# Setting Everything Up

In [139]:
import pandas as pd
from datetime import datetime

In [140]:
#import the file from the web
import requests
from bs4 import BeautifulSoup
import pandas as pd
from unicodedata import normalize
import re
from datetime import datetime
from pandas import read_csv 
from sklearn.feature_extraction.text import CountVectorizer
import json
from flatten_json import flatten


# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None

# # I want to be able to see the entire narrative, so remove the maximum width for each column
# pd.options.display.max_colwidth = None

# pd.options.display.float_format = '{:,.0f}'.format

import string

%matplotlib inline

In [141]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 8)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [143]:
%%R

# My commonly used R imports

require('tidyverse')

In [144]:
#To Download PDFs
from urllib.parse import urljoin

In [145]:
#To import camelot and PDF-related items
import camelot
import ghostscript
import sys

In [146]:
%%R

library(jpeg)
library(wordcloud)
library(RColorBrewer)
library(wordcloud2)
library(tm)


In [147]:
from sklearn.feature_extraction.text import CountVectorizer

In [148]:
%%R
library(vistime)
library(shiny)
library(plotly)
#for timeline plotting

In [149]:
from collections import Counter


# Preparing the company titling data for relevant companies

# Reading a json to pandas
Source of this data as accessed on July 5th: https://www.sec.gov/files/company_tickers.json
* copy pasting ended up being easier given sec blockers

In [171]:
company_tickers={"0":{"cik_str":320193,"ticker":"AAPL","title":"Apple Inc."},"1":{"cik_str":789019,"ticker":"MSFT","title":"MICROSOFT CORP"},"2":{"cik_str":1652044,"ticker":"GOOGL","title":"Alphabet Inc."},"3":{"cik_str":1018724,"ticker":"AMZN","title":"AMAZON COM INC"},"4":{"cik_str":1045810,"ticker":"NVDA","title":"NVIDIA CORP"},"5":{"cik_str":1318605,"ticker":"TSLA","title":"Tesla, Inc."},"6":{"cik_str":1067983,"ticker":"BRK-B","title":"BERKSHIRE HATHAWAY INC"},"7":{"cik_str":1326801,"ticker":"META","title":"Meta Platforms, Inc."},"8":{"cik_str":1046179,"ticker":"TSM","title":"TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD"},"9":{"cik_str":1403161,"ticker":"V","title":"VISA INC."},"10":{"cik_str":824046,"ticker":"LVMUY","title":"LVMH MOET HENNESSY LOUIS VUITTON"},"11":{"cik_str":731766,"ticker":"UNH","title":"UNITEDHEALTH GROUP INC"},"12":{"cik_str":59478,"ticker":"LLY","title":"ELI LILLY & Co"},"13":{"cik_str":34088,"ticker":"XOM","title":"EXXON MOBIL CORP"},"14":{"cik_str":19617,"ticker":"JPM","title":"JPMORGAN CHASE & CO"},"15":{"cik_str":104169,"ticker":"WMT","title":"Walmart Inc."},"16":{"cik_str":200406,"ticker":"JNJ","title":"JOHNSON & JOHNSON"},"17":{"cik_str":884394,"ticker":"SPY","title":"SPDR S&P 500 ETF TRUST"},"18":{"cik_str":1141391,"ticker":"MA","title":"Mastercard Inc"},"19":{"cik_str":1730168,"ticker":"AVGO","title":"Broadcom Inc."},"20":{"cik_str":80424,"ticker":"PG","title":"PROCTER & GAMBLE Co"},"21":{"cik_str":353278,"ticker":"NVO","title":"NOVO NORDISK A S"},"22":{"cik_str":1047716,"ticker":"LTMAY","title":"LATAM AIRLINES GROUP S.A."},"23":{"cik_str":1545460,"ticker":"OLCLY","title":"Oriental Land Co., Ltd./ADR"},"24":{"cik_str":1341439,"ticker":"ORCL","title":"ORACLE CORP"},"25":{"cik_str":354950,"ticker":"HD","title":"HOME DEPOT, INC."},"26":{"cik_str":93410,"ticker":"CVX","title":"CHEVRON CORP"},"27":{"cik_str":937966,"ticker":"ASML","title":"ASML HOLDING NV"},"28":{"cik_str":310158,"ticker":"MRK","title":"Merck & Co., Inc."},"29":{"cik_str":21344,"ticker":"KO","title":"COCA COLA CO"},"30":{"cik_str":77476,"ticker":"PEP","title":"PEPSICO INC"},"31":{"cik_str":909832,"ticker":"COST","title":"COSTCO WHOLESALE CORP /NEW"},"32":{"cik_str":1551152,"ticker":"ABBV","title":"AbbVie Inc."},"33":{"cik_str":70858,"ticker":"BAC","title":"BANK OF AMERICA CORP /DE/"},"34":{"cik_str":796343,"ticker":"ADBE","title":"ADOBE INC."},"35":{"cik_str":1094517,"ticker":"TM","title":"TOYOTA MOTOR CORP/"},"36":{"cik_str":811809,"ticker":"BHP","title":"BHP Group Ltd"},"37":{"cik_str":1577552,"ticker":"BABA","title":"Alibaba Group Holding Ltd"},"38":{"cik_str":63908,"ticker":"MCD","title":"MCDONALDS CORP"},"39":{"cik_str":858877,"ticker":"CSCO","title":"CISCO SYSTEMS, INC."},"40":{"cik_str":1114448,"ticker":"NVS","title":"NOVARTIS AG"},"41":{"cik_str":78003,"ticker":"PFE","title":"PFIZER INC"},"42":{"cik_str":1108524,"ticker":"CRM","title":"Salesforce, Inc."},"43":{"cik_str":1306965,"ticker":"SHEL","title":"Shell plc"},"44":{"cik_str":901832,"ticker":"AZN","title":"ASTRAZENECA PLC"},"45":{"cik_str":1061736,"ticker":"FMX","title":"MEXICAN ECONOMIC DEVELOPMENT INC"},"46":{"cik_str":97745,"ticker":"TMO","title":"THERMO FISHER SCIENTIFIC INC."},"47":{"cik_str":1467373,"ticker":"ACN","title":"Accenture plc"},"48":{"cik_str":1065280,"ticker":"NFLX","title":"NETFLIX INC"},"49":{"cik_str":2488,"ticker":"AMD","title":"ADVANCED MICRO DEVICES INC"},"50":{"cik_str":1800,"ticker":"ABT","title":"ABBOTT LABORATORIES"},"51":{"cik_str":1707925,"ticker":"LIN","title":"LINDE PLC"},"52":{"cik_str":313616,"ticker":"DHR","title":"DANAHER CORP /DE/"},"53":{"cik_str":1166691,"ticker":"CMCSA","title":"COMCAST CORP"},"54":{"cik_str":320187,"ticker":"NKE","title":"NIKE, Inc."},"55":{"cik_str":1283699,"ticker":"TMUS","title":"T-Mobile US, Inc."},"56":{"cik_str":1744489,"ticker":"DIS","title":"Walt Disney Co"},"57":{"cik_str":97476,"ticker":"TXN","title":"TEXAS INSTRUMENTS INC"},"58":{"cik_str":72971,"ticker":"WFC","title":"WELLS FARGO & COMPANY/MN"},"59":{"cik_str":1089113,"ticker":"HSBC","title":"HSBC HOLDINGS PLC"},"60":{"cik_str":1090727,"ticker":"UPS","title":"UNITED PARCEL SERVICE INC"},"61":{"cik_str":1000184,"ticker":"SAP","title":"SAP SE"},"62":{"cik_str":732712,"ticker":"VZ","title":"VERIZON COMMUNICATIONS INC"},"63":{"cik_str":1413329,"ticker":"PM","title":"Philip Morris International Inc."},"64":{"cik_str":753308,"ticker":"NEE","title":"NEXTERA ENERGY INC"},"65":{"cik_str":1067839,"ticker":"QQQ","title":"INVESCO QQQ TRUST, SERIES 1"},"66":{"cik_str":895421,"ticker":"MS","title":"MORGAN STANLEY"},"67":{"cik_str":101829,"ticker":"RTX","title":"RAYTHEON TECHNOLOGIES CORP"},"68":{"cik_str":879764,"ticker":"TTE","title":"TotalEnergies SE"},"69":{"cik_str":50863,"ticker":"INTC","title":"INTEL CORP"},"70":{"cik_str":773840,"ticker":"HON","title":"HONEYWELL INTERNATIONAL INC"},"71":{"cik_str":1442651,"ticker":"SHECY","title":"Shin-Etsu Chemical Co., Ltd."},"72":{"cik_str":14272,"ticker":"BMY","title":"BRISTOL MYERS SQUIBB CO"},"73":{"cik_str":1121404,"ticker":"SNY","title":"Sanofi"},"74":{"cik_str":1000275,"ticker":"RY","title":"ROYAL BANK OF CANADA"},"75":{"cik_str":804328,"ticker":"QCOM","title":"QUALCOMM INC/DE"},"76":{"cik_str":1144967,"ticker":"HDB","title":"HDFC BANK LTD"},"77":{"cik_str":60667,"ticker":"LOW","title":"LOWES COMPANIES INC"},"78":{"cik_str":217410,"ticker":"UL","title":"UNILEVER PLC"},"79":{"cik_str":4962,"ticker":"AXP","title":"AMERICAN EXPRESS CO"},"80":{"cik_str":18230,"ticker":"CAT","title":"CATERPILLAR INC"},"81":{"cik_str":64040,"ticker":"SPGI","title":"S&P Global Inc."},"82":{"cik_str":12927,"ticker":"BA","title":"BOEING CO"},"83":{"cik_str":896878,"ticker":"INTU","title":"INTUIT INC."},"84":{"cik_str":887028,"ticker":"RTNTF","title":"RIO TINTO LTD"},"85":{"cik_str":1163165,"ticker":"COP","title":"CONOCOPHILLIPS"},"86":{"cik_str":100885,"ticker":"UNP","title":"UNION PACIFIC CORP"},"87":{"cik_str":6951,"ticker":"AMAT","title":"APPLIED MATERIALS INC /DE"},"88":{"cik_str":51143,"ticker":"IBM","title":"INTERNATIONAL BUSINESS MACHINES CORP"},"89":{"cik_str":318154,"ticker":"AMGN","title":"AMGEN INC"},"90":{"cik_str":315189,"ticker":"DE","title":"DEERE & CO"},"91":{"cik_str":40545,"ticker":"GE","title":"GENERAL ELECTRIC CO"},"92":{"cik_str":1035267,"ticker":"ISRG","title":"INTUITIVE SURGICAL INC"},"93":{"cik_str":936468,"ticker":"LMT","title":"LOCKHEED MARTIN CORP"},"94":{"cik_str":1613103,"ticker":"MDT","title":"Medtronic plc"},"95":{"cik_str":947263,"ticker":"TD","title":"TORONTO DOMINION BANK"},"96":{"cik_str":732717,"ticker":"T","title":"AT&T INC."},"97":{"cik_str":313838,"ticker":"SONY","title":"Sony Group Corp"},"98":{"cik_str":1373715,"ticker":"NOW","title":"ServiceNow, Inc."},"99":{"cik_str":1045609,"ticker":"PLD","title":"Prologis, Inc."},"100":{"cik_str":1378580,"ticker":"EADSY","title":"Airbus SE/ADR"},"101":{"cik_str":1393818,"ticker":"BX","title":"Blackstone Inc."},"102":{"cik_str":1668717,"ticker":"BUD","title":"Anheuser-Busch InBev SA/NV"},"103":{"cik_str":829224,"ticker":"SBUX","title":"STARBUCKS CORP"},"104":{"cik_str":310764,"ticker":"SYK","title":"STRYKER CORP"},"105":{"cik_str":886982,"ticker":"GS","title":"GOLDMAN SACHS GROUP INC"},"106":{"cik_str":863064,"ticker":"RIO","title":"RIO TINTO PLC"},"107":{"cik_str":1156039,"ticker":"ELV","title":"Elevance Health, Inc."},"108":{"cik_str":316709,"ticker":"SCHW","title":"SCHWAB CHARLES CORP"},"109":{"cik_str":313807,"ticker":"BP","title":"BP PLC"},"110":{"cik_str":1364742,"ticker":"BLK","title":"BlackRock Inc."},"111":{"cik_str":1075531,"ticker":"BKNG","title":"Booking Holdings Inc."},"112":{"cik_str":1103982,"ticker":"MDLZ","title":"Mondelez International, Inc."},"113":{"cik_str":835403,"ticker":"DEO","title":"DIAGEO PLC"},"114":{"cik_str":6281,"ticker":"ADI","title":"ANALOG DEVICES INC"},"115":{"cik_str":109198,"ticker":"TJX","title":"TJX COMPANIES INC /DE/"},"116":{"cik_str":882095,"ticker":"GILD","title":"GILEAD SCIENCES, INC."},"117":{"cik_str":1737806,"ticker":"PDD","title":"PDD Holdings Inc."},"118":{"cik_str":62709,"ticker":"MMC","title":"MARSH & MCLENNAN COMPANIES, INC."},"119":{"cik_str":1140625,"ticker":"EQNR","title":"EQUINOR ASA"},"120":{"cik_str":1119639,"ticker":"PBR","title":"PETROBRAS - PETROLEO BRASILEIRO SA"},"121":{"cik_str":67088,"ticker":"MUFG","title":"MITSUBISHI UFJ FINANCIAL GROUP INC"},"122":{"cik_str":831001,"ticker":"C","title":"CITIGROUP INC"},"123":{"cik_str":1053507,"ticker":"AMT","title":"AMERICAN TOWER CORP /MA/"},"124":{"cik_str":8670,"ticker":"ADP","title":"AUTOMATIC DATA PROCESSING INC"},"125":{"cik_str":1274152,"ticker":"CSLLY","title":"CSL LTD"},"126":{"cik_str":875320,"ticker":"VRTX","title":"VERTEX PHARMACEUTICALS INC / MA"},"127":{"cik_str":64803,"ticker":"CVS","title":"CVS HEALTH Corp"},"128":{"cik_str":707549,"ticker":"LRCX","title":"LAM RESEARCH CORP"},"129":{"cik_str":1543151,"ticker":"UBER","title":"Uber Technologies, Inc"},"130":{"cik_str":1067318,"ticker":"MBGYY","title":"DAIMLER AG"},"131":{"cik_str":1559720,"ticker":"ABNB","title":"Airbnb, Inc."},"132":{"cik_str":1739940,"ticker":"CI","title":"Cigna Group"},"133":{"cik_str":1594805,"ticker":"SHOP","title":"SHOPIFY INC."},"134":{"cik_str":764180,"ticker":"MO","title":"ALTRIA GROUP, INC."},"135":{"cik_str":860730,"ticker":"HCA","title":"HCA Healthcare, Inc."},"136":{"cik_str":1103838,"ticker":"IBN","title":"ICICI BANK LTD"},"137":{"cik_str":16868,"ticker":"CNI","title":"CANADIAN NATIONAL RAILWAY CO"},"138":{"cik_str":1551182,"ticker":"ETN","title":"Eaton Corp plc"},"139":{"cik_str":896159,"ticker":"CB","title":"Chubb Ltd"},"140":{"cik_str":872589,"ticker":"REGN","title":"REGENERON PHARMACEUTICALS, INC."},"141":{"cik_str":1555280,"ticker":"ZTS","title":"Zoetis Inc."},"142":{"cik_str":1327567,"ticker":"PANW","title":"Palo Alto Networks Inc"},"143":{"cik_str":798354,"ticker":"FI","title":"FISERV INC"},"144":{"cik_str":80661,"ticker":"PGR","title":"PROGRESSIVE CORP/OH/"},"145":{"cik_str":885725,"ticker":"BSX","title":"BOSTON SCIENTIFIC CORP"},"146":{"cik_str":92122,"ticker":"…T AIRPORT GROUP"},"924":{"cik_str":842162,"ticker":"LEA","title":"LEAR CORP"},"925":{"cik_str":1669162,"ticker":"KNSL","title":"Kinsale Capital Group, Inc."},"926":{"cik_str":1720635,"ticker":"NVT","title":"nVent Electric plc"},"927":{"cik_str":1849853,"ticker":"STVN","title":"Stevanato Group S.p.A."},"928":{"cik_str":937556,"ticker":"MASI","title":"MASIMO CORP"},"929":{"cik_str":1531152,"ticker":"BJ","title":"BJ's Wholesale Club Holdings, Inc."},"930":{"cik_str":818686,"ticker":"TEVA","title":"TEVA PHARMACEUTICAL INDUSTRIES LTD"},"931":{"cik_str":818479,"ticker":"XRAY","title":"DENTSPLY SIRONA Inc."},"932":{"cik_str":1633931,"ticker":"BLD","title":"TopBuild Corp"},"933":{"cik_str":1023128,"ticker":"LAD","title":"LITHIA MOTORS INC"},"934":{"cik_str":1289419,"ticker":"MORN","title":"Morningstar, Inc."},"935":{"cik_str":70318,"ticker":"THC","title":"TENET HEALTHCARE CORP"},"936":{"cik_str":1450123,"ticker":"JBSAY","title":"JBS S.A."},"937":{"cik_str":1065837,"ticker":"SKX","title":"SKECHERS USA INC"},"938":{"cik_str":1639438,"ticker":"CAVA","title":"CAVA GROUP, INC."},"939":{"cik_str":40729,"ticker":"ALLY","title":"Ally Financial Inc."},"940":{"cik_str":910073,"ticker":"NYCB","title":"NEW YORK COMMUNITY BANCORP INC"},"941":{"cik_str":1533232,"ticker":"BEP","title":"Brookfield Renewable Partners L.P."},"942":{"cik_str":106640,"ticker":"WHR","title":"WHIRLPOOL CORP /DE/"},"943":{"cik_str":1771917,"ticker":"KRTX","title":"Karuna Therapeutics, Inc."},"944":{"cik_str":719955,"ticker":"WSM","title":"WILLIAMS SONOMA INC"},"945":{"cik_str":315213,"ticker":"RHI","title":"ROBERT HALF INTERNATIONAL INC."},"946":{"cik_str":7536,"ticker":"ARW","title":"ARROW ELECTRONICS, INC"},"947":{"cik_str":1446457,"ticker":"BDNNY","title":"Boliden AB"},"948":{"cik_str":19584,"ticker":"CHE","title":"CHEMED CORP"},"949":{"cik_str":1091907,"ticker":"WWE","title":"WORLD WRESTLING ENTERTAINMENTINC"},"950":{"cik_str":1449003,"ticker":"ASXFY","title":"ASX Ltd."},"951":{"cik_str":1037038,"ticker":"RL","title":"RALPH LAUREN CORP"},"952":{"cik_str":1845338,"ticker":"MNDY","title":"monday.com Ltd."},"953":{"cik_str":1170858,"ticker":"SBS","title":"COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SAO PAULO-SABESP"},"954":{"cik_str":1818874,"ticker":"SOFI","title":"SoFi Technologies, Inc."},"955":{"cik_str":34903,"ticker":"FRT","title":"FEDERAL REALTY INVESTMENT TRUST"},"956":{"cik_str":1308547,"ticker":"DLB","title":"Dolby Laboratories, Inc."},"957":{"cik_str":314590,"ticker":"SSL","title":"SASOL LTD"},"958":{"cik_str":1596783,"ticker":"CTLT","title":"Catalent, Inc."},"959":{"cik_str":1232524,"ticker":"JAZZ","title":"Jazz Pharmaceuticals plc"},"960":{"cik_str":350894,"ticker":"SEIC","title":"SEI INVESTMENTS CO"},"961":{"cik_str":1570585,"ticker":"LBTYA","title":"Liberty Global plc"},"962":{"cik_str":730272,"ticker":"RGEN","title":"REPLIGEN CORP"},"963":{"cik_str":769520,"ticker":"MIDD","title":"MIDDLEBY Corp"},"964":{"cik_str":1342874,"ticker":"TX","title":"Ternium S.A."},"965":{"cik_str":914208,"ticker":"IVZ","title":"Invesco Ltd."},"966":{"cik_str":1309108,"ticker":"WEX","title":"WEX Inc."},"967":{"cik_str":751364,"ticker":"NNN","title":"NNN REIT, INC."},"968":{"cik_str":1438569,"ticker":"GRFS","title":"Grifols SA"},"969":{"cik_str":1015650,"ticker":"SKM","title":"SK TELECOM CO LTD"},"970":{"cik_str":96223,"ticker":"JEF","title":"Jefferies Financial Group Inc."},"971":{"cik_str":16058,"ticker":"CACI","title":"CACI INTERNATIONAL INC /DE/"},"972":{"cik_str":49600,"ticker":"EGP","title":"EASTGROUP PROPERTIES INC"},"973":{"cik_str":1653482,"ticker":"GTLB","title":"Gitlab Inc."},"974":{"cik_str":1418819,"ticker":"IRDM","title":"Iridium Communications Inc."},"975":{"cik_str":1378992,"ticker":"BERY","title":"BERRY GLOBAL GROUP, INC."},"976":{"cik_str":1557860,"ticker":"GLOB","title":"Globant S.A."},"977":{"cik_str":1789832,"ticker":"HESM","title":"Hess Midstream LP"},"978":{"cik_str":1616707,"ticker":"W","title":"Wayfair Inc."},"979":{"cik_str":85535,"ticker":"RGLD","title":"ROYAL GOLD INC"},"980":{"cik_str":1910851,"ticker":"RCM","title":"R1 RCM Inc. /DE"},"981":{"cik_str":216228,"ticker":"ITT","title":"ITT INC."},"982":{"cik_str":896622,"ticker":"ATR","title":"APTARGROUP, INC."},"983":{"cik_str":935494,"ticker":"NATI","title":"NATIONAL INSTRUMENTS CORP"},"984":{"cik_str":350698,"ticker":"AN","title":"AUTONATION, INC."},"985":{"cik_str":1326380,"ticker":"GME","title":"GameStop Corp."},"986":{"cik_str":1289460,"ticker":"TXRH","title":"Texas Roadhouse, Inc."},"987":{"cik_str":1481045,"ticker":"DSEEY","title":"Daiwa Securities Group Inc."},"988":{"cik_str":29644,"ticker":"DCI","title":"DONALDSON Co INC"},"989":{"cik_str":100826,"ticker":"UELMO","title":"UNION ELECTRIC CO"},"990":{"cik_str":1069157,"ticker":"EWBC","title":"EAST WEST BANCORP INC"},"991":{"cik_str":1327068,"ticker":"USO","title":"United States Oil Fund, LP"},"992":{"cik_str":1852244,"ticker":"GXO","title":"GXO Logistics, Inc."},"993":{"cik_str":1732845,"ticker":"WRK","title":"WestRock Co"},"994":{"cik_str":1408075,"ticker":"GPK","title":"GRAPHIC PACKAGING HOLDING CO"},"995":{"cik_str":1816007,"ticker":"LU","title":"Lufax Holding Ltd"},"996":{"cik_str":201533,"ticker":"CMS-PB","title":"CONSUMERS ENERGY CO"},"997":{"cik_str":938323,"ticker":"PSO","title":"PEARSON PLC"},"998":{"cik_str":1650729,"ticker":"SITE","title":"SiteOne Landscape Supply, Inc."},"999":{"cik_str":103379,"ticker":"VFC","title":"V F CORP"},"1000":{"cik_str":1037976,"ticker":"JLL","title":"JONES LANG LASALLE INC"},"1001":{"cik_str":888491,"ticker":"OHI","title":"OMEGA HEALTHCARE INVESTORS INC"},"1002":{"cik_str":1681459,"ticker":"FTI","title":"TechnipFMC plc"},"1003":{"cik_str":1635088,"ticker":"ROIV","title":"Roivant Sciences Ltd."},"1004":{"cik_str":1826168,"ticker":"TIMB","title":"TIM S.A."},"1005":{"cik_str":74260,"ticker":"ORI","title":"OLD REPUBLIC INTERNATIONAL CORP"},"1006":{"cik_str":1034670,"ticker":"ALV","title":"AUTOLIV INC"},"1007":{"cik_str":1668397,"ticker":"MEDP","title":"Medpace Holdings, Inc."},"1008":{"cik_str":1520697,"ticker":"ACHC","title":"Acadia Healthcare Company, Inc."},"1009":{"cik_str":1049502,"ticker":"MKSI","title":"MKS INSTRUMENTS INC"},"1010":{"cik_str":1616862,"ticker":"AXTA","title":"Axalta Coating Systems Ltd."},"1011":{"cik_str":1528849,"ticker":"RH","title":"RH"},"1012":{"cik_str":1334036,"ticker":"CROX","title":"Crocs, Inc."},"1013":{"cik_str":1047127,"ticker":"AMKR","title":"AMKOR TECHNOLOGY, INC."},"1014":{"cik_str":1021635,"ticker":"OGE","title":"OGE ENERGY CORP."},"1015":{"cik_str":921825,"ticker":"FR","title":"FIRST INDUSTRIAL REALTY TRUST INC"},"1016":{"cik_str":1860742,"ticker":"BLCO","title":"Bausch & Lomb Corp"},"1017":{"cik_str":1360604,"ticker":"HR","title":"Healthcare Realty Trust Inc"},"1018":{"cik_str":941221,"ticker":"ICL","title":"ICL Group Ltd."},"1019":{"cik_str":1131383,"ticker":"STN","title":"STANTEC INC"},"1020":{"cik_str":1402388,"ticker":"WFG","title":"WEST FRASER TIMBER CO., LTD"},"1021":{"cik_str":1430162,"ticker":"CSAN","title":"Cosan S.A."},"1022":{"cik_str":1475922,"ticker":"PRI","title":"Primerica, Inc."},"1023":{"cik_str":1330568,"ticker":"SLV","title":"iShares Silver Trust"},"1024":{"cik_str":889331,"ticker":"LFUS","title":"LITTELFUSE INC /DE"},"1025":{"cik_str":1856525,"ticker":"CNM","title":"Core & Main, Inc."},"1026":{"cik_str":1766363,"ticker":"EDR","title":"Endeavor Group Holdings, Inc."},"1027":{"cik_str":108312,"ticker":"WWD","title":"Woodward, Inc."},"1028":{"cik_str":1173382,"ticker":"CAE","title":"CAE INC"},"1029":{"cik_str":63276,"ticker":"MAT","title":"MATTEL INC /DE/"},"1030":{"cik_str":39263,"ticker":"CFR","title":"CULLEN/FROST BANKERS, INC."},"1031":{"cik_str":315852,"ticker":"RRC","title":"RANGE RESOURCES CORP"},"1032":{"cik_str":1069878,"ticker":"TREX","title":"TREX CO INC"},"1033":{"cik_str":1535929,"ticker":"VOYA","title":"Voya Financial, Inc."},"1034":{"cik_str":895419,"ticker":"WOLF","title":"WOLFSPEED, INC."},"1035":{"cik_str":1046257,"ticker":"INGR","title":"Ingredion Inc"},"1036":{"cik_str":1831097,"ticker":"AGL","title":"agilon health, inc."},"1037":{"cik_str":1818201,"ticker":"CCCS","title":"CCC Intelligent Solutions Holdings Inc."},"1038":{"cik_str":26324,"ticker":"CW","title":"CURTISS WRIGHT CORP"},"1039":{"cik_str":906553,"ticker":"BYD","title":"BOYD GAMING CORP"},"1040":{"cik_str":917273,"ticker":"RMBS","title":"RAMBUS INC"},"1041":{"cik_str":931015,"ticker":"PII","title":"Polaris Inc."},"1042":{"cik_str":1398659,"ticker":"G","title":"Genpact LTD"},"1043":{"cik_str":853816,"ticker":"LSTR","title":"LANDSTAR SYSTEM INC"},"1044":{"cik_str":1433270,"ticker":"AR","title":"ANTERO RESOURCES Corp"},"1045":{"cik_str":1838413,"ticker":"YMM","title":"Full Truck Alliance Co. Ltd."},"1046":{"cik_str":1637810,"ticker":"FSV","title":"FirstService Corp"},"1047":{"cik_str":1092699,"ticker":"SPSC","title":"SPS COMMERCE INC"},"1048":{"cik_str":1166003,"ticker":"XPO","title":"XPO, Inc."},"1049":{"cik_str":1819848,"ticker":"JOBY","title":"Joby Aviation, Inc."},"1050":{"cik_str":355811,"ticker":"GNTX","title":"GENTEX CORP"},"1051":{"cik_str":1206264,"ticker":"TPX","title":"TEMPUR SEALY INTERNATIONAL, INC."},"1052":{"cik_str":1835963,"ticker":"GLBE","title":"Global-E Online Ltd."},"1053":{"cik_str":1005284,"ticker":"OLED","title":"UNIVERSAL DISPLAY CORP PA"},"1054":{"cik_str":766421,"ticker":"ALK","title":"ALASKA AIR GROUP, INC."},"1055":{"cik_str":1513845,"ticker":"YNDX","title":"Yandex N.V."},"1056":{"cik_str":1477294,"ticker":"ST","title":"Sensata Technologies Holding plc"},"1057":{"cik_str":74303,"ticker":"OLN","title":"OLIN Corp"},"1058":{"cik_str":1372920,"ticker":"EDU","title":"New Oriental Education & Technology Group Inc."},"1059":{"cik_str":1573516,"ticker":"MUSA","title":"Murphy USA Inc."},"1060":{"cik_str":1842827,"ticker":"BZ","title":"Kanzhun Ltd"},"1061":{"cik_str":1050140,"ticker":"DSGX","title":"DESCARTES SYSTEMS GROUP INC"},"1062":{"cik_str":1767582,"ticker":"LKNCY","title":"Luckin Coffee Inc."},"1063":{"cik_str":785161,"ticker":"EHC","title":"Encompass Health Corp"},"1064":{"cik_str":1302215,"ticker":"HLI","title":"HOULIHAN LOKEY, INC."},"1065":{"cik_str":1264136,"ticker":"WF","title":"WOORI FINANCIAL GROUP INC."},"1066":{"cik_str":892553,"ticker":"GTLS","title":"CHART INDUSTRIES INC"},"1067":{"cik_str":98362,"ticker":"TKR","title":"TIMKEN CO"},"1068":{"cik_str":801337,"ticker":"WBS","title":"WEBSTER FINANCIAL CORP"},"1069":{"cik_str":1267238,"ticker":"AIZ","title":"ASSURANT, INC."},"1070":{"cik_str":1479094,"ticker":"STAG","title":"STAG Industrial, Inc."},"1071":{"cik_str":66570,"ticker":"MSA","title":"MSA Safety Inc"},"1072":{"cik_str":1618732,"ticker":"NTNX","title":"Nutanix, Inc."},"1073":{"cik_str":7332,"ticker":"SWN","title":"SOUTHWESTERN ENERGY CO"},"1074":{"cik_str":1581068,"ticker":"BRX","title":"Brixmor Property Group Inc."},"1075":{"cik_str":918646,"ticker":"EXP","title":"EAGLE MATERIALS INC"},"1076":{"cik_str":1770787,"ticker":"TXG","title":"10x Genomics, Inc."},"1077":{"cik_str":1157557,"ticker":"CIG","title":"ENERGY CO OF MINAS GERAIS"},"1078":{"cik_str":1748797,"ticker":"DOOO","title":"BRP Inc."},"1079":{"cik_str":851968,"ticker":"MHK","title":"MOHAWK INDUSTRIES INC"},"1080":{"cik_str":885550,"ticker":"CACC","title":"CREDIT ACCEPTANCE CORP"},"1081":{"cik_str":1071371,"ticker":"CIB","title":"BANCOLOMBIA SA"},"1082":{"cik_str":71691,"ticker":"NYT","title":"NEW YORK TIMES CO"},"1083":{"cik_str":1546354,"ticker":"DUFRY","title":"Dufry AG/ADR"},"1084":{"cik_str":887936,"ticker":"FCN","title":"FTI CONSULTING, INC"},"1085":{"cik_str":1486957,"ticker":"BWXT","title":"BWX Technologies, Inc."},"1086":{"cik_str":1723690,"ticker":"BILI","title":"Bilibili Inc."},"1087":{"cik_str":1764046,"ticker":"CLVT","title":"CLARIVATE Plc"},"1088":{"cik_str":1598110,"ticker":"CYBR","title":"CyberArk Software Ltd."},"1089":{"cik_str":1819928,"ticker":"DV","title":"DoubleVerify Holdings, Inc."},"1090":{"cik_str":1486159,"ticker":"CHRD","title":"Chord Energy Corp"},"1091":{"cik_str":1076930,"ticker":"NOVT","title":"NOVANTA INC"},"1092":{"cik_str":36966,"ticker":"FHN","title":"FIRST HORIZON CORP"},"1093":{"cik_str":1516513,"ticker":"DOCS","title":"Doximity, Inc."},"1094":{"cik_str":1021860,"ticker":"NOV","title":"NOV Inc."},"1095":{"cik_str":1796209,"ticker":"APG","title":"APi Group Corp"},"1096":{"cik_str":717605,"ticker":"HXL","title":"HEXCEL CORP /DE/"},"1097":{"cik_str":1111928,"ticker":"IPGP","title":"IPG PHOTONICS CORP"},"1098":{"cik_str":720672,"ticker":"SF","title":"STIFEL FINANCIAL CORP"},"1099":{"cik_str":1619762,"ticker":"IGT","title":"International Game Technology PLC"},"1100":{"cik_str":1324948,"ticker":"RBC","title":"RBC Bearings INC"},"1101":{"cik_str":936395,"ticker":"CIEN","title":"CIENA CORP"},"1102":{"cik_str":1707753,"ticker":"ESTC","title":"Elastic N.V."},"1103":{"cik_str":750004,"ticker":"LNW","title":"Light & Wonder, Inc."},"1104":{"cik_str":1465128,"ticker":"STWD","title":"STARWOOD PROPERTY TRUST, INC."},"1105":{"cik_str":22444,"ticker":"CMC","title":"COMMERCIAL METALS Co"},"1106":{"cik_str":1520006,"ticker":"MTDR","title":"Matador Resources Co"},"1107":{"cik_str":939767,"ticker":"EXEL","title":"EXELIXIS, INC."},"1108":{"cik_str":84246,"ticker":"RLI","title":"RLI CORP"},"1109":{"cik_str":1093691,"ticker":"PLUG","title":"PLUG POWER INC"},"1110":{"cik_str":77877,"ticker":"PDCE","title":"PDC ENERGY, INC."},"1111":{"cik_str":1723089,"ticker":"CHX","title":"ChampionX Corp"},"1112":{"cik_str":1437178,"ticker":"EGFEY","title":"EFG Eurobank Ergasias S.A./ADR"},"1113":{"cik_str":917251,"ticker":"ADC","title":"AGREE REALTY CORP"},"1114":{"cik_str":795403,"ticker":"WTS","title":"WATTS WATER TECHNOLOGIES INC"},"1115":{"cik_str":102729,"ticker":"VMI","title":"VALMONT INDUSTRIES INC"},"1116":{"cik_str":14930,"ticker":"BC","title":"BRUNSWICK CORP"},"1117":{"cik_str":22356,"ticker":"CBSH","title":"COMMERCE BANCSHARES INC /MO/"},"1118":{"cik_str":1433195,"ticker":"APPF","title":"APPFOLIO INC"},"1119":{"cik_str":1558812,"ticker":"MEJHY","title":"MEIJI Holdings Co Ltd/ADR"},"1120":{"cik_str":1528396,"ticker":"GWRE","title":"Guidewire Software, Inc."},"1121":{"cik_str":1675149,"ticker":"AA","title":"Alcoa Corp"},"1122":{"cik_str":1600033,"ticker":"ELF","title":"e.l.f. Beauty, Inc."},"1123":{"cik_str":1658566,"ticker":"PR","title":"Permian Resources Corp"},"1124":{"cik_str":1701732,"ticker":"ALTR","title":"Altair Engineering Inc."},"1125":{"cik_str":1571123,"ticker":"SAIC","title":"Science Applications International Corp"},"1126":{"cik_str":1637207,"ticker":"PLNT","title":"Planet Fitness, Inc."},"1127":{"cik_str":1567514,"ticker":"ITCI","title":"Intra-Cellular Therapies, Inc."},"1128":{"cik_str":317540,"ticker":"COKE","title":"Coca-Cola Consolidated, Inc."},"1129":{"cik_str":1142417,"ticker":"NXST","title":"NEXSTAR MEDIA GROUP, INC."},"1130":{"cik_str":912767,"ticker":"UFPI","title":"UFP INDUSTRIES INC"},"1131":{"cik_str":1046311,"ticker":"CHH","title":"CHOICE HOTELS INTERNATIONAL INC /DE"},"1132":{"cik_str":1786431,"ticker":"REYN","title":"Reynolds Consumer Products Inc."},"1133":{"cik_str":717423,"ticker":"MUR","title":"MURPHY OIL CORP"},"1134":{"cik_str":1666138,"ticker":"ATKR","title":"Atkore Inc."},"1135":{"cik_str":1477049,"ticker":"PHYS","title":"Sprott Physical Gold Trust"},"1136":{"cik_str":1237831,"ticker":"GMED","title":"GLOBUS MEDICAL INC"},"1137":{"cik_str":1113232,"ticker":"ACLS","title":"AXCELIS TECHNOLOGIES INC"},"1138":{"cik_str":1423689,"ticker":"AGNC","title":"AGNC Investment Corp."},"1139":{"cik_str":1449005,"ticker":"JSCPY","title":"JSR Corp."},"1140":{"cik_str":701818,"ticker":"KGC","title":"KINROSS GOLD CORP"},"1141":{"cik_str":1035983,"ticker":"FIX","title":"COMFORT SYSTEMS USA INC"},"1142":{"cik_str":874015,"ticker":"IONS","title":"IONIS PHARMACEUTICALS INC"},"1143":{"cik_str":1029199,"ticker":"EEFT","title":"EURONET WORLDWIDE, INC."},"1144":{"cik_str":1895262,"ticker":"NE","title":"Noble Corp plc"},"1145":{"cik_str":1472787,"ticker":"FAF","title":"First American Financial Corp"},"1146":{"cik_str":1174169,"ticker":"AQN","title":"ALGONQUIN POWER & UTILITIES CORP."},"1147":{"cik_str":1636222,"ticker":"WING","title":"Wingstop Inc."},"1148":{"cik_str":91767,"ticker":"SON","title":"SONOCO PRODUCTS CO"},"1149":{"cik_str":1562088,"ticker":"DUOL","title":"Duolingo, Inc."},"1150":{"cik_str":1448978,"ticker":"SMBMY","title":"SembCorp Marine Ltd."},"1151":{"cik_str":1722684,"ticker":"WH","title":"WYNDHAM HOTELS & RESORTS, INC."},"1152":{"cik_str":1288847,"ticker":"FIVN","title":"Five9, Inc."},"1153":{"cik_str":1012100,"ticker":"SEE","title":"SEALED AIR CORP/DE"},"1154":{"cik_str":1014739,"ticker":"OPCH","title":"Option Care Health, Inc."},"1155":{"cik_str":1061894,"ticker":"GIL","title":"Gildan Activewear Inc."},"1156":{"cik_str":920371,"ticker":"SSD","title":"Simpson Manufacturing Co., Inc."},"1157":{"cik_str":1657853,"ticker":"HTZ","title":"HERTZ GLOBAL HOLDINGS, INC"},"1158":{"cik_str":230557,"ticker":"SIGI","title":"SELECTIVE INSURANCE GROUP INC"},"1159":{"cik_str":28412,"ticker":"CMA","title":"COMERICA INC /NEW/"},"1160":{"cik_str":1512544,"ticker":"ITTOY","title":"ITOCHU Techno-Solutions Corporation/ADR"},"1161":{"cik_str":1754581,"ticker":"FUTU","title":"Futu Holdings Ltd"},"1162":{"cik_str":1018963,"ticker":"ATI","title":"ATI INC"},"1163":{"cik_str":1521036,"ticker":"LNTH","title":"Lantheus Holdings, Inc."},"1164":{"cik_str":1688568,"ticker":"DXC","title":"DXC Technology Co"},"1165":{"cik_str":94845,"ticker":"LEVI","title":"LEVI STRAUSS & CO"},"1166":{"cik_str":1287865,"ticker":"MPW","title":"MEDICAL PROPERTIES TRUST INC"},"1167":{"cik_str":1163302,"ticker":"X","title":"UNITED STATES STEEL CORP"},"1168":{"cik_str":884614,"ticker":"UGI","title":"UGI CORP /PA/"},"1169":{"cik_str":775158,"ticker":"OSK","title":"OSHKOSH CORP"},"1170":{"cik_str":937098,"ticker":"TNET","title":"TRINET GROUP, INC."},"1171":{"cik_str":892450,"ticker":"KT","title":"KT CORP"},"1172":{"cik_str":1852131,"ticker":"NXT","title":"Nextracker Inc."},"1173":{"cik_str":1703056,"ticker":"ADT","title":"ADT Inc."},"1174":{"cik_str":1567683,"ticker":"CWEN","title":"Clearway Energy, Inc."},"1175":{"cik_str":1710864,"ticker":"MSSMY","title":"Misumi Group Inc./ADR"},"1176":{"cik_str":704532,"ticker":"ONTO","title":"ONTO INNOVATION INC."},"1177":{"cik_str":1040829,"ticker":"RHP","title":"Ryman Hospitality Properties, Inc."},"1178":{"cik_str":1794669,"ticker":"FOUR","title":"Shift4 Payments, Inc."},"1179":{"cik_str":1494319,"ticker":"UNVR","title":"Univar Solutions Inc."},"1180":{"cik_str":1791863,"ticker":"BEPC","title":"Brookfield Renewable Corp"},"1181":{"cik_str":1830043,"ticker":"BMBL","title":"Bumble Inc."},"1182":{"cik_str":109563,"ticker":"AIT","title":"APPLIED INDUSTRIAL TECHNOLOGIES INC"},"1183":{"cik_str":1308606,"ticker":"SRC","title":"SPIRIT REALTY CAPITAL, INC."},"1184":{"cik_str":1509589,"ticker":"CIVI","title":"CIVITAS RESOURCES, INC."},"1185":{"cik_str":1455955,"ticker":"PUTKY","title":"PT United Tractors Tbk / ADR"},"1186":{"cik_str":1553915,"ticker":"CRCBY","title":"Chongqing Rural Commercial Bank Co., Ltd./ADR"},"1187":{"cik_str":1604821,"ticker":"NTRA","title":"Natera, Inc."},"1188":{"cik_str":1530950,"ticker":"POST","title":"Post Holdings, Inc."},"1189":{"cik_str":1623925,"ticker":"AM","title":"Antero Midstream Corp"},"1190":{"cik_str":1370418,"ticker":"SLFPY","title":"STANDARD LIFE PLC"},"1191":{"cik_str":1492074,"ticker":"ABCM","title":"Abcam plc"},"1192":{"cik_str":1627223,"ticker":"CC","title":"Chemours Co"},"1193":{"cik_str":1906324,"ticker":"QDEL","title":"QuidelOrtho Corp"},"1194":{"cik_str":1757073,"ticker":"NVST","title":"Envista Holdings Corp"},"1195":{"cik_str":730263,"ticker":"THO","title":"THOR INDUSTRIES INC"},"1196":{"cik_str":1638833,"ticker":"SGRY","title":"Surgery Partners, Inc."},"1197":{"cik_str":57515,"ticker":"LANC","title":"LANCASTER COLONY CORP"},"1198":{"cik_str":1815846,"ticker":"MNSO","title":"MINISO Group Holding Ltd"},"1199":{"cik_str":887153,"ticker":"SIM","title":"GRUPO SIMEC, S.A.B. de C.V."},"1200":{"cik_str":78239,"ticker":"PVH","title":"PVH CORP. /DE/"},"1201":{"cik_str":875357,"ticker":"BOKF","title":"BOK FINANCIAL CORP"},"1202":{"cik_str":1800227,"ticker":"IAC","title":"IAC Inc."},"1203":{"cik_str":1749723,"ticker":"NFE","title":"New Fortress Energy Inc."},"1204":{"cik_str":28917,"ticker":"DDS","title":"DILLARD'S, INC."},"1205":{"cik_str":1820877,"ticker":"AIRC","title":"Apartment Income REIT Corp."},"1206":{"cik_str":1410939,"ticker":"ISEE","title":"IVERIC bio, Inc."},"1207":{"cik_str":1004434,"ticker":"AMG","title":"AFFILIATED MANAGERS GROUP, INC."},"1208":{"cik_str":833640,"ticker":"POWI","title":"POWER INTEGRATIONS INC"},"1209":{"cik_str":1068851,"ticker":"PB","title":"PROSPERITY BANCSHARES INC"},"1210":{"cik_str":816761,"ticker":"TDC","title":"TERADATA CORP /DE/"},"1211":{"cik_str":1447137,"ticker":"BKGFY","title":"Berkeley Group Holdings plc"},"1212":{"cik_str":1840776,"ticker":"HGTY","title":"Hagerty, Inc."},"1213":{"cik_str":771992,"ticker":"PAAS","title":"PAN AMERICAN SILVER CORP"},"1214":{"cik_str":1422183,"ticker":"FSK","title":"FS KKR Capital Corp"},"1215":{"cik_str":60519,"ticker":"LPX","title":"LOUISIANA-PACIFIC CORP"},"1216":{"cik_str":1003078,"ticker":"MSM","title":"MSC INDUSTRIAL DIRECT CO INC"},"1217":{"cik_str":1124941,"ticker":"BECN","title":"BEACON ROOFING SUPPLY INC"},"1218":{"cik_str":1451505,"ticker":"RIG","title":"Transocean Ltd."},"1219":{"cik_str":1428336,"ticker":"HQY","title":"HEALTHEQUITY, INC."},"1220":{"cik_str":1437578,"ticker":"BFAM","title":"BRIGHT HORIZONS FAMILY SOLUTIONS INC."},"1221":{"cik_str":1128928,"ticker":"FLO","title":"FLOWERS FOODS INC"},"1222":{"cik_str":1562476,"ticker":"TMHC","title":"Taylor Morrison Home Corp"},"1223":{"cik_str":1584207,"ticker":"OMF","title":"OneMain Holdings, Inc."},"1224":{"cik_str":1698990,"ticker":"MGY","title":"Magnolia Oil & Gas Corp"},"1225":{"cik_str":1868778,"ticker":"INFA","title":"Informatica Inc."},"1226":{"cik_str":1038074,"ticker":"SLAB","title":"SILICON LABORATORIES INC."},"1227":{"cik_str":1655888,"ticker":"ORCC","title":"Owl Rock Capital Corp"},"1228":{"cik_str":1125376,"ticker":"ENSG","title":"ENSIGN GROUP, INC"},"1229":{"cik_str":1821825,"ticker":"OGN","title":"Organon & Co."},"1230":{"cik_str":849869,"ticker":"SLGN","title":"SILGAN HOLDINGS INC"},"1231":{"cik_str":793952,"ticker":"HOG","title":"HARLEY-DAVIDSON, INC."},"1232":{"cik_str":1435969,"ticker":"MXCHY","title":"Orbia Advance Corporation, S.A.B. de C.V./ADR"},"1233":{"cik_str":1057877,"ticker":"IDA","title":"IDACORP INC"},"1234":{"cik_str":1674910,"ticker":"VVV","title":"VALVOLINE INC"},"1235":{"cik_str":1411207,"ticker":"ALSN","title":"Allison Transmission Holdings Inc"},"1236":{"cik_str":1445305,"ticker":"WK","title":"WORKIVA INC"},"1237":{"cik_str":824142,"ticker":"AAON","title":"AAON, INC."},"1238":{"cik_str":1144215,"ticker":"AYI","title":"ACUITY BRANDS INC"},"1239":{"cik_str":833079,"ticker":"MTH","title":"Meritage Homes CORP"},"1240":{"cik_str":1722608,"ticker":"IQ","title":"iQIYI, Inc."},"1241":{"cik_str":1809104,"ticker":"ALIT","title":"Alight, Inc. / Delaware"},"1242":{"cik_str":78150,"ticker":"PHI","title":"PLDT Inc."},"1243":{"cik_str":1822479,"ticker":"SHC","title":"Sotera Health Co"},"1244":{"cik_str":1032220,"ticker":"MMS","title":"MAXIMUS, INC."},"1245":{"cik_str":1144980,"ticker":"ABG","title":"ASBURY AUTOMOTIVE GROUP INC"},"1246":{"cik_str":1366561,"ticker":"SMAR","title":"SMARTSHEET INC"},"1247":{"cik_str":1603145,"ticker":"NEP","title":"NEXTERA ENERGY PARTNERS, LP"},"1248":{"cik_str":1674168,"ticker":"HGV","title":"Hilton Grand Vacations Inc."},"1249":{"cik_str":802481,"ticker":"PPC","title":"PILGRIMS PRIDE CORP"},"1250":{"cik_str":797721,"ticker":"VSAT","title":"VIASAT INC"},"1251":{"cik_str":1544522,"ticker":"FRSH","title":"Freshworks Inc."},"1252":{"cik_str":911177,"ticker":"CWST","title":"CASELLA WASTE SYSTEMS INC"},"1253":{"cik_str":1534504,"ticker":"PBF","title":"PBF Energy Inc."},"1254":{"cik_str":1114700,"ticker":"GGAL","title":"GRUPO FINANCIERO GALICIA SA"},"1255":{"cik_str":1692063,"ticker":"SNDR","title":"Schneider National, Inc."},"1256":{"cik_str":1799208,"ticker":"DNB","title":"Dun & Bradstreet Holdings, Inc."},"1257":{"cik_str":764038,"ticker":"SSB","title":"SouthState Corp"},"1258":{"cik_str":1381640,"ticker":"MLCO","title":"Melco Resorts & Entertainment LTD"},"1259":{"cik_str":1448893,"ticker":"ESNT","title":"Essent Group Ltd."},"1260":{"cik_str":1742924,"ticker":"LTHM","title":"Livent Corp."},"1261":{"cik_str":1520262,"ticker":"ALKS","title":"Alkermes plc."},"1262":{"cik_str":1592000,"ticker":"ENLC","title":"EnLink Midstream, LLC"},"1263":{"cik_str":275880,"ticker":"PSN","title":"PARSONS CORP"},"1264":{"cik_str":1286043,"ticker":"KRG","title":"KITE REALTY GROUP TRUST"},"1265":{"cik_str":1476150,"ticker":"TRNO","title":"Terreno Realty Corp"},"1266":{"cik_str":1788348,"ticker":"BIPC","title":"Brookfield Infrastructure Corp"},"1267":{"cik_str":1720671,"ticker":"HCP","title":"HashiCorp, Inc."},"1268":{"cik_str":1297989,"ticker":"EXLS","title":"ExlService Holdings, Inc."},"1269":{"cik_str":1944013,"ticker":"CR","title":"Crane Co"},"1270":{"cik_str":1307954,"ticker":"HUN","title":"Huntsman CORP"},"1271":{"cik_str":89439,"ticker":"MLI","title":"MUELLER INDUSTRIES INC"},"1272":{"cik_str":1618563,"ticker":"NSA","title":"National Storage Affiliates Trust"},"1273":{"cik_str":1786842,"ticker":"VNT","title":"Vontier Corp"},"1274":{"cik_str":1612042,"ticker":"ASND","title":"Ascendis Pharma A/S"},"1275":{"cik_str":1739104,"ticker":"ELAN","title":"Elanco Animal Health Inc"},"1276":{"cik_str":1805387,"ticker":"CERE","title":"Cerevel Therapeutics Holdings, Inc."},"1277":{"cik_str":12659,"ticker":"HRB","title":"H&R BLOCK INC"},"1278":{"cik_str":1489079,"ticker":"ADOOY","title":"Adaro Energy PT/ADR/"},"1279":{"cik_str":1660280,"ticker":"TENB","title":"Tenable Holdings, Inc."},"1280":{"cik_str":1050446,"ticker":"MSTR","title":"MICROSTRATEGY Inc"},"1281":{"cik_str":1772016,"ticker":"BRBR","title":"BELLRING BRANDS, INC."},"1282":{"cik_str":1510599,"ticker":"PDI","title":"PIMCO Dynamic Income Fund"},"1283":{"cik_str":932696,"ticker":"NSIT","title":"INSIGHT ENTERPRISES INC"},"1284":{"cik_str":30625,"ticker":"FLS","title":"FLOWSERVE CORP"},"1285":{"cik_str":1776967,"ticker":"NTCO","title":"Natura &Co Holding S.A."},"1286":{"cik_str":924805,"ticker":"FRHC","title":"Freedom Holding Corp."},"1287":{"cik_str":1159036,"ticker":"HALO","title":"HALOZYME THERAPEUTICS, INC."},"1288":{"cik_str":1603923,"ticker":"WFRD","title":"Weatherford International plc"},"1289":{"cik_str":1842022,"ticker":"DTM","title":"DT Midstream, Inc."},"1290":{"cik_str":855654,"ticker":"IMGN","title":"ImmunoGen, Inc."},"1291":{"cik_str":1178819,"ticker":"AGI","title":"ALAMOS GOLD INC"},"1292":{"cik_str":1296445,"ticker":"ORA","title":"ORMAT TECHNOLOGIES, INC."},"1293":{"cik_str":1050797,"ticker":"COLM","title":"COLUMBIA SPORTSWEAR CO"},"1294":{"cik_str":1360901,"ticker":"EVR","title":"Evercore Inc."},"1295":{"cik_str":1477720,"ticker":"ASAN","title":"Asana, Inc."},"1296":{"cik_str":38725,"ticker":"FELE","title":"FRANKLIN ELECTRIC CO INC"},"1297":{"cik_str":1820953,"ticker":"AFRM","title":"Affirm Holdings, Inc."},"1298":{"cik_str":314808,"ticker":"VAL","title":"Valaris Ltd"},"1299":{"cik_str":856982,"ticker":"MMSI","title":"MERIT MEDICAL SYSTEMS INC"},"1300":{"cik_str":1107843,"ticker":"QLYS","title":"QUALYS, INC."},"1301":{"cik_str":1429937,"ticker":"BTG","title":"B2GOLD CORP"},"1302":{"cik_str":70145,"ticker":"NFG","title":"NATIONAL FUEL GAS CO"},"1303":{"cik_str":52827,"ticker":"RYN","title":"RAYONIER INC"},"1304":{"cik_str":851520,"ticker":"EXPO","title":"EXPONENT INC"},"1305":{"cik_str":1378239,"ticker":"OMAB","title":"Central North Airport Group"},"1306":{"cik_str":1331520,"ticker":"HOMB","title":"HOME BANCSHARES INC"},"1307":{"cik_str":1469395,"ticker":"PAM","title":"Pampa Energy Inc."},"1308":{"cik_str":1437774,"ticker":"BZZUY","title":"BUZZI UNICEM S.p.A."},"1309":{"cik_str":1408710,"ticker":"FN","title":"Fabrinet"},"1310":{"cik_str":1439288,"ticker":"ZWS","title":"Zurn Elkay Water Solutions Corp"},"1311":{"cik_str":1448056,"ticker":"NEWR","title":"NEW RELIC, INC."},"1312":{"cik_str":1071438,"ticker":"BAK","title":"BRASKEM SA"},"1313":{"cik_str":711377,"ticker":"NEOG","title":"NEOGEN CORP"},"1314":{"cik_str":1487712,"ticker":"AL","title":"AIR LEASE CORP"},"1315":{"cik_str":1590714,"ticker":"ESI","title":"Element Solutions Inc"},"1316":{"cik_str":1569650,"ticker":"OZK","title":"Bank OZK"},"1317":{"cik_str":56047,"ticker":"KEX","title":"KIRBY CORP"},"1318":{"cik_str":1424929,"ticker":"FOXF","title":"FOX FACTORY HOLDING CORP"},"1319":{"cik_str":1493594,"ticker":"MTSI","title":"MACOM Technology Solutions Holdings, Inc."},"1320":{"cik_str":1649094,"ticker":"PCVX","title":"Vaxcyte, Inc."},"1321":{"cik_str":69891,"ticker":"FIZZ","title":"NATIONAL BEVERAGE CORP"},"1322":{"cik_str":1394832,"ticker":"ATS","title":"ATS Corp /ATS"},"1323":{"cik_str":356309,"ticker":"NJR","title":"NEW JERSEY RESOURCES CORP"},"1324":{"cik_str":30697,"ticker":"WEN","title":"Wendy's Co"},"1325":{"cik_str":1868941,"ticker":"FLNC","title":"Fluence Energy, Inc."},"1326":{"cik_str":40211,"ticker":"GATX","title":"GATX CORP"},"1327":{"cik_str":1660734,"ticker":"TRTN","title":"Triton International Ltd"},"1328":{"cik_str":1866368,"ticker":"CWAN","title":"Clearwater Analytics Holdings, Inc."},"1329":{"cik_str":1690820,"ticker":"CVNA","title":"CARVANA CO."},"1330":{"cik_str":1214816,"ticker":"AXS","title":"AXIS CAPITAL HOLDINGS LTD"},"1331":{"cik_str":876437,"ticker":"MTG","title":"MGIC INVESTMENT CORP"},"1332":{"cik_str":8858,"ticker":"AVT","title":"AVNET INC"},"1333":{"cik_str":1674862,"ticker":"ASH","title":"ASHLAND INC."},"1334":{"cik_str":1000753,"ticker":"NSP","title":"INSPERITY, INC."},"1335":{"cik_str":1659939,"ticker":"ENIC","title":"Enel Chile S.A."},"1336":{"cik_str":784977,"ticker":"POR","title":"PORTLAND GENERAL ELECTRIC CO /OR/"},"1337":{"cik_str":1524358,"ticker":"VAC","title":"MARRIOTT VACATIONS WORLDWIDE Corp"},"1338":{"cik_str":1692115,"ticker":"SWX","title":"Southwest Gas Holdings, Inc."},"1339":{"cik_str":1833756,"ticker":"DRS","title":"Leonardo DRS, Inc."},"1340":{"cik_str":1585364,"ticker":"PRGO","title":"PERRIGO Co plc"},"1341":{"cik_str":1274173,"ticker":"JHG","title":"JANUS HENDERSON GROUP PLC"},"1342":{"cik_str":18349,"ticker":"SNV","title":"SYNOVUS FINANCIAL CORP"},"1343":{"cik_str":1786909,"ticker":"SBSW","title":"Sibanye Stillwater Ltd"},"1344":{"cik_str":1556593,"ticker":"RITM","title":"Rithm Capital Corp."},"1345":{"cik_str":1476204,"ticker":"PECO","title":"Phillips Edison & Company, Inc."},"1346":{"cik_str":1645113,"ticker":"NVCR","title":"NovoCure Ltd"},"1347":{"cik_str":794367,"ticker":"M","title":"Macy's, Inc."},"1348":{"cik_str":1804745,"ticker":"DRVN","title":"Driven Brands Holdings Inc."},"1349":{"cik_str":20520,"ticker":"FYBR","title":"Frontier Communications Parent, Inc."},"1350":{"cik_str":1636519,"ticker":"MSGS","title":"Madison Square Garden Sports Corp."},"1351":{"cik_str":763901,"ticker":"BPOP","title":"POPULAR, INC."},"1352":{"cik_str":1015328,"ticker":"WTFC","title":"WINTRUST FINANCIAL CORP"},"1353":{"cik_str":1290109,"ticker":"LPL","title":"LG Display Co., Ltd."},"1354":{"cik_str":1289308,"ticker":"ENS","title":"EnerSys"},"1355":{"cik_str":40888,"ticker":"AJRD","title":"AEROJET ROCKETDYNE HOLDINGS, INC."},"1356":{"cik_str":1576789,"ticker":"WIX","title":"Wix.com Ltd."},"1357":{"cik_str":1674416,"ticker":"CRSP","title":"CRISPR Therapeutics AG"},"1358":{"cik_str":772406,"ticker":"CRUS","title":"CIRRUS LOGIC, INC."},"1359":{"cik_str":1782754,"ticker":"AZEK","title":"AZEK Co Inc."},"1360":{"cik_str":59558,"ticker":"LNC","title":"LINCOLN NATIONAL CORP"},"1361":{"cik_str":1583708,"ticker":"S","title":"SentinelOne, Inc."},"1362":{"cik_str":1115055,"ticker":"PNFP","title":"PINNACLE FINANCIAL PARTNERS INC"},"1363":{"cik_str":1621563,"ticker":"SUM","title":"Summit Materials, Inc."},"1364":{"cik_str":1094972,"ticker":"UGP","title":"ULTRAPAR HOLDINGS INC"},"1365":{"cik_str":822663,"ticker":"IPAR","title":"INTER PARFUMS INC"},"1366":{"cik_str":1365135,"ticker":"WU","title":"Western Union CO"},"1367":{"cik_str":913353,"ticker":"CIGI","title":"Colliers International Group Inc."},"1368":{"cik_str":1345105,"ticker":"CPA","title":"Copa Holdings, S.A."},"1369":{"cik_str":1610950,"ticker":"SYNH","title":"Syneos Health, Inc."},"1370":{"cik_str":1736035,"ticker":"BXSL","title":"Blackstone Secured Lending Fund"},"1371":{"cik_str":887343,"ticker":"COLB","title":"COLUMBIA BANKING SYSTEM, INC."},"1372":{"cik_str":1828016,"ticker":"PLTK","title":"Playtika Holding Corp."},"1373":{"cik_str":9326,"ticker":"BCPC","title":"BALCHEM CORP"},"1374":{"cik_str":1577526,"ticker":"AI","title":"C3.ai, Inc."},"1375":{"cik_str":1803599,"ticker":"CNXC","title":"Concentrix Corp"},"1376":{"cik_str":945764,"ticker":"DEN","title":"DENBURY INC"},"1377":{"cik_str":1496963,"ticker":"SQSP","title":"Squarespace, Inc."},"1378":{"cik_str":1530721,"ticker":"CPRI","title":"Capri Holdings Ltd"},"1379":{"cik_str":932417,"ticker":"VIAAY","title":"VIENNA INTERNATIONAL AIRPORT"},"1380":{"cik_str":1433642,"ticker":"HLNE","title":"Hamilton Lane INC"},"1381":{"cik_str":9092,"ticker":"BMI","title":"BADGER METER INC"},"1382":{"cik_str":1848309,"ticker":"SGML","title":"Sigma Lithium Corp"},"1383":{"cik_str":1347426,"ticker":"BMA","title":"Macro Bank Inc."},"1384":{"cik_str":861878,"ticker":"SRCL","title":"STERICYCLE INC"},"1385":{"cik_str":67716,"ticker":"MDU","title":"MDU RESOURCES GROUP INC"},"1386":{"cik_str":29002,"ticker":"DIOD","title":"DIODES INC /DEL/"},"1387":{"cik_str":1587732,"ticker":"OGS","title":"ONE Gas, Inc."},"1388":{"cik_str":1142750,"ticker":"AMN","title":"AMN HEALTHCARE SERVICES INC"},"1389":{"cik_str":1831651,"ticker":"SHLS","title":"Shoals Technologies Group, Inc."},"1390":{"cik_str":1602065,"ticker":"VNOM","title":"Viper Energy Partners LP"},"1391":{"cik_str":795266,"ticker":"KBH","title":"KB HOME"},"1392":{"cik_str":1466085,"ticker":"IRT","title":"INDEPENDENCE REALTY TRUST, INC."},"1393":{"cik_str":840489,"ticker":"FCFS","title":"FirstCash Holdings, Inc."},"1394":{"cik_str":1372612,"ticker":"BOX","title":"BOX INC"},"1395":{"cik_str":1676238,"ticker":"BRZE","title":"Braze, Inc."},"1396":{"cik_str":313143,"ticker":"HAE","title":"HAEMONETICS CORP"},"1397":{"cik_str":1124198,"ticker":"FLR","title":"FLUOR CORP"},"1398":{"cik_str":1411579,"ticker":"AMC","title":"AMC ENTERTAINMENT HOLDINGS, INC."},"1399":{"cik_str":1338749,"ticker":"PCH","title":"POTLATCHDELTIC CORP"},"1400":{"cik_str":1158449,"ticker":"AAP","title":"ADVANCE AUTO PARTS INC"},"1401":{"cik_str":1447945,"ticker":"ASMVY","title":"ASM Pacific Technology Ltd."},"1402":{"cik_str":37808,"ticker":"FNB","title":"FNB CORP/PA/"},"1403":{"cik_str":883984,"ticker":"ICUI","title":"ICU MEDICAL INC/DE"},"1404":{"cik_str":1477449,"ticker":"TDOC","title":"Teladoc Health, Inc."},"1405":{"cik_str":927003,"ticker":"AEIS","title":"ADVANCED ENERGY INDUSTRIES INC"},"1406":{"cik_str":1808220,"ticker":"GOCO","title":"GoHealth, Inc."},"1407":{"cik_str":36029,"ticker":"FFIN","title":"FIRST FINANCIAL BANKSHARES INC"},"1408":{"cik_str":1765159,"ticker":"NVEI","title":"Nuvei Corp"},"1409":{"cik_str":109380,"ticker":"ZION","title":"ZIONS BANCORPORATION, NATIONAL ASSOCIATION /UT/"},"1410":{"cik_str":1508633,"ticker":"IHCPF","title":"Inchcape plc/ADR"},"1411":{"cik_str":931427,"ticker":"TGS","title":"GAS TRANSPORTER OF THE SOUTH INC"},"1412":{"cik_str":1823529,"ticker":"ACT","title":"Enact Holdings, Inc."},"1413":{"cik_str":1747009,"ticker":"ETRN","title":"Equitrans Midstream Corp"},"1414":{"cik_str":88121,"ticker":"SEB","title":"SEABOARD CORP /DE/"},"1415":{"cik_str":1576280,"ticker":"GH","title":"Guardant Health, Inc."},"1416":{"cik_str":928876,"ticker":"TSEM","title":"TOWER SEMICONDUCTOR LTD"},"1417":{"cik_str":97216,"ticker":"TEX","title":"TEREX CORP"},"1418":{"cik_str":103730,"ticker":"VSH","title":"VISHAY INTERTECHNOLOGY INC"},"1419":{"cik_str":1157601,"ticker":"MDGL","title":"MADRIGAL PHARMACEUTICALS, INC."},"1420":{"cik_str":707179,"ticker":"ONB","title":"OLD NATIONAL BANCORP /IN/"},"1421":{"cik_str":1825155,"ticker":"TIXT","title":"TELUS International (Cda) Inc."},"1422":{"cik_str":1111335,"ticker":"VC","title":"VISTEON CORP"},"1423":{"cik_str":1801368,"ticker":"MP","title":"MP Materials Corp. / DE"},"1424":{"cik_str":1817358,"ticker":"ASO","title":"Academy Sports & Outdoors, Inc."},"1425":{"cik_str":1212545,"ticker":"WAL","title":"WESTERN ALLIANCE BANCORPORATION"},"1426":{"cik_str":816956,"ticker":"CNMD","title":"CONMED Corp"},"1427":{"cik_str":913142,"ticker":"BDC","title":"BELDEN INC."},"1428":{"cik_str":729986,"ticker":"UBSI","title":"UNITED BANKSHARES INC/WV"},"1429":{"cik_str":1839439,"ticker":"PYCR","title":"PAYCOR HCM, INC."},"1430":{"cik_str":1949543,"ticker":"STR","title":"Sitio Royalties Corp."},"1431":{"cik_str":1013857,"ticker":"PEGA","title":"PEGASYSTEMS INC"},"1432":{"cik_str":1714899,"ticker":"DNLI","title":"Denali Therapeutics Inc."},"1433":{"cik_str":714310,"ticker":"VLY","title":"VALLEY NATIONAL BANCORP"},"1434":{"cik_str":944695,"ticker":"THG","title":"HANOVER INSURANCE GROUP, INC."},"1435":{"cik_str":1130464,"ticker":"BKH","title":"BLACK HILLS CORP /SD/"},"1436":{"cik_str":1039828,"ticker":"AEL","title":"AMERICAN EQUITY INVESTMENT LIFE HOLDING CO"},"1437":{"cik_str":871763,"ticker":"MAN","title":"ManpowerGroup Inc."},"1438":{"cik_str":354707,"ticker":"HE","title":"HAWAIIAN ELECTRIC INDUSTRIES INC"},"1439":{"cik_str":890926,"ticker":"RDN","title":"RADIAN GROUP INC"},"1440":{"cik_str":85961,"ticker":"R","title":"RYDER SYSTEM INC"},"1441":{"cik_str":1032033,"ticker":"SLM","title":"SLM Corp"},"1442":{"cik_str":1498828,"ticker":"HHC","title":"Howard Hughes Corp"},"1443":{"cik_str":1549107,"ticker":"MANU","title":"Manchester United plc"},"1444":{"cik_str":1282637,"ticker":"NEU","title":"NEWMARKET CORP"},"1445":{"cik_str":58492,"ticker":"LEG","title":"LEGGETT & PLATT INC"},"1446":{"cik_str":1877322,"ticker":"ESAB","title":"ESAB Corp"},"1447":{"cik_str":1442655,"ticker":"NGKSY","title":"NGK Spark Plug Co., Ltd."},"1448":{"cik_str":1835681,"ticker":"PWSC","title":"POWERSCHOOL HOLDINGS, INC."},"1449":{"cik_str":1745431,"ticker":"STNE","title":"StoneCo Ltd."},"1450":{"cik_str":1320414,"ticker":"SEM","title":"SELECT MEDICAL HOLDINGS CORP"},"1451":{"cik_str":1633978,"ticker":"LITE","title":"Lumentum Holdings Inc."},"1452":{"cik_str":1364479,"ticker":"HRI","title":"HERC HOLDINGS INC"},"1453":{"cik_str":1580905,"ticker":"IBP","title":"Installed Building Products, Inc."},"1454":{"cik_str":277509,"ticker":"FSS","title":"FEDERAL SIGNAL CORP /DE/"},"1455":{"cik_str":1412558,"ticker":"EVO","title":"Evotec SE"},"1456":{"cik_str":1433660,"ticker":"JBT","title":"John Bean Technologies CORP"},"1457":{"cik_str":1437441,"ticker":"BPIRY","title":"Piraeus Bank S.A."},"1458":{"cik_str":1070494,"ticker":"ACAD","title":"ACADIA PHARMACEUTICALS INC"},"1459":{"cik_str":1108426,"ticker":"PNM","title":"PNM RESOURCES INC"},"1460":{"cik_str":1499620,"ticker":"TAL","title":"TAL Education Group"},"1461":{"cik_str":42582,"ticker":"GT","title":"GOODYEAR TIRE & RUBBER CO /OH/"},"1462":{"cik_str":1726122,"ticker":"CEF","title":"Sprott Physical Gold & Silver Trust"},"1463":{"cik_str":1363829,"ticker":"ESGR","title":"Enstar Group LTD"},"1464":{"cik_str":88205,"ticker":"SPXC","title":"SPX Technologies, Inc."},"1465":{"cik_str":1836470,"ticker":"SRAD","title":"Sportradar Group AG"},"1466":{"cik_str":1358762,"ticker":"RETA","title":"REATA PHARMACEUTICALS INC"},"1467":{"cik_str":1869198,"ticker":"LTH","title":"Life Time Group Holdings, Inc."},"1468":{"cik_str":39368,"ticker":"FUL","title":"FULLER H B CO"},"1469":{"cik_str":1469367,"ticker":"RUN","title":"Sunrun Inc."},"1470":{"cik_str":1834048,"ticker":"ASAI","title":"Sendas Distributor S.A."},"1471":{"cik_str":899689,"ticker":"VNO","title":"VORNADO REALTY TRUST"},"1472":{"cik_str":1532173,"ticker":"KOZAY","title":"Koza Altin Isletmeleri A.S./ADR"},"1473":{"cik_str":1280058,"ticker":"BLKB","title":"BLACKBAUD INC"},"1474":{"cik_str":1759509,"ticker":"LYFT","title":"Lyft, Inc."},"1475":{"cik_str":1718512,"ticker":"GTES","title":"Gates Industrial Corp plc"},"1476":{"cik_str":1597264,"ticker":"BPMC","title":"Blueprint Medicines Corp"},"1477":{"cik_str":16040,"ticker":"CBT","title":"CABOT CORP"},"1478":{"cik_str":949870,"ticker":"SAM","title":"BOSTON BEER CO INC"},"1479":{"cik_str":1494728,"ticker":"PSLV","title":"Sprott Physical Silver Trust"},"1480":{"cik_str":871459,"ticker":"DFIHY","title":"DAIRY FARM INTERNATIONAL HOLDINGS LTD /FI"},"1481":{"cik_str":1575515,"ticker":"SFM","title":"Sprouts Farmers Market, Inc."},"1482":{"cik_str":893538,"ticker":"SM","title":"SM Energy Co"},"1483":{"cik_str":879407,"ticker":"ARWR","title":"ARROWHEAD PHARMACEUTICALS, INC."},"1484":{"cik_str":1564538,"ticker":"GRP-UN","title":"GRANITE REAL ESTATE INVESTMENT TRUST"},"1485":{"cik_str":1122976,"ticker":"AVNT","title":"AVIENT CORP"},"1486":{"cik_str":90896,"ticker":"SKY","title":"Skyline Champion Corp"},"1487":{"cik_str":921738,"ticker":"PENN","title":"PENN Entertainment, Inc."},"1488":{"cik_str":1639300,"ticker":"OLLI","title":"Ollie's Bargain Outlet Holdings, Inc."},"1489":{"cik_str":837465,"ticker":"MODG","title":"Topgolf Callaway Brands Corp."},"1490":{"cik_str":1883313,"ticker":"SVV","title":"Savers Value Village, Inc."},"1491":{"cik_str":890564,"ticker":"ASGN","title":"ASGN Inc"},"1492":{"cik_str":814453,"ticker":"NWL","title":"NEWELL BRANDS INC."},"1493":{"cik_str":1830214,"ticker":"DNA","title":"Ginkgo Bioworks Holdings, Inc."},"1494":{"cik_str":806628,"ticker":"DNP","title":"DNP SELECT INCOME FUND INC"},"1495":{"cik_str":1672013,"ticker":"GOLF","title":"Acushnet Holdings Corp."},"1496":{"cik_str":1569345,"ticker":"CXM","title":"Sprinklr, Inc."},"1497":{"cik_str":1739445,"ticker":"ACA","title":"Arcosa, Inc."},"1498":{"cik_str":825313,"ticker":"AB","title":"ALLIANCEBERNSTEIN HOLDING L.P."},"1499":{"cik_str":1670541,"ticker":"ADNT","title":"Adient plc"},"1500":{"cik_str":46765,"ticker":"HP","title":"Helmerich & Payne, Inc."},"1501":{"cik_str":1564902,"ticker":"SEAS","title":"SeaWorld Entertainment, Inc."},"1502":{"cik_str":1632127,"ticker":"CABO","title":"Cable One, Inc."},"1503":{"cik_str":1025996,"ticker":"KRC","title":"KILROY REALTY CORP"},"1504":{"cik_str":1552275,"ticker":"SUN","title":"Sunoco LP"},"1505":{"cik_str":1545851,"ticker":"CPG","title":"Crescent Point Energy Corp."},"1506":{"cik_str":1499785,"ticker":"TJBH","title":"Tengjun Biotechnology Corp."},"1507":{"cik_str":1299939,"ticker":"CADE","title":"Cadence Bank"},"1508":{"cik_str":1811414,"ticker":"QS","title":"QuantumScape Corp"},"1509":{"cik_str":39899,"ticker":"TGNA","title":"TEGNA INC"},"1510":{"cik_str":1702744,"ticker":"SMPL","title":"Simply Good Foods Co"},"1511":{"cik_str":1031203,"ticker":"GPI","title":"GROUP 1 AUTOMOTIVE INC"},"1512":{"cik_str":1527636,"ticker":"ATHM","title":"Autohome Inc."},"1513":{"cik_str":1551306,"ticker":"PGNY","title":"Progyny, Inc."},"1514":{"cik_str":1846832,"ticker":"DLO","title":"dLocal Ltd"},"1515":{"cik_str":1559053,"ticker":"PRTA","title":"PROTHENA CORP PUBLIC LTD CO"},"1516":{"cik_str":825542,"ticker":"SMG","title":"SCOTTS MIRACLE-GRO CO"},"1517":{"cik_str":1841804,"ticker":"INST","title":"INSTRUCTURE HOLDINGS, INC."},"1518":{"cik_str":1258602,"ticker":"NNI","title":"NELNET INC"},"1519":{"cik_str":1001082,"ticker":"DISH","title":"DISH Network CORP"},"1520":{"cik_str":1417398,"ticker":"HI","title":"Hillenbrand, Inc."},"1521":{"cik_str":1769731,"ticker":"AMTD","title":"AMTD IDEA GROUP"},"1522":{"cik_str":1581990,"ticker":"PAGP","title":"PLAINS GP HOLDINGS LP"},"1523":{"cik_str":70866,"ticker":"NCR","title":"NCR CORP"},"1524":{"cik_str":1588489,"ticker":"GBTC","title":"Grayscale Bitcoin Trust (BTC)"},"1525":{"cik_str":1652130,"ticker":"NTLA","title":"Intellia Therapeutics, Inc."},"1526":{"cik_str":1328581,"ticker":"BCC","title":"BOISE CASCADE Co"},"1527":{"cik_str":1061630,"ticker":"BXMT","title":"BLACKSTONE MORTGAGE TRUST, INC."},"1528":{"cik_str":1728951,"ticker":"EPRT","title":"ESSENTIAL PROPERTIES REALTY TRUST, INC."},"1529":{"cik_str":81362,"ticker":"KWR","title":"QUAKER CHEMICAL CORP"},"1530":{"cik_str":1045450,"ticker":"EPR","title":"EPR PROPERTIES"},"1531":{"cik_str":1356570,"ticker":"WNS","title":"WNS (HOLDINGS) LTD"},"1532":{"cik_str":25232,"ticker":"CUZ","title":"COUSINS PROPERTIES INC"},"1533":{"cik_str":1381668,"ticker":"TFSL","title":"TFS Financial CORP"},"1534":{"cik_str":776867,"ticker":"WTM","title":"WHITE MOUNTAINS INSURANCE GROUP LTD"},"1535":{"cik_str":932470,"ticker":"TEO","title":"TELECOM ARGENTINA SA"},"1536":{"cik_str":1418121,"ticker":"APLE","title":"Apple Hospitality REIT, Inc."},"1537":{"cik_str":1856314,"ticker":"YOU","title":"Clear Secure, Inc."},"1538":{"cik_str":1380366,"ticker":"GNNDY","title":"GN STORE NORD A/S"},"1539":{"cik_str":868671,"ticker":"GBCI","title":"GLACIER BANCORP, INC."},"1540":{"cik_str":1441683,"ticker":"APPN","title":"APPIAN CORP"},"1541":{"cik_str":1582961,"ticker":"DOCN","title":"DigitalOcean Holdings, Inc."},"1542":{"cik_str":1745916,"ticker":"PFSI","title":"PennyMac Financial Services, Inc."},"1543":{"cik_str":897723,"ticker":"SANM","title":"SANMINA CORP"},"1544":{"cik_str":1049659,"ticker":"SID","title":"NATIONAL STEEL CO"},"1545":{"cik_str":1178879,"ticker":"FOLD","title":"AMICUS THERAPEUTICS, INC."},"1546":{"cik_str":1828108,"ticker":"AUR","title":"Aurora Innovation, Inc."},"1547":{"cik_str":1090633,"ticker":"IIJIY","title":"INTERNET INITIATIVE JAPAN INC"},"1548":{"cik_str":1490978,"ticker":"SDGR","title":"Schrodinger, Inc."},"1549":{"cik_str":1651052,"ticker":"FOCS","title":"Focus Financial Partners Inc."},"1550":{"cik_str":1446702,"ticker":"UBSFY","title":"Ubisoft Entertainment SA"},"1551":{"cik_str":933136,"ticker":"COOP","title":"Mr. Cooper Group Inc."},"1552":{"cik_str":108516,"ticker":"WOR","title":"WORTHINGTON INDUSTRIES INC"},"1553":{"cik_str":1574540,"ticker":"DOC","title":"Physicians Realty Trust"},"1554":{"cik_str":1126956,"ticker":"SR","title":"SPIRE INC"},"1555":{"cik_str":1420800,"ticker":"ENOV","title":"Enovis CORP"},"1556":{"cik_str":43920,"ticker":"GEF","title":"GREIF, INC"},"1557":{"cik_str":773141,"ticker":"MDC","title":"M.D.C. HOLDINGS, INC."},"1558":{"cik_str":67887,"ticker":"MOG-A","title":"MOOG INC."},"1559":{"cik_str":817720,"ticker":"SYNA","title":"SYNAPTICS Inc"},"1560":{"cik_str":1580560,"ticker":"FLYW","title":"Flywire Corp"},"1561":{"cik_str":73088,"ticker":"NWE","title":"NORTHWESTERN CORP"},"1562":{"cik_str":1012019,"ticker":"RUSHA","title":"RUSH ENTERPRISES INC TX"},"1563":{"cik_str":1664703,"ticker":"BE","title":"Bloom Energy Corp"},"1564":{"cik_str":812074,"ticker":"OI","title":"O-I Glass, Inc. /DE/"},"1565":{"cik_str":1001316,"ticker":"TGTX","title":"TG THERAPEUTICS, INC."},"1566":{"cik_str":1078271,"ticker":"EXTR","title":"EXTREME NETWORKS INC"},"1567":{"cik_str":1628908,"ticker":"EVH","title":"Evolent Health, Inc."},"1568":{"cik_str":750577,"ticker":"HWC","title":"HANCOCK WHITNEY CORP"},"1569":{"cik_str":1902733,"ticker":"NCNO","title":"nCino, Inc."},"1570":{"cik_str":1845257,"ticker":"LFST","title":"LifeStance Health Group, Inc."},"1571":{"cik_str":1280263,"ticker":"AMBA","title":"AMBARELLA INC"},"1572":{"cik_str":1670592,"ticker":"YETI","title":"YETI Holdings, Inc."},"1573":{"cik_str":1820721,"ticker":"ARRY","title":"Array Technologies, Inc."},"1574":{"cik_str":1192448,"ticker":"GKOS","title":"GLAUKOS Corp"},"1575":{"cik_str":874866,"ticker":"CRVL","title":"CORVEL CORP"},"1576":{"cik_str":917520,"ticker":"IART","title":"INTEGRA LIFESCIENCES HOLDINGS CORP"},"1577":{"cik_str":7431,"ticker":"AWI","title":"ARMSTRONG WORLD INDUSTRIES INC"},"1578":{"cik_str":1644406,"ticker":"TWNK","title":"Hostess Brands, Inc."},"1579":{"cik_str":66756,"ticker":"ALE","title":"ALLETE INC"},"1580":{"cik_str":1782170,"ticker":"RELY","title":"Remitly Global, Inc."},"1581":{"cik_str":72333,"ticker":"JWN","title":"NORDSTROM INC"},"1582":{"cik_str":1621434,"ticker":"BSM","title":"Black Stone Minerals, L.P."},"1583":{"cik_str":1647822,"ticker":"SVMB","title":"SavMobi Technology Inc."},"1584":{"cik_str":56978,"ticker":"KLIC","title":"KULICKE & SOFFA INDUSTRIES INC"},"1585":{"cik_str":1084048,"ticker":"ZD","title":"ZIFF DAVIS, INC."},"1586":{"cik_str":1820872,"ticker":"GBTG","title":"Global Business Travel Group, Inc."},"1587":{"cik_str":67215,"ticker":"DY","title":"DYCOM INDUSTRIES INC"},"1588":{"cik_str":1697884,"ticker":"VSBC","title":"VITASPRING BIOMEDICAL CO. LTD."},"1589":{"cik_str":1515673,"ticker":"RARE","title":"Ultragenyx Pharmaceutical Inc."},"1590":{"cik_str":1620533,"ticker":"SHAK","title":"Shake Shack Inc."},"1591":{"cik_str":1706431,"ticker":"VIR","title":"Vir Biotechnology, Inc."},"1592":{"cik_str":1109345,"ticker":"NVMI","title":"NOVA LTD."},"1593":{"cik_str":1466593,"ticker":"OTTR","title":"Otter Tail Corp"},"1594":{"cik_str":1577916,"ticker":"PINC","title":"Premier, Inc."},"1595":{"cik_str":1195737,"ticker":"NEA","title":"Nuveen AMT-Free Quality Municipal Income Fund"},"1596":{"cik_str":926326,"ticker":"OMCL","title":"OMNICELL, INC."},"1597":{"cik_str":1406666,"ticker":"CALX","title":"CALIX, INC"},"1598":{"cik_str":850460,"ticker":"WIRE","title":"ENCORE WIRE CORP"},"1599":{"cik_str":1273813,"ticker":"AGO","title":"ASSURED GUARANTY LTD"},"1600":{"cik_str":1880866,"ticker":"VTSCY","title":"Vitesco Technologies Group AG /ADR"},"1601":{"cik_str":1737706,"ticker":"SDRL","title":"Seadrill Ltd"},"1602":{"cik_str":1561680,"ticker":"TPH","title":"Tri Pointe Homes, Inc."},"1603":{"cik_str":39911,"ticker":"GPS","title":"GAP INC"},"1604":{"cik_str":1440972,"ticker":"LAC","title":"LITHIUM AMERICAS CORP."},"1605":{"cik_str":1666134,"ticker":"BL","title":"BLACKLINE, INC."},"1606":{"cik_str":913290,"ticker":"FRO","title":"Frontline plc"},"1607":{"cik_str":719413,"ticker":"HL","title":"HECLA MINING CO/DE/"},"1608":{"cik_str":1396440,"ticker":"MAIN","title":"Main Street Capital CORP"},"1609":{"cik_str":1056288,"ticker":"FHI","title":"FEDERATED HERMES, INC."},"1610":{"cik_str":1531048,"ticker":"NARI","title":"Inari Medical, Inc."},"1611":{"cik_str":780571,"ticker":"ITRI","title":"ITRON, INC."},"1612":{"cik_str":1056903,"ticker":"AWR","title":"AMERICAN STATES WATER CO"},"1613":{"cik_str":1158324,"ticker":"CCOI","title":"COGENT COMMUNICATIONS HOLDINGS, INC."},"1614":{"cik_str":1384905,"ticker":"RNG","title":"RingCentral, Inc."},"1615":{"cik_str":1611647,"ticker":"FRPT","title":"Freshpet, Inc."},"1616":{"cik_str":1299130,"ticker":"PACB","title":"PACIFIC BIOSCIENCES OF CALIFORNIA, INC."},"1617":{"cik_str":1609253,"ticker":"CRC","title":"California Resources Corp"},"1618":{"cik_str":1337619,"ticker":"ENV","title":"ENVESTNET, INC."},"1619":{"cik_str":109177,"ticker":"SPB","title":"Spectrum Brands Holdings, Inc."},"1620":{"cik_str":763744,"ticker":"LCII","title":"LCI INDUSTRIES"},"1621":{"cik_str":1876183,"ticker":"IHS","title":"IHS Holding Ltd"},"1622":{"cik_str":1104485,"ticker":"NOG","title":"NORTHERN OIL & GAS, INC."},"1623":{"cik_str":78890,"ticker":"BCO","title":"BRINKS CO"},"1624":{"cik_str":1685040,"ticker":"BHF","title":"Brighthouse Financial, Inc."},"1625":{"cik_str":1061983,"ticker":"CYTK","title":"CYTOKINETICS INC"},"1626":{"cik_str":1434588,"ticker":"LOPE","title":"Grand Canyon Education, Inc."},"1627":{"cik_str":23194,"ticker":"CRK","title":"COMSTOCK RESOURCES INC"},"1628":{"cik_str":1689923,"ticker":"AYX","title":"Alteryx, Inc."},"1629":{"cik_str":1169561,"ticker":"CVLT","title":"COMMVAULT SYSTEMS INC"},"1630":{"cik_str":1064728,"ticker":"BTU","title":"PEABODY ENERGY CORP"},"1631":{"cik_str":1625297,"ticker":"INDV","title":"INDIVIOR PLC"},"1632":{"cik_str":1595761,"ticker":"WB","title":"WEIBO Corp"},"1633":{"cik_str":1388658,"ticker":"IRTC","title":"iRhythm Technologies, Inc."},"1634":{"cik_str":1553920,"ticker":"BBMPY","title":"BBMG Corporation/ADR"},"1635":{"cik_str":1647639,"ticker":"UPST","title":"Upstart Holdings, Inc."},"1636":{"cik_str":1604481,"ticker":"EURN","title":"Euronav NV"},"1637":{"cik_str":25445,"ticker":"CXT","title":"Crane NXT, Co."},"1638":{"cik_str":1336917,"ticker":"UAA","title":"Under Armour, Inc."},"1639":{"cik_str":1590364,"ticker":"FTAI","title":"FTAI Aviation Ltd."},"1640":{"cik_str":1071321,"ticker":"TKC","title":"TURKCELL ILETISIM HIZMETLERI A S"},"1641":{"cik_str":1364885,"ticker":"SPR","title":"Spirit AeroSystems Holdings, Inc."},"1642":{"cik_str":912615,"ticker":"URBN","title":"URBAN OUTFITTERS INC"},"1643":{"cik_str":1877787,"ticker":"ZGN","title":"Ermenegildo Zegna N.V."},"1644":{"cik_str":1823239,"ticker":"MRVI","title":"MARAVAI LIFESCIENCES HOLDINGS, INC."},"1645":{"cik_str":1712807,"ticker":"PAGS","title":"PagSeguro Digital Ltd."},"1646":{"cik_str":1742692,"ticker":"INMD","title":"InMode Ltd."},"1647":{"cik_str":1580670,"ticker":"LGIH","title":"LGI Homes, Inc."},"1648":{"cik_str":876883,"ticker":"STGW","title":"Stagwell Inc"},"1649":{"cik_str":1495932,"ticker":"EXPI","title":"EXP World Holdings, Inc."},"1650":{"cik_str":891024,"ticker":"PDCO","title":"PATTERSON COMPANIES, INC."},"1651":{"cik_str":1651717,"ticker":"NOMD","title":"Nomad Foods Ltd"},"1652":{"cik_str":1863105,"ticker":"ESMT","title":"EngageSmart, Inc."},"1653":{"cik_str":860748,"ticker":"KMPR","title":"KEMPER Corp"},"1654":{"cik_str":1126874,"ticker":"ERF","title":"ENERPLUS Corp"},"1655":{"cik_str":1424182,"ticker":"BNL","title":"Broadstone Net Lease, Inc."},"1656":{"cik_str":1361658,"ticker":"TNL","title":"Travel & Leisure Co."},"1657":{"cik_str":1777393,"ticker":"CHPT","title":"ChargePoint Holdings, Inc."},"1658":{"cik_str":1070081,"ticker":"PTCT","title":"PTC THERAPEUTICS, INC."},"1659":{"cik_str":888746,"ticker":"CCU","title":"UNITED BREWERIES CO INC"},"1660":{"cik_str":1934850,"ticker":"FG","title":"F&G Annuities & Life, Inc."},"1661":{"cik_str":1932737,"ticker":"NWTN","title":"NWTN, Inc."},"1662":{"cik_str":1579428,"ticker":"AXSM","title":"Axsome Therapeutics, Inc."},"1663":{"cik_str":1853513,"ticker":"MCW","title":"Mister Car Wash, Inc."},"1664":{"cik_str":785956,"ticker":"JJSF","title":"J&J SNACK FOODS CORP"},"1665":{"cik_str":1070235,"ticker":"BB","title":"BLACKBERRY Ltd"},"1666":{"cik_str":832988,"ticker":"SIG","title":"SIGNET JEWELERS LTD"},"1667":{"cik_str":84748,"ticker":"ROG","title":"ROGERS CORP"},"1668":{"cik_str":1596967,"ticker":"MC","title":"Moelis & Co"},"1669":{"cik_str":760498,"ticker":"BANF","title":"BANCFIRST CORP /OK/"},"1670":{"cik_str":1477641,"ticker":"DQ","title":"DAQO NEW ENERGY CORP."},"1671":{"cik_str":1867072,"ticker":"KD","title":"Kyndryl Holdings, Inc."},"1672":{"cik_str":1711279,"ticker":"KRYS","title":"Krystal Biotech, Inc."},"1673":{"cik_str":101382,"ticker":"UMBF","title":"UMB FINANCIAL CORP"},"1674":{"cik_str":310142,"ticker":"SXT","title":"SENSIENT TECHNOLOGIES CORP"},"1675":{"cik_str":933974,"ticker":"AZTA","title":"Azenta, Inc."},"1676":{"cik_str":1771515,"ticker":"GO","title":"Grocery Outlet Holding Corp."},"1677":{"cik_str":1376139,"ticker":"CVI","title":"CVR ENERGY INC"},"1678":{"cik_str":896262,"ticker":"AMED","title":"AMEDISYS INC"},"1679":{"cik_str":1828318,"ticker":"ENVX","title":"Enovix Corp"},"1680":{"cik_str":1790982,"ticker":"ARNC","title":"Arconic Corp"},"1681":{"cik_str":1682745,"ticker":"VRRM","title":"VERRA MOBILITY Corp"},"1682":{"cik_str":104918,"ticker":"AVA","title":"AVISTA CORP"},"1683":{"cik_str":1759655,"ticker":"PRVA","title":"Privia Health Group, Inc."},"1684":{"cik_str":857855,"ticker":"UCBI","title":"UNITED COMMUNITY BANKS INC"},"1685":{"cik_str":921638,"ticker":"SSRM","title":"SSR MINING INC."},"1686":{"cik_str":1158463,"ticker":"JBLU","title":"JETBLUE AIRWAYS CORP"},"1687":{"cik_str":1617406,"ticker":"PK","title":"Park Hotels & Resorts Inc."},"1688":{"cik_str":1070412,"ticker":"CNX","title":"CNX Resources Corp"},"1689":{"cik_str":912892,"ticker":"TV","title":"GRUPO TELEVISA, S.A.B."},"1690":{"cik_str":1114483,"ticker":"ITGR","title":"Integer Holdings Corp"},"1691":{"cik_str":1295947,"ticker":"PBH","title":"Prestige Consumer Healthcare Inc."},"1692":{"cik_str":1827090,"ticker":"CERT","title":"Certara, Inc."},"1693":{"cik_str":1627272,"ticker":"OR","title":"Osisko Gold Royalties LTD"},"1694":{"cik_str":1085869,"ticker":"PRFT","title":"PERFICIENT INC"},"1695":{"cik_str":1741530,"ticker":"QFIN","title":"Qifu Technology, Inc."},"1696":{"cik_str":1035201,"ticker":"CWT","title":"CALIFORNIA WATER SERVICE GROUP"},"1697":{"cik_str":1361113,"ticker":"VRNS","title":"VARONIS SYSTEMS INC"},"1698":{"cik_str":1656081,"ticker":"DAVA","title":"Endava plc"},"1699":{"cik_str":717954,"ticker":"UNF","title":"UNIFIRST CORP"},"1700":{"cik_str":1639825,"ticker":"PTON","title":"PELOTON INTERACTIVE, INC."},"1701":{"cik_str":910108,"ticker":"LXP","title":"LXP Industrial Trust"},"1702":{"cik_str":1828365,"ticker":"RLX","title":"RLX Technology Inc."},"1703":{"cik_str":1284812,"ticker":"CNS","title":"COHEN & STEERS, INC."},"1704":{"cik_str":1446694,"ticker":"SHWDY","title":"Showa Denko K.K./ADR"},"1705":{"cik_str":819793,"ticker":"AIN","title":"ALBANY INTERNATIONAL CORP /DE/"},"1706":{"cik_str":1161728,"ticker":"MGEE","title":"MGE ENERGY INC"},"1707":{"cik_str":1597553,"ticker":"SAGE","title":"Sage Therapeutics, Inc."},"1708":{"cik_str":1806837,"ticker":"VERX","title":"Vertex, Inc."},"1709":{"cik_str":1819994,"ticker":"RKLB","title":"Rocket Lab USA, Inc."},"1710":{"cik_str":1320695,"ticker":"THS","title":"TreeHouse Foods, Inc."},"1711":{"cik_str":745308,"ticker":"JOE","title":"ST JOE Co"},"1712":{"cik_str":1628171,"ticker":"RVMD","title":"Revolution Medicines, Inc."},"1713":{"cik_str":1600438,"ticker":"GMS","title":"GMS Inc."},"1714":{"cik_str":885590,"ticker":"BHC","title":"Bausch Health Companies Inc."},"1715":{"cik_str":1495153,"ticker":"MMYT","title":"MakeMyTrip Ltd"},"1716":{"cik_str":1829726,"ticker":"TFPM","title":"Triple Flag Precious Metals Corp."},"1717":{"cik_str":1615903,"ticker":"IFS","title":"Intercorp Financial Services Inc."},"1718":{"cik_str":886977,"ticker":"MEOH","title":"METHANEX CORP"},"1719":{"cik_str":3453,"ticker":"MATX","title":"Matson, Inc."},"1720":{"cik_str":1311370,"ticker":"LAZ","title":"Lazard Ltd"},"1721":{"cik_str":1454480,"ticker":"SHZNY","title":"Shenzhen Expressway Co. / ADR"},"1722":{"cik_str":1104506,"ticker":"INSM","title":"INSMED Inc"},"1723":{"cik_str":1704292,"ticker":"ZLAB","title":"Zai Lab Ltd"},"1724":{"cik_str":1355444,"ticker":"ERJ","title":"EMBRAER S.A."},"1725":{"cik_str":1279495,"ticker":"BTE","title":"BAYTEX ENERGY CORP."},"1726":{"cik_str":1136352,"ticker":"CEQP","title":"Crestwood Equity Partners LP"},"1727":{"cik_str":1060822,"ticker":"CRI","title":"CARTERS INC"},"1728":{"cik_str":793074,"ticker":"WERN","title":"WERNER ENTERPRISES INC"},"1729":{"cik_str":771497,"ticker":"ABM","title":"ABM INDUSTRIES INC /DE/"},"1730":{"cik_str":315709,"ticker":"IBOC","title":"INTERNATIONAL BANCSHARES CORP"},"1731":{"cik_str":1164863,"ticker":"NPO","title":"ENPRO INDUSTRIES, INC"},"1732":{"cik_str":894315,"ticker":"SITC","title":"SITE Centers Corp."},"1733":{"cik_str":1823652,"ticker":"EVEX","title":"Eve Holding, Inc."},"1734":{"cik_str":1592386,"ticker":"VIRT","title":"Virtu Financial, Inc."},"1735":{"cik_str":1743881,"ticker":"BBIO","title":"BridgeBio Pharma, Inc."},"1736":{"cik_str":1297184,"ticker":"AMPH","title":"Amphastar Pharmaceuticals, Inc."},"1737":{"cik_str":1509991,"ticker":"KOS","title":"Kosmos Energy Ltd."},"1738":{"cik_str":1504764,"ticker":"AVAL","title":"Grupo Aval Acciones Y Valores S.A."},"1739":{"cik_str":860546,"ticker":"OFC","title":"CORPORATE OFFICE PROPERTIES TRUST"},"1740":{"cik_str":1626115,"ticker":"PJT","title":"PJT Partners Inc."},"1741":{"cik_str":1801169,"ticker":"OPEN","title":"Opendoor Technologies Inc."},"1742":{"cik_str":1699136,"ticker":"WHD","title":"Cactus, Inc."},"1743":{"cik_str":1800667,"ticker":"FROG","title":"JFrog Ltd"},"1744":{"cik_str":1842718,"ticker":"IAS","title":"INTEGRAL AD SCIENCE HOLDING CORP."},"1745":{"cik_str":912728,"ticker":"FWRD","title":"FORWARD AIR CORP"},"1746":{"cik_str":1828962,"ticker":"CRCT","title":"Cricut, Inc."},"1747":{"cik_str":98222,"ticker":"TDW","title":"TIDEWATER INC"},"1748":{"cik_str":1639691,"ticker":"LIVN","title":"LivaNova PLC"},"1749":{"cik_str":1492298,"ticker":"SBRA","title":"Sabra Health Care REIT, Inc."},"1750":{"cik_str":1224608,"ticker":"CNO","title":"CNO Financial Group, Inc."},"1751":{"cik_str":17843,"ticker":"CRS","title":"CARPENTER TECHNOLOGY CORP"},"1752":{"cik_str":1565687,"ticker":"INTA","title":"Intapp, Inc."},"1753":{"cik_str":921082,"ticker":"HIW","title":"HIGHWOODS PROPERTIES, INC."},"1754":{"cik_str":1560327,"ticker":"RPD","title":"Rapid7, Inc."},"1755":{"cik_str":1253986,"ticker":"ABR","title":"ARBOR REALTY TRUST INC"},"1756":{"cik_str":1831617,"ticker":"DSEY","title":"Diversey Holdings, Ltd."},"1757":{"cik_str":1653653,"ticker":"RRR","title":"Red Rock Resorts, Inc."},"1758":{"cik_str":1601072,"ticker":"AY","title":"Atlantica Sustainable Infrastructure plc"},"1759":{"cik_str":940942,"ticker":"HUBG","title":"Hub Group, Inc."},"1760":{"cik_str":785786,"ticker":"PLXS","title":"PLEXUS CORP"},"1761":{"cik_str":1824920,"ticker":"IONQ","title":"IonQ, Inc."},"1762":{"cik_str":1796022,"ticker":"STEP","title":"StepStone Group Inc."},"1763":{"cik_str":1714174,"ticker":"BUR","title":"Burford Capital Ltd"},"1764":{"cik_str":885639,"ticker":"KSS","title":"KOHLS Corp"},"1765":{"cik_str":1517302,"ticker":"APAM","title":"Artisan Partners Asset Management Inc."},"1766":{"cik_str":1671927,"ticker":"IMCR","title":"Immunocore Holdings plc"},"1767":{"cik_str":104889,"ticker":"GHC","title":"Graham Holdings Co"},"1768":{"cik_str":1834622,"ticker":"HAYW","title":"Hayward Holdings, Inc."},"1769":{"cik_str":1850270,"ticker":"PROK","title":"PROKIDNEY CORP."},"1770":{"cik_str":1421876,"ticker":"GLPG","title":"GALAPAGOS NV"},"1771":{"cik_str":1592438,"ticker":"ROYMY","title":"Royal Mail plc/ADR"},"1772":{"cik_str":1756262,"ticker":"TMDX","title":"TransMedics Group, Inc."},"1773":{"cik_str":944148,"ticker":"CBZ","title":"CBIZ, Inc."},"1774":{"cik_str":1497770,"ticker":"WD","title":"Walker & Dunlop, Inc."},"1775":{"cik_str":912958,"ticker":"TIGO","title":"MILLICOM INTERNATIONAL CELLULAR SA"},"1776":{"cik_str":1561894,"ticker":"HASI","title":"Hannon Armstrong Sustainable Infrastructure Capital, Inc."},"1777":{"cik_str":1023514,"ticker":"HMY","title":"HARMONY GOLD MINING CO LTD"},"1778":{"cik_str":1929561,"ticker":"RXO","title":"RXO, Inc."},"1779":{"cik_str":866706,"ticker":"ESE","title":"ESCO TECHNOLOGIES INC"},"1780":{"cik_str":1807192,"ticker":"CD","title":"Chindata Group Holdings Ltd"},"1781":{"cik_str":1637873,"ticker":"ACVA","title":"ACV Auctions Inc."},"1782":{"cik_str":1364250,"ticker":"DEI","title":"Douglas Emmett Inc"},"1783":{"cik_str":916789,"ticker":"HELE","title":"HELEN OF TROY LTD"},"1784":{"cik_str":1368622,"ticker":"AVAV","title":"AeroVironment Inc"},"1785":{"cik_str":1451809,"ticker":"SITM","title":"SITIME Corp"},"1786":{"cik_str":1083839,"ticker":"NAD","title":"Nuveen Quality Municipal Income Fund"},"1787":{"cik_str":1522540,"ticker":"MQ","title":"Marqeta, Inc."},"1788":{"cik_str":1622194,"ticker":"DEA","title":"Easterly Government Properties, Inc."},"1789":{"cik_str":1834488,"ticker":"NABL","title":"N-able, Inc."},"1790":{"cik_str":1570187,"ticker":"OSTIY","title":"Osterreichische Post AG ADR"},"1791":{"cik_str":1679363,"ticker":"MORF","title":"Morphic Holding, Inc."},"1792":{"cik_str":1579877,"ticker":"OUT","title":"OUTFRONT Media Inc."},"1793":{"cik_str":723188,"ticker":"CBU","title":"COMMUNITY BANK SYSTEM, INC."},"1794":{"cik_str":1507605,"ticker":"MARA","title":"MARATHON DIGITAL HOLDINGS, INC."},"1795":{"cik_str":1727263,"ticker":"FTDR","title":"Frontdoor, Inc."},"1796":{"cik_str":1039399,"ticker":"FORM","title":"FORMFACTOR INC"},"1797":{"cik_str":1435049,"ticker":"KDNY","title":"CHINOOK THERAPEUTICS, INC."},"1798":{"cik_str":1758057,"ticker":"LAZR","title":"Luminar Technologies, Inc./DE"},"1799":{"cik_str":56679,"ticker":"KFY","title":"KORN FERRY"},"1800":{"cik_str":1740332,"ticker":"REZI","title":"RESIDEO TECHNOLOGIES, INC."},"1801":{"cik_str":1823306,"ticker":"LSPD","title":"Lightspeed Commerce Inc."},"1802":{"cik_str":1405495,"ticker":"IDCC","title":"InterDigital, Inc."},"1803":{"cik_str":105132,"ticker":"WDFC","title":"WD 40 CO"},"1804":{"cik_str":1494259,"ticker":"CARG","title":"CarGurus, Inc."},"1805":{"cik_str":1517375,"ticker":"SPT","title":"Sprout Social, Inc."},"1806":{"cik_str":1868726,"ticker":"OLPX","title":"OLAPLEX HOLDINGS, INC."},"1807":{"cik_str":876167,"ticker":"PRGS","title":"PROGRESS SOFTWARE CORP /MA"},"1808":{"cik_str":889900,"ticker":"PTEN","title":"PATTERSON UTI ENERGY INC"},"1809":{"cik_str":1288469,"ticker":"MXL","title":"MAXLINEAR, INC"},"1810":{"cik_str":1603756,"ticker":"AXNX","title":"Axonics, Inc."},"1811":{"cik_str":886346,"ticker":"KAI","title":"KADANT INC"},"1812":{"cik_str":912242,"ticker":"MAC","title":"MACERICH CO"},"1813":{"cik_str":1624794,"ticker":"CSWI","title":"CSW INDUSTRIALS, INC."},"1814":{"cik_str":1610250,"ticker":"BOOT","title":"Boot Barn Holdings, Inc."},"1815":{"cik_str":1955520,"ticker":"KNF","title":"Knife River Corp"},"1816":{"cik_str":1077428,"ticker":"TCBI","title":"TEXAS CAPITAL BANCSHARES INC/TX"},"1817":{"cik_str":1373670,"ticker":"GRBK","title":"Green Brick Partners, Inc."},"1818":{"cik_str":1459200,"ticker":"ALRM","title":"Alarm.com Holdings, Inc."},"1819":{"cik_str":1822993,"ticker":"JXN","title":"Jackson Financial Inc."},"1820":{"cik_str":1350593,"ticker":"MWA","title":"Mueller Water Products, Inc."},"1821":{"cik_str":278166,"ticker":"CVCO","title":"CAVCO INDUSTRIES INC."},"1822":{"cik_str":1483934,"ticker":"STNG","title":"Scorpio Tankers Inc."},"1823":{"cik_str":718937,"ticker":"STAA","title":"STAAR SURGICAL CO"},"1824":{"cik_str":1328237,"ticker":"DBC","title":"Invesco DB Commodity Index Tracking Fund"},"1825":{"cik_str":912093,"ticker":"VIAV","title":"VIAVI SOLUTIONS INC."},"1826":{"cik_str":1054905,"ticker":"IOSP","title":"INNOSPEC INC."},"1827":{"cik_str":860413,"ticker":"FIBK","title":"FIRST INTERSTATE BANCSYSTEM INC"},"1828":{"cik_str":935036,"ticker":"ACIW","title":"ACI WORLDWIDE, INC."},"1829":{"cik_str":1807794,"ticker":"CRDO","title":"Credo Technology Group Holding Ltd"},"1830":{"cik_str":1090116,"ticker":"NVG","title":"Nuveen AMT-Free Municipal Credit Income Fund"},"1831":{"cik_str":1345016,"ticker":"YELP","title":"YELP INC"},"1832":{"cik_str":7789,"ticker":"ASB","title":"ASSOCIATED BANC-CORP"},"1833":{"cik_str":1287032,"ticker":"PSEC","title":"PROSPECT CAPITAL CORP"},"1834":{"cik_str":1276520,"ticker":"GNW","title":"GENWORTH FINANCIAL INC"},"1835":{"cik_str":98677,"ticker":"TR","title":"TOOTSIE ROLL INDUSTRIES INC"},"1836":{"cik_str":913241,"ticker":"SHOO","title":"STEVEN MADDEN, LTD."},"1837":{"cik_str":850209,"ticker":"FL","title":"FOOT LOCKER, INC."},"1838":{"cik_str":1062750,"ticker":"SAPMY","title":"SAIPEM S P A /FI"},"1839":{"cik_str":1144800,"ticker":"TAC","title":"TRANSALTA CORP"},"1840":{"cik_str":1857154,"ticker":"DNUT","title":"Krispy Kreme, Inc."},"1841":{"cik_str":868780,"ticker":"DORM","title":"Dorman Products, Inc."},"1842":{"cik_str":1534675,"ticker":"TGLS","title":"Tecnoglass Inc."},"1843":{"cik_str":1488139,"ticker":"AMRC","title":"Ameresco, Inc."},"1844":{"cik_str":1563411,"ticker":"CSTM","title":"CONSTELLIUM SE"},"1845":{"cik_str":1375877,"ticker":"CSIQ","title":"Canadian Solar Inc."},"1846":{"cik_str":26780,"ticker":"DAN","title":"DANA INC"},"1847":{"cik_str":901491,"ticker":"PZZA","title":"PAPA JOHNS INTERNATIONAL INC"},"1848":{"cik_str":1744659,"ticker":"AKRO","title":"Akero Therapeutics, Inc."},"1849":{"cik_str":1559865,"ticker":"EVTC","title":"EVERTEC, Inc."},"1850":{"cik_str":1584425,"ticker":"TCN","title":"Tricon Residential Inc."},"1851":{"cik_str":1764013,"ticker":"IMVT","title":"Immunovant, Inc."},"1852":{"cik_str":1740915,"ticker":"FTCH","title":"Farfetch Ltd"},"1853":{"cik_str":1576940,"ticker":"CCS","title":"Century Communities, Inc."},"1854":{"cik_str":1632790,"ticker":"ENR","title":"ENERGIZER HOLDINGS, INC."},"1855":{"cik_str":1379438,"ticker":"EXG","title":"Eaton Vance Tax-Managed Global Diversified Equity Income Fund"},"1856":{"cik_str":1745999,"ticker":"BEAM","title":"Beam Therapeutics Inc."},"1857":{"cik_str":1866550,"ticker":"TWKS","title":"Thoughtworks Holding, Inc."},"1858":{"cik_str":1582313,"ticker":"XENE","title":"Xenon Pharmaceuticals Inc."},"1859":{"cik_str":1830081,"ticker":"RUM","title":"Rumble Inc."},"1860":{"cik_str":351569,"ticker":"ABCB","title":"Ameris Bancorp"},"1861":{"cik_str":1721947,"ticker":"JAMF","title":"Jamf Holding Corp."},"1862":{"cik_str":1822829,"ticker":"OZON","title":"Ozon Holdings PLC"},"1863":{"cik_str":1114995,"ticker":"PI","title":"IMPINJ INC"},"1864":{"cik_str":1898795,"ticker":"LVWR","title":"LiveWire Group, Inc."},"1865":{"cik_str":751978,"ticker":"VICR","title":"VICOR CORP"},"1866":{"cik_str":1809122,"ticker":"CVAC","title":"CureVac N.V."},"1867":{"cik_str":919012,"ticker":"AEO","title":"AMERICAN EAGLE OUTFITTERS INC"},"1868":{"cik_str":1362004,"ticker":"ICFI","title":"ICF International, Inc."},"1869":{"cik_str":1694028,"ticker":"LBRT","title":"Liberty Energy Inc."},"1870":{"cik_str":1086600,"ticker":"ARLP","title":"ALLIANCE RESOURCE PARTNERS LP"},"1871":{"cik_str":1620393,"ticker":"NXRT","title":"NexPoint Residential Trust, Inc."},"1872":{"cik_str":799292,"ticker":"MHO","title":"M/I HOMES, INC."},"1873":{"cik_str":1526520,"ticker":"TRIP","title":"TripAdvisor, Inc."},"1874":{"cik_str":1760965,"ticker":"KTB","title":"Kontoor Brands, Inc."},"1875":{"cik_str":1679688,"ticker":"DBRG","title":"DigitalBridge Group, Inc."},"1876":{"cik_str":1455633,"ticker":"PEGRY","title":"Pennon Group Plc / ADR"},"1877":{"cik_str":1546417,"ticker":"BLMN","title":"Bloomin' Brands, Inc."},"1878":{"cik_str":1704715,"ticker":"AMR","title":"Alpha Metallurgical Resources, Inc."},"1879":{"cik_str":746598,"ticker":"BRC","title":"BRADY CORP"},"1880":{"cik_str":828944,"ticker":"WSFS","title":"WSFS FINANCIAL CORP"},"1881":{"cik_str":1275214,"ticker":"CSQ","title":"CALAMOS STRATEGIC TOTAL RETURN FUND"},"1882":{"cik_str":1167419,"ticker":"RIOT","title":"Riot Platforms, Inc."},"1883":{"cik_str":1756770,"ticker":"CURLF","title":"Curaleaf Holdings, Inc."},"1884":{"cik_str":1362468,"ticker":"ALGT","title":"Allegiant Travel CO"},"1885":{"cik_str":1593538,"ticker":"NAVI","title":"NAVIENT CORP"},"1886":{"cik_str":861842,"ticker":"CATY","title":"CATHAY GENERAL BANCORP"},"1887":{"cik_str":1835539,"ticker":"OLK","title":"Olink Holding AB (publ)"},"1888":{"cik_str":1853717,"ticker":"ATAT","title":"Atour Lifestyle Holdings Ltd"},"1889":{"cik_str":1861560,"ticker":"NUVL","title":"Nuvalent, Inc."},"1890":{"cik_str":1104657,"ticker":"MTRN","title":"MATERION Corp"},"1891":{"cik_str":36377,"ticker":"FHB","title":"FIRST HAWAIIAN, INC."},"1892":{"cik_str":1826470,"ticker":"WOOF","title":"Petco Health & Wellness Company, Inc."},"1893":{"cik_str":894405,"ticker":"ARCB","title":"ARCBEST CORP /DE/"},"1894":{"cik_str":835011,"ticker":"MGPI","title":"MGP INGREDIENTS INC"},"1895":{"cik_str":1770450,"ticker":"XRX","title":"Xerox Holdings Corp"},"1896":{"cik_str":1851194,"ticker":"VTYX","title":"Ventyx Biosciences, Inc."},"1897":{"cik_str":1299709,"ticker":"AX","title":"Axos Financial, Inc."},"1898":{"cik_str":1726978,"ticker":"GSHD","title":"Goosehead Insurance, Inc."},"1899":{"cik_str":1283140,"ticker":"HEP","title":"HOLLY ENERGY PARTNERS LP"},"1900":{"cik_str":1710366,"ticker":"CEIX","title":"CONSOL Energy Inc."},"1901":{"cik_str":1329394,"ticker":"SIMO","title":"Silicon Motion Technology CORP"},"1902":{"cik_str":50725,"ticker":"GFF","title":"GRIFFON CORP"},"1903":{"cik_str":899715,"ticker":"SKT","title":"TANGER FACTORY OUTLET CENTERS, INC"},"1904":{"cik_str":1845097,"ticker":"AMBP","title":"Ardagh Metal Packaging S.A."},"1905":{"cik_str":1767258,"ticker":"XPEL","title":"XPEL, Inc."},"1906":{"cik_str":1323885,"ticker":"ATRC","title":"AtriCure, Inc."},"1907":{"cik_str":1552797,"ticker":"DKL","title":"Delek Logistics Partners, LP"},"1908":{"cik_str":700923,"ticker":"MYRG","title":"MYR GROUP INC."},"1909":{"cik_str":1408100,"ticker":"KW","title":"Kennedy-Wilson Holdings, Inc."},"1910":{"cik_str":55242,"ticker":"KMT","title":"KENNAMETAL INC"},"1911":{"cik_str":1041803,"ticker":"PSMT","title":"PRICESMART INC"},"1912":{"cik_str":877860,"ticker":"NHI","title":"NATIONAL HEALTH INVESTORS INC"},"1913":{"cik_str":1430723,"ticker":"SFBS","title":"ServisFirst Bancshares, Inc."},"1914":{"cik_str":1476765,"ticker":"GBDC","title":"GOLUB CAPITAL BDC, Inc."},"1915":{"cik_str":1698535,"ticker":"NXE","title":"NexGen Energy Ltd."},"1916":{"cik_str":1276671,"ticker":"ITCL","title":"BANCO ITAU CHILE"},"1917":{"cik_str":1825088,"ticker":"DFH","title":"Dream Finders Homes, Inc."},"1918":{"cik_str":752714,"ticker":"MGRC","title":"MCGRATH RENTCORP"},"1919":{"cik_str":1676163,"ticker":"SSII","title":"SS Innovations International, Inc."},"1920":{"cik_str":1286139,"ticker":"LZ","title":"LEGALZOOM.COM, INC."},"1921":{"cik_str":18926,"ticker":"LUMN","title":"Lumen Technologies, Inc."},"1922":{"cik_str":1481513,"ticker":"JKS","title":"JinkoSolar Holding Co., Ltd."},"1923":{"cik_str":1057706,"ticker":"FBP","title":"FIRST BANCORP /PR/"},"1924":{"cik_str":16160,"ticker":"CALM","title":"CAL-MAINE FOODS INC"},"1925":{"cik_str":1762506,"ticker":"VIST","title":"Vista Energy, S.A.B. de C.V."},"1926":{"cik_str":1088856,"ticker":"CORT","title":"CORCEPT THERAPEUTICS INC"},"1927":{"cik_str":803649,"ticker":"EQC","title":"Equity Commonwealth"},"1928":{"cik_str":90498,"ticker":"SFNC","title":"SIMMONS FIRST NATIONAL CORP"},"1929":{"cik_str":893691,"ticker":"DOOR","title":"MASONITE INTERNATIONAL CORP"},"1930":{"cik_str":1166388,"ticker":"VRNT","title":"VERINT SYSTEMS INC"},"1931":{"cik_str":1275617,"ticker":"UTF","title":"COHEN & STEERS INFRASTRUCTURE FUND INC"},"1932":{"cik_str":1650132,"ticker":"FCPT","title":"Four Corners Property Trust, Inc."},"1933":{"cik_str":1888447,"ticker":"EE","title":"Excelerate Energy, Inc."},"1934":{"cik_str":1853145,"ticker":"EVCM","title":"EverCommerce Inc."},"1935":{"cik_str":1898416,"ticker":"ALVO","title":"Alvotech"},"1936":{"cik_str":1645569,"ticker":"DICE","title":"DICE Therapeutics, Inc."},"1937":{"cik_str":897077,"ticker":"ALG","title":"ALAMO GROUP INC"},"1938":{"cik_str":1576263,"ticker":"MRTX","title":"Mirati Therapeutics, Inc."},"1939":{"cik_str":1821586,"ticker":"MLTX","title":"MoonLake Immunotherapeutics"},"1940":{"cik_str":766829,"ticker":"SJW","title":"SJW GROUP"},"1941":{"cik_str":701374,"ticker":"SIX","title":"Six Flags Entertainment Corp"},"1942":{"cik_str":1078075,"ticker":"NTCT","title":"NETSCOUT SYSTEMS INC"},"1943":{"cik_str":1530238,"ticker":"YY","title":"JOYY Inc."},"1944":{"cik_str":1210677,"ticker":"FA","title":"FIRST ADVANTAGE CORP"},"1945":{"cik_str":1498547,"ticker":"CMRF","title":"CIM REAL ESTATE FINANCE TRUST, INC."},"1946":{"cik_str":1810546,"ticker":"EBC","title":"Eastern Bankshares, Inc."},"1947":{"cik_str":1547903,"ticker":"NMIH","title":"NMI Holdings, Inc."},"1948":{"cik_str":1749704,"ticker":"AGTI","title":"AGILITI, INC. \\DE"},"1949":{"cik_str":1526125,"ticker":"GDS","title":"GDS Holdings Ltd"},"1950":{"cik_str":1809519,"ticker":"GDRX","title":"GoodRx Holdings, Inc."},"1951":{"cik_str":1537054,"ticker":"GOGO","title":"Gogo Inc."},"1952":{"cik_str":928022,"ticker":"CPE","title":"Callon Petroleum Co"},"1953":{"cik_str":1848763,"ticker":"RNW","title":"ReNew Energy Global plc"},"1954":{"cik_str":1591587,"ticker":"AMK","title":"AssetMark Financial Holdings, Inc."},"1955":{"cik_str":1570827,"ticker":"VCTR","title":"Victory Capital Holdings, Inc."},"1956":{"cik_str":1508478,"ticker":"ARCO","title":"Arcos Dorados Holdings Inc."},"1957":{"cik_str":1648257,"ticker":"HCM","title":"HUTCHMED (China) Ltd"},"1958":{"cik_str":1207179,"ticker":"GLNG","title":"GOLAR LNG LTD"},"1959":{"cik_str":1295810,"ticker":"SHO","title":"Sunstone Hotel Investors, Inc."},"1960":{"cik_str":1541401,"ticker":"ESRT","title":"Empire State Realty Trust, Inc."},"1961":{"cik_str":1670076,"ticker":"ULCC","title":"Frontier Group Holdings, Inc."},"1962":{"cik_str":1142596,"ticker":"NUVA","title":"NUVASIVE INC"},"1963":{"cik_str":1037676,"ticker":"ARCH","title":"ARCH RESOURCES, INC."},"1964":{"cik_str":9984,"ticker":"B","title":"BARNES GROUP INC"},"1965":{"cik_str":1096752,"ticker":"EPC","title":"EDGEWELL PERSONAL CARE Co"},"1966":{"cik_str":94049,"ticker":"SCL","title":"STEPAN CO"},"1967":{"cik_str":1653477,"ticker":"NGVT","title":"Ingevity Corp"},"1968":{"cik_str":1772695,"ticker":"NOVA","title":"Sunnova Energy International Inc."},"1969":{"cik_str":1024795,"ticker":"HLIO","title":"HELIOS TECHNOLOGIES, INC."},"1970":{"cik_str":790526,"ticker":"RDNT","title":"RadNet, Inc."},"1971":{"cik_str":1280784,"ticker":"HTGC","title":"Hercules Capital, Inc."},"1972":{"cik_str":19745,"ticker":"CPK","title":"CHESAPEAKE UTILITIES CORP"},"1973":{"cik_str":925261,"ticker":"AKO-A","title":"ANDINA BOTTLING CO INC"},"1974":{"cik_str":1479290,"ticker":"RVNC","title":"Revance Therapeutics, Inc."},"1975":{"cik_str":1655891,"ticker":"TRMD","title":"TORM plc"},"1976":{"cik_str":1825570,"ticker":"PAX","title":"Patria Investments Ltd"},"1977":{"cik_str":99780,"ticker":"TRN","title":"TRINITY INDUSTRIES INC"},"1978":{"cik_str":1688476,"ticker":"NEX","title":"NEXTIER OILFIELD SOLUTIONS INC."},"1979":{"cik_str":1802665,"ticker":"HRMY","title":"Harmony Biosciences Holdings, Inc."},"1980":{"cik_str":1350653,"ticker":"ATEC","title":"Alphatec Holdings, Inc."},"1981":{"cik_str":1314727,"ticker":"SONO","title":"Sonos Inc"},"1982":{"cik_str":1677576,"ticker":"IIPR","title":"INNOVATIVE INDUSTRIAL PROPERTIES INC"},"1983":{"cik_str":1568100,"ticker":"PD","title":"PagerDuty, Inc."},"1984":{"cik_str":1922641,"ticker":"ENLT","title":"Enlight Renewable Energy Ltd."},"1985":{"cik_str":1293135,"ticker":"VET","title":"VERMILION ENERGY INC."},"1986":{"cik_str":918608,"ticker":"EGO","title":"ELDORADO GOLD CORP /FI"},"1987":{"cik_str":1067294,"ticker":"CBRL","title":"CRACKER BARREL OLD COUNTRY STORE, INC"},"1988":{"cik_str":1691303,"ticker":"HCC","title":"WARRIOR MET COAL, INC."},"1989":{"cik_str":1717161,"ticker":"CEPU","title":"CENTRAL PUERTO S.A."},"1990":{"cik_str":1858257,"ticker":"AVDX","title":"AvidXchange Holdings, Inc."},"1991":{"cik_str":811532,"ticker":"FUN","title":"CEDAR FAIR L P"},"1992":{"cik_str":1553079,"ticker":"ESBA","title":"Empire State Realty OP, L.P."},"1993":{"cik_str":1265131,"ticker":"HTH","title":"Hilltop Holdings Inc."},"1994":{"cik_str":1122491,"ticker":"BRFS","title":"BRF S.A."},"1995":{"cik_str":2230,"ticker":"ADX","title":"ADAMS DIVERSIFIED EQUITY FUND, INC."},"1996":{"cik_str":776901,"ticker":"INDB","title":"INDEPENDENT BANK CORP"},"1997":{"cik_str":1049521,"ticker":"MRCY","title":"MERCURY SYSTEMS INC"},"1998":{"cik_str":107687,"ticker":"WGO","title":"WINNEBAGO INDUSTRIES INC"},"1999":{"cik_str":1028918,"ticker":"PPBI","title":"PACIFIC PREMIER BANCORP INC"},"2000":{"cik_str":59440,"ticker":"VGR","title":"VECTOR GROUP LTD"},"2001":{"cik_str":1530804,"ticker":"TROX","title":"Tronox Holdings plc"},"2002":{"cik_str":1517413,"ticker":"FSLY","title":"Fastly, Inc."},"2003":{"cik_str":1263994,"ticker":"UTG","title":"REAVES UTILITY INCOME FUND"},"2004":{"cik_str":1385280,"ticker":"CNK","title":"Cinemark Holdings, Inc."},"2005":{"cik_str":1077183,"ticker":"NEO","title":"NEOGENOMICS INC"},"2006":{"cik_str":1936804,"ticker":"SDA","title":"SunCar Technology Group Inc."},"2007":{"cik_str":1735707,"ticker":"GTX","title":"Garrett Motion Inc."},"2008":{"cik_str":1856608,"ticker":"SOVO","title":"Sovos Brands, Inc."},"2009":{"cik_str":700564,"ticker":"FULT","title":"FULTON FINANCIAL CORP"},"2010":{"cik_str":887733,"ticker":"CENT","title":"CENTRAL GARDEN & PET CO"},"2011":{"cik_str":1750,"ticker":"AIR","title":"AAR CORP"},"2012":{"cik_str":10254,"ticker":"ESTE","title":"EARTHSTONE ENERGY INC"},"2013":{"cik_str":1725160,"ticker":"ZNTL","title":"Zentalis Pharmaceuticals, Inc."},"2014":{"cik_str":884713,"ticker":"PRMW","title":"Primo Water Corp /CN/"},"2015":{"cik_str":1962918,"ticker":"SLRN","title":"ACELYRIN, Inc."},"2016":{"cik_str":807882,"ticker":"JACK","title":"JACK IN THE BOX INC"},"2017":{"cik_str":883948,"ticker":"AUB","title":"Atlantic Union Bankshares Corp"},"2018":{"cik_str":1039065,"ticker":"OSIS","title":"OSI SYSTEMS INC"},"2019":{"cik_str":1040971,"ticker":"SLG","title":"SL GREEN REALTY CORP"},"2020":{"cik_str":708955,"ticker":"FFBC","title":"FIRST FINANCIAL BANCORP /OH/"},"2021":{"cik_str":1717393,"ticker":"CAAP","title":"CORPORACION AMERICA AIRPORTS S.A."},"2022":{"cik_str":1590717,"ticker":"CTRE","title":"CareTrust REIT, Inc."},"2023":{"cik_str":1840572,"ticker":"BOWL","title":"Bowlero Corp."},"2024":{"cik_str":1527590,"ticker":"RC","title":"Ready Capital Corp"},"2025":{"cik_str":1651562,"ticker":"COUR","title":"Coursera, Inc."},"2026":{"cik_str":1522727,"ticker":"USAC","title":"USA Compression Partners, LP"},"2027":{"cik_str":21535,"ticker":"COHU","title":"COHU INC"},"2028":{"cik_str":1309402,"ticker":"GPRE","title":"Green Plains Inc."},"2029":{"cik_str":1340736,"ticker":"ETY","title":"Eaton Vance Tax-Managed Diversified Equity Income Fund"},"2030":{"cik_str":929351,"ticker":"LGF-A","title":"LIONS GATE ENTERTAINMENT CORP /CN/"},"2031":{"cik_str":1898496,"ticker":"GETY","title":"Getty Images Holdings, Inc."},"2032":{"cik_str":1366868,"ticker":"GSAT","title":"Globalstar, Inc."},"2033":{"cik_str":1137887,"ticker":"NZF","title":"Nuveen Municipal Credit Income Fund"},"2034":{"cik_str":73756,"ticker":"OII","title":"OCEANEERING INTERNATIONAL INC"},"2035":{"cik_str":1795139,"ticker":"GTBIF","title":"Green Thumb Industries Inc."},"2036":{"cik_str":1230245,"ticker":"PIPR","title":"PIPER SANDLER COMPANIES"},"2037":{"cik_str":1853860,"ticker":"ERO","title":"Ero Copper Corp."},"2038":{"cik_str":354647,"ticker":"CVBF","title":"CVB FINANCIAL CORP"},"2039":{"cik_str":1737450,"ticker":"OPRA","title":"Opera Ltd"},"2040":{"cik_str":1575828,"ticker":"XPRO","title":"EXPRO GROUP HOLDINGS N.V."},"2041":{"cik_str":1710155,"ticker":"EYE","title":"National Vision Holdings, Inc."},"2042":{"cik_str":733269,"ticker":"RAMP","title":"LiveRamp Holdings, Inc."},"2043":{"cik_str":1611547,"ticker":"UE","title":"Urban Edge Properties"},"2044":{"cik_str":1921963,"ticker":"ATMU","title":"Atmus Filtration Technologies Inc."},"2045":{"cik_str":1110805,"ticker":"NS","title":"NuStar Energy L.P."},"2046":{"cik_str":1260729,"ticker":"GDV","title":"GABELLI DIVIDEND & INCOME TRUST"},"2047":{"cik_str":1525769,"ticker":"PLAY","title":"Dave & Buster's Entertainment, Inc."},"2048":{"cik_str":1861974,"ticker":"ECX","title":"ECARX Holdings Inc."},"2049":{"cik_str":1773751,"ticker":"HIMS","title":"Hims & Hers Health, Inc."},"2050":{"cik_str":903129,"ticker":"THRM","title":"GENTHERM Inc"},"2051":{"cik_str":1576427,"ticker":"CRTO","title":"Criteo S.A."},"2052":{"cik_str":1703057,"ticker":"ABCL","title":"AbCellera Biologics Inc."},"2053":{"cik_str":730708,"ticker":"SBCF","title":"SEACOAST BANKING CORP OF FLORIDA"},"2054":{"cik_str":874499,"ticker":"GPOR","title":"GULFPORT ENERGY CORP"},"2055":{"cik_str":1013131,"ticker":"BVN","title":"BUENAVENTURA MINING CO INC"},"2056":{"cik_str":912766,"ticker":"LAUR","title":"LAUREATE EDUCATION, INC."},"2057":{"cik_str":899923,"ticker":"MYGN","title":"MYRIAD GENETICS INC"},"2058":{"cik_str":1720990,"ticker":"FSR","title":"Fisker Inc./DE"},"2059":{"cik_str":107140,"ticker":"WLY","title":"JOHN WILEY & SONS, INC."},"2060":{"cik_str":1498710,"ticker":"SAVE","title":"Spirit Airlines, Inc."},"2061":{"cik_str":1628369,"ticker":"CWK","title":"Cushman & Wakefield plc"},"2062":{"cik_str":1809987,"ticker":"MIR","title":"Mirion Technologies, Inc."},"2063":{"cik_str":912562,"ticker":"ROCK","title":"GIBRALTAR INDUSTRIES, INC."},"2064":{"cik_str":1723596,"ticker":"CLBK","title":"Columbia Financial, Inc."},"2065":{"cik_str":18654,"ticker":"AILIH","title":"Ameren Illinois Co"},"2066":{"cik_str":891014,"ticker":"MTX","title":"MINERALS TECHNOLOGIES INC"},"2067":{"cik_str":1690511,"ticker":"GOOS","title":"Canada Goose Holdings Inc."},"2068":{"cik_str":1018840,"ticker":"ANF","title":"ABERCROMBIE & FITCH CO /DE/"},"2069":{"cik_str":1610601,"ticker":"MOMO","title":"Hello Group Inc."},"2070":{"cik_str":1595974,"ticker":"MGNI","title":"MAGNITE, INC."},"2071":{"cik_str":1712184,"ticker":"LILA","title":"Liberty Latin America Ltd."},"2072":{"cik_str":1679049,"ticker":"INSW","title":"International Seaways, Inc."},"2073":{"cik_str":1317833,"ticker":"AQPW","title":"Golden Ally Lifetech Group, Inc."},"2074":{"cik_str":800240,"ticker":"ODP","title":"ODP Corp"},"2075":{"cik_str":1190935,"ticker":"PTY","title":"PIMCO CORPORATE & INCOME OPPORTUNITY FUND"},"2076":{"cik_str":1396814,"ticker":"PCRX","title":"Pacira BioSciences, Inc."},"2077":{"cik_str":1830033,"ticker":"PCT","title":"PureCycle Technologies, Inc."},"2078":{"cik_str":814083,"ticker":"CLM","title":"CORNERSTONE STRATEGIC VALUE FUND INC"},"2079":{"cik_str":1692787,"ticker":"KNTK","title":"Kinetik Holdings Inc."},"2080":{"cik_str":1715497,"ticker":"BORR","title":"Borr Drilling Ltd"},"2081":{"cik_str":1743759,"ticker":"CRSR","title":"Corsair Gaming, Inc."},"2082":{"cik_str":1881487,"ticker":"ACDC","title":"ProFrac Holding Corp."},"2083":{"cik_str":793733,"ticker":"SKYW","title":"SKYWEST INC"},"2084":{"cik_str":851310,"ticker":"HLIT","title":"HARMONIC INC"},"2085":{"cik_str":812801,"ticker":"NUV","title":"NUVEEN MUNICIPAL VALUE FUND INC"},"2086":{"cik_str":1295401,"ticker":"TBBK","title":"Bancorp, Inc."},"2087":{"cik_str":887596,"ticker":"CAKE","title":"CHEESECAKE FACTORY INC"},"2088":{"cik_str":861459,"ticker":"GVA","title":"GRANITE CONSTRUCTION INC"},"2089":{"cik_str":1854401,"ticker":"BRDG","title":"Bridge Investment Group Holdings Inc."},"2090":{"cik_str":1386716,"ticker":"SBLK","title":"Star Bulk Carriers Corp."},"2091":{"cik_str":1069258,"ticker":"KTOS","title":"KRATOS DEFENSE & SECURITY SOLUTIONS, INC."},"2092":{"cik_str":1836057,"ticker":"BIGZ","title":"BlackRock Innovation & Growth Term Trust"},"2093":{"cik_str":1384101,"ticker":"VCYT","title":"VERACYTE, INC."},"2094":{"cik_str":67347,"ticker":"MOD","title":"MODINE MANUFACTURING CO"},"2095":{"cik_str":1617553,"ticker":"ZIP","title":"ZIPRECRUITER, INC."},"2096":{"cik_str":1785971,"ticker":"BMEZ","title":"BlackRock Health Sciences Term Trust"},"2097":{"cik_str":1821769,"ticker":"NVTS","title":"Navitas Semiconductor Corp"},"2098":{"cik_str":874238,"ticker":"STRL","title":"STERLING INFRASTRUCTURE, INC."},"2099":{"cik_str":1410384,"ticker":"QTWO","title":"Q2 Holdings, Inc."},"2100":{"cik_str":1549346,"ticker":"SSTK","title":"Shutterstock, Inc."},"2101":{"cik_str":936528,"ticker":"WAFD","title":"WASHINGTON FEDERAL INC"},"2102":{"cik_str":76605,"ticker":"PATK","title":"PATRICK INDUSTRIES INC"},"2103":{"cik_str":1543916,"ticker":"FORG","title":"ForgeRock, Inc."},"2104":{"cik_str":1380936,"ticker":"GOF","title":"GUGGENHEIM STRATEGIC OPPORTUNITIES FUND"},"2105":{"cik_str":885245,"ticker":"BKE","title":"BUCKLE INC"},"2106":{"cik_str":1828248,"ticker":"CVII","title":"Churchill Capital Corp VII"},"2107":{"cik_str":887359,"ticker":"VCEL","title":"Vericel Corp"},"2108":{"cik_str":933036,"ticker":"UPBD","title":"UPBOUND GROUP, INC."},"2109":{"cik_str":1568651,"ticker":"OSCR","title":"Oscar Health, Inc."},"2110":{"cik_str":1474098,"ticker":"PEB","title":"Pebblebrook Hotel Trust"},"2111":{"cik_str":1821806,"ticker":"LESL","title":"Leslie's, Inc."},"2112":{"cik_str":1822492,"ticker":"HLMN","title":"Hillman Solutions Corp."},"2113":{"cik_str":105418,"ticker":"WMK","title":"WEIS MARKETS INC"},"2114":{"cik_str":1689796,"ticker":"JBGS","title":"JBG SMITH Properties"},"2115":{"cik_str":885978,"ticker":"USPH","title":"U S PHYSICAL THERAPY INC /NV"},"2116":{"cik_str":799195,"ticker":"USA","title":"LIBERTY ALL STAR EQUITY FUND"},"2117":{"cik_str":1724965,"ticker":"TALO","title":"TALOS ENERGY INC."},"2118":{"cik_str":1845815,"ticker":"PAYO","title":"Payoneer Global Inc."},"2119":{"cik_str":1856485,"ticker":"SLVM","title":"Sylvamo Corp"},"2120":{"cik_str":794685,"ticker":"GAB","title":"GABELLI EQUITY TRUST INC"},"2121":{"cik_str":1866175,"ticker":"CRGY","title":"Crescent Energy Co"},"2122":{"cik_str":1030894,"ticker":"CLS","title":"CELESTICA INC"},"2123":{"cik_str":799167,"ticker":"MRTN","title":"MARTEN TRANSPORT LTD"},"2124":{"cik_str":1532961,"ticker":"NVEE","title":"NV5 Global, Inc."},"2125":{"cik_str":1298946,"ticker":"DRH","title":"DiamondRock Hospitality Co"},"2126":{"cik_str":1837429,"ticker":"BNRE","title":"Brookfield Reinsurance Ltd."},"2127":{"cik_str":913760,"ticker":"SNEX","title":"StoneX Group Inc."},"2128":{"cik_str":1599489,"ticker":"VRTV","title":"Veritiv Corp"},"2129":{"cik_str":1851003,"ticker":"ZETA","title":"Zeta Global Holdings Corp."},"2130":{"cik_str":78749,"ticker":"AGYS","title":"AGILYSYS INC"},"2131":{"cik_str":1120914,"ticker":"PDFS","title":"PDF SOLUTIONS INC"},"2132":{"cik_str":1688757,"ticker":"ESTA","title":"ESTABLISHMENT LABS HOLDINGS INC."},"2133":{"cik_str":1043509,"ticker":"SAH","title":"SONIC AUTOMOTIVE INC"},"2134":{"cik_str":867773,"ticker":"SPWR","title":"SUNPOWER CORP"},"2135":{"cik_str":1681769,"ticker":"CCYC","title":"Clancy Corp"},"2136":{"cik_str":1407623,"ticker":"ROIC","title":"RETAIL OPPORTUNITY INVESTMENTS CORP"},"2137":{"cik_str":1910950,"ticker":"AESI","title":"Atlas Energy Solutions Inc."},"2138":{"cik_str":1354327,"ticker":"PGTI","title":"PGT Innovations, Inc."},"2139":{"cik_str":712534,"ticker":"FRME","title":"FIRST MERCHANTS CORP"},"2140":{"cik_str":1253327,"ticker":"EVT","title":"EATON VANCE TAX ADVANTAGED DIVIDEND INCOME FUND"},"2141":{"cik_str":1739942,"ticker":"SWI","title":"SolarWinds Corp"},"2142":{"cik_str":1275014,"ticker":"UCTT","title":"Ultra Clean Holdings, Inc."},"2143":{"cik_str":1952073,"ticker":"MSGE","title":"Madison Square Garden Entertainment Corp."},"2144":{"cik_str":1529864,"ticker":"ENVA","title":"Enova International, Inc."},"2145":{"cik_str":1800347,"ticker":"ETWO","title":"E2open Parent Holdings, Inc."},"2146":{"cik_str":1413159,"ticker":"TGH","title":"TEXTAINER GROUP HOLDINGS LTD"},"2147":{"cik_str":1511337,"ticker":"RLJ","title":"RLJ Lodging Trust"},"2148":{"cik_str":1563190,"ticker":"COMP","title":"Compass, Inc."},"2149":{"cik_str":1599298,"ticker":"SMMT","title":"Summit Therapeutics Inc."},"2150":{"cik_str":1052752,"ticker":"GTY","title":"GETTY REALTY CORP /MD/"},"2151":{"cik_str":1705110,"ticker":"ANGI","title":"Angi Inc."},"2152":{"cik_str":46195,"ticker":"BOH","title":"BANK OF HAWAII CORP"},"2153":{"cik_str":1322422,"ticker":"HBM","title":"Hudbay Minerals Inc."},"2154":{"cik_str":1915760,"ticker":"HMNTY","title":"Hemnet Group AB/ADR"},"2155":{"cik_str":310354,"ticker":"SXI","title":"STANDEX INTERNATIONAL CORP/DE/"},"2156":{"cik_str":1359841,"ticker":"HBI","title":"Hanesbrands Inc."},"2157":{"cik_str":64996,"ticker":"MCY","title":"MERCURY GENERAL CORP"},"2158":{"cik_str":1021561,"ticker":"NUS","title":"NU SKIN ENTERPRISES, INC."},"2159":{"cik_str":88941,"ticker":"SMTC","title":"SEMTECH CORP"},"2160":{"cik_str":805676,"ticker":"PRK","title":"PARK NATIONAL CORP /OH/"},"2161":{"cik_str":1834494,"ticker":"MLNK","title":"MeridianLink, Inc."},"2162":{"cik_str":1545057,"ticker":"PTXKY","title":"PT XL Axiata Tbk/ADR"},"2163":{"cik_str":1005757,"ticker":"CSGS","title":"CSG SYSTEMS INTERNATIONAL INC"},"2164":{"cik_str":1013934,"ticker":"STRA","title":"Strategic Education, Inc."},"2165":{"cik_str":1866581,"ticker":"BROS","title":"Dutch Bros Inc."},"2166":{"cik_str":1504008,"ticker":"BKU","title":"BankUnited, Inc."},"2167":{"cik_str":1709682,"ticker":"CTOS","title":"Custom Truck One Source, Inc."},"2168":{"cik_str":1339605,"ticker":"HEES","title":"H&E Equipment Services, Inc."},"2169":{"cik_str":1809541,"ticker":"BCAT","title":"BlackRock Capital Allocation Term Trust"},"2170":{"cik_str":1395942,"ticker":"KAR","title":"OPENLANE, Inc."},"2171":{"cik_str":1289848,"ticker":"HURN","title":"Huron Consulting Group Inc."},"2172":{"cik_str":1587523,"ticker":"KN","title":"Knowles Corp"},"2173":{"cik_str":1773427,"ticker":"SWTX","title":"SpringWorks Therapeutics, Inc."},"2174":{"cik_str":1772253,"ticker":"FLNG","title":"Flex LNG Ltd."},"2175":{"cik_str":944809,"ticker":"OPK","title":"OPKO HEALTH, INC."},"2176":{"cik_str":1029142,"ticker":"DVAX","title":"DYNAVAX TECHNOLOGIES CORP"},"2177":{"cik_str":703351,"ticker":"EAT","title":"BRINKER INTERNATIONAL, INC"},"2178":{"cik_str":744218,"ticker":"CLDX","title":"Celldex Therapeutics, Inc."},"2179":{"cik_str":1718227,"ticker":"ROAD","title":"Construction Partners, Inc."},"2180":{"cik_str":924901,"ticker":"VRE","title":"Veris Residential, Inc."},"2181":{"cik_str":810136,"ticker":"PLAB","title":"PHOTRONICS INC"},"2182":{"cik_str":1389050,"ticker":"AROC","title":"Archrock, Inc."},"2183":{"cik_str":1520504,"ticker":"VLRS","title":"Controladora Vuela Compania de Aviacion, S.A.B. de C.V."},"2184":{"cik_str":821483,"ticker":"PARR","title":"PAR PACIFIC HOLDINGS, INC."},"2185":{"cik_str":75288,"ticker":"OXM","title":"OXFORD INDUSTRIES INC"},"2186":{"cik_str":1356576,"ticker":"SUPN","title":"SUPERNUS PHARMACEUTICALS, INC."},"2187":{"cik_str":1508655,"ticker":"TSLX","title":"Sixth Street Specialty Lending, Inc."},"2188":{"cik_str":1013880,"ticker":"TTEC","title":"TTEC Holdings, Inc."},"2189":{"cik_str":1361538,"ticker":"PRIM","title":"Primoris Services Corp"},"2190":{"cik_str":1412408,"ticker":"PHR","title":"Phreesia, Inc."},"2191":{"cik_str":1616318,"ticker":"VSTO","title":"Vista Outdoor Inc."},"2192":{"cik_str":886744,"ticker":"GERN","title":"GERON CORP"},"2193":{"cik_str":821026,"ticker":"ANDE","title":"Andersons, Inc."},"2194":{"cik_str":1446847,"ticker":"IRWD","title":"IRONWOOD PHARMACEUTICALS INC"},"2195":{"cik_str":1421517,"ticker":"ERII","title":"Energy Recovery, Inc."},"2196":{"cik_str":1725255,"ticker":"AHCO","title":"AdaptHealth Corp."},"2197":{"cik_str":1281895,"ticker":"RCKT","title":"ROCKET PHARMACEUTICALS, INC."},"2198":{"cik_str":1636639,"ticker":"FIHL","title":"Fidelis Insurance Holdings Ltd"},"2199":{"cik_str":1157408,"ticker":"LRN","title":"Stride, Inc."},"2200":{"cik_str":1722438,"ticker":"DOMA","title":"Doma Holdings, Inc."},"2201":{"cik_str":1864843,"ticker":"ECAT","title":"BlackRock ESG Capital Allocation Term Trust"},"2202":{"cik_str":1657312,"ticker":"VRNA","title":"Verona Pharma plc"},"2203":{"cik_str":1711375,"ticker":"LOMA","title":"Loma Negra Compania Industrial Argentina Sociedad Anonima"},"2204":{"cik_str":745543,"ticker":"PBAJ","title":"PETRO USA, INC."},"2205":{"cik_str":1588978,"ticker":"PRCT","title":"PROCEPT BioRobotics Corp"},"2206":{"cik_str":1569963,"ticker":"LTGHY","title":"Life Healthcare Group Holdings Limited/ADR"},"2207":{"cik_str":1101215,"ticker":"BFH","title":"BREAD FINANCIAL HOLDINGS, INC."},"2208":{"cik_str":1307748,"ticker":"IVT","title":"InvenTrust Properties Corp."},"2209":{"cik_str":1425205,"ticker":"IOVA","title":"IOVANCE BIOTHERAPEUTICS, INC."},"2210":{"cik_str":1438133,"ticker":"TNDM","title":"TANDEM DIABETES CARE INC"},"2211":{"cik_str":1467760,"ticker":"ARI","title":"Apollo Commercial Real Estate Finance, Inc."},"2212":{"cik_str":1888886,"ticker":"GPCR","title":"Structure Therapeutics Inc."},"2213":{"cik_str":1308648,"ticker":"AG","title":"FIRST MAJESTIC SILVER CORP"},"2214":{"cik_str":1157842,"ticker":"RQI","title":"COHEN & STEERS QUALITY INCOME REALTY FUND INC"},"2215":{"cik_str":1434614,"ticker":"SAND","title":"SANDSTORM GOLD LTD"},"2216":{"cik_str":1074952,"ticker":"NAC","title":"Nuveen California Quality Municipal Income Fund"},"2217":{"cik_str":50493,"ticker":"IMKTA","title":"INGLES MARKETS INC"},"2218":{"cik_str":1935979,"ticker":"BHVN","title":"Biohaven Ltd."},"2219":{"cik_str":1607939,"ticker":"UDMY","title":"Udemy, Inc."},"2220":{"cik_str":1694426,"ticker":"DK","title":"Delek US Holdings, Inc."},"2221":{"cik_str":1439222,"ticker":"AGIO","title":"AGIOS PHARMACEUTICALS, INC."},"2222":{"cik_str":1432364,"ticker":"AZUL","title":"AZUL SA"},"2223":{"cik_str":1109138,"ticker":"CAMT","title":"CAMTEK LTD"},"2224":{"cik_str":1666291,"ticker":"CMTG","title":"Claros Mortgage Trust, Inc."},"2225":{"cik_str":1332283,"ticker":"BDJ","title":"BlackRock Enhanced Equity Dividend Trust"},"2226":{"cik_str":1095651,"ticker":"SAFE","title":"Safehold Inc."},"2227":{"cik_str":203596,"ticker":"WSBC","title":"WESBANCO INC"},"2228":{"cik_str":1733998,"ticker":"NWN","title":"Northwest Natural Holding Co"},"2229":{"cik_str":1704720,"ticker":"CNNE","title":"Cannae Holdings, Inc."},"2230":{"cik_str":1808834,"ticker":"PRG","title":"PROG Holdings, Inc."},"2231":{"cik_str":1345126,"ticker":"CODI","title":"Compass Diversified Holdings"},"2232":{"cik_str":742278,"ticker":"RES","title":"RPC INC"},"2233":{"cik_str":1839839,"ticker":"JBI","title":"Janus International Group, Inc."},"2234":{"cik_str":1856031,"ticker":"SEAT","title":"Vivid Seats Inc."},"2235":{"cik_str":1781755,"ticker":"BRP","title":"BRP Group, Inc."},"2236":{"cik_str":1262976,"ticker":"CMPR","title":"CIMPRESS plc"},"2237":{"cik_str":1832511,"ticker":"PIII","title":"P3 Health Partners Inc."},"2238":{"cik_str":1607678,"ticker":"VKTX","title":"Viking Therapeutics, Inc."},"2239":{"cik_str":946673,"ticker":"BANR","title":"BANNER CORP"},"2240":{"cik_str":845877,"ticker":"AGM","title":"FEDERAL AGRICULTURAL MORTGAGE CORP"},"2241":{"cik_str":1812364,"ticker":"RLAY","title":"Relay Therapeutics, Inc."},"2242":{"cik_str":1654126,"ticker":"ZIM","title":"ZIM Integrated Shipping Services Ltd."},"2243":{"cik_str":1423774,"ticker":"ZUO","title":"ZUORA INC"},"2244":{"cik_str":1601548,"ticker":"VVX","title":"V2X, Inc."},"2245":{"cik_str":804116,"ticker":"RVT","title":"ROYCE VALUE TRUST, INC."},"2246":{"cik_str":899629,"ticker":"AKR","title":"ACADIA REALTY TRUST"},"2247":{"cik_str":1338940,"ticker":"PERI","title":"Perion Network Ltd."},"2248":{"cik_str":1572694,"ticker":"GSBD","title":"Goldman Sachs BDC, Inc."},"2249":{"cik_str":1690680,"ticker":"NMRK","title":"NEWMARK GROUP, INC."},"2250":{"cik_str":821130,"ticker":"USM","title":"UNITED STATES CELLULAR CORP"},"2251":{"cik_str":1481582,"ticker":"RYI","title":"Ryerson Holding Corp"},"2252":{"cik_str":1500217,"ticker":"AAT","title":"American Assets Trust, Inc."},"2253":{"cik_str":1786205,"ticker":"ACLX","title":"Arcellx, Inc."},"2254":{"cik_str":1529274,"ticker":"ALKT","title":"ALKAMI TECHNOLOGY, INC."},"2255":{"cik_str":1414932,"ticker":"OCSL","title":"Oaktree Specialty Lending Corp"},"2256":{"cik_str":1029145,"ticker":"GOGL","title":"Golden Ocean Group Ltd"},"2257":{"cik_str":97134,"ticker":"TNC","title":"TENNANT CO"},"2258":{"cik_str":730464,"ticker":"ATGE","title":"Adtalem Global Education Inc."},"2259":{"cik_str":6955,"ticker":"EPAC","title":"ENERPAC TOOL GROUP CORP"},"2260":{"cik_str":1326732,"ticker":"XNCR","title":"Xencor Inc"},"2261":{"cik_str":1022408,"ticker":"PLUS","title":"EPLUS INC"},"2262":{"cik_str":1635984,"ticker":"NWLI","title":"National Western Life Group, Inc."},"2263":{"cik_str":1630627,"ticker":"TMCI","title":"TREACE MEDICAL CONCEPTS, INC."},"2264":{"cik_str":1482981,"ticker":"COCO","title":"Vita Coco Company, Inc."},"2265":{"cik_str":914156,"ticker":"UFPT","title":"UFP TECHNOLOGIES INC"},"2266":{"cik_str":1724521,"ticker":"RCUS","title":"Arcus Biosciences, Inc."},"2267":{"cik_str":945394,"ticker":"SVC","title":"Service Properties Trust"},"2268":{"cik_str":1756607,"ticker":"EQX","title":"Equinox Gold Corp."},"2269":{"cik_str":1158895,"ticker":"LMAT","title":"LEMAITRE VASCULAR INC"},"2270":{"cik_str":1941365,"ticker":"MBC","title":"MasterBrand, Inc."},"2271":{"cik_str":1025835,"ticker":"EFSC","title":"ENTERPRISE FINANCIAL SERVICES CORP"},"2272":{"cik_str":715072,"ticker":"RNST","title":"RENASANT CORP"},"2273":{"cik_str":1564618,"ticker":"IBTX","title":"Independent Bank Group, Inc."},"2274":{"cik_str":1810739,"ticker":"RADI","title":"Radius Global Infrastructure, Inc."},"2275":{"cik_str":1267395,"ticker":"AHL-PC","title":"ASPEN INSURANCE HOLDINGS LTD"},"2276":{"cik_str":99614,"ticker":"TY","title":"TRI-CONTINENTAL CORP"},"2277":{"cik_str":1382821,"ticker":"RDFN","title":"Redfin Corp"},"2278":{"cik_str":1835256,"ticker":"NAPA","title":"Duckhorn Portfolio, Inc."},"2279":{"cik_str":1576018,"ticker":"SPNT","title":"SiriusPoint Ltd"},"2280":{"cik_str":1528437,"ticker":"BTT","title":"BlackRock Municipal 2030 Target Term Trust"},"2281":{"cik_str":1878057,"ticker":"SGHC","title":"Super Group (SGHC) Ltd"},"2282":{"cik_str":1477815,"ticker":"SG","title":"Sweetgreen, Inc."},"2283":{"cik_str":1468328,"ticker":"ADUS","title":"Addus HomeCare Corp"},"2284":{"cik_str":885740,"ticker":"SPNS","title":"SAPIENS INTERNATIONAL CORP N V"},"2285":{"cik_str":949039,"ticker":"DO","title":"DIAMOND OFFSHORE DRILLING, INC."},"2286":{"cik_str":1616000,"ticker":"XHR","title":"Xenia Hotels & Resorts, Inc."},"2287":{"cik_str":1875444,"ticker":"ARHS","title":"Arhaus, Inc."},"2288":{"cik_str":1674335,"ticker":"JELD","title":"JELD-WEN Holding, Inc."},"2289":{"cik_str":104894,"ticker":"ELME","title":"Elme Communities"},"2290":{"cik_str":1795589,"ticker":"KC","title":"Kingsoft Cloud Holdings Ltd"},"2291":{"cik_str":1083446,"ticker":"AMEH","title":"Apollo Medical Holdings, Inc."},"2292":{"cik_str":1625791,"ticker":"KRNT","title":"Kornit Digital Ltd."},"2293":{"cik_str":1798618,"ticker":"PDO","title":"PIMCO Dynamic Income Opportunities Fund"},"2294":{"cik_str":883237,"ticker":"VRTS","title":"VIRTUS INVESTMENT PARTNERS, INC."},"2295":{"cik_str":1702780,"ticker":"ATUS","title":"Altice USA, Inc."},"2296":{"cik_str":1761312,"ticker":"PLMR","title":"Palomar Holdings, Inc."},"2297":{"cik_str":1130713,"ticker":"OSTK","title":"OVERSTOCK.COM, INC"},"2298":{"cik_str":1415404,"ticker":"SATS","title":"EchoStar CORP"},"2299":{"cik_str":1531978,"ticker":"FNA","title":"Paragon 28, Inc."},"2300":{"cik_str":1395937,"ticker":"SNDX","title":"Syndax Pharmaceuticals Inc"},"2301":{"cik_str":1793862,"ticker":"DADA","title":"Dada Nexus Ltd"},"2302":{"cik_str":1600620,"ticker":"AUPH","title":"Aurinia Pharmaceuticals Inc."},"2303":{"cik_str":75252,"ticker":"OMI","title":"OWENS & MINOR INC/VA/"},"2304":{"cik_str":1768666,"ticker":"BSTZ","title":"BlackRock Science & Technology Term Trust"},"2305":{"cik_str":1517175,"ticker":"CHEF","title":"Chefs' Warehouse, Inc."},"2306":{"cik_str":66004,"ticker":"MSEX","title":"MIDDLESEX WATER CO"},"2307":{"cik_str":1861737,"ticker":"HSAI","title":"Hesai Group"},"2308":{"cik_str":1116942,"ticker":"TTMI","title":"TTM TECHNOLOGIES INC"},"2309":{"cik_str":1601830,"ticker":"RXRX","title":"RECURSION PHARMACEUTICALS, INC."},"2310":{"cik_str":1796898,"ticker":"MAXN","title":"Maxeon Solar Technologies, Ltd."},"2311":{"cik_str":854775,"ticker":"DGII","title":"DIGI INTERNATIONAL INC"},"2312":{"cik_str":1871130,"ticker":"BBUC","title":"Brookfield Business Corp"},"2313":{"cik_str":1227654,"ticker":"CMP","title":"COMPASS MINERALS INTERNATIONAL INC"},"2314":{"cik_str":1894562,"ticker":"PRME","title":"Prime Medicine, Inc."},"2315":{"cik_str":1658551,"ticker":"AMLX","title":"Amylyx Pharmaceuticals, Inc."},"2316":{"cik_str":1653242,"ticker":"NTB","title":"Bank of N.T. Butterfield & Son Ltd"},"2317":{"cik_str":1854587,"ticker":"CLBT","title":"Cellebrite DI Ltd."},"2318":{"cik_str":1819796,"ticker":"GCMG","title":"GCM Grosvenor Inc."},"2319":{"cik_str":1614178,"ticker":"YEXT","title":"Yext, Inc."},"2320":{"cik_str":1657788,"ticker":"KRP","title":"Kimbell Royalty Partners, LP"},"2321":{"cik_str":1298699,"ticker":"BXMX","title":"Nuveen S&P 500 BuyWrite Income Fund"},"2322":{"cik_str":906338,"ticker":"TARO","title":"TARO PHARMACEUTICAL INDUSTRIES LTD"},"2323":{"cik_str":790359,"ticker":"NBTB","title":"NBT BANCORP INC"},"2324":{"cik_str":1322436,"ticker":"ETV","title":"Eaton Vance Tax-Managed Buy-Write Opportunities Fund"},"2325":{"cik_str":1577670,"ticker":"LADR","title":"Ladder Capital Corp"},"2326":{"cik_str":726854,"ticker":"CHCO","title":"CITY HOLDING CO"},"2327":{"cik_str":1856437,"ticker":"VSCO","title":"Victoria's Secret & Co."},"2328":{"cik_str":1504776,"ticker":"WRBY","title":"Warby Parker Inc."},"2329":{"cik_str":887905,"ticker":"LTC","title":"LTC PROPERTIES INC"},"2330":{"cik_str":1124804,"ticker":"MDRX","title":"Veradigm Inc."},"2331":{"cik_str":1180262,"ticker":"HLF","title":"HERBALIFE LTD."},"2332":{"cik_str":1331284,"ticker":"DHT","title":"DHT Holdings, Inc."},"2333":{"cik_str":1468091,"ticker":"VEON","title":"VEON Ltd."},"2334":{"cik_str":1616533,"ticker":"SGH","title":"SMART Global Holdings, Inc."},"2335":{"cik_str":1843724,"ticker":"LNZA","title":"LanzaTech Global, Inc."},"2336":{"cik_str":894081,"ticker":"ATSG","title":"Air Transport Services Group, Inc."},"2337":{"cik_str":1713334,"ticker":"MSC","title":"STUDIO CITY INTERNATIONAL HOLDINGS Ltd"},"2338":{"cik_str":923120,"ticker":"GBX","title":"GREENBRIER COMPANIES INC"},"2339":{"cik_str":1527508,"ticker":"PTVE","title":"Pactiv Evergreen Inc."},"2340":{"cik_str":1392972,"ticker":"PRO","title":"PROS Holdings, Inc."},"2341":{"cik_str":1545654,"ticker":"ALEX","title":"Alexander & Baldwin, Inc."},"2342":{"cik_str":1793229,"ticker":"MPLN","title":"MultiPlan Corp"},"2343":{"cik_str":880117,"ticker":"JBSS","title":"SANFILIPPO JOHN B & SON INC"},"2344":{"cik_str":1369241,"ticker":"DAC","title":"Danaos Corp"},"2345":{"cik_str":1471265,"ticker":"NWBI","title":"Northwest Bancshares, Inc."},"2346":{"cik_str":1844224,"ticker":"FREY","title":"FREYR Battery"},"2347":{"cik_str":1173420,"ticker":"NG","title":"NOVAGOLD RESOURCES INC"},"2348":{"cik_str":1539638,"ticker":"TFIN","title":"Triumph Financial, Inc."},"2349":{"cik_str":835324,"ticker":"SYBT","title":"Stock Yards Bancorp, Inc."},"2350":{"cik_str":1369568,"ticker":"CPRX","title":"CATALYST PHARMACEUTICALS, INC."},"2351":{"cik_str":1831840,"ticker":"SEMR","title":"SEMrush Holdings, Inc."},"2352":{"cik_str":907242,"ticker":"MCRI","title":"MONARCH CASINO & RESORT INC"},"2353":{"cik_str":1368458,"ticker":"SBH","title":"Sally Beauty Holdings, Inc."},"2354":{"cik_str":1649749,"ticker":"FBK","title":"FB Financial Corp"},"2355":{"cik_str":1846576,"ticker":"FIGS","title":"FIGS, Inc."},"2356":{"cik_str":1395213,"ticker":"EDN","title":"EDENOR"},"2357":{"cik_str":1655759,"ticker":"ARVN","title":"ARVINAS, INC."},"2358":{"cik_str":1854640,"ticker":"CGAU","title":"Centerra Gold Inc."},"2359":{"cik_str":1669811,"ticker":"DFIN","title":"Donnelley Financial Solutions, Inc."},"2360":{"cik_str":1712189,"ticker":"TH","title":"Target Hospitality Corp."},"2361":{"cik_str":1708035,"ticker":"ECVT","title":"Ecovyst Inc."},"2362":{"cik_str":26058,"ticker":"CTS","title":"CTS CORP"},"2363":{"cik_str":1785173,"ticker":"ETNB","title":"89bio, Inc."},"2364":{"cik_str":1841925,"ticker":"INDI","title":"indie Semiconductor, Inc."},"2365":{"cik_str":103145,"ticker":"VECO","title":"VEECO INSTRUMENTS INC"},"2366":{"cik_str":110095,"ticker":"NISUY","title":"NIPPON SUISAN KAISHA LTD /ADR/"},"2367":{"cik_str":882796,"ticker":"BCRX","title":"BIOCRYST PHARMACEUTICALS INC"},"2368":{"cik_str":933267,"ticker":"IRS","title":"IRSA INVESTMENTS & REPRESENTATIONS INC"},"2369":{"cik_str":712537,"ticker":"FCF","title":"FIRST COMMONWEALTH FINANCIAL CORP /PA/"},"2370":{"cik_str":48287,"ticker":"HNI","title":"HNI CORP"},"2371":{"cik_str":1739566,"ticker":"UTZ","title":"Utz Brands, Inc."},"2372":{"cik_str":1176433,"ticker":"JPS","title":"Nuveen Preferred & Income Securities Fund"},"2373":{"cik_str":1118417,"ticker":"MODN","title":"MODEL N, INC."},"2374":{"cik_str":1669779,"ticker":"CWH","title":"Camping World Holdings, Inc."},"2375":{"cik_str":1683606,"ticker":"CARS","title":"Cars.com Inc."},"2376":{"cik_str":1792849,"ticker":"HPK","title":"HighPeak Energy, Inc."},"2377":{"cik_str":1651311,"ticker":"MRUS","title":"Merus N.V."},"2378":{"cik_str":1680056,"ticker":"ORLA","title":"Orla Mining Ltd."},"2379":{"cik_str":63296,"ticker":"MATW","title":"MATTHEWS INTERNATIONAL CORP"},"2380":{"cik_str":1203464,"ticker":"IAG","title":"IAMGOLD CORP"},"2381":{"cik_str":1835591,"ticker":"VZIO","title":"Vizio Holding Corp."},"2382":{"cik_str":36146,"ticker":"TRMK","title":"TRUSTMARK CORP"},"2383":{"cik_str":1771007,"ticker":"AFYA","title":"Afya Ltd"},"2384":{"cik_str":1654795,"ticker":"BBU","title":"Brookfield Business Partners L.P."},"2385":{"cik_str":1676725,"ticker":"IDYA","title":"IDEAYA Biosciences, Inc."},"2386":{"cik_str":1842279,"ticker":"OPAL","title":"OPAL Fuels Inc."},"2387":{"cik_str":1841968,"ticker":"PX","title":"P10, Inc."},"2388":{"cik_str":836157,"ticker":"LNN","title":"LINDSAY CORP"},"2389":{"cik_str":931148,"ticker":"EAF","title":"GRAFTECH INTERNATIONAL LTD"},"2390":{"cik_str":1409493,"ticker":"CIM","title":"CHIMERA INVESTMENT CORP"},"2391":{"cik_str":1815442,"ticker":"KYMR","title":"Kymera Therapeutics, Inc."},"2392":{"cik_str":1318568,"ticker":"EVRI","title":"Everi Holdings Inc."},"2393":{"cik_str":799233,"ticker":"HTLD","title":"HEARTLAND EXPRESS INC"},"2394":{"cik_str":1453015,"ticker":"BLDP","title":"Ballard Power Systems Inc."},"2395":{"cik_str":1464790,"ticker":"RILY","title":"B. Riley Financial, Inc."},"2396":{"cik_str":910638,"ticker":"DDD","title":"3D SYSTEMS CORP"},"2397":{"cik_str":1419945,"ticker":"TNK","title":"TEEKAY TANKERS LTD."},"2398":{"cik_str":1643615,"ticker":"MEG","title":"Montrose Environmental Group, Inc."},"2399":{"cik_str":876427,"ticker":"MNRO","title":"MONRO, INC."},"2400":{"cik_str":922864,"ticker":"AIV","title":"APARTMENT INVESTMENT & MANAGEMENT CO"},"2401":{"cik_str":102426,"ticker":"STEW","title":"SRH Total Return Fund, Inc."},"2402":{"cik_str":1691445,"ticker":"FINV","title":"FinVolution Group"},"2403":{"cik_str":1612720,"ticker":"NEXT","title":"NextDecade Corp."},"2404":{"cik_str":907471,"ticker":"CASH","title":"PATHWARD FINANCIAL, INC."},"2405":{"cik_str":1482512,"ticker":"HPP","title":"Hudson Pacific Properties, Inc."},"2406":{"cik_str":1465740,"ticker":"TWO","title":"TWO HARBORS INVESTMENT CORP."},"2407":{"cik_str":1841156,"ticker":"PAY","title":"Paymentus Holdings, Inc."},"2408":{"cik_str":866729,"ticker":"SCHL","title":"SCHOLASTIC CORP"},"2409":{"cik_str":1834489,"ticker":"GENI","title":"Genius Sports Ltd"},"2410":{"cik_str":1334933,"ticker":"UEC","title":"URANIUM ENERGY CORP"},"2411":{"cik_str":1609804,"ticker":"OEC","title":"Orion S.A."},"2412":{"cik_str":1806201,"ticker":"LPRO","title":"Open Lending Corp"},"2413":{"cik_str":1899123,"ticker":"BTDR","title":"Bitdeer Technologies Group"},"2414":{"cik_str":1270523,"ticker":"ETG","title":"EATON VANCE TAX ADVANTAGED GLOBAL DIVIDEND INCOME FUND"},"2415":{"cik_str":1377121,"ticker":"PTGX","title":"Protagonist Therapeutics, Inc"},"2416":{"cik_str":789460,"ticker":"WKC","title":"WORLD KINECT CORP"},"2417":{"cik_str":1829959,"ticker":"DCBO","title":"Docebo Inc."},"2418":{"cik_str":1857475,"ticker":"DOLE","title":"Dole plc"},"2419":{"cik_str":930420,"ticker":"KFRC","title":"KFORCE INC"},"2420":{"cik_str":1866692,"ticker":"AMPL","title":"Amplitude, Inc."},"2421":{"cik_str":1222401,"ticker":"HYT","title":"BLACKROCK CORPORATE HIGH YIELD FUND, INC."},"2422":{"cik_str":1496099,"ticker":"NMFC","title":"New Mountain Finance Corp"},"2423":{"cik_str":1737953,"ticker":"REPL","title":"Replimune Group, Inc."},"2424":{"cik_str":1821159,"ticker":"EVGO","title":"EVgo Inc."},"2425":{"cik_str":1030469,"ticker":"OFG","title":"OFG BANCORP"},"2426":{"cik_str":913059,"ticker":"BBAR","title":"Banco BBVA Argentina S.A."},"2427":{"cik_str":1340122,"ticker":"CLMT","title":"Calumet Specialty Products Partners, L.P."},"2428":{"cik_str":1611005,"ticker":"KEN","title":"Kenon Holdings Ltd."},"2429":{"cik_str":69633,"ticker":"NSSC","title":"NAPCO SECURITY TECHNOLOGIES, INC"},"2430":{"cik_str":1178970,"ticker":"PFS","title":"PROVIDENT FINANCIAL SERVICES INC"},"2431":{"cik_str":1743907,"ticker":"SNCY","title":"Sun Country Airlines Holdings, Inc."},"2432":{"cik_str":1809691,"ticker":"HKD","title":"AMTD Digital Inc."},"2433":{"cik_str":1848739,"ticker":"GDEV","title":"GDEV Inc."},"2434":{"cik_str":886163,"ticker":"LGND","title":"LIGAND PHARMACEUTICALS INC"},"2435":{"cik_str":1040470,"ticker":"AEHR","title":"AEHR TEST SYSTEMS"},"2436":{"cik_str":1846069,"ticker":"KIND","title":"Nextdoor Holdings, Inc."},"2437":{"cik_str":1159167,"ticker":"IRBT","title":"IROBOT CORP"},"2438":{"cik_str":65270,"ticker":"MEI","title":"METHODE ELECTRONICS INC"},"2439":{"cik_str":721994,"ticker":"LKFN","title":"LAKELAND FINANCIAL CORP"},"2440":{"cik_str":1692412,"ticker":"PLYA","title":"Playa Hotels & Resorts N.V."},"2441":{"cik_str":794619,"ticker":"AMWD","title":"AMERICAN WOODMARK CORP"},"2442":{"cik_str":102037,"ticker":"UVV","title":"UNIVERSAL CORP /VA/"},"2443":{"cik_str":1517228,"ticker":"COMM","title":"CommScope Holding Company, Inc."},"2444":{"cik_str":1879016,"ticker":"IE","title":"Ivanhoe Electric Inc."},"2445":{"cik_str":811589,"ticker":"FBNC","title":"FIRST BANCORP /NC/"},"2446":{"cik_str":1578732,"ticker":"MMI","title":"Marcus & Millichap, Inc."},"2447":{"cik_str":1871509,"ticker":"PTLO","title":"Portillo's Inc."},"2448":{"cik_str":1397702,"ticker":"SILK","title":"Silk Road Medical Inc"},"2449":{"cik_str":879526,"ticker":"WNC","title":"WABASH NATIONAL Corp"},"2450":{"cik_str":1473844,"ticker":"STEL","title":"Stellar Bancorp, Inc."},"2451":{"cik_str":1047340,"ticker":"FDP","title":"FRESH DEL MONTE PRODUCE INC"},"2452":{"cik_str":57131,"ticker":"LZB","title":"LA-Z-BOY INC"},"2453":{"cik_str":703604,"ticker":"DSGR","title":"Distribution Solutions Group, Inc."},"2454":{"cik_str":1746618,"ticker":"RVLV","title":"Revolve Group, Inc."},"2455":{"cik_str":896264,"ticker":"USNA","title":"USANA HEALTH SCIENCES INC"},"2456":{"cik_str":1590976,"ticker":"MBUU","title":"MALIBU BOATS, INC."},"2457":{"cik_str":1758488,"ticker":"OSW","title":"ONESPAWORLD HOLDINGS Ltd"},"2458":{"cik_str":1503584,"ticker":"CMRE","title":"Costamare Inc."},"2459":{"cik_str":1864163,"ticker":"INTR","title":"Inter & Co, Inc."},"2460":{"cik_str":1566388,"ticker":"DSL","title":"DoubleLine Income Solutions Fund"},"2461":{"cik_str":1627475,"ticker":"UPWK","title":"UPWORK, INC"},"2462":{"cik_str":920112,"ticker":"HTLF","title":"HEARTLAND FINANCIAL USA INC"},"2463":{"cik_str":1332174,"ticker":"GSG","title":"iShares S&P GSCI Commodity-Indexed Trust"},"2464":{"cik_str":907254,"ticker":"BFS","title":"SAUL CENTERS, INC."},"2465":{"cik_str":1608741,"ticker":"QQQX","title":"Nuveen NASDAQ 100 Dynamic Overwrite Fund"},"2466":{"cik_str":850141,"ticker":"HMN","title":"HORACE MANN EDUCATORS CORP /DE/"},"2467":{"cik_str":1691421,"ticker":"LMND","title":"Lemonade, Inc."},"2468":{"cik_str":1861795,"ticker":"DH","title":"Definitive Healthcare Corp."},"2469":{"cik_str":1872789,"ticker":"EMBC","title":"Embecta Corp."},"2470":{"cik_str":1071255,"ticker":"GDEN","title":"GOLDEN ENTERTAINMENT, INC."},"2471":{"cik_str":1840574,"ticker":"VERV","title":"Verve Therapeutics, Inc."},"2472":{"cik_str":1645070,"ticker":"STER","title":"Sterling Check Corp."},"2473":{"cik_str":1326110,"ticker":"IBRX","title":"ImmunityBio, Inc."},"2474":{"cik_str":1289877,"ticker":"SFL","title":"SFL Corp Ltd."},"2475":{"cik_str":1517396,"ticker":"SSYS","title":"STRATASYS LTD."},"2476":{"cik_str":1050377,"ticker":"CNXN","title":"PC CONNECTION INC"},"2477":{"cik_str":1767042,"ticker":"KGS","title":"Kodiak Gas Services, Inc."},"2478":{"cik_str":1843586,"ticker":"OTLY","title":"Oatly Group AB"},"2479":{"cik_str":1606498,"ticker":"AVNS","title":"AVANOS MEDICAL, INC."},"2480":{"cik_str":1022321,"ticker":"GEL","title":"GENESIS ENERGY LP"},"2481":{"cik_str":110471,"ticker":"WWW","title":"WOLVERINE WORLD WIDE INC /DE/"},"2482":{"cik_str":1856236,"ticker":"EWCZ","title":"European Wax Center, Inc."},"2483":{"cik_str":1462120,"ticker":"LOB","title":"Live Oak Bancshares, Inc."},"2484":{"cik_str":1464423,"ticker":"PMT","title":"PennyMac Mortgage Investment Trust"},"2485":{"cik_str":1230992,"ticker":"MAG","title":"MAG SILVER CORP"},"2486":{"cik_str":1664710,"ticker":"KROS","title":"Keros Therapeutics, Inc."},"2487":{"cik_str":1005229,"ticker":"CMCO","title":"COLUMBUS MCKINNON CORP"},"2488":{"cik_str":82020,"ticker":"USLM","title":"UNITED STATES LIME & MINERALS INC"},"2489":{"cik_str":1266585,"ticker":"NMZ","title":"NUVEEN MUNICIPAL HIGH INCOME OPPORTUNITY FUND"},"2490":{"cik_str":319654,"ticker":"PBT","title":"PERMIAN BASIN ROYALTY TRUST"},"2491":{"cik_str":1020859,"ticker":"UNFI","title":"UNITED NATURAL FOODS INC"},"2492":{"cik_str":1768267,"ticker":"CRNC","title":"Cerence Inc."},"2493":{"cik_str":910406,"ticker":"HAIN","title":"HAIN CELESTIAL GROUP INC"},"2494":{"cik_str":893949,"ticker":"MD","title":"Pediatrix Medical Group, Inc."},"2495":{"cik_str":1562528,"ticker":"FBRT","title":"Franklin BSP Realty Trust, Inc."},"2496":{"cik_str":1712178,"ticker":"NAAS","title":"NaaS Technology Inc."},"2497":{"cik_str":908315,"ticker":"WINA","title":"WINMARK CORP"},"2498":{"cik_str":1798100,"ticker":"NTST","title":"NETSTREIT Corp."},"2499":{"cik_str":1616678,"ticker":"BST","title":"BlackRock Science & Technology Trust"},"2500":{"cik_str":1172358,"ticker":"DMLP","title":"DORCHESTER MINERALS, L.P."},"2501":{"cik_str":811596,"ticker":"KALU","title":"KAISER ALUMINUM CORP"},"2502":{"cik_str":1803696,"ticker":"ADEA","title":"Adeia Inc."},"2503":{"cik_str":1591956,"ticker":"ANY","title":"Sphere 3D Corp."},"2504":{"cik_str":1829118,"ticker":"TUYA","title":"Tuya Inc."},"2505":{"cik_str":1260563,"ticker":"NFJ","title":"Virtus Dividend, Interest & Premium Strategy Fund"},"2506":{"cik_str":1012477,"ticker":"AVDL","title":"AVADEL PHARMACEUTICALS PLC"},"2507":{"cik_str":1438533,"ticker":"TVTX","title":"Travere Therapeutics, Inc."},"2508":{"cik_str":1084961,"ticker":"ECPG","title":"ENCORE CAPITAL GROUP INC"},"2509":{"cik_str":1499543,"ticker":"NOAH","title":"Noah Holdings Ltd"},"2510":{"cik_str":1342338,"ticker":"HIMX","title":"Himax Technologies, Inc."},"2511":{"cik_str":1770121,"ticker":"SANA","title":"Sana Biotechnology, Inc."},"2512":{"cik_str":842633,"ticker":"TRS","title":"TRIMAS CORP"},"2513":{"cik_str":830524,"ticker":"CCF","title":"CHASE CORP"},"2514":{"cik_str":1038572,"ticker":"CBD","title":"BRAZILIAN DISTRIBUTION CO COMPANHIA BRASILEIRA DE DISTR CBD"},"2515":{"cik_str":1048268,"ticker":"IESC","title":"IES Holdings, Inc."},"2516":{"cik_str":1803112,"ticker":"TDCX","title":"TDCX Inc."},"2517":{"cik_str":1739614,"ticker":"INBX","title":"Inhibrx, Inc."},"2518":{"cik_str":66382,"ticker":"MLKN","title":"MILLERKNOLL, INC."},"2519":{"cik_str":1091883,"ticker":"CIR","title":"CIRCOR INTERNATIONAL INC"},"2520":{"cik_str":1620280,"ticker":"UNIT","title":"Uniti Group Inc."},"2521":{"cik_str":1731348,"ticker":"TLRY","title":"Tilray Brands, Inc."},"2522":{"cik_str":1045986,"ticker":"FORTY","title":"FORMULA SYSTEMS (1985) LTD"},"2523":{"cik_str":896841,"ticker":"AVID","title":"AVID TECHNOLOGY, INC."},"2524":{"cik_str":1042074,"ticker":"CBAY","title":"CymaBay Therapeutics, Inc."},"2525":{"cik_str":1406587,"ticker":"FOR","title":"Forestar Group Inc."},"2526":{"cik_str":1055160,"ticker":"MFA","title":"MFA FINANCIAL, INC."},"2527":{"cik_str":1581280,"ticker":"TWST","title":"Twist Bioscience Corp"},"2528":{"cik_str":866829,"ticker":"HLX","title":"HELIX ENERGY SOLUTIONS GROUP INC"},"2529":{"cik_str":94344,"ticker":"STC","title":"STEWART INFORMATION SERVICES CORP"},"2530":{"cik_str":1364954,"ticker":"CHGG","title":"CHEGG, INC"},"2531":{"cik_str":1818093,"ticker":"SKIN","title":"Beauty Health Co"},"2532":{"cik_str":1624140,"ticker":"JMKJ","title":"Nine Alliance Science & Technology Group"},"2533":{"cik_str":356171,"ticker":"TCBK","title":"TRICO BANCSHARES /"},"2534":{"cik_str":1293613,"ticker":"KYN","title":"Kayne Anderson Energy Infrastructure Fund, Inc."},"2535":{"cik_str":1526113,"ticker":"GNL","title":"Global Net Lease, Inc."},"2536":{"cik_str":1599617,"ticker":"DNOW","title":"NOW Inc."},"2537":{"cik_str":314489,"ticker":"BUSE","title":"FIRST BUSEY CORP /NV/"},"2538":{"cik_str":1831915,"ticker":"CTKB","title":"Cytek Biosciences, Inc."},"2539":{"cik_str":1629019,"ticker":"MBIN","title":"Merchants Bancorp"},"2540":{"cik_str":1855447,"ticker":"TYGO","title":"TIGO ENERGY, INC."},"2541":{"cik_str":1475841,"ticker":"NBHC","title":"National Bank Holdings Corp"},"2542":{"cik_str":1728205,"ticker":"PLL","title":"Piedmont Lithium Inc."},"2543":{"cik_str":1437352,"ticker":"EVBG","title":"EVERBRIDGE, INC."},"2544":{"cik_str":731012,"ticker":"HCSG","title":"HEALTHCARE SERVICES GROUP INC"},"2545":{"cik_str":1654151,"ticker":"DCPH","title":"Deciphera Pharmaceuticals, Inc."},"2546":{"cik_str":1829864,"ticker":"TASK","title":"TaskUs, Inc."},"2547":{"cik_str":1777921,"ticker":"AVPT","title":"AvePoint, Inc."},"2548":{"cik_str":1652535,"ticker":"ICHR","title":"ICHOR HOLDINGS, LTD."},"2549":{"cik_str":1357450,"ticker":"HOLI","title":"Hollysys Automation Technologies, Ltd."},"2550":{"cik_str":1597033,"ticker":"SABR","title":"Sabre Corp"},"2551":{"cik_str":1848416,"ticker":"VRNOF","title":"Verano Holdings Corp."},"2552":{"cik_str":1832466,"ticker":"ALHC","title":"Alignment Healthcare, Inc."},"2553":{"cik_str":1222922,"ticker":"EVV","title":"EATON VANCE LTD DURATION INCOME FUND"},"2554":{"cik_str":1023024,"ticker":"ANIP","title":"ANI PHARMACEUTICALS INC"},"2555":{"cik_str":70487,"ticker":"NRC","title":"NATIONAL RESEARCH CORP"},"2556":{"cik_str":1070985,"ticker":"CXW","title":"CoreCivic, Inc."},"2557":{"cik_str":8947,"ticker":"AZZ","title":"AZZ INC"},"2558":{"cik_str":1674930,"ticker":"FLGT","title":"Fulgent Genetics, Inc."},"2559":{"cik_str":719220,"ticker":"STBA","title":"S&T BANCORP INC"},"2560":{"cik_str":7323,"ticker":"EAI","title":"ENTERGY ARKANSAS, LLC"},"2561":{"cik_str":1368265,"ticker":"CLNE","title":"Clean Energy Fuels Corp."},"2562":{"cik_str":1891101,"ticker":"BRCC","title":"BRC Inc."},"2563":{"cik_str":1824502,"ticker":"ACHR","title":"Archer Aviation Inc."},"2564":{"cik_str":708818,"ticker":"NXGN","title":"NEXTGEN HEALTHCARE, INC."},"2565":{"cik_str":1869858,"ticker":"SSU","title":"SIGNA Sports United N.V."},"2566":{"cik_str":1572957,"ticker":"BIOGY","title":"BioGaia AB/ADR"},"2567":{"cik_str":1409970,"ticker":"LC","title":"LendingClub Corp"},"2568":{"cik_str":1746473,"ticker":"PLRX","title":"PLIANT THERAPEUTICS, INC."},"2569":{"cik_str":1590750,"ticker":"VRDN","title":"Viridian Therapeutics, Inc.\\DE"},"2570":{"cik_str":1706946,"ticker":"SPCE","title":"Virgin Galactic Holdings, Inc"},"2571":{"cik_str":1172052,"ticker":"SAFT","title":"SAFETY INSURANCE GROUP INC"},"2572":{"cik_str":1063259,"ticker":"DNN","title":"DENISON MINES CORP."},"2573":{"cik_str":1802450,"ticker":"LIFW","title":"MSP Recovery, Inc."},"2574":{"cik_str":1051004,"ticker":"MUC","title":"BLACKROCK MUNIHOLDINGS CALIFORNIA QUALITY FUND, INC."},"2575":{"cik_str":34782,"ticker":"SRCE","title":"1ST SOURCE CORP"},"2576":{"cik_str":912463,"ticker":"GES","title":"GUESS INC"},"2577":{"cik_str":6845,"ticker":"APOG","title":"APOGEE ENTERPRISES, INC."},"2578":{"cik_str":1138639,"ticker":"INFN","title":"Infinera Corp"},"2579":{"cik_str":1883085,"ticker":"PGY","title":"Pagaya Technologies Ltd."},"2580":{"cik_str":1069530,"ticker":"SAVA","title":"CASSAVA SCIENCES INC"},"2581":{"cik_str":1580808,"ticker":"ATEN","title":"A10 Networks, Inc."},"2582":{"cik_str":1459839,"ticker":"SIBN","title":"SI-BONE, Inc."},"2583":{"cik_str":945114,"ticker":"GIC","title":"GLOBAL INDUSTRIAL Co"},"2584":{"cik_str":1605607,"ticker":"PGRE","title":"Paramount Group, Inc."},"2585":{"cik_str":899460,"ticker":"MNKD","title":"MANNKIND CORP"},"2586":{"cik_str":1431695,"ticker":"OLO","title":"Olo Inc."},"2587":{"cik_str":1846510,"ticker":"SHCO","title":"Soho House & Co Inc."},"2588":{"cik_str":1596993,"ticker":"LPG","title":"DORIAN LPG LTD."},"2589":{"cik_str":1569187,"ticker":"AHH","title":"Armada Hoffler Properties, Inc."},"2590":{"cik_str":1323468,"ticker":"GLP","title":"GLOBAL PARTNERS LP"},"2591":{"cik_str":311094,"ticker":"WABC","title":"WESTAMERICA BANCORPORATION"},"2592":{"cik_str":1091748,"ticker":"ARGO","title":"Argo Group International Holdings, Ltd."},"2593":{"cik_str":1826600,"ticker":"MNTK","title":"Montauk Renewables, Inc."},"2594":{"cik_str":824410,"ticker":"SASR","title":"SANDY SPRING BANCORP INC"},"2595":{"cik_str":1958086,"ticker":"CLB","title":"Core Laboratories Inc. /DE/"},"2596":{"cik_str":1836564,"ticker":"VALN","title":"Valneva SE"},"2597":{"cik_str":1278027,"ticker":"BGS","title":"B&G Foods, Inc."},"2598":{"cik_str":1428205,"ticker":"ARR","title":"Armour Residential REIT, Inc."},"2599":{"cik_str":1845337,"ticker":"DAWN","title":"Day One Biopharmaceuticals, Inc."},"2600":{"cik_str":1317945,"ticker":"OFLX","title":"Omega Flex, Inc."},"2601":{"cik_str":792987,"ticker":"ASTE","title":"ASTEC INDUSTRIES INC"},"2602":{"cik_str":1493683,"ticker":"BBN","title":"BlackRock Taxable Municipal Bond Trust"},"2603":{"cik_str":1128361,"ticker":"HOPE","title":"HOPE BANCORP INC"},"2604":{"cik_str":1257640,"ticker":"KRO","title":"KRONOS WORLDWIDE INC"},"2605":{"cik_str":910329,"ticker":"MED","title":"MEDIFAST INC"},"2606":{"cik_str":1762301,"ticker":"FVRR","title":"Fiverr International Ltd."},"2607":{"cik_str":1789940,"ticker":"FWRG","title":"First Watch Restaurant Group, Inc."},"2608":{"cik_str":1452857,"ticker":"SPLP","title":"STEEL PARTNERS HOLDINGS L.P."},"2609":{"cik_str":1836379,"ticker":"VECT","title":"VectivBio Holding AG"},"2610":{"cik_str":18748,"ticker":"CET","title":"CENTRAL SECURITIES CORP"},"2611":{"cik_str":1499505,"ticker":"AGRO","title":"Adecoagro S.A."},"2612":{"cik_str":1018979,"ticker":"AMSF","title":"AMERISAFE INC"},"2613":{"cik_str":1515816,"ticker":"PLYM","title":"Plymouth Industrial REIT, Inc."},"2614":{"cik_str":1484612,"ticker":"OM","title":"Outset Medical, Inc."},"2615":{"cik_str":880631,"ticker":"WT","title":"WisdomTree, Inc."},"2616":{"cik_str":46250,"ticker":"HWKN","title":"HAWKINS INC"},"2617":{"cik_str":1231346,"ticker":"SA","title":"SEABRIDGE GOLD INC"},"2618":{"cik_str":1102112,"ticker":"PACW","title":"PACWEST BANCORP"},"2619":{"cik_str":1772177,"ticker":"KRUS","title":"KURA SUSHI USA, INC."},"2620":{"cik_str":1379041,"ticker":"EIG","title":"Employers Holdings, Inc."},"2621":{"cik_str":1828811,"ticker":"LICY","title":"Li-Cycle Holdings Corp."},"2622":{"cik_str":1816736,"ticker":"IRON","title":"Disc Medicine, Inc."},"2623":{"cik_str":1528930,"ticker":"FRG","title":"Franchise Group, Inc."},"2624":{"cik_str":1174850,"ticker":"NIC","title":"NICOLET BANKSHARES INC"},"2625":{"cik_str":1425450,"ticker":"KIDS","title":"ORTHOPEDIATRICS CORP"},"2626":{"cik_str":1568162,"ticker":"RTL","title":"Necessity Retail REIT, Inc."},"2627":{"cik_str":40417,"ticker":"GAM","title":"GENERAL AMERICAN INVESTORS CO INC"},"2628":{"cik_str":1141103,"ticker":"CCRN","title":"CROSS COUNTRY HEALTHCARE INC"},"2629":{"cik_str":701288,"ticker":"ATRI","title":"ATRION CORP"},"2630":{"cik_str":1820144,"ticker":"GRND","title":"Grindr Inc."},"2631":{"cik_str":1501570,"ticker":"VBTX","title":"Veritex Holdings, Inc."},"2632":{"cik_str":1622229,"ticker":"COGT","title":"Cogent Biosciences, Inc."},"2633":{"cik_str":1736946,"ticker":"ARLO","title":"Arlo Technologies, Inc."},"2634":{"cik_str":1386278,"ticker":"GDOT","title":"GREEN DOT CORP"},"2635":{"cik_str":354963,"ticker":"SHEN","title":"SHENANDOAH TELECOMMUNICATIONS CO/VA/"},"2636":{"cik_str":798359,"ticker":"CSR","title":"CENTERSPACE"},"2637":{"cik_str":1673985,"ticker":"ASIX","title":"AdvanSix Inc."},"2638":{"cik_str":1657573,"ticker":"XMTR","title":"Xometry, Inc."},"2639":{"cik_str":1759783,"ticker":"EH","title":"EHang Holdings Ltd"},"2640":{"cik_str":1581760,"ticker":"LIFX","title":"Life360, Inc."},"2641":{"cik_str":1936258,"ticker":"NAMS","title":"NewAmsterdam Pharma Co N.V."},"2642":{"cik_str":1840856,"ticker":"SOUN","title":"SOUNDHOUND AI, INC."},"2643":{"cik_str":1834376,"ticker":"INNV","title":"InnovAge Holding Corp."},"2644":{"cik_str":1600626,"ticker":"PKST","title":"Peakstone Realty Trust"},"2645":{"cik_str":752642,"ticker":"UMH","title":"UMH PROPERTIES, INC."},"2646":{"cik_str":1731289,"ticker":"NKLA","title":"Nikola Corp"},"2647":{"cik_str":1062231,"ticker":"AXL","title":"AMERICAN AXLE & MANUFACTURING HOLDINGS INC"},"2648":{"cik_str":1730430,"ticker":"KNSA","title":"Kiniksa Pharmaceuticals, Ltd."},"2649":{"cik_str":1759425,"ticker":"MIRM","title":"Mirum Pharmaceuticals, Inc."},"2650":{"cik_str":1658247,"ticker":"CRNX","title":"Crinetics Pharmaceuticals, Inc."},"2651":{"cik_str":1865187,"ticker":"ARIS","title":"Aris Water Solutions, Inc."},"2652":{"cik_str":1568194,"ticker":"FSCO","title":"FS Credit Opportunities Corp."},"2653":{"cik_str":1793882,"ticker":"PTA","title":"Cohen & Steers Tax-Advantaged Preferred Securities & Income Fund"},"2654":{"cik_str":215466,"ticker":"CDE","title":"Coeur Mining, Inc."},"2655":{"cik_str":1385849,"ticker":"UUUU","title":"ENERGY FUELS INC"},"2656":{"cik_str":1811764,"ticker":"PNT","title":"POINT Biopharma Global Inc."},"2657":{"cik_str":1340243,"ticker":"MOR","title":"MorphoSys AG"},"2658":{"cik_str":1680247,"ticker":"PUMP","title":"ProPetro Holding Corp."},"2659":{"cik_str":1481646,"ticker":"ACCD","title":"Accolade, Inc."},"2660":{"cik_str":1598428,"ticker":"TMST","title":"TimkenSteel Corp"},"2661":{"cik_str":1893325,"ticker":"SCRM","title":"Screaming Eagle Acquisition Corp."},"2662":{"cik_str":1799983,"ticker":"GB","title":"Global Blue Group Holding AG"},"2663":{"cik_str":1488813,"ticker":"CUBI","title":"Customers Bancorp, Inc."},"2664":{"cik_str":1823794,"ticker":"ARKO","title":"ARKO Corp."},"2665":{"cik_str":1475115,"ticker":"EB","title":"Eventbrite, Inc."},"2666":{"cik_str":1341335,"ticker":"FSM","title":"FORTUNA SILVER MINES INC"},"2667":{"cik_str":1478320,"ticker":"ADPT","title":"Adaptive Biotechnologies Corp"},"2668":{"cik_str":1051512,"ticker":"TDS","title":"TELEPHONE & DATA SYSTEMS INC /DE/"},"2669":{"cik_str":1681622,"ticker":"VREX","title":"Varex Imaging Corp"},"2670":{"cik_str":1496919,"ticker":"MRRTY","title":"MARFRIG GLOBAL FOODS S.A."},"2671":{"cik_str":1111485,"ticker":"RXST","title":"RxSight, Inc."},"2672":{"cik_str":1838615,"ticker":"ALTI","title":"AlTi Global, Inc."},"2673":{"cik_str":1173514,"ticker":"HY","title":"HYSTER-YALE MATERIALS HANDLING, INC."},"2674":{"cik_str":1581804,"ticker":"NVGS","title":"Navigator Holdings Ltd."},"2675":{"cik_str":317788,"ticker":"APPS","title":"Digital Turbine, Inc."},"2676":{"cik_str":1780312,"ticker":"ASTS","title":"AST SpaceMobile, Inc."},"2677":{"cik_str":1795250,"ticker":"SPHR","title":"Sphere Entertainment Co."},"2678":{"cik_str":1519449,"ticker":"SKWD","title":"Skyward Specialty Insurance Group, Inc."},"2679":{"cik_str":710752,"ticker":"SBR","title":"SABINE ROYALTY TRUST"},"2680":{"cik_str":1822359,"ticker":"DCGO","title":"DocGo Inc."},"2681":{"cik_str":1144879,"ticker":"APLD","title":"Applied Digital Corp."},"2682":{"cik_str":1379384,"ticker":"BTZ","title":"BLACKROCK CREDIT ALLOCATION INCOME TRUST"},"2683":{"cik_str":921582,"ticker":"IMAX","title":"IMAX CORP"},"2684":{"cik_str":1004702,"ticker":"OCFC","title":"OCEANFIRST FINANCIAL CORP"},"2685":{"cik_str":318300,"ticker":"PEBO","title":"PEOPLES BANCORP INC"},"2686":{"cik_str":1840502,"ticker":"TBLA","title":"Taboola.com Ltd."},"2687":{"cik_str":1567569,"ticker":"FPF","title":"First Trust Intermediate Duration Preferred & Income Fund"},"2688":{"cik_str":95029,"ticker":"RGR","title":"STURM RUGER & CO INC"},"2689":{"cik_str":3499,"ticker":"ALX","title":"ALEXANDERS INC"},"2690":{"cik_str":1880319,"ticker":"PRM","title":"Perimeter Solutions, SA"},"2691":{"cik_str":1042776,"ticker":"PDM","title":"Piedmont Office Realty Trust, Inc."},"2692":{"cik_str":1422930,"ticker":"PUBM","title":"PubMatic, Inc."},"2693":{"cik_str":1108134,"ticker":"BHLB","title":"BERKSHIRE HILLS BANCORP INC"},"2694":{"cik_str":1308335,"ticker":"EOS","title":"Eaton Vance Enhanced Equity Income Fund II"},"2695":{"cik_str":1023512,"ticker":"DRD","title":"DRDGOLD LTD"},"2696":{"cik_str":1005210,"ticker":"SPH","title":"SUBURBAN PROPANE PARTNERS LP"},"2697":{"cik_str":1655099,"ticker":"RA","title":"Brookfield Real Assets Income Fund Inc."},"2698":{"cik_str":834365,"ticker":"BLFS","title":"BIOLIFE SOLUTIONS INC"},"2699":{"cik_str":1723128,"ticker":"AMRX","title":"Amneal Pharmaceuticals, Inc."},"2700":{"cik_str":842183,"ticker":"RPT","title":"RPT Realty"},"2701":{"cik_str":1524741,"ticker":"SLCA","title":"U.S. SILICA HOLDINGS, INC."},"2702":{"cik_str":1819394,"ticker":"MTTR","title":"Matterport, Inc./DE"},"2703":{"cik_str":1047335,"ticker":"NHC","title":"NATIONAL HEALTHCARE CORP"},"2704":{"cik_str":1403431,"ticker":"HCCI","title":"Heritage-Crystal Clean, Inc."},"2705":{"cik_str":1443669,"ticker":"PRLB","title":"Proto Labs Inc"},"2706":{"cik_str":1444380,"ticker":"NVRO","title":"NEVRO CORP"},"2707":{"cik_str":863436,"ticker":"BHE","title":"BENCHMARK ELECTRONICS INC"},"2708":{"cik_str":1034957,"ticker":"CRESY","title":"CRESUD INC"},"2709":{"cik_str":49754,"ticker":"DIN","title":"Dine Brands Global, Inc."},"2710":{"cik_str":1928446,"ticker":"GRNT","title":"Granite Ridge Resources, Inc."},"2711":{"cik_str":923796,"ticker":"GEO","title":"GEO GROUP INC"},"2712":{"cik_str":1649904,"ticker":"RYTM","title":"RHYTHM PHARMACEUTICALS, INC."},"2713":{"cik_str":1698991,"ticker":"ACEL","title":"Accel Entertainment, Inc."},"2714":{"cik_str":1224450,"ticker":"RNP","title":"COHEN & STEERS REIT & PREFERRED & INCOME FUND INC"},"2715":{"cik_str":1639877,"ticker":"GSM","title":"Ferroglobe PLC"},"2716":{"cik_str":1516912,"ticker":"OBK","title":"Origin Bancorp, Inc."},"2717":{"cik_str":1843762,"ticker":"EQRX","title":"EQRx, Inc."},"2718":{"cik_str":1185348,"ticker":"PRAA","title":"PRA GROUP INC"},"2719":{"cik_str":1411342,"ticker":"EFC","title":"Ellington Financial Inc."},"2720":{"cik_str":821002,"ticker":"GIII","title":"G III APPAREL GROUP LTD /DE/"},"2721":{"cik_str":708821,"ticker":"PAR","title":"PAR TECHNOLOGY CORP"},"2722":{"cik_str":1683695,"ticker":"IMXI","title":"International Money Express, Inc."},"2723":{"cik_str":1758766,"ticker":"STEM","title":"STEM, INC."},"2724":{"cik_str":1826018,"ticker":"ROVR","title":"ROVER GROUP, INC."},"2725":{"cik_str":1195739,"ticker":"NRK","title":"NUVEEN NEW YORK AMT-FREE QUALITY MUNICIPAL INCOME FUND"},"2726":{"cik_str":1273685,"ticker":"NYMT","title":"NEW YORK MORTGAGE TRUST INC"},"2727":{"cik_str":1322435,"ticker":"ETW","title":"Eaton Vance Tax-Managed Global Buy-Write Opportunities Fund"},"2728":{"cik_str":1387467,"ticker":"AOSL","title":"ALPHA & OMEGA SEMICONDUCTOR Ltd"},"2729":{"cik_str":1163739,"ticker":"NBR","title":"NABORS INDUSTRIES LTD"},"2730":{"cik_str":1693256,"ticker":"WTTR","title":"Select Water Solutions, Inc."},"2731":{"cik_str":1050825,"ticker":"SCS","title":"STEELCASE INC"},"2732":{"cik_str":1489096,"ticker":"THR","title":"Thermon Group Holdings, Inc."},"2733":{"cik_str":14707,"ticker":"CAL","title":"CALERES INC"},"2734":{"cik_str":1728190,"ticker":"HUYA","title":"HUYA Inc."},"2735":{"cik_str":1836833,"ticker":"PL","title":"Planet Labs PBC"},"2736":{"cik_str":1836056,"ticker":"AMAM","title":"Ambrx Biopharma Inc."},"2737":{"cik_str":1840616,"ticker":"NFGC","title":"New Found Gold Corp."},"2738":{"cik_str":1809196,"ticker":"IMTX","title":"Immatics N.V."},"2739":{"cik_str":1659520,"ticker":"SILV","title":"SilverCrest Metals Inc."},"2740":{"cik_str":1793663,"ticker":"VTEX","title":"VTEX"},"2741":{"cik_str":1680062,"ticker":"ACMR","title":"ACM Research, Inc."},"2742":{"cik_str":1805385,"ticker":"EVLV","title":"Evolv Technologies Holdings, Inc."},"2743":{"cik_str":846901,"ticker":"LBAI","title":"LAKELAND BANCORP INC"},"2744":{"cik_str":1423221,"ticker":"NX","title":"Quanex Building Products CORP"},"2745":{"cik_str":915840,"ticker":"BZH","title":"BEAZER HOMES USA INC"},"2746":{"cik_str":1293282,"ticker":"TTGT","title":"TechTarget Inc"},"2747":{"cik_str":1802156,"ticker":"XPOF","title":"Xponential Fitness, Inc."},"2748":{"cik_str":1068875,"ticker":"AVTA","title":"AVANTAX, INC."},"2749":{"cik_str":1023459,"ticker":"SLP","title":"Simulations Plus, Inc."},"2750":{"cik_str":886128,"ticker":"FCEL","title":"FUELCELL ENERGY INC"},"2751":{"cik_str":1828723,"ticker":"AMPS","title":"Altus Power, Inc."},"2752":{"cik_str":1631569,"ticker":"CHCT","title":"Community Healthcare Trust Inc"},"2753":{"cik_str":1592057,"ticker":"EVA","title":"Enviva Inc."},"2754":{"cik_str":1717547,"ticker":"BRSP","title":"BrightSpire Capital, Inc."},"2755":{"cik_str":704562,"ticker":"CDMO","title":"Avid Bioservices, Inc."},"2756":{"cik_str":1425292,"ticker":"UAN","title":"CVR PARTNERS, LP"},"2757":{"cik_str":1591588,"ticker":"AMRK","title":"A-Mark Precious Metals, Inc."},"2758":{"cik_str":1845437,"ticker":"NPWR","title":"NET Power Inc."},"2759":{"cik_str":1379400,"ticker":"AOD","title":"abrdn Total Dynamic Dividend Fund"},"2760":{"cik_str":1746109,"ticker":"BFC","title":"Bank First Corp"},"2761":{"cik_str":1821349,"ticker":"NGMS","title":"NeoGames S.A."},"2762":{"cik_str":1843181,"ticker":"NBXG","title":"Neuberger Berman Next Generation Connectivity Fund Inc."},"2763":{"cik_str":1655050,"ticker":"BCSF","title":"Bain Capital Specialty Finance, Inc."},"2764":{"cik_str":1971213,"ticker":"SBGI","title":"Sinclair, Inc."},"2765":{"cik_str":1748824,"ticker":"BSIG","title":"BrightSphere Investment Group Inc."},"2766":{"cik_str":1590877,"ticker":"RGNX","title":"REGENXBIO Inc."},"2767":{"cik_str":1802974,"ticker":"AVO","title":"Mission Produce, Inc."},"2768":{"cik_str":1222719,"ticker":"CHY","title":"CALAMOS CONVERTIBLE & HIGH INCOME FUND"},"2769":{"cik_str":1371285,"ticker":"TRUP","title":"TRUPANION, INC."},"2770":{"cik_str":1495222,"ticker":"OXLC","title":"Oxford Lane Capital Corp."},"2771":{"cik_str":1868995,"ticker":"CINT","title":"CI&T Inc"},"2772":{"cik_str":890196,"ticker":"MQY","title":"BLACKROCK MUNIYIELD QUALITY FUND, INC."},"2773":{"cik_str":906013,"ticker":"AWF","title":"ALLIANCEBERNSTEIN GLOBAL HIGH INCOME FUND INC"},"2774":{"cik_str":1819142,"ticker":"SES","title":"SES AI Corp"},"2775":{"cik_str":1847584,"ticker":"WKME","title":"WalkMe Ltd."},"2776":{"cik_str":1856430,"ticker":"SNPO","title":"Snap One Holdings Corp."},"2777":{"cik_str":1528129,"ticker":"VTLE","title":"Vital Energy, Inc."},"2778":{"cik_str":1439095,"ticker":"MRC","title":"MRC GLOBAL INC."},"2779":{"cik_str":1904856,"ticker":"EFXT","title":"Enerflex Ltd."},"2780":{"cik_str":1000623,"ticker":"MATV","title":"Mativ Holdings, Inc."},"2781":{"cik_str":1490906,"ticker":"CFFN","title":"Capitol Federal Financial, Inc."},"2782":{"cik_str":1094366,"ticker":"RDWR","title":"RADWARE LTD"},"2783":{"cik_str":1379785,"ticker":"BBDC","title":"Barings BDC, Inc."},"2784":{"cik_str":1769484,"ticker":"BIOX","title":"Bioceres Crop Solutions Corp."},"2785":{"cik_str":1868912,"ticker":"ENFN","title":"Enfusion, Inc."},"2786":{"cik_str":1232860,"ticker":"MUI","title":"BLACKROCK MUNICIPAL INCOME FUND, INC."},"2787":{"cik_str":1851112,"ticker":"RSKD","title":"Riskified Ltd."},"2788":{"cik_str":1631596,"ticker":"KREF","title":"KKR Real Estate Finance Trust Inc."},"2789":{"cik_str":1301787,"ticker":"BXC","title":"BlueLinx Holdings Inc."},"2790":{"cik_str":1556739,"ticker":"THRY","title":"Thryv Holdings, Inc."},"2791":{"cik_str":1655210,"ticker":"BYND","title":"BEYOND MEAT, INC."},"2792":{"cik_str":921557,"ticker":"RBCAA","title":"REPUBLIC BANCORP INC /KY/"},"2793":{"cik_str":1029800,"ticker":"UBP","title":"URSTADT BIDDLE PROPERTIES INC"},"2794":{"cik_str":790816,"ticker":"BDN","title":"BRANDYWINE REALTY TRUST"},"2795":{"cik_str":108385,"ticker":"WRLD","title":"WORLD ACCEPTANCE CORP"},"2796":{"cik_str":1123134,"ticker":"IMOS","title":"CHIPMOS TECHNOLOGIES INC"},"2797":{"cik_str":1795251,"ticker":"NNOX","title":"Nano-X Imaging Ltd."},"2798":{"cik_str":1171471,"ticker":"CHI","title":"CALAMOS CONVERTIBLE OPPORTUNITIES & INCOME FUND"},"2799":{"cik_str":949157,"ticker":"CENX","title":"CENTURY ALUMINUM CO"},"2800":{"cik_str":1886190,"ticker":"GGR","title":"Gogoro Inc."},"2801":{"cik_str":1127703,"ticker":"PRA","title":"PROASSURANCE CORP"},"2802":{"cik_str":33934,"ticker":"CRF","title":"CORNERSTONE TOTAL RETURN FUND INC"},"2803":{"cik_str":1820190,"ticker":"SCLX","title":"Scilex Holding Co"},"2804":{"cik_str":926282,"ticker":"ADTN","title":"ADTRAN Holdings, Inc."},"2805":{"cik_str":1806347,"ticker":"WEST","title":"Westrock Coffee Co"},"2806":{"cik_str":1080014,"ticker":"INVA","title":"Innoviva, Inc."},"2807":{"cik_str":1860543,"ticker":"CDRE","title":"Cadre Holdings, Inc."},"2808":{"cik_str":1278752,"ticker":"MFIC","title":"MidCap Financial Investment Corp"},"2809":{"cik_str":1525221,"ticker":"VTOL","title":"Bristow Group Inc."},"2810":{"cik_str":1512920,"ticker":"SII","title":"SPROTT INC."},"2811":{"cik_str":1005817,"ticker":"TMP","title":"TOMPKINS FINANCIAL CORP"},"2812":{"cik_str":805267,"ticker":"HQH","title":"TEKLA HEALTHCARE INVESTORS"},"2813":{"cik_str":912603,"ticker":"SCHN","title":"SCHNITZER STEEL INDUSTRIES, INC."},"2814":{"cik_str":705432,"ticker":"SBSI","title":"SOUTHSIDE BANCSHARES INC"},"2815":{"cik_str":1859285,"ticker":"HRT","title":"HireRight Holdings Corp"},"2816":{"cik_str":890541,"ticker":"BLX","title":"FOREIGN TRADE BANK OF LATIN AMERICA, INC."},"2817":{"cik_str":93389,"ticker":"SMP","title":"STANDARD MOTOR PRODUCTS, INC."},"2818":{"cik_str":1278895,"ticker":"CII","title":"BLACKROCK ENHANCED CAPITAL & INCOME FUND, INC."},"2819":{"cik_str":755001,"ticker":"UTL","title":"UNITIL CORP"},"2820":{"cik_str":1046568,"ticker":"PRDO","title":"PERDOCEO EDUCATION Corp"},"2821":{"cik_str":1835724,"ticker":"ZH","title":"Zhihu Inc."},"2822":{"cik_str":1503274,"ticker":"QTRX","title":"Quanterix Corp"},"2823":{"cik_str":1368514,"ticker":"ADMA","title":"ADMA BIOLOGICS, INC."},"2824":{"cik_str":1782999,"ticker":"PRTC","title":"PureTech Health plc"},"2825":{"cik_str":714395,"ticker":"GABC","title":"GERMAN AMERICAN BANCORP, INC."},"2826":{"cik_str":814549,"ticker":"EBIX","title":"EBIX INC"},"2827":{"cik_str":1840199,"ticker":"WALD","title":"Waldencast plc"},"2828":{"cik_str":1492165,"ticker":"PFBC","title":"Preferred Bank"},"2829":{"cik_str":1021162,"ticker":"TGI","title":"TRIUMPH GROUP INC"},"2830":{"cik_str":1740594,"ticker":"ARCE","title":"Arco Platform Ltd."},"2831":{"cik_str":1599901,"ticker":"RNA","title":"Avidity Biosciences, Inc."},"2832":{"cik_str":947559,"ticker":"FBMS","title":"FIRST BANCSHARES INC /MS/"},"2833":{"cik_str":1819989,"ticker":"CIFR","title":"Cipher Mining Inc."},"2834":{"cik_str":743238,"ticker":"SHYF","title":"SHYFT GROUP, INC."},"2835":{"cik_str":1332349,"ticker":"BKD","title":"Brookdale Senior Living Inc."},"2836":{"cik_str":1042893,"ticker":"DRQ","title":"DRIL-QUIP INC"},"2837":{"cik_str":1506289,"ticker":"BCX","title":"BlackRock Resources & Commodities Strategy Trust"},"2838":{"cik_str":877422,"ticker":"SPTN","title":"SpartanNash Co"},"2839":{"cik_str":65770,"ticker":"MVIS","title":"MICROVISION, INC."},"2840":{"cik_str":45876,"ticker":"NVRI","title":"ENVIRI Corp"},"2841":{"cik_str":1124524,"ticker":"CYRX","title":"Cryoport, Inc."},"2842":{"cik_str":1687221,"ticker":"REVG","title":"REV Group, Inc."},"2843":{"cik_str":80035,"ticker":"PLPC","title":"PREFORMED LINE PRODUCTS CO"},"2844":{"cik_str":1325964,"ticker":"LWLG","title":"Lightwave Logic, Inc."},"2845":{"cik_str":1728688,"ticker":"IIIV","title":"i3 Verticals, Inc."},"2846":{"cik_str":1604665,"ticker":"WLKP","title":"Westlake Chemical Partners LP"},"2847":{"cik_str":1604522,"ticker":"THQ","title":"Tekla Healthcare Opportunities Fund"},"2848":{"cik_str":1049782,"ticker":"BRKL","title":"BROOKLINE BANCORP INC"},"2849":{"cik_str":1422143,"ticker":"KURA","title":"Kura Oncology, Inc."},"2850":{"cik_str":1754195,"ticker":"TCNNF","title":"Trulieve Cannabis Corp."},"2851":{"cik_str":1059262,"ticker":"SP","title":"SP Plus Corp"},"2852":{"cik_str":1308208,"ticker":"ULH","title":"UNIVERSAL LOGISTICS HOLDINGS, INC."},"2853":{"cik_str":1537137,"ticker":"SLI","title":"STANDARD LITHIUM LTD."},"2854":{"cik_str":1418076,"ticker":"SLRC","title":"SLR Investment Corp."},"2855":{"cik_str":832428,"ticker":"SSP","title":"E.W. SCRIPPS Co"},"2856":{"cik_str":1804591,"ticker":"ME","title":"23andMe Holding Co."},"2857":{"cik_str":1731805,"ticker":"HUT","title":"HUT 8 MINING CORP."},"2858":{"cik_str":800166,"ticker":"NGD","title":"New Gold Inc. /FI"},"2859":{"cik_str":1672619,"ticker":"ELVN","title":"Enliven Therapeutics, Inc."},"2860":{"cik_str":351834,"ticker":"STKL","title":"SunOpta Inc."},"2861":{"cik_str":43196,"ticker":"GTN","title":"GRAY TELEVISION INC"},"2862":{"cik_str":1656472,"ticker":"CRON","title":"Cronos Group Inc."},"2863":{"cik_str":1642122,"ticker":"AC","title":"Associated Capital Group, Inc."},"2864":{"cik_str":1806952,"ticker":"LYEL","title":"Lyell Immunopharma, Inc."},"2865":{"cik_str":1677703,"ticker":"CNDT","title":"CONDUENT Inc"},"2866":{"cik_str":1768259,"ticker":"GOTU","title":"Gaotu Techedu Inc."},"2867":{"cik_str":1837014,"ticker":"SMRT","title":"SmartRent, Inc."},"2868":{"cik_str":1000177,"ticker":"NAT","title":"NORDIC AMERICAN TANKERS Ltd"},"2869":{"cik_str":61398,"ticker":"TELL","title":"TELLURIAN INC. /DE/"},"2870":{"cik_str":1776661,"ticker":"ADV","title":"Advantage Solutions Inc."},"2871":{"cik_str":27996,"ticker":"DLX","title":"DELUXE CORP"},"2872":{"cik_str":1231457,"ticker":"GNLX","title":"GENELUX Corp"},"2873":{"cik_str":1013488,"ticker":"BJRI","title":"BJs RESTAURANTS INC"},"2874":{"cik_str":1823986,"ticker":"WDH","title":"Waterdrop Inc."},"2875":{"cik_str":1376339,"ticker":"MDXG","title":"MIMEDX GROUP, INC."},"2876":{"cik_str":1398987,"ticker":"HOUS","title":"Anywhere Real Estate Inc."},"2877":{"cik_str":1768224,"ticker":"ARCT","title":"Arcturus Therapeutics Holdings Inc."},"2878":{"cik_str":1828316,"ticker":"PHAR","title":"Pharming Group N.V."},"2879":{"cik_str":1609151,"ticker":"WEAV","title":"Weave Communications, Inc."},"2880":{"cik_str":1829432,"ticker":"AAC","title":"Ares Acquisition Corp"},"2881":{"cik_str":60714,"ticker":"LXU","title":"LSB INDUSTRIES, INC."},"2882":{"cik_str":1840439,"ticker":"BMEA","title":"Biomea Fusion, Inc."},"2883":{"cik_str":1643303,"ticker":"NNDM","title":"Nano Dimension Ltd."},"2884":{"cik_str":17313,"ticker":"CSWC","title":"CAPITAL SOUTHWEST CORP"},"2885":{"cik_str":1828852,"ticker":"MOND","title":"Mondee Holdings, Inc."},"2886":{"cik_str":1524931,"ticker":"CHUY","title":"CHUY'S HOLDINGS, INC."},"2887":{"cik_str":729580,"ticker":"BELFA","title":"BEL FUSE INC /NJ"},"2888":{"cik_str":1754170,"ticker":"ASLE","title":"AerSale Corp"},"2889":{"cik_str":1095565,"ticker":"HSTM","title":"HEALTHSTREAM INC"},"2890":{"cik_str":1783183,"ticker":"PHAT","title":"Phathom Pharmaceuticals, Inc."},"2891":{"cik_str":1944057,"ticker":"CLCO","title":"Cool Co Ltd."},"2892":{"cik_str":42682,"ticker":"GRC","title":"GORMAN RUPP CO"},"2893":{"cik_str":1865408,"ticker":"EXAI","title":"Exscientia plc"},"2894":{"cik_str":1057060,"ticker":"HZO","title":"MARINEMAX INC"},"2895":{"cik_str":1650648,"ticker":"FDMT","title":"4D Molecular Therapeutics, Inc."},"2896":{"cik_str":883412,"ticker":"MYI","title":"BLACKROCK MUNIYIELD QUALITY FUND III, INC."},"2897":{"cik_str":1860805,"ticker":"ASTL","title":"Algoma Steel Group Inc."},"2898":{"cik_str":1544206,"ticker":"CGBD","title":"Carlyle Secured Lending, Inc."},"2899":{"cik_str":1761612,"ticker":"BCYC","title":"BICYCLE THERAPEUTICS plc"},"2900":{"cik_str":1085913,"ticker":"FMCB","title":"FARMERS & MERCHANTS BANCORP"},"2901":{"cik_str":1538849,"ticker":"CAPL","title":"CrossAmerica Partners LP"},"2902":{"cik_str":1267565,"ticker":"COLL","title":"COLLEGIUM PHARMACEUTICAL, INC"},"2903":{"cik_str":1557746,"ticker":"ACRS","title":"Aclaris Therapeutics, Inc."},"2904":{"cik_str":1219360,"ticker":"PHK","title":"PIMCO HIGH INCOME FUND"},"2905":{"cik_str":918965,"ticker":"SCSC","title":"SCANSOURCE, INC."},"2906":{"cik_str":1626450,"ticker":"BIGC","title":"BigCommerce Holdings, Inc."},"2907":{"cik_str":1720592,"ticker":"RPAY","title":"Repay Holdings Corp"},"2908":{"cik_str":1282631,"ticker":"NLST","title":"NETLIST INC"},"2909":{"cik_str":1874474,"ticker":"ALLG","title":"Allego N.V."},"2910":{"cik_str":899751,"ticker":"TWI","title":"TITAN INTERNATIONAL INC"},"2911":{"cik_str":1863218,"ticker":"SFWL","title":"SHENGFENG DEVELOPMENT Ltd"},"2912":{"cik_str":1873331,"ticker":"PGRU","title":"PROPERTYGURU GROUP LTD"},"2913":{"cik_str":896156,"ticker":"ETD","title":"ETHAN ALLEN INTERIORS INC"},"2914":{"cik_str":1625278,"ticker":"NRDS","title":"NERDWALLET, INC."},"2915":{"cik_str":1001614,"ticker":"REPX","title":"Riley Exploration Permian, Inc."},"2916":{"cik_str":1701051,"ticker":"WOW","title":"WideOpenWest, Inc."},"2917":{"cik_str":1260041,"ticker":"HTD","title":"JOHN HANCOCK TAX-ADVANTAGED DIVIDEND INCOME FUND"},"2918":{"cik_str":1747079,"ticker":"BALY","title":"Bally's Corp"},"2919":{"cik_str":930236,"ticker":"RWT","title":"REDWOOD TRUST INC"},"2920":{"cik_str":1777835,"ticker":"PWP","title":"Perella Weinberg Partners"},"2921":{"cik_str":910612,"ticker":"CBL","title":"CBL & ASSOCIATES PROPERTIES INC"},"2922":{"cik_str":796505,"ticker":"CLFD","title":"Clearfield, Inc."},"2923":{"cik_str":1737287,"ticker":"ALLO","title":"Allogene Therapeutics, Inc."},"2924":{"cik_str":69488,"ticker":"MYE","title":"MYERS INDUSTRIES INC"},"2925":{"cik_str":80420,"ticker":"POWL","title":"POWELL INDUSTRIES INC"},"2926":{"cik_str":1497645,"ticker":"INN","title":"Summit Hotel Properties, Inc."},"2927":{"cik_str":1589526,"ticker":"BLBD","title":"Blue Bird Corp"},"2928":{"cik_str":935419,"ticker":"RICK","title":"RCI HOSPITALITY HOLDINGS, INC."},"2929":{"cik_str":1334978,"ticker":"CCO","title":"Clear Channel Outdoor Holdings, Inc."},"2930":{"cik_str":1053706,"ticker":"CRAI","title":"CRA INTERNATIONAL, INC."},"2931":{"cik_str":1855756,"ticker":"LILM","title":"Lilium N.V."},"2932":{"cik_str":1315257,"ticker":"KOP","title":"Koppers Holdings Inc."},"2933":{"cik_str":1945711,"ticker":"LVRO","title":"Lavoro Ltd"},"2934":{"cik_str":1855066,"ticker":"MEGI","title":"MainStay CBRE Global Infrastructure Megatrends Fund"},"2935":{"cik_str":1828102,"ticker":"KARO","title":"Karooooo Ltd."},"2936":{"cik_str":1124796,"ticker":"LASR","title":"NLIGHT, INC."},"2937":{"cik_str":1845022,"ticker":"BASE","title":"Couchbase, Inc."},"2938":{"cik_str":1176984,"ticker":"EIM","title":"EATON VANCE MUNICIPAL BOND FUND"},"2939":{"cik_str":1651958,"ticker":"EOSS","title":"EOS INC."},"2940":{"cik_str":1828250,"ticker":"CCVI","title":"Churchill Capital Corp VI"},"2941":{"cik_str":906465,"ticker":"QCRH","title":"QCR HOLDINGS INC"},"2942":{"cik_str":1756908,"ticker":"NRGX","title":"PIMCO Energy & Tactical Credit Opportunities Fund"},"2943":{"cik_str":1437402,"ticker":"ARDX","title":"ARDELYX, INC."},"2944":{"cik_str":1793659,"ticker":"RSI","title":"Rush Street Interactive, Inc."},"2945":{"cik_str":852772,"ticker":"DENN","title":"DENNY'S Corp"},"2946":{"cik_str":846617,"ticker":"DCOM","title":"Dime Community Bancshares, Inc. /NY/"},"2947":{"cik_str":784199,"ticker":"AORT","title":"ARTIVION, INC."},"2948":{"cik_str":1819404,"ticker":"NRDY","title":"Nerdy Inc."},"2949":{"cik_str":1831828,"ticker":"VERA","title":"Vera Therapeutics, Inc."},"2950":{"cik_str":1430725,"ticker":"GSL","title":"Global Ship Lease, Inc."},"2951":{"cik_str":1788399,"ticker":"DLY","title":"DoubleLine Yield Opportunities Fund"},"2952":{"cik_str":1819559,"ticker":"WDI","title":"Western Asset Diversified Income Fund"},"2953":{"cik_str":1821160,"ticker":"GHLD","title":"Guild Holdings Co"},"2954":{"cik_str":1287213,"ticker":"PLOW","title":"DOUGLAS DYNAMICS, INC"},"2955":{"cik_str":1381074,"ticker":"RTC","title":"Baijiayun Group Ltd"},"2956":{"cik_str":1702750,"ticker":"BY","title":"BYLINE BANCORP, INC."},"2957":{"cik_str":1216583,"ticker":"JPC","title":"Nuveen Preferred & Income Opportunities Fund"},"2958":{"cik_str":1620459,"ticker":"JRVR","title":"James River Group Holdings, Ltd."},"2959":{"cik_str":102752,"ticker":"VSEC","title":"VSE CORP"},"2960":{"cik_str":54381,"ticker":"KAMN","title":"KAMAN Corp"},"2961":{"cik_str":1837686,"ticker":"VMEO","title":"Vimeo, Inc."},"2962":{"cik_str":724004,"ticker":"MLAB","title":"MESA LABORATORIES INC /CO/"},"2963":{"cik_str":1794350,"ticker":"YALA","title":"Yalla Group Ltd"},"2964":{"cik_str":897429,"ticker":"CHS","title":"CHICO'S FAS, INC."},"2965":{"cik_str":1227476,"ticker":"JQC","title":"Nuveen Credit Strategies Income Fund"},"2966":{"cik_str":1636422,"ticker":"HCAT","title":"Health Catalyst, Inc."},"2967":{"cik_str":1800682,"ticker":"CANO","title":"Cano Health, Inc."},"2968":{"cik_str":1319947,"ticker":"DBI","title":"Designer Brands Inc."},"2969":{"cik_str":1169770,"ticker":"BANC","title":"BANC OF CALIFORNIA, INC."},"2970":{"cik_str":1050441,"ticker":"EGBN","title":"EAGLE BANCORP INC"},"2971":{"cik_str":1606757,"ticker":"KE","title":"Kimball Electronics, Inc."},"2972":{"cik_str":1409171,"ticker":"TITN","title":"Titan Machinery Inc."},"2973":{"cik_str":1743725,"ticker":"GDYN","title":"GRID DYNAMICS HOLDINGS, INC."},"2974":{"cik_str":876779,"ticker":"MGIC","title":"MAGIC SOFTWARE ENTERPRISES LTD"},"2975":{"cik_str":1013605,"ticker":"PDS","title":"PRECISION DRILLING Corp"},"2976":{"cik_str":790500,"ticker":"FAX","title":"ABRDN ASIA-PACIFIC INCOME FUND, INC."},"2977":{"cik_str":1514705,"ticker":"SXC","title":"SunCoke Energy, Inc."},"2978":{"cik_str":826675,"ticker":"DX","title":"DYNEX CAPITAL INC"},"2979":{"cik_str":883618,"ticker":"NXP","title":"NUVEEN SELECT TAX FREE INCOME PORTFOLIO"},"2980":{"cik_str":1838957,"ticker":"RERE","title":"ATRenew Inc."},"2981":{"cik_str":1831631,"ticker":"LDI","title":"loanDepot, Inc."},"2982":{"cik_str":1734713,"ticker":"BV","title":"BrightView Holdings, Inc."},"2983":{"cik_str":1894630,"ticker":"GHIX","title":"Gores Holdings IX, Inc."},"2984":{"cik_str":1650664,"ticker":"EDIT","title":"Editas Medicine, Inc."},"2985":{"cik_str":1855129,"ticker":"GHRS","title":"GH Research PLC"},"2986":{"cik_str":1174164,"ticker":"FFC","title":"Flaherty & Crumrine PREFERRED & INCOME SECURITIES FUND INC"},"2987":{"cik_str":1713930,"ticker":"NEXA","title":"Nexa Resources S.A."},"2988":{"cik_str":1300391,"ticker":"EOI","title":"Eaton Vance Enhanced Equity Income Fund"},"2989":{"cik_str":1849056,"ticker":"ALCC","title":"AltC Acquisition Corp."},"2990":{"cik_str":1171486,"ticker":"NRP","title":"NATURAL RESOURCE PARTNERS LP"},"2991":{"cik_str":1415921,"ticker":"NMM","title":"Navios Maritime Partners L.P."},"2992":{"cik_str":1005731,"ticker":"IDT","title":"IDT CORP"},"2993":{"cik_str":712771,"ticker":"CNOB","title":"ConnectOne Bancorp, Inc."},"2994":{"cik_str":61986,"ticker":"MTW","title":"MANITOWOC CO INC"},"2995":{"cik_str":798783,"ticker":"UHT","title":"UNIVERSAL HEALTH REALTY INCOME TRUST"},"2996":{"cik_str":1778129,"ticker":"TRSSF","title":"TerrAscend Corp."},"2997":{"cik_str":884624,"ticker":"OFIX","title":"Orthofix Medical Inc."},"2998":{"cik_str":895447,"ticker":"SCVL","title":"SHOE CARNIVAL INC"},"2999":{"cik_str":1220754,"ticker":"MODV","title":"ModivCare Inc"},"3000":{"cik_str":1476840,"ticker":"EXFY","title":"Expensify, Inc."},"3001":{"cik_str":1484769,"ticker":"FUBO","title":"fuboTV Inc. /FL"},"3002":{"cik_str":1559432,"ticker":"TXO","title":"TXO Energy Partners, L.P."},"3003":{"cik_str":351817,"ticker":"SBOW","title":"SILVERBOW RESOURCES, INC."},"3004":{"cik_str":1591670,"ticker":"FPI","title":"Farmland Partners Inc."},"3005":{"cik_str":1888012,"ticker":"HLVX","title":"HilleVax, Inc."},"3006":{"cik_str":1818794,"ticker":"DYN","title":"Dyne Therapeutics, Inc."},"3007":{"cik_str":8063,"ticker":"ATRO","title":"ASTRONICS CORP"},"3008":{"cik_str":1863127,"ticker":"TYRA","title":"Tyra Biosciences, Inc."},"3009":{"cik_str":350852,"ticker":"CTBI","title":"COMMUNITY TRUST BANCORP INC /KY/"},"3010":{"cik_str":99302,"ticker":"TRNS","title":"TRANSCAT INC"},"3011":{"cik_str":1933414,"ticker":"MLYS","title":"Mineralys Therapeutics, Inc."},"3012":{"cik_str":1500435,"ticker":"GPRO","title":"GoPro, Inc."},"3013":{"cik_str":1673772,"ticker":"RAPT","title":"RAPT Therapeutics, Inc."},"3014":{"cik_str":1944558,"ticker":"VTS","title":"Vitesse Energy, Inc."},"3015":{"cik_str":1886878,"ticker":"PAXS","title":"PIMCO Access Income Fund"},"3016":{"cik_str":858655,"ticker":"HAYN","title":"HAYNES INTERNATIONAL INC"},"3017":{"cik_str":1020710,"ticker":"DXPE","title":"DXP ENTERPRISES INC"},"3018":{"cik_str":1823878,"ticker":"MYPS","title":"PLAYSTUDIOS, Inc."},"3019":{"cik_str":1345099,"ticker":"MESO","title":"MESOBLAST LTD"},"3020":{"cik_str":1314052,"ticker":"AVXL","title":"ANAVEX LIFE SCIENCES CORP."},"3021":{"cik_str":874501,"ticker":"AMBC","title":"AMBAC FINANCIAL GROUP INC"},"3022":{"cik_str":1946425,"ticker":"HHRS","title":"Hammerhead Energy Inc."},"3023":{"cik_str":1533615,"ticker":"GMRE","title":"Global Medical REIT Inc."},"3024":{"cik_str":46129,"ticker":"AMOT","title":"ALLIED MOTION TECHNOLOGIES INC"},"3025":{"cik_str":1177161,"ticker":"EVM","title":"EATON VANCE CALIFORNIA MUNICIPAL BOND FUND"},"3026":{"cik_str":1084201,"ticker":"SVA","title":"SINOVAC BIOTECH LTD"},"3027":{"cik_str":55135,"ticker":"KELYA","title":"KELLY SERVICES INC"},"3028":{"cik_str":799850,"ticker":"CRMT","title":"AMERICAS CARMART INC"},"3029":{"cik_str":1778114,"ticker":"AIO","title":"Virtus Artificial Intelligence & Technology Opportunities Fund"},"3030":{"cik_str":884152,"ticker":"VMO","title":"Invesco Municipal Opportunity Trust"},"3031":{"cik_str":1853138,"ticker":"AACT","title":"Ares Acquisition Corp II"},"3032":{"cik_str":30305,"ticker":"DCO","title":"DUCOMMUN INC /DE/"},"3033":{"cik_str":1671858,"ticker":"SPRY","title":"ARS Pharmaceuticals, Inc."},"3034":{"cik_str":1866633,"ticker":"CCSI","title":"Consensus Cloud Solutions, Inc."},"3035":{"cik_str":1000694,"ticker":"NVAX","title":"NOVAVAX INC"},"3036":{"cik_str":1370755,"ticker":"TCPC","title":"BlackRock TCP Capital Corp."},"3037":{"cik_str":1687926,"ticker":"ZUUS","title":"ZEUUS, INC."},"3038":{"cik_str":1320375,"ticker":"BOE","title":"BlackRock Enhanced Global Dividend Trust"},"3039":{"cik_str":1833835,"ticker":"PSFE","title":"Paysafe Ltd"},"3040":{"cik_str":1010000,"ticker":"RUPRF","title":"RUPERT RESOURCES LTD"},"3041":{"cik_str":788329,"ticker":"JOUT","title":"JOHNSON OUTDOORS INC"},"3042":{"cik_str":827187,"ticker":"SNBR","title":"Sleep Number Corp"},"3043":{"cik_str":1609809,"ticker":"MCRB","title":"Seres Therapeutics, Inc."},"3044":{"cik_str":1853962,"ticker":"IAUX","title":"i-80 Gold Corp."},"3045":{"cik_str":78814,"ticker":"PBI","title":"PITNEY BOWES INC /DE/"},"3046":{"cik_str":854560,"ticker":"GSBC","title":"GREAT SOUTHERN BANCORP, INC."},"3047":{"cik_str":1034665,"ticker":"MHD","title":"BLACKROCK MUNIHOLDINGS FUND, INC."},"3048":{"cik_str":1710680,"ticker":"HFRO","title":"HIGHLAND INCOME FUND\\MA"},"3049":{"cik_str":1468666,"ticker":"SCWX","title":"SecureWorks Corp"},"3050":{"cik_str":1854545,"ticker":"DDL","title":"Dingdong (Cayman) Ltd"},"3051":{"cik_str":1816233,"ticker":"SHCR","title":"Sharecare, Inc."},"3052":{"cik_str":1604174,"ticker":"ECC","title":"Eagle Point Credit Co Inc."},"3053":{"cik_str":1438231,"ticker":"DMRC","title":"Digimarc CORP"},"3054":{"cik_str":1866501,"ticker":"WBX","title":"Wallbox N.V."},"3055":{"cik_str":1268884,"ticker":"IGR","title":"CBRE GLOBAL REAL ESTATE INCOME FUND"},"3056":{"cik_str":732417,"ticker":"HBIA","title":"HILLS BANCORPORATION"},"3057":{"cik_str":1108109,"ticker":"CYH","title":"COMMUNITY HEALTH SYSTEMS INC"},"3058":{"cik_str":1464343,"ticker":"ATLC","title":"Atlanticus Holdings Corp"},"3059":{"cik_str":1160990,"ticker":"PCN","title":"PIMCO CORPORATE & INCOME STRATEGY FUND"},"3060":{"cik_str":1781753,"ticker":"DAO","title":"Youdao, Inc."},"3061":{"cik_str":886206,"ticker":"FC","title":"FRANKLIN COVEY CO"},"3062":{"cik_str":1053988,"ticker":"MUJ","title":"BLACKROCK MUNIHOLDINGS NEW JERSEY QUALITY FUND, INC."},"3063":{"cik_str":1793129,"ticker":"NDMO","title":"Nuveen Dynamic Municipal Opportunities Fund"},"3064":{"cik_str":1802457,"ticker":"ORGN","title":"Origin Materials, Inc."},"3065":{"cik_str":1494582,"ticker":"BOC","title":"BOSTON OMAHA Corp"},"3066":{"cik_str":72573,"ticker":"MOV","title":"MOVADO GROUP INC"},"3067":{"cik_str":853180,"ticker":"VSLAX","title":"Invesco Senior Loan Fund"},"3068":{"cik_str":764401,"ticker":"IIIN","title":"INSTEEL INDUSTRIES INC"},"3069":{"cik_str":1726173,"ticker":"BH-A","title":"Biglari Holdings Inc."},"3070":{"cik_str":1296250,"ticker":"PFN","title":"PIMCO Income Strategy Fund II"},"3071":{"cik_str":1134982,"ticker":"PVNC","title":"PREVENTION INSURANCE COM"},"3072":{"cik_str":1922097,"ticker":"LANV","title":"Lanvin Group Holdings Ltd"},"3073":{"cik_str":1071058,"ticker":"ABST","title":"ABSOLUTE SOFTWARE CORP"},"3074":{"cik_str":1057379,"ticker":"HCKT","title":"HACKETT GROUP, INC."},"3075":{"cik_str":1092796,"ticker":"SWBI","title":"SMITH & WESSON BRANDS, INC."},"3076":{"cik_str":744187,"ticker":"REX","title":"REX AMERICAN RESOURCES Corp"},"3077":{"cik_str":1304492,"ticker":"ATEX","title":"Anterix Inc."},"3078":{"cik_str":1290877,"ticker":"CLVLY","title":"Clinuvel Pharmaceuticals Ltd"},"3079":{"cik_str":1583107,"ticker":"TBPH","title":"Theravance Biopharma, Inc."},"3080":{"cik_str":911971,"ticker":"TK","title":"TEEKAY CORP"},"3081":{"cik_str":1173489,"ticker":"CEVA","title":"CEVA INC"},"3082":{"cik_str":1325618,"ticker":"IRMD","title":"IRADIMED CORP"},"3083":{"cik_str":1775734,"ticker":"BENF","title":"Beneficient"},"3084":{"cik_str":1170299,"ticker":"PML","title":"PIMCO MUNICIPAL INCOME FUND II"},"3085":{"cik_str":1901279,"ticker":"NYAX","title":"Nayax Ltd."},"3086":{"cik_str":1512499,"ticker":"LIND","title":"LINDBLAD EXPEDITIONS HOLDINGS, INC."},"3087":{"cik_str":1072379,"ticker":"NWBO","title":"NORTHWEST BIOTHERAPEUTICS INC"},"3088":{"cik_str":1899287,"ticker":"AMPX","title":"Amprius Technologies, Inc."},"3089":{"cik_str":1581091,"ticker":"RMAX","title":"RE/MAX Holdings, Inc."},"3090":{"cik_str":1044777,"ticker":"OSPN","title":"OneSpan Inc."},"3091":{"cik_str":1774342,"ticker":"NMCO","title":"Nuveen Municipal Credit Opportunities Fund"},"3092":{"cik_str":315858,"ticker":"BVH","title":"Bluegreen Vacations Holding Corp"},"3093":{"cik_str":902791,"ticker":"BBSI","title":"BARRETT BUSINESS SERVICES INC"},"3094":{"cik_str":1059386,"ticker":"VVR","title":"Invesco Senior Income Trust"},"3095":{"cik_str":798287,"ticker":"PTSI","title":"PAM TRANSPORTATION SERVICES INC"},"3096":{"cik_str":108985,"ticker":"YORW","title":"YORK WATER CO"},"3097":{"cik_str":1495240,"ticker":"LAND","title":"GLADSTONE LAND Corp"},"3098":{"cik_str":1383441,"ticker":"NIE","title":"Virtus Equity & Convertible Income Fund"},"3099":{"cik_str":1393584,"ticker":"AMWL","title":"American Well Corp"},"3100":{"cik_str":1326200,"ticker":"GNK","title":"GENCO SHIPPING & TRADING LTD"},"3101":{"cik_str":1383082,"ticker":"DBA","title":"INVESCO DB AGRICULTURE FUND"},"3102":{"cik_str":1517399,"ticker":"SUPV","title":"Grupo Supervielle S.A."},"3103":{"cik_str":1145986,"ticker":"ASPN","title":"ASPEN AEROGELS INC"},"3104":{"cik_str":890447,"ticker":"VTNR","title":"Vertex Energy Inc."},"3105":{"cik_str":357173,"ticker":"OSBC","title":"OLD SECOND BANCORP INC"},"3106":{"cik_str":775215,"ticker":"HBT","title":"HBT Financial, Inc."},"3107":{"cik_str":1129155,"ticker":"MPX","title":"MARINE PRODUCTS CORP"},"3108":{"cik_str":946647,"ticker":"PFC","title":"PREMIER FINANCIAL CORP"},"3109":{"cik_str":101199,"ticker":"UFCS","title":"UNITED FIRE GROUP INC"},"3110":{"cik_str":879585,"ticker":"ATNI","title":"ATN International, Inc."},"3111":{"cik_str":1844270,"ticker":"ZING","title":"FTAC ZEUS ACQUISITION CORP."},"3112":{"cik_str":859070,"ticker":"FCBC","title":"FIRST COMMUNITY BANKSHARES INC /VA/"},"3113":{"cik_str":1899830,"ticker":"PERF","title":"Perfect Corp."},"3114":{"cik_str":928658,"ticker":"CVLG","title":"COVENANT LOGISTICS GROUP, INC."},"3115":{"cik_str":1313510,"ticker":"GGN","title":"GAMCO Global Gold, Natural Resources & Income Trust"},"3116":{"cik_str":1160864,"ticker":"BHK","title":"BLACKROCK CORE BOND TRUST"},"3117":{"cik_str":1602584,"ticker":"CCD","title":"Calamos Dynamic Convertible & Income Fund"},"3118":{"cik_str":1847903,"ticker":"CNTA","title":"Centessa Pharmaceuticals plc"},"3119":{"cik_str":1633336,"ticker":"CCAP","title":"Crescent Capital BDC, Inc."},"3120":{"cik_str":896429,"ticker":"CTLP","title":"CANTALOUPE, INC."},"3121":{"cik_str":1534254,"ticker":"CION","title":"CION Investment Corp"},"3122":{"cik_str":1046025,"ticker":"HFWA","title":"HERITAGE FINANCIAL CORP /WA/"},"3123":{"cik_str":1172222,"ticker":"HA","title":"HAWAIIAN HOLDINGS INC"},"3124":{"cik_str":1818383,"ticker":"MAX","title":"MediaAlpha, Inc."},"3125":{"cik_str":1062822,"ticker":"LXRX","title":"LEXICON PHARMACEUTICALS, INC."},"3126":{"cik_str":1464591,"ticker":"GPRK","title":"GeoPark Ltd"},"3127":{"cik_str":1630472,"ticker":"TRTX","title":"TPG RE Finance Trust, Inc."},"3128":{"cik_str":1288403,"ticker":"WTI","title":"W&T OFFSHORE INC"},"3129":{"cik_str":1704711,"ticker":"FNKO","title":"Funko, Inc."},"3130":{"cik_str":1027838,"ticker":"TCMD","title":"TACTILE SYSTEMS TECHNOLOGY INC"},"3131":{"cik_str":1810019,"ticker":"RXT","title":"Rackspace Technology, Inc."},"3132":{"cik_str":1759824,"ticker":"ALTG","title":"ALTA EQUIPMENT GROUP INC."},"3133":{"cik_str":1772921,"ticker":"ONEW","title":"OneWater Marine Inc."},"3134":{"cik_str":1277866,"ticker":"EXK","title":"ENDEAVOUR SILVER CORP"},"3135":{"cik_str":884219,"ticker":"VVI","title":"VIAD CORP"},"3136":{"cik_str":1436208,"ticker":"LEGH","title":"Legacy Housing Corp"},"3137":{"cik_str":1788999,"ticker":"XPER","title":"Xperi Inc."},"3138":{"cik_str":1590560,"ticker":"QURE","title":"uniQure N.V."},"3139":{"cik_str":1754820,"ticker":"DM","title":"Desktop Metal, Inc."},"3140":{"cik_str":1803737,"ticker":"EHAB","title":"Enhabit, Inc."},"3141":{"cik_str":855886,"ticker":"PDT","title":"JOHN HANCOCK PREMIUM DIVIDEND FUND"},"3142":{"cik_str":1349436,"ticker":"SD","title":"SANDRIDGE ENERGY INC"},"3143":{"cik_str":357301,"ticker":"TRST","title":"TRUSTCO BANK CORP N Y"},"3144":{"cik_str":1393299,"ticker":"BGY","title":"BlackRock Enhanced International Dividend Trust"},"3145":{"cik_str":885601,"ticker":"IIM","title":"Invesco Value Municipal Income Trust"},"3146":{"cik_str":1395325,"ticker":"ETJ","title":"Eaton Vance Risk-Managed Diversified Equity Income Fund"},"3147":{"cik_str":1314966,"ticker":"BME","title":"BlackRock Health Sciences Trust"},"3148":{"cik_str":1023313,"ticker":"FORR","title":"FORRESTER RESEARCH, INC."},"3149":{"cik_str":1069899,"ticker":"PAHC","title":"PHIBRO ANIMAL HEALTH CORP"},"3150":{"cik_str":1075415,"ticker":"DHC","title":"DIVERSIFIED HEALTHCARE TRUST"},"3151":{"cik_str":1805012,"ticker":"VTRU","title":"Vitru Ltd"},"3152":{"cik_str":879911,"ticker":"AERG","title":"APPLIED ENERGETICS, INC."},"3153":{"cik_str":1850235,"ticker":"HEPS","title":"D-MARKET Electronic Services & Trading"},"3154":{"cik_str":1393726,"ticker":"TIPT","title":"TIPTREE INC."},"3155":{"cik_str":1529377,"ticker":"ACRE","title":"Ares Commercial Real Estate Corp"},"3156":{"cik_str":1819580,"ticker":"YSG","title":"Yatsen Holding Ltd"},"3157":{"cik_str":1360214,"ticker":"HROW","title":"HARROW HEALTH, INC."},"3158":{"cik_str":1805077,"ticker":"EOSE","title":"Eos Energy Enterprises, Inc."},"3159":{"cik_str":105319,"ticker":"WW","title":"WW INTERNATIONAL, INC."},"3160":{"cik_str":768899,"ticker":"TBI","title":"TrueBlue, Inc."},"3161":{"cik_str":1333274,"ticker":"MERC","title":"MERCER INTERNATIONAL INC."},"3162":{"cik_str":1937653,"ticker":"ZYME","title":"Zymeworks Inc."},"3163":{"cik_str":1846253,"ticker":"OABI","title":"OmniAb, Inc."},"3164":{"cik_str":827876,"ticker":"CLSK","title":"CLEANSPARK, INC."},"3165":{"cik_str":1195738,"ticker":"NKX","title":"NUVEEN CALIFORNIA AMT-FREE QUALITY MUNICIPAL INCOME FUND"},"3166":{"cik_str":917470,"ticker":"ZEUS","title":"OLYMPIC STEEL INC"},"3167":{"cik_str":1823144,"ticker":"CMPO","title":"CompoSecure, Inc."},"3168":{"cik_str":1372020,"ticker":"GLDD","title":"Great Lakes Dredge & Dock CORP"},"3169":{"cik_str":1798562,"ticker":"TMC","title":"TMC the metals Co Inc."},"3170":{"cik_str":1267902,"ticker":"WIW","title":"WESTERN ASSET INFLATION-LINKED OPPORTUNITIES & INCOME FUND"},"3171":{"cik_str":1562818,"ticker":"BIT","title":"BlackRock Multi-Sector Income Trust"},"3172":{"cik_str":1724755,"ticker":"GHG","title":"GreenTree Hospitality Group Ltd."},"3173":{"cik_str":1948455,"ticker":"ISPR","title":"Ispire Technology Inc."},"3174":{"cik_str":1705873,"ticker":"BRY","title":"Berry Corp (bry)"},"3175":{"cik_str":844059,"ticker":"FRPH","title":"FRP HOLDINGS, INC."},"3176":{"cik_str":1638290,"ticker":"MCFT","title":"MasterCraft Boat Holdings, Inc."},"3177":{"cik_str":1857853,"ticker":"COOK","title":"Traeger, Inc."},"3178":{"cik_str":1084765,"ticker":"RGP","title":"RESOURCES CONNECTION, INC."},"3179":{"cik_str":1081834,"ticker":"KUBR","title":"Kuber Resources Corp"},"3180":{"cik_str":925683,"ticker":"BTO","title":"JOHN HANCOCK FINANCIAL OPPORTUNITIES FUND"},"3181":{"cik_str":1267813,"ticker":"MRNS","title":"MARINUS PHARMACEUTICALS, INC."},"3182":{"cik_str":1505952,"ticker":"DOMO","title":"DOMO, INC."},"3183":{"cik_str":1787306,"ticker":"ARQT","title":"Arcutis Biotherapeutics, Inc."},"3184":{"cik_str":100591,"ticker":"AGX","title":"ARGAN INC"},"3185":{"cik_str":1937441,"ticker":"AMBI","title":"Ambipar Emergency Response"},"3186":{"cik_str":708781,"ticker":"CASS","title":"CASS INFORMATION SYSTEMS INC"},"3187":{"cik_str":1548717,"ticker":"LDP","title":"Cohen & Steers Ltd Duration Preferred & Income Fund, Inc."},"3188":{"cik_str":1400810,"ticker":"HCI","title":"HCI Group, Inc."},"3189":{"cik_str":102212,"ticker":"UVSP","title":"UNIVEST FINANCIAL Corp"},"3190":{"cik_str":1066605,"ticker":"HSII","title":"HEIDRICK & STRUGGLES INTERNATIONAL INC"},"3191":{"cik_str":5981,"ticker":"AVD","title":"AMERICAN VANGUARD CORP"},"3192":{"cik_str":1441236,"ticker":"CLW","title":"Clearwater Paper Corp"},"3193":{"cik_str":1504619,"ticker":"PFLT","title":"PennantPark Floating Rate Capital Ltd."},"3194":{"cik_str":33002,"ticker":"EBF","title":"ENNIS, INC."},"3195":{"cik_str":1734342,"ticker":"AMTB","title":"Amerant Bancorp Inc."},"3196":{"cik_str":1098972,"ticker":"AGEN","title":"AGENUS INC"},"3197":{"cik_str":880892,"ticker":"VGM","title":"Invesco Trust for Investment Grade Municipals"},"3198":{"cik_str":1635977,"ticker":"THW","title":"Tekla World Healthcare Fund"},"3199":{"cik_str":216851,"ticker":"PEO","title":"ADAMS NATURAL RESOURCES FUND, INC."},"3200":{"cik_str":726601,"ticker":"CCBG","title":"CAPITAL CITY BANK GROUP INC"},"3201":{"cik_str":80172,"ticker":"NPK","title":"NATIONAL PRESTO INDUSTRIES INC"},"3202":{"cik_str":1227862,"ticker":"EMD","title":"WESTERN ASSET EMERGING MARKETS DEBT FUND INC."},"3203":{"cik_str":1504461,"ticker":"NGL","title":"NGL Energy Partners LP"},"3204":{"cik_str":877463,"ticker":"VKQ","title":"Invesco Municipal Trust"},"3205":{"cik_str":1630805,"ticker":"BW","title":"Babcock & Wilcox Enterprises, Inc."},"3206":{"cik_str":715787,"ticker":"TILE","title":"INTERFACE INC"},"3207":{"cik_str":1830188,"ticker":"UHG","title":"United Homes Group, Inc."},"3208":{"cik_str":1133470,"ticker":"CVGW","title":"CALAVO GROWERS INC"},"3209":{"cik_str":1166663,"ticker":"TNP","title":"TSAKOS ENERGY NAVIGATION LTD"},"3210":{"cik_str":1774983,"ticker":"BROG","title":"Brooge Energy Ltd"},"3211":{"cik_str":1842731,"ticker":"SMWB","title":"SIMILARWEB LTD."},"3212":{"cik_str":1499849,"ticker":"LND","title":"BrasilAgro - Brazilian Agricultural Real Estate Co"},"3213":{"cik_str":1340677,"ticker":"SVM","title":"SILVERCORP METALS INC"},"3214":{"cik_str":1514991,"ticker":"AMCX","title":"AMC Networks Inc."},"3215":{"cik_str":1822966,"ticker":"SMR","title":"NUSCALE POWER Corp"},"3216":{"cik_str":1370053,"ticker":"ANAB","title":"ANAPTYSBIO, INC"},"3217":{"cik_str":1043337,"ticker":"SRI","title":"STONERIDGE INC"},"3218":{"cik_str":1437958,"ticker":"CCB","title":"COASTAL FINANCIAL CORP"},"3219":{"cik_str":1576873,"ticker":"ABML","title":"AMERICAN BATTERY TECHNOLOGY Co"},"3220":{"cik_str":1084869,"ticker":"FLWS","title":"1 800 FLOWERS COM INC"},"3221":{"cik_str":1493225,"ticker":"NFBK","title":"Northfield Bancorp, Inc."},"3222":{"cik_str":1760689,"ticker":"MVST","title":"Microvast Holdings, Inc."},"3223":{"cik_str":1355096,"ticker":"QRTEA","title":"Qurate Retail, Inc."},"3224":{"cik_str":1577437,"ticker":"ASC","title":"Ardmore Shipping Corp"},"3225":{"cik_str":1608742,"ticker":"DIAX","title":"Nuveen Dow 30sm Dynamic Overwrite Fund"},"3226":{"cik_str":1826286,"ticker":"VINP","title":"Vinci Partners Investments Ltd."},"3227":{"cik_str":1817713,"ticker":"JANX","title":"Janux Therapeutics, Inc."},"3228":{"cik_str":1628063,"ticker":"SRG","title":"Seritage Growth Properties"},"3229":{"cik_str":1053352,"ticker":"HTBK","title":"HERITAGE COMMERCE CORP"},"3230":{"cik_str":1487952,"ticker":"VPG","title":"Vishay Precision Group, Inc."},"3231":{"cik_str":712034,"ticker":"ACCO","title":"ACCO BRANDS Corp"},"3232":{"cik_str":1234006,"ticker":"GOOD","title":"GLADSTONE COMMERCIAL CORP"},"3233":{"cik_str":1023731,"ticker":"EGHT","title":"8X8 INC /DE/"},"3234":{"cik_str":1080720,"ticker":"GUT","title":"GABELLI UTILITY TRUST"},"3235":{"cik_str":1368519,"ticker":"NOA","title":"North American Construction Group Ltd."},"3236":{"cik_str":909108,"ticker":"DHIL","title":"DIAMOND HILL INVESTMENT GROUP INC"},"3237":{"cik_str":885125,"ticker":"IQI","title":"Invesco Quality Municipal Income Trust"},"3238":{"cik_str":1176194,"ticker":"BLE","title":"BLACKROCK MUNICIPAL INCOME TRUST II"},"3239":{"cik_str":700565,"ticker":"FMBH","title":"FIRST MID BANCSHARES, INC."},"3240":{"cik_str":1798749,"ticker":"AVTE","title":"Aerovate Therapeutics, Inc."},"3241":{"cik_str":1091223,"ticker":"MTLS","title":"MATERIALISE NV"},"3242":{"cik_str":1849396,"ticker":"TRMR","title":"Tremor International Ltd."},"3243":{"cik_str":1819576,"ticker":"LQDA","title":"Liquidia Corp"},"3244":{"cik_str":1235468,"ticker":"LQDT","title":"LIQUIDITY SERVICES INC"},"3245":{"cik_str":25475,"ticker":"CRD-A","title":"CRAWFORD & CO"},"3246":{"cik_str":310522,"ticker":"FNMA","title":"FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE"},"3247":{"cik_str":1860782,"ticker":"TSVT","title":"2seventy bio, Inc."},"3248":{"cik_str":1065059,"ticker":"LEU","title":"CENTRUS ENERGY CORP"},"3249":{"cik_str":1830487,"ticker":"PHVS","title":"Pharvaris N.V."},"3250":{"cik_str":1653087,"ticker":"ALEC","title":"Alector, Inc."},"3251":{"cik_str":357294,"ticker":"HOV","title":"HOVNANIAN ENTERPRISES INC"},"3252":{"cik_str":1653384,"ticker":"RWAY","title":"Runway Growth Finance Corp."},"3253":{"cik_str":807863,"ticker":"MITK","title":"MITEK SYSTEMS INC"},"3254":{"cik_str":1625641,"ticker":"LAW","title":"CS Disco, Inc."},"3255":{"cik_str":1810523,"ticker":"FTHY","title":"FIRST TRUST HIGH YIELD OPPORTUNITIES 2027 TERM FUND"},"3256":{"cik_str":1471420,"ticker":"STK","title":"Columbia Seligman Premium Technology Growth Fund, Inc."},"3257":{"cik_str":1458412,"ticker":"CFB","title":"CROSSFIRST BANKSHARES, INC."},"3258":{"cik_str":1823608,"ticker":"AMAL","title":"Amalgamated Financial Corp."},"3259":{"cik_str":1944885,"ticker":"APLM","title":"Apollomics Inc."},"3260":{"cik_str":1070524,"ticker":"GCBC","title":"GREENE COUNTY BANCORP INC"},"3261":{"cik_str":1528988,"ticker":"BUI","title":"BlackRock Utilities, Infrastructure & Power Opportunities Trust"},"3262":{"cik_str":1313310,"ticker":"GRVY","title":"GRAVITY Co., Ltd."},"3263":{"cik_str":1016281,"ticker":"CSV","title":"CARRIAGE SERVICES INC"},"3264":{"cik_str":1786108,"ticker":"TRIN","title":"Trinity Capital Inc."},"3265":{"cik_str":1822928,"ticker":"HLLY","title":"Holley Inc."},"3266":{"cik_str":917100,"ticker":"IFN","title":"INDIA FUND, INC."},"3267":{"cik_str":1870940,"ticker":"AIRS","title":"Airsculpt Technologies, Inc."},"3268":{"cik_str":879361,"ticker":"MYD","title":"BLACKROCK MUNIYIELD FUND, INC."},"3269":{"cik_str":1579733,"ticker":"VITL","title":"Vital Farms, Inc."},"3270":{"cik_str":1513363,"ticker":"FDUS","title":"FIDUS INVESTMENT Corp"},"3271":{"cik_str":1050743,"ticker":"PGC","title":"PEAPACK GLADSTONE FINANCIAL CORP"},"3272":{"cik_str":1950246,"ticker":"MTAL","title":"Metals Acquisition Ltd"},"3273":{"cik_str":1546429,"ticker":"BGB","title":"Blackstone Strategic Credit 2027 Term Fund"},"3274":{"cik_str":1476045,"ticker":"CLDT","title":"Chatham Lodging Trust"},"3275":{"cik_str":1175535,"ticker":"WSR","title":"Whitestone REIT"},"3276":{"cik_str":1127537,"ticker":"LUNG","title":"Pulmonx Corp"},"3277":{"cik_str":1640266,"ticker":"VYGR","title":"Voyager Therapeutics, Inc."},"3278":{"cik_str":814585,"ticker":"MBI","title":"MBIA INC"},"3279":{"cik_str":1820378,"ticker":"TBLD","title":"Thornburg Income Builder Opportunities Trust"},"3280":{"cik_str":1121484,"ticker":"OIS","title":"OIL STATES INTERNATIONAL, INC"},"3281":{"cik_str":1759138,"ticker":"CABA","title":"Cabaletta Bio, Inc."},"3282":{"cik_str":863110,"ticker":"ARTNA","title":"ARTESIAN RESOURCES CORP"},"3283":{"cik_str":1708055,"ticker":"RBBN","title":"Ribbon Communications Inc."},"3284":{"cik_str":1334388,"ticker":"OBE","title":"OBSIDIAN ENERGY LTD."},"3285":{"cik_str":1626199,"ticker":"ALPN","title":"ALPINE IMMUNE SCIENCES, INC."},"3286":{"cik_str":216085,"ticker":"HVT","title":"HAVERTY FURNITURE COMPANIES INC"},"3287":{"cik_str":1788257,"ticker":"BWMX","title":"BETTERWARE DE MEXICO, S.A.P.I. DE C.V"},"3288":{"cik_str":1689375,"ticker":"TRDA","title":"Entrada Therapeutics, Inc."},"3289":{"cik_str":1413855,"ticker":"FANH","title":"FANHUA INC."},"3290":{"cik_str":1823466,"ticker":"NOTE","title":"FiscalNote Holdings, Inc."},"3291":{"cik_str":1804176,"ticker":"BFLY","title":"Butterfly Network, Inc."},"3292":{"cik_str":1576942,"ticker":"SFIX","title":"Stitch Fix, Inc."},"3293":{"cik_str":1819253,"ticker":"BTMD","title":"biote Corp."},"3294":{"cik_str":1087786,"ticker":"NXJ","title":"NUVEEN NEW JERSEY QUALITY MUNICIPAL INCOME FUND"},"3295":{"cik_str":1832928,"ticker":"CRLBF","title":"Cresco Labs Inc."},"3296":{"cik_str":3197,"ticker":"CECO","title":"CECO ENVIRONMENTAL CORP"},"3297":{"cik_str":891166,"ticker":"UVE","title":"UNIVERSAL INSURANCE HOLDINGS, INC."},"3298":{"cik_str":1808865,"ticker":"ITOS","title":"iTeos Therapeutics, Inc."},"3299":{"cik_str":1587264,"ticker":"NETI","title":"ENETI INC."},"3300":{"cik_str":1117297,"ticker":"QNST","title":"QUINSTREET, INC"},"3301":{"cik_str":1549595,"ticker":"NRIX","title":"Nurix Therapeutics, Inc."},"3302":{"cik_str":1710072,"ticker":"EWTX","title":"Edgewise Therapeutics, Inc."},"3303":{"cik_str":897448,"ticker":"AMRN","title":"AMARIN CORP PLC\\UK"},"3304":{"cik_str":1518557,"ticker":"MMD","title":"MainStay MacKay DefinedTerm Municipal Opportunities Fund"},"3305":{"cik_str":1617242,"ticker":"KRNY","title":"Kearny Financial Corp."},"3306":{"cik_str":1932770,"ticker":"QSG","title":"QuantaSing Group Ltd"},"3307":{"cik_str":1805833,"ticker":"SST","title":"System1, Inc."},"3308":{"cik_str":1434316,"ticker":"FATE","title":"FATE THERAPEUTICS INC"},"3309":{"cik_str":62234,"ticker":"MCS","title":"MARCUS CORP"},"3310":{"cik_str":706129,"ticker":"HBNC","title":"HORIZON BANCORP INC /IN/"},"3311":{"cik_str":1864208,"ticker":"GUG","title":"Guggenheim Active Allocation Fund"},"3312":{"cik_str":1337117,"ticker":"ITRN","title":"Ituran Location & Control Ltd."},"3313":{"cik_str":800457,"ticker":"DGICA","title":"DONEGAL GROUP INC"},"3314":{"cik_str":1528985,"ticker":"INRE","title":"Inland Real Estate Income Trust, Inc."},"3315":{"cik_str":1623526,"ticker":"STOK","title":"Stoke Therapeutics, Inc."},"3316":{"cik_str":1808805,"ticker":"NAUT","title":"Nautilus Biotechnology, Inc."},"3317":{"cik_str":709337,"ticker":"FMNB","title":"FARMERS NATIONAL BANC CORP /OH/"},"3318":{"cik_str":1849635,"ticker":"DWAC","title":"Digital World Acquisition Corp."},"3319":{"cik_str":1845840,"ticker":"TSAT","title":"Telesat Corp"},"3320":{"cik_str":1437071,"ticker":"IVR","title":"Invesco Mortgage Capital Inc."},"3321":{"cik_str":1002590,"ticker":"SGU","title":"STAR GROUP, L.P."},"3322":{"cik_str":1287098,"ticker":"MXCT","title":"MAXCYTE, INC."},"3323":{"cik_str":1831363,"ticker":"TERN","title":"Terns Pharmaceuticals, Inc."},"3324":{"cik_str":1964333,"ticker":"BHRB","title":"Burke & Herbert Financial Services Corp."},"3325":{"cik_str":1341235,"ticker":"ALDX","title":"Aldeyra Therapeutics, Inc."},"3326":{"cik_str":1109242,"ticker":"HAFC","title":"HANMI FINANCIAL CORP"},"3327":{"cik_str":1325702,"ticker":"MX","title":"MAGNACHIP SEMICONDUCTOR Corp"},"3328":{"cik_str":1400891,"ticker":"IHRT","title":"iHeartMedia, Inc."},"3329":{"cik_str":1270073,"ticker":"ICPT","title":"INTERCEPT PHARMACEUTICALS, INC."},"3330":{"cik_str":1842937,"ticker":"HCVI","title":"Hennessy Capital Investment Corp. VI"},"3331":{"cik_str":1017480,"ticker":"HIBB","title":"HIBBETT INC"},"3332":{"cik_str":1488775,"ticker":"CEM","title":"ClearBridge MLP & Midstream Fund Inc."},"3333":{"cik_str":1177648,"ticker":"ENTA","title":"ENANTA PHARMACEUTICALS INC"},"3334":{"cik_str":1844862,"ticker":"SLDP","title":"Solid Power, Inc."},"3335":{"cik_str":798949,"ticker":"UNTC","title":"UNIT CORP"},"3336":{"cik_str":1501796,"ticker":"AURA","title":"Aura Biosciences, Inc."},"3337":{"cik_str":1322439,"ticker":"EGLE","title":"Eagle Bulk Shipping Inc."},"3338":{"cik_str":1703141,"ticker":"DESP","title":"Despegar.com, Corp."},"3339":{"cik_str":1519061,"ticker":"TSE","title":"Trinseo PLC"},"3340":{"cik_str":1847590,"ticker":"BWMN","title":"Bowman Consulting Group Ltd."},"3341":{"cik_str":1819790,"ticker":"TARS","title":"Tarsus Pharmaceuticals, Inc."},"3342":{"cik_str":96869,"ticker":"TRC","title":"TEJON RANCH CO"},"3343":{"cik_str":1233681,"ticker":"BLW","title":"BLACKROCK Ltd DURATION INCOME TRUST"},"3344":{"cik_str":1877333,"ticker":"THCH","title":"TH International Ltd"},"3345":{"cik_str":1475348,"ticker":"LBC","title":"Luther Burbank Corp"},"3346":{"cik_str":876523,"ticker":"EZPW","title":"EZCORP INC"},"3347":{"cik_str":1799567,"ticker":"DDI","title":"DoubleDown Interactive Co., Ltd."},"3348":{"cik_str":895531,"ticker":"VCV","title":"Invesco California Value Municipal Income Trust"},"3349":{"cik_str":1834974,"ticker":"LEV","title":"Lion Electric Co"},"3350":{"cik_str":1478888,"ticker":"NBB","title":"Nuveen Taxable Municipal Income Fund"},"3351":{"cik_str":1356115,"ticker":"NXDT","title":"NEXPOINT DIVERSIFIED REAL ESTATE TRUST"},"3352":{"cik_str":1109116,"ticker":"EVC","title":"ENTRAVISION COMMUNICATIONS CORP"},"3353":{"cik_str":1683825,"ticker":"TRVG","title":"trivago N.V."},"3354":{"cik_str":1896084,"ticker":"IONR","title":"ioneer Ltd"},"3355":{"cik_str":1073748,"ticker":"YUMM","title":"YUMMIES INC"},"3356":{"cik_str":737468,"ticker":"WASH","title":"WASHINGTON TRUST BANCORP INC"},"3357":{"cik_str":1051003,"ticker":"DSU","title":"BLACKROCK DEBT STRATEGIES FUND, INC."},"3358":{"cik_str":1880441,"ticker":"BFAC","title":"Battery Future Acquisition Corp."},"3359":{"cik_str":1623590,"ticker":"HWGC","title":"Fintech Scion Ltd"},"3360":{"cik_str":1863362,"ticker":"PROC","title":"Procaps Group, S.A."},"3361":{"cik_str":1217234,"ticker":"CDNA","title":"CareDx, Inc."},"3362":{"cik_str":1692705,"ticker":"QD","title":"Qudian Inc."},"3363":{"cik_str":1841137,"ticker":"CPAA","title":"Conyers Park III Acquisition Corp."},"3364":{"cik_str":1598655,"ticker":"GLOP","title":"GasLog Partners LP"},"3365":{"cik_str":1747068,"ticker":"MCBS","title":"MetroCity Bankshares, Inc."},"3366":{"cik_str":1807846,"ticker":"ML","title":"MONEYLION INC."},"3367":{"cik_str":1474627,"ticker":"NEGG","title":"Newegg Commerce, Inc."},"3368":{"cik_str":1554697,"ticker":"GHY","title":"PGIM Global High Yield Fund, Inc."},"3369":{"cik_str":1713210,"ticker":"AATP","title":"Agape ATP Corp"},"3370":{"cik_str":1494904,"ticker":"GBLI","title":"Global Indemnity Group, LLC"},"3371":{"cik_str":1304421,"ticker":"CNSL","title":"Consolidated Communications Holdings, Inc."},"3372":{"cik_str":1276533,"ticker":"JFR","title":"NUVEEN FLOATING RATE INCOME FUND"},"3373":{"cik_str":1756699,"ticker":"TIGR","title":"UP Fintech Holding Ltd"},"3374":{"cik_str":750686,"ticker":"CAC","title":"CAMDEN NATIONAL CORP"},"3375":{"cik_str":1137393,"ticker":"BFK","title":"BLACKROCK MUNICIPAL INCOME TRUST"},"3376":{"cik_str":1821393,"ticker":"AAN","title":"Aaron's Company, Inc."},"3377":{"cik_str":1793855,"ticker":"ASGI","title":"abrdn Global Infrastructure Income Fund"},"3378":{"cik_str":724910,"ticker":"NVEC","title":"NVE CORP /NEW/"},"3379":{"cik_str":1785493,"ticker":"PXPC","title":"Phoenix Plus Corp."},"3380":{"cik_str":1668673,"ticker":"PETQ","title":"PetIQ, Inc."},"3381":{"cik_str":1953530,"ticker":"OCS","title":"Oculis Holding AG"},"3382":{"cik_str":1579597,"ticker":"ITEPF","title":"ITE Group plc/ADR"},"3383":{"cik_str":1455684,"ticker":"TPIC","title":"TPI COMPOSITES, INC"},"3384":{"cik_str":1713748,"ticker":"SKE","title":"Skeena Resources Ltd"},"3385":{"cik_str":1887388,"ticker":"BCHPY","title":"Brainchip Holdings Limited/ADR"},"3386":{"cik_str":1850502,"ticker":"NFYS","title":"Enphys Acquisition Corp."},"3387":{"cik_str":1847112,"ticker":"TRTL","title":"TortoiseEcofin Acquisition Corp. III"},"3388":{"cik_str":1215913,"ticker":"HPS","title":"JOHN HANCOCK PREFERRED INCOME FUND III"},"3389":{"cik_str":1221029,"ticker":"CPAC","title":"CEMENTOS PACASMAYO SAA"},"3390":{"cik_str":1466026,"ticker":"MSBI","title":"Midland States Bancorp, Inc."},"3391":{"cik_str":1042729,"ticker":"MBWM","title":"MERCANTILE BANK CORP"},"3392":{"cik_str":71829,"ticker":"NR","title":"NEWPARK RESOURCES INC"},"3393":{"cik_str":916907,"ticker":"SMBC","title":"SOUTHERN MISSOURI BANCORP, INC."},"3394":{"cik_str":1703956,"ticker":"BBCP","title":"Concrete Pumping Holdings, Inc."},"3395":{"cik_str":1808158,"ticker":"RPTX","title":"Repare Therapeutics Inc."},"3396":{"cik_str":1321741,"ticker":"GAIN","title":"GLADSTONE INVESTMENT CORPORATION\\DE"},"3397":{"cik_str":1508475,"ticker":"VNET","title":"VNET Group, Inc."},"3398":{"cik_str":1020214,"ticker":"CERS","title":"CERUS CORP"},"3399":{"cik_str":791963,"ticker":"OPY","title":"OPPENHEIMER HOLDINGS INC"},"3400":{"cik_str":925528,"ticker":"HDSN","title":"HUDSON TECHNOLOGIES INC /NY"},"3401":{"cik_str":712770,"ticker":"OLP","title":"ONE LIBERTY PROPERTIES INC"},"3402":{"cik_str":844965,"ticker":"TTI","title":"TETRA TECHNOLOGIES INC"},"3403":{"cik_str":1661181,"ticker":"ORGO","title":"Organogenesis Holdings Inc."},"3404":{"cik_str":1833197,"ticker":"SWIM","title":"Latham Group, Inc."},"3405":{"cik_str":835948,"ticker":"MVF","title":"BLACKROCK MUNIVEST FUND, INC."},"3406":{"cik_str":1730463,"ticker":"AUTL","title":"Autolus Therapeutics plc"},"3407":{"cik_str":1649989,"ticker":"OTLK","title":"Outlook Therapeutics, Inc."},"3408":{"cik_str":1847891,"ticker":"ACRO","title":"Acropolis Infrastructure Acquisition Corp."},"3409":{"cik_str":924717,"ticker":"SRDX","title":"SURMODICS INC"},"3410":{"cik_str":1885461,"ticker":"SUAC","title":"ShoulderUP Technology Acquisition Corp."},"3411":{"cik_str":701347,"ticker":"CPF","title":"CENTRAL PACIFIC FINANCIAL CORP"},"3412":{"cik_str":717720,"ticker":"VALU","title":"VALUE LINE INC"},"3413":{"cik_str":1259708,"ticker":"FRA","title":"BLACKROCK FLOATING RATE INCOME STRATEGIES FUND, INC."},"3414":{"cik_str":932695,"ticker":"CYD","title":"CHINA YUCHAI INTERNATIONAL LTD"},"3415":{"cik_str":711772,"ticker":"CATC","title":"CAMBRIDGE BANCORP"},"3416":{"cik_str":1699880,"ticker":"AMLI","title":"American Lithium Corp."},"3417":{"cik_str":886043,"ticker":"MMU","title":"WESTERN ASSET MANAGED MUNICIPALS FUND INC."},"3418":{"cik_str":1365916,"ticker":"AMRS","title":"AMYRIS, INC."},"3419":{"cik_str":1780652,"ticker":"CAN","title":"Canaan Inc."},"3420":{"cik_str":1518621,"ticker":"ORC","title":"Orchid Island Capital, Inc."},"3421":{"cik_str":1412095,"ticker":"TROO","title":"Troops, Inc. /Cayman Islands/"},"3422":{"cik_str":1635282,"ticker":"RMNI","title":"Rimini Street, Inc."},"3423":{"cik_str":828803,"ticker":"GIM","title":"TEMPLETON GLOBAL INCOME FUND"},"3424":{"cik_str":1867102,"ticker":"EVTL","title":"Vertical Aerospace Ltd."},"3425":{"cik_str":1083301,"ticker":"WULF","title":"TERAWULF INC."},"3426":{"cik_str":1574197,"ticker":"FPH","title":"Five Point Holdings, LLC"},"3427":{"cik_str":1794338,"ticker":"IGIC","title":"International General Insurance Holdings Ltd."},"3428":{"cik_str":1837412,"ticker":"SLGC","title":"SomaLogic, Inc."},"3429":{"cik_str":1795579,"ticker":"CALT","title":"Calliditas Therapeutics AB"},"3430":{"cik_str":1838162,"ticker":"SLAM","title":"Slam Corp."},"3431":{"cik_str":1827821,"ticker":"FRGE","title":"Forge Global Holdings, Inc."},"3432":{"cik_str":74046,"ticker":"ODC","title":"Oil-Dri Corp of America"},"3433":{"cik_str":1580345,"ticker":"TPVG","title":"TriplePoint Venture Growth BDC Corp."},"3434":{"cik_str":870780,"ticker":"NQP","title":"NUVEEN PENNSYLVANIA QUALITY MUNICIPAL INCOME FUND"},"3435":{"cik_str":1801170,"ticker":"CLOV","title":"CLOVER HEALTH INVESTMENTS, CORP. /DE"},"3436":{"cik_str":1738177,"ticker":"CMBM","title":"Cambium Networks Corp"},"3437":{"cik_str":1720424,"ticker":"HIVE","title":"HIVE Blockchain Technologies Ltd."},"3438":{"cik_str":878518,"ticker":"TGB","title":"TASEKO MINES LTD"},"3439":{"cik_str":1789972,"ticker":"CGEM","title":"Cullinan Oncology, Inc."},"3440":{"cik_str":912147,"ticker":"RMT","title":"ROYCE MICRO-CAP TRUST, INC."},"3441":{"cik_str":1895597,"ticker":"ICG","title":"Intchains Group Ltd"},"3442":{"cik_str":1122904,"ticker":"NTGR","title":"NETGEAR, INC."},"3443":{"cik_str":1762303,"ticker":"RCEL","title":"AVITA Medical, Inc."},"3444":{"cik_str":1760717,"ticker":"SCPL","title":"SciPlay Corp"},"3445":{"cik_str":1290677,"ticker":"TPB","title":"Turning Point Brands, Inc."},"3446":{"cik_str":1841873,"ticker":"KVSA","title":"Khosla Ventures Acquisition Co."},"3447":{"cik_str":1512762,"ticker":"CHRS","title":"Coherus BioSciences, Inc."},"3448":{"cik_str":1531177,"ticker":"SGHT","title":"Sight Sciences, Inc."},"3449":{"cik_str":1761918,"ticker":"ERAS","title":"Erasca, Inc."},"3450":{"cik_str":1285890,"ticker":"IGD","title":"Voya GLOBAL EQUITY DIVIDEND & PREMIUM OPPORTUNITY FUND"},"3451":{"cik_str":1823593,"ticker":"TSP","title":"TuSimple Holdings Inc."},"3452":{"cik_str":1769617,"ticker":"HONE","title":"HarborOne Bancorp, Inc."},"3453":{"cik_str":1819974,"ticker":"SKYT","title":"SkyWater Technology, Inc"},"3454":{"cik_str":1534880,"ticker":"ISD","title":"PGIM High Yield Bond Fund, Inc."},"3455":{"cik_str":1570562,"ticker":"EOLS","title":"Evolus, Inc."},"3456":{"cik_str":1852940,"ticker":"GOGN","title":"GoGreen Investments Corp"},"3457":{"cik_str":894627,"ticker":"EGY","title":"VAALCO ENERGY INC /DE/"},"3458":{"cik_str":1219120,"ticker":"AVK","title":"ADVENT CONVERTIBLE & INCOME FUND"},"3459":{"cik_str":1825079,"ticker":"VLD","title":"Velo3D, Inc."},"3460":{"cik_str":1308927,"ticker":"ETB","title":"Eaton Vance Tax-Managed Buy-Write Income Fund"},"3461":{"cik_str":1957413,"ticker":"CJET","title":"Chijet Motor Company, Inc."},"3462":{"cik_str":1750264,"ticker":"PUYI","title":"PUYI, INC."},"3463":{"cik_str":1824814,"ticker":"CGNT","title":"Cognyte Software Ltd."},"3464":{"cik_str":1431852,"ticker":"ODV","title":"Osisko Development Corp."},"3465":{"cik_str":1275187,"ticker":"ANGO","title":"ANGIODYNAMICS INC"},"3466":{"cik_str":1745020,"ticker":"THRX","title":"Theseus Pharmaceuticals, Inc."},"3467":{"cik_str":924822,"ticker":"MLR","title":"MILLER INDUSTRIES INC /TN/"},"3468":{"cik_str":1176199,"ticker":"HPI","title":"JOHN HANCOCK PREFERRED INCOME FUND"},"3469":{"cik_str":1383058,"ticker":"DBO","title":"Invesco DB Oil Fund"},"3470":{"cik_str":1835654,"ticker":"INVZ","title":"Innoviz Technologies Ltd."},"3471":{"cik_str":1860879,"ticker":"RRAC","title":"Rigel Resource Acquisition Corp."},"3472":{"cik_str":882150,"ticker":"MYN","title":"BLACKROCK MUNIYIELD NEW YORK QUALITY FUND, INC."},"3473":{"cik_str":1727196,"ticker":"SRRK","title":"Scholar Rock Holding Corp"},"3474":{"cik_str":1074540,"ticker":"EVN","title":"EATON VANCE MUNICIPAL INCOME TRUST"},"3475":{"cik_str":1735438,"ticker":"MGTX","title":"MeiraGTx Holdings plc"},"3476":{"cik_str":783412,"ticker":"DJCO","title":"DAILY JOURNAL CORP"},"3477":{"cik_str":1496323,"ticker":"IGMS","title":"IGM Biosciences, Inc."},"3478":{"cik_str":1812477,"ticker":"BITF","title":"Bitfarms Ltd"},"3479":{"cik_str":1701758,"ticker":"LOVE","title":"Lovesac Co"},"3480":{"cik_str":714562,"ticker":"THFF","title":"FIRST FINANCIAL CORP /IN/"},"3481":{"cik_str":1476034,"ticker":"MCB","title":"Metropolitan Bank Holding Corp."},"3482":{"cik_str":1970509,"ticker":"HYAC","title":"Haymaker Acquisition Corp. 4"},"3483":{"cik_str":1094324,"ticker":"SIFY","title":"SIFY TECHNOLOGIES LTD"},"3484":{"cik_str":23795,"ticker":"CTO","title":"CTO Realty Growth, Inc."},"3485":{"cik_str":1738021,"ticker":"CMPX","title":"Compass Therapeutics, Inc."},"3486":{"cik_str":866028,"ticker":"PLM","title":"POLYMET MINING CORP"},"3487":{"cik_str":1337298,"ticker":"FF","title":"FutureFuel Corp."},"3488":{"cik_str":1883814,"ticker":"SLND","title":"Southland Holdings, Inc."},"3489":{"cik_str":1318885,"ticker":"DSX","title":"DIANA SHIPPING INC."},"3490":{"cik_str":1644378,"ticker":"RMR","title":"RMR GROUP INC."},"3491":{"cik_str":1163668,"ticker":"SPFI","title":"SOUTH PLAINS FINANCIAL, INC."},"3492":{"cik_str":922357,"ticker":"IOCJY","title":"IOCHPE-MAXION SA /FI"},"3493":{"cik_str":1487428,"ticker":"HRZN","title":"Horizon Technology Finance Corp"},"3494":{"cik_str":1587987,"ticker":"NEWT","title":"NewtekOne, Inc."},"3495":{"cik_str":1562051,"ticker":"NML","title":"Neuberger Berman MLP & Energy Income Fund Inc."},"3496":{"cik_str":1856589,"ticker":"MBSC","title":"M3-Brigade Acquisition III Corp."},"3497":{"cik_str":1096056,"ticker":"LXFR","title":"LUXFER HOLDINGS PLC"},"3498":{"cik_str":1861115,"ticker":"NMAI","title":"Nuveen Multi-Asset Income Fund"},"3499":{"cik_str":1721386,"ticker":"LSEA","title":"Landsea Homes Corp"},"3500":{"cik_str":1828105,"ticker":"HIPO","title":"Hippo Holdings Inc."},"3501":{"cik_str":1442836,"ticker":"MRSN","title":"Mersana Therapeutics, Inc."},"3502":{"cik_str":1720420,"ticker":"IBEX","title":"IBEX Ltd"},"3503":{"cik_str":1712463,"ticker":"PACK","title":"Ranpak Holdings Corp."},"3504":{"cik_str":1873923,"ticker":"ONL","title":"Orion Office REIT Inc."},"3505":{"cik_str":1171326,"ticker":"EUBG","title":"ENTREPRENEUR UNIVERSE BRIGHT GROUP"},"3506":{"cik_str":1889109,"ticker":"BLTE","title":"BELITE BIO, INC"},"3507":{"cik_str":818972,"ticker":"LEO","title":"BNY MELLON STRATEGIC MUNICIPALS, INC."},"3508":{"cik_str":1383414,"ticker":"PNNT","title":"PENNANTPARK INVESTMENT CORP"},"3509":{"cik_str":1624322,"ticker":"BFST","title":"Business First Bancshares, Inc."},"3510":{"cik_str":1393434,"ticker":"OCUL","title":"OCULAR THERAPEUTIX, INC"},"3511":{"cik_str":1816590,"ticker":"CMPS","title":"COMPASS Pathways plc"},"3512":{"cik_str":1615063,"ticker":"INSE","title":"Inspired Entertainment, Inc."},"3513":{"cik_str":1396277,"ticker":"CHW","title":"Calamos Global Dynamic Income Fund"},"3514":{"cik_str":1811063,"ticker":"NUVB","title":"Nuvation Bio Inc."},"3515":{"cik_str":1697500,"ticker":"SOI","title":"Solaris Oilfield Infrastructure, Inc."},"3516":{"cik_str":1708259,"ticker":"LX","title":"LexinFintech Holdings Ltd."},"3517":{"cik_str":1547994,"ticker":"JPI","title":"Nuveen Preferred & Income Term Fund"},"3518":{"cik_str":898437,"ticker":"ANIK","title":"Anika Therapeutics, Inc."},"3519":{"cik_str":1517518,"ticker":"EMO","title":"ClearBridge Energy Midstream Opportunity Fund Inc."},"3520":{"cik_str":1210123,"ticker":"EAD","title":"ALLSPRING INCOME OPPORTUNITIES FUND"},"3521":{"cik_str":1692376,"ticker":"VEL","title":"Velocity Financial, Inc."},"3522":{"cik_str":1429764,"ticker":"BLNK","title":"Blink Charging Co."},"3523":{"cik_str":1377789,"ticker":"AVNW","title":"AVIAT NETWORKS, INC."},"3524":{"cik_str":1866757,"ticker":"BRLT","title":"Brilliant Earth Group, Inc."},"3525":{"cik_str":1856242,"ticker":"CXAC","title":"C5 Acquisition Corp"},"3526":{"cik_str":1824403,"ticker":"RSVR","title":"Reservoir Media, Inc."},"3527":{"cik_str":1456772,"ticker":"OPI","title":"OFFICE PROPERTIES INCOME TRUST"},"3528":{"cik_str":1275168,"ticker":"FSBC","title":"FIVE STAR BANCORP"},"3529":{"cik_str":1367644,"ticker":"EBS","title":"Emergent BioSolutions Inc."},"3530":{"cik_str":1515671,"ticker":"DPG","title":"Duff & Phelps Utility & Infrastructure Fund Inc."},"3531":{"cik_str":14846,"ticker":"BRT","title":"BRT Apartments Corp."},"3532":{"cik_str":879635,"ticker":"MPB","title":"MID PENN BANCORP INC"},"3533":{"cik_str":1616543,"ticker":"SENS","title":"Senseonics Holdings, Inc."},"3534":{"cik_str":1729149,"ticker":"VMD","title":"VIEMED HEALTHCARE, INC."},"3535":{"cik_str":928340,"ticker":"CWCO","title":"Consolidated Water Co. Ltd."},"3536":{"cik_str":1687187,"ticker":"METC","title":"Ramaco Resources, Inc."},"3537":{"cik_str":805928,"ticker":"AXGN","title":"Axogen, Inc."},"3538":{"cik_str":1935172,"ticker":"AIXI","title":"Xiao-I Corp"},"3539":{"cik_str":1115128,"ticker":"QUOT","title":"Quotient Technology Inc."},"3540":{"cik_str":1710350,"ticker":"BTBT","title":"Bit Digital, Inc"},"3541":{"cik_str":31235,"ticker":"KODK","title":"EASTMAN KODAK CO"},"3542":{"cik_str":1110611,"ticker":"ONTF","title":"ON24 INC."},"3543":{"cik_str":1756708,"ticker":"JMIA","title":"Jumia Technologies AG"},"3544":{"cik_str":1782430,"ticker":"STRW","title":"Strawberry Fields REIT, Inc."},"3545":{"cik_str":1839799,"ticker":"GAMB","title":"Gambling.com Group Ltd"},"3546":{"cik_str":1494530,"ticker":"FSD","title":"FIRST TRUST HIGH INCOME LONG/SHORT FUND"},"3547":{"cik_str":1327688,"ticker":"OOMA","title":"OOMA INC"},"3548":{"cik_str":1564824,"ticker":"ALLK","title":"Allakos Inc."},"3549":{"cik_str":1531031,"ticker":"ESQ","title":"Esquire Financial Holdings, Inc."},"3550":{"cik_str":1447028,"ticker":"ABUS","title":"Arbutus Biopharma Corp"},"3551":{"cik_str":908993,"ticker":"VKI","title":"Invesco Advantage Municipal Income Trust II"},"3552":{"cik_str":1865389,"ticker":"NPFD","title":"Nuveen Variable Rate Preferred & Income Fund"},"3553":{"cik_str":1434754,"ticker":"SB","title":"SAFE BULKERS, INC."},"3554":{"cik_str":1845550,"ticker":"APTM","title":"Alpha Partners Technology Merger Corp."},"3555":{"cik_str":1281926,"ticker":"ETO","title":"Eaton Vance Tax-Advantaged Global Dividend Opportunities Fund"},"3556":{"cik_str":743367,"ticker":"BHB","title":"BAR HARBOR BANKSHARES"},"3557":{"cik_str":1290900,"ticker":"CVGI","title":"Commercial Vehicle Group, Inc."},"3558":{"cik_str":923139,"ticker":"FFIC","title":"FLUSHING FINANCIAL CORP"},"3559":{"cik_str":1847874,"ticker":"FWAC","title":"Fifth Wall Acquisition Corp. III"},"3560":{"cik_str":1734107,"ticker":"SOHU","title":"Sohu.com Ltd"},"3561":{"cik_str":736772,"ticker":"CCNE","title":"CNB FINANCIAL CORP/PA"},"3562":{"cik_str":1855474,"ticker":"XPDB","title":"Power & Digital Infrastructure Acquisition II Corp."},"3563":{"cik_str":1059142,"ticker":"GHI","title":"Greystone Housing Impact Investors LP"},"3564":{"cik_str":77543,"ticker":"TPC","title":"TUTOR PERINI CORP"},"3565":{"cik_str":1759774,"ticker":"PSTL","title":"Postal Realty Trust, Inc."},"3566":{"cik_str":1143513,"ticker":"GLAD","title":"GLADSTONE CAPITAL CORP"},"3567":{"cik_str":1116463,"ticker":"OSUR","title":"ORASURE TECHNOLOGIES INC"},"3568":{"cik_str":903419,"ticker":"ALRS","title":"ALERUS FINANCIAL CORP"},"3569":{"cik_str":1766600,"ticker":"SNDL","title":"SNDL Inc."},"3570":{"cik_str":1347178,"ticker":"VNDA","title":"Vanda Pharmaceuticals Inc."},"3571":{"cik_str":1882198,"ticker":"ATEK","title":"Athena Technology Acquisition Corp. II"},"3572":{"cik_str":1038773,"ticker":"SMBK","title":"SMARTFINANCIAL INC."},"3573":{"cik_str":1369085,"ticker":"NEWP","title":"NEW PACIFIC METALS CORP"},"3574":{"cik_str":1495825,"ticker":"GBAB","title":"Guggenheim Taxable Municipal Bond & Investment Grade Debt Trust"},"3575":{"cik_str":39311,"ticker":"IBCP","title":"INDEPENDENT BANK CORP /MI/"},"3576":{"cik_str":1754068,"ticker":"ALVR","title":"Allovir, Inc."},"3577":{"cik_str":1758021,"ticker":"KRT","title":"Karat Packaging Inc."},"3578":{"cik_str":1812923,"ticker":"SDHY","title":"PGIM Short Duration High Yield Opportunities Fund"},"3579":{"cik_str":1392380,"ticker":"GEVO","title":"Gevo, Inc."},"3580":{"cik_str":59255,"ticker":"VHI","title":"VALHI INC /DE/"},"3581":{"cik_str":1169445,"ticker":"CPSI","title":"COMPUTER PROGRAMS & SYSTEMS INC"},"3582":{"cik_str":763532,"ticker":"LYTS","title":"LSI INDUSTRIES INC"},"3583":{"cik_str":1854458,"ticker":"NETC","title":"Nabors Energy Transition Corp."},"3584":{"cik_str":1766400,"ticker":"PNTG","title":"Pennant Group, Inc."},"3585":{"cik_str":1865697,"ticker":"GGAA","title":"Genesis Growth Tech Acquisition Corp."},"3586":{"cik_str":1858848,"ticker":"TNYA","title":"Tenaya Therapeutics, Inc."},"3587":{"cik_str":1538263,"ticker":"HTBI","title":"HomeTrust Bancshares, Inc."},"3588":{"cik_str":1385613,"ticker":"GLRE","title":"GREENLIGHT CAPITAL RE, LTD."},"3589":{"cik_str":1939780,"ticker":"UCAR","title":"U Power Ltd"},"3590":{"cik_str":1828161,"ticker":"FTCI","title":"FTC Solar, Inc."},"3591":{"cik_str":1528356,"ticker":"GNE","title":"Genie Energy Ltd."},"3592":{"cik_str":1828536,"ticker":"NRGV","title":"Energy Vault Holdings, Inc."},"3593":{"cik_str":1862327,"ticker":"NFNT","title":"Infinite Acquisition Corp."},"3594":{"cik_str":1853070,"ticker":"PACI","title":"PROOF Acquisition Corp I"},"3595":{"cik_str":1865407,"ticker":"KCGI","title":"Kensington Capital Acquisition Corp. V"},"3596":{"cik_str":1838337,"ticker":"APGB","title":"Apollo Strategic Growth Capital II"},"3597":{"cik_str":1468748,"ticker":"KOD","title":"Kodiak Sciences Inc."},"3598":{"cik_str":1010086,"ticker":"SIGA","title":"SIGA TECHNOLOGIES INC"},"3599":{"cik_str":1138723,"ticker":"ARAY","title":"ACCURAY INC"},"3600":{"cik_str":1447362,"ticker":"CSTL","title":"CASTLE BIOSCIENCES INC"},"3601":{"cik_str":1102993,"ticker":"LPSN","title":"LIVEPERSON INC"},"3602":{"cik_str":1160308,"ticker":"SVRA","title":"Savara Inc"},"3603":{"cik_str":1807120,"ticker":"DSGN","title":"Design Therapeutics, Inc."},"3604":{"cik_str":910068,"ticker":"HIO","title":"WESTERN ASSET HIGH INCOME OPPORTUNITY FUND INC."},"3605":{"cik_str":1137391,"ticker":"BFZ","title":"BLACKROCK CALIFORNIA MUNICIPAL INCOME TRUST"},"3606":{"cik_str":1559991,"ticker":"DFP","title":"Flaherty & Crumrine Dynamic Preferred & Income Fund Inc"},"3607":{"cik_str":1694665,"ticker":"EVLO","title":"Evelo Biosciences, Inc."},"3608":{"cik_str":1861121,"ticker":"EVE","title":"EVe Mobility Acquisition Corp"},"3609":{"cik_str":1829576,"ticker":"CARE","title":"Carter Bankshares, Inc."},"3610":{"cik_str":1851959,"ticker":"CNDA","title":"Concord Acquisition Corp II"},"3611":{"cik_str":1870600,"ticker":"DTC","title":"Solo Brands, Inc."},"3612":{"cik_str":1784440,"ticker":"NOTR","title":"Nowtransit Inc"},"3613":{"cik_str":34067,"ticker":"BOOM","title":"DMC Global Inc."},"3614":{"cik_str":713425,"ticker":"AMSWA","title":"AMERICAN SOFTWARE INC"},"3615":{"cik_str":1899883,"ticker":"FIP","title":"FTAI Infrastructure Inc."},"3616":{"cik_str":1071236,"ticker":"RRBI","title":"RED RIVER BANCSHARES INC"},"3617":{"cik_str":1227500,"ticker":"EQBK","title":"EQUITY BANCSHARES INC"},"3618":{"cik_str":1503290,"ticker":"ACP","title":"abrdn Income Credit Strategies Fund"},"3619":{"cik_str":1750284,"ticker":"OLMA","title":"Olema Pharmaceuticals, Inc."},"3620":{"cik_str":1889123,"ticker":"EMLD","title":"FTAC Emerald Acquisition Corp."},"3621":{"cik_str":1018399,"ticker":"EBTC","title":"ENTERPRISE BANCORP INC /MA/"},"3622":{"cik_str":1561566,"ticker":"QIWI","title":"QIWI"},"3623":{"cik_str":1293971,"ticker":"BLUE","title":"bluebird bio, Inc."},"3624":{"cik_str":1786255,"ticker":"ICVX","title":"Icosavax, Inc."},"3625":{"cik_str":1291334,"ticker":"FFA","title":"FIRST TRUST ENHANCED EQUITY INCOME FUND"},"3626":{"cik_str":1827087,"ticker":"VIGL","title":"Vigil Neuroscience, Inc."},"3627":{"cik_str":1843370,"ticker":"BLEU","title":"bleuacacia ltd"},"3628":{"cik_str":739421,"ticker":"CZFS","title":"CITIZENS FINANCIAL SERVICES INC"},"3629":{"cik_str":1613780,"ticker":"DBVT","title":"DBV Technologies S.A."},"3630":{"cik_str":884121,"ticker":"HQL","title":"TEKLA LIFE SCIENCES INVESTORS"},"3631":{"cik_str":897322,"ticker":"GILT","title":"GILAT SATELLITE NETWORKS LTD"},"3632":{"cik_str":1140102,"ticker":"HQI","title":"HireQuest, Inc."},"3633":{"cik_str":1604950,"ticker":"SCPH","title":"scPharmaceuticals Inc."},"3634":{"cik_str":1514416,"ticker":"BAND","title":"Bandwidth Inc."},"3635":{"cik_str":1786248,"ticker":"NREF","title":"NexPoint Real Estate Finance, Inc."},"3636":{"cik_str":1631574,"ticker":"WVE","title":"Wave Life Sciences Ltd."},"3637":{"cik_str":1817159,"ticker":"RFMZ","title":"RiverNorth Flexible Municipal Income Fund II, Inc."},"3638":{"cik_str":1867096,"ticker":"XERS","title":"Xeris Biopharma Holdings, Inc."},"3639":{"cik_str":1556336,"ticker":"FEI","title":"FIRST TRUST MLP & ENERGY INCOME FUND"},"3640":{"cik_str":1102238,"ticker":"ARL","title":"AMERICAN REALTY INVESTORS INC"},"3641":{"cik_str":1258623,"ticker":"EFR","title":"EATON VANCE SENIOR FLOATING RATE TRUST"},"3642":{"cik_str":900422,"ticker":"PMO","title":"PUTNAM MUNICIPAL OPPORTUNITIES TRUST"},"3643":{"cik_str":1500881,"ticker":"EU","title":"enCore Energy Corp."},"3644":{"cik_str":319655,"ticker":"SJT","title":"SAN JUAN BASIN ROYALTY TRUST"},"3645":{"cik_str":1852889,"ticker":"IVCA","title":"Investcorp India Acquisition Corp"},"3646":{"cik_str":1737927,"ticker":"CGC","title":"Canopy Growth Corp"},"3647":{"cik_str":827773,"ticker":"PPT","title":"PUTNAM PREMIER INCOME TRUST"},"3648":{"cik_str":1796280,"ticker":"ORIC","title":"Oric Pharmaceuticals, Inc."},"3649":{"cik_str":1855644,"ticker":"ZURA","title":"Zura Bio Ltd"},"3650":{"cik_str":1574085,"ticker":"BHR","title":"Braemar Hotels & Resorts Inc."},"3651":{"cik_str":1794783,"ticker":"SLQT","title":"SelectQuote, Inc."},"3652":{"cik_str":1041934,"ticker":"EDAP","title":"EDAP TMS SA"},"3653":{"cik_str":846475,"ticker":"ZYXI","title":"ZYNEX INC"},"3654":{"cik_str":1384195,"ticker":"REI","title":"RING ENERGY, INC."},"3655":{"cik_str":1862461,"ticker":"REAX","title":"Real Brokerage Inc"},"3656":{"cik_str":1762417,"ticker":"DOYU","title":"DouYu International Holdings Ltd"},"3657":{"cik_str":1847241,"ticker":"PEGR","title":"Project Energy Reimagined Acquisition Corp."},"3658":{"cik_str":1288992,"ticker":"EFT","title":"Eaton Vance Floating-Rate Income Trust"},"3659":{"cik_str":1836981,"ticker":"BBAI","title":"BigBear.ai Holdings, Inc."},"3660":{"cik_str":1318008,"ticker":"ZUMZ","title":"Zumiez Inc"},"3661":{"cik_str":839533,"ticker":"KTF","title":"DWS MUNICIPAL INCOME TRUST"},"3662":{"cik_str":1390195,"ticker":"AWP","title":"abrdn Global Premier Properties Fund"},"3663":{"cik_str":1824734,"ticker":"BGRY","title":"Berkshire Grey, Inc."},"3664":{"cik_str":314203,"ticker":"MUX","title":"McEwen Mining Inc."},"3665":{"cik_str":1306550,"ticker":"BGR","title":"BlackRock Energy & Resources Trust"},"3666":{"cik_str":103595,"ticker":"VLGEA","title":"VILLAGE SUPER MARKET INC"},"3667":{"cik_str":1876714,"ticker":"GFGD","title":"Growth for Good Acquisition Corp"},"3668":{"cik_str":1485003,"ticker":"CARM","title":"Carisma Therapeutics Inc."},"3669":{"cik_str":1412665,"ticker":"MOFG","title":"MidWestOne Financial Group, Inc."},"3670":{"cik_str":706698,"ticker":"UTMD","title":"UTAH MEDICAL PRODUCTS INC"},"3671":{"cik_str":1500096,"ticker":"KMF","title":"Kayne Anderson NextGen Energy & Infrastructure, Inc."},"3672":{"cik_str":1853580,"ticker":"PFTA","title":"PORTAGE FINTECH ACQUISITION CORP."},"3673":{"cik_str":717538,"ticker":"AROW","title":"ARROW FINANCIAL CORP"},"3674":{"cik_str":1831907,"ticker":"MYTE","title":"MYT Netherlands Parent B.V."},"3675":{"cik_str":714256,"ticker":"SMTI","title":"Sanara MedTech Inc."},"3676":{"cik_str":1268533,"ticker":"TYG","title":"TORTOISE ENERGY INFRASTRUCTURE CORP"},"3677":{"cik_str":1733294,"ticker":"GBIO","title":"Generation Bio Co."},"3678":{"cik_str":1839341,"ticker":"CORZQ","title":"Core Scientific, Inc./tx"},"3679":{"cik_str":18498,"ticker":"GCO","title":"GENESCO INC"},"3680":{"cik_str":913277,"ticker":"CLAR","title":"Clarus Corp"},"3681":{"cik_str":1074769,"ticker":"NAN","title":"NUVEEN NEW YORK QUALITY MUNICIPAL INCOME FUND"},"3682":{"cik_str":1539337,"ticker":"JRI","title":"Nuveen Real Asset Income & Growth Fund"},"3683":{"cik_str":1125345,"ticker":"MGNX","title":"MACROGENICS INC"},"3684":{"cik_str":1813914,"ticker":"CMAX","title":"CareMax, Inc."},"3685":{"cik_str":1642453,"ticker":"DSKE","title":"Daseke, Inc."},"3686":{"cik_str":1579684,"ticker":"GCI","title":"Gannett Co., Inc."},"3687":{"cik_str":826020,"ticker":"BRW","title":"Saba Capital Income & Opportunities Fund"},"3688":{"cik_str":890393,"ticker":"MIY","title":"BLACKROCK MUNIYIELD MICHIGAN QUALITY FUND, INC."},"3689":{"cik_str":1859795,"ticker":"NVX","title":"NOVONIX Ltd"},"3690":{"cik_str":1285819,"ticker":"OMER","title":"OMEROS CORP"},"3691":{"cik_str":1883984,"ticker":"CLIN","title":"Clean Earth Acquisitions Corp."},"3692":{"cik_str":1828608,"ticker":"CPUH","title":"Compute Health Acquisition Corp."},"3693":{"cik_str":1463972,"ticker":"VUZI","title":"Vuzix Corp"},"3694":{"cik_str":1711933,"ticker":"AKYA","title":"Akoya Biosciences, Inc."},"3695":{"cik_str":1759631,"ticker":"HYLN","title":"Hyliion Holdings Corp."},"3696":{"cik_str":1189740,"ticker":"HPF","title":"JOHN HANCOCK PREFERRED INCOME FUND II"},"3697":{"cik_str":1058867,"ticker":"GNTY","title":"GUARANTY BANCSHARES INC /TX/"},"3698":{"cik_str":1865631,"ticker":"NN","title":"NEXTNAV INC."},"3699":{"cik_str":1802883,"ticker":"API","title":"Agora, Inc."},"3700":{"cik_str":1842644,"ticker":"SLVR","title":"SILVERspac Inc."},"3701":{"cik_str":1857518,"ticker":"JGGC","title":"Jaguar Global Growth Corp I"},"3702":{"cik_str":1606909,"ticker":"PANL","title":"Pangaea Logistics Solutions Ltd."},"3703":{"cik_str":1843477,"ticker":"SVII","title":"Spring Valley Acquisition Corp. II"},"3704":{"cik_str":1865975,"ticker":"AFTR","title":"AfterNext HealthTech Acquisition Corp."},"3705":{"cik_str":1038186,"ticker":"MHN","title":"BLACKROCK MUNIHOLDINGS NEW YORK QUALITY FUND, INC."},"3706":{"cik_str":1606366,"ticker":"LOCO","title":"El Pollo Loco Holdings, Inc."},"3707":{"cik_str":1095146,"ticker":"ABIT","title":"Athena Bitcoin Global"},"3708":{"cik_str":91847,"ticker":"SOR","title":"SOURCE CAPITAL INC /DE/"},"3709":{"cik_str":75208,"ticker":"OSG","title":"OVERSEAS SHIPHOLDING GROUP INC"},"3710":{"cik_str":1171155,"ticker":"RLGT","title":"RADIANT LOGISTICS, INC"},"3711":{"cik_str":1874944,"ticker":"VCSA","title":"Vacasa, Inc."},"3712":{"cik_str":1377936,"ticker":"SAR","title":"SARATOGA INVESTMENT CORP."},"3713":{"cik_str":1744781,"ticker":"NIU","title":"Niu Technologies"},"3714":{"cik_str":1845013,"ticker":"HWEL","title":"Healthwell Acquisition Corp. I"},"3715":{"cik_str":1067837,"ticker":"AUDA","title":"AUDACY, INC."},"3716":{"cik_str":1569994,"ticker":"WSBF","title":"Waterstone Financial, Inc."},"3717":{"cik_str":1830197,"ticker":"HMPT","title":"Home Point Capital Inc."},"3718":{"cik_str":733590,"ticker":"TCI","title":"TRANSCONTINENTAL REALTY INVESTORS INC"},"3719":{"cik_str":1235912,"ticker":"CVRX","title":"CVRx, Inc."},"3720":{"cik_str":1053584,"ticker":"MCBC","title":"MACATAWA BANK CORP"},"3721":{"cik_str":1805890,"ticker":"FUSN","title":"Fusion Pharmaceuticals Inc."},"3722":{"cik_str":1826457,"ticker":"GLUE","title":"Monte Rosa Therapeutics, Inc."},"3723":{"cik_str":813298,"ticker":"DXLG","title":"DESTINATION XL GROUP, INC."},"3724":{"cik_str":909531,"ticker":"CRNCY","title":"CAIRN ENERGY PLC"},"3725":{"cik_str":1289213,"ticker":"JRO","title":"Nuveen Floating Rate Income Opportunity Fund"},"3726":{"cik_str":1879248,"ticker":"ADSE","title":"Ads-Tec Energy Public Ltd Co"},"3727":{"cik_str":1851657,"ticker":"VAXX","title":"Vaxxinity, Inc."},"3728":{"cik_str":1058239,"ticker":"HIX","title":"WESTERN ASSET HIGH INCOME FUND II INC."},"3729":{"cik_str":1113809,"ticker":"BBW","title":"BUILD-A-BEAR WORKSHOP INC"},"3730":{"cik_str":1625101,"ticker":"PLSE","title":"Pulse Biosciences, Inc."},"3731":{"cik_str":69733,"ticker":"NATH","title":"NATHANS FAMOUS, INC."},"3732":{"cik_str":1676580,"ticker":"VENG","title":"VISION ENERGY Corp"},"3733":{"cik_str":786035,"ticker":"ASG","title":"LIBERTY ALL STAR GROWTH FUND INC."},"3734":{"cik_str":1681206,"ticker":"NODK","title":"NI Holdings, Inc."},"3735":{"cik_str":1214935,"ticker":"NCV","title":"Virtus Convertible & Income Fund"},"3736":{"cik_str":826735,"ticker":"MIN","title":"MFS INTERMEDIATE INCOME TRUST"},"3737":{"cik_str":1690334,"ticker":"SMHI","title":"SEACOR Marine Holdings Inc."},"3738":{"cik_str":1162027,"ticker":"AFB","title":"ALLIANCEBERNSTEIN NATIONAL MUNICIPAL INCOME FUND"},"3739":{"cik_str":792966,"ticker":"FMAO","title":"FARMERS & MERCHANTS BANCORP INC"},"3740":{"cik_str":1166928,"ticker":"WTBA","title":"WEST BANCORPORATION INC"},"3741":{"cik_str":1840904,"ticker":"ATAI","title":"ATAI Life Sciences N.V."},"3742":{"cik_str":1808665,"ticker":"ASRT","title":"Assertio Holdings, Inc."},"3743":{"cik_str":919893,"ticker":"TDF","title":"TEMPLETON DRAGON FUND INC"},"3744":{"cik_str":1388141,"ticker":"EDD","title":"Morgan Stanley Emerging Markets Domestic Debt Fund, Inc."},"3745":{"cik_str":1807046,"ticker":"OZ","title":"Belpointe PREP, LLC"},"3746":{"cik_str":1878848,"ticker":"IREN","title":"Iris Energy Ltd"},"3747":{"cik_str":909494,"ticker":"TCX","title":"TUCOWS INC /PA/"},"3748":{"cik_str":1459417,"ticker":"TWOU","title":"2U, Inc."},"3749":{"cik_str":1700849,"ticker":"VADP","title":"Vado Corp."},"3750":{"cik_str":1852061,"ticker":"GAQ","title":"Generation Asia I Acquisition Ltd"},"3751":{"cik_str":741516,"ticker":"AMNB","title":"AMERICAN NATIONAL BANKSHARES INC."},"3752":{"cik_str":1348911,"ticker":"KALV","title":"KalVista Pharmaceuticals, Inc."},"3753":{"cik_str":1849737,"ticker":"PLAO","title":"Patria Latin American Opportunity Acquisition Corp."},"3754":{"cik_str":1717457,"ticker":"CPZ","title":"Calamos Long/Short Equity & Dynamic Income Trust"},"3755":{"cik_str":788965,"ticker":"HNRG","title":"HALLADOR ENERGY CO"},"3756":{"cik_str":1056943,"ticker":"PFIS","title":"PEOPLES FINANCIAL SERVICES CORP."},"3757":{"cik_str":1842659,"ticker":"CORS","title":"Corsair Partnering Corp"},"3758":{"cik_str":1815086,"ticker":"BTWN","title":"Bridgetown Holdings Ltd"},"3759":{"cik_str":1889112,"ticker":"RENE","title":"Cartesian Growth Corp II"},"3760":{"cik_str":832370,"ticker":"EAXR","title":"Ealixir, Inc."},"3761":{"cik_str":1041859,"ticker":"PLCE","title":"Childrens Place, Inc."},"3762":{"cik_str":917491,"ticker":"FARO","title":"FARO TECHNOLOGIES INC"},"3763":{"cik_str":1848437,"ticker":"CITE","title":"Cartica Acquisition Corp"},"3764":{"cik_str":1434728,"ticker":"GWRS","title":"Global Water Resources, Inc."},"3765":{"cik_str":903651,"ticker":"INOD","title":"INNODATA INC"},"3766":{"cik_str":891290,"ticker":"RFI","title":"COHEN & STEERS TOTAL RETURN REALTY FUND INC"},"3767":{"cik_str":829365,"ticker":"NTPIF","title":"NAM TAI PROPERTY INC."},"3768":{"cik_str":1792781,"ticker":"CURV","title":"Torrid Holdings Inc."},"3769":{"cik_str":1843714,"ticker":"WNNR","title":"Andretti Acquisition Corp."},"3770":{"cik_str":1843993,"ticker":"THCP","title":"Thunder Bridge Capital Partners IV, Inc."},"3771":{"cik_str":1001385,"ticker":"NWPX","title":"NORTHWEST PIPE CO"},"3772":{"cik_str":1635650,"ticker":"GPP","title":"Green Plains Partners LP"},"3773":{"cik_str":811808,"ticker":"SMMF","title":"SUMMIT FINANCIAL GROUP, INC."},"3774":{"cik_str":1501134,"ticker":"NVTA","title":"Invitae Corp"},"3775":{"cik_str":46207,"ticker":"HAWEL","title":"HAWAIIAN ELECTRIC CO INC"},"3776":{"cik_str":1879221,"ticker":"UTAA","title":"UTA Acquisition Corp"},"3777":{"cik_str":1162461,"ticker":"CUTR","title":"CUTERA INC"},"3778":{"cik_str":1854964,"ticker":"NLCP","title":"NewLake Capital Partners, Inc."},"3779":{"cik_str":1854480,"ticker":"FIAC","title":"Focus Impact Acquisition Corp."},"3780":{"cik_str":1885444,"ticker":"KNSW","title":"KnightSwan Acquisition Corp"},"3781":{"cik_str":1552198,"ticker":"WHF","title":"WhiteHorse Finance, Inc."},"3782":{"cik_str":1244183,"ticker":"PFL","title":"PIMCO INCOME STRATEGY FUND"},"3783":{"cik_str":1593899,"ticker":"AVIR","title":"Atea Pharmaceuticals, Inc."},"3784":{"cik_str":275694,"ticker":"MCI","title":"BARINGS CORPORATE INVESTORS"},"3785":{"cik_str":1841024,"ticker":"LCAA","title":"L Catterton Asia Acquisition Corp"},"3786":{"cik_str":1421461,"ticker":"IPI","title":"Intrepid Potash, Inc."},"3787":{"cik_str":1847351,"ticker":"BRD","title":"Beard Energy Transition Acquisition Corp."},"3788":{"cik_str":1434621,"ticker":"TREE","title":"LendingTree, Inc."},"3789":{"cik_str":1239819,"ticker":"LUNA","title":"LUNA INNOVATIONS INC"},"3790":{"cik_str":1865200,"ticker":"PORT","title":"Southport Acquisition Corp"},"3791":{"cik_str":1851230,"ticker":"CREC","title":"Crescera Capital Acquisition Corp."},"3792":{"cik_str":1857803,"ticker":"DPCS","title":"DP Cap Acquisition Corp I"},"3793":{"cik_str":1823584,"ticker":"AENT","title":"ALLIANCE ENTERTAINMENT HOLDING CORP"},"3794":{"cik_str":1852736,"ticker":"TRIS","title":"Tristar Acquisition I Corp."},"3795":{"cik_str":1771226,"ticker":"RMM","title":"RiverNorth Managed Duration Municipal Income Fund, Inc."},"3796":{"cik_str":1847577,"ticker":"LCW","title":"Learn CW Investment Corp"},"3797":{"cik_str":1181187,"ticker":"BYM","title":"BLACKROCK MUNICIPAL INCOME QUALITY TRUST"},"3798":{"cik_str":1810182,"ticker":"ALXO","title":"ALX ONCOLOGY HOLDINGS INC"},"3799":{"cik_str":1861541,"ticker":"PGSS","title":"Pegasus Digital Mobility Acquisition Corp."},"3800":{"cik_str":1753539,"ticker":"BKSY","title":"BlackSky Technology Inc."},"3801":{"cik_str":1852016,"ticker":"AEAE","title":"AltEnergy Acquisition Corp"},"3802":{"cik_str":1822145,"ticker":"PRST","title":"Presto Automation Inc."},"3803":{"cik_str":1831359,"ticker":"JWSM","title":"Jaws Mustang Acquisition Corp"},"3804":{"cik_str":1501697,"ticker":"XFOR","title":"X4 Pharmaceuticals, Inc"},"3805":{"cik_str":810958,"ticker":"CZNC","title":"CITIZENS & NORTHERN CORP"},"3806":{"cik_str":835333,"ticker":"OIA","title":"Invesco Municipal Income Opportunities Trust"},"3807":{"cik_str":1375340,"ticker":"FOF","title":"Cohen & Steers Closed-End Opportunity Fund, Inc."},"3808":{"cik_str":1845149,"ticker":"CBRG","title":"Chain Bridge I"},"3809":{"cik_str":1551901,"ticker":"SCM","title":"Stellus Capital Investment Corp"},"3810":{"cik_str":844790,"ticker":"PMM","title":"PUTNAM MANAGED MUNICIPAL INCOME TRUST"},"3811":{"cik_str":1903870,"ticker":"AFRI","title":"Forafric Global PLC"},"3812":{"cik_str":1519401,"ticker":"RM","title":"Regional Management Corp."},"3813":{"cik_str":1745059,"ticker":"FINS","title":"Angel Oak Financial Strategies Income Term Trust"},"3814":{"cik_str":1005286,"ticker":"LFCR","title":"LIFECORE BIOMEDICAL, INC. \\DE\\"},"3815":{"cik_str":1086434,"ticker":"AUDC","title":"AUDIOCODES LTD"},"3816":{"cik_str":1710482,"ticker":"JMSB","title":"John Marshall Bancorp, Inc."},"3817":{"cik_str":807707,"ticker":"VOXX","title":"VOXX International Corp"},"3818":{"cik_str":1892747,"ticker":"VMCA","title":"Valuence Merger Corp. I"},"3819":{"cik_str":1857410,"ticker":"IVCB","title":"Investcorp Europe Acquisition Corp I"},"3820":{"cik_str":1818382,"ticker":"HUMA","title":"Humacyte, Inc."},"3821":{"cik_str":1720116,"ticker":"RDVT","title":"Red Violet, Inc."},"3822":{"cik_str":1432133,"ticker":"KLTR","title":"KALTURA INC"},"3823":{"cik_str":1041657,"ticker":"UONE","title":"URBAN ONE, INC."},"3824":{"cik_str":1277902,"ticker":"MVBF","title":"MVB FINANCIAL CORP"},"3825":{"cik_str":1805521,"ticker":"FFIE","title":"FARADAY FUTURE INTELLIGENT ELECTRIC INC."},"3826":{"cik_str":1699382,"ticker":"PMVP","title":"PMV Pharmaceuticals, Inc."},"3827":{"cik_str":1779128,"ticker":"BLDE","title":"Blade Air Mobility, Inc."},"3828":{"cik_str":1832332,"ticker":"AVAH","title":"Aveanna Healthcare Holdings, Inc."},"3829":{"cik_str":1036262,"ticker":"INTT","title":"INTEST CORP"},"3830":{"cik_str":915779,"ticker":"DAKT","title":"DAKTRONICS INC /SD/"},"3831":{"cik_str":1590584,"ticker":"CVEO","title":"Civeo Corp"},"3832":{"cik_str":1857816,"ticker":"GCT","title":"GigaCloud Technology Inc"},"3833":{"cik_str":1643953,"ticker":"PRPL","title":"Purple Innovation, Inc."},"3834":{"cik_str":1040130,"ticker":"PETS","title":"PETMED EXPRESS INC"},"3835":{"cik_str":1230869,"ticker":"ASA","title":"ASA Gold & Precious Metals Ltd"},"3836":{"cik_str":933972,"ticker":"SCWO","title":"374Water Inc."},"3837":{"cik_str":1403475,"ticker":"BMRC","title":"Bank of Marin Bancorp"},"3838":{"cik_str":1722964,"ticker":"YMAB","title":"Y-mAbs Therapeutics, Inc."},"3839":{"cik_str":63330,"ticker":"MLP","title":"MAUI LAND & PINEAPPLE CO INC"},"3840":{"cik_str":1597672,"ticker":"RYAM","title":"RAYONIER ADVANCED MATERIALS INC."},"3841":{"cik_str":1356090,"ticker":"PGEN","title":"PRECIGEN, INC."},"3842":{"cik_str":1822462,"ticker":"FHTX","title":"Foghorn Therapeutics Inc."},"3843":{"cik_str":1781174,"ticker":"ACRV","title":"Acrivon Therapeutics, Inc."},"3844":{"cik_str":1848861,"ticker":"HAIA","title":"Healthcare AI Acquisition Corp."},"3845":{"cik_str":1854139,"ticker":"ZVIA","title":"Zevia PBC"},"3846":{"cik_str":836412,"ticker":"ZTR","title":"Virtus Total Return Fund Inc."},"3847":{"cik_str":1847355,"ticker":"TGAA","title":"Target Global Acquisition I Corp."},"3848":{"cik_str":1316517,"ticker":"KNDI","title":"Kandi Technologies Group, Inc."},"3849":{"cik_str":1829667,"ticker":"RGC","title":"Regencell Bioscience Holdings Ltd"},"3850":{"cik_str":1479681,"ticker":"NUTX","title":"Nutex Health, Inc."},"3851":{"cik_str":356590,"ticker":"GTII","title":"GLOBAL TECH INDUSTRIES GROUP, INC."},"3852":{"cik_str":1853397,"ticker":"XPAX","title":"XPAC Acquisition Corp."},"3853":{"cik_str":1392326,"ticker":"CPLP","title":"Capital Product Partners L.P."},"3854":{"cik_str":1790340,"ticker":"IMRX","title":"Immuneering Corp"},"3855":{"cik_str":715579,"ticker":"ACNB","title":"ACNB CORP"},"3856":{"cik_str":1026214,"ticker":"FMCC","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"3857":{"cik_str":818851,"ticker":"NCA","title":"NUVEEN CALIFORNIA MUNICIPAL VALUE FUND"},"3858":{"cik_str":1828791,"ticker":"DSP","title":"Viant Technology Inc."},"3859":{"cik_str":1515324,"ticker":"ARDC","title":"Ares Dynamic Credit Allocation Fund, Inc."},"3860":{"cik_str":1567924,"ticker":"MIESY","title":"Mitsui E&S Holdings Co., Ltd./ADR"},"3861":{"cik_str":1628808,"ticker":"PROF","title":"Profound Medical Corp."},"3862":{"cik_str":855887,"ticker":"DSM","title":"BNY MELLON STRATEGIC MUNICIPAL BOND FUND, INC."},"3863":{"cik_str":1338561,"ticker":"SPXX","title":"Nuveen S&P 500 Dynamic Overwrite Fund"},"3864":{"cik_str":1382101,"ticker":"STRO","title":"SUTRO BIOPHARMA, INC."},"3865":{"cik_str":1823587,"ticker":"SKYH","title":"Sky Harbour Group Corp"},"3866":{"cik_str":1848275,"ticker":"TOP","title":"TOP Financial Group Ltd"},"3867":{"cik_str":1653558,"ticker":"PRTH","title":"Priority Technology Holdings, Inc."},"3868":{"cik_str":1871321,"ticker":"DRTS","title":"Alpha Tau Medical Ltd."},"3869":{"cik_str":1517006,"ticker":"GATO","title":"Gatos Silver, Inc."},"3870":{"cik_str":1342423,"ticker":"LMNR","title":"Limoneira CO"},"3871":{"cik_str":1680379,"ticker":"SBT","title":"Sterling Bancorp, Inc."},"3872":{"cik_str":76267,"ticker":"PKE","title":"PARK AEROSPACE CORP"},"3873":{"cik_str":720858,"ticker":"ITIC","title":"INVESTORS TITLE CO"},"3874":{"cik_str":1849580,"ticker":"WAVC","title":"Waverley Capital Acquisition Corp. 1"},"3875":{"cik_str":1819133,"ticker":"TNGX","title":"Tango Therapeutics, Inc."},"3876":{"cik_str":1368493,"ticker":"CAF","title":"Morgan Stanley China A Share Fund, Inc."},"3877":{"cik_str":851170,"ticker":"MCR","title":"MFS CHARTER INCOME TRUST"},"3878":{"cik_str":1823000,"ticker":"CONX","title":"CONX Corp."},"3879":{"cik_str":1375205,"ticker":"URG","title":"UR-ENERGY INC"},"3880":{"cik_str":1314102,"ticker":"EYPT","title":"EyePoint Pharmaceuticals, Inc."},"3881":{"cik_str":1547459,"ticker":"NGVC","title":"Natural Grocers by Vitamin Cottage, Inc."},"3882":{"cik_str":1820630,"ticker":"PTRA","title":"Proterra Inc"},"3883":{"cik_str":1912461,"ticker":"SKGR","title":"SK Growth Opportunities Corp"},"3884":{"cik_str":1400438,"ticker":"LGO","title":"Largo Inc."},"3885":{"cik_str":1812234,"ticker":"CCV","title":"Churchill Capital Corp V"},"3886":{"cik_str":1743102,"ticker":"JFIN","title":"Jiayin Group Inc."},"3887":{"cik_str":1098151,"ticker":"FDBC","title":"FIDELITY D & D BANCORP INC"},"3888":{"cik_str":1851909,"ticker":"CDAQ","title":"Compass Digital Acquisition Corp."},"3889":{"cik_str":746838,"ticker":"UIS","title":"UNISYS CORP"},"3890":{"cik_str":1282977,"ticker":"GHL","title":"GREENHILL & CO INC"},"3891":{"cik_str":1876588,"ticker":"ZIMV","title":"ZimVie Inc."},"3892":{"cik_str":740663,"ticker":"FLIC","title":"FIRST OF LONG ISLAND CORP"},"3893":{"cik_str":65312,"ticker":"EVI","title":"EVI INDUSTRIES, INC."},"3894":{"cik_str":944745,"ticker":"CIVB","title":"CIVISTA BANCSHARES, INC."},"3895":{"cik_str":1792829,"ticker":"VSTA","title":"Vasta Platform Ltd"},"3896":{"cik_str":1840706,"ticker":"SOPH","title":"SOPHiA GENETICS SA"},"3897":{"cik_str":1163609,"ticker":"SDSYA","title":"SOUTH DAKOTA SOYBEAN PROCESSORS LLC"},"3898":{"cik_str":1018164,"ticker":"WLFC","title":"WILLIS LEASE FINANCE CORP"},"3899":{"cik_str":921299,"ticker":"FGEN","title":"FIBROGEN INC"},"3900":{"cik_str":1867949,"ticker":"REFI","title":"Chicago Atlantic Real Estate Finance, Inc."},"3901":{"cik_str":1454741,"ticker":"EOT","title":"Eaton Vance National Municipal Opportunities Trust"},"3902":{"cik_str":1606163,"ticker":"LMB","title":"Limbach Holdings, Inc."},"3903":{"cik_str":765207,"ticker":"FNLC","title":"First Bancorp, Inc /ME/"},"3904":{"cik_str":1284940,"ticker":"FEN","title":"FIRST TRUST ENERGY INCOME & GROWTH FUND"},"3905":{"cik_str":1750153,"ticker":"GOEV","title":"Canoo Inc."},"3906":{"cik_str":72162,"ticker":"NL","title":"NL INDUSTRIES INC"},"3907":{"cik_str":1436425,"ticker":"HBCP","title":"HOME BANCORP, INC."},"3908":{"cik_str":1703644,"ticker":"GPMT","title":"Granite Point Mortgage Trust Inc."},"3909":{"cik_str":1140392,"ticker":"PMF","title":"PIMCO MUNICIPAL INCOME FUND"},"3910":{"cik_str":1907982,"ticker":"QBTS","title":"D-Wave Quantum Inc."},"3911":{"cik_str":1341317,"ticker":"BWB","title":"Bridgewater Bancshares Inc"},"3912":{"cik_str":1629665,"ticker":"MULG","title":"MULIANG VIAGOO TECHNOLOGY, INC."},"3913":{"cik_str":1049606,"ticker":"CIX","title":"COMPX INTERNATIONAL INC"},"3914":{"cik_str":1181506,"ticker":"PMX","title":"PIMCO MUNICIPAL INCOME FUND III"},"3915":{"cik_str":1846017,"ticker":"BLFY","title":"Blue Foundry Bancorp"},"3916":{"cik_str":65172,"ticker":"MSB","title":"MESABI TRUST"},"3917":{"cik_str":1195734,"ticker":"PBPB","title":"POTBELLY CORP"},"3918":{"cik_str":738214,"ticker":"AMTX","title":"AEMETIS, INC"},"3919":{"cik_str":1497186,"ticker":"HYI","title":"Western Asset High Yield Defined Opportunity Fund Inc."},"3920":{"cik_str":1876716,"ticker":"ASCB","title":"ASPAC II Acquisition Corp."},"3921":{"cik_str":1839990,"ticker":"ARTE","title":"Artemis Strategic Investment Corp"},"3922":{"cik_str":1868640,"ticker":"VHNA","title":"Vahanna Tech Edge Acquisition I Corp."},"3923":{"cik_str":1006655,"ticker":"EPM","title":"EVOLUTION PETROLEUM CORP"},"3924":{"cik_str":1769256,"ticker":"FLJ","title":"FLJ Group Ltd"},"3925":{"cik_str":1795815,"ticker":"BCAL","title":"Southern California Bancorp CA"},"3926":{"cik_str":1641614,"ticker":"PMTS","title":"CPI Card Group Inc."},"3927":{"cik_str":809173,"ticker":"MMT","title":"MFS MULTIMARKET INCOME TRUST"},"3928":{"cik_str":1533924,"ticker":"AMPY","title":"Amplify Energy Corp."},"3929":{"cik_str":1287480,"ticker":"BGT","title":"BLACKROCK FLOATING RATE INCOME TRUST"},"3930":{"cik_str":768835,"ticker":"BIG","title":"BIG LOTS INC"},"3931":{"cik_str":1454938,"ticker":"OB","title":"Outbrain Inc."},"3932":{"cik_str":846676,"ticker":"AEF","title":"ABRDN EMERGING MARKETS EQUITY INCOME FUND, INC."},"3933":{"cik_str":1850838,"ticker":"OMGA","title":"Omega Therapeutics, Inc."},"3934":{"cik_str":809248,"ticker":"TAST","title":"CARROLS RESTAURANT GROUP, INC."},"3935":{"cik_str":276720,"ticker":"PCYO","title":"PURE CYCLE CORP"},"3936":{"cik_str":1465872,"ticker":"VTDRF","title":"VANTAGE DRILLING INTERNATIONAL"},"3937":{"cik_str":1351636,"ticker":"SSTI","title":"SOUNDTHINKING, INC."},"3938":{"cik_str":1856161,"ticker":"PRLH","title":"Pearl Holdings Acquisition Corp"},"3939":{"cik_str":1830210,"ticker":"BHIL","title":"Benson Hill, Inc."},"3940":{"cik_str":1521404,"ticker":"BGH","title":"BARINGS GLOBAL SHORT DURATION HIGH YIELD FUND"},"3941":{"cik_str":1863294,"ticker":"DALS","title":"DA32 Life Science Tech Acquisition Corp."},"3942":{"cik_str":1834026,"ticker":"GROY","title":"Gold Royalty Corp."},"3943":{"cik_str":1401257,"ticker":"FET","title":"FORUM ENERGY TECHNOLOGIES, INC."},"3944":{"cik_str":275053,"ticker":"NATR","title":"NATURES SUNSHINE PRODUCTS INC"},"3945":{"cik_str":1789029,"ticker":"AEVA","title":"Aeva Technologies, Inc."},"3946":{"cik_str":1766368,"ticker":"MEC","title":"Mayville Engineering Company, Inc."},"3947":{"cik_str":1417892,"ticker":"SOL","title":"Emeren Group Ltd"},"3948":{"cik_str":1676479,"ticker":"CSTR","title":"CapStar Financial Holdings, Inc."},"3949":{"cik_str":1846975,"ticker":"SEDA","title":"SDCL EDGE Acquisition Corp"},"3950":{"cik_str":1294133,"ticker":"INGN","title":"Inogen Inc"},"3951":{"cik_str":1875558,"ticker":"NVCT","title":"Nuvectis Pharma, Inc."},"3952":{"cik_str":1364125,"ticker":"WRN","title":"Western Copper & Gold Corp"},"3953":{"cik_str":727273,"ticker":"CDZI","title":"CADIZ INC"},"3954":{"cik_str":1698514,"ticker":"NESR","title":"National Energy Services Reunited Corp."},"3955":{"cik_str":1863006,"ticker":"VLN","title":"Valens Semiconductor Ltd."},"3956":{"cik_str":746514,"ticker":"NEN","title":"NEW ENGLAND REALTY ASSOCIATES LIMITED PARTNERSHIP"},"3957":{"cik_str":1726445,"ticker":"SEER","title":"Seer, Inc."},"3958":{"cik_str":1826492,"ticker":"GRCL","title":"Gracell Biotechnologies Inc."},"3959":{"cik_str":789933,"ticker":"NC","title":"NACCO INDUSTRIES INC"},"3960":{"cik_str":1855693,"ticker":"FICV","title":"Frontier Investment Corp"},"3961":{"cik_str":832327,"ticker":"BKT","title":"BLACKROCK INCOME TRUST, INC."},"3962":{"cik_str":827871,"ticker":"EGRX","title":"EAGLE PHARMACEUTICALS, INC."},"3963":{"cik_str":1854863,"ticker":"BWC","title":"Blue Whale Acquisition Corp I"},"3964":{"cik_str":1615905,"ticker":"JGH","title":"Nuveen Global High Income Fund"},"3965":{"cik_str":1130144,"ticker":"BSRR","title":"SIERRA BANCORP"},"3966":{"cik_str":1441082,"ticker":"HLCO","title":"Healing Co Inc."},"3967":{"cik_str":1403256,"ticker":"SCU","title":"Sculptor Capital Management, Inc."},"3968":{"cik_str":1855747,"ticker":"BLND","title":"Blend Labs, Inc."},"3969":{"cik_str":1227073,"ticker":"ERC","title":"ALLSPRING MULTI-SECTOR INCOME FUND"},"3970":{"cik_str":1419536,"ticker":"CBNK","title":"Capital Bancorp Inc"},"3971":{"cik_str":1289945,"ticker":"SPOK","title":"Spok Holdings, Inc"},"3972":{"cik_str":1371489,"ticker":"III","title":"Information Services Group Inc."},"3973":{"cik_str":1282850,"ticker":"FCT","title":"FIRST TRUST SENIOR FLOATING RATE INCOME FUND II"},"3974":{"cik_str":1484778,"ticker":"TDUP","title":"ThredUp Inc."},"3975":{"cik_str":23197,"ticker":"CMTL","title":"COMTECH TELECOMMUNICATIONS CORP /DE/"},"3976":{"cik_str":1822523,"ticker":"AFCG","title":"AFC Gamma, Inc."},"3977":{"cik_str":1680873,"ticker":"HFFG","title":"HF Foods Group Inc."},"3978":{"cik_str":1685428,"ticker":"LMDX","title":"LumiraDx Ltd"},"3979":{"cik_str":1703079,"ticker":"XFLT","title":"XAI Octagon Floating Rate & Alternative Income Term Trust"},"3980":{"cik_str":1941536,"ticker":"BAER","title":"Bridger Aerospace Group Holdings, Inc."},"3981":{"cik_str":88948,"ticker":"SENEA","title":"Seneca Foods Corp"},"3982":{"cik_str":1848323,"ticker":"LOCC","title":"Live Oak Crestview Climate Acquisition Corp."},"3983":{"cik_str":65433,"ticker":"MXF","title":"MEXICO FUND INC"},"3984":{"cik_str":1844280,"ticker":"THRN","title":"Thorne Healthtech, Inc."},"3985":{"cik_str":1320461,"ticker":"CPS","title":"Cooper-Standard Holdings Inc."},"3986":{"cik_str":1579214,"ticker":"EEX","title":"Emerald Holding, Inc."},"3987":{"cik_str":1796073,"ticker":"VZLA","title":"Vizsla Silver Corp."},"3988":{"cik_str":1708688,"ticker":"IFRX","title":"InflaRx N.V."},"3989":{"cik_str":1786117,"ticker":"PINE","title":"Alpine Income Property Trust, Inc."},"3990":{"cik_str":1137390,"ticker":"BNY","title":"BLACKROCK NEW YORK MUNICIPAL INCOME TRUST"},"3991":{"cik_str":1801417,"ticker":"BYNO","title":"byNordic Acquisition Corp"},"3992":{"cik_str":1723648,"ticker":"LVOX","title":"LiveVox Holdings, Inc."},"3993":{"cik_str":1850529,"ticker":"HMA","title":"Heartland Media Acquisition Corp."},"3994":{"cik_str":1699039,"ticker":"RNGR","title":"Ranger Energy Services, Inc."},"3995":{"cik_str":1834253,"ticker":"EM","title":"Smart Share Global Ltd"},"3996":{"cik_str":1816431,"ticker":"QSI","title":"Quantum-Si Inc"},"3997":{"cik_str":1506929,"ticker":"VRDR","title":"VERDE RESOURCES, INC."},"3998":{"cik_str":1063344,"ticker":"HT","title":"HERSHA HOSPITALITY TRUST"},"3999":{"cik_str":1233087,"ticker":"FTF","title":"FRANKLIN LTD DURATION INCOME TRUST"},"4000":{"cik_str":1370450,"ticker":"WLDN","title":"Willdan Group, Inc."},"4001":{"cik_str":1856961,"ticker":"BOCN","title":"Blue Ocean Acquisition Corp"},"4002":{"cik_str":862831,"ticker":"FISI","title":"FINANCIAL INSTITUTIONS INC"},"4003":{"cik_str":1660334,"ticker":"VRCA","title":"Verrica Pharmaceuticals Inc."},"4004":{"cik_str":799288,"ticker":"LE","title":"LANDS' END, INC."},"4005":{"cik_str":1619856,"ticker":"CRBU","title":"Caribou Biosciences, Inc."},"4006":{"cik_str":106532,"ticker":"WEYS","title":"WEYCO GROUP INC"},"4007":{"cik_str":1779020,"ticker":"DNMR","title":"Danimer Scientific, Inc."},"4008":{"cik_str":1825249,"ticker":"CFIV","title":"CF ACQUISITION CORP. IV"},"4009":{"cik_str":916793,"ticker":"SILC","title":"SILICOM LTD."},"4010":{"cik_str":1875493,"ticker":"CSLM","title":"Consilium Acquisition Corp I, Ltd."},"4011":{"cik_str":1521951,"ticker":"FBIZ","title":"FIRST BUSINESS FINANCIAL SERVICES, INC."},"4012":{"cik_str":1450445,"ticker":"NUW","title":"Nuveen AMT-Free Municipal Value Fund"},"4013":{"cik_str":1835512,"ticker":"LLAP","title":"Terran Orbital Corp"},"4014":{"cik_str":1625414,"ticker":"BZUN","title":"Baozun Inc."},"4015":{"cik_str":1839519,"ticker":"CFFS","title":"CF Acquisition Corp. VII"},"4016":{"cik_str":1552800,"ticker":"TTSH","title":"TILE SHOP HOLDINGS, INC."},"4017":{"cik_str":1831283,"ticker":"LIAN","title":"LianBio"},"4018":{"cik_str":1667011,"ticker":"AIP","title":"Arteris, Inc."},"4019":{"cik_str":1819574,"ticker":"BARK","title":"Bark, Inc."},"4020":{"cik_str":1501072,"ticker":"RIV","title":"RIVERNORTH OPPORTUNITIES FUND, INC."},"4021":{"cik_str":889609,"ticker":"CPSS","title":"CONSUMER PORTFOLIO SERVICES, INC."},"4022":{"cik_str":1778016,"ticker":"IMAB","title":"I-Mab"},"4023":{"cik_str":1561921,"ticker":"TELA","title":"TELA Bio, Inc."},"4024":{"cik_str":1013272,"ticker":"NWFL","title":"NORWOOD FINANCIAL CORP"},"4025":{"cik_str":884144,"ticker":"ASUR","title":"ASURE SOFTWARE INC"},"4026":{"cik_str":76282,"ticker":"PKOH","title":"PARK OHIO HOLDINGS CORP"},"4027":{"cik_str":1716583,"ticker":"HYZN","title":"Hyzon Motors Inc."},"4028":{"cik_str":1015383,"ticker":"POWW","title":"AMMO, INC."},"4029":{"cik_str":1503707,"ticker":"NHHS","title":"NorthStar Healthcare Income, Inc."},"4030":{"cik_str":909112,"ticker":"TEI","title":"TEMPLETON EMERGING MARKETS INCOME FUND"},"4031":{"cik_str":1448431,"ticker":"OPRX","title":"OptimizeRx Corp"},"4032":{"cik_str":1651625,"ticker":"ACIU","title":"AC Immune SA"},"4033":{"cik_str":1066194,"ticker":"EGAN","title":"EGAIN Corp"},"4034":{"cik_str":1598599,"ticker":"IPHA","title":"Innate Pharma SA"},"4035":{"cik_str":1649096,"ticker":"CLPR","title":"Clipper Realty Inc."},"4036":{"cik_str":1326003,"ticker":"BKCC","title":"BlackRock Capital Investment Corp"},"4037":{"cik_str":916183,"ticker":"RCS","title":"PIMCO STRATEGIC INCOME FUND, INC"},"4038":{"cik_str":1612875,"ticker":"GER","title":"GOLDMAN SACHS MLP & ENERGY RENAISSANCE FUND"},"4039":{"cik_str":934549,"ticker":"ACTG","title":"ACACIA RESEARCH CORP"},"4040":{"cik_str":1211583,"ticker":"FENC","title":"FENNEC PHARMACEUTICALS INC."},"4041":{"cik_str":876343,"ticker":"LCTX","title":"Lineage Cell Therapeutics, Inc."},"4042":{"cik_str":1525201,"ticker":"DBL","title":"DoubleLine Opportunistic Credit Fund"},"4043":{"cik_str":1636651,"ticker":"OVID","title":"Ovid Therapeutics Inc."},"4044":{"cik_str":1717307,"ticker":"ILPT","title":"Industrial Logistics Properties Trust"},"4045":{"cik_str":920427,"ticker":"UNTY","title":"UNITY BANCORP INC /NJ/"},"4046":{"cik_str":1175680,"ticker":"CYDY","title":"CytoDyn Inc."},"4047":{"cik_str":825202,"ticker":"FUND","title":"SPROTT FOCUS TRUST INC."},"4048":{"cik_str":804123,"ticker":"TWN","title":"TAIWAN FUND INC"},"4049":{"cik_str":1261654,"ticker":"UTI","title":"UNIVERSAL TECHNICAL INSTITUTE INC"},"4050":{"cik_str":1454789,"ticker":"ATXS","title":"Astria Therapeutics, Inc."},"4051":{"cik_str":1436126,"ticker":"MG","title":"Mistras Group, Inc."},"4052":{"cik_str":1593222,"ticker":"CIO","title":"City Office REIT, Inc."},"4053":{"cik_str":1515940,"ticker":"KIO","title":"KKR Income Opportunities Fund"},"4054":{"cik_str":1362481,"ticker":"AGD","title":"abrdn Global Dynamic Dividend Fund"},"4055":{"cik_str":1867752,"ticker":"CIAN","title":"Cian PLC"},"4056":{"cik_str":1227857,"ticker":"NCZ","title":"Virtus Convertible & Income Fund II"},"4057":{"cik_str":1866874,"ticker":"RLTY","title":"Cohen & Steers Real Estate Opportunities & Income Fund"},"4058":{"cik_str":1378950,"ticker":"PRTS","title":"CarParts.com, Inc."},"4059":{"cik_str":1668243,"ticker":"URGN","title":"UroGen Pharma Ltd."},"4060":{"cik_str":1678660,"ticker":"PRLD","title":"Prelude Therapeutics Inc"},"4061":{"cik_str":1693011,"ticker":"INZY","title":"Inozyme Pharma, Inc."},"4062":{"cik_str":1041514,"ticker":"LSAK","title":"LESAKA TECHNOLOGIES INC"},"4063":{"cik_str":1035092,"ticker":"SHBI","title":"SHORE BANCSHARES INC"},"4064":{"cik_str":1814114,"ticker":"OBIO","title":"Orchestra BioMed Holdings, Inc."},"4065":{"cik_str":1530249,"ticker":"FSBW","title":"FS Bancorp, Inc."},"4066":{"cik_str":1058811,"ticker":"IMMR","title":"IMMERSION CORP"},"4067":{"cik_str":1769663,"ticker":"PBFS","title":"Pioneer Bancorp, Inc./MD"},"4068":{"cik_str":778164,"ticker":"ALTO","title":"Alto Ingredients, Inc."},"4069":{"cik_str":895528,"ticker":"VPV","title":"Invesco Pennsylvania Value Municipal Income Trust"},"4070":{"cik_str":355948,"ticker":"RELL","title":"RICHARDSON ELECTRONICS, LTD."},"4071":{"cik_str":1858351,"ticker":"ICNC","title":"Iconic Sports Acquisition Corp."},"4072":{"cik_str":1066719,"ticker":"TIHE","title":"Taihe Group, Inc."},"4073":{"cik_str":1722606,"ticker":"MTA","title":"Metalla Royalty & Streaming Ltd."},"4074":{"cik_str":1573221,"ticker":"REAL","title":"TheRealReal, Inc."},"4075":{"cik_str":850429,"ticker":"TG","title":"TREDEGAR CORP"},"4076":{"cik_str":1123494,"ticker":"HBIO","title":"HARVARD BIOSCIENCE INC"},"4077":{"cik_str":1818502,"ticker":"OPFI","title":"OppFi Inc."},"4078":{"cik_str":1819438,"ticker":"GWH","title":"ESS Tech, Inc."},"4079":{"cik_str":1399520,"ticker":"STKS","title":"ONE Group Hospitality, Inc."},"4080":{"cik_str":771266,"ticker":"KOPN","title":"KOPIN CORP"},"4081":{"cik_str":1598981,"ticker":"SKYX","title":"SKYX Platforms Corp."},"4082":{"cik_str":1812173,"ticker":"RBOT","title":"Vicarious Surgical Inc."},"4083":{"cik_str":1862993,"ticker":"APCA","title":"AP Acquisition Corp"},"4084":{"cik_str":1001233,"ticker":"SGMO","title":"SANGAMO THERAPEUTICS, INC"},"4085":{"cik_str":1567529,"ticker":"KMDA","title":"KAMADA LTD"},"4086":{"cik_str":1746129,"ticker":"BSVN","title":"Bank7 Corp."},"4087":{"cik_str":1959455,"ticker":"HSHP","title":"Himalaya Shipping Ltd."},"4088":{"cik_str":1835579,"ticker":"IKNA","title":"Ikena Oncology, Inc."},"4089":{"cik_str":891482,"ticker":"FLL","title":"FULL HOUSE RESORTS INC"},"4090":{"cik_str":1513789,"ticker":"FIF","title":"FIRST TRUST ENERGY INFRASTRUCTURE FUND"},"4091":{"cik_str":1894951,"ticker":"GBBK","title":"Global Blockchain Acquisition Corp."},"4092":{"cik_str":41091,"ticker":"GPJA","title":"GEORGIA POWER CO"},"4093":{"cik_str":1720893,"ticker":"BTAI","title":"BioXcel Therapeutics, Inc."},"4094":{"cik_str":1594204,"ticker":"RBTK","title":"ZHEN DING RESOURCES INC."},"4095":{"cik_str":1863719,"ticker":"MNTN","title":"Everest Consolidator Acquisition Corp"},"4096":{"cik_str":1930021,"ticker":"PTHR","title":"Pono Capital Three, Inc."},"4097":{"cik_str":1584754,"ticker":"AKTS","title":"Akoustis Technologies, Inc."},"4098":{"cik_str":1874495,"ticker":"FRBN","title":"Forbion European Acquisition Corp."},"4099":{"cik_str":1413837,"ticker":"FFWM","title":"First Foundation Inc."},"4100":{"cik_str":1171759,"ticker":"RRGB","title":"RED ROBIN GOURMET BURGERS INC"},"4101":{"cik_str":887394,"ticker":"MQT","title":"BLACKROCK MUNIYIELD QUALITY FUND II, INC."},"4102":{"cik_str":916618,"ticker":"IIF","title":"MORGAN STANLEY INDIA INVESTMENT FUND, INC."},"4103":{"cik_str":873860,"ticker":"OCN","title":"OCWEN FINANCIAL CORP"},"4104":{"cik_str":1845618,"ticker":"TWCB","title":"Bilander Acquisition Corp."},"4105":{"cik_str":1333493,"ticker":"EHTH","title":"eHealth, Inc."},"4106":{"cik_str":1816613,"ticker":"MKFG","title":"Markforged Holding Corp"},"4107":{"cik_str":1928340,"ticker":"GDHG","title":"GOLDEN HEAVEN GROUP HOLDINGS LTD."},"4108":{"cik_str":64472,"ticker":"GENC","title":"GENCOR INDUSTRIES INC"},"4109":{"cik_str":1657045,"ticker":"CHMX","title":"NEXT-ChemX Corporation."},"4110":{"cik_str":1034842,"ticker":"RIGL","title":"RIGEL PHARMACEUTICALS INC"},"4111":{"cik_str":766011,"ticker":"CMCL","title":"Caledonia Mining Corp Plc"},"4112":{"cik_str":1526243,"ticker":"PPTA","title":"PERPETUA RESOURCES CORP."},"4113":{"cik_str":945983,"ticker":"CLMB","title":"Climb Global Solutions, Inc."},"4114":{"cik_str":897269,"ticker":"MVT","title":"BLACKROCK MUNIVEST FUND II, INC."},"4115":{"cik_str":1861841,"ticker":"ARBE","title":"Arbe Robotics Ltd."},"4116":{"cik_str":1555279,"ticker":"MASS","title":"908 Devices Inc."},"4117":{"cik_str":23426,"ticker":"CNLHN","title":"CONNECTICUT LIGHT & POWER CO"},"4118":{"cik_str":1847398,"ticker":"NECB","title":"NorthEast Community Bancorp, Inc./MD/"},"4119":{"cik_str":1687932,"ticker":"JILL","title":"J.Jill, Inc."},"4120":{"cik_str":1818089,"ticker":"BOAC","title":"Bluescape Opportunities Acquisition Corp."},"4121":{"cik_str":1499422,"ticker":"RBB","title":"RBB Bancorp"},"4122":{"cik_str":1163370,"ticker":"NRIM","title":"NORTHRIM BANCORP INC"},"4123":{"cik_str":1640428,"ticker":"EVER","title":"EverQuote, Inc."},"4124":{"cik_str":922358,"ticker":"FGPR","title":"FERRELLGAS PARTNERS L P"},"4125":{"cik_str":1679268,"ticker":"TUSK","title":"MAMMOTH ENERGY SERVICES, INC."},"4126":{"cik_str":1646109,"ticker":"FDEU","title":"FIRST TRUST DYNAMIC EUROPE EQUITY INCOME FUND"},"4127":{"cik_str":1846750,"ticker":"TCOA","title":"Trajectory Alpha Acquisition Corp."},"4128":{"cik_str":1637866,"ticker":"DGWR","title":"Deep Green Waste & Recycling, Inc."},"4129":{"cik_str":1739410,"ticker":"RLYB","title":"Rallybio Corp"},"4130":{"cik_str":886158,"ticker":"BBBYQ","title":"BED BATH & BEYOND INC"},"4131":{"cik_str":1603454,"ticker":"CELC","title":"Celcuity Inc."},"4132":{"cik_str":1857190,"ticker":"NYXH","title":"Nyxoah SA"},"4133":{"cik_str":1426332,"ticker":"NGM","title":"NGM BIOPHARMACEUTICALS INC"},"4134":{"cik_str":1350869,"ticker":"GLO","title":"Clough Global Opportunities Fund"},"4135":{"cik_str":1845459,"ticker":"GFOR","title":"Graf Acquisition Corp. IV"},"4136":{"cik_str":1847064,"ticker":"CLBR","title":"Colombier Acquisition Corp."},"4137":{"cik_str":1270131,"ticker":"SCD","title":"LMP CAPITAL & INCOME FUND INC."},"4138":{"cik_str":1848731,"ticker":"GLASF","title":"Glass House Brands Inc."},"4139":{"cik_str":1547341,"ticker":"CTR","title":"ClearBridge MLP & Midstream Total Return Fund Inc."},"4140":{"cik_str":1540013,"ticker":"QIPT","title":"Quipt Home Medical Corp."},"4141":{"cik_str":1071899,"ticker":"MUE","title":"BLACKROCK MUNIHOLDINGS QUALITY FUND II, INC."},"4142":{"cik_str":946394,"ticker":"ELLO","title":"Ellomay Capital Ltd."},"4143":{"cik_str":1852353,"ticker":"DC","title":"Dakota Gold Corp."},"4144":{"cik_str":809559,"ticker":"TSI","title":"TCW STRATEGIC INCOME FUND INC"},"4145":{"cik_str":1745114,"ticker":"MOLN","title":"MOLECULAR PARTNERS AG"},"4146":{"cik_str":1498612,"ticker":"PSF","title":"Cohen & Steers Select Preferred & Income Fund, Inc."},"4147":{"cik_str":1158289,"ticker":"JRS","title":"NUVEEN REAL ESTATE INCOME FUND"},"4148":{"cik_str":1604868,"ticker":"GRWG","title":"GrowGeneration Corp."},"4149":{"cik_str":1874071,"ticker":"PDLB","title":"Ponce Financial Group, Inc."},"4150":{"cik_str":1851908,"ticker":"BSAQ","title":"Black Spade Acquisition Co"},"4151":{"cik_str":1649744,"ticker":"DLTH","title":"DULUTH HOLDINGS INC."},"4152":{"cik_str":1838987,"ticker":"FACT","title":"Freedom Acquisition I Corp."},"4153":{"cik_str":1781730,"ticker":"TCBX","title":"Third Coast Bancshares, Inc."},"4154":{"cik_str":1616156,"ticker":"WEWA","title":"WEWARDS, INC."},"4155":{"cik_str":1835597,"ticker":"PEPG","title":"PepGen Inc."},"4156":{"cik_str":1637715,"ticker":"RPHM","title":"Reneo Pharmaceuticals, Inc."},"4157":{"cik_str":1132105,"ticker":"SPWH","title":"SPORTSMAN'S WAREHOUSE HOLDINGS, INC."},"4158":{"cik_str":911147,"ticker":"CNTY","title":"CENTURY CASINOS INC /CO/"},"4159":{"cik_str":1089907,"ticker":"SWKH","title":"SWK Holdings Corp"},"4160":{"cik_str":1593548,"ticker":"AGS","title":"PlayAGS, Inc."},"4161":{"cik_str":836147,"ticker":"MBCN","title":"MIDDLEFIELD BANC CORP"},"4162":{"cik_str":1420520,"ticker":"ATOM","title":"Atomera Inc"},"4163":{"cik_str":1072627,"ticker":"KFS","title":"KINGSWAY FINANCIAL SERVICES INC"},"4164":{"cik_str":1479615,"ticker":"SLN","title":"Silence Therapeutics plc"},"4165":{"cik_str":1580864,"ticker":"VRM","title":"Vroom, Inc."},"4166":{"cik_str":1285786,"ticker":"EMX","title":"EMX Royalty Corp"},"4167":{"cik_str":1666071,"ticker":"CDLX","title":"Cardlytics, Inc."},"4168":{"cik_str":1874218,"ticker":"RCAC","title":"Revelstone Capital Acquisition Corp."},"4169":{"cik_str":1651561,"ticker":"TRHC","title":"Tabula Rasa HealthCare, Inc."},"4170":{"cik_str":1725033,"ticker":"XYF","title":"X Financial"},"4171":{"cik_str":1423869,"ticker":"PCB","title":"PCB BANCORP"},"4172":{"cik_str":1821329,"ticker":"PRPC","title":"CC Neuberger Principal Holdings III"},"4173":{"cik_str":1792267,"ticker":"BNR","title":"Burning Rock Biotech Ltd"},"4174":{"cik_str":801961,"ticker":"MFM","title":"MFS MUNICIPAL INCOME TRUST"},"4175":{"cik_str":1168455,"ticker":"PLBC","title":"PLUMAS BANCORP"},"4176":{"cik_str":1072725,"ticker":"GDRZF","title":"GOLD RESERVE INC"},"4177":{"cik_str":1481792,"ticker":"QUAD","title":"Quad/Graphics, Inc."},"4178":{"cik_str":1277250,"ticker":"CRGE","title":"Charge Enterprises, Inc."},"4179":{"cik_str":1524025,"ticker":"TLYS","title":"TILLY'S, INC."},"4180":{"cik_str":1046050,"ticker":"TSBK","title":"TIMBERLAND BANCORP INC"},"4181":{"cik_str":1775625,"ticker":"SDC","title":"SmileDirectClub, Inc."},"4182":{"cik_str":1842566,"ticker":"BYTS","title":"BYTE Acquisition Corp."},"4183":{"cik_str":1769804,"ticker":"AUGX","title":"Augmedix, Inc."},"4184":{"cik_str":1621227,"ticker":"ADAP","title":"Adaptimmune Therapeutics PLC"},"4185":{"cik_str":1854795,"ticker":"IRRX","title":"INTEGRATED RAIL & RESOURCES ACQUISITION CORP"},"4186":{"cik_str":1636289,"ticker":"ACV","title":"Virtus Diversified Income & Convertible Fund"},"4187":{"cik_str":1803901,"ticker":"TALK","title":"Talkspace, Inc."},"4188":{"cik_str":1325670,"ticker":"FRST","title":"Primis Financial Corp."},"4189":{"cik_str":1859686,"ticker":"SBXC","title":"SilverBox Corp III"},"4190":{"cik_str":885508,"ticker":"STRS","title":"STRATUS PROPERTIES INC"},"4191":{"cik_str":1540684,"ticker":"ATLX","title":"Atlas Lithium Corp"},"4192":{"cik_str":1733868,"ticker":"CNF","title":"CNFinance Holdings Ltd."},"4193":{"cik_str":1816581,"ticker":"OUST","title":"Ouster, Inc."},"4194":{"cik_str":1631761,"ticker":"YRD","title":"Yiren Digital Ltd."},"4195":{"cik_str":826154,"ticker":"ORRF","title":"ORRSTOWN FINANCIAL SERVICES INC"},"4196":{"cik_str":1431567,"ticker":"OVLY","title":"Oak Valley Bancorp"},"4197":{"cik_str":1412100,"ticker":"MHLD","title":"Maiden Holdings, Ltd."},"4198":{"cik_str":1915328,"ticker":"ISRL","title":"Israel Acquisitions Corp"},"4199":{"cik_str":894242,"ticker":"BKN","title":"BLACKROCK INVESTMENT QUALITY MUNICIPAL TRUST, INC."},"4200":{"cik_str":1286613,"ticker":"LINC","title":"LINCOLN EDUCATIONAL SERVICES CORP"},"4201":{"cik_str":1869974,"ticker":"OCEA","title":"Ocean Biomedical, Inc."},"4202":{"cik_str":1077688,"ticker":"HOFT","title":"HOOKER FURNISHINGS Corp"},"4203":{"cik_str":1328598,"ticker":"FXE","title":"Invesco CurrencyShares Euro Trust"},"4204":{"cik_str":1813756,"ticker":"WE","title":"WeWork Inc."},"4205":{"cik_str":1856653,"ticker":"BIOS","title":"BioPlus Acquisition Corp."},"4206":{"cik_str":1738906,"ticker":"YI","title":"111, Inc."},"4207":{"cik_str":1856725,"ticker":"RANI","title":"Rani Therapeutics Holdings, Inc."},"4208":{"cik_str":1597095,"ticker":"TOUR","title":"Tuniu Corp"},"4209":{"cik_str":1683541,"ticker":"ACB","title":"AURORA CANNABIS INC"},"4210":{"cik_str":1327273,"ticker":"LYRA","title":"Lyra Therapeutics, Inc."},"4211":{"cik_str":1901637,"ticker":"USCB","title":"USCB FINANCIAL HOLDINGS, INC."},"4212":{"cik_str":1730984,"ticker":"BCML","title":"BayCom Corp"},"4213":{"cik_str":1869601,"ticker":"EMCG","title":"Embrace Change Acquisition Corp."},"4214":{"cik_str":1385763,"ticker":"JCE","title":"Nuveen Core Equity Alpha Fund"},"4215":{"cik_str":859796,"ticker":"JOF","title":"JAPAN SMALLER CAPITALIZATION FUND INC"},"4216":{"cik_str":1315399,"ticker":"PKBK","title":"PARKE BANCORP, INC."},"4217":{"cik_str":1754226,"ticker":"OBT","title":"Orange County Bancorp, Inc. /DE/"},"4218":{"cik_str":1842939,"ticker":"ALPA","title":"ALPHA HEALTHCARE ACQUISITION CORP III"},"4219":{"cik_str":1493761,"ticker":"HEAR","title":"Turtle Beach Corp"},"4220":{"cik_str":1826000,"ticker":"LTCH","title":"Latch, Inc."},"4221":{"cik_str":1576885,"ticker":"ABOS","title":"Acumen Pharmaceuticals, Inc."},"4222":{"cik_str":1278211,"ticker":"LGI","title":"LAZARD GLOBAL TOTAL RETURN & INCOME FUND INC"},"4223":{"cik_str":1327318,"ticker":"TRUE","title":"TrueCar, Inc."},"4224":{"cik_str":1848756,"ticker":"PHYT","title":"Pyrophyte Acquisition Corp."},"4225":{"cik_str":1156388,"ticker":"BGFV","title":"BIG 5 SPORTING GOODS Corp"},"4226":{"cik_str":1090009,"ticker":"SFST","title":"SOUTHERN FIRST BANCSHARES INC"},"4227":{"cik_str":1776738,"ticker":"CCHWF","title":"Columbia Care Inc."},"4228":{"cik_str":1538716,"ticker":"OPRT","title":"Oportun Financial Corp"},"4229":{"cik_str":831547,"ticker":"SPPI","title":"SPECTRUM PHARMACEUTICALS INC"},"4230":{"cik_str":1502573,"ticker":"AFT","title":"Apollo Senior Floating Rate Fund Inc."},"4231":{"cik_str":1711570,"ticker":"UROY","title":"Uranium Royalty Corp."},"4232":{"cik_str":791908,"ticker":"XOMA","title":"XOMA Corp"},"4233":{"cik_str":1036044,"ticker":"INVE","title":"Identiv, Inc."},"4234":{"cik_str":1715768,"ticker":"TGAN","title":"Transphorm, Inc."},"4235":{"cik_str":1596961,"ticker":"RMBL","title":"RumbleOn, Inc."},"4236":{"cik_str":1801661,"ticker":"SKLZ","title":"Skillz Inc."},"4237":{"cik_str":1495320,"ticker":"VRA","title":"Vera Bradley, Inc."},"4238":{"cik_str":1766478,"ticker":"AOMR","title":"Angel Oak Mortgage REIT, Inc."},"4239":{"cik_str":1684144,"ticker":"ZOM","title":"Zomedica Corp."},"4240":{"cik_str":1870143,"ticker":"RCFA","title":"RCF Acquisition Corp."},"4241":{"cik_str":1953366,"ticker":"STHO","title":"Star Holdings"},"4242":{"cik_str":1680581,"ticker":"FULC","title":"Fulcrum Therapeutics, Inc."},"4243":{"cik_str":1774675,"ticker":"SKIL","title":"Skillsoft Corp."},"4244":{"cik_str":1129928,"ticker":"ONCY","title":"ONCOLYTICS BIOTECH INC"},"4245":{"cik_str":1337013,"ticker":"INFU","title":"InfuSystem Holdings, Inc"},"4246":{"cik_str":1534992,"ticker":"FRGI","title":"Fiesta Restaurant Group, Inc."},"4247":{"cik_str":883265,"ticker":"VTN","title":"Invesco Trust for Investment Grade New York Municipals"},"4248":{"cik_str":1009829,"ticker":"JAKK","title":"JAKKS PACIFIC INC"},"4249":{"cik_str":1550695,"ticker":"PFMT","title":"Performant Financial Corp"},"4250":{"cik_str":1894057,"ticker":"PPYA","title":"Papaya Growth Opportunity Corp. I"},"4251":{"cik_str":1565381,"ticker":"DMB","title":"BNY Mellon Municipal Bond Infrastructure Fund, Inc."},"4252":{"cik_str":1690639,"ticker":"VBNK","title":"VersaBank"},"4253":{"cik_str":320340,"ticker":"CCRD","title":"CoreCard Corp"},"4254":{"cik_str":1228454,"ticker":"BCBP","title":"BCB BANCORP INC"},"4255":{"cik_str":1606745,"ticker":"LTRPA","title":"Liberty TripAdvisor Holdings, Inc."},"4256":{"cik_str":1593773,"ticker":"KGNR","title":"AMJ Global Technology"},"4257":{"cik_str":1166258,"ticker":"PHT","title":"PIONEER HIGH INCOME FUND, INC."},"4258":{"cik_str":1852432,"ticker":"ARYE","title":"ARYA Sciences Acquisition Corp V"},"4259":{"cik_str":1923840,"ticker":"THRD","title":"Third Harmonic Bio, Inc."},"4260":{"cik_str":1506251,"ticker":"CTXR","title":"Citius Pharmaceuticals, Inc."},"4261":{"cik_str":1817229,"ticker":"VOR","title":"Vor Biopharma Inc."},"4262":{"cik_str":1612630,"ticker":"JYNT","title":"JOINT Corp"},"4263":{"cik_str":1061353,"ticker":"DHY","title":"CREDIT SUISSE HIGH YIELD BOND FUND"},"4264":{"cik_str":1600847,"ticker":"GMDA","title":"Gamida Cell Ltd."},"4265":{"cik_str":1785680,"ticker":"HYW","title":"Hywin Holdings Ltd."},"4266":{"cik_str":1069533,"ticker":"RGCO","title":"RGC RESOURCES INC"},"4267":{"cik_str":1896049,"ticker":"STIX","title":"Semantix, Inc."},"4268":{"cik_str":1010134,"ticker":"ICTSF","title":"ICTS INTERNATIONAL N V"},"4269":{"cik_str":1878897,"ticker":"DOUG","title":"Douglas Elliman Inc."},"4270":{"cik_str":1503802,"ticker":"KPTI","title":"Karyopharm Therapeutics Inc."},"4271":{"cik_str":1223026,"ticker":"MHI","title":"PIONEER MUNICIPAL HIGH INCOME FUND, INC."},"4272":{"cik_str":1847440,"ticker":"MITA","title":"Coliseum Acquisition Corp."},"4273":{"cik_str":1526119,"ticker":"VSTM","title":"Verastem, Inc."},"4274":{"cik_str":1856995,"ticker":"MCAA","title":"Mountain & Co. I Acquisition Corp."},"4275":{"cik_str":66901,"ticker":"EMP","title":"ENTERGY MISSISSIPPI, LLC"},"4276":{"cik_str":1678130,"ticker":"OPP","title":"RiverNorth/DoubleLine Strategic Opportunity Fund, Inc."},"4277":{"cik_str":854800,"ticker":"TIO","title":"Tingo Group, Inc."},"4278":{"cik_str":3545,"ticker":"ALCO","title":"ALICO, INC."},"4279":{"cik_str":1505732,"ticker":"BWFG","title":"Bankwell Financial Group, Inc."},"4280":{"cik_str":1411690,"ticker":"BNGO","title":"Bionano Genomics, Inc."},"4281":{"cik_str":71508,"ticker":"ENJ","title":"ENTERGY NEW ORLEANS, LLC"},"4282":{"cik_str":1388320,"ticker":"ATNM","title":"Actinium Pharmaceuticals, Inc."},"4283":{"cik_str":850261,"ticker":"SRNEQ","title":"Sorrento Therapeutics, Inc."},"4284":{"cik_str":318673,"ticker":"SNFCA","title":"SECURITY NATIONAL FINANCIAL CORP"},"4285":{"cik_str":1178839,"ticker":"NBH","title":"NEUBERGER BERMAN MUNICIPAL FUND INC."},"4286":{"cik_str":897421,"ticker":"NPV","title":"NUVEEN VIRGINIA QUALITY MUNICIPAL INCOME FUND"},"4287":{"cik_str":1653909,"ticker":"BIRD","title":"Allbirds, Inc."},"4288":{"cik_str":1254370,"ticker":"WIA","title":"WESTERN ASSET INFLATION-LINKED INCOME FUND"},"4289":{"cik_str":887396,"ticker":"EP","title":"EMPIRE PETROLEUM CORP"},"4290":{"cik_str":1438423,"ticker":"MRAM","title":"EVERSPIN TECHNOLOGIES INC"},"4291":{"cik_str":728387,"ticker":"CATX","title":"Perspective Therapeutics, Inc."},"4292":{"cik_str":1506184,"ticker":"IMMP","title":"IMMUTEP Ltd"},"4293":{"cik_str":1549966,"ticker":"SAMG","title":"Silvercrest Asset Management Group Inc."},"4294":{"cik_str":1857662,"ticker":"APMI","title":"AxonPrime Infrastructure Acquisition Corp"},"4295":{"cik_str":1502377,"ticker":"CTGO","title":"Contango ORE, Inc."},"4296":{"cik_str":1622148,"ticker":"HGLB","title":"HIGHLAND GLOBAL ALLOCATION FUND"},"4297":{"cik_str":1302084,"ticker":"NSRCF","title":"NextSource Materials Inc."},"4298":{"cik_str":1170300,"ticker":"PCK","title":"PIMCO CALIFORNIA MUNICIPAL INCOME FUND II"},"4299":{"cik_str":1563696,"ticker":"ETX","title":"Eaton Vance Municipal Income 2028 Term Trust"},"4300":{"cik_str":1675644,"ticker":"FVCB","title":"FVCBankcorp, Inc."},"4301":{"cik_str":1519505,"ticker":"HIE","title":"Miller/Howard High Income Equity Fund"},"4302":{"cik_str":1372807,"ticker":"PTMN","title":"Portman Ridge Finance Corp"},"4303":{"cik_str":1026655,"ticker":"CMT","title":"CORE MOLDING TECHNOLOGIES INC"},"4304":{"cik_str":1811530,"ticker":"NXMH","title":"Next Meats Holdings, Inc."},"4305":{"cik_str":1490281,"ticker":"GRPN","title":"Groupon, Inc."},"4306":{"cik_str":1383084,"ticker":"DBB","title":"INVESCO DB BASE METALS FUND"},"4307":{"cik_str":1258943,"ticker":"MAV","title":"PIONEER MUNICIPAL HIGH INCOME ADVANTAGE FUND, INC."},"4308":{"cik_str":1758530,"ticker":"SY","title":"So-Young International Inc."},"4309":{"cik_str":806279,"ticker":"CVLY","title":"CODORUS VALLEY BANCORP INC"},"4310":{"cik_str":1401521,"ticker":"UIHC","title":"UNITED INSURANCE HOLDINGS CORP."},"4311":{"cik_str":701719,"ticker":"ELA","title":"Envela Corp"},"4312":{"cik_str":1914818,"ticker":"STBX","title":"Starbox Group Holdings Ltd."},"4313":{"cik_str":1930510,"ticker":"VCIG","title":"VCI Global Ltd"},"4314":{"cik_str":1851266,"ticker":"LFAC","title":"LF Capital Acquisition Corp. II"},"4315":{"cik_str":320121,"ticker":"TLS","title":"TELOS CORP"},"4316":{"cik_str":1852767,"ticker":"GLTA","title":"Galata Acquisition Corp."},"4317":{"cik_str":1386067,"ticker":"EOD","title":"ALLSPRING GLOBAL DIVIDEND OPPORTUNITY FUND"},"4318":{"cik_str":1273441,"ticker":"GTE","title":"GRAN TIERRA ENERGY INC."},"4319":{"cik_str":1602409,"ticker":"FNGR","title":"FingerMotion, Inc."},"4320":{"cik_str":1499832,"ticker":"TSQ","title":"Townsquare Media, Inc."},"4321":{"cik_str":1140411,"ticker":"PCQ","title":"PIMCO CALIFORNIA MUNICIPAL INCOME FUND"},"4322":{"cik_str":809708,"ticker":"EMF","title":"TEMPLETON EMERGING MARKETS FUND"},"4323":{"cik_str":913341,"ticker":"CFFI","title":"C & F FINANCIAL CORP"},"4324":{"cik_str":1851484,"ticker":"TENK","title":"TenX Keane Acquisition"},"4325":{"cik_str":1649752,"ticker":"NMG","title":"Nouveau Monde Graphite Inc."},"4326":{"cik_str":880807,"ticker":"AMSC","title":"AMERICAN SUPERCONDUCTOR CORP /DE/"},"4327":{"cik_str":1401708,"ticker":"NSTG","title":"NanoString Technologies Inc"},"4328":{"cik_str":1850119,"ticker":"IPSC","title":"Century Therapeutics, Inc."},"4329":{"cik_str":1000209,"ticker":"MFIN","title":"MEDALLION FINANCIAL CORP"},"4330":{"cik_str":1490286,"ticker":"NTG","title":"Tortoise Midstream Energy Fund, Inc."},"4331":{"cik_str":1757064,"ticker":"GNFT","title":"Genfit S.A."},"4332":{"cik_str":1689731,"ticker":"SSBK","title":"Southern States Bancshares, Inc."},"4333":{"cik_str":1086303,"ticker":"HYBT","title":"Heyu Biological Technology Corp"},"4334":{"cik_str":1526697,"ticker":"AIF","title":"Apollo Tactical Income Fund Inc."},"4335":{"cik_str":1119769,"ticker":"CRNT","title":"CERAGON NETWORKS LTD"},"4336":{"cik_str":33488,"ticker":"ESCA","title":"ESCALADE INC"},"4337":{"cik_str":50471,"ticker":"PCYG","title":"PARK CITY GROUP INC"},"4338":{"cik_str":1472341,"ticker":"GDO","title":"WESTERN ASSET GLOBAL CORPORATE DEFINED OPPORTUNITY FUND INC."},"4339":{"cik_str":1425287,"ticker":"WKHS","title":"Workhorse Group Inc."},"4340":{"cik_str":1504379,"ticker":"CSTE","title":"Caesarstone Ltd."},"4341":{"cik_str":1874315,"ticker":"SATL","title":"Satellogic Inc."},"4342":{"cik_str":1528115,"ticker":"ANNX","title":"Annexon, Inc."},"4343":{"cik_str":719135,"ticker":"APYX","title":"Apyx Medical Corp"},"4344":{"cik_str":1864290,"ticker":"MIO","title":"Pioneer Municipal High Income Opportunities Fund, Inc."},"4345":{"cik_str":1871638,"ticker":"BRKH","title":"BurTech Acquisition Corp."},"4346":{"cik_str":921671,"ticker":"GGT","title":"GABELLI MULTIMEDIA TRUST INC."},"4347":{"cik_str":763563,"ticker":"CHMG","title":"CHEMUNG FINANCIAL CORP"},"4348":{"cik_str":1259429,"ticker":"OXSQ","title":"Oxford Square Capital Corp."},"4349":{"cik_str":1200375,"ticker":"CDXS","title":"CODEXIS, INC."},"4350":{"cik_str":1437283,"ticker":"RPMT","title":"REGO PAYMENT ARCHITECTURES, INC."},"4351":{"cik_str":1682811,"ticker":"IHIT","title":"INVESCO HIGH INCOME 2023 TARGET TERM FUND"},"4352":{"cik_str":1393883,"ticker":"DHX","title":"DHI GROUP, INC."},"4353":{"cik_str":1760854,"ticker":"NBTX","title":"Nanobiotix S.A."},"4354":{"cik_str":1127371,"ticker":"CVCY","title":"CENTRAL VALLEY COMMUNITY BANCORP"},"4355":{"cik_str":1554859,"ticker":"SMLR","title":"Semler Scientific, Inc."},"4356":{"cik_str":1650101,"ticker":"ATXG","title":"ADDENTAX GROUP CORP."},"4357":{"cik_str":1886799,"ticker":"BGXX","title":"Bright Green Corp"},"4358":{"cik_str":1166834,"ticker":"SHLT","title":"SHL TELEMEDICINE LTD"},"4359":{"cik_str":1093428,"ticker":"NSL","title":"NUVEEN SENIOR INCOME FUND"},"4360":{"cik_str":716605,"ticker":"PWOD","title":"PENNS WOODS BANCORP INC"},"4361":{"cik_str":1645666,"ticker":"KZR","title":"Kezar Life Sciences, Inc."},"4362":{"cik_str":1656634,"ticker":"GRTS","title":"Gritstone bio, Inc."},"4363":{"cik_str":1327607,"ticker":"MYFW","title":"First Western Financial Inc"},"4364":{"cik_str":1007587,"ticker":"KVHI","title":"KVH INDUSTRIES INC DE"},"4365":{"cik_str":1158114,"ticker":"AAOI","title":"APPLIED OPTOELECTRONICS, INC."},"4366":{"cik_str":1588871,"ticker":"OAKC","title":"Oakworth Capital, Inc."},"4367":{"cik_str":1462586,"ticker":"IGI","title":"Western Asset Investment Grade Defined Opportunity Trust Inc."},"4368":{"cik_str":1512931,"ticker":"MRCC","title":"MONROE CAPITAL Corp"},"4369":{"cik_str":1410098,"ticker":"CRMD","title":"CorMedix Inc."},"4370":{"cik_str":1437424,"ticker":"POET","title":"POET TECHNOLOGIES INC."},"4371":{"cik_str":1840225,"ticker":"FSNB","title":"Fusion Acquisition Corp. II"},"4372":{"cik_str":1532383,"ticker":"VNJA","title":"VANJIA CORP"},"4373":{"cik_str":1564180,"ticker":"KNOP","title":"KNOT Offshore Partners LP"},"4374":{"cik_str":1274792,"ticker":"MACK","title":"MERRIMACK PHARMACEUTICALS INC"},"4375":{"cik_str":1181504,"ticker":"PZC","title":"PIMCO CALIFORNIA MUNICIPAL INCOME FUND III"},"4376":{"cik_str":1665988,"ticker":"BVS","title":"Bioventus Inc."},"4377":{"cik_str":1836100,"ticker":"PUCK","title":"Goal Acquisitions Corp."},"4378":{"cik_str":1285550,"ticker":"CLPT","title":"ClearPoint Neuro, Inc."},"4379":{"cik_str":1841209,"ticker":"HOLO","title":"MicroCloud Hologram Inc."},"4380":{"cik_str":1313275,"ticker":"BCOV","title":"BRIGHTCOVE INC"},"4381":{"cik_str":1838359,"ticker":"RGTI","title":"Rigetti Computing, Inc."},"4382":{"cik_str":1824185,"ticker":"HHLA","title":"HH&L Acquisition Co."},"4383":{"cik_str":350868,"ticker":"ITI","title":"ITERIS, INC."},"4384":{"cik_str":1771910,"ticker":"ADCT","title":"ADC Therapeutics SA"},"4385":{"cik_str":1449488,"ticker":"CCLP","title":"CSI Compressco LP"},"4386":{"cik_str":1828937,"ticker":"FOA","title":"Finance of America Companies Inc."},"4387":{"cik_str":803164,"ticker":"COFS","title":"CHOICEONE FINANCIAL SERVICES INC"},"4388":{"cik_str":1030997,"ticker":"WILC","title":"G WILLI FOOD INTERNATIONAL LTD"},"4389":{"cik_str":1434647,"ticker":"ZVRA","title":"ZEVRA THERAPEUTICS, INC."},"4390":{"cik_str":1946216,"ticker":"SYT","title":"SYLA Technologies Co., Ltd."},"4391":{"cik_str":1453687,"ticker":"SELB","title":"SELECTA BIOSCIENCES INC"},"4392":{"cik_str":1630113,"ticker":"BTCY","title":"BIOTRICITY INC."},"4393":{"cik_str":1913971,"ticker":"BPRN","title":"Princeton Bancorp, Inc."},"4394":{"cik_str":1701541,"ticker":"BDTX","title":"Black Diamond Therapeutics, Inc."},"4395":{"cik_str":1271554,"ticker":"ERLFF","title":"Entree Resources Ltd."},"4396":{"cik_str":56868,"ticker":"PNRG","title":"PRIMEENERGY RESOURCES CORP"},"4397":{"cik_str":796534,"ticker":"NKSH","title":"NATIONAL BANKSHARES INC"},"4398":{"cik_str":1693577,"ticker":"MNSB","title":"MainStreet Bancshares, Inc."},"4399":{"cik_str":5094,"ticker":"VBF","title":"Invesco Bond Fund"},"4400":{"cik_str":1572334,"ticker":"VABK","title":"Virginia National Bankshares Corp"},"4401":{"cik_str":1815620,"ticker":"OPT","title":"Opthea Ltd"},"4402":{"cik_str":1557265,"ticker":"PIIVX","title":"Private Shares Fund"},"4403":{"cik_str":1626878,"ticker":"XBIT","title":"XBiotech Inc."},"4404":{"cik_str":833040,"ticker":"FT","title":"FRANKLIN UNIVERSAL TRUST"},"4405":{"cik_str":1326190,"ticker":"ALT","title":"Altimmune, Inc."},"4406":{"cik_str":1177162,"ticker":"ENX","title":"EATON VANCE NEW YORK MUNICIPAL BOND FUND"},"4407":{"cik_str":1819810,"ticker":"RDW","title":"Redwire Corp"},"4408":{"cik_str":1704299,"ticker":"TEAF","title":"ECOFIN SUSTAINABLE & SOCIAL IMPACT TERM FUND"},"4409":{"cik_str":1689548,"ticker":"PRAX","title":"Praxis Precision Medicines, Inc."},"4410":{"cik_str":1860871,"ticker":"LGST","title":"Semper Paratus Acquisition Corp"},"4411":{"cik_str":1098009,"ticker":"AAGH","title":"America Great Health"},"4412":{"cik_str":1926599,"ticker":"TDAC","title":"Translational Development Acquisition Corp."},"4413":{"cik_str":1517022,"ticker":"AKBA","title":"Akebia Therapeutics, Inc."},"4414":{"cik_str":842717,"ticker":"BRBS","title":"BLUE RIDGE BANKSHARES, INC."},"4415":{"cik_str":1391437,"ticker":"GRX","title":"Gabelli Healthcare & WellnessRx Trust"},"4416":{"cik_str":1549922,"ticker":"SMLP","title":"Summit Midstream Partners, LP"},"4417":{"cik_str":1862490,"ticker":"DCFC","title":"Tritium DCFC Ltd"},"4418":{"cik_str":1401040,"ticker":"DMAC","title":"DiaMedica Therapeutics Inc."},"4419":{"cik_str":1486298,"ticker":"BSL","title":"Blackstone Senior Floating Rate 2027 Term Fund"},"4420":{"cik_str":1842386,"ticker":"VAQC","title":"Vector Acquisition Corp II"},"4421":{"cik_str":30125,"ticker":"INSI","title":"Insight Select Income Fund"},"4422":{"cik_str":1756594,"ticker":"IVA","title":"Inventiva S.A."},"4423":{"cik_str":1711754,"ticker":"INMB","title":"Inmune Bio, Inc."},"4424":{"cik_str":1862935,"ticker":"IFIN","title":"InFinT Acquisition Corp"},"4425":{"cik_str":1228509,"ticker":"EHI","title":"WESTERN ASSET GLOBAL HIGH INCOME FUND INC."},"4426":{"cik_str":711669,"ticker":"CBAN","title":"COLONY BANKCORP INC"},"4427":{"cik_str":1881551,"ticker":"NUBI","title":"Nubia Brand International Corp."},"4428":{"cik_str":1504678,"ticker":"LOOP","title":"Loop Industries, Inc."},"4429":{"cik_str":1833909,"ticker":"AGAC","title":"African Gold Acquisition Corp"},"4430":{"cik_str":1859690,"ticker":"ARQQ","title":"Arqit Quantum Inc."},"4431":{"cik_str":1753706,"ticker":"FREE","title":"Whole Earth Brands, Inc."},"4432":{"cik_str":816332,"ticker":"LYBC","title":"LYONS BANCORP INC"},"4433":{"cik_str":1411688,"ticker":"TCS","title":"Container Store Group, Inc."},"4434":{"cik_str":1488039,"ticker":"ATOS","title":"ATOSSA THERAPEUTICS, INC."},"4435":{"cik_str":1530979,"ticker":"HNST","title":"Honest Company, Inc."},"4436":{"cik_str":1866782,"ticker":"CDRO","title":"Codere Online Luxembourg, S.A."},"4437":{"cik_str":1328919,"ticker":"GASS","title":"StealthGas Inc."},"4438":{"cik_str":842517,"ticker":"ISBA","title":"ISABELLA BANK Corp"},"4439":{"cik_str":1074902,"ticker":"LCNB","title":"LCNB CORP"},"4440":{"cik_str":1398172,"ticker":"NASO","title":"Naples Soap Company, Inc."},"4441":{"cik_str":18255,"ticker":"CATO","title":"CATO CORP"},"4442":{"cik_str":1927719,"ticker":"CRGO","title":"Freightos Ltd"},"4443":{"cik_str":1046995,"ticker":"EMAN","title":"EMAGIN CORP"},"4444":{"cik_str":1132651,"ticker":"ATLO","title":"AMES NATIONAL CORP"},"4445":{"cik_str":1614806,"ticker":"AJX","title":"Great Ajax Corp."},"4446":{"cik_str":704440,"ticker":"KRMD","title":"KORU Medical Systems, Inc."},"4447":{"cik_str":1422142,"ticker":"AADI","title":"Aadi Bioscience, Inc."},"4448":{"cik_str":1880438,"ticker":"ANTX","title":"AN2 Therapeutics, Inc."},"4449":{"cik_str":1725293,"ticker":"GRIN","title":"Grindrod Shipping Holdings Ltd."},"4450":{"cik_str":1836295,"ticker":"RTGN","title":"RetinalGenix Technologies Inc."},"4451":{"cik_str":1842916,"ticker":"FMIV","title":"Forum Merger IV Corp"},"4452":{"cik_str":1832168,"ticker":"LBPH","title":"Longboard Pharmaceuticals, Inc."},"4453":{"cik_str":1719714,"ticker":"MREO","title":"Mereo Biopharma Group plc"},"4454":{"cik_str":1738827,"ticker":"KLXE","title":"KLX Energy Services Holdings, Inc."},"4455":{"cik_str":1462056,"ticker":"BLZE","title":"Backblaze, Inc."},"4456":{"cik_str":1057861,"ticker":"DHF","title":"BNY MELLON HIGH YIELD STRATEGIES FUND"},"4457":{"cik_str":1721741,"ticker":"LAZY","title":"Lazydays Holdings, Inc."},"4458":{"cik_str":1889450,"ticker":"FTII","title":"FutureTech II Acquisition Corp."},"4459":{"cik_str":1870925,"ticker":"ENTF","title":"Enterprise 4.0 Technology Acquisition Corp"},"4460":{"cik_str":1831481,"ticker":"FLME","title":"Flame Acquisition Corp."},"4461":{"cik_str":1947244,"ticker":"USGO","title":"U.S. GoldMining Inc."},"4462":{"cik_str":918251,"ticker":"MPAA","title":"MOTORCAR PARTS OF AMERICA INC"},"4463":{"cik_str":1780531,"ticker":"OCFT","title":"ONECONNECT FINANCIAL TECHNOLOGY CO., LTD."},"4464":{"cik_str":1815436,"ticker":"AHI","title":"Advanced Health Intelligence Ltd"},"4465":{"cik_str":1590715,"ticker":"AREC","title":"American Resources Corp"},"4466":{"cik_str":1822250,"ticker":"WISH","title":"ContextLogic Inc."},"4467":{"cik_str":1848948,"ticker":"VCXB","title":"10X Capital Venture Acquisition Corp. III"},"4468":{"cik_str":818850,"ticker":"NNY","title":"NUVEEN NEW YORK MUNICIPAL VALUE FUND"},"4469":{"cik_str":1873441,"ticker":"BCSA","title":"Blockchain Coinvestors Acquisition Corp. I"},"4470":{"cik_str":1835378,"ticker":"CTV","title":"Innovid Corp."},"4471":{"cik_str":1725123,"ticker":"CANG","title":"Cango Inc."},"4472":{"cik_str":1841610,"ticker":"ROSS","title":"Ross Acquisition Corp II"},"4473":{"cik_str":1701809,"ticker":"CBH","title":"Virtus Convertible & Income 2024 Target Term Fund"},"4474":{"cik_str":1383057,"ticker":"DBP","title":"Invesco DB Precious Metals Fund"},"4475":{"cik_str":1734520,"ticker":"ALYA","title":"Alithya Group inc"},"4476":{"cik_str":1832950,"ticker":"KRNL","title":"Kernel Group Holdings, Inc."},"4477":{"cik_str":1865701,"ticker":"KYCH","title":"Keyarch Acquisition Corp"},"4478":{"cik_str":866273,"ticker":"MTRX","title":"MATRIX SERVICE CO"},"4479":{"cik_str":1865111,"ticker":"ALSA","title":"Alpha Star Acquisition Corp"},"4480":{"cik_str":1382230,"ticker":"ESSA","title":"ESSA Bancorp, Inc."},"4481":{"cik_str":352825,"ticker":"FSTR","title":"FOSTER L B CO"},"4482":{"cik_str":1275158,"ticker":"NDLS","title":"NOODLES & Co"},"4483":{"cik_str":100378,"ticker":"TWIN","title":"TWIN DISC INC"},"4484":{"cik_str":1501756,"ticker":"ADVM","title":"Adverum Biotechnologies, Inc."},"4485":{"cik_str":895456,"ticker":"RCKY","title":"ROCKY BRANDS, INC."},"4486":{"cik_str":1346830,"ticker":"CARA","title":"Cara Therapeutics, Inc."},"4487":{"cik_str":1864032,"ticker":"ADRT","title":"Ault Disruptive Technologies Corp"},"4488":{"cik_str":784539,"ticker":"EACO","title":"EACO CORP"},"4489":{"cik_str":1904501,"ticker":"ZTEK","title":"Zentek Ltd."},"4490":{"cik_str":1289868,"ticker":"MCN","title":"Madison Covered Call & Equity Strategy Fund"},"4491":{"cik_str":1814423,"ticker":"IH","title":"iHuman Inc."},"4492":{"cik_str":1604464,"ticker":"ATRA","title":"Atara Biotherapeutics, Inc."},"4493":{"cik_str":825345,"ticker":"HYB","title":"NEW AMERICA HIGH INCOME FUND INC"},"4494":{"cik_str":830622,"ticker":"PIM","title":"PUTNAM MASTER INTERMEDIATE INCOME TRUST"},"4495":{"cik_str":1157762,"ticker":"CAAS","title":"CHINA AUTOMOTIVE SYSTEMS INC"},"4496":{"cik_str":700841,"ticker":"RCMT","title":"RCM TECHNOLOGIES, INC."},"4497":{"cik_str":948320,"ticker":"LFMD","title":"LifeMD, Inc."},"4498":{"cik_str":1175151,"ticker":"CTSO","title":"Cytosorbents Corp"},"4499":{"cik_str":1031316,"ticker":"FSP","title":"FRANKLIN STREET PROPERTIES CORP /MA/"},"4500":{"cik_str":1341170,"ticker":"ESEA","title":"EUROSEAS LTD."},"4501":{"cik_str":95574,"ticker":"SGC","title":"SUPERIOR GROUP OF COMPANIES, INC."},"4502":{"cik_str":1538847,"ticker":"GLDG","title":"GoldMining Inc."},"4503":{"cik_str":891038,"ticker":"MPA","title":"BLACKROCK MUNIYIELD PENNSYLVANIA QUALITY FUND"},"4504":{"cik_str":858706,"ticker":"GF","title":"NEW GERMANY FUND INC"},"4505":{"cik_str":810766,"ticker":"CIK","title":"CREDIT SUISSE ASSET MANAGEMENT INCOME FUND INC"},"4506":{"cik_str":1162194,"ticker":"LAB","title":"STANDARD BIOTOOLS INC."},"4507":{"cik_str":1900402,"ticker":"EVGR","title":"Evergreen Corp"},"4508":{"cik_str":1833769,"ticker":"HYPR","title":"Hyperfine, Inc."},"4509":{"cik_str":1682220,"ticker":"SACH","title":"Sachem Capital Corp."},"4510":{"cik_str":1467808,"ticker":"CORBF","title":"Global Cord Blood Corp"},"4511":{"cik_str":1815776,"ticker":"GRPH","title":"Graphite Bio, Inc."},"4512":{"cik_str":1877557,"ticker":"WEL","title":"Integrated Wellness Acquisition Corp"},"4513":{"cik_str":1472091,"ticker":"PDSB","title":"PDS Biotechnology Corp"},"4514":{"cik_str":1788028,"ticker":"JSPR","title":"Jasper Therapeutics, Inc."},"4515":{"cik_str":1901886,"ticker":"AFAR","title":"Aura Fat Projects Acquisition Corp"},"4516":{"cik_str":1114925,"ticker":"LTRX","title":"LANTRONIX INC"},"4517":{"cik_str":1856948,"ticker":"CHEA","title":"Chenghe Acquisition Co."},"4518":{"cik_str":1885998,"ticker":"ROCL","title":"Roth CH Acquisition V Co."},"4519":{"cik_str":1512228,"ticker":"NB","title":"NIOCORP DEVELOPMENTS LTD"},"4520":{"cik_str":1840877,"ticker":"ANZU","title":"Anzu Special Acquisition Corp I"},"4521":{"cik_str":1580149,"ticker":"BIVI","title":"BIOVIE INC."},"4522":{"cik_str":1832871,"ticker":"BMN","title":"BlackRock 2037 Municipal Target Term Trust"},"4523":{"cik_str":1051627,"ticker":"AXTI","title":"AXT INC"},"4524":{"cik_str":1452477,"ticker":"SEVN","title":"Seven Hills Realty Trust"},"4525":{"cik_str":1401667,"ticker":"PBYI","title":"PUMA BIOTECHNOLOGY, INC."},"4526":{"cik_str":1852407,"ticker":"FEXD","title":"Fintech Ecosystem Development Corp."},"4527":{"cik_str":1779474,"ticker":"MAPS","title":"WM TECHNOLOGY, INC."},"4528":{"cik_str":1828377,"ticker":"FTCO","title":"Fortitude Gold Corp"},"4529":{"cik_str":1645460,"ticker":"CUE","title":"Cue Biopharma, Inc."},"4530":{"cik_str":1600641,"ticker":"DIBS","title":"1stdibs.com, Inc."},"4531":{"cik_str":1572621,"ticker":"AENZ","title":"AENZA S.A.A."},"4532":{"cik_str":1491419,"ticker":"LVO","title":"LiveOne, Inc."},"4533":{"cik_str":1477081,"ticker":"KGEIF","title":"Kolibri Global Energy Inc."},"4534":{"cik_str":1865191,"ticker":"TGVC","title":"TG Venture Acquisition Corp."},"4535":{"cik_str":1126741,"ticker":"GSIT","title":"GSI TECHNOLOGY INC"},"4536":{"cik_str":1818331,"ticker":"WGS","title":"GeneDx Holdings Corp."},"4537":{"cik_str":1778784,"ticker":"PVBC","title":"Provident Bancorp, Inc. /MD/"},"4538":{"cik_str":1922858,"ticker":"EFHT","title":"EF Hutton Acquisition Corp I"},"4539":{"cik_str":1245648,"ticker":"FLC","title":"FLAHERTY & CRUMRINE TOTAL RETURN FUND INC"},"4540":{"cik_str":1417802,"ticker":"IDE","title":"Voya Infrastructure, Industrials & Materials Fund"},"4541":{"cik_str":1845368,"ticker":"HCMA","title":"HCM Acquisition Corp"},"4542":{"cik_str":1661460,"ticker":"PSTX","title":"Poseida Therapeutics, Inc."},"4543":{"cik_str":1839175,"ticker":"MBAC","title":"M3-Brigade Acquisition II Corp."},"4544":{"cik_str":1371782,"ticker":"MVO","title":"MV Oil Trust"},"4545":{"cik_str":1318484,"ticker":"CTRN","title":"Citi Trends Inc"},"4546":{"cik_str":1824153,"ticker":"LHC","title":"Leo Holdings Corp. II"},"4547":{"cik_str":1522767,"ticker":"MRMD","title":"MARIMED INC."},"4548":{"cik_str":1960262,"ticker":"LUDG","title":"LUDWIG ENTERPRISES, INC."},"4549":{"cik_str":1615165,"ticker":"VERI","title":"Veritone, Inc."},"4550":{"cik_str":1261166,"ticker":"NRO","title":"NEUBERGER BERMAN REAL ESTATE SECURITIES INCOME FUND INC"},"4551":{"cik_str":1888654,"ticker":"FEAM","title":"5E Advanced Materials, Inc."},"4552":{"cik_str":1850453,"ticker":"MYNA","title":"Mynaric AG"},"4553":{"cik_str":1643988,"ticker":"LPTV","title":"Loop Media, Inc."},"4554":{"cik_str":1305168,"ticker":"ARC","title":"ARC DOCUMENT SOLUTIONS, INC."},"4555":{"cik_str":1835814,"ticker":"NSTD","title":"Northern Star Investment Corp. IV"},"4556":{"cik_str":1838108,"ticker":"RMGC","title":"RMG Acquisition Corp. III"},"4557":{"cik_str":1504234,"ticker":"BGX","title":"Blackstone Long-Short Credit Income Fund"},"4558":{"cik_str":1391127,"ticker":"EGIO","title":"Edgio, Inc."},"4559":{"cik_str":1811115,"ticker":"RNLX","title":"Renalytix plc"},"4560":{"cik_str":1487610,"ticker":"NHS","title":"Neuberger Berman High Yield Strategies Fund Inc."},"4561":{"cik_str":883569,"ticker":"FOSL","title":"Fossil Group, Inc."},"4562":{"cik_str":1017491,"ticker":"SEEL","title":"SEELOS THERAPEUTICS, INC."},"4563":{"cik_str":1232582,"ticker":"AHT","title":"ASHFORD HOSPITALITY TRUST INC"},"4564":{"cik_str":1835817,"ticker":"NSTC","title":"Northern Star Investment Corp. III"},"4565":{"cik_str":41719,"ticker":"GLT","title":"Glatfelter Corp"},"4566":{"cik_str":100726,"ticker":"UFI","title":"UNIFI INC"},"4567":{"cik_str":1895251,"ticker":"SGTM","title":"Sustainable Green Team, Ltd."},"4568":{"cik_str":1576914,"ticker":"MIXT","title":"MiX Telematics Ltd"},"4569":{"cik_str":1372299,"ticker":"OCGN","title":"Ocugen, Inc."},"4570":{"cik_str":1788882,"ticker":"ROOT","title":"Root, Inc."},"4571":{"cik_str":1876431,"ticker":"PRE","title":"Prenetics Global Ltd"},"4572":{"cik_str":1006837,"ticker":"VATE","title":"INNOVATE Corp."},"4573":{"cik_str":1443575,"ticker":"AVBH","title":"Avidbank Holdings, Inc."},"4574":{"cik_str":1059213,"ticker":"DNIF","title":"DIVIDEND & INCOME FUND"},"4575":{"cik_str":891532,"ticker":"PESI","title":"PERMA FIX ENVIRONMENTAL SERVICES INC"},"4576":{"cik_str":1884516,"ticker":"DTI","title":"Drilling Tools International Corp"},"4577":{"cik_str":716314,"ticker":"GHM","title":"GRAHAM CORP"},"4578":{"cik_str":1937737,"ticker":"MLEC","title":"Moolec Science SA"},"4579":{"cik_str":1562151,"ticker":"CWGL","title":"Crimson Wine Group, Ltd"},"4580":{"cik_str":818033,"ticker":"HRTX","title":"HERON THERAPEUTICS, INC. /DE/"},"4581":{"cik_str":1563880,"ticker":"TRVI","title":"Trevi Therapeutics, Inc."},"4582":{"cik_str":830487,"ticker":"MHF","title":"WESTERN ASSET MUNICIPAL HIGH INCOME FUND INC."},"4583":{"cik_str":1589420,"ticker":"FPL","title":"FIRST TRUST NEW OPPORTUNITIES MLP & ENERGY FUND"},"4584":{"cik_str":1903145,"ticker":"GRRR","title":"Gorilla Technology Group Inc."},"4585":{"cik_str":785557,"ticker":"DLHC","title":"DLH Holdings Corp."},"4586":{"cik_str":1784535,"ticker":"PRCH","title":"Porch Group, Inc."},"4587":{"cik_str":36840,"ticker":"FREVS","title":"FIRST REAL ESTATE INVESTMENT TRUST OF NEW JERSEY"},"4588":{"cik_str":1176309,"ticker":"ORMP","title":"ORAMED PHARMACEUTICALS INC."},"4589":{"cik_str":1505413,"ticker":"VOC","title":"VOC Energy Trust"},"4590":{"cik_str":1672688,"ticker":"ABSI","title":"Absci Corp"},"4591":{"cik_str":1697851,"ticker":"REKR","title":"Rekor Systems, Inc."},"4592":{"cik_str":803016,"ticker":"CFNB","title":"CALIFORNIA FIRST LEASING CORP"},"4593":{"cik_str":1879814,"ticker":"TLGY","title":"TLGY ACQUISITION CORP"},"4594":{"cik_str":1813814,"ticker":"MNMD","title":"Mind Medicine (MindMed) Inc."},"4595":{"cik_str":1844642,"ticker":"LCA","title":"Landcadia Holdings IV, Inc."},"4596":{"cik_str":1095052,"ticker":"PLG","title":"PLATINUM GROUP METALS LTD"},"4597":{"cik_str":1016838,"ticker":"RDCM","title":"RADCOM LTD"},"4598":{"cik_str":1797768,"ticker":"KNTE","title":"Kinnate Biopharma Inc."},"4599":{"cik_str":1826892,"ticker":"BCAB","title":"BioAtla, Inc."},"4600":{"cik_str":315374,"ticker":"HURC","title":"HURCO COMPANIES INC"},"4601":{"cik_str":1709164,"ticker":"HBB","title":"Hamilton Beach Brands Holding Co"},"4602":{"cik_str":1532286,"ticker":"NINE","title":"Nine Energy Service, Inc."},"4603":{"cik_str":874396,"ticker":"LCUT","title":"LIFETIME BRANDS, INC"},"4604":{"cik_str":1398805,"ticker":"BEEM","title":"Beam Global"},"4605":{"cik_str":849145,"ticker":"HGBL","title":"Heritage Global Inc."},"4606":{"cik_str":1931691,"ticker":"MOBV","title":"Mobiv Acquisition Corp"},"4607":{"cik_str":901243,"ticker":"MUA","title":"BLACKROCK MUNIASSETS FUND, INC."},"4608":{"cik_str":1840317,"ticker":"PLMI","title":"Plum Acquisition Corp. I"},"4609":{"cik_str":1827506,"ticker":"TALS","title":"Talaris Therapeutics, Inc."},"4610":{"cik_str":1662579,"ticker":"CCCC","title":"C4 Therapeutics, Inc."},"4611":{"cik_str":931584,"ticker":"USAP","title":"UNIVERSAL STAINLESS & ALLOY PRODUCTS INC"},"4612":{"cik_str":1353538,"ticker":"APGT","title":"Appgate, Inc."},"4613":{"cik_str":1126115,"ticker":"LZGI","title":"LZG INTERNATIONAL, INC."},"4614":{"cik_str":1813452,"ticker":"PLNHF","title":"Planet 13 Holdings Inc."},"4615":{"cik_str":1843100,"ticker":"SAMA","title":"Schultze Special Purpose Acquisition Corp. II"},"4616":{"cik_str":1060219,"ticker":"SAL","title":"SALISBURY BANCORP, INC."},"4617":{"cik_str":1468327,"ticker":"RENT","title":"Rent the Runway, Inc."},"4618":{"cik_str":10329,"ticker":"BSET","title":"BASSETT FURNITURE INDUSTRIES INC"},"4619":{"cik_str":1827871,"ticker":"TLGA","title":"TLG Acquisition One Corp."},"4620":{"cik_str":831655,"ticker":"MPV","title":"BARINGS PARTICIPATION INVESTORS"},"4621":{"cik_str":808219,"ticker":"MHGU","title":"MERITAGE HOSPITALITY GROUP INC"},"4622":{"cik_str":842518,"ticker":"EVBN","title":"EVANS BANCORP INC"},"4623":{"cik_str":1504545,"ticker":"BWG","title":"BrandywineGLOBAL-Global Income Opportunities Fund Inc"},"4624":{"cik_str":1523836,"ticker":"LITB","title":"LightInTheBox Holding Co., Ltd."},"4625":{"cik_str":1370416,"ticker":"WPRT","title":"WESTPORT FUEL SYSTEMS INC."},"4626":{"cik_str":1383395,"ticker":"SQNS","title":"SEQUANS COMMUNICATIONS"},"4627":{"cik_str":1562463,"ticker":"INBK","title":"First Internet Bancorp"},"4628":{"cik_str":1870833,"ticker":"RMMZ","title":"RiverNorth Managed Duration Municipal Income Fund II, Inc."},"4629":{"cik_str":1819175,"ticker":"PIAI","title":"Prime Impact Acquisition I"},"4630":{"cik_str":1840161,"ticker":"FRXB","title":"Forest Road Acquisition Corp. II"},"4631":{"cik_str":1494650,"ticker":"OPTN","title":"OptiNose, Inc."},"4632":{"cik_str":1377757,"ticker":"GAU","title":"Galiano Gold Inc."},"4633":{"cik_str":1860657,"ticker":"ALLR","title":"Allarity Therapeutics, Inc."},"4634":{"cik_str":1434868,"ticker":"ESPR","title":"Esperion Therapeutics, Inc."},"4635":{"cik_str":1834336,"ticker":"FSRX","title":"Finserv Acquisition Corp. II"},"4636":{"cik_str":1912582,"ticker":"FLFV","title":"Feutune Light Acquisition Corp"},"4637":{"cik_str":1680367,"ticker":"STTK","title":"Shattuck Labs, Inc."},"4638":{"cik_str":1819498,"ticker":"TWLV","title":"Twelve Seas Investment Co. II"},"4639":{"cik_str":1803914,"ticker":"PLBY","title":"PLBY Group, Inc."},"4640":{"cik_str":1487918,"ticker":"OFS","title":"OFS Capital Corp"},"4641":{"cik_str":101984,"ticker":"UEIC","title":"UNIVERSAL ELECTRONICS INC"},"4642":{"cik_str":1374690,"ticker":"LRMR","title":"Larimar Therapeutics, Inc."},"4643":{"cik_str":1408534,"ticker":"FGBI","title":"First Guaranty Bancshares, Inc."},"4644":{"cik_str":1839824,"ticker":"FZT","title":"FAST Acquisition Corp. II"},"4645":{"cik_str":904112,"ticker":"MSD","title":"MORGAN STANLEY EMERGING MARKETS DEBT FUND INC"},"4646":{"cik_str":1752036,"ticker":"CALB","title":"California BanCorp"},"4647":{"cik_str":1612940,"ticker":"PRQR","title":"ProQR Therapeutics N.V."},"4648":{"cik_str":1753162,"ticker":"FTHM","title":"Fathom Holdings Inc."},"4649":{"cik_str":1353613,"ticker":"FXY","title":"Invesco CurrencyShares Japanese Yen Trust"},"4650":{"cik_str":932781,"ticker":"FCCO","title":"FIRST COMMUNITY CORP /SC/"},"4651":{"cik_str":1951222,"ticker":"LAES","title":"SEALSQ Corp"},"4652":{"cik_str":1287498,"ticker":"EVG","title":"Eaton Vance Short Duration Diversified Income Fund"},"4653":{"cik_str":1851961,"ticker":"CNDB","title":"Concord Acquisition Corp III"},"4654":{"cik_str":1729997,"ticker":"GDLC","title":"Grayscale Digital Large Cap Fund LLC"},"4655":{"cik_str":881787,"ticker":"CRT","title":"CROSS TIMBERS ROYALTY TRUST"},"4656":{"cik_str":1133311,"ticker":"TZOO","title":"TRAVELZOO"},"4657":{"cik_str":1826889,"ticker":"BODY","title":"Beachbody Company, Inc."},"4658":{"cik_str":1442236,"ticker":"QRHC","title":"Quest Resource Holding Corp"},"4659":{"cik_str":1157647,"ticker":"WNEB","title":"Western New England Bancorp, Inc."},"4660":{"cik_str":839122,"ticker":"DMF","title":"BNY MELLON MUNICIPAL INCOME, INC."},"4661":{"cik_str":795800,"ticker":"PNRLF","title":"Premium Nickel Resources Ltd."},"4662":{"cik_str":1616788,"ticker":"LGYV","title":"LEGACY VENTURES INTERNATIONAL INC."},"4663":{"cik_str":1332943,"ticker":"IGA","title":"Voya Global Advantage & Premium Opportunity Fund"},"4664":{"cik_str":1343793,"ticker":"BTA","title":"BlackRock Long-Term Municipal Advantage Trust"},"4665":{"cik_str":928465,"ticker":"DIT","title":"AMCON DISTRIBUTING CO"},"4666":{"cik_str":1879297,"ticker":"LGVC","title":"LAMF Global Ventures Corp. I"},"4667":{"cik_str":1823896,"ticker":"GFX","title":"Golden Falcon Acquisition Corp."},"4668":{"cik_str":1742927,"ticker":"RVPH","title":"REVIVA PHARMACEUTICALS HOLDINGS, INC."},"4669":{"cik_str":720500,"ticker":"ASYS","title":"AMTECH SYSTEMS INC"},"4670":{"cik_str":897802,"ticker":"SPE","title":"SPECIAL OPPORTUNITIES FUND, INC."},"4671":{"cik_str":1383062,"ticker":"DBE","title":"Invesco DB Energy Fund"},"4672":{"cik_str":1834032,"ticker":"CSTA","title":"Constellation Acquisition Corp I"},"4673":{"cik_str":1510593,"ticker":"XNET","title":"Xunlei Ltd"},"4674":{"cik_str":886136,"ticker":"SGA","title":"SAGA COMMUNICATIONS INC"},"4675":{"cik_str":1828588,"ticker":"HNVR","title":"Hanover Bancorp, Inc. /NY"},"4676":{"cik_str":1832038,"ticker":"IVVD","title":"Invivyd, Inc."},"4677":{"cik_str":1821812,"ticker":"SCAQ","title":"Stratim Cloud Acquisition Corp."},"4678":{"cik_str":1496443,"ticker":"PAYS","title":"Paysign, Inc."},"4679":{"cik_str":1828376,"ticker":"WMPN","title":"William Penn Bancorporation"},"4680":{"cik_str":1209028,"ticker":"AAIC","title":"Arlington Asset Investment Corp."},"4681":{"cik_str":1620737,"ticker":"OGI","title":"ORGANIGRAM HOLDINGS INC."},"4682":{"cik_str":1563577,"ticker":"GRTX","title":"Galera Therapeutics, Inc."},"4683":{"cik_str":1844452,"ticker":"LUNR","title":"Intuitive Machines, Inc."},"4684":{"cik_str":1496749,"ticker":"HEQ","title":"John Hancock Hedged Equity & Income Fund"},"4685":{"cik_str":1746278,"ticker":"GOLQ","title":"GoLogiq, Inc."},"4686":{"cik_str":924719,"ticker":"SMID","title":"SMITH MIDLAND CORP"},"4687":{"cik_str":882508,"ticker":"QUIK","title":"QUICKLOGIC Corp"},"4688":{"cik_str":1641631,"ticker":"XAIR","title":"Beyond Air, Inc."},"4689":{"cik_str":1560241,"ticker":"GTHX","title":"G1 Therapeutics, Inc."},"4690":{"cik_str":1946399,"ticker":"YS","title":"YS Biopharma Co., Ltd."},"4691":{"cik_str":1898601,"ticker":"IPX","title":"IPERIONX Ltd"},"4692":{"cik_str":1722010,"ticker":"OPBK","title":"OP Bancorp"},"4693":{"cik_str":1815753,"ticker":"TBCP","title":"Thunder Bridge Capital Partners III Inc."},"4694":{"cik_str":1743344,"ticker":"PYR","title":"PyroGenesis Canada Inc."},"4695":{"cik_str":1173643,"ticker":"TRX","title":"TRX GOLD Corp"},"4696":{"cik_str":1838163,"ticker":"DHCA","title":"DHC Acquisition Corp."},"4697":{"cik_str":1477845,"ticker":"ANVS","title":"Annovis Bio, Inc."},"4698":{"cik_str":1838814,"ticker":"JUN","title":"Juniper II Corp."},"4699":{"cik_str":893847,"ticker":"HWBK","title":"HAWTHORN BANCSHARES, INC."},"4700":{"cik_str":1922331,"ticker":"GLST","title":"Global Star Acquisition Inc."},"4701":{"cik_str":1838513,"ticker":"GATE","title":"Marblegate Acquisition Corp."},"4702":{"cik_str":1178711,"ticker":"PRTK","title":"Paratek Pharmaceuticals, Inc."},"4703":{"cik_str":1410738,"ticker":"VPLM","title":"Voip-pal.com Inc"},"4704":{"cik_str":1866547,"ticker":"GMFI","title":"Aetherium Acquisition Corp"},"4705":{"cik_str":1848020,"ticker":"BMAC","title":"Black Mountain Acquisition Corp."},"4706":{"cik_str":1728117,"ticker":"GOSS","title":"Gossamer Bio, Inc."},"4707":{"cik_str":868578,"ticker":"PFD","title":"FLAHERTY & CRUMRINE PREFERRED & INCOME FUND INC"},"4708":{"cik_str":1101680,"ticker":"DZSI","title":"DZS INC."},"4709":{"cik_str":1866226,"ticker":"WTMA","title":"Welsbach Technology Metals Acquisition Corp."},"4710":{"cik_str":1820875,"ticker":"CXAI","title":"CXApp Inc."},"4711":{"cik_str":1516899,"ticker":"YTRA","title":"Yatra Online, Inc."},"4712":{"cik_str":1767837,"ticker":"RMBI","title":"Richmond Mutual Bancorporation, Inc."},"4713":{"cik_str":759866,"ticker":"JHS","title":"JOHN HANCOCK INCOME SECURITIES TRUST"},"4714":{"cik_str":723646,"ticker":"FRAF","title":"FRANKLIN FINANCIAL SERVICES CORP /PA/"},"4715":{"cik_str":892992,"ticker":"NAZ","title":"NUVEEN ARIZONA QUALITY MUNICIPAL INCOME FUND"},"4716":{"cik_str":1696195,"ticker":"KNIT","title":"KINETIC GROUP INC."},"4717":{"cik_str":72633,"ticker":"NRT","title":"NORTH EUROPEAN OIL ROYALTY TRUST"},"4718":{"cik_str":1836274,"ticker":"ACAH","title":"Atlantic Coastal Acquisition Corp."},"4719":{"cik_str":1571776,"ticker":"CHMI","title":"Cherry Hill Mortgage Investment Corp"},"4720":{"cik_str":836690,"ticker":"ISSC","title":"INNOVATIVE SOLUTIONS & SUPPORT INC"},"4721":{"cik_str":1164771,"ticker":"NAK","title":"NORTHERN DYNASTY MINERALS LTD"},"4722":{"cik_str":868278,"ticker":"PRPH","title":"ProPhase Labs, Inc."},"4723":{"cik_str":1753368,"ticker":"SANG","title":"Sangoma Technologies Corp"},"4724":{"cik_str":1006281,"ticker":"PLX","title":"Protalix BioTherapeutics, Inc."},"4725":{"cik_str":1289340,"ticker":"STXS","title":"Stereotaxis, Inc."},"4726":{"cik_str":1799788,"ticker":"GLSI","title":"Greenwich LifeSciences, Inc."},"4727":{"cik_str":720154,"ticker":"NOTV","title":"Inotiv, Inc."},"4728":{"cik_str":1756390,"ticker":"AAWH","title":"Ascend Wellness Holdings, Inc."},"4729":{"cik_str":87050,"ticker":"NEON","title":"Neonode Inc."},"4730":{"cik_str":1726126,"ticker":"EPSN","title":"Epsilon Energy Ltd."},"4731":{"cik_str":895574,"ticker":"VFL","title":"Delaware Investments National Municipal Income Fund"},"4732":{"cik_str":832480,"ticker":"UTGN","title":"UTG INC"},"4733":{"cik_str":1514281,"ticker":"MITT","title":"AG Mortgage Investment Trust, Inc."},"4734":{"cik_str":1622996,"ticker":"ACBM","title":"ACRO BIOMEDICAL CO., LTD."},"4735":{"cik_str":1834518,"ticker":"NSTB","title":"Northern Star Investment Corp. II"},"4736":{"cik_str":1437226,"ticker":"MHH","title":"Mastech Digital, Inc."},"4737":{"cik_str":1550603,"ticker":"MLVF","title":"MALVERN BANCORP, INC."},"4738":{"cik_str":1914605,"ticker":"ECBK","title":"ECB Bancorp, Inc. /MD/"},"4739":{"cik_str":1854275,"ticker":"IOAC","title":"Innovative International Acquisition Corp."},"4740":{"cik_str":1163792,"ticker":"WEA","title":"WESTERN ASSET PREMIER BOND FUND"},"4741":{"cik_str":1084991,"ticker":"NGS","title":"NATURAL GAS SERVICES GROUP INC"},"4742":{"cik_str":1199004,"ticker":"FMN","title":"Federated Hermes Premier Municipal Income Fund"},"4743":{"cik_str":1895249,"ticker":"MCAC","title":"Monterey Capital Acquisition Corp"},"4744":{"cik_str":1888151,"ticker":"BCAN","title":"BYND CANNASOFT ENTERPRISES INC."},"4745":{"cik_str":24090,"ticker":"CIA","title":"CITIZENS, INC."},"4746":{"cik_str":1082038,"ticker":"DRRX","title":"DURECT CORP"},"4747":{"cik_str":1772720,"ticker":"SPRU","title":"SPRUCE POWER HOLDING CORP"},"4748":{"cik_str":1835972,"ticker":"ARRW","title":"Arrowroot Acquisition Corp."},"4749":{"cik_str":1840353,"ticker":"TRCA","title":"Twin Ridge Capital Acquisition Corp."},"4750":{"cik_str":1871745,"ticker":"DSAQ","title":"Direct Selling Acquisition Corp."},"4751":{"cik_str":1831979,"ticker":"GPAC","title":"Global Partner Acquisition Corp II"},"4752":{"cik_str":1869141,"ticker":"ACAQ","title":"Athena Consumer Acquisition Corp."},"4753":{"cik_str":1158420,"ticker":"GGE","title":"Green Giant Inc."},"4754":{"cik_str":1339970,"ticker":"LIFE","title":"aTYR PHARMA INC"},"4755":{"cik_str":1843121,"ticker":"OPA","title":"Magnum Opus Acquisition Ltd"},"4756":{"cik_str":1478102,"ticker":"DMO","title":"Western Asset Mortgage Opportunity Fund Inc."},"4757":{"cik_str":1578987,"ticker":"BANX","title":"ArrowMark Financial Corp."},"4758":{"cik_str":1837929,"ticker":"NPAB","title":"New Providence Acquisition Corp. II"},"4759":{"cik_str":1505155,"ticker":"UPLD","title":"Upland Software, Inc."},"4760":{"cik_str":1838672,"ticker":"ADTH","title":"AdTheorent Holding Company, Inc."},"4761":{"cik_str":1316463,"ticker":"GLQ","title":"Clough Global Equity Fund"},"4762":{"cik_str":1752372,"ticker":"EZOO","title":"Ezagoo Ltd"},"4763":{"cik_str":1107421,"ticker":"TCRT","title":"Alaunos Therapeutics, Inc."},"4764":{"cik_str":1829953,"ticker":"COOL","title":"Corner Growth Acquisition Corp."},"4765":{"cik_str":1633438,"ticker":"AZRE","title":"Azure Power Global Ltd"},"4766":{"cik_str":1518715,"ticker":"HMST","title":"HomeStreet, Inc."},"4767":{"cik_str":727207,"ticker":"AXDX","title":"Accelerate Diagnostics, Inc"},"4768":{"cik_str":23111,"ticker":"CTG","title":"COMPUTER TASK GROUP INC"},"4769":{"cik_str":1633932,"ticker":"EPIX","title":"ESSA Pharma Inc."},"4770":{"cik_str":1365767,"ticker":"ALLT","title":"Allot Ltd."},"4771":{"cik_str":1602658,"ticker":"ISTR","title":"Investar Holding Corp"},"4772":{"cik_str":1860514,"ticker":"USCT","title":"TKB Critical Technologies 1"},"4773":{"cik_str":1133869,"ticker":"CAPR","title":"CAPRICOR THERAPEUTICS, INC."},"4774":{"cik_str":876378,"ticker":"ASXC","title":"ASENSUS SURGICAL, INC."},"4775":{"cik_str":1823575,"ticker":"ZFOX","title":"ZeroFox Holdings, Inc."},"4776":{"cik_str":793040,"ticker":"ECF","title":"ELLSWORTH GROWTH & INCOME FUND LTD"},"4777":{"cik_str":1035976,"ticker":"FNCB","title":"FNCB Bancorp, Inc."},"4778":{"cik_str":1031029,"ticker":"SRT","title":"Startek, Inc."},"4779":{"cik_str":1501989,"ticker":"CTMX","title":"CytomX Therapeutics, Inc."},"4780":{"cik_str":1873875,"ticker":"IXHL","title":"Incannex Healthcare Ltd"},"4781":{"cik_str":1820302,"ticker":"BKKT","title":"Bakkt Holdings, Inc."},"4782":{"cik_str":908187,"ticker":"PCM","title":"PCM FUND, INC."},"4783":{"cik_str":1808997,"ticker":"AOUT","title":"American Outdoor Brands, Inc."},"4784":{"cik_str":1627281,"ticker":"CLLS","title":"Cellectis S.A."},"4785":{"cik_str":894671,"ticker":"OVBC","title":"OHIO VALLEY BANC CORP"},"4786":{"cik_str":1520358,"ticker":"MMMB","title":"MamaMancini's Holdings, Inc."},"4787":{"cik_str":1849548,"ticker":"ONYX","title":"Onyx Acquisition Co. I"},"4788":{"cik_str":1720580,"ticker":"ACET","title":"Adicet Bio, Inc."},"4789":{"cik_str":1620749,"ticker":"PHCI","title":"Panamera Holdings Corp"},"4790":{"cik_str":1509253,"ticker":"JSD","title":"Nuveen Short Duration Credit Opportunities Fund"},"4791":{"cik_str":1868573,"ticker":"APXI","title":"APx Acquisition Corp. I"},"4792":{"cik_str":1852633,"ticker":"BYN","title":"Banyan Acquisition Corp"},"4793":{"cik_str":1897245,"ticker":"ACAX","title":"Alset Capital Acquisition Corp."},"4794":{"cik_str":1396033,"ticker":"LL","title":"LL Flooring Holdings, Inc."},"4795":{"cik_str":748691,"ticker":"KF","title":"KOREA FUND INC"},"4796":{"cik_str":1907909,"ticker":"VOXR","title":"Vox Royalty Corp."},"4797":{"cik_str":1333835,"ticker":"CASA","title":"Casa Systems Inc"},"4798":{"cik_str":1883788,"ticker":"ATAK","title":"Aurora Technology Acquisition Corp."},"4799":{"cik_str":1137547,"ticker":"UBFO","title":"UNITED SECURITY BANCSHARES"},"4800":{"cik_str":1819395,"ticker":"SOND","title":"Sonder Holdings Inc."},"4801":{"cik_str":1386570,"ticker":"CDXC","title":"ChromaDex Corp."},"4802":{"cik_str":1913577,"ticker":"DECA","title":"Denali Capital Acquisition Corp."},"4803":{"cik_str":725363,"ticker":"CVM","title":"CEL SCI CORP"},"4804":{"cik_str":1856365,"ticker":"FINW","title":"Finwise Bancorp"},"4805":{"cik_str":845379,"ticker":"CHN","title":"CHINA FUND INC"},"4806":{"cik_str":1055726,"ticker":"INO","title":"INOVIO PHARMACEUTICALS, INC."},"4807":{"cik_str":1141688,"ticker":"LARK","title":"LANDMARK BANCORP INC"},"4808":{"cik_str":1462418,"ticker":"ASPS","title":"ALTISOURCE PORTFOLIO SOLUTIONS S.A."},"4809":{"cik_str":1829322,"ticker":"SZZL","title":"Sizzle Acquisition Corp."},"4810":{"cik_str":894351,"ticker":"MNP","title":"WESTERN ASSET MUNICIPAL PARTNERS FUND INC."},"4811":{"cik_str":1822711,"ticker":"PRDS","title":"PARDES BIOSCIENCES, INC."},"4812":{"cik_str":810943,"ticker":"PCF","title":"HIGH INCOME SECURITIES FUND"},"4813":{"cik_str":1769759,"ticker":"MGRM","title":"MONOGRAM ORTHOPAEDICS INC"},"4814":{"cik_str":1528287,"ticker":"NPCE","title":"NeuroPace Inc"},"4815":{"cik_str":315131,"ticker":"PHX","title":"PHX MINERALS INC."},"4816":{"cik_str":1853920,"ticker":"VBOC","title":"Viscogliosi Brothers Acquisition Corp"},"4817":{"cik_str":1826011,"ticker":"VII","title":"7GC & Co. Holdings Inc."},"4818":{"cik_str":1893219,"ticker":"ACAB","title":"Atlantic Coastal Acquisition Corp. II"},"4819":{"cik_str":1859310,"ticker":"MEOA","title":"Minority Equality Opportunities Acquisition Inc."},"4820":{"cik_str":1178253,"ticker":"SCYX","title":"SCYNEXIS INC"},"4821":{"cik_str":1509397,"ticker":"LOMLF","title":"Lion One Metals Ltd."},"4822":{"cik_str":1827899,"ticker":"CPTK","title":"Crown PropTech Acquisitions"},"4823":{"cik_str":1627854,"ticker":"DCF","title":"BNY Mellon Alcentra Global Credit Income 2024 Target Term Fund, Inc."},"4824":{"cik_str":1165002,"ticker":"WHG","title":"WESTWOOD HOLDINGS GROUP INC"},"4825":{"cik_str":1872302,"ticker":"NA","title":"Nano Labs Ltd"},"4826":{"cik_str":1783328,"ticker":"TCRX","title":"TScan Therapeutics, Inc."},"4827":{"cik_str":355019,"ticker":"FONR","title":"FONAR CORP"},"4828":{"cik_str":890119,"ticker":"NIM","title":"NUVEEN SELECT MATURITIES MUNICIPAL FUND"},"4829":{"cik_str":1787414,"ticker":"BSBK","title":"Bogota Financial Corp."},"4830":{"cik_str":1855457,"ticker":"KORE","title":"KORE Group Holdings, Inc."},"4831":{"cik_str":832090,"ticker":"PTRS","title":"PARTNERS BANCORP"},"4832":{"cik_str":1226616,"ticker":"MNOV","title":"MEDICINOVA INC"},"4833":{"cik_str":1705012,"ticker":"FAT","title":"Fat Brands, Inc"},"4834":{"cik_str":1449794,"ticker":"EMYB","title":"Embassy Bancorp, Inc."},"4835":{"cik_str":1641229,"ticker":"FFMGF","title":"First Mining Gold Corp."},"4836":{"cik_str":1648636,"ticker":"TRLEF","title":"Trillion Energy International Inc."},"4837":{"cik_str":1672909,"ticker":"CPHC","title":"Canterbury Park Holding Corp"},"4838":{"cik_str":1408443,"ticker":"MIST","title":"Milestone Pharmaceuticals Inc."},"4839":{"cik_str":1822791,"ticker":"CLNN","title":"Clene Inc."},"4840":{"cik_str":1753673,"ticker":"SJ","title":"Scienjoy Holding Corp"},"4841":{"cik_str":1909417,"ticker":"SLNA","title":"SELINA HOSPITALITY PLC"},"4842":{"cik_str":1865468,"ticker":"KACL","title":"Kairous Acquisition Corp. Ltd"},"4843":{"cik_str":1851612,"ticker":"BHAC","title":"Crixus BH3 Acquisition Co"},"4844":{"cik_str":1750735,"ticker":"MRBK","title":"Meridian Corp"},"4845":{"cik_str":1253689,"ticker":"GLYC","title":"GLYCOMIMETICS INC"},"4846":{"cik_str":779336,"ticker":"IAF","title":"ABRDN AUSTRALIA EQUITY FUND, INC."},"4847":{"cik_str":1822912,"ticker":"ADER","title":"26 Capital Acquisition Corp."},"4848":{"cik_str":759828,"ticker":"JHI","title":"JOHN HANCOCK INVESTORS TRUST"},"4849":{"cik_str":75398,"ticker":"PAI","title":"Western Asset Investment Grade Income Fund Inc."},"4850":{"cik_str":1750149,"ticker":"IKT","title":"Inhibikase Therapeutics, Inc."},"4851":{"cik_str":1556727,"ticker":"FNWB","title":"First Northwest Bancorp"},"4852":{"cik_str":1732406,"ticker":"LTCN","title":"Grayscale Litecoin Trust (LTC)"},"4853":{"cik_str":1855555,"ticker":"ENER","title":"ACCRETION ACQUISITION CORP."},"4854":{"cik_str":1645873,"ticker":"MDV","title":"MODIV INC."},"4855":{"cik_str":31107,"ticker":"EML","title":"EASTERN CO"},"4856":{"cik_str":845606,"ticker":"CXE","title":"MFS HIGH INCOME MUNICIPAL TRUST"},"4857":{"cik_str":1390352,"ticker":"ORRCF","title":"OROCO RESOURCE CORP"},"4858":{"cik_str":908311,"ticker":"CMCT","title":"Creative Media & Community Trust Corp"},"4859":{"cik_str":1838716,"ticker":"GNTA","title":"Genenta Science S.p.A."},"4860":{"cik_str":737875,"ticker":"FKYS","title":"FIRST KEYSTONE CORP"},"4861":{"cik_str":918541,"ticker":"NNBR","title":"NN INC"},"4862":{"cik_str":1852019,"ticker":"IXAQ","title":"IX Acquisition Corp."},"4863":{"cik_str":1302028,"ticker":"MNTX","title":"Manitex International, Inc."},"4864":{"cik_str":72444,"ticker":"VXRT","title":"Vaxart, Inc."},"4865":{"cik_str":1852749,"ticker":"XFIN","title":"ExcelFin Acquisition Corp."},"4866":{"cik_str":1752474,"ticker":"KLDI","title":"KLDiscovery Inc."},"4867":{"cik_str":1533998,"ticker":"DRIO","title":"DarioHealth Corp."},"4868":{"cik_str":1620463,"ticker":"ATHA","title":"Athira Pharma, Inc."},"4869":{"cik_str":1853044,"ticker":"WWAC","title":"Worldwide Webb Acquisition Corp."},"4870":{"cik_str":1875496,"ticker":"YGF","title":"YanGuFang International Group Co., Ltd"},"4871":{"cik_str":813623,"ticker":"SWZ","title":"SWISS HELVETIA FUND, INC."},"4872":{"cik_str":1447051,"ticker":"TBNK","title":"Territorial Bancorp Inc."},"4873":{"cik_str":1830795,"ticker":"QFTA","title":"Quantum FinTech Acquisition Corp"},"4874":{"cik_str":1643154,"ticker":"ITHUF","title":"iANTHUS CAPITAL HOLDINGS, INC."},"4875":{"cik_str":1051343,"ticker":"CWBC","title":"COMMUNITY WEST BANCSHARES /"},"4876":{"cik_str":1566243,"ticker":"ARAT","title":"Arax Holdings Corp"},"4877":{"cik_str":1598665,"ticker":"HRTG","title":"Heritage Insurance Holdings, Inc."},"4878":{"cik_str":906709,"ticker":"NKTR","title":"NEKTAR THERAPEUTICS"},"4879":{"cik_str":1354866,"ticker":"BYRN","title":"Byrna Technologies Inc."},"4880":{"cik_str":810332,"ticker":"MESA","title":"MESA AIR GROUP INC"},"4881":{"cik_str":1893311,"ticker":"LUXH","title":"LUXURBAN HOTELS INC."},"4882":{"cik_str":1839132,"ticker":"MVLA","title":"Movella Holdings Inc."},"4883":{"cik_str":1346610,"ticker":"SOS","title":"SOS Ltd"},"4884":{"cik_str":1305767,"ticker":"PHD","title":"Pioneer Floating Rate Fund, Inc."},"4885":{"cik_str":1881462,"ticker":"SHAP","title":"Spree Acquisition Corp. 1 Ltd"},"4886":{"cik_str":1132979,"ticker":"FRCB","title":"FIRST REPUBLIC BANK"},"4887":{"cik_str":882300,"ticker":"SBI","title":"WESTERN ASSET INTERMEDIATE MUNI FUND INC."},"4888":{"cik_str":1527762,"ticker":"MFH","title":"Mercurity Fintech Holding Inc."},"4889":{"cik_str":1041368,"ticker":"RVSB","title":"RIVERVIEW BANCORP INC"},"4890":{"cik_str":1825473,"ticker":"PRSR","title":"Prospector Capital Corp."},"4891":{"cik_str":1117171,"ticker":"CBAT","title":"CBAK Energy Technology, Inc."},"4892":{"cik_str":798081,"ticker":"LAKE","title":"LAKELAND INDUSTRIES INC"},"4893":{"cik_str":1906133,"ticker":"FGMC","title":"FG Merger Corp."},"4894":{"cik_str":1748137,"ticker":"NEOV","title":"NeoVolta Inc."},"4895":{"cik_str":93314,"ticker":"VNRX","title":"VOLITIONRX LTD"},"4896":{"cik_str":1783875,"ticker":"FFNTF","title":"4Front Ventures Corp."},"4897":{"cik_str":863894,"ticker":"VERU","title":"VERU INC."},"4898":{"cik_str":1095315,"ticker":"PFSW","title":"PFSWEB INC"},"4899":{"cik_str":1610820,"ticker":"BCTX","title":"BriaCell Therapeutics Corp."},"4900":{"cik_str":1785530,"ticker":"HOWL","title":"Werewolf Therapeutics, Inc."},"4901":{"cik_str":8146,"ticker":"ALOT","title":"AstroNova, Inc."},"4902":{"cik_str":1839610,"ticker":"IPVF","title":"InterPrivate III Financial Partners Inc."},"4903":{"cik_str":1787400,"ticker":"NKTX","title":"Nkarta, Inc."},"4904":{"cik_str":1843988,"ticker":"TWOA","title":"two"},"4905":{"cik_str":1865506,"ticker":"ESAC","title":"ESGEN Acquisition Corp"},"4906":{"cik_str":16859,"ticker":"SRL","title":"Scully Royalty Ltd."},"4907":{"cik_str":14177,"ticker":"BRID","title":"BRIDGFORD FOODS CORP"},"4908":{"cik_str":1478454,"ticker":"EBMT","title":"Eagle Bancorp Montana, Inc."},"4909":{"cik_str":1853436,"ticker":"MPRA","title":"Mercato Partners Acquisition Corp"},"4910":{"cik_str":949858,"ticker":"ACHV","title":"ACHIEVE LIFE SCIENCES, INC."},"4911":{"cik_str":1093672,"ticker":"PEBK","title":"PEOPLES BANCORP OF NORTH CAROLINA INC"},"4912":{"cik_str":731122,"ticker":"CCFN","title":"CCFNB BANCORP INC"},"4913":{"cik_str":1605301,"ticker":"CBFV","title":"CB Financial Services, Inc."},"4914":{"cik_str":880641,"ticker":"EFSI","title":"EAGLE FINANCIAL SERVICES INC"},"4915":{"cik_str":1814974,"ticker":"BBXIA","title":"BBX Capital, Inc."},"4916":{"cik_str":1865248,"ticker":"CMCA","title":"Capitalworks Emerging Markets Acquisition Corp"},"4917":{"cik_str":1117480,"ticker":"CMRX","title":"CHIMERIX INC"},"4918":{"cik_str":1539190,"ticker":"SPPP","title":"SPROTT PHYSICAL PLATINUM & PALLADIUM TRUST"},"4919":{"cik_str":1839998,"ticker":"DTOC","title":"Digital Transformation Opportunities Corp."},"4920":{"cik_str":1070296,"ticker":"FCAP","title":"FIRST CAPITAL INC"},"4921":{"cik_str":1774170,"ticker":"PWFL","title":"PowerFleet, Inc."},"4922":{"cik_str":95552,"ticker":"SUP","title":"SUPERIOR INDUSTRIES INTERNATIONAL INC"},"4923":{"cik_str":1831006,"ticker":"BLUA","title":"BlueRiver Acquisition Corp."},"4924":{"cik_str":1626971,"ticker":"CRVS","title":"Corvus Pharmaceuticals, Inc."},"4925":{"cik_str":1832161,"ticker":"KEYR","title":"KeyStar Corp."},"4926":{"cik_str":894871,"ticker":"AREN","title":"Arena Group Holdings, Inc."},"4927":{"cik_str":811922,"ticker":"MGF","title":"MFS GOVERNMENT MARKETS INCOME TRUST"},"4928":{"cik_str":1585855,"ticker":"GGZ","title":"Gabelli Global Small & Mid Cap Value Trust"},"4929":{"cik_str":882071,"ticker":"PFO","title":"Flaherty & Crumrine PREFERRED & INCOME OPPORTUNITY FUND INC"},"4930":{"cik_str":1814329,"ticker":"ASTR","title":"Astra Space, Inc."},"4931":{"cik_str":1837248,"ticker":"SDAC","title":"Sustainable Development Acquisition I Corp."},"4932":{"cik_str":1746967,"ticker":"RMI","title":"RiverNorth Opportunistic Municipal Income Fund, Inc."},"4933":{"cik_str":880417,"ticker":"CSBB","title":"CSB Bancorp, Inc."},"4934":{"cik_str":1780201,"ticker":"LVLU","title":"Lulu's Fashion Lounge Holdings, Inc."},"4935":{"cik_str":1821606,"ticker":"TMKR","title":"Tastemaker Acquisition Corp."},"4936":{"cik_str":1849820,"ticker":"KITT","title":"Nauticus Robotics, Inc."},"4937":{"cik_str":1502557,"ticker":"KPEA","title":"Kun Peng International Ltd."},"4938":{"cik_str":1834755,"ticker":"SLAC","title":"Social Leverage Acquisition Corp I"},"4939":{"cik_str":804563,"ticker":"BMBN","title":"BENCHMARK BANKSHARES INC"},"4940":{"cik_str":1741830,"ticker":"KRON","title":"Kronos Bio, Inc."},"4941":{"cik_str":1855485,"ticker":"FLAG","title":"First Light Acquisition Group, Inc."},"4942":{"cik_str":1593984,"ticker":"MDWD","title":"MediWound Ltd."},"4943":{"cik_str":924383,"ticker":"GNSS","title":"Genasys Inc."},"4944":{"cik_str":719402,"ticker":"FXNC","title":"FIRST NATIONAL CORP /VA/"},"4945":{"cik_str":1282648,"ticker":"BATL","title":"BATTALION OIL CORP"},"4946":{"cik_str":1280776,"ticker":"IMUX","title":"IMMUNIC, INC."},"4947":{"cik_str":1517767,"ticker":"VCIF","title":"Vertical Capital Income Fund"},"4948":{"cik_str":1474903,"ticker":"BGSF","title":"BGSF, INC."},"4949":{"cik_str":1114927,"ticker":"FNRN","title":"FIRST NORTHERN COMMUNITY BANCORP"},"4950":{"cik_str":1401564,"ticker":"FFNW","title":"First Financial Northwest, Inc."},"4951":{"cik_str":1303942,"ticker":"BFIN","title":"BankFinancial CORP"},"4952":{"cik_str":709283,"ticker":"QMCO","title":"QUANTUM CORP /DE/"},"4953":{"cik_str":1392694,"ticker":"SURG","title":"SurgePays, Inc."},"4954":{"cik_str":706863,"ticker":"UNB","title":"UNION BANKSHARES INC"},"4955":{"cik_str":1707502,"ticker":"SLDB","title":"Solid Biosciences Inc."},"4956":{"cik_str":1800392,"ticker":"MLGO","title":"MicroAlgo Inc."},"4957":{"cik_str":1838293,"ticker":"CHAA","title":"Catcha Investment Corp"},"4958":{"cik_str":1817868,"ticker":"JAQC","title":"Jupiter Acquisition Corp"},"4959":{"cik_str":65596,"ticker":"SIEB","title":"SIEBERT FINANCIAL CORP"},"4960":{"cik_str":1844505,"ticker":"GIA","title":"GigCapital5, Inc."},"4961":{"cik_str":1610618,"ticker":"CDTX","title":"Cidara Therapeutics, Inc."},"4962":{"cik_str":1843973,"ticker":"EGGF","title":"EG Acquisition Corp."},"4963":{"cik_str":1179821,"ticker":"BGI","title":"BIRKS GROUP INC."},"4964":{"cik_str":1878074,"ticker":"BWAQ","title":"Blue World Acquisition Corp"},"4965":{"cik_str":1794276,"ticker":"YCQH","title":"YCQH Agricultural Technology Co. Ltd"},"4966":{"cik_str":1506488,"ticker":"NXG","title":"NXG NextGen Infrastructure Income Fund"},"4967":{"cik_str":1782223,"ticker":"PYXS","title":"Pyxis Oncology, Inc."},"4968":{"cik_str":1070680,"ticker":"CFBK","title":"CF BANKSHARES INC."},"4969":{"cik_str":831378,"ticker":"LVPA","title":"LVPAI GROUP Ltd"},"4970":{"cik_str":72205,"ticker":"NOBH","title":"NOBILITY HOMES INC"},"4971":{"cik_str":1978133,"ticker":"YERBF","title":"YERBAE BRANDS CORP."},"4972":{"cik_str":899394,"ticker":"HRBR","title":"HARBOR DIVERSIFIED, INC."},"4973":{"cik_str":1775898,"ticker":"UCL","title":"uCloudlink Group Inc."},"4974":{"cik_str":9521,"ticker":"BCV","title":"BANCROFT FUND LTD"},"4975":{"cik_str":1849475,"ticker":"NCAC","title":"Newcourt Acquisition Corp"},"4976":{"cik_str":1590418,"ticker":"FCUV","title":"FOCUS UNIVERSAL INC."},"4977":{"cik_str":875582,"ticker":"NTIC","title":"NORTHERN TECHNOLOGIES INTERNATIONAL CORP"},"4978":{"cik_str":1534969,"ticker":"SERA","title":"SERA PROGNOSTICS, INC."},"4979":{"cik_str":1841144,"ticker":"HWKZ","title":"Hawks Acquisition Corp"},"4980":{"cik_str":1719395,"ticker":"EAR","title":"Eargo, Inc."},"4981":{"cik_str":1909747,"ticker":"JUSHF","title":"Jushi Holdings Inc."},"4982":{"cik_str":919864,"ticker":"FNWD","title":"Finward Bancorp"},"4983":{"cik_str":1448397,"ticker":"SHIP","title":"Seanergy Maritime Holdings Corp."},"4984":{"cik_str":1001115,"ticker":"GEOS","title":"GEOSPACE TECHNOLOGIES CORP"},"4985":{"cik_str":1847409,"ticker":"HITI","title":"High Tide Inc."},"4986":{"cik_str":1498403,"ticker":"BLRX","title":"BioLineRx Ltd."},"4987":{"cik_str":1119774,"ticker":"CGEN","title":"COMPUGEN LTD"},"4988":{"cik_str":37472,"ticker":"FLXS","title":"FLEXSTEEL INDUSTRIES INC"},"4989":{"cik_str":850033,"ticker":"BPT","title":"BP PRUDHOE BAY ROYALTY TRUST"},"4990":{"cik_str":1378701,"ticker":"GDL","title":"GDL FUND"},"4991":{"cik_str":1543418,"ticker":"TMQ","title":"Trilogy Metals Inc."},"4992":{"cik_str":1560672,"ticker":"EARN","title":"Ellington Residential Mortgage REIT"},"4993":{"cik_str":1756701,"ticker":"LNKB","title":"LINKBANCORP, Inc."},"4994":{"cik_str":355379,"ticker":"MDWT","title":"MIDWEST HOLDING INC."},"4995":{"cik_str":1021917,"ticker":"AWCA","title":"Awaysis Capital, Inc."},"4996":{"cik_str":1830029,"ticker":"ADEX","title":"Adit EdTech Acquisition Corp."},"4997":{"cik_str":1790177,"ticker":"RFM","title":"RiverNorth Flexible Municipal Income Fund, Inc."},"4998":{"cik_str":1828748,"ticker":"ASFH","title":"ASIAFIN HOLDINGS CORP."},"4999":{"cik_str":1472012,"ticker":"IMNM","title":"Immunome Inc."},"5000":{"cik_str":1818605,"ticker":"DIST","title":"Distoken Acquisition Corp"},"5001":{"cik_str":718413,"ticker":"CMTV","title":"COMMUNITY BANCORP /VT"},"5002":{"cik_str":897419,"ticker":"NMT","title":"NUVEEN MASSACHUSETTS QUALITY MUNICIPAL INCOME FUND"},"5003":{"cik_str":1869673,"ticker":"BACA","title":"Berenson Acquisition Corp. I"},"5004":{"cik_str":865400,"ticker":"GSVRF","title":"GUANAJUATO SILVER CO LTD"},"5005":{"cik_str":1077640,"ticker":"WMLLF","title":"Wealth Minerals Ltd."},"5006":{"cik_str":1001902,"ticker":"IVAC","title":"INTEVAC INC"},"5007":{"cik_str":715446,"ticker":"ANIX","title":"Anixa Biosciences Inc"},"5008":{"cik_str":1848898,"ticker":"VCXA","title":"10X Capital Venture Acquisition Corp. II"},"5009":{"cik_str":1748907,"ticker":"ORTX","title":"Orchard Therapeutics plc"},"5010":{"cik_str":1576340,"ticker":"CEN","title":"CENTER COAST BROOKFIELD MLP & ENERGY INFRASTRUCTURE FUND"},"5011":{"cik_str":1844817,"ticker":"AACI","title":"Armada Acquisition Corp. I"},"5012":{"cik_str":1547546,"ticker":"LFT","title":"Lument Finance Trust, Inc."},"5013":{"cik_str":6207,"ticker":"AXR","title":"AMREP CORP."},"5014":{"cik_str":1892922,"ticker":"MARX","title":"Mars Acquisition Corp."},"5015":{"cik_str":1621832,"ticker":"AQMS","title":"Aqua Metals, Inc."},"5016":{"cik_str":1867443,"ticker":"CNGL","title":"Canna-Global Acquisition Corp"},"5017":{"cik_str":1040161,"ticker":"PXLW","title":"PIXELWORKS, INC"},"5018":{"cik_str":1662244,"ticker":"CYT","title":"Cyteir Therapeutics, Inc."},"5019":{"cik_str":1857855,"ticker":"FNVT","title":"Finnovate Acquisition Corp."},"5020":{"cik_str":763907,"ticker":"FUNC","title":"FIRST UNITED CORP/MD/"},"5021":{"cik_str":1070732,"ticker":"EVF","title":"EATON VANCE SENIOR INCOME TRUST"},"5022":{"cik_str":1445831,"ticker":"NPFC","title":"Newpoint Financial Corp"},"5023":{"cik_str":1449792,"ticker":"PPSI","title":"PIONEER POWER SOLUTIONS, INC."},"5024":{"cik_str":1946021,"ticker":"HSPO","title":"Horizon Space Acquisition I Corp."},"5025":{"cik_str":1882464,"ticker":"MSSA","title":"Metal Sky Star Acquisition Corp"},"5026":{"cik_str":1865120,"ticker":"BRAC","title":"Broad Capital Acquisition Corp"},"5027":{"cik_str":1823826,"ticker":"ARBG","title":"Aequi Acquisition Corp."},"5028":{"cik_str":1855168,"ticker":"WRAC","title":"Williams Rowland Acquisition Corp."},"5029":{"cik_str":846596,"ticker":"KSM","title":"DWS STRATEGIC MUNICIPAL INCOME TRUST"},"5030":{"cik_str":1741257,"ticker":"BFGX","title":"BANGFU TECHNOLOGY GROUP CO., LTD."},"5031":{"cik_str":39092,"ticker":"FRD","title":"FRIEDMAN INDUSTRIES INC"},"5032":{"cik_str":1843588,"ticker":"REE","title":"REE Automotive Ltd."},"5033":{"cik_str":1841125,"ticker":"GAMC","title":"Golden Arrow Merger Corp."},"5034":{"cik_str":1894370,"ticker":"HMAC","title":"Hainan Manaslu Acquisition Corp."},"5035":{"cik_str":1367859,"ticker":"CZWI","title":"Citizens Community Bancorp Inc."},"5036":{"cik_str":1439725,"ticker":"BDSX","title":"BIODESIX INC"},"5037":{"cik_str":830271,"ticker":"NMI","title":"NUVEEN MUNICIPAL INCOME FUND INC"},"5038":{"cik_str":1285650,"ticker":"CGO","title":"CALAMOS GLOBAL TOTAL RETURN FUND"},"5039":{"cik_str":1937891,"ticker":"ATMV","title":"AlphaVest Acquisition Corp."},"5040":{"cik_str":1868269,"ticker":"LATG","title":"LatAmGrowth SPAC"},"5041":{"cik_str":1496292,"ticker":"IHD","title":"Voya Emerging Markets High Dividend Equity Fund"},"5042":{"cik_str":1752828,"ticker":"CELU","title":"Celularity Inc"},"5043":{"cik_str":1770088,"ticker":"WIMI","title":"WiMi Hologram Cloud Inc."},"5044":{"cik_str":1871149,"ticker":"RGF","title":"Real Good Food Company, Inc."},"5045":{"cik_str":54187,"ticker":"MAYS","title":"MAYS J W INC"},"5046":{"cik_str":1882078,"ticker":"ARIZ","title":"Arisz Acquisition Corp."},"5047":{"cik_str":740971,"ticker":"OPOF","title":"OLD POINT FINANCIAL CORP"},"5048":{"cik_str":716634,"ticker":"RDI","title":"READING INTERNATIONAL INC"},"5049":{"cik_str":1279014,"ticker":"ERH","title":"ALLSPRING UTILITIES & HIGH INCOME FUND"},"5050":{"cik_str":1847345,"ticker":"PWUP","title":"PowerUp Acquisition Corp."},"5051":{"cik_str":1656536,"ticker":"DBTX","title":"Decibel Therapeutics, Inc."},"5052":{"cik_str":1437925,"ticker":"GMGI","title":"Golden Matrix Group, Inc."},"5053":{"cik_str":1872529,"ticker":"MDXH","title":"MDxHealth SA"},"5054":{"cik_str":1431959,"ticker":"MMAT","title":"META MATERIALS INC."},"5055":{"cik_str":1889106,"ticker":"ATMC","title":"ALPHATIME ACQUISITION CORP"},"5056":{"cik_str":1861233,"ticker":"ILLM","title":"AcuityAds Holdings Inc."},"5057":{"cik_str":1043186,"ticker":"SLNG","title":"Stabilis Solutions, Inc."},"5058":{"cik_str":1824036,"ticker":"IDBA","title":"IDEX Biometrics ASA"},"5059":{"cik_str":1588972,"ticker":"SCTL","title":"Societal CDMO, Inc."},"5060":{"cik_str":1835856,"ticker":"AURC","title":"Aurora Acquisition Corp."},"5061":{"cik_str":316253,"ticker":"ENZ","title":"ENZO BIOCHEM INC"},"5062":{"cik_str":1398972,"ticker":"ERDCF","title":"Erdene Resource Development Corp"},"5063":{"cik_str":1840425,"ticker":"BLAC","title":"Bellevue Life Sciences Acquisition Corp."},"5064":{"cik_str":1832136,"ticker":"OSI","title":"Osiris Acquisition Corp."},"5065":{"cik_str":1866030,"ticker":"AZ","title":"A2Z Smart Technologies Corp"},"5066":{"cik_str":1831978,"ticker":"NBST","title":"Newbury Street Acquisition Corp"},"5067":{"cik_str":1435508,"ticker":"FSFG","title":"First Savings Financial Group, Inc."},"5068":{"cik_str":1862463,"ticker":"INAQ","title":"Insight Acquisition Corp. /DE"},"5069":{"cik_str":866643,"ticker":"JDVB","title":"JD BANCSHARES INC"},"5070":{"cik_str":1907223,"ticker":"RWOD","title":"Redwoods Acquisition Corp."},"5071":{"cik_str":1541119,"ticker":"SFBC","title":"Sound Financial Bancorp, Inc."},"5072":{"cik_str":96536,"ticker":"TAYD","title":"TAYLOR DEVICES INC"},"5073":{"cik_str":1825962,"ticker":"QDRO","title":"Quadro Acquisition One Corp."},"5074":{"cik_str":1846235,"ticker":"IMAQ","title":"International Media Acquisition Corp."},"5075":{"cik_str":1934945,"ticker":"TBMC","title":"Trailblazer Merger Corp I"},"5076":{"cik_str":1684693,"ticker":"SLGL","title":"Sol-Gel Technologies Ltd."},"5077":{"cik_str":95953,"ticker":"ACNT","title":"ASCENT INDUSTRIES CO."},"5078":{"cik_str":921183,"ticker":"HMNF","title":"HMN FINANCIAL INC"},"5079":{"cik_str":814586,"ticker":"LWAY","title":"Lifeway Foods, Inc."},"5080":{"cik_str":1716951,"ticker":"OCCI","title":"OFS Credit Company, Inc."},"5081":{"cik_str":1556266,"ticker":"GLG","title":"TD Holdings, Inc."},"5082":{"cik_str":1223389,"ticker":"CONN","title":"CONNS INC"},"5083":{"cik_str":1578453,"ticker":"DLNG","title":"Dynagas LNG Partners LP"},"5084":{"cik_str":1488917,"ticker":"ELMD","title":"Electromed, Inc."},"5085":{"cik_str":1057083,"ticker":"PCTI","title":"PC TEL INC"},"5086":{"cik_str":1682639,"ticker":"EYEN","title":"EYENOVIA, INC."},"5087":{"cik_str":1769624,"ticker":"AGBA","title":"AGBA Group Holding Ltd."},"5088":{"cik_str":1608390,"ticker":"AFMD","title":"Affimed N.V."},"5089":{"cik_str":1832765,"ticker":"EAC","title":"Edify Acquisition Corp."},"5090":{"cik_str":809844,"ticker":"CMU","title":"MFS HIGH YIELD MUNICIPAL TRUST"},"5091":{"cik_str":1895144,"ticker":"GODN","title":"Golden Star Acquisition Corp"},"5092":{"cik_str":1402829,"ticker":"ORN","title":"Orion Group Holdings Inc"},"5093":{"cik_str":1849294,"ticker":"FRLA","title":"Fortune Rise Acquisition Corp"},"5094":{"cik_str":1850266,"ticker":"AKLI","title":"Akili, Inc."},"5095":{"cik_str":1555074,"ticker":"AAMC","title":"Altisource Asset Management Corp"},"5096":{"cik_str":1631487,"ticker":"CLGN","title":"CollPlant Biotechnologies Ltd"},"5097":{"cik_str":1224133,"ticker":"MCHX","title":"MARCHEX INC"},"5098":{"cik_str":1870778,"ticker":"OHAA","title":"Opy Acquisition Corp. I"},"5099":{"cik_str":815577,"ticker":"MPIR","title":"Empire Diversified Energy Inc"},"5100":{"cik_str":1438893,"ticker":"GNT","title":"GAMCO Natural Resources, Gold & Income Trust"},"5101":{"cik_str":1398733,"ticker":"AQST","title":"Aquestive Therapeutics, Inc."},"5102":{"cik_str":1719489,"ticker":"KLR","title":"Kaleyra, Inc."},"5103":{"cik_str":1334325,"ticker":"CODA","title":"Coda Octopus Group, Inc."},"5104":{"cik_str":1548536,"ticker":"SZSMF","title":"Santacruz Silver Mining Ltd."},"5105":{"cik_str":1575434,"ticker":"VERY","title":"Vericity, Inc."},"5106":{"cik_str":1228627,"ticker":"OCUP","title":"Ocuphire Pharma, Inc."},"5107":{"cik_str":1831964,"ticker":"NRAC","title":"NORTHERN REVIVAL ACQUISITION Corp"},"5108":{"cik_str":1017303,"ticker":"TACT","title":"TRANSACT TECHNOLOGIES INC"},"5109":{"cik_str":1835800,"ticker":"PICC","title":"Pivotal Investment Corp III"},"5110":{"cik_str":1831270,"ticker":"BITE","title":"Bite Acquisition Corp."},"5111":{"cik_str":1010470,"ticker":"PROV","title":"PROVIDENT FINANCIAL HOLDINGS INC"},"5112":{"cik_str":1710340,"ticker":"ETON","title":"Eton Pharmaceuticals, Inc."},"5113":{"cik_str":1839530,"ticker":"CFFE","title":"CF Acquisition Corp. VIII"},"5114":{"cik_str":1703157,"ticker":"SCTH","title":"Securetech Innovations, Inc."},"5115":{"cik_str":1671284,"ticker":"BHG","title":"Bright Health Group Inc."},"5116":{"cik_str":1858180,"ticker":"PNAC","title":"Prime Number Acquisition I Corp."},"5117":{"cik_str":1651944,"ticker":"DMTK","title":"DermTech, Inc."},"5118":{"cik_str":1822366,"ticker":"ALTU","title":"Altitude Acquisition Corp."},"5119":{"cik_str":1472215,"ticker":"JLS","title":"Nuveen Mortgage & Income Fund/MA/"},"5120":{"cik_str":2178,"ticker":"AE","title":"ADAMS RESOURCES & ENERGY, INC."},"5121":{"cik_str":2098,"ticker":"ACU","title":"ACME UNITED CORP"},"5122":{"cik_str":1637736,"ticker":"SOLO","title":"ELECTRAMECCANICA VEHICLES CORP."},"5123":{"cik_str":1134115,"ticker":"THM","title":"INTERNATIONAL TOWER HILL MINES LTD"},"5124":{"cik_str":1482554,"ticker":"HFUS","title":"Hartford Great Health Corp."},"5125":{"cik_str":1066684,"ticker":"TGLO","title":"THEGLOBE COM INC"},"5126":{"cik_str":1740797,"ticker":"TREN","title":"TREND INNOVATIONS HOLDING INC."},"5127":{"cik_str":1844149,"ticker":"PCCT","title":"Perception Capital Corp. II"},"5128":{"cik_str":1376227,"ticker":"UNG","title":"United States Natural Gas Fund, LP"},"5129":{"cik_str":1828185,"ticker":"VACC","title":"Vaccitech plc"},"5130":{"cik_str":767405,"ticker":"SBFG","title":"SB FINANCIAL GROUP, INC."},"5131":{"cik_str":1831874,"ticker":"IRAA","title":"Iris Acquisition Corp"},"5132":{"cik_str":1527753,"ticker":"PSNL","title":"Personalis, Inc."},"5133":{"cik_str":1782594,"ticker":"GTH","title":"Genetron Holdings Ltd"},"5134":{"cik_str":1844981,"ticker":"APAC","title":"StoneBridge Acquisition Corp."},"5135":{"cik_str":1842384,"ticker":"BLNG","title":"Belong Acquisition Corp."},"5136":{"cik_str":1888734,"ticker":"GLLI","title":"GLOBALINK INVESTMENT INC."},"5137":{"cik_str":1914805,"ticker":"BANL","title":"CBL International Ltd"},"5138":{"cik_str":1837821,"ticker":"ZJYL","title":"Jin Medical International Ltd."},"5139":{"cik_str":948368,"ticker":"PSBQ","title":"PSB HOLDINGS INC /WI/"},"5140":{"cik_str":1853112,"ticker":"GENQ","title":"Genesis Unicorn Capital Corp."},"5141":{"cik_str":1201792,"ticker":"APEI","title":"AMERICAN PUBLIC EDUCATION INC"},"5142":{"cik_str":1785424,"ticker":"KPLT","title":"Katapult Holdings, Inc."},"5143":{"cik_str":1131554,"ticker":"SNCR","title":"SYNCHRONOSS TECHNOLOGIES INC"},"5144":{"cik_str":1847986,"ticker":"DFLI","title":"Dragonfly Energy Holdings Corp."},"5145":{"cik_str":1833141,"ticker":"CYBN","title":"CYBIN INC."},"5146":{"cik_str":1683553,"ticker":"SPRB","title":"SPRUCE BIOSCIENCES, INC."},"5147":{"cik_str":1133416,"ticker":"GALT","title":"GALECTIN THERAPEUTICS INC"},"5148":{"cik_str":1170311,"ticker":"PNI","title":"PIMCO NEW YORK MUNICIPAL INCOME FUND II"},"5149":{"cik_str":1530766,"ticker":"BSGM","title":"BioSig Technologies, Inc."},"5150":{"cik_str":771856,"ticker":"CSBR","title":"CHAMPIONS ONCOLOGY, INC."},"5151":{"cik_str":1634117,"ticker":"BNED","title":"Barnes & Noble Education, Inc."},"5152":{"cik_str":84129,"ticker":"RAD","title":"RITE AID CORP"},"5153":{"cik_str":1528811,"ticker":"VGI","title":"Virtus Global Multi-Sector Income Fund"},"5154":{"cik_str":1520048,"ticker":"PVL","title":"Permianville Royalty Trust"},"5155":{"cik_str":806172,"ticker":"SOTK","title":"SONO TEK CORP"},"5156":{"cik_str":1453593,"ticker":"XTNT","title":"Xtant Medical Holdings, Inc."},"5157":{"cik_str":1282957,"ticker":"GLU","title":"GABELLI GLOBAL UTILITY & INCOME TRUST"},"5158":{"cik_str":1388126,"ticker":"HNW","title":"Pioneer Diversified High Income Fund, Inc."},"5159":{"cik_str":1872356,"ticker":"MKUL","title":"Molekule Group, Inc."},"5160":{"cik_str":1512717,"ticker":"THTX","title":"Theratechnologies Inc."},"5161":{"cik_str":1838821,"ticker":"ARYD","title":"ARYA Sciences Acquisition Corp IV"},"5162":{"cik_str":1806904,"ticker":"EZGO","title":"EZGO Technologies Ltd."},"5163":{"cik_str":1609065,"ticker":"PBHC","title":"Pathfinder Bancorp, Inc."},"5164":{"cik_str":1662684,"ticker":"KULR","title":"KULR Technology Group, Inc."},"5165":{"cik_str":1120970,"ticker":"LODE","title":"Comstock Inc."},"5166":{"cik_str":1707919,"ticker":"CENN","title":"CENNTRO ELECTRIC GROUP Ltd"},"5167":{"cik_str":1879851,"ticker":"TMTC","title":"TMT Acquisition Corp."},"5168":{"cik_str":1045742,"ticker":"LIVE","title":"LIVE VENTURES Inc"},"5169":{"cik_str":1400897,"ticker":"SRV","title":"NXG Cushing Midstream Energy Fund"},"5170":{"cik_str":845611,"ticker":"GCV","title":"GABELLI CONVERTIBLE & INCOME SECURITIES FUND INC"},"5171":{"cik_str":927355,"ticker":"TESS","title":"TESSCO TECHNOLOGIES INC"},"5172":{"cik_str":866095,"ticker":"JEQ","title":"ABRDN JAPAN EQUITY FUND, INC."},"5173":{"cik_str":1318025,"ticker":"PGP","title":"PIMCO Global StocksPLUS & Income Fund"},"5174":{"cik_str":885732,"ticker":"NXC","title":"NUVEEN CALIFORNIA SELECT TAX FREE INCOME PORTFOLIO"},"5175":{"cik_str":1828972,"ticker":"BZFD","title":"BuzzFeed, Inc."},"5176":{"cik_str":750558,"ticker":"QNBC","title":"QNB CORP"},"5177":{"cik_str":1120370,"ticker":"BWEN","title":"BROADWIND, INC."},"5178":{"cik_str":1286973,"ticker":"USAS","title":"Americas Gold & Silver Corp"},"5179":{"cik_str":1849380,"ticker":"DKDCA","title":"Data Knights Acquisition Corp."},"5180":{"cik_str":1498233,"ticker":"CPTN","title":"Cepton, Inc."},"5181":{"cik_str":1408201,"ticker":"TPZ","title":"TORTOISE POWER & ENERGY INFRASTRUCTURE FUND INC"},"5182":{"cik_str":1137883,"ticker":"BCLI","title":"BRAINSTORM CELL THERAPEUTICS INC."},"5183":{"cik_str":1085243,"ticker":"VTSI","title":"VirTra, Inc"},"5184":{"cik_str":1869824,"ticker":"SGII","title":"Seaport Global Acquisition II Corp."},"5185":{"cik_str":1601485,"ticker":"ELTX","title":"Elicio Therapeutics, Inc."},"5186":{"cik_str":93676,"ticker":"SCX","title":"STARRETT L S CO"},"5187":{"cik_str":1845123,"ticker":"IVCP","title":"Swiftmerge Acquisition Corp."},"5188":{"cik_str":818677,"ticker":"SFDL","title":"SECURITY FEDERAL CORP"},"5189":{"cik_str":1841004,"ticker":"ATAQ","title":"Altimar Acquisition Corp. III"},"5190":{"cik_str":47307,"ticker":"CRAWA","title":"CRAWFORD UNITED Corp"},"5191":{"cik_str":1509470,"ticker":"SSSS","title":"SURO CAPITAL CORP."},"5192":{"cik_str":1768446,"ticker":"ELYM","title":"Eliem Therapeutics, Inc."},"5193":{"cik_str":1437479,"ticker":"ENBP","title":"ENB Financial Corp"},"5194":{"cik_str":318306,"ticker":"ABEO","title":"ABEONA THERAPEUTICS INC."},"5195":{"cik_str":1816017,"ticker":"SPIR","title":"Spire Global, Inc."},"5196":{"cik_str":58361,"ticker":"LEE","title":"LEE ENTERPRISES, Inc"},"5197":{"cik_str":1419806,"ticker":"REEMF","title":"RARE ELEMENT RESOURCES LTD"},"5198":{"cik_str":316888,"ticker":"ASM","title":"AVINO SILVER & GOLD MINES LTD"},"5199":{"cik_str":1858007,"ticker":"GDST","title":"Goldenstone Acquisition Ltd."},"5200":{"cik_str":1829280,"ticker":"FORA","title":"Forian Inc."},"5201":{"cik_str":1863460,"ticker":"PPHP","title":"PHP Ventures Acquisition Corp."},"5202":{"cik_str":1881741,"ticker":"AOGO","title":"Arogo Capital Acquisition Corp."},"5203":{"cik_str":1501078,"ticker":"OFED","title":"Oconee Federal Financial Corp."},"5204":{"cik_str":1729173,"ticker":"UXIN","title":"Uxin Ltd"},"5205":{"cik_str":1022652,"ticker":"INSG","title":"INSEEGO CORP."},"5206":{"cik_str":1823406,"ticker":"AFBI","title":"Affinity Bancshares, Inc."},"5207":{"cik_str":1758009,"ticker":"QUBT","title":"Quantum Computing Inc."},"5208":{"cik_str":1820175,"ticker":"OCAX","title":"OCA Acquisition Corp."},"5209":{"cik_str":1080259,"ticker":"TCLRY","title":"Technicolor"},"5210":{"cik_str":1890671,"ticker":"DUET","title":"DUET Acquisition Corp."},"5211":{"cik_str":1842556,"ticker":"HNRA","title":"HNR Acquisition Corp."},"5212":{"cik_str":875657,"ticker":"ULBI","title":"ULTRALIFE CORP"},"5213":{"cik_str":1955104,"ticker":"ZAPP","title":"Zapp Electric Vehicles Group Ltd"},"5214":{"cik_str":1176334,"ticker":"MMLP","title":"MARTIN MIDSTREAM PARTNERS L.P."},"5215":{"cik_str":1845799,"ticker":"CULL","title":"Cullman Bancorp, Inc. /MD/"},"5216":{"cik_str":1513818,"ticker":"ARAV","title":"Aravive, Inc."},"5217":{"cik_str":1841675,"ticker":"ARBK","title":"Argo Blockchain Plc"},"5218":{"cik_str":1854233,"ticker":"EGLX","title":"Enthusiast Gaming Holdings Inc. / Canada"},"5219":{"cik_str":1141197,"ticker":"PED","title":"PEDEVCO CORP"},"5220":{"cik_str":1672886,"ticker":"ZHYBF","title":"Zhong Yuan Bio-Technology Holdings Ltd"},"5221":{"cik_str":1360565,"ticker":"WFCF","title":"Where Food Comes From, Inc."},"5222":{"cik_str":1108205,"ticker":"CRIS","title":"CURIS INC"},"5223":{"cik_str":1230058,"ticker":"KANP","title":"KAANAPALI LAND LLC"},"5224":{"cik_str":69422,"ticker":"INTG","title":"INTERGROUP CORP"},"5225":{"cik_str":1096938,"ticker":"UEEC","title":"United Health Products, Inc."},"5226":{"cik_str":1401914,"ticker":"DARE","title":"Dare Bioscience, Inc."},"5227":{"cik_str":1751783,"ticker":"RBKB","title":"Rhinebeck Bancorp, Inc."},"5228":{"cik_str":1857030,"ticker":"INCR","title":"Intercure Ltd."},"5229":{"cik_str":1842356,"ticker":"PET","title":"Wag! Group Co."},"5230":{"cik_str":1880151,"ticker":"LIBY","title":"Liberty Resources Acquisition Corp."},"5231":{"cik_str":1490349,"ticker":"PFX","title":"PhenixFIN Corp"},"5232":{"cik_str":1501103,"ticker":"EDF","title":"Virtus Stone Harbor Emerging Markets Income Fund"},"5233":{"cik_str":1929231,"ticker":"PLTN","title":"Plutonian Acquisition Corp."},"5234":{"cik_str":1799332,"ticker":"GAN","title":"GAN Ltd"},"5235":{"cik_str":1826667,"ticker":"MTAC","title":"MedTech Acquisition Corp"},"5236":{"cik_str":1007273,"ticker":"BKSC","title":"BANK OF SOUTH CAROLINA CORP"},"5237":{"cik_str":1701108,"ticker":"SPRO","title":"Spero Therapeutics, Inc."},"5238":{"cik_str":1936702,"ticker":"CETU","title":"Cetus Capital Acquisition Corp."},"5239":{"cik_str":1945422,"ticker":"OAKU","title":"Oak Woods Acquisition Corp"},"5240":{"cik_str":914122,"ticker":"PPIH","title":"Perma-Pipe International Holdings, Inc."},"5241":{"cik_str":900391,"ticker":"NTZ","title":"NATUZZI S P A"},"5242":{"cik_str":1481028,"ticker":"HYSR","title":"SUNHYDROGEN, INC."},"5243":{"cik_str":1823857,"ticker":"VHAQ","title":"Viveon Health Acquisition Corp."},"5244":{"cik_str":933034,"ticker":"STRT","title":"STRATTEC SECURITY CORP"},"5245":{"cik_str":1883962,"ticker":"ALOR","title":"ALSP Orchid Acquisition Corp I"},"5246":{"cik_str":1722387,"ticker":"ITRG","title":"Integra Resources Corp."},"5247":{"cik_str":1429260,"ticker":"FBIO","title":"Fortress Biotech, Inc."},"5248":{"cik_str":1907730,"ticker":"YOTA","title":"Yotta Acquisition Corp"},"5249":{"cik_str":1553643,"ticker":"RLMD","title":"RELMADA THERAPEUTICS, INC."},"5250":{"cik_str":879535,"ticker":"DTF","title":"DTF TAX-FREE INCOME 2028 TERM FUND INC"},"5251":{"cik_str":1838000,"ticker":"SWSS","title":"Springwater Special Situations Corp."},"5252":{"cik_str":1715925,"ticker":"IPA","title":"ImmunoPrecise Antibodies Ltd."},"5253":{"cik_str":1828673,"ticker":"HCWB","title":"HCW Biologics Inc."},"5254":{"cik_str":1732409,"ticker":"BCHG","title":"Grayscale Bitcoin Cash Trust (BCH)"},"5255":{"cik_str":1582554,"ticker":"MTNB","title":"Matinas BioPharma Holdings, Inc."},"5256":{"cik_str":1332551,"ticker":"ACR","title":"ACRES Commercial Realty Corp."},"5257":{"cik_str":876717,"ticker":"FCO","title":"ABRDN GLOBAL INCOME FUND, INC."},"5258":{"cik_str":1301236,"ticker":"SOHO","title":"Sotherly Hotels Inc."},"5259":{"cik_str":835662,"ticker":"AIXN","title":"AiXin Life International, Inc."},"5260":{"cik_str":1847607,"ticker":"RFAC","title":"RF Acquisition Corp."},"5261":{"cik_str":1850902,"ticker":"TKNO","title":"Alpha Teknova, Inc."},"5262":{"cik_str":1509261,"ticker":"RZLT","title":"Rezolute, Inc."},"5263":{"cik_str":1041024,"ticker":"RMTI","title":"ROCKWELL MEDICAL, INC."},"5264":{"cik_str":353184,"ticker":"AIRT","title":"AIR T INC"},"5265":{"cik_str":1863181,"ticker":"HPLT","title":"Home Plate Acquisition Corp"},"5266":{"cik_str":1697532,"ticker":"APLT","title":"Applied Therapeutics Inc."},"5267":{"cik_str":1008586,"ticker":"STRM","title":"STREAMLINE HEALTH SOLUTIONS INC."},"5268":{"cik_str":1158172,"ticker":"SCOR","title":"COMSCORE, INC."},"5269":{"cik_str":750574,"ticker":"AUBN","title":"AUBURN NATIONAL BANCORPORATION, INC"},"5270":{"cik_str":1866838,"ticker":"IGTA","title":"Inception Growth Acquisition Ltd"},"5271":{"cik_str":1848821,"ticker":"GTAC","title":"Global Technology Acquisition Corp. I"},"5272":{"cik_str":1839133,"ticker":"IRME","title":"IR-Med, Inc."},"5273":{"cik_str":1840233,"ticker":"XLO","title":"Xilio Therapeutics, Inc."},"5274":{"cik_str":1870404,"ticker":"PBAX","title":"PHOENIX BIOTECH ACQUISITION CORP."},"5275":{"cik_str":1058623,"ticker":"CMLS","title":"CUMULUS MEDIA INC"},"5276":{"cik_str":202947,"ticker":"CPTP","title":"CAPITAL PROPERTIES INC /RI/"},"5277":{"cik_str":1789769,"ticker":"TIL","title":"Instil Bio, Inc."},"5278":{"cik_str":1944902,"ticker":"OMH","title":"Ohmyhome Ltd"},"5279":{"cik_str":1861063,"ticker":"AQU","title":"Aquaron Acquisition Corp."},"5280":{"cik_str":1713539,"ticker":"KXIN","title":"Kaixin Auto Holdings"},"5281":{"cik_str":1083743,"ticker":"FLUX","title":"Flux Power Holdings, Inc."},"5282":{"cik_str":740806,"ticker":"FMBM","title":"F&M BANK CORP"},"5283":{"cik_str":714712,"ticker":"JUVF","title":"JUNIATA VALLEY FINANCIAL CORP"},"5284":{"cik_str":1753217,"ticker":"JHAA","title":"Nuveen Corporate Income 2023 Target Term Fund"},"5285":{"cik_str":1101396,"ticker":"DLA","title":"DELTA APPAREL, INC"},"5286":{"cik_str":1874252,"ticker":"MYNZ","title":"MAINZ BIOMED N.V."},"5287":{"cik_str":1930179,"ticker":"ARBB","title":"ARB IOT Group Ltd"},"5288":{"cik_str":1894210,"ticker":"QOMO","title":"Qomolangma Acquisition Corp."},"5289":{"cik_str":843006,"ticker":"ISDR","title":"ISSUER DIRECT CORP"},"5290":{"cik_str":1679033,"ticker":"JPT","title":"Nuveen Preferred & Income Fund"},"5291":{"cik_str":1288795,"ticker":"GLV","title":"Clough Global Dividend & Income Fund"},"5292":{"cik_str":1833498,"ticker":"RCLF","title":"Rosecliff Acquisition Corp I"},"5293":{"cik_str":1875257,"ticker":"LIVB","title":"LIV Capital Acquisition Corp. II"},"5294":{"cik_str":1347858,"ticker":"XXII","title":"22nd Century Group, Inc."},"5295":{"cik_str":793306,"ticker":"BDCO","title":"BLUE DOLPHIN ENERGY CO"},"5296":{"cik_str":1805651,"ticker":"MKTW","title":"MARKETWISE, INC."},"5297":{"cik_str":1074692,"ticker":"CEV","title":"EATON VANCE CALIFORNIA MUNICIPAL INCOME TRUST"},"5298":{"cik_str":1490596,"ticker":"GVXXF","title":"GoviEx Uranium Inc."},"5299":{"cik_str":1849058,"ticker":"CLOE","title":"Clover Leaf Capital Corp."},"5300":{"cik_str":828146,"ticker":"LINK","title":"INTERLINK ELECTRONICS INC"},"5301":{"cik_str":1001171,"ticker":"BYFC","title":"BROADWAY FINANCIAL CORP DE"},"5302":{"cik_str":948708,"ticker":"SMSI","title":"SMITH MICRO SOFTWARE, INC."},"5303":{"cik_str":1821146,"ticker":"BWAC","title":"Better World Acquisition Corp."},"5304":{"cik_str":1817565,"ticker":"FXCO","title":"Financial Strategies Acquisition Corp."},"5305":{"cik_str":1852973,"ticker":"OXUS","title":"Oxus Acquisition Corp."},"5306":{"cik_str":1529628,"ticker":"SND","title":"Smart Sand, Inc."},"5307":{"cik_str":1584549,"ticker":"VFF","title":"Village Farms International, Inc."},"5308":{"cik_str":798528,"ticker":"OMEX","title":"ODYSSEY MARINE EXPLORATION INC"},"5309":{"cik_str":1791725,"ticker":"HUDI","title":"Huadi International Group Co., Ltd."},"5310":{"cik_str":1720446,"ticker":"ZEPP","title":"Zepp Health Corp"},"5311":{"cik_str":1584547,"ticker":"GP","title":"GREENPOWER MOTOR Co INC."},"5312":{"cik_str":862651,"ticker":"INVU","title":"Investview, Inc."},"5313":{"cik_str":1507957,"ticker":"IPWR","title":"Ideal Power Inc."},"5314":{"cik_str":1558740,"ticker":"WNLV","title":"Winvest Group Ltd"},"5315":{"cik_str":1847513,"ticker":"TRON","title":"CORNER GROWTH ACQUISITION CORP. 2"},"5316":{"cik_str":1742770,"ticker":"VIOT","title":"Viomi Technology Co., Ltd"},"5317":{"cik_str":1407973,"ticker":"SONX","title":"Sonendo, Inc."},"5318":{"cik_str":1489874,"ticker":"GGROU","title":"Golden Growers Cooperative"},"5319":{"cik_str":1539029,"ticker":"CLSD","title":"Clearside Biomedical, Inc."},"5320":{"cik_str":731653,"ticker":"UBCP","title":"UNITED BANCORP INC /OH/"},"5321":{"cik_str":1850398,"ticker":"TCBC","title":"TC Bancshares, Inc."},"5322":{"cik_str":1778982,"ticker":"HUIZ","title":"Huize Holding Ltd"},"5323":{"cik_str":1840229,"ticker":"INKT","title":"MiNK Therapeutics, Inc."},"5324":{"cik_str":1337068,"ticker":"MGYR","title":"Magyar Bancorp, Inc."},"5325":{"cik_str":1723069,"ticker":"TLSA","title":"Tiziana Life Sciences Ltd"},"5326":{"cik_str":748268,"ticker":"RCAT","title":"Red Cat Holdings, Inc."},"5327":{"cik_str":1862068,"ticker":"RBT","title":"Rubicon Technologies, Inc."},"5328":{"cik_str":1597313,"ticker":"VRAY","title":"ViewRay, Inc."},"5329":{"cik_str":1718500,"ticker":"RVIV","title":"Reviv3 Procare Co"},"5330":{"cik_str":1801834,"ticker":"PRFX","title":"PAINREFORM LTD."},"5331":{"cik_str":1448597,"ticker":"AUGG","title":"AUGUSTA GOLD CORP."},"5332":{"cik_str":1385632,"ticker":"IAE","title":"Voya Asia Pacific High Dividend Equity Income Fund"},"5333":{"cik_str":1827980,"ticker":"EMBK","title":"Embark Technology, Inc."},"5334":{"cik_str":837852,"ticker":"IDEX","title":"IDEANOMICS, INC."},"5335":{"cik_str":1456189,"ticker":"LEAT","title":"Leatt Corp"},"5336":{"cik_str":1075706,"ticker":"CIZN","title":"CITIZENS HOLDING CO /MS/"},"5337":{"cik_str":1721484,"ticker":"LGVN","title":"Longeveron Inc."},"5338":{"cik_str":846671,"ticker":"VLT","title":"Invesco High Income Trust II"},"5339":{"cik_str":1780731,"ticker":"EPOW","title":"Sunrise New Energy Co., Ltd."},"5340":{"cik_str":1597835,"ticker":"CMCM","title":"Cheetah Mobile Inc."},"5341":{"cik_str":1835615,"ticker":"MHUA","title":"Meihua International Medical Technologies Co., Ltd."},"5342":{"cik_str":928054,"ticker":"FTK","title":"FLOTEK INDUSTRIES INC/CN/"},"5343":{"cik_str":1473086,"ticker":"OXBC","title":"Oxford Bank Corp"},"5344":{"cik_str":1941131,"ticker":"TORO","title":"TORO CORP."},"5345":{"cik_str":1724009,"ticker":"PRT","title":"PermRock Royalty Trust"},"5346":{"cik_str":811211,"ticker":"UNIB","title":"UNIVERSITY BANCORP INC /DE/"},"5347":{"cik_str":1586454,"ticker":"PMHG","title":"Prime Meridian Holding Co"},"5348":{"cik_str":1851182,"ticker":"FHLT","title":"Future Health ESG Corp."},"5349":{"cik_str":1140410,"ticker":"PNF","title":"PIMCO NEW YORK MUNICIPAL INCOME FUND"},"5350":{"cik_str":847411,"ticker":"CXH","title":"MFS INVESTMENT GRADE MUNICIPAL TRUST"},"5351":{"cik_str":1936255,"ticker":"FORL","title":"Four Leaf Acquisition Corp"},"5352":{"cik_str":1867287,"ticker":"LBBB","title":"Lakeshore Acquisition II Corp."},"5353":{"cik_str":1867834,"ticker":"BRAG","title":"Bragg Gaming Group Inc."},"5354":{"cik_str":1896212,"ticker":"MURF","title":"Murphy Canyon Acquisition Corp."},"5355":{"cik_str":1826671,"ticker":"NIR","title":"Near Intelligence, Inc."},"5356":{"cik_str":1742341,"ticker":"HKIT","title":"HiTek Global Inc."},"5357":{"cik_str":779544,"ticker":"ARKR","title":"ARK RESTAURANTS CORP"},"5358":{"cik_str":1276469,"ticker":"MFD","title":"MACQUARIE/FIRST TRUST GLOBAL INFRASTR/UTIL DIV & INC FUND"},"5359":{"cik_str":1820566,"ticker":"ISPO","title":"Inspirato Inc"},"5360":{"cik_str":1837607,"ticker":"PMGM","title":"Priveterra Acquisition Corp."},"5361":{"cik_str":1030192,"ticker":"IDR","title":"Idaho Strategic Resources, Inc."},"5362":{"cik_str":808439,"ticker":"TATT","title":"TAT TECHNOLOGIES LTD"},"5363":{"cik_str":1900679,"ticker":"TETE","title":"Technology & Telecommunication Acquisition Corp"},"5364":{"cik_str":1760542,"ticker":"HOOK","title":"HOOKIPA Pharma Inc."},"5365":{"cik_str":1844419,"ticker":"MAQC","title":"Maquia Capital Acquisition Corp"},"5366":{"cik_str":1084554,"ticker":"LTBR","title":"LIGHTBRIDGE Corp"},"5367":{"cik_str":1938186,"ticker":"JYD","title":"Jayud Global Logistics Ltd"},"5368":{"cik_str":71557,"ticker":"NUVR","title":"Nuvera Communications, Inc."},"5369":{"cik_str":1804583,"ticker":"RAASY","title":"Cloopen Group Holding Ltd"},"5370":{"cik_str":1939696,"ticker":"ETAO","title":"ETAO International Co., Ltd."},"5371":{"cik_str":1607997,"ticker":"NMS","title":"Nuveen Minnesota Quality Municipal Income Fund"},"5372":{"cik_str":1227636,"ticker":"STIM","title":"Neuronetics, Inc."},"5373":{"cik_str":751365,"ticker":"VIRC","title":"VIRCO MFG CORPORATION"},"5374":{"cik_str":834285,"ticker":"FRBK","title":"REPUBLIC FIRST BANCORP INC"},"5375":{"cik_str":1800315,"ticker":"GLTO","title":"Galecto, Inc."},"5376":{"cik_str":1915380,"ticker":"DMYY","title":"dMY Squared Technology Group, Inc."},"5377":{"cik_str":1557523,"ticker":"PGZ","title":"Principal Real Estate Income Fund"},"5378":{"cik_str":1847462,"ticker":"AYRWF","title":"Ayr Wellness Inc."},"5379":{"cik_str":1742518,"ticker":"MTC","title":"MMTec, Inc."},"5380":{"cik_str":1807707,"ticker":"APPH","title":"AppHarvest, Inc."},"5381":{"cik_str":1818838,"ticker":"ADAG","title":"Adagene Inc."},"5382":{"cik_str":1859807,"ticker":"NVAC","title":"NorthView Acquisition Corp"},"5383":{"cik_str":1514443,"ticker":"AGSS","title":"AMERIGUARD SECURITY SERVICES, INC."},"5384":{"cik_str":1841408,"ticker":"DAVE","title":"Dave Inc./DE"},"5385":{"cik_str":1290476,"ticker":"VBFC","title":"Village Bank & Trust Financial Corp."},"5386":{"cik_str":1779372,"ticker":"BEAT","title":"HeartBeam, Inc."},"5387":{"cik_str":788920,"ticker":"PDEX","title":"PRO DEX INC"},"5388":{"cik_str":1718405,"ticker":"HYMC","title":"HYCROFT MINING HOLDING CORP"},"5389":{"cik_str":1903382,"ticker":"BHM","title":"Bluerock Homes Trust, Inc."},"5390":{"cik_str":1841761,"ticker":"GROV","title":"Grove Collaborative Holdings, Inc."},"5391":{"cik_str":1329606,"ticker":"CETY","title":"Clean Energy Technologies, Inc."},"5392":{"cik_str":13156,"ticker":"GLXZ","title":"Galaxy Gaming, Inc."},"5393":{"cik_str":1831096,"ticker":"GEG","title":"Great Elm Group, Inc."},"5394":{"cik_str":1328792,"ticker":"TPCS","title":"TECHPRECISION CORP"},"5395":{"cik_str":1341318,"ticker":"LSBK","title":"LAKE SHORE BANCORP, INC."},"5396":{"cik_str":1698508,"ticker":"IHTA","title":"Invesco High Income 2024 Target Term Fund"},"5397":{"cik_str":912061,"ticker":"NHTC","title":"NATURAL HEALTH TRENDS CORP"},"5398":{"cik_str":1266806,"ticker":"VANI","title":"Vivani Medical, Inc."},"5399":{"cik_str":1302624,"ticker":"FAM","title":"FIRST TRUST/ABRDN GLOBAL OPPORTUNITY INCOME FUND"},"5400":{"cik_str":1914023,"ticker":"ACAC","title":"Acri Capital Acquisition Corp"},"5401":{"cik_str":1835416,"ticker":"MACA","title":"Moringa Acquisition Corp"},"5402":{"cik_str":855683,"ticker":"MLSS","title":"MILESTONE SCIENTIFIC INC."},"5403":{"cik_str":1556263,"ticker":"SYRS","title":"Syros Pharmaceuticals, Inc."},"5404":{"cik_str":1807765,"ticker":"PMVC","title":"PMV Consumer Acquisition Corp."},"5405":{"cik_str":1835268,"ticker":"CNTB","title":"Connect Biopharma Holdings Ltd"},"5406":{"cik_str":1210708,"ticker":"HSON","title":"Hudson Global, Inc."},"5407":{"cik_str":1850262,"ticker":"INTE","title":"Integral Acquisition Corp 1"},"5408":{"cik_str":723603,"ticker":"CULP","title":"CULP INC"},"5409":{"cik_str":1445942,"ticker":"TMRC","title":"Texas Mineral Resources Corp."},"5410":{"cik_str":883107,"ticker":"NANX","title":"NANOPHASE TECHNOLOGIES Corp"},"5411":{"cik_str":1137091,"ticker":"PSIX","title":"POWER SOLUTIONS INTERNATIONAL, INC."},"5412":{"cik_str":1386301,"ticker":"RSSS","title":"Research Solutions, Inc."},"5413":{"cik_str":6176,"ticker":"AP","title":"AMPCO PITTSBURGH CORP"},"5414":{"cik_str":1784058,"ticker":"PNYG","title":"Pony Group Inc."},"5415":{"cik_str":1213037,"ticker":"CRDF","title":"Cardiff Oncology, Inc."},"5416":{"cik_str":1870129,"ticker":"ROSE","title":"Rose Hill Acquisition Corp"},"5417":{"cik_str":1357874,"ticker":"DTIL","title":"PRECISION BIOSCIENCES INC"},"5418":{"cik_str":1099668,"ticker":"FOTB","title":"FIRST OTTAWA BANCSHARES, INC"},"5419":{"cik_str":783324,"ticker":"VGZ","title":"VISTA GOLD CORP"},"5420":{"cik_str":1626745,"ticker":"FVTI","title":"Fortune Valley Treasures, Inc."},"5421":{"cik_str":1849011,"ticker":"APNC","title":"Apeiron Capital Investment Corp."},"5422":{"cik_str":1024672,"ticker":"ELTK","title":"ELTEK LTD"},"5423":{"cik_str":40570,"ticker":"JOB","title":"GEE Group Inc."},"5424":{"cik_str":1866816,"ticker":"OLIT","title":"OmniLit Acquisition Corp."},"5425":{"cik_str":39020,"ticker":"FEIM","title":"FREQUENCY ELECTRONICS INC"},"5426":{"cik_str":15847,"ticker":"BUKS","title":"BUTLER NATIONAL CORP"},"5427":{"cik_str":1737523,"ticker":"BGLC","title":"BioNexus Gene Lab Corp"},"5428":{"cik_str":1821866,"ticker":"BTQQF","title":"Sonora Gold & Silver Corp."},"5429":{"cik_str":719733,"ticker":"KTCC","title":"KEY TRONIC CORP"},"5430":{"cik_str":1043000,"ticker":"SNDA","title":"SONIDA SENIOR LIVING, INC."},"5431":{"cik_str":1852440,"ticker":"JZ","title":"Jianzhi Education Technology Group Co Ltd"},"5432":{"cik_str":34563,"ticker":"FARM","title":"FARMER BROTHERS CO"},"5433":{"cik_str":1657642,"ticker":"SLBK","title":"Skyline Bankshares, Inc."},"5434":{"cik_str":1622879,"ticker":"SHWZ","title":"Medicine Man Technologies, Inc."},"5435":{"cik_str":1355848,"ticker":"TOON","title":"Kartoon Studios, Inc."},"5436":{"cik_str":1769116,"ticker":"VIAO","title":"VIA optronics AG"},"5437":{"cik_str":1763950,"ticker":"LTRN","title":"Lantern Pharma Inc."},"5438":{"cik_str":1702924,"ticker":"WRAP","title":"WRAP TECHNOLOGIES, INC."},"5439":{"cik_str":1095435,"ticker":"PRTG","title":"PORTAGE BIOTECH INC."},"5440":{"cik_str":1534154,"ticker":"AUID","title":"authID Inc."},"5441":{"cik_str":1865861,"ticker":"CCTS","title":"Cactus Acquisition Corp. 1 Ltd"},"5442":{"cik_str":1641489,"ticker":"VTVT","title":"vTv Therapeutics Inc."},"5443":{"cik_str":1722926,"ticker":"ASLN","title":"ASLAN Pharmaceuticals Ltd"},"5444":{"cik_str":1472072,"ticker":"CCM","title":"Concord Medical Services Holdings Ltd"},"5445":{"cik_str":791718,"ticker":"EEA","title":"EUROPEAN EQUITY FUND, INC / MD"},"5446":{"cik_str":1746119,"ticker":"MUGH","title":"MU GLOBAL HOLDING Ltd"},"5447":{"cik_str":1178840,"ticker":"NBW","title":"NEUBERGER BERMAN CALIFORNIA MUNICIPAL FUND INC."},"5448":{"cik_str":1837344,"ticker":"MBTC","title":"Nocturne Acquisition Corp"},"5449":{"cik_str":872912,"ticker":"DCTH","title":"DELCATH SYSTEMS, INC."},"5450":{"cik_str":1841586,"ticker":"ITAQ","title":"Industrial Tech Acquisitions II, Inc."},"5451":{"cik_str":1850906,"ticker":"OMIC","title":"Singular Genomics Systems, Inc."},"5452":{"cik_str":1321834,"ticker":"MYMD","title":"MyMD Pharmaceuticals, Inc."},"5453":{"cik_str":849146,"ticker":"LFVN","title":"Lifevantage Corp"},"5454":{"cik_str":1487718,"ticker":"BLTH","title":"AMERICAN BATTERY MATERIALS, INC."},"5455":{"cik_str":1675033,"ticker":"GECC","title":"Great Elm Capital Corp."},"5456":{"cik_str":1799011,"ticker":"LUCD","title":"Lucid Diagnostics Inc."},"5457":{"cik_str":1815974,"ticker":"ANEB","title":"Anebulo Pharmaceuticals, Inc."},"5458":{"cik_str":1628945,"ticker":"HLTH","title":"Cue Health Inc."},"5459":{"cik_str":1080340,"ticker":"LOAN","title":"MANHATTAN BRIDGE CAPITAL, INC"},"5460":{"cik_str":1514490,"ticker":"RGT","title":"ROYCE GLOBAL VALUE TRUST, INC."},"5461":{"cik_str":1394056,"ticker":"OSS","title":"ONE STOP SYSTEMS, INC."},"5462":{"cik_str":1841425,"ticker":"VGAS","title":"Verde Clean Fuels, Inc."},"5463":{"cik_str":1526329,"ticker":"TTP","title":"TORTOISE PIPELINE & ENERGY FUND, INC."},"5464":{"cik_str":12040,"ticker":"BDL","title":"FLANIGANS ENTERPRISES INC"},"5465":{"cik_str":1849902,"ticker":"SEPA","title":"SEP Acquisition Corp."},"5466":{"cik_str":1734750,"ticker":"MOVE","title":"Movano Inc."},"5467":{"cik_str":1874875,"ticker":"HOUR","title":"Hour Loop, Inc"},"5468":{"cik_str":1842498,"ticker":"OTMO","title":"Otonomo Technologies Ltd."},"5469":{"cik_str":1708176,"ticker":"HOFV","title":"Hall of Fame Resort & Entertainment Co"},"5470":{"cik_str":860489,"ticker":"CEE","title":"CENTRAL & EASTERN EUROPE FUND, INC."},"5471":{"cik_str":1289636,"ticker":"PFIE","title":"PROFIRE ENERGY INC"},"5472":{"cik_str":1879373,"ticker":"ENCP","title":"Energem Corp"},"5473":{"cik_str":1514597,"ticker":"FURY","title":"FURY GOLD MINES LTD"},"5474":{"cik_str":1929589,"ticker":"MRDB","title":"MariaDB plc"},"5475":{"cik_str":1886894,"ticker":"SNAL","title":"Snail, Inc."},"5476":{"cik_str":1702123,"ticker":"CRDL","title":"Cardiol Therapeutics Inc."},"5477":{"cik_str":1868775,"ticker":"ASCA","title":"ASPAC I Acquisition Corp."},"5478":{"cik_str":875729,"ticker":"BNET","title":"BION ENVIRONMENTAL TECHNOLOGIES INC"},"5479":{"cik_str":1873324,"ticker":"PEPL","title":"PepperLime Health Acquisition Corp"},"5480":{"cik_str":1845942,"ticker":"BNIX","title":"Bannix Acquisition Corp."},"5481":{"cik_str":1213809,"ticker":"DYAI","title":"DYADIC INTERNATIONAL INC"},"5482":{"cik_str":356037,"ticker":"CSPI","title":"CSP INC /MA/"},"5483":{"cik_str":1377630,"ticker":"NCMI","title":"National CineMedia, Inc."},"5484":{"cik_str":1006830,"ticker":"CBKM","title":"CONSUMERS BANCORP INC /OH/"},"5485":{"cik_str":1957401,"ticker":"ZYRX","title":"GLOBAL EARNINGS CAPITAL LTD."},"5486":{"cik_str":1865107,"ticker":"AKA","title":"A.K.A. BRANDS HOLDING CORP."},"5487":{"cik_str":1160791,"ticker":"GORO","title":"GOLD RESOURCE CORP"},"5488":{"cik_str":1476573,"ticker":"AIRRF","title":"Aurion Resources Ltd."},"5489":{"cik_str":898171,"ticker":"UWHR","title":"UWHARRIE CAPITAL CORP"},"5490":{"cik_str":1886268,"ticker":"SHUA","title":"SHUAA Partners Acquisition Corp I"},"5491":{"cik_str":1930313,"ticker":"PTWO","title":"Pono Capital Two, Inc."},"5492":{"cik_str":770460,"ticker":"PFBX","title":"PEOPLES FINANCIAL CORP /MS/"},"5493":{"cik_str":1448815,"ticker":"RFLFY","title":"Raffles Education Corp Ltd"},"5494":{"cik_str":1821323,"ticker":"SGTX","title":"Sigilon Therapeutics, Inc."},"5495":{"cik_str":1031235,"ticker":"SELF","title":"Global Self Storage, Inc."},"5496":{"cik_str":1849867,"ticker":"CLST","title":"Catalyst Bancorp, Inc."},"5497":{"cik_str":1819411,"ticker":"GANX","title":"Gain Therapeutics, Inc."},"5498":{"cik_str":1539850,"ticker":"STCB","title":"Starco Brands, Inc."},"5499":{"cik_str":1711291,"ticker":"CURO","title":"CURO Group Holdings Corp."},"5500":{"cik_str":1272842,"ticker":"AIRG","title":"AIRGAIN INC"},"5501":{"cik_str":2186,"ticker":"BKTI","title":"BK Technologies Corp"},"5502":{"cik_str":1426800,"ticker":"ASMB","title":"ASSEMBLY BIOSCIENCES, INC."},"5503":{"cik_str":1379329,"ticker":"IBTN","title":"InsCorp Inc"},"5504":{"cik_str":1571329,"ticker":"LRFC","title":"Logan Ridge Finance Corp."},"5505":{"cik_str":1524769,"ticker":"CHKR","title":"CHESAPEAKE GRANITE WASH TRUST"},"5506":{"cik_str":1065078,"ticker":"NTIP","title":"NETWORK-1 TECHNOLOGIES, INC."},"5507":{"cik_str":1649739,"ticker":"BAFN","title":"BayFirst Financial Corp."},"5508":{"cik_str":1865494,"ticker":"IOBT","title":"IO Biotech, Inc."},"5509":{"cik_str":1863990,"ticker":"SMAP","title":"Sportsmap Tech Acquisition Corp."},"5510":{"cik_str":1563665,"ticker":"BSTG","title":"Biostage, Inc."},"5511":{"cik_str":793524,"ticker":"REFR","title":"RESEARCH FRONTIERS INC"},"5512":{"cik_str":1816723,"ticker":"GRFX","title":"Graphex Group Ltd"},"5513":{"cik_str":1163389,"ticker":"NWPP","title":"NEW PEOPLES BANKSHARES INC"},"5514":{"cik_str":838131,"ticker":"JMM","title":"Nuveen Multi-Market Income Fund"},"5515":{"cik_str":1854445,"ticker":"VRAR","title":"Glimpse Group, Inc."},"5516":{"cik_str":914712,"ticker":"STCN","title":"Steel Connect, Inc."},"5517":{"cik_str":1739426,"ticker":"RVLP","title":"RVL Pharmaceuticals plc"},"5518":{"cik_str":1817232,"ticker":"DUNE","title":"Dune Acquisition Corp"},"5519":{"cik_str":1750593,"ticker":"DUO","title":"Fangdd Network Group Ltd."},"5520":{"cik_str":1718108,"ticker":"SURF","title":"Surface Oncology, Inc."},"5521":{"cik_str":1665300,"ticker":"PHUN","title":"Phunware, Inc."},"5522":{"cik_str":1514056,"ticker":"TRIQ","title":"TRAQIQ, INC."},"5523":{"cik_str":1031623,"ticker":"GIFI","title":"GULF ISLAND FABRICATION INC"},"5524":{"cik_str":1145255,"ticker":"HNNA","title":"HENNESSY ADVISORS INC"},"5525":{"cik_str":1320854,"ticker":"RAIL","title":"FreightCar America, Inc."},"5526":{"cik_str":1853774,"ticker":"MCAF","title":"Mountain Crest Acquisition Corp. IV"},"5527":{"cik_str":1860484,"ticker":"RACY","title":"Relativity Acquisition Corp"},"5528":{"cik_str":1302387,"ticker":"BVFL","title":"BV Financial, Inc."},"5529":{"cik_str":1113313,"ticker":"AVNI","title":"ARVANA INC"},"5530":{"cik_str":880242,"ticker":"BLGO","title":"BIOLARGO, INC."},"5531":{"cik_str":1433607,"ticker":"NSPR","title":"InspireMD, Inc."},"5532":{"cik_str":1860663,"ticker":"MTRY","title":"Monterey Innovation Acquisition Corp"},"5533":{"cik_str":1822886,"ticker":"HHGC","title":"HHG Capital Corp"},"5534":{"cik_str":1876581,"ticker":"IMPP","title":"Imperial Petroleum Inc./Marshall Islands"},"5535":{"cik_str":1409375,"ticker":"OESX","title":"ORION ENERGY SYSTEMS, INC."},"5536":{"cik_str":925741,"ticker":"BCDA","title":"BioCardia, Inc."},"5537":{"cik_str":1460329,"ticker":"FLNT","title":"Fluent, Inc."},"5538":{"cik_str":717806,"ticker":"FUSB","title":"FIRST US BANCSHARES, INC."},"5539":{"cik_str":1662774,"ticker":"QNCX","title":"Quince Therapeutics, Inc."},"5540":{"cik_str":1141240,"ticker":"LQMT","title":"LIQUIDMETAL TECHNOLOGIES INC"},"5541":{"cik_str":862692,"ticker":"CCEL","title":"CRYO CELL INTERNATIONAL INC"},"5542":{"cik_str":1362190,"ticker":"AEYE","title":"AUDIOEYE INC"},"5543":{"cik_str":1061027,"ticker":"VIRX","title":"Viracta Therapeutics, Inc."},"5544":{"cik_str":1879103,"ticker":"CFSB","title":"CFSB Bancorp, Inc. /MA/"},"5545":{"cik_str":1578348,"ticker":"ICMB","title":"Investcorp Credit Management BDC, Inc."},"5546":{"cik_str":1494891,"ticker":"SRTS","title":"Sensus Healthcare, Inc."},"5547":{"cik_str":1857086,"ticker":"DMAQ","title":"Deep Medicine Acquisition Corp."},"5548":{"cik_str":1606268,"ticker":"VIA","title":"Via Renewables, Inc."},"5549":{"cik_str":1786286,"ticker":"DPRO","title":"Draganfly Inc."},"5550":{"cik_str":912764,"ticker":"KSBI","title":"KS BANCORP INC"},"5551":{"cik_str":1854368,"ticker":"DGHI","title":"Digihost Technology Inc."},"5552":{"cik_str":1551040,"ticker":"EDI","title":"Virtus Stone Harbor Emerging Markets Total Income Fund"},"5553":{"cik_str":1434524,"ticker":"CLIR","title":"ClearSign Technologies Corp"},"5554":{"cik_str":1584371,"ticker":"ICCM","title":"IceCure Medical Ltd."},"5555":{"cik_str":1091596,"ticker":"AURX","title":"Nuo Therapeutics, Inc."},"5556":{"cik_str":1574565,"ticker":"EVGN","title":"Evogene Ltd."},"5557":{"cik_str":943535,"ticker":"WHEN","title":"WORLD HEALTH ENERGY HOLDINGS, INC."},"5558":{"cik_str":84112,"ticker":"RSKIA","title":"GEORGE RISK INDUSTRIES, INC."},"5559":{"cik_str":1088034,"ticker":"USIO","title":"Usio, Inc."},"5560":{"cik_str":1465885,"ticker":"WMC","title":"Western Asset Mortgage Capital Corp"},"5561":{"cik_str":1216128,"ticker":"ORPB","title":"OREGON PACIFIC BANCORP"},"5562":{"cik_str":1606698,"ticker":"ALPP","title":"ALPINE 4 HOLDINGS, INC."},"5563":{"cik_str":1455365,"ticker":"CGTX","title":"COGNITION THERAPEUTICS INC"},"5564":{"cik_str":1641281,"ticker":"BOLT","title":"Bolt Biotherapeutics, Inc."},"5565":{"cik_str":1713863,"ticker":"RFL","title":"Rafael Holdings, Inc."},"5566":{"cik_str":1836176,"ticker":"FATH","title":"Fathom Digital Manufacturing Corp"},"5567":{"cik_str":1496690,"ticker":"BCRD","title":"BlueOne Card, Inc."},"5568":{"cik_str":1834045,"ticker":"VWE","title":"Vintage Wine Estates, Inc."},"5569":{"cik_str":1644771,"ticker":"RSF","title":"RiverNorth Capital & Income Fund, Inc."},"5570":{"cik_str":1788717,"ticker":"FXLV","title":"F45 Training Holdings Inc."},"5571":{"cik_str":1602842,"ticker":"MOGO","title":"Mogo Inc."},"5572":{"cik_str":1598924,"ticker":"ICGL","title":"Image Chain Group Limited, Inc."},"5573":{"cik_str":1477246,"ticker":"SANW","title":"S&W Seed Co"},"5574":{"cik_str":1447380,"ticker":"MFON","title":"MOBIVITY HOLDINGS CORP."},"5575":{"cik_str":1396502,"ticker":"HTY","title":"John Hancock Tax-Advantaged Global Shareholder Yield Fund"},"5576":{"cik_str":1514743,"ticker":"IROQ","title":"IF Bancorp, Inc."},"5577":{"cik_str":754811,"ticker":"GROW","title":"U S GLOBAL INVESTORS INC"},"5578":{"cik_str":1337090,"ticker":"SPAZF","title":"SPANISH MOUNTAIN GOLD LTD."},"5579":{"cik_str":1843162,"ticker":"SSIC","title":"Silver Spike Investment Corp."},"5580":{"cik_str":1580063,"ticker":"BIOR","title":"BIORA THERAPEUTICS, INC."},"5581":{"cik_str":315545,"ticker":"PVCT","title":"PROVECTUS BIOPHARMACEUTICALS, INC."},"5582":{"cik_str":1074828,"ticker":"KNW","title":"KNOW LABS, INC."},"5583":{"cik_str":1710495,"ticker":"MNAO","title":"PINEAPPLE EXPRESS CANNABIS Co"},"5584":{"cik_str":1598646,"ticker":"NERV","title":"Minerva Neurosciences, Inc."},"5585":{"cik_str":25895,"ticker":"CRWS","title":"CROWN CRAFTS INC"},"5586":{"cik_str":771999,"ticker":"DSS","title":"DSS, INC."},"5587":{"cik_str":1547158,"ticker":"NDP","title":"TORTOISE ENERGY INDEPENDENCE FUND, INC."},"5588":{"cik_str":1835059,"ticker":"ARVL","title":"Arrival"},"5589":{"cik_str":1828522,"ticker":"EFTR","title":"eFFECTOR Therapeutics, Inc."},"5590":{"cik_str":1515156,"ticker":"ADES","title":"Advanced Emissions Solutions, Inc."},"5591":{"cik_str":1310630,"ticker":"CFOO","title":"China Foods Holdings Ltd."},"5592":{"cik_str":716643,"ticker":"RGS","title":"REGIS CORP"},"5593":{"cik_str":1934064,"ticker":"OSA","title":"ProSomnus, Inc."},"5594":{"cik_str":1297341,"ticker":"KFFB","title":"Kentucky First Federal Bancorp"},"5595":{"cik_str":1840748,"ticker":"LVTX","title":"LAVA Therapeutics NV"},"5596":{"cik_str":1723935,"ticker":"STG","title":"Sunlands Technology Group"},"5597":{"cik_str":1681903,"ticker":"ICCH","title":"ICC Holdings, Inc."},"5598":{"cik_str":1903392,"ticker":"CLRC","title":"ClimateRock"},"5599":{"cik_str":889971,"ticker":"LPTH","title":"LIGHTPATH TECHNOLOGIES INC"},"5600":{"cik_str":1595248,"ticker":"GNPX","title":"Genprex, Inc."},"5601":{"cik_str":1320350,"ticker":"LNSR","title":"LENSAR, Inc."},"5602":{"cik_str":766792,"ticker":"CVV","title":"CVD EQUIPMENT CORP"},"5603":{"cik_str":1661059,"ticker":"NXTC","title":"NextCure, Inc."},"5604":{"cik_str":1840292,"ticker":"HLGN","title":"Heliogen, Inc."},"5605":{"cik_str":1058307,"ticker":"NXPL","title":"NextPlat Corp"},"5606":{"cik_str":1787297,"ticker":"PASG","title":"Passage BIO, Inc."},"5607":{"cik_str":1883983,"ticker":"VSAC","title":"VISION SENSING ACQUISITION CORP."},"5608":{"cik_str":1602380,"ticker":"JROOF","title":"Jericho Energy Ventures Inc."},"5609":{"cik_str":1680048,"ticker":"MBIO","title":"MUSTANG BIO, INC."},"5610":{"cik_str":1677077,"ticker":"ALZN","title":"Alzamend Neuro, Inc."},"5611":{"cik_str":1689657,"ticker":"CELL","title":"PhenomeX Inc."},"5612":{"cik_str":1740279,"ticker":"INAB","title":"IN8BIO, INC."},"5613":{"cik_str":1274737,"ticker":"XGN","title":"EXAGEN INC."},"5614":{"cik_str":1776909,"ticker":"CURI","title":"CuriosityStream Inc."},"5615":{"cik_str":716006,"ticker":"YELL","title":"Yellow Corp"},"5616":{"cik_str":1810140,"ticker":"POL","title":"Polished.com Inc."},"5617":{"cik_str":1846809,"ticker":"OTEC","title":"OceanTech Acquisitions I Corp."},"5618":{"cik_str":1648365,"ticker":"TMNA","title":"AGRI-FINTECH HOLDINGS, INC."},"5619":{"cik_str":1881592,"ticker":"NSTS","title":"NSTS Bancorp, Inc."},"5620":{"cik_str":1661998,"ticker":"FIXX","title":"Homology Medicines, Inc."},"5621":{"cik_str":1864531,"ticker":"DHAC","title":"DIGITAL HEALTH ACQUISITION CORP."},"5622":{"cik_str":1855351,"ticker":"SAGA","title":"Sagaliam Acquisition Corp"},"5623":{"cik_str":1178841,"ticker":"NBO","title":"NEUBERGER BERMAN NEW YORK MUNICIPAL FUND INC."},"5624":{"cik_str":1701478,"ticker":"AZTR","title":"Azitra Inc"},"5625":{"cik_str":1549145,"ticker":"BIOF","title":"BLUE BIOFUELS, INC."},"5626":{"cik_str":1844392,"ticker":"MRAI","title":"Marpai, Inc."},"5627":{"cik_str":1357971,"ticker":"ESOA","title":"Energy Services of America CORP"},"5628":{"cik_str":1178697,"ticker":"SONM","title":"SONIM TECHNOLOGIES INC"},"5629":{"cik_str":1498148,"ticker":"AITX","title":"Artificial Intelligence Technology Solutions Inc."},"5630":{"cik_str":1102432,"ticker":"VKIN","title":"VIKING ENERGY GROUP, INC."},"5631":{"cik_str":1596812,"ticker":"ENLV","title":"Enlivex Therapeutics Ltd."},"5632":{"cik_str":884269,"ticker":"APT","title":"ALPHA PRO TECH LTD"},"5633":{"cik_str":949961,"ticker":"IOR","title":"INCOME OPPORTUNITY REALTY INVESTORS INC /TX/"},"5634":{"cik_str":889348,"ticker":"CVU","title":"CPI AEROSTRUCTURES INC"},"5635":{"cik_str":1762239,"ticker":"KAVL","title":"Kaival Brands Innovations Group, Inc."},"5636":{"cik_str":1407583,"ticker":"BHLL","title":"Bunker Hill Mining Corp."},"5637":{"cik_str":1089872,"ticker":"GAIA","title":"GAIA, INC"},"5638":{"cik_str":1690585,"ticker":"MGTA","title":"Magenta Therapeutics, Inc."},"5639":{"cik_str":1075736,"ticker":"CXDO","title":"Crexendo, Inc."},"5640":{"cik_str":1319183,"ticker":"FMY","title":"FIRST TRUST MORTGAGE INCOME FUND"},"5641":{"cik_str":1696558,"ticker":"JRSH","title":"Jerash Holdings (US), Inc."},"5642":{"cik_str":1452965,"ticker":"UTRS","title":"MINERVA SURGICAL INC"},"5643":{"cik_str":885731,"ticker":"NXN","title":"NUVEEN NEW YORK SELECT TAX -FREE INCOME PORTFOLIO"},"5644":{"cik_str":1582982,"ticker":"CCLD","title":"CareCloud, Inc."},"5645":{"cik_str":1392994,"ticker":"FGB","title":"FIRST TRUST SPECIALTY FINANCE & FINANCIAL OPPORTUNITIES FUND"},"5646":{"cik_str":1832487,"ticker":"GUER","title":"Guerrilla RF, Inc."},"5647":{"cik_str":1817640,"ticker":"BREZ","title":"Breeze Holdings Acquisition Corp."},"5648":{"cik_str":1799290,"ticker":"EBON","title":"Ebang International Holdings Inc."},"5649":{"cik_str":1783407,"ticker":"LIZI","title":"LIZHI INC."},"5650":{"cik_str":1659494,"ticker":"COE","title":"51Talk Online Education Group"},"5651":{"cik_str":1826681,"ticker":"STRC","title":"Sarcos Technology & Robotics Corp"},"5652":{"cik_str":1847360,"ticker":"BCOW","title":"1895 Bancorp of Wisconsin, Inc. /MD/"},"5653":{"cik_str":1646188,"ticker":"ONDS","title":"Ondas Holdings Inc."},"5654":{"cik_str":893739,"ticker":"TURN","title":"180 DEGREE CAPITAL CORP. /NY/"},"5655":{"cik_str":1500375,"ticker":"HFBL","title":"Home Federal Bancorp, Inc. of Louisiana"},"5656":{"cik_str":1085921,"ticker":"MMTIF","title":"MICROMEM TECHNOLOGIES INC"},"5657":{"cik_str":1884164,"ticker":"KWIK","title":"KwikClick, Inc."},"5658":{"cik_str":864240,"ticker":"SYPR","title":"SYPRIS SOLUTIONS INC"},"5659":{"cik_str":749660,"ticker":"ICAD","title":"ICAD INC"},"5660":{"cik_str":33533,"ticker":"ESP","title":"ESPEY MFG & ELECTRONICS CORP"},"5661":{"cik_str":1861622,"ticker":"OXAC","title":"Oxbridge Acquisition Corp."},"5662":{"cik_str":1859639,"ticker":"CZOO","title":"Cazoo Group Ltd"},"5663":{"cik_str":313364,"ticker":"MTR","title":"MESA ROYALTY TRUST/TX"},"5664":{"cik_str":1040896,"ticker":"IDN","title":"Intellicheck, Inc."},"5665":{"cik_str":1483510,"ticker":"EXPR","title":"EXPRESS, INC."},"5666":{"cik_str":1414382,"ticker":"CLEV","title":"Concrete Leveling Systems Inc"},"5667":{"cik_str":1553788,"ticker":"SBEV","title":"SPLASH BEVERAGE GROUP, INC."},"5668":{"cik_str":27367,"ticker":"DXR","title":"DAXOR CORP"},"5669":{"cik_str":1874074,"ticker":"MMV","title":"MultiMetaVerse Holdings Ltd"},"5670":{"cik_str":1440153,"ticker":"BKUH","title":"Bakhu Holdings, Corp."},"5671":{"cik_str":1677940,"ticker":"BYSI","title":"BeyondSpring Inc."},"5672":{"cik_str":1523289,"ticker":"DMA","title":"Destra Multi-Alternative Fund"},"5673":{"cik_str":799165,"ticker":"DWSN","title":"DAWSON GEOPHYSICAL CO"},"5674":{"cik_str":1822691,"ticker":"GRNA","title":"GreenLight Biosciences Holdings, PBC"},"5675":{"cik_str":863900,"ticker":"MXE","title":"MEXICO EQUITY & INCOME FUND INC"},"5676":{"cik_str":1843248,"ticker":"GSD","title":"Global System Dynamics, Inc."},"5677":{"cik_str":1873835,"ticker":"IMMX","title":"Immix Biopharma, Inc."},"5678":{"cik_str":1836242,"ticker":"TKLF","title":"Yoshitsu Co., Ltd"},"5679":{"cik_str":55529,"ticker":"KEQU","title":"KEWAUNEE SCIENTIFIC CORP /DE/"},"5680":{"cik_str":878828,"ticker":"WTT","title":"WIRELESS TELECOM GROUP INC"},"5681":{"cik_str":1420529,"ticker":"AACG","title":"ATA Creativity Global"},"5682":{"cik_str":1439237,"ticker":"QOEG","title":"Quality Online Education Group Inc."},"5683":{"cik_str":1854463,"ticker":"WINV","title":"WinVest Acquisition Corp."},"5684":{"cik_str":1771885,"ticker":"HUGE","title":"FSD Pharma Inc."},"5685":{"cik_str":1699906,"ticker":"BKRRF","title":"BLACKROCK SILVER CORP."},"5686":{"cik_str":1096275,"ticker":"WKSP","title":"Worksport Ltd"},"5687":{"cik_str":1846968,"ticker":"CCAI","title":"Cascadia Acquisition Corp."},"5688":{"cik_str":1262104,"ticker":"MEIP","title":"MEI Pharma, Inc."},"5689":{"cik_str":1757840,"ticker":"INDO","title":"Indonesia Energy Corp Ltd"},"5690":{"cik_str":1459188,"ticker":"FDOC","title":"FUEL DOCTOR HOLDINGS, INC."},"5691":{"cik_str":1888175,"ticker":"PXCLY","title":"Phoenix Copper Limited/ADR"},"5692":{"cik_str":1527096,"ticker":"MIRO","title":"Miromatrix Medical Inc."},"5693":{"cik_str":1757143,"ticker":"AIH","title":"Aesthetic Medical International Holdings Group Ltd"},"5694":{"cik_str":1627282,"ticker":"CWD","title":"CaliberCos Inc."},"5695":{"cik_str":1390478,"ticker":"SLS","title":"SELLAS Life Sciences Group, Inc."},"5696":{"cik_str":1114936,"ticker":"UMEWF","title":"UMeWorld Ltd"},"5697":{"cik_str":1654672,"ticker":"PNPL","title":"PINEAPPLE, INC."},"5698":{"cik_str":1858912,"ticker":"GDNR","title":"Gardiner Healthcare Acquisitions Corp."},"5699":{"cik_str":1723580,"ticker":"BFI","title":"BurgerFi International, Inc."},"5700":{"cik_str":1396536,"ticker":"DUOT","title":"DUOS TECHNOLOGIES GROUP, INC."},"5701":{"cik_str":1709626,"ticker":"NCNA","title":"NuCana plc"},"5702":{"cik_str":946936,"ticker":"DSWL","title":"DESWELL INDUSTRIES INC"},"5703":{"cik_str":1681087,"ticker":"AVRO","title":"AVROBIO, Inc."},"5704":{"cik_str":1413119,"ticker":"KBLB","title":"Kraig Biocraft Laboratories, Inc"},"5705":{"cik_str":1821468,"ticker":"YQ","title":"17 Education & Technology Group Inc."},"5706":{"cik_str":1546296,"ticker":"IPDN","title":"Professional Diversity Network, Inc."},"5707":{"cik_str":934796,"ticker":"NWCN","title":"NETWORK CN INC"},"5708":{"cik_str":1724979,"ticker":"RAIN","title":"Rain Oncology Inc."},"5709":{"cik_str":1651407,"ticker":"CKPT","title":"Checkpoint Therapeutics, Inc."},"5710":{"cik_str":839470,"ticker":"WWR","title":"WESTWATER RESOURCES, INC."},"5711":{"cik_str":787253,"ticker":"NAII","title":"NATURAL ALTERNATIVES INTERNATIONAL INC"},"5712":{"cik_str":748790,"ticker":"GCEH","title":"Global Clean Energy Holdings, Inc."},"5713":{"cik_str":707605,"ticker":"ASRV","title":"AMERISERV FINANCIAL INC /PA/"},"5714":{"cik_str":894556,"ticker":"GEVI","title":"General Enterprise Ventures, Inc."},"5715":{"cik_str":1005101,"ticker":"MGLD","title":"Marygold Companies, Inc."},"5716":{"cik_str":1799191,"ticker":"TOI","title":"Oncology Institute, Inc."},"5717":{"cik_str":1799448,"ticker":"ALGS","title":"Aligos Therapeutics, Inc."},"5718":{"cik_str":25743,"ticker":"TXMD","title":"TherapeuticsMD, Inc."},"5719":{"cik_str":1624326,"ticker":"PAVM","title":"PAVmed Inc."},"5720":{"cik_str":722572,"ticker":"FKWL","title":"FRANKLIN WIRELESS CORP"},"5721":{"cik_str":1854572,"ticker":"IZM","title":"ICZOOM Group Inc."},"5722":{"cik_str":1880613,"ticker":"DRCT","title":"Direct Digital Holdings, Inc."},"5723":{"cik_str":1874097,"ticker":"CYN","title":"Cyngn Inc."},"5724":{"cik_str":1397616,"ticker":"SSVFF","title":"SOUTHERN SILVER EXPLORATION CORP"},"5725":{"cik_str":895464,"ticker":"YBGJ","title":"Yubo International Biotech Ltd"},"5726":{"cik_str":1844389,"ticker":"ACBA","title":"Ace Global Business Acquisition Ltd"},"5727":{"cik_str":740664,"ticker":"RFIL","title":"R F INDUSTRIES LTD"},"5728":{"cik_str":1310488,"ticker":"BFNH","title":"BIOFORCE NANOSCIENCES HOLDINGS, INC."},"5729":{"cik_str":942895,"ticker":"REDW","title":"REDWOOD FINANCIAL INC /MN/"},"5730":{"cik_str":1111741,"ticker":"DYNR","title":"DYNARESOURCE INC"},"5731":{"cik_str":1709505,"ticker":"BEST","title":"BEST Inc."},"5732":{"cik_str":808326,"ticker":"EMKR","title":"EMCORE CORP"},"5733":{"cik_str":839087,"ticker":"VASO","title":"VASO Corp"},"5734":{"cik_str":1038277,"ticker":"INIS","title":"INTERNATIONAL ISOTOPES INC"},"5735":{"cik_str":921114,"ticker":"ARMP","title":"Armata Pharmaceuticals, Inc."},"5736":{"cik_str":1720161,"ticker":"CTRM","title":"Castor Maritime Inc."},"5737":{"cik_str":1833214,"ticker":"SABS","title":"SAB Biotherapeutics, Inc."},"5738":{"cik_str":1769697,"ticker":"LITM","title":"Snow Lake Resources Ltd."},"5739":{"cik_str":1031233,"ticker":"PPBN","title":"PINNACLE BANKSHARES CORP"},"5740":{"cik_str":1868419,"ticker":"WAVS","title":"Western Acquisition Ventures Corp."},"5741":{"cik_str":1210618,"ticker":"SPI","title":"SPI Energy Co., Ltd."},"5742":{"cik_str":1607962,"ticker":"RWLK","title":"ReWalk Robotics Ltd."},"5743":{"cik_str":1495651,"ticker":"UURAF","title":"Ucore Rare Metals Inc."},"5744":{"cik_str":1785056,"ticker":"TRNR","title":"Interactive Strength, Inc."},"5745":{"cik_str":1583771,"ticker":"HEPA","title":"Hepion Pharmaceuticals, Inc."},"5746":{"cik_str":1692427,"ticker":"NCSM","title":"NCS Multistage Holdings, Inc."},"5747":{"cik_str":1419275,"ticker":"RVYL","title":"RYVYL Inc."},"5748":{"cik_str":45919,"ticker":"HHS","title":"HARTE HANKS INC"},"5749":{"cik_str":1275101,"ticker":"BOTJ","title":"BANK OF THE JAMES FINANCIAL GROUP INC"},"5750":{"cik_str":1172631,"ticker":"SUNW","title":"Sunworks, Inc."},"5751":{"cik_str":1421642,"ticker":"MSCLF","title":"Satellos Bioscience Inc."},"5752":{"cik_str":1757715,"ticker":"ATER","title":"Aterian, Inc."},"5753":{"cik_str":814676,"ticker":"CPSH","title":"CPS TECHNOLOGIES CORP/DE/"},"5754":{"cik_str":1380106,"ticker":"RPID","title":"RAPID MICRO BIOSYSTEMS, INC."},"5755":{"cik_str":1505611,"ticker":"DPSI","title":"DecisionPoint Systems, Inc."},"5756":{"cik_str":318833,"ticker":"TISI","title":"TEAM INC"},"5757":{"cik_str":1766526,"ticker":"TECTP","title":"Tectonic Financial, Inc."},"5758":{"cik_str":1849466,"ticker":"TCBS","title":"Texas Community Bancshares, Inc."},"5759":{"cik_str":1835022,"ticker":"COYA","title":"Coya Therapeutics, Inc."},"5760":{"cik_str":1036848,"ticker":"MPU","title":"Mega Matrix Corp."},"5761":{"cik_str":1943802,"ticker":"FSEA","title":"First Seacoast Bancorp, Inc."},"5762":{"cik_str":1816708,"ticker":"OWLT","title":"Owlet, Inc."},"5763":{"cik_str":1556179,"ticker":"RMRI","title":"Rocky Mountain Industrials, Inc."},"5764":{"cik_str":1472619,"ticker":"LONCF","title":"Loncor Gold Inc."},"5765":{"cik_str":1819794,"ticker":"HTOO","title":"Fusion Fuel Green PLC"},"5766":{"cik_str":1775194,"ticker":"UPXI","title":"UPEXI, INC."},"5767":{"cik_str":1824884,"ticker":"ADOC","title":"Edoc Acquisition Corp."},"5768":{"cik_str":1682241,"ticker":"MATH","title":"Metalpha Technology Holding Ltd"},"5769":{"cik_str":1162896,"ticker":"CRKR","title":"Prairie Operating Co."},"5770":{"cik_str":1806310,"ticker":"TSHA","title":"Taysha Gene Therapies, Inc."},"5771":{"cik_str":846913,"ticker":"FTEK","title":"FUEL TECH, INC."},"5772":{"cik_str":1299969,"ticker":"CHCI","title":"Comstock Holding Companies, Inc."},"5773":{"cik_str":1640043,"ticker":"PXS","title":"Pyxis Tankers Inc."},"5774":{"cik_str":1757097,"ticker":"CNTG","title":"Centogene N.V."},"5775":{"cik_str":1759186,"ticker":"COEP","title":"Coeptis Therapeutics Holdings, Inc."},"5776":{"cik_str":768216,"ticker":"LOGQ","title":"Coyni, Inc."},"5777":{"cik_str":1905660,"ticker":"HUBC","title":"Hub Cyber Security Ltd."},"5778":{"cik_str":27093,"ticker":"USAU","title":"U.S. GOLD CORP."},"5779":{"cik_str":1714562,"ticker":"GAME","title":"GameSquare Holdings, Inc."},"5780":{"cik_str":1705402,"ticker":"AGMH","title":"AGM GROUP HOLDINGS, INC."},"5781":{"cik_str":825324,"ticker":"GTIM","title":"Good Times Restaurants Inc."},"5782":{"cik_str":811641,"ticker":"ICCC","title":"IMMUCELL CORP /DE/"},"5783":{"cik_str":1790625,"ticker":"AGIL","title":"AgileThought, Inc."},"5784":{"cik_str":1823086,"ticker":"KWAC","title":"Kingswood Acquisition Corp."},"5785":{"cik_str":1119083,"ticker":"MNDO","title":"MIND CTI LTD"},"5786":{"cik_str":730255,"ticker":"CAMP","title":"CalAmp Corp."},"5787":{"cik_str":1042187,"ticker":"CTIB","title":"Yunhong CTI Ltd."},"5788":{"cik_str":1815849,"ticker":"ATIP","title":"ATI Physical Therapy, Inc."},"5789":{"cik_str":1813783,"ticker":"VMAR","title":"Vision Marine Technologies Inc."},"5790":{"cik_str":1939801,"ticker":"SMDRF","title":"SIERRA MADRE GOLD & SILVER LTD."},"5791":{"cik_str":1708341,"ticker":"AGAE","title":"Allied Gaming & Entertainment Inc."},"5792":{"cik_str":1731388,"ticker":"EDRY","title":"EuroDry Ltd."},"5793":{"cik_str":1263364,"ticker":"GTVI","title":"Idaho Copper Corp"},"5794":{"cik_str":1460602,"ticker":"ORGS","title":"Orgenesis Inc."},"5795":{"cik_str":1821850,"ticker":"SUNL","title":"Sunlight Financial Holdings Inc."},"5796":{"cik_str":8177,"ticker":"AAME","title":"ATLANTIC AMERICAN CORP"},"5797":{"cik_str":1882963,"ticker":"AIB","title":"AIB Acquisition Corp"},"5798":{"cik_str":1817153,"ticker":"EDTX","title":"EdtechX Holdings Acquisition Corp. II"},"5799":{"cik_str":1652539,"ticker":"SNRG","title":"SusGlobal Energy Corp."},"5800":{"cik_str":351998,"ticker":"DAIO","title":"DATA I/O CORP"},"5801":{"cik_str":1846084,"ticker":"JETMF","title":"Global Crossing Airlines Group Inc."},"5802":{"cik_str":1463361,"ticker":"UBX","title":"Unity Biotechnology, Inc."},"5803":{"cik_str":1876766,"ticker":"LICN","title":"Lichen China Ltd"},"5804":{"cik_str":1391933,"ticker":"QNTO","title":"QUAINT OAK BANCORP INC"},"5805":{"cik_str":1783036,"ticker":"NLSP","title":"NLS Pharmaceutics Ltd."},"5806":{"cik_str":1902794,"ticker":"MGOL","title":"MGO Global Inc."},"5807":{"cik_str":1495231,"ticker":"IZEA","title":"IZEA Worldwide, Inc."},"5808":{"cik_str":1858028,"ticker":"NOVV","title":"Nova Vision Acquisition Corp"},"5809":{"cik_str":1849670,"ticker":"PBBK","title":"PB Bankshares, Inc."},"5810":{"cik_str":1834975,"ticker":"VEV","title":"VICINITY MOTOR CORP"},"5811":{"cik_str":1830749,"ticker":"ACHL","title":"Achilles Therapeutics plc"},"5812":{"cik_str":1832415,"ticker":"BTTX","title":"Better Therapeutics, Inc."},"5813":{"cik_str":101295,"ticker":"UG","title":"UNITED GUARDIAN INC"},"5814":{"cik_str":1498067,"ticker":"CTGL","title":"CITRINE GLOBAL, CORP."},"5815":{"cik_str":101594,"ticker":"USEG","title":"US ENERGY CORP"},"5816":{"cik_str":1734875,"ticker":"MSVB","title":"Mid-Southern Bancorp, Inc."},"5817":{"cik_str":1509745,"ticker":"LPTX","title":"LEAP THERAPEUTICS, INC."},"5818":{"cik_str":1078207,"ticker":"NLS","title":"NAUTILUS, INC."},"5819":{"cik_str":1537028,"ticker":"ICD","title":"Independence Contract Drilling, Inc."},"5820":{"cik_str":1679817,"ticker":"OZSC","title":"OZOP ENERGY SOLUTIONS, INC."},"5821":{"cik_str":1532346,"ticker":"BCEL","title":"Atreca, Inc."},"5822":{"cik_str":1053369,"ticker":"ELTP","title":"ELITE PHARMACEUTICALS INC /NV/"},"5823":{"cik_str":1775085,"ticker":"CLEU","title":"China Liberal Education Holdings Ltd"},"5824":{"cik_str":1333172,"ticker":"NM","title":"Navios Maritime Holdings Inc."},"5825":{"cik_str":1844507,"ticker":"AVHI","title":"Achari Ventures Holdings Corp. I"},"5826":{"cik_str":1642380,"ticker":"OCX","title":"Oncocyte Corp"},"5827":{"cik_str":1898766,"ticker":"TPET","title":"Trio Petroleum Corp."},"5828":{"cik_str":1708527,"ticker":"AZYO","title":"AZIYO BIOLOGICS, INC."},"5829":{"cik_str":1701756,"ticker":"GRIL","title":"Muscle Maker, Inc."},"5830":{"cik_str":1336050,"ticker":"EGF","title":"BlackRock Enhanced Government Fund, Inc."},"5831":{"cik_str":1695295,"ticker":"HYFM","title":"HYDROFARM HOLDINGS GROUP, INC."},"5832":{"cik_str":1819493,"ticker":"XOS","title":"Xos, Inc."},"5833":{"cik_str":1783032,"ticker":"ELEV","title":"Elevation Oncology, Inc."},"5834":{"cik_str":1801762,"ticker":"MLYF","title":"Western Magnesium Corp."},"5835":{"cik_str":1722969,"ticker":"NXU","title":"Nxu, Inc."},"5836":{"cik_str":866439,"ticker":"DPLS","title":"DarkPulse, Inc."},"5837":{"cik_str":1854078,"ticker":"RLFTY","title":"Relief Therapeutics Holding SA"},"5838":{"cik_str":888721,"ticker":"TRIB","title":"TRINITY BIOTECH PLC"},"5839":{"cik_str":1482436,"ticker":"AGXPF","title":"SILVER X MINING CORP."},"5840":{"cik_str":1828805,"ticker":"ALMU","title":"Aeluma, Inc."},"5841":{"cik_str":1505065,"ticker":"BWAY","title":"Brainsway Ltd."},"5842":{"cik_str":1839360,"ticker":"FAZE","title":"FaZe Holdings Inc."},"5843":{"cik_str":1600422,"ticker":"SDPI","title":"Superior Drilling Products, Inc."},"5844":{"cik_str":867840,"ticker":"POCI","title":"PRECISION OPTICS CORPORATION, INC."},"5845":{"cik_str":1026980,"ticker":"APWC","title":"ASIA PACIFIC WIRE & CABLE CORP LTD"},"5846":{"cik_str":1070050,"ticker":"APCX","title":"AppTech Payments Corp."},"5847":{"cik_str":1899005,"ticker":"BMR","title":"Beamr Imaging Ltd."},"5848":{"cik_str":1907184,"ticker":"ELBM","title":"Electra Battery Materials Corp"},"5849":{"cik_str":1807983,"ticker":"MDNA","title":"Medicenna Therapeutics Corp."},"5850":{"cik_str":1619544,"ticker":"JFU","title":"9F Inc."},"5851":{"cik_str":1873093,"ticker":"GDTC","title":"CytoMed Therapeutics Ltd"},"5852":{"cik_str":1887603,"ticker":"HCNWF","title":"Hypercharge Networks Corp."},"5853":{"cik_str":1847806,"ticker":"GNS","title":"Genius Group Ltd"},"5854":{"cik_str":1865045,"ticker":"FATP","title":"FAT PROJECTS ACQUISITION CORP"},"5855":{"cik_str":1131312,"ticker":"ZNOG","title":"ZION OIL & GAS INC"},"5856":{"cik_str":1008654,"ticker":"TUP","title":"TUPPERWARE BRANDS CORP"},"5857":{"cik_str":1678848,"ticker":"BIMT","title":"BITMIS CORP."},"5858":{"cik_str":1867066,"ticker":"DERM","title":"Journey Medical Corp"},"5859":{"cik_str":917225,"ticker":"XPL","title":"SOLITARIO ZINC CORP."},"5860":{"cik_str":1596856,"ticker":"LEJU","title":"Leju Holdings Ltd"},"5861":{"cik_str":87802,"ticker":"SCND","title":"SCIENTIFIC INDUSTRIES INC"},"5862":{"cik_str":1181505,"ticker":"PYN","title":"PIMCO NEW YORK MUNICIPAL INCOME FUND III"},"5863":{"cik_str":1841387,"ticker":"CADL","title":"Candel Therapeutics, Inc."},"5864":{"cik_str":1051514,"ticker":"SSKN","title":"STRATA Skin Sciences, Inc."},"5865":{"cik_str":1056285,"ticker":"KIRK","title":"KIRKLAND'S, INC"},"5866":{"cik_str":1406944,"ticker":"TRBMF","title":"TORQ RESOURCES INC."},"5867":{"cik_str":857949,"ticker":"CGA","title":"China Green Agriculture, Inc."},"5868":{"cik_str":831609,"ticker":"CXXIF","title":"C21 Investments Inc."},"5869":{"cik_str":946454,"ticker":"FGH","title":"FG Group Holdings Inc."},"5870":{"cik_str":1182731,"ticker":"CNNN","title":"ConneXionONE Corp."},"5871":{"cik_str":1762322,"ticker":"SFT","title":"SHIFT TECHNOLOGIES, INC."},"5872":{"cik_str":1894954,"ticker":"XPON","title":"Expion360 Inc."},"5873":{"cik_str":1082324,"ticker":"VHC","title":"VirnetX Holding Corp"},"5874":{"cik_str":909724,"ticker":"TLF","title":"TANDY LEATHER FACTORY INC"},"5875":{"cik_str":1687919,"ticker":"FGNV","title":"FORGE INNOVATION DEVELOPMENT CORP."},"5876":{"cik_str":1830072,"ticker":"IPW","title":"iPower Inc."},"5877":{"cik_str":1834585,"ticker":"JOAN","title":"JOANN Inc."},"5878":{"cik_str":862022,"ticker":"HGTXU","title":"HUGOTON ROYALTY TRUST"},"5879":{"cik_str":1365388,"ticker":"HSTI","title":"High Sierra Technologies, Inc."},"5880":{"cik_str":845819,"ticker":"KTEL","title":"KonaTel, Inc."},"5881":{"cik_str":1000045,"ticker":"NICK","title":"NICHOLAS FINANCIAL INC"},"5882":{"cik_str":1517496,"ticker":"BTCM","title":"BIT Mining Ltd"},"5883":{"cik_str":1404644,"ticker":"NLTX","title":"Neoleukin Therapeutics, Inc."},"5884":{"cik_str":89140,"ticker":"SVT","title":"SERVOTRONICS INC /DE/"},"5885":{"cik_str":1600983,"ticker":"KSCP","title":"Knightscope, Inc."},"5886":{"cik_str":1638298,"ticker":"UGEIF","title":"UGE International Ltd."},"5887":{"cik_str":1818644,"ticker":"LIDR","title":"AEye, Inc."},"5888":{"cik_str":1696355,"ticker":"BEDU","title":"Bright Scholar Education Holdings Ltd"},"5889":{"cik_str":1701114,"ticker":"APRN","title":"Blue Apron Holdings, Inc."},"5890":{"cik_str":1021096,"ticker":"TRKA","title":"Troika Media Group, Inc."},"5891":{"cik_str":1719406,"ticker":"NRXP","title":"NRX Pharmaceuticals, Inc."},"5892":{"cik_str":1855557,"ticker":"HLP","title":"Hongli Group Inc."},"5893":{"cik_str":1643715,"ticker":"LISMF","title":"LITHIUM SOUTH DEVELOPMENT Corp"},"5894":{"cik_str":1496383,"ticker":"ILUS","title":"Ilustrato Pictures International Inc."},"5895":{"cik_str":1725872,"ticker":"BMTX","title":"BM Technologies, Inc."},"5896":{"cik_str":1761911,"ticker":"NOWG","title":"Nowigence Inc."},"5897":{"cik_str":1595097,"ticker":"CRBP","title":"Corbus Pharmaceuticals Holdings, Inc."},"5898":{"cik_str":1881472,"ticker":"MEGL","title":"Magic Empire Global Ltd"},"5899":{"cik_str":1117057,"ticker":"PLAG","title":"Planet Green Holdings Corp."},"5900":{"cik_str":1692415,"ticker":"CODX","title":"Co-Diagnostics, Inc."},"5901":{"cik_str":1679063,"ticker":"CSSE","title":"Chicken Soup for the Soul Entertainment, Inc."},"5902":{"cik_str":1479419,"ticker":"KALA","title":"Kala Pharmaceuticals, Inc."},"5903":{"cik_str":1374339,"ticker":"PMN","title":"ProMIS Neurosciences Inc."},"5904":{"cik_str":1133062,"ticker":"JANL","title":"JANEL CORP"},"5905":{"cik_str":1310291,"ticker":"BWVI","title":"PSYCHECEUTICAL BIOSCIENCE, INC."},"5906":{"cik_str":1828253,"ticker":"EVAX","title":"Evaxion Biotech A/S"},"5907":{"cik_str":1915403,"ticker":"SATX","title":"SatixFy Communications Ltd."},"5908":{"cik_str":1759614,"ticker":"YJ","title":"Yunji Inc."},"5909":{"cik_str":1484565,"ticker":"SLNO","title":"SOLENO THERAPEUTICS INC"},"5910":{"cik_str":1804469,"ticker":"GFAI","title":"Guardforce AI Co., Ltd."},"5911":{"cik_str":1006045,"ticker":"IRIX","title":"IRIDEX CORP"},"5912":{"cik_str":850027,"ticker":"GRF","title":"EAGLE CAPITAL GROWTH FUND, INC."},"5913":{"cik_str":1551693,"ticker":"SIEN","title":"Sientra, Inc."},"5914":{"cik_str":56701,"ticker":"KOSS","title":"KOSS CORP"},"5915":{"cik_str":1579157,"ticker":"VNCE","title":"VINCE HOLDING CORP."},"5916":{"cik_str":1857910,"ticker":"DRFS","title":"Dr. Foods, Inc."},"5917":{"cik_str":1627469,"ticker":"PTZH","title":"Photozou Holdings, Inc."},"5918":{"cik_str":1826135,"ticker":"MHHC","title":"MHHC Enterprises Inc."},"5919":{"cik_str":1069394,"ticker":"FSI","title":"FLEXIBLE SOLUTIONS INTERNATIONAL INC"},"5920":{"cik_str":1098146,"ticker":"PNBK","title":"PATRIOT NATIONAL BANCORP INC"},"5921":{"cik_str":946563,"ticker":"RVP","title":"RETRACTABLE TECHNOLOGIES INC"},"5922":{"cik_str":81955,"ticker":"RAND","title":"RAND CAPITAL CORP"},"5923":{"cik_str":1590496,"ticker":"AKOM","title":"Aerkomm Inc."},"5924":{"cik_str":1015739,"ticker":"AWRE","title":"AWARE INC /MA/"},"5925":{"cik_str":1779476,"ticker":"KRKR","title":"36Kr Holdings Inc."},"5926":{"cik_str":1847075,"ticker":"SAI","title":"SAI.TECH Global Corp"},"5927":{"cik_str":278165,"ticker":"OMQS","title":"OMNIQ Corp."},"5928":{"cik_str":1843656,"ticker":"AMAO","title":"American Acquisition Opportunity Inc."},"5929":{"cik_str":1738758,"ticker":"GSMG","title":"GLORY STAR NEW MEDIA GROUP HOLDINGS Ltd"},"5930":{"cik_str":1759546,"ticker":"RIDE","title":"Lordstown Motors Corp."},"5931":{"cik_str":1158780,"ticker":"PLUR","title":"Pluri Inc."},"5932":{"cik_str":1776932,"ticker":"MMNFF","title":"MedMen Enterprises, Inc."},"5933":{"cik_str":1378140,"ticker":"OPTT","title":"Ocean Power Technologies, Inc."},"5934":{"cik_str":1839608,"ticker":"GETR","title":"Getaround, Inc"},"5935":{"cik_str":1621906,"ticker":"WSTRF","title":"Western Uranium & Vanadium Corp."},"5936":{"cik_str":1474167,"ticker":"COSM","title":"Cosmos Health Inc."},"5937":{"cik_str":764897,"ticker":"BRST","title":"Broad Street Realty, Inc."},"5938":{"cik_str":1962738,"ticker":"CASI","title":"CASI Pharmaceuticals, Inc."},"5939":{"cik_str":1602078,"ticker":"NMRD","title":"Nemaura Medical Inc."},"5940":{"cik_str":1030471,"ticker":"UTSI","title":"UTSTARCOM HOLDINGS CORP."},"5941":{"cik_str":1610682,"ticker":"USDP","title":"USD Partners LP"},"5942":{"cik_str":1577445,"ticker":"SCTC","title":"Odysight.ai Inc."},"5943":{"cik_str":1738699,"ticker":"WKEY","title":"Wisekey International Holding S.A."},"5944":{"cik_str":1023109,"ticker":"CVVUF","title":"CANALASKA URANIUM LTD"},"5945":{"cik_str":1616262,"ticker":"RMCF","title":"Rocky Mountain Chocolate Factory, Inc."},"5946":{"cik_str":1829802,"ticker":"SNSE","title":"Sensei Biotherapeutics, Inc."},"5947":{"cik_str":1667313,"ticker":"ZDGE","title":"Zedge, Inc."},"5948":{"cik_str":946644,"ticker":"AIM","title":"AIM ImmunoTech Inc."},"5949":{"cik_str":1098880,"ticker":"IGXT","title":"IntelGenx Technologies Corp."},"5950":{"cik_str":1882607,"ticker":"ALPS","title":"ALPINE SUMMIT ENERGY PARTNERS, INC."},"5951":{"cik_str":101538,"ticker":"UAMY","title":"UNITED STATES ANTIMONY CORP"},"5952":{"cik_str":833021,"ticker":"CIF","title":"MFS INTERMEDIATE HIGH INCOME FUND"},"5953":{"cik_str":1763543,"ticker":"UWHGF","title":"United World Holding Group Ltd."},"5954":{"cik_str":1737339,"ticker":"JG","title":"Aurora Mobile Ltd"},"5955":{"cik_str":1468639,"ticker":"VICP","title":"Vicapsys Life Sciences, Inc."},"5956":{"cik_str":1305253,"ticker":"EIGR","title":"Eiger BioPharmaceuticals, Inc."},"5957":{"cik_str":1083220,"ticker":"XELB","title":"XCel Brands, Inc."},"5958":{"cik_str":1575858,"ticker":"PUBC","title":"PureBase Corp"},"5959":{"cik_str":1854583,"ticker":"ASPA","title":"Abri SPAC I, Inc."},"5960":{"cik_str":1391135,"ticker":"LIFD","title":"LFTD PARTNERS INC."},"5961":{"cik_str":1864055,"ticker":"MOXC","title":"Moxian (BVI) Inc"},"5962":{"cik_str":1744494,"ticker":"ADN","title":"ADVENT TECHNOLOGIES HOLDINGS, INC."},"5963":{"cik_str":1777319,"ticker":"CISO","title":"CISO Global, Inc."},"5964":{"cik_str":1836934,"ticker":"ZENV","title":"Zenvia Inc."},"5965":{"cik_str":1397047,"ticker":"FPAY","title":"FlexShopper, Inc."},"5966":{"cik_str":829323,"ticker":"INUV","title":"Inuvo, Inc."},"5967":{"cik_str":1430300,"ticker":"GWSO","title":"Global Warming Solutions, Inc."},"5968":{"cik_str":1094038,"ticker":"MRKR","title":"Marker Therapeutics, Inc."},"5969":{"cik_str":1962911,"ticker":"INFT","title":"Infinity Bancorp"},"5970":{"cik_str":1404281,"ticker":"ELDN","title":"Eledon Pharmaceuticals, Inc."},"5971":{"cik_str":1779578,"ticker":"QLI","title":"Qilian International Holding Group Ltd"},"5972":{"cik_str":883975,"ticker":"MBOT","title":"Microbot Medical Inc."},"5973":{"cik_str":1218683,"ticker":"MIGI","title":"Mawson Infrastructure Group Inc."},"5974":{"cik_str":1902314,"ticker":"MPTI","title":"M-tron Industries, Inc."},"5975":{"cik_str":1855467,"ticker":"CLAY","title":"Chavant Capital Acquisition Corp."},"5976":{"cik_str":845385,"ticker":"PIAC","title":"PRINCETON CAPITAL CORP"},"5977":{"cik_str":1736243,"ticker":"ACXP","title":"Acurx Pharmaceuticals, Inc."},"5978":{"cik_str":1785566,"ticker":"ZCMD","title":"Zhongchao Inc."},"5979":{"cik_str":1770141,"ticker":"UPH","title":"UpHealth, Inc."},"5980":{"cik_str":1686000,"ticker":"JGLDF","title":"Japan Gold Corp."},"5981":{"cik_str":1871983,"ticker":"ANGH","title":"Anghami Inc"},"5982":{"cik_str":1563568,"ticker":"EVTV","title":"Envirotech Vehicles, Inc."},"5983":{"cik_str":927719,"ticker":"DWNX","title":"DELHI BANK CORP"},"5984":{"cik_str":1000230,"ticker":"OCC","title":"OPTICAL CABLE CORP"},"5985":{"cik_str":1614744,"ticker":"PPBT","title":"PURPLE BIOTECH LTD."},"5986":{"cik_str":1811856,"ticker":"VIEW","title":"View, Inc."},"5987":{"cik_str":1892322,"ticker":"HTCR","title":"HeartCore Enterprises, Inc."},"5988":{"cik_str":1819516,"ticker":"UP","title":"Wheels Up Experience Inc."},"5989":{"cik_str":65759,"ticker":"MPAD","title":"MICROPAC INDUSTRIES INC"},"5990":{"cik_str":1604738,"ticker":"AINC","title":"Ashford Inc."},"5991":{"cik_str":1620179,"ticker":"XELA","title":"Exela Technologies, Inc."},"5992":{"cik_str":1101433,"ticker":"QMCI","title":"QUOTEMEDIA INC"},"5993":{"cik_str":1661053,"ticker":"NVNO","title":"enVVeno Medical Corp"},"5994":{"cik_str":1756180,"ticker":"NCRA","title":"NOCERA, INC."},"5995":{"cik_str":1425627,"ticker":"SOBR","title":"SOBR Safe, Inc."},"5996":{"cik_str":1160798,"ticker":"OOGI","title":"C2E ENERGY, INC."},"5997":{"cik_str":1784254,"ticker":"MDIA","title":"Mediaco Holding Inc."},"5998":{"cik_str":1651932,"ticker":"XITO","title":"Xenous Holdings, Inc."},"5999":{"cik_str":1465470,"ticker":"SHMP","title":"NaturalShrimp Inc"},"6000":{"cik_str":1781162,"ticker":"MNTS","title":"Momentus Inc."},"6001":{"cik_str":1928948,"ticker":"CACO","title":"Caravelle International Group"},"6002":{"cik_str":1616741,"ticker":"PATI","title":"PATRIOT TRANSPORTATION HOLDING, INC."},"6003":{"cik_str":1636282,"ticker":"AGLE","title":"Aeglea BioTherapeutics, Inc."},"6004":{"cik_str":1741220,"ticker":"CWPE","title":"CW Petroleum Corp"},"6005":{"cik_str":856128,"ticker":"MFV","title":"MFS SPECIAL VALUE TRUST"},"6006":{"cik_str":1099160,"ticker":"BBGI","title":"BEASLEY BROADCAST GROUP INC"},"6007":{"cik_str":1493130,"ticker":"MAPPF","title":"PROSTAR HOLDINGS INC. /BC"},"6008":{"cik_str":1527728,"ticker":"ENOB","title":"Enochian Biosciences Inc"},"6009":{"cik_str":1707303,"ticker":"DOGZ","title":"Dogness (International) Corp"},"6010":{"cik_str":882361,"ticker":"APTO","title":"Aptose Biosciences Inc."},"6011":{"cik_str":1445499,"ticker":"IMPL","title":"IMPEL PHARMACEUTICALS INC"},"6012":{"cik_str":1840416,"ticker":"SEV","title":"Sono Group N.V."},"6013":{"cik_str":1878313,"ticker":"MAIA","title":"MAIA Biotechnology, Inc."},"6014":{"cik_str":888981,"ticker":"NNUP","title":"NOCOPI TECHNOLOGIES INC/MD/"},"6015":{"cik_str":907654,"ticker":"ABIO","title":"ARCA biopharma, Inc."},"6016":{"cik_str":1592782,"ticker":"NUKK","title":"Nukkleus Inc."},"6017":{"cik_str":1650575,"ticker":"BRQS","title":"Borqs Technologies, Inc."},"6018":{"cik_str":1699709,"ticker":"YJGJ","title":"YIJIA GROUP CORP."},"6019":{"cik_str":1505512,"ticker":"RGLS","title":"Regulus Therapeutics Inc."},"6020":{"cik_str":1938046,"ticker":"MGRX","title":"MANGOCEUTICALS, INC."},"6021":{"cik_str":1004989,"ticker":"SGRP","title":"SPAR Group, Inc."},"6022":{"cik_str":1408057,"ticker":"ICCT","title":"iCoreConnect Inc."},"6023":{"cik_str":1288770,"ticker":"XTGRF","title":"XTRA-GOLD RESOURCES CORP"},"6024":{"cik_str":719739,"ticker":"SIVBQ","title":"SVB FINANCIAL GROUP"},"6025":{"cik_str":1559356,"ticker":"IMUN","title":"Immune Therapeutics, Inc."},"6026":{"cik_str":1650287,"ticker":"QYOUF","title":"QYOU Media Inc."},"6027":{"cik_str":1708493,"ticker":"HARP","title":"Harpoon Therapeutics, Inc."},"6028":{"cik_str":718332,"ticker":"RAVE","title":"RAVE RESTAURANT GROUP, INC."},"6029":{"cik_str":1509223,"ticker":"MTBL","title":"Moatable, Inc."},"6030":{"cik_str":315958,"ticker":"CRMZ","title":"CREDITRISKMONITOR COM INC"},"6031":{"cik_str":1519469,"ticker":"ANLDF","title":"ANFIELD ENERGY INC."},"6032":{"cik_str":896494,"ticker":"SNT","title":"SENSTAR TECHNOLOGIES LTD."},"6033":{"cik_str":1340476,"ticker":"DRTT","title":"DIRTT ENVIRONMENTAL SOLUTIONS LTD"},"6034":{"cik_str":1335112,"ticker":"LGIQ","title":"LOGIQ, INC."},"6035":{"cik_str":1854270,"ticker":"SNTI","title":"Senti Biosciences, Inc."},"6036":{"cik_str":1015155,"ticker":"CTHR","title":"CHARLES & COLVARD LTD"},"6037":{"cik_str":1708599,"ticker":"AGE","title":"AgeX Therapeutics, Inc."},"6038":{"cik_str":1533030,"ticker":"CRYM","title":"Cryomass Technologies, Inc."},"6039":{"cik_str":1676047,"ticker":"NTRB","title":"NutriBand Inc."},"6040":{"cik_str":1750155,"ticker":"CWBHF","title":"Charlotte's Web Holdings, Inc."},"6041":{"cik_str":320017,"ticker":"LSTA","title":"LISATA THERAPEUTICS, INC."},"6042":{"cik_str":1418149,"ticker":"HLRTD","title":"Hillcrest Energy Technologies Ltd."},"6043":{"cik_str":83350,"ticker":"RSRV","title":"RESERVE PETROLEUM CO"},"6044":{"cik_str":1592560,"ticker":"TEDU","title":"Tarena International, Inc."},"6045":{"cik_str":1856028,"ticker":"SDIG","title":"Stronghold Digital Mining, Inc."},"6046":{"cik_str":838875,"ticker":"WVVI","title":"WILLAMETTE VALLEY VINEYARDS INC"},"6047":{"cik_str":1138867,"ticker":"EESH","title":"EESTech, Inc."},"6048":{"cik_str":1527599,"ticker":"SYBX","title":"SYNLOGIC, INC."},"6049":{"cik_str":1724542,"ticker":"CLPS","title":"CLPS Inc"},"6050":{"cik_str":1691936,"ticker":"SNAX","title":"STRYVE FOODS, INC."},"6051":{"cik_str":1796129,"ticker":"VINC","title":"Vincerx Pharma, Inc."},"6052":{"cik_str":1359931,"ticker":"TARA","title":"Protara Therapeutics, Inc."},"6053":{"cik_str":1735948,"ticker":"GBNH","title":"Greenbrook TMS Inc."},"6054":{"cik_str":1859035,"ticker":"MCAG","title":"Mountain Crest Acquisition Corp. V"},"6055":{"cik_str":1282224,"ticker":"DLPN","title":"Dolphin Entertainment, Inc."},"6056":{"cik_str":1939965,"ticker":"BREA","title":"Brera Holdings PLC"},"6057":{"cik_str":1039280,"ticker":"NTWK","title":"NETSOL TECHNOLOGIES INC"},"6058":{"cik_str":926617,"ticker":"AWH","title":"Aspira Women's Health Inc."},"6059":{"cik_str":79661,"ticker":"PRSI","title":"PORTSMOUTH SQUARE INC"},"6060":{"cik_str":1076691,"ticker":"OSBK","title":"OCONEE FINANCIAL CORP"},"6061":{"cik_str":728385,"ticker":"MEEC","title":"Midwest Energy Emissions Corp."},"6062":{"cik_str":1545634,"ticker":"CDBMF","title":"Cordoba Minerals Corp."},"6063":{"cik_str":1746466,"ticker":"EQ","title":"Equillium, Inc."},"6064":{"cik_str":1760233,"ticker":"RSTN","title":"RDE, Inc."},"6065":{"cik_str":885462,"ticker":"GURE","title":"GULF RESOURCES, INC."},"6066":{"cik_str":1815632,"ticker":"TGCB","title":"Tego Cyber, Inc."},"6067":{"cik_str":1938865,"ticker":"TCJH","title":"Top KingWin Ltd"},"6068":{"cik_str":1183765,"ticker":"MTEM","title":"Molecular Templates, Inc."},"6069":{"cik_str":1631282,"ticker":"DTSS","title":"DATASEA INC."},"6070":{"cik_str":1786511,"ticker":"FRES","title":"Fresh2 Group Ltd"},"6071":{"cik_str":1819113,"ticker":"SNCE","title":"Science 37 Holdings, Inc."},"6072":{"cik_str":1861449,"ticker":"BRDS","title":"Bird Global, Inc."},"6073":{"cik_str":736012,"ticker":"INTZ","title":"INTRUSION INC"},"6074":{"cik_str":1467913,"ticker":"RNWR","title":"808 RENEWABLE ENERGY CORP"},"6075":{"cik_str":1865429,"ticker":"OAKV","title":"Oak View Bankshares, Inc."},"6076":{"cik_str":880406,"ticker":"CUBA","title":"HERZFELD CARIBBEAN BASIN FUND INC"},"6077":{"cik_str":1403749,"ticker":"FLYLF","title":"FLYHT Aerospace Solutions Ltd."},"6078":{"cik_str":86115,"ticker":"SFE","title":"SAFEGUARD SCIENTIFICS INC"},"6079":{"cik_str":1126234,"ticker":"LUMO","title":"LUMOS PHARMA, INC."},"6080":{"cik_str":1050606,"ticker":"SALM","title":"SALEM MEDIA GROUP, INC. /DE/"},"6081":{"cik_str":1054721,"ticker":"BSQR","title":"BSQUARE CORP /WA"},"6082":{"cik_str":1819704,"ticker":"MRM","title":"Medirom Healthcare Technologies Inc."},"6083":{"cik_str":1402945,"ticker":"RXMD","title":"Progressive Care Inc."},"6084":{"cik_str":1658678,"ticker":"BSEM","title":"BIOSTEM TECHNOLOGIES"},"6085":{"cik_str":722313,"ticker":"NSYS","title":"NORTECH SYSTEMS INC"},"6086":{"cik_str":1603978,"ticker":"AQB","title":"AquaBounty Technologies, Inc."},"6087":{"cik_str":1522860,"ticker":"AFIB","title":"Acutus Medical, Inc."},"6088":{"cik_str":1267602,"ticker":"ALIM","title":"ALIMERA SCIENCES INC"},"6089":{"cik_str":806517,"ticker":"PMD","title":"PSYCHEMEDICS CORP"},"6090":{"cik_str":1321851,"ticker":"SEED","title":"Origin Agritech LTD"},"6091":{"cik_str":66418,"ticker":"MXC","title":"MEXCO ENERGY CORP"},"6092":{"cik_str":1144546,"ticker":"HWTR","title":"HFactor, Inc."},"6093":{"cik_str":1088413,"ticker":"GLGI","title":"GREYSTONE LOGISTICS, INC."},"6094":{"cik_str":1535955,"ticker":"LPCN","title":"Lipocine Inc."},"6095":{"cik_str":1815021,"ticker":"BQ","title":"Boqii Holding Ltd"},"6096":{"cik_str":1086745,"ticker":"AYRO","title":"AYRO, Inc."},"6097":{"cik_str":1913838,"ticker":"VWFB","title":"VWF Bancorp, Inc."},"6098":{"cik_str":1296774,"ticker":"NCTY","title":"The9 LTD"},"6099":{"cik_str":1961592,"ticker":"GSHRF","title":"GOLDSHORE RESOURCES INC."},"6100":{"cik_str":10048,"ticker":"BRN","title":"BARNWELL INDUSTRIES INC"},"6101":{"cik_str":1842952,"ticker":"CNTX","title":"Context Therapeutics Inc."},"6102":{"cik_str":1553404,"ticker":"PGTK","title":"Pacific Green Technologies Inc."},"6103":{"cik_str":1844417,"ticker":"UPTD","title":"TradeUP Acquisition Corp."},"6104":{"cik_str":1703073,"ticker":"VIVC","title":"VIVIC CORP."},"6105":{"cik_str":1638097,"ticker":"ENTX","title":"Entera Bio Ltd."},"6106":{"cik_str":939930,"ticker":"PYYX","title":"PYXUS INTERNATIONAL, INC."},"6107":{"cik_str":1061164,"ticker":"PGID","title":"PEREGRINE INDUSTRIES INC"},"6108":{"cik_str":8504,"ticker":"UAVS","title":"AgEagle Aerial Systems Inc."},"6109":{"cik_str":1527102,"ticker":"EGSE","title":"Evergreen Sustainable Enterprises, Inc."},"6110":{"cik_str":1412486,"ticker":"COCP","title":"Cocrystal Pharma, Inc."},"6111":{"cik_str":942126,"ticker":"TAIT","title":"TAITRON COMPONENTS INC"},"6112":{"cik_str":61004,"ticker":"LGL","title":"LGL GROUP INC"},"6113":{"cik_str":1754323,"ticker":"PBTS","title":"Powerbridge Technologies Co., Ltd."},"6114":{"cik_str":1884072,"ticker":"JEWL","title":"Adamas One Corp."},"6115":{"cik_str":922247,"ticker":"CYTH","title":"Cyclo Therapeutics, Inc."},"6116":{"cik_str":1818844,"ticker":"VIRI","title":"Virios Therapeutics, Inc."},"6117":{"cik_str":1672572,"ticker":"LNBY","title":"Landbay Inc"},"6118":{"cik_str":1691221,"ticker":"FRSX","title":"Foresight Autonomous Holdings Ltd."},"6119":{"cik_str":1157075,"ticker":"PMCB","title":"PharmaCyte Biotech, Inc."},"6120":{"cik_str":1445815,"ticker":"BIXT","title":"BIOXYTRAN, INC"},"6121":{"cik_str":19871,"ticker":"CVR","title":"CHICAGO RIVET & MACHINE CO"},"6122":{"cik_str":1512922,"ticker":"PETV","title":"PetVivo Holdings, Inc."},"6123":{"cik_str":1108645,"ticker":"CIPI","title":"Correlate Infrastructure Partners, Inc."},"6124":{"cik_str":1393781,"ticker":"QIND","title":"Quality Industrial Corp."},"6125":{"cik_str":1751707,"ticker":"UNXP","title":"United Express Inc."},"6126":{"cik_str":1537435,"ticker":"TGEN","title":"TECOGEN INC."},"6127":{"cik_str":1499961,"ticker":"MULN","title":"MULLEN AUTOMOTIVE INC."},"6128":{"cik_str":1912847,"ticker":"USEA","title":"United Maritime Corp"},"6129":{"cik_str":1854963,"ticker":"SHFS","title":"SHF Holdings, Inc."},"6130":{"cik_str":1625285,"ticker":"ARMV","title":"Arma Services Inc"},"6131":{"cik_str":920990,"ticker":"WOWI","title":"METRO ONE TELECOMMUNICATIONS INC"},"6132":{"cik_str":911216,"ticker":"PTN","title":"PALATIN TECHNOLOGIES INC"},"6133":{"cik_str":1741231,"ticker":"TTCF","title":"Tattooed Chef, Inc."},"6134":{"cik_str":1877939,"ticker":"CTM","title":"Castellum, Inc."},"6135":{"cik_str":1807887,"ticker":"LASE","title":"Laser Photonics Corp"},"6136":{"cik_str":1575295,"ticker":"ALID","title":"Allied Corp."},"6137":{"cik_str":1349706,"ticker":"IMHC","title":"IMPERALIS HOLDING CORP."},"6138":{"cik_str":1297937,"ticker":"PRKA","title":"PARKS AMERICA, INC"},"6139":{"cik_str":1106213,"ticker":"SFRX","title":"SEAFARER EXPLORATION CORP"},"6140":{"cik_str":1672571,"ticker":"ACHN","title":"Antiaging Quantum Living Inc."},"6141":{"cik_str":1783432,"ticker":"TETOF","title":"Tectonic Metals Inc."},"6142":{"cik_str":1539680,"ticker":"HMMR","title":"HAMMER FIBER OPTICS HOLDINGS CORP"},"6143":{"cik_str":1842138,"ticker":"WRPT","title":"WARPSPEED TAXI INC."},"6144":{"cik_str":1713923,"ticker":"JT","title":"Jianpu Technology Inc."},"6145":{"cik_str":1784851,"ticker":"SHPW","title":"Shapeways Holdings, Inc."},"6146":{"cik_str":1018735,"ticker":"NYMX","title":"NYMOX PHARMACEUTICAL CORP"},"6147":{"cik_str":1509646,"ticker":"FENG","title":"Phoenix New Media Ltd"},"6148":{"cik_str":1855631,"ticker":"AWIN","title":"AERWINS Technologies Inc."},"6149":{"cik_str":1570843,"ticker":"GEBRF","title":"Greenbriar Capital Corp."},"6150":{"cik_str":1802749,"ticker":"ZEV","title":"Lightning eMotors, Inc."},"6151":{"cik_str":1920406,"ticker":"ASST","title":"Asset Entities Inc."},"6152":{"cik_str":1075880,"ticker":"KZIA","title":"KAZIA THERAPEUTICS LTD"},"6153":{"cik_str":1781726,"ticker":"NPLS","title":"NP Life Sciences Health Industry Group Inc."},"6154":{"cik_str":1445283,"ticker":"KA","title":"KINETA, INC./DE"},"6155":{"cik_str":1825875,"ticker":"RAYA","title":"Erayak Power Solution Group Inc."},"6156":{"cik_str":899782,"ticker":"NOM","title":"NUVEEN MISSOURI QUALITY MUNICIPAL INCOME FUND"},"6157":{"cik_str":1876945,"ticker":"GRAMF","title":"TPCO Holding Corp."},"6158":{"cik_str":1575793,"ticker":"WATT","title":"Energous Corp"},"6159":{"cik_str":1560258,"ticker":"ECOR","title":"electroCore, Inc."},"6160":{"cik_str":1574094,"ticker":"RNXT","title":"RenovoRx, Inc."},"6161":{"cik_str":1444192,"ticker":"ACST","title":"Acasti Pharma Inc."},"6162":{"cik_str":1813191,"ticker":"AUCUF","title":"Inflection Resources Ltd."},"6163":{"cik_str":1377149,"ticker":"CRVW","title":"CareView Communications Inc"},"6164":{"cik_str":1921865,"ticker":"ASPI","title":"ASP Isotopes Inc."},"6165":{"cik_str":1675634,"ticker":"PIXY","title":"ShiftPixy, Inc."},"6166":{"cik_str":1156041,"ticker":"HUSA","title":"HOUSTON AMERICAN ENERGY CORP"},"6167":{"cik_str":1205922,"ticker":"VCNX","title":"VACCINEX, INC."},"6168":{"cik_str":1069308,"ticker":"ACER","title":"Acer Therapeutics Inc."},"6169":{"cik_str":1356093,"ticker":"CREX","title":"CREATIVE REALITIES, INC."},"6170":{"cik_str":1741534,"ticker":"MDJH","title":"MDJM LTD"},"6171":{"cik_str":1101026,"ticker":"ZIVO","title":"Zivo Bioscience, Inc."},"6172":{"cik_str":1901305,"ticker":"FRZA","title":"Forza X1, Inc."},"6173":{"cik_str":1444839,"ticker":"BRVO","title":"Bravo Multinational Inc."},"6174":{"cik_str":1413891,"ticker":"HWNI","title":"HIGH WIRE NETWORKS, INC."},"6175":{"cik_str":1130598,"ticker":"ONTX","title":"Onconova Therapeutics, Inc."},"6176":{"cik_str":1518461,"ticker":"AMPG","title":"AmpliTech Group, Inc."},"6177":{"cik_str":764195,"ticker":"VBIV","title":"VBI Vaccines Inc/BC"},"6178":{"cik_str":1955083,"ticker":"GESI","title":"General European Strategic Investments, Inc."},"6179":{"cik_str":1056358,"ticker":"MTEX","title":"MANNATECH INC"},"6180":{"cik_str":868822,"ticker":"CRCUF","title":"Canagold Resources Ltd."},"6181":{"cik_str":1605481,"ticker":"NGLD","title":"Nevada Canyon Gold Corp."},"6182":{"cik_str":1397016,"ticker":"OPXS","title":"Optex Systems Holdings Inc"},"6183":{"cik_str":82473,"ticker":"IHT","title":"INNSUITES HOSPITALITY TRUST"},"6184":{"cik_str":1518561,"ticker":"EWGFF","title":"Eat Well Investment Group Inc."},"6185":{"cik_str":91668,"ticker":"SODI","title":"SOLITRON DEVICES INC"},"6186":{"cik_str":1762359,"ticker":"ACRHF","title":"Acreage Holdings, Inc."},"6187":{"cik_str":1814215,"ticker":"BURU","title":"Nuburu, Inc."},"6188":{"cik_str":1534133,"ticker":"CALC","title":"CalciMedica, Inc."},"6189":{"cik_str":1527702,"ticker":"IQST","title":"iQSTEL Inc"},"6190":{"cik_str":1367408,"ticker":"OILY","title":"Sino American Oil Co"},"6191":{"cik_str":1829247,"ticker":"BFRG","title":"BullFrog AI Holdings, Inc."},"6192":{"cik_str":890066,"ticker":"GLBZ","title":"GLEN BURNIE BANCORP"},"6193":{"cik_str":1138978,"ticker":"NVOS","title":"Novo Integrated Sciences, Inc."},"6194":{"cik_str":1840780,"ticker":"LOCL","title":"Local Bounti Corporation/DE"},"6195":{"cik_str":1777946,"ticker":"IRNT","title":"IronNet, Inc."},"6196":{"cik_str":1687451,"ticker":"ZKIN","title":"ZK International Group Co., Ltd."},"6197":{"cik_str":1001907,"ticker":"ASTC","title":"ASTROTECH Corp"},"6198":{"cik_str":1887673,"ticker":"WLDS","title":"Wearable Devices Ltd."},"6199":{"cik_str":1502292,"ticker":"CNFR","title":"Conifer Holdings, Inc."},"6200":{"cik_str":1873722,"ticker":"BBLR","title":"Bubblr Inc."},"6201":{"cik_str":1419041,"ticker":"FBRX","title":"Forte Biosciences, Inc."},"6202":{"cik_str":1288855,"ticker":"OPHC","title":"OptimumBank Holdings, Inc."},"6203":{"cik_str":73290,"ticker":"BMRA","title":"BIOMERICA INC"},"6204":{"cik_str":1087294,"ticker":"CPIX","title":"CUMBERLAND PHARMACEUTICALS INC"},"6205":{"cik_str":1173204,"ticker":"CNVS","title":"Cineverse Corp."},"6206":{"cik_str":724742,"ticker":"TPHS","title":"Trinity Place Holdings Inc."},"6207":{"cik_str":1083522,"ticker":"JSDA","title":"JONES SODA CO"},"6208":{"cik_str":1682056,"ticker":"GTAGF","title":"GOLDEN TAG RESOURCES LTD."},"6209":{"cik_str":1823365,"ticker":"GBNY","title":"Generations Bancorp NY, Inc."},"6210":{"cik_str":1084133,"ticker":"RLBD","title":"Real Brands, Inc."},"6211":{"cik_str":1903995,"ticker":"MGIH","title":"Millennium Group International Holdings Ltd"},"6212":{"cik_str":1009759,"ticker":"CGRN","title":"Capstone Green Energy Corp"},"6213":{"cik_str":1166272,"ticker":"GENE","title":"GENETIC TECHNOLOGIES LTD"},"6214":{"cik_str":1893448,"ticker":"SGE","title":"Strong Global Entertainment, Inc."},"6215":{"cik_str":1735041,"ticker":"GTEC","title":"Greenland Technologies Holding Corp."},"6216":{"cik_str":1624512,"ticker":"BOXL","title":"Boxlight Corp"},"6217":{"cik_str":1825349,"ticker":"CHSN","title":"Chanson International Holding"},"6218":{"cik_str":1844971,"ticker":"GREE","title":"Greenidge Generation Holdings Inc."},"6219":{"cik_str":1109486,"ticker":"EDXC","title":"Endexx Corp"},"6220":{"cik_str":1074871,"ticker":"MODD","title":"Modular Medical, Inc."},"6221":{"cik_str":1500305,"ticker":"SPGX","title":"Sustainable Projects Group Inc."},"6222":{"cik_str":1689448,"ticker":"PNXLF","title":"Argentina Lithium & Energy Corp."},"6223":{"cik_str":1134765,"ticker":"CHUC","title":"Charlie's Holdings, Inc."},"6224":{"cik_str":1066764,"ticker":"BTTC","title":"Bitech Technologies Corp"},"6225":{"cik_str":1403674,"ticker":"OMHI","title":"Portage Resources Inc."},"6226":{"cik_str":1260990,"ticker":"ONCT","title":"Oncternal Therapeutics, Inc."},"6227":{"cik_str":1413898,"ticker":"DALN","title":"DallasNews Corp"},"6228":{"cik_str":1652561,"ticker":"DQWS","title":"DSwiss Inc"},"6229":{"cik_str":1708441,"ticker":"GEHI","title":"Gravitas Education Holdings, Inc."},"6230":{"cik_str":1829794,"ticker":"VLCN","title":"Volcon, Inc."},"6231":{"cik_str":1899658,"ticker":"WLGS","title":"WANG & LEE GROUP, Inc."},"6232":{"cik_str":1019671,"ticker":"SEAC","title":"SEACHANGE INTERNATIONAL INC"},"6233":{"cik_str":1083490,"ticker":"SRRE","title":"SUNRISE REAL ESTATE GROUP INC"},"6234":{"cik_str":1103795,"ticker":"QSEP","title":"QS Energy, Inc."},"6235":{"cik_str":1743971,"ticker":"MOGU","title":"MOGU Inc."},"6236":{"cik_str":1450704,"ticker":"VIVK","title":"Vivakor, Inc."},"6237":{"cik_str":1814963,"ticker":"ILAG","title":"Intelligent Living Application Group Inc."},"6238":{"cik_str":1304161,"ticker":"PBSV","title":"Pharma-Bio Serv, Inc."},"6239":{"cik_str":797465,"ticker":"STLY","title":"HG Holdings, Inc."},"6240":{"cik_str":1168981,"ticker":"LVRLF","title":"CordovaCann Corp."},"6241":{"cik_str":1787518,"ticker":"BRLI","title":"Brilliant Acquisition Corp"},"6242":{"cik_str":878932,"ticker":"EQS","title":"EQUUS TOTAL RETURN, INC."},"6243":{"cik_str":1013706,"ticker":"WHLM","title":"Wilhelmina International, Inc."},"6244":{"cik_str":1751876,"ticker":"PAVS","title":"Paranovus Entertainment Technology Ltd."},"6245":{"cik_str":1449349,"ticker":"RDGL","title":"VIVOS INC"},"6246":{"cik_str":1547660,"ticker":"VSOLF","title":"THREE SIXTY SOLAR LTD."},"6247":{"cik_str":1490161,"ticker":"SOWG","title":"Sow Good Inc."},"6248":{"cik_str":867038,"ticker":"SPND","title":"SPINDLETOP OIL & GAS CO"},"6249":{"cik_str":1487798,"ticker":"ECTM","title":"ECA Marcellus Trust I"},"6250":{"cik_str":1066923,"ticker":"FTFT","title":"Future FinTech Group Inc."},"6251":{"cik_str":1516551,"ticker":"SKYE","title":"Skye Bioscience, Inc."},"6252":{"cik_str":1782037,"ticker":"YGMZ","title":"MingZhu Logistics Holdings Ltd"},"6253":{"cik_str":1500198,"ticker":"NMTC","title":"NEUROONE MEDICAL TECHNOLOGIES Corp"},"6254":{"cik_str":1855509,"ticker":"VEEE","title":"Twin Vee PowerCats, Co."},"6255":{"cik_str":29332,"ticker":"DXYN","title":"DIXIE GROUP INC"},"6256":{"cik_str":1090396,"ticker":"TBTC","title":"TABLE TRAC INC"},"6257":{"cik_str":895665,"ticker":"CLRD","title":"Clearday, Inc."},"6258":{"cik_str":915358,"ticker":"SGMA","title":"SIGMATRON INTERNATIONAL INC"},"6259":{"cik_str":1253176,"ticker":"VAPO","title":"VAPOTHERM INC"},"6260":{"cik_str":1730346,"ticker":"CHRA","title":"Charah Solutions, Inc."},"6261":{"cik_str":1487839,"ticker":"LKCO","title":"Luokung Technology Corp."},"6262":{"cik_str":75340,"ticker":"PFIN","title":"P&F INDUSTRIES INC"},"6263":{"cik_str":1279704,"ticker":"CLRB","title":"Cellectar Biosciences, Inc."},"6264":{"cik_str":1875091,"ticker":"NRSN","title":"NeuroSense Therapeutics Ltd."},"6265":{"cik_str":1402479,"ticker":"MDVL","title":"MedAvail Holdings, Inc."},"6266":{"cik_str":352955,"ticker":"CKX","title":"CKX LANDS, INC."},"6267":{"cik_str":732026,"ticker":"TRT","title":"TRIO-TECH INTERNATIONAL"},"6268":{"cik_str":1826376,"ticker":"GSUN","title":"Golden Sun Education Group Ltd"},"6269":{"cik_str":312257,"ticker":"IVFH","title":"INNOVATIVE FOOD HOLDINGS INC"},"6270":{"cik_str":1191070,"ticker":"BNOX","title":"BIONOMICS LIMITED/FI"},"6271":{"cik_str":1351172,"ticker":"LMNL","title":"Liminal BioSciences Inc."},"6272":{"cik_str":1684888,"ticker":"GFGSF","title":"GFG Resources Inc."},"6273":{"cik_str":1689382,"ticker":"TLLYF","title":"Trilogy International Partners Inc."},"6274":{"cik_str":1771706,"ticker":"GDNSF","title":"Goodness Growth Holdings, Inc."},"6275":{"cik_str":1446159,"ticker":"POAI","title":"Predictive Oncology Inc."},"6276":{"cik_str":1905956,"ticker":"TGL","title":"TREASURE GLOBAL INC"},"6277":{"cik_str":1892274,"ticker":"VEDU","title":"Visionary Education Technology Holdings Group Inc."},"6278":{"cik_str":840715,"ticker":"CLRO","title":"CLEARONE INC"},"6279":{"cik_str":1005516,"ticker":"BOSC","title":"BOS BETTER ONLINE SOLUTIONS LTD"},"6280":{"cik_str":1582249,"ticker":"RASP","title":"Rasna Therapeutics Inc."},"6281":{"cik_str":1908984,"ticker":"ENDI","title":"ENDI Corp."},"6282":{"cik_str":1368148,"ticker":"ATHX","title":"ATHERSYS, INC / NEW"},"6283":{"cik_str":1698702,"ticker":"ARGC","title":"ARION GROUP CORP."},"6284":{"cik_str":1782309,"ticker":"EDTK","title":"Skillful Craftsman Education Technology Ltd"},"6285":{"cik_str":1487197,"ticker":"BRFH","title":"BARFRESH FOOD GROUP INC."},"6286":{"cik_str":1865127,"ticker":"LSDI","title":"Lucy Scientific Discovery, Inc."},"6287":{"cik_str":1787803,"ticker":"WNW","title":"Meiwu Technology Co Ltd"},"6288":{"cik_str":1339688,"ticker":"LCGMF","title":"LION COPPER & GOLD CORP."},"6289":{"cik_str":1549084,"ticker":"EKSO","title":"EKSO BIONICS HOLDINGS, INC."},"6290":{"cik_str":1307579,"ticker":"LIQT","title":"LIQTECH INTERNATIONAL INC"},"6291":{"cik_str":1123267,"ticker":"HANNF","title":"Hannan Metals Ltd."},"6292":{"cik_str":1391426,"ticker":"CLNV","title":"Clean Vision Corp"},"6293":{"cik_str":1009922,"ticker":"NSFDF","title":"NXT Energy Solutions Inc."},"6294":{"cik_str":1930419,"ticker":"ALRTF","title":"ALR Technologies SG Ltd."},"6295":{"cik_str":1941500,"ticker":"MWG","title":"Multi Ways Holdings Ltd"},"6296":{"cik_str":1074543,"ticker":"FBPI","title":"FIRST BANCORP OF INDIANA INC"},"6297":{"cik_str":1397183,"ticker":"IVDA","title":"Iveda Solutions, Inc."},"6298":{"cik_str":1824893,"ticker":"SRZN","title":"Surrozen, Inc./DE"},"6299":{"cik_str":1410428,"ticker":"XWEL","title":"XWELL, Inc."},"6300":{"cik_str":1499780,"ticker":"GLBS","title":"GLOBUS MARITIME LTD"},"6301":{"cik_str":1603207,"ticker":"VBLT","title":"Vascular Biogenics Ltd."},"6302":{"cik_str":1089297,"ticker":"NVGT","title":"NOVAGANT CORP"},"6303":{"cik_str":1398453,"ticker":"XIN","title":"Xinyuan Real Estate Co., Ltd."},"6304":{"cik_str":1834645,"ticker":"PKBO","title":"Peak Bio, Inc."},"6305":{"cik_str":704172,"ticker":"PHIL","title":"PHI GROUP INC"},"6306":{"cik_str":1293310,"ticker":"HGEN","title":"HUMANIGEN, INC"},"6307":{"cik_str":1116548,"ticker":"MXROF","title":"MAX RESOURCE CORP."},"6308":{"cik_str":1285170,"ticker":"ITMSF","title":"INTERMAP TECHNOLOGIES CORP"},"6309":{"cik_str":1383088,"ticker":"CYCA","title":"CYTTA CORP."},"6310":{"cik_str":830616,"ticker":"SCIA","title":"SCI Engineered Materials, Inc."},"6311":{"cik_str":1801602,"ticker":"SBIG","title":"SpringBig Holdings, Inc."},"6312":{"cik_str":1172102,"ticker":"FSRL","title":"FIRST RELIANCE BANCSHARES INC"},"6313":{"cik_str":1318482,"ticker":"KDOZF","title":"KIDOZ INC."},"6314":{"cik_str":1807893,"ticker":"SPFX","title":"STANDARD PREMIUM FINANCE HOLDINGS, INC."},"6315":{"cik_str":1809158,"ticker":"KUKE","title":"Kuke Music Holding Ltd"},"6316":{"cik_str":1978811,"ticker":"GOVB","title":"Gouverneur Bancorp, Inc./MD/"},"6317":{"cik_str":1591890,"ticker":"FGF","title":"FG Financial Group, Inc."},"6318":{"cik_str":1789299,"ticker":"UTME","title":"UTime Ltd"},"6319":{"cik_str":1621443,"ticker":"ZYNE","title":"Zynerba Pharmaceuticals, Inc."},"6320":{"cik_str":1236275,"ticker":"SSNT","title":"SilverSun Technologies, Inc."},"6321":{"cik_str":1782107,"ticker":"BWV","title":"Blue Water Biotech, Inc."},"6322":{"cik_str":1681348,"ticker":"VVPR","title":"VivoPower International PLC"},"6323":{"cik_str":1113148,"ticker":"INFI","title":"INFINITY PHARMACEUTICALS, INC."},"6324":{"cik_str":1836875,"ticker":"NVVE","title":"Nuvve Holding Corp."},"6325":{"cik_str":1505497,"ticker":"BRTX","title":"BioRestorative Therapies, Inc."},"6326":{"cik_str":1436229,"ticker":"BTCS","title":"BTCS Inc."},"6327":{"cik_str":1355736,"ticker":"AVCRF","title":"Avricore Health Inc."},"6328":{"cik_str":1347652,"ticker":"CORR","title":"CorEnergy Infrastructure Trust, Inc."},"6329":{"cik_str":1015647,"ticker":"AAU","title":"ALMADEN MINERALS LTD"},"6330":{"cik_str":747540,"ticker":"SPRS","title":"SURGE COMPONENTS INC"},"6331":{"cik_str":1173313,"ticker":"ABVC","title":"ABVC BIOPHARMA, INC."},"6332":{"cik_str":1766140,"ticker":"UNCY","title":"Unicycive Therapeutics, Inc."},"6333":{"cik_str":1408146,"ticker":"SCYYF","title":"SCANDIUM INTERNATIONAL MINING CORP."},"6334":{"cik_str":1531266,"ticker":"ELIO","title":"Elio Motors, Inc."},"6335":{"cik_str":1739174,"ticker":"PHGE","title":"BiomX Inc."},"6336":{"cik_str":1937993,"ticker":"CVKD","title":"Cadrenal Therapeutics, Inc."},"6337":{"cik_str":1733413,"ticker":"TFFP","title":"TFF Pharmaceuticals, Inc."},"6338":{"cik_str":1790169,"ticker":"FLGC","title":"Flora Growth Corp."},"6339":{"cik_str":1169034,"ticker":"BHV","title":"BLACKROCK VIRGINIA MUNICIPAL BOND TRUST"},"6340":{"cik_str":1680139,"ticker":"HLYK","title":"HealthLynked Corp"},"6341":{"cik_str":1610590,"ticker":"CHEK","title":"Check-Cap Ltd"},"6342":{"cik_str":1816319,"ticker":"LYT","title":"Lytus Technologies Holdings PTV. Ltd."},"6343":{"cik_str":1305773,"ticker":"CFMS","title":"Conformis Inc"},"6344":{"cik_str":1355839,"ticker":"PALT","title":"PALTALK, INC."},"6345":{"cik_str":1326205,"ticker":"IGC","title":"IGC Pharma, Inc."},"6346":{"cik_str":1476963,"ticker":"NHWK","title":"NightHawk Biosciences, Inc."},"6347":{"cik_str":1540159,"ticker":"EDSA","title":"Edesa Biotech, Inc."},"6348":{"cik_str":1677897,"ticker":"UPYY","title":"UPAY"},"6349":{"cik_str":1717556,"ticker":"BBIG","title":"Vinco Ventures, Inc."},"6350":{"cik_str":1858994,"ticker":"RSMXF","title":"Regency Silver Corp."},"6351":{"cik_str":1821075,"ticker":"SFR","title":"Appreciate Holdings, Inc."},"6352":{"cik_str":1659617,"ticker":"MBRX","title":"Moleculin Biotech, Inc."},"6353":{"cik_str":1709819,"ticker":"FEDU","title":"Four Seasons Education (Cayman) Inc."},"6354":{"cik_str":1837493,"ticker":"IINN","title":"Inspira Technologies OXY B.H.N. Ltd"},"6355":{"cik_str":1630176,"ticker":"HYEX","title":"HEALTHY EXTRACTS INC."},"6356":{"cik_str":1175596,"ticker":"AXREF","title":"AMARC RESOURCES LTD"},"6357":{"cik_str":1776197,"ticker":"AKU","title":"AKUMIN INC."},"6358":{"cik_str":1445109,"ticker":"NSGP","title":"Clean Coal Technologies Inc."},"6359":{"cik_str":744452,"ticker":"APDN","title":"APPLIED DNA SCIENCES INC"},"6360":{"cik_str":1196298,"ticker":"NEPH","title":"NEPHROS INC"},"6361":{"cik_str":1541157,"ticker":"AKTX","title":"Akari Therapeutics Plc"},"6362":{"cik_str":1757499,"ticker":"SHPH","title":"Shuttle Pharmaceuticals Holdings, Inc."},"6363":{"cik_str":1593184,"ticker":"FDIT","title":"FINDIT, INC."},"6364":{"cik_str":1412126,"ticker":"RMSL","title":"RemSleep Holdings Inc."},"6365":{"cik_str":1847846,"ticker":"EUDA","title":"EUDA Health Holdings Ltd"},"6366":{"cik_str":1622345,"ticker":"POLA","title":"Polar Power, Inc."},"6367":{"cik_str":1864943,"ticker":"FGI","title":"FGI Industries Ltd."},"6368":{"cik_str":1510518,"ticker":"GFOO","title":"Genufood Energy Enzymes Corp."},"6369":{"cik_str":1595527,"ticker":"NYC","title":"American Strategic Investment Co."},"6370":{"cik_str":1430306,"ticker":"TNXP","title":"Tonix Pharmaceuticals Holding Corp."},"6371":{"cik_str":1763925,"ticker":"CJAX","title":"CoJax Oil & Gas Corp"},"6372":{"cik_str":793628,"ticker":"CHNR","title":"CHINA NATURAL RESOURCES INC"},"6373":{"cik_str":744825,"ticker":"AMS","title":"AMERICAN SHARED HOSPITAL SERVICES"},"6374":{"cik_str":1582581,"ticker":"VJET","title":"voxeljet AG"},"6375":{"cik_str":1034760,"ticker":"WYY","title":"WIDEPOINT CORP"},"6376":{"cik_str":1532390,"ticker":"WTER","title":"ALKALINE WATER Co INC"},"6377":{"cik_str":1421819,"ticker":"HJGP","title":"Hanjiao Group, Inc."},"6378":{"cik_str":1768910,"ticker":"GRCYF","title":"Greencity Acquisition Corp"},"6379":{"cik_str":707388,"ticker":"STRR","title":"STAR EQUITY HOLDINGS, INC."},"6380":{"cik_str":1544227,"ticker":"TPST","title":"Tempest Therapeutics, Inc."},"6381":{"cik_str":1712762,"ticker":"BIAF","title":"bioAffinity Technologies, Inc."},"6382":{"cik_str":1629210,"ticker":"PZG","title":"Paramount Gold Nevada Corp."},"6383":{"cik_str":1590695,"ticker":"TLCC","title":"TWINLAB CONSOLIDATED HOLDINGS, INC."},"6384":{"cik_str":1781397,"ticker":"EEIQ","title":"EpicQuest Education Group International Ltd"},"6385":{"cik_str":1495648,"ticker":"PARG","title":"Power Americas Resource Group Ltd."},"6386":{"cik_str":1730869,"ticker":"EBZT","title":"Everything Blockchain, Inc."},"6387":{"cik_str":1633070,"ticker":"AXLA","title":"Axcella Health Inc."},"6388":{"cik_str":1309082,"ticker":"CEI","title":"CAMBER ENERGY, INC."},"6389":{"cik_str":1567892,"ticker":"MNK","title":"Mallinckrodt plc"},"6390":{"cik_str":1718224,"ticker":"BTBD","title":"BT Brands, Inc."},"6391":{"cik_str":1901215,"ticker":"BNRG","title":"Brenmiller Energy Ltd."},"6392":{"cik_str":1892025,"ticker":"OHCS","title":"Optimus Healthcare Services, Inc."},"6393":{"cik_str":743758,"ticker":"AIAD","title":"AiAdvertising, Inc."},"6394":{"cik_str":1755058,"ticker":"ATIF","title":"ATIF Holdings Ltd"},"6395":{"cik_str":1311673,"ticker":"YTFD","title":"Yale Transaction Finders, Inc."},"6396":{"cik_str":1105101,"ticker":"GIGM","title":"GIGAMEDIA Ltd"},"6397":{"cik_str":1750106,"ticker":"AEI","title":"Alset Inc."},"6398":{"cik_str":1384437,"ticker":"CSKL","title":"CATSKILL HUDSON BANCORP INC"},"6399":{"cik_str":1419951,"ticker":"DTST","title":"Data Storage Corp"},"6400":{"cik_str":1368365,"ticker":"MARK","title":"REMARK HOLDINGS, INC."},"6401":{"cik_str":1470683,"ticker":"AEHL","title":"Antelope Enterprise Holdings Ltd"},"6402":{"cik_str":885307,"ticker":"JCTCF","title":"JEWETT CAMERON TRADING CO LTD"},"6403":{"cik_str":1568385,"ticker":"BMTM","title":"Bright Mountain Media, Inc."},"6404":{"cik_str":1857044,"ticker":"INDP","title":"Indaptus Therapeutics, Inc."},"6405":{"cik_str":1362703,"ticker":"THER","title":"THERALINK TECHNOLOGIES, INC."},"6406":{"cik_str":1080319,"ticker":"ELYS","title":"Elys Game Technology, Corp."},"6407":{"cik_str":725394,"ticker":"DFCO","title":"DALRADA FINANCIAL CORP"},"6408":{"cik_str":1442620,"ticker":"RCON","title":"Recon Technology, Ltd"},"6409":{"cik_str":1534248,"ticker":"CMMB","title":"Chemomab Therapeutics Ltd."},"6410":{"cik_str":351789,"ticker":"ELSE","title":"ELECTRO SENSORS INC"},"6411":{"cik_str":1716338,"ticker":"PT","title":"Pintec Technology Holdings Ltd"},"6412":{"cik_str":832489,"ticker":"GOVX","title":"GeoVax Labs, Inc."},"6413":{"cik_str":1425355,"ticker":"MCVT","title":"Mill City Ventures III, Ltd"},"6414":{"cik_str":1008848,"ticker":"ACOR","title":"Acorda Therapeutics, Inc."},"6415":{"cik_str":1158399,"ticker":"BRCNF","title":"Burcon NutraScience Corp"},"6416":{"cik_str":34285,"ticker":"RLBY","title":"RELIABILITY INC"},"6417":{"cik_str":1501958,"ticker":"CLAD","title":"China Liaoning Dingxu Ecological Agriculture Development, Inc."},"6418":{"cik_str":1697818,"ticker":"ICLK","title":"iClick Interactive Asia Group Ltd"},"6419":{"cik_str":1497253,"ticker":"ONVO","title":"ORGANOVO HOLDINGS, INC."},"6420":{"cik_str":1716166,"ticker":"VVOS","title":"Vivos Therapeutics, Inc."},"6421":{"cik_str":1022505,"ticker":"SDCH","title":"SideChannel, Inc."},"6422":{"cik_str":1081745,"ticker":"INLX","title":"INTELLINETICS, INC."},"6423":{"cik_str":90168,"ticker":"SIF","title":"SIFCO INDUSTRIES INC"},"6424":{"cik_str":1621672,"ticker":"SLGG","title":"Super League Gaming, Inc."},"6425":{"cik_str":1879848,"ticker":"PEV","title":"PHOENIX MOTOR INC."},"6426":{"cik_str":1627554,"ticker":"PDRO","title":"Pedro's List, Inc."},"6427":{"cik_str":1443863,"ticker":"BICX","title":"BioCorRx Inc."},"6428":{"cik_str":1023994,"ticker":"SGBX","title":"SAFE & GREEN HOLDINGS CORP."},"6429":{"cik_str":1552743,"ticker":"UGHB","title":"UNIVERSAL GLOBAL HUB INC."},"6430":{"cik_str":1630212,"ticker":"ALBT","title":"Avalon GloboCare Corp."},"6431":{"cik_str":727510,"ticker":"ENZN","title":"ENZON PHARMACEUTICALS, INC."},"6432":{"cik_str":1411685,"ticker":"VTGN","title":"Vistagen Therapeutics, Inc."},"6433":{"cik_str":1871737,"ticker":"ACLHF","title":"ACME Lithium Inc."},"6434":{"cik_str":1698022,"ticker":"FMFG","title":"Farmers & Merchants Bancshares, Inc."},"6435":{"cik_str":1846715,"ticker":"WAVE","title":"Eco Wave Power Global AB (publ)"},"6436":{"cik_str":826253,"ticker":"AUSI","title":"AURA SYSTEMS INC"},"6437":{"cik_str":1546853,"ticker":"SKKY","title":"Skkynet Cloud Systems, Inc."},"6438":{"cik_str":1611842,"ticker":"PYPD","title":"PolyPid Ltd."},"6439":{"cik_str":1136174,"ticker":"OTRK","title":"Ontrak, Inc."},"6440":{"cik_str":1603993,"ticker":"NISN","title":"Nisun International Enterprise Development Group Co., Ltd"},"6441":{"cik_str":797542,"ticker":"VBIX","title":"Viewbix Inc."},"6442":{"cik_str":1812360,"ticker":"FOXO","title":"FOXO TECHNOLOGIES INC."},"6443":{"cik_str":1299865,"ticker":"GXSFF","title":"Goldsource Mines Inc."},"6444":{"cik_str":98338,"ticker":"TSRI","title":"TSR INC"},"6445":{"cik_str":1858685,"ticker":"BFRI","title":"Biofrontera Inc."},"6446":{"cik_str":1546383,"ticker":"JXJT","title":"JX Luxventure Ltd"},"6447":{"cik_str":894158,"ticker":"TOVX","title":"Theriva Biologics, Inc."},"6448":{"cik_str":1756499,"ticker":"MCLD","title":"mCloud Technologies Corp."},"6449":{"cik_str":1841800,"ticker":"NOGN","title":"Nogin, Inc."},"6450":{"cik_str":1314196,"ticker":"OLB","title":"OLB GROUP, INC."},"6451":{"cik_str":1106838,"ticker":"SONN","title":"Sonnet BioTherapeutics Holdings, Inc."},"6452":{"cik_str":1659323,"ticker":"ITRM","title":"Iterum Therapeutics plc"},"6453":{"cik_str":1698113,"ticker":"ID","title":"PARTS iD, Inc."},"6454":{"cik_str":1637890,"ticker":"CYAD","title":"Celyad Oncology SA"},"6455":{"cik_str":719494,"ticker":"INRD","title":"Inrad Optics, Inc."},"6456":{"cik_str":1321517,"ticker":"SRNW","title":"Stratos Renewables Corp"},"6457":{"cik_str":1597846,"ticker":"GRNQ","title":"Greenpro Capital Corp."},"6458":{"cik_str":1133818,"ticker":"BPTH","title":"BIO-PATH HOLDINGS INC"},"6459":{"cik_str":1557376,"ticker":"OCEL","title":"Organicell Regenerative Medicine, Inc."},"6460":{"cik_str":1589061,"ticker":"GYRO","title":"Gyrodyne, LLC"},"6461":{"cik_str":1468929,"ticker":"NXGL","title":"NEXGEL, INC."},"6462":{"cik_str":1706524,"ticker":"UGRO","title":"urban-gro, Inc."},"6463":{"cik_str":1828098,"ticker":"STKH","title":"Steakholder Foods Ltd."},"6464":{"cik_str":1538495,"ticker":"ETST","title":"Earth Science Tech, Inc."},"6465":{"cik_str":33992,"ticker":"KINS","title":"KINGSTONE COMPANIES, INC."},"6466":{"cik_str":933738,"ticker":"ENG","title":"ENGLOBAL CORP"},"6467":{"cik_str":1733257,"ticker":"FNCH","title":"Finch Therapeutics Group, Inc."},"6468":{"cik_str":1370292,"ticker":"VSMR","title":"VERIFY SMART CORP."},"6469":{"cik_str":861838,"ticker":"ACGN","title":"Aceragen, Inc."},"6470":{"cik_str":1583648,"ticker":"PIRS","title":"PIERIS PHARMACEUTICALS, INC."},"6471":{"cik_str":1406434,"ticker":"PCST","title":"PURE CAPITAL SOLUTIONS, INC."},"6472":{"cik_str":22701,"ticker":"PEGY","title":"Pineapple Energy Inc."},"6473":{"cik_str":1889823,"ticker":"KWE","title":"KWESST Micro Systems Inc."},"6474":{"cik_str":1817511,"ticker":"SOPA","title":"SOCIETY PASS INCORPORATED."},"6475":{"cik_str":764630,"ticker":"SPEV","title":"UNITED STATES BASKETBALL LEAGUE INC"},"6476":{"cik_str":1843388,"ticker":"VORBQ","title":"Virgin Orbit Holdings, Inc."},"6477":{"cik_str":1565228,"ticker":"VISL","title":"Vislink Technologies, Inc."},"6478":{"cik_str":1451512,"ticker":"UNRV","title":"Unrivaled Brands, Inc."},"6479":{"cik_str":1014763,"ticker":"AIMD","title":"Ainos, Inc."},"6480":{"cik_str":1605057,"ticker":"VMNT","title":"Vemanti Group, Inc."},"6481":{"cik_str":1467154,"ticker":"NOVN","title":"Novan, Inc."},"6482":{"cik_str":1379006,"ticker":"NNVC","title":"NANOVIRICIDES, INC."},"6483":{"cik_str":880984,"ticker":"ACFN","title":"ACORN ENERGY, INC."},"6484":{"cik_str":1017655,"ticker":"PAYD","title":"PAID INC"},"6485":{"cik_str":1792030,"ticker":"IMCC","title":"IM Cannabis Corp."},"6486":{"cik_str":1463978,"ticker":"BKUCF","title":"BLUE SKY URANIUM CORP."},"6487":{"cik_str":1825024,"ticker":"OPAD","title":"Offerpad Solutions Inc."},"6488":{"cik_str":1026662,"ticker":"CLWT","title":"EURO TECH HOLDINGS CO LTD"},"6489":{"cik_str":924396,"ticker":"OLKR","title":"OpenLocker Holdings, Inc."},"6490":{"cik_str":875355,"ticker":"ISIG","title":"INSIGNIA SYSTEMS INC/MN"},"6491":{"cik_str":1113423,"ticker":"AEZS","title":"Aeterna Zentaris Inc."},"6492":{"cik_str":1171838,"ticker":"SUND","title":"Sundance Strategies, Inc."},"6493":{"cik_str":1566044,"ticker":"VYNE","title":"VYNE Therapeutics Inc."},"6494":{"cik_str":1792554,"ticker":"EQMEF","title":"Equity Metals Corp"},"6495":{"cik_str":1124105,"ticker":"CBIO","title":"CATALYST BIOSCIENCES, INC."},"6496":{"cik_str":1725134,"ticker":"DMS","title":"Digital Media Solutions, Inc."},"6497":{"cik_str":12239,"ticker":"DOMH","title":"Dominari Holdings Inc."},"6498":{"cik_str":1747661,"ticker":"ADD","title":"Color Star Technology Co., Ltd."},"6499":{"cik_str":1276531,"ticker":"SCGY","title":"SCIENTIFIC ENERGY, INC"},"6500":{"cik_str":1865482,"ticker":"REUN","title":"Reunion Neuroscience Inc."},"6501":{"cik_str":1785345,"ticker":"LABP","title":"Landos Biopharma, Inc."},"6502":{"cik_str":1161814,"ticker":"KIQ","title":"KELSO TECHNOLOGIES INC"},"6503":{"cik_str":1536196,"ticker":"CANF","title":"Can-Fite BioPharma Ltd."},"6504":{"cik_str":1121702,"ticker":"YTEN","title":"YIELD10 BIOSCIENCE, INC."},"6505":{"cik_str":886093,"ticker":"NUNZ","title":"NUNZIA PHARMACEUTICAL Co"},"6506":{"cik_str":1448705,"ticker":"BASA","title":"BASANITE, INC."},"6507":{"cik_str":1533743,"ticker":"PCSA","title":"Processa Pharmaceuticals, Inc."},"6508":{"cik_str":1104038,"ticker":"VRME","title":"VerifyMe, Inc."},"6509":{"cik_str":1824293,"ticker":"GRI","title":"GRI BIO, Inc."},"6510":{"cik_str":1636051,"ticker":"FUST","title":"FUSE GROUP HOLDING INC."},"6511":{"cik_str":1369290,"ticker":"MYO","title":"MYOMO, INC."},"6512":{"cik_str":1376231,"ticker":"VPRB","title":"VPR Brands, LP."},"6513":{"cik_str":919567,"ticker":"RCG","title":"RENN Fund, Inc."},"6514":{"cik_str":1421289,"ticker":"LVVV","title":"LiveWire Ergogenics, Inc."},"6515":{"cik_str":1785592,"ticker":"LFLY","title":"Leafly Holdings, Inc. /DE"},"6516":{"cik_str":1584751,"ticker":"TLIS","title":"Talis Biomedical Corp"},"6517":{"cik_str":1708331,"ticker":"VRPX","title":"Virpax Pharmaceuticals, Inc."},"6518":{"cik_str":1770236,"ticker":"MITQ","title":"MOVING iMAGE TECHNOLOGIES INC."},"6519":{"cik_str":721693,"ticker":"CREG","title":"Smart Powerr Corp."},"6520":{"cik_str":723533,"ticker":"LWLW","title":"Longwen Group Corp."},"6521":{"cik_str":1962481,"ticker":"BOF","title":"BranchOut Food Inc."},"6522":{"cik_str":1589361,"ticker":"WTRV","title":"White River Energy Corp."},"6523":{"cik_str":1723047,"ticker":"VAUCF","title":"Viva Gold Corp."},"6524":{"cik_str":1827401,"ticker":"DRUG","title":"BRIGHT MINDS BIOSCIENCES INC."},"6525":{"cik_str":1277998,"ticker":"MHPC","title":"MANUFACTURED HOUSING PROPERTIES INC."},"6526":{"cik_str":1402328,"ticker":"SBFM","title":"Sunshine Biopharma, Inc"},"6527":{"cik_str":1761510,"ticker":"TLLTF","title":"TILT Holdings Inc."},"6528":{"cik_str":1729214,"ticker":"XRTX","title":"XORTX Therapeutics Inc."},"6529":{"cik_str":314227,"ticker":"TOMZ","title":"TOMI Environmental Solutions, Inc."},"6530":{"cik_str":1776067,"ticker":"OCG","title":"Oriental Culture Holding LTD"},"6531":{"cik_str":1438943,"ticker":"MLCT","title":"MALACHITE INNOVATIONS, INC."},"6532":{"cik_str":1485029,"ticker":"NAHD","title":"New Asia Holdings, Inc."},"6533":{"cik_str":32621,"ticker":"MSN","title":"EMERSON RADIO CORP"},"6534":{"cik_str":1378590,"ticker":"BLIN","title":"Bridgeline Digital, Inc."},"6535":{"cik_str":1703647,"ticker":"FREQ","title":"Frequency Therapeutics, Inc."},"6536":{"cik_str":822370,"ticker":"EMMA","title":"Emmaus Life Sciences, Inc."},"6537":{"cik_str":1558569,"ticker":"ISPC","title":"iSpecimen Inc."},"6538":{"cik_str":1823882,"ticker":"MIMO","title":"Airspan Networks Holdings Inc."},"6539":{"cik_str":749647,"ticker":"IMNN","title":"Imunon, Inc."},"6540":{"cik_str":1811216,"ticker":"BAOS","title":"Baosheng Media Group Holdings Ltd"},"6541":{"cik_str":1781193,"ticker":"QH","title":"QUHUO Ltd"},"6542":{"cik_str":1463208,"ticker":"TLSS","title":"Transportation & Logistics Systems, Inc."},"6543":{"cik_str":1296484,"ticker":"TOPS","title":"TOP SHIPS INC."},"6544":{"cik_str":1971544,"ticker":"AIRO","title":"AIRO Group, Inc."},"6545":{"cik_str":1310527,"ticker":"QSAM","title":"QSAM Biosciences, Inc."},"6546":{"cik_str":1813658,"ticker":"TMPO","title":"Tempo Automation Holdings, Inc."},"6547":{"cik_str":1841330,"ticker":"KTTA","title":"Pasithea Therapeutics Corp."},"6548":{"cik_str":1684425,"ticker":"PETZ","title":"TDH Holdings, Inc."},"6549":{"cik_str":748592,"ticker":"ERNA","title":"Eterna Therapeutics Inc."},"6550":{"cik_str":1705338,"ticker":"LOV","title":"Spark Networks SE"},"6551":{"cik_str":1781983,"ticker":"APRE","title":"Aprea Therapeutics, Inc."},"6552":{"cik_str":1743340,"ticker":"TC","title":"TuanChe Ltd"},"6553":{"cik_str":1413745,"ticker":"ANTE","title":"AIRNET TECHNOLOGY INC."},"6554":{"cik_str":1190370,"ticker":"IVDN","title":"INNOVATIVE DESIGNS INC"},"6555":{"cik_str":109657,"ticker":"GTBP","title":"GT Biopharma, Inc."},"6556":{"cik_str":1347242,"ticker":"LIPO","title":"LIPELLA PHARMACEUTICALS INC"},"6557":{"cik_str":1870144,"ticker":"CDIO","title":"Cardio Diagnostics Holdings, Inc."},"6558":{"cik_str":1415332,"ticker":"LTUM","title":"Lithium Corp"},"6559":{"cik_str":1552670,"ticker":"TAOP","title":"Taoping Inc."},"6560":{"cik_str":1427925,"ticker":"ACRX","title":"ACELRX PHARMACEUTICALS INC"},"6561":{"cik_str":1803407,"ticker":"OST","title":"Ostin Technology Group Co., Ltd."},"6562":{"cik_str":1419793,"ticker":"OCLN","title":"ORIGINCLEAR, INC."},"6563":{"cik_str":1734005,"ticker":"APM","title":"Aptorum Group Ltd"},"6564":{"cik_str":1777765,"ticker":"VQS","title":"VIQ Solutions Inc."},"6565":{"cik_str":1011509,"ticker":"AUMN","title":"Golden Minerals Co"},"6566":{"cik_str":7623,"ticker":"ARTW","title":"ARTS WAY MANUFACTURING CO INC"},"6567":{"cik_str":1807389,"ticker":"CPOP","title":"Pop Culture Group Co., Ltd"},"6568":{"cik_str":1576197,"ticker":"SENR","title":"Strategic Environmental & Energy Resources, Inc."},"6569":{"cik_str":1638287,"ticker":"NRBO","title":"NeuroBo Pharmaceuticals, Inc."},"6570":{"cik_str":1331612,"ticker":"IMTH","title":"INNOVATIVE MEDTECH, INC."},"6571":{"cik_str":1907685,"ticker":"CMRA","title":"Comera Life Sciences Holdings, Inc."},"6572":{"cik_str":1491487,"ticker":"TKAT","title":"Takung Art Co., Ltd"},"6573":{"cik_str":1654595,"ticker":"MDRR","title":"Medalist Diversified REIT, Inc."},"6574":{"cik_str":1357459,"ticker":"PALI","title":"PALISADE BIO, INC."},"6575":{"cik_str":1648960,"ticker":"DATS","title":"DatChat, Inc."},"6576":{"cik_str":1066130,"ticker":"FEMFF","title":"First Energy Metals Ltd"},"6577":{"cik_str":1759136,"ticker":"BHAT","title":"Blue Hat Interactive Entertainment Technology"},"6578":{"cik_str":924515,"ticker":"GTHP","title":"GUIDED THERAPEUTICS INC"},"6579":{"cik_str":1892480,"ticker":"HPCO","title":"Hempacco Co., Inc."},"6580":{"cik_str":1634293,"ticker":"EXDW","title":"Exceed World, Inc."},"6581":{"cik_str":1817740,"ticker":"AUST","title":"Austin Gold Corp."},"6582":{"cik_str":1862150,"ticker":"CING","title":"Cingulate Inc."},"6583":{"cik_str":1811623,"ticker":"PXMD","title":"PaxMedica, Inc."},"6584":{"cik_str":1138476,"ticker":"PFHO","title":"PACIFIC HEALTH CARE ORGANIZATION INC"},"6585":{"cik_str":1333822,"ticker":"LEDS","title":"SemiLEDs Corp"},"6586":{"cik_str":1450416,"ticker":"LXXGQ","title":"LexaGene Holdings Inc."},"6587":{"cik_str":1737193,"ticker":"LDSN","title":"Luduson G Inc."},"6588":{"cik_str":1119190,"ticker":"HMBL","title":"HUMBL, INC."},"6589":{"cik_str":1821175,"ticker":"MSGM","title":"Motorsport Games Inc."},"6590":{"cik_str":1016178,"ticker":"CARV","title":"CARVER BANCORP INC"},"6591":{"cik_str":1477009,"ticker":"CANN","title":"TREES Corp (Colorado)"},"6592":{"cik_str":1009891,"ticker":"AIRI","title":"AIR INDUSTRIES GROUP"},"6593":{"cik_str":724445,"ticker":"BZYR","title":"BURZYNSKI RESEARCH INSTITUTE INC"},"6594":{"cik_str":1808377,"ticker":"LUCY","title":"Innovative Eyewear Inc"},"6595":{"cik_str":1389002,"ticker":"MRIN","title":"MARIN SOFTWARE INC"},"6596":{"cik_str":1660046,"ticker":"IMRN","title":"Immuron Ltd"},"6597":{"cik_str":1342958,"ticker":"DGLY","title":"DIGITAL ALLY, INC."},"6598":{"cik_str":1681682,"ticker":"NDRA","title":"ENDRA Life Sciences Inc."},"6599":{"cik_str":891417,"ticker":"NMGX","title":"Nano Magic Inc."},"6600":{"cik_str":844887,"ticker":"DHCC","title":"DIAMONDHEAD CASINO CORP"},"6601":{"cik_str":1810031,"ticker":"FRLN","title":"Freeline Therapeutics Holdings plc"},"6602":{"cik_str":1558583,"ticker":"FUV","title":"Arcimoto Inc"},"6603":{"cik_str":819913,"ticker":"HALL","title":"HALLMARK FINANCIAL SERVICES INC"},"6604":{"cik_str":1096950,"ticker":"ACTX","title":"ADVANCED CONTAINER TECHNOLOGIES, INC."},"6605":{"cik_str":1131343,"ticker":"ATHE","title":"ALTERITY THERAPEUTICS LTD"},"6606":{"cik_str":1645469,"ticker":"MNPR","title":"Monopar Therapeutics"},"6607":{"cik_str":1780785,"ticker":"CNEY","title":"CN ENERGY GROUP. INC."},"6608":{"cik_str":1760903,"ticker":"JUPW","title":"Jupiter Wellness, Inc."},"6609":{"cik_str":1538217,"ticker":"SRAX","title":"SRAX, Inc."},"6610":{"cik_str":1651721,"ticker":"GIPR","title":"GENERATION INCOME PROPERTIES, INC."},"6611":{"cik_str":1801777,"ticker":"AMTI","title":"Applied Molecular Transport Inc."},"6612":{"cik_str":1814140,"ticker":"APGN","title":"Apexigen, Inc."},"6613":{"cik_str":1136294,"ticker":"WLMS","title":"Williams Industrial Services Group Inc."},"6614":{"cik_str":944075,"ticker":"SCKT","title":"SOCKET MOBILE, INC."},"6615":{"cik_str":890394,"ticker":"PRSO","title":"Peraso Inc."},"6616":{"cik_str":912544,"ticker":"NSTM","title":"NovelStem International Corp."},"6617":{"cik_str":1488638,"ticker":"MBGH","title":"MBG Holdings, Inc."},"6618":{"cik_str":1640384,"ticker":"LMFA","title":"LM FUNDING AMERICA, INC."},"6619":{"cik_str":1409269,"ticker":"VERO","title":"Venus Concept Inc."},"6620":{"cik_str":1080657,"ticker":"SQFT","title":"Presidio Property Trust, Inc."},"6621":{"cik_str":1803977,"ticker":"VYBE","title":"Limitless X Holdings Inc."},"6622":{"cik_str":1307624,"ticker":"HLTT","title":"Healthtech Solutions, Inc./UT"},"6623":{"cik_str":1586495,"ticker":"LTES","title":"Leet Technology Inc."},"6624":{"cik_str":1743905,"ticker":"RVSN","title":"Rail Vision Ltd."},"6625":{"cik_str":1029744,"ticker":"SOFO","title":"SONIC FOUNDRY INC"},"6626":{"cik_str":1707910,"ticker":"REBN","title":"Reborn Coffee, Inc."},"6627":{"cik_str":1671584,"ticker":"APVO","title":"Aptevo Therapeutics Inc."},"6628":{"cik_str":1585608,"ticker":"JAGX","title":"Jaguar Health, Inc."},"6629":{"cik_str":1698530,"ticker":"XCUR","title":"EXICURE, INC."},"6630":{"cik_str":1555746,"ticker":"MNRLF","title":"MINERAL MOUNTAIN RESOURCES LTD."},"6631":{"cik_str":1043961,"ticker":"PRPO","title":"Precipio, Inc."},"6632":{"cik_str":1886362,"ticker":"MGAM","title":"Mobile Global Esports, Inc."},"6633":{"cik_str":66496,"ticker":"MMTRS","title":"MILLS MUSIC TRUST"},"6634":{"cik_str":1420720,"ticker":"IBIO","title":"iBio, Inc."},"6635":{"cik_str":1642375,"ticker":"GHSI","title":"Guardion Health Sciences, Inc."},"6636":{"cik_str":31667,"ticker":"EDUC","title":"EDUCATIONAL DEVELOPMENT CORP"},"6637":{"cik_str":1781405,"ticker":"ODII","title":"Odyssey Semiconductor Technologies, Inc."},"6638":{"cik_str":1898604,"ticker":"YOSH","title":"Yoshiharu Global Co."},"6639":{"cik_str":1275477,"ticker":"BMNM","title":"BIMINI CAPITAL MANAGEMENT, INC."},"6640":{"cik_str":830656,"ticker":"PBIO","title":"PRESSURE BIOSCIENCES INC"},"6641":{"cik_str":1574235,"ticker":"PULM","title":"Pulmatrix, Inc."},"6642":{"cik_str":1701261,"ticker":"FAMI","title":"Farmmi, Inc."},"6643":{"cik_str":1108967,"ticker":"OIG","title":"Orbital Infrastructure Group, Inc."},"6644":{"cik_str":1755237,"ticker":"CYCN","title":"Cyclerion Therapeutics, Inc."},"6645":{"cik_str":1026785,"ticker":"HIHO","title":"HIGHWAY HOLDINGS LTD"},"6646":{"cik_str":1584831,"ticker":"OXBR","title":"OXBRIDGE RE HOLDINGS Ltd"},"6647":{"cik_str":1705843,"ticker":"CBUS","title":"Cibus, Inc."},"6648":{"cik_str":1417663,"ticker":"SNWV","title":"SANUWAVE Health, Inc."},"6649":{"cik_str":1891791,"ticker":"TGGI","title":"TRANS GLOBAL GROUP, INC."},"6650":{"cik_str":908259,"ticker":"OTLC","title":"Oncotelic Therapeutics, Inc."},"6651":{"cik_str":55234,"ticker":"KENS","title":"KENILWORTH SYSTEMS CORP"},"6652":{"cik_str":1673481,"ticker":"LTRY","title":"Lottery.com Inc."},"6653":{"cik_str":861972,"ticker":"SRLZF","title":"Salazar Resources Ltd"},"6654":{"cik_str":1016504,"ticker":"INBP","title":"INTEGRATED BIOPHARMA INC"},"6655":{"cik_str":1355250,"ticker":"IPIX","title":"Innovation Pharmaceuticals Inc."},"6656":{"cik_str":910267,"ticker":"TTNP","title":"TITAN PHARMACEUTICALS INC"},"6657":{"cik_str":1711786,"ticker":"HOTH","title":"Hoth Therapeutics, Inc."},"6658":{"cik_str":1839285,"ticker":"HCTI","title":"Healthcare Triangle, Inc."},"6659":{"cik_str":62362,"ticker":"MARPS","title":"MARINE PETROLEUM TRUST"},"6660":{"cik_str":1784970,"ticker":"WETG","title":"WeTrade Group Inc."},"6661":{"cik_str":1716770,"ticker":"WAFU","title":"Wah Fu Education Group Ltd"},"6662":{"cik_str":1304409,"ticker":"AHNR","title":"ATHENA GOLD CORP"},"6663":{"cik_str":1309057,"ticker":"CHHE","title":"China Health Industries Holdings, Inc."},"6664":{"cik_str":1279620,"ticker":"ZDPY","title":"Zoned Properties, Inc."},"6665":{"cik_str":1429560,"ticker":"TRVN","title":"TREVENA INC"},"6666":{"cik_str":1518353,"ticker":"EVGDF","title":"ELEVATION GOLD MINING Corp"},"6667":{"cik_str":1385818,"ticker":"AYTU","title":"AYTU BIOPHARMA, INC"},"6668":{"cik_str":1819615,"ticker":"CLVR","title":"Clever Leaves Holdings Inc."},"6669":{"cik_str":1393772,"ticker":"BUDZ","title":"WEED, INC."},"6670":{"cik_str":1551986,"ticker":"NMTR","title":"9 METERS BIOPHARMA, INC."},"6671":{"cik_str":1574232,"ticker":"ADXN","title":"Addex Therapeutics Ltd."},"6672":{"cik_str":1382574,"ticker":"MEDS","title":"TRxADE HEALTH, INC"},"6673":{"cik_str":1481241,"ticker":"PSHG","title":"Performance Shipping Inc."},"6674":{"cik_str":1231339,"ticker":"ASRE","title":"Astra Energy, Inc."},"6675":{"cik_str":38264,"ticker":"FORD","title":"Forward Industries, Inc."},"6676":{"cik_str":1481504,"ticker":"XERI","title":"XERIANT, INC."},"6677":{"cik_str":1163560,"ticker":"GTMAY","title":"GRUPO TMM SAB"},"6678":{"cik_str":1588084,"ticker":"TANH","title":"TANTECH HOLDINGS LTD"},"6679":{"cik_str":874292,"ticker":"AEY","title":"ADDVANTAGE TECHNOLOGIES GROUP INC"},"6680":{"cik_str":1376321,"ticker":"CNET","title":"ZW Data Action Technologies Inc."},"6681":{"cik_str":1849296,"ticker":"OKYO","title":"OKYO Pharma Ltd"},"6682":{"cik_str":1807166,"ticker":"AMST","title":"Amesite Inc."},"6683":{"cik_str":1715611,"ticker":"BMMJ","title":"BODY & MIND INC."},"6684":{"cik_str":1061069,"ticker":"AWX","title":"AVALON HOLDINGS CORP"},"6685":{"cik_str":1567900,"ticker":"BLBX","title":"BLACKBOXSTOCKS INC."},"6686":{"cik_str":1718939,"ticker":"IDAI","title":"T Stamp Inc"},"6687":{"cik_str":1288750,"ticker":"TLRS","title":"Timberline Resources Corp"},"6688":{"cik_str":1856084,"ticker":"CJJD","title":"China Jo-Jo Drugstores Holdings, Inc."},"6689":{"cik_str":1335105,"ticker":"LIXT","title":"LIXTE BIOTECHNOLOGY HOLDINGS, INC."},"6690":{"cik_str":1729427,"ticker":"CNSP","title":"CNS Pharmaceuticals, Inc."},"6691":{"cik_str":1879726,"ticker":"SIDU","title":"Sidus Space Inc."},"6692":{"cik_str":1880343,"ticker":"VINE","title":"Fresh Vine Wine, Inc."},"6693":{"cik_str":1424657,"ticker":"CUEN","title":"Cuentas Inc."},"6694":{"cik_str":1677522,"ticker":"BKTPF","title":"CRUZ BATTERY METALS CORP."},"6695":{"cik_str":1779303,"ticker":"DCSX","title":"Direct Communication Solutions, Inc."},"6696":{"cik_str":1400271,"ticker":"CUBT","title":"Curative Biotechnology Inc"},"6697":{"cik_str":1468492,"ticker":"HSCS","title":"Heart Test Laboratories, Inc."},"6698":{"cik_str":1644963,"ticker":"ATXI","title":"AVENUE THERAPEUTICS, INC."},"6699":{"cik_str":1437517,"ticker":"CLOQ","title":"CYBERLOQ TECHNOLOGIES, INC."},"6700":{"cik_str":1725332,"ticker":"ALAR","title":"Alarum Technologies Ltd."},"6701":{"cik_str":1737995,"ticker":"STSS","title":"Sharps Technology Inc."},"6702":{"cik_str":1006028,"ticker":"PURE","title":"PURE BIOSCIENCE, INC."},"6703":{"cik_str":1422892,"ticker":"SGLY","title":"Singularity Future Technology Ltd."},"6704":{"cik_str":1589149,"ticker":"GWAV","title":"Greenwave Technology Solutions, Inc."},"6705":{"cik_str":1545224,"ticker":"SILEF","title":"Silver Elephant Mining Corp."},"6706":{"cik_str":1022899,"ticker":"PTIX","title":"Protagenic Therapeutics, Inc.new"},"6707":{"cik_str":1814102,"ticker":"BBOP","title":"BeBop Channel Corp"},"6708":{"cik_str":1386044,"ticker":"FHLD","title":"Freedom Holdings, Inc."},"6709":{"cik_str":1414767,"ticker":"NCPL","title":"Netcapital Inc."},"6710":{"cik_str":1320760,"ticker":"TSSI","title":"TSS, Inc."},"6711":{"cik_str":1025561,"ticker":"SBET","title":"SharpLink Gaming Ltd."},"6712":{"cik_str":1035354,"ticker":"ELOX","title":"Eloxx Pharmaceuticals, Inc."},"6713":{"cik_str":1319150,"ticker":"PBMLF","title":"Pacific Booker Minerals Inc."},"6714":{"cik_str":944480,"ticker":"GVP","title":"GSE SYSTEMS INC"},"6715":{"cik_str":1522602,"ticker":"CWBR","title":"CohBar, Inc."},"6716":{"cik_str":1420565,"ticker":"ALRN","title":"AILERON THERAPEUTICS INC"},"6717":{"cik_str":1641398,"ticker":"GDC","title":"GD Culture Group Ltd"},"6718":{"cik_str":1718817,"ticker":"MSNVF","title":"MISSION READY SOLUTIONS INC"},"6719":{"cik_str":1895618,"ticker":"NFTG","title":"NFT Gaming Co Inc."},"6720":{"cik_str":1371128,"ticker":"NEWH","title":"NewHydrogen, Inc."},"6721":{"cik_str":1817004,"ticker":"EZFL","title":"EzFill Holdings Inc"},"6722":{"cik_str":1579026,"ticker":"TOWTF","title":"TOWER ONE WIRELESS CORP."},"6723":{"cik_str":926423,"ticker":"MIND","title":"MIND TECHNOLOGY, INC"},"6724":{"cik_str":1480170,"ticker":"BEOB","title":"BEO BANCORP"},"6725":{"cik_str":1811109,"ticker":"AUVI","title":"Applied UV, Inc."},"6726":{"cik_str":1554818,"ticker":"AUUD","title":"AUDDIA INC."},"6727":{"cik_str":876235,"ticker":"EGYF","title":"ENERGY FINDERS INC"},"6728":{"cik_str":1809616,"ticker":"UPC","title":"Universe Pharmaceuticals INC"},"6729":{"cik_str":1166708,"ticker":"BWMG","title":"Brownie's Marine Group, Inc"},"6730":{"cik_str":1467761,"ticker":"MINM","title":"MINIM, INC."},"6731":{"cik_str":1606457,"ticker":"ATTO","title":"Atento S.A."},"6732":{"cik_str":1069680,"ticker":"HHSE","title":"Hannover House, Inc."},"6733":{"cik_str":1426506,"ticker":"SMGI","title":"SMG Industries Inc."},"6734":{"cik_str":1011662,"ticker":"GDSI","title":"GLOBAL DIGITAL SOLUTIONS INC"},"6735":{"cik_str":1684688,"ticker":"JUPGF","title":"Jupiter Gold Corp"},"6736":{"cik_str":1614067,"ticker":"ARDS","title":"Aridis Pharmaceuticals, Inc."},"6737":{"cik_str":1784168,"ticker":"ELMSQ","title":"Electric Last Mile Solutions, Inc."},"6738":{"cik_str":849401,"ticker":"ADMT","title":"ADM TRONICS UNLIMITED, INC."},"6739":{"cik_str":1007019,"ticker":"JVA","title":"COFFEE HOLDING CO INC"},"6740":{"cik_str":1729750,"ticker":"KBNT","title":"Kubient, Inc."},"6741":{"cik_str":1099369,"ticker":"DSNY","title":"DESTINY MEDIA TECHNOLOGIES INC"},"6742":{"cik_str":1878835,"ticker":"VGFCQ","title":"Very Good Food Co Inc."},"6743":{"cik_str":1415744,"ticker":"NMEX","title":"NORTHERN MINERALS & EXPLORATION LTD."},"6744":{"cik_str":1442999,"ticker":"ABTI","title":"ALTEROLA BIOTECH INC."},"6745":{"cik_str":1650696,"ticker":"LSF","title":"Laird Superfood, Inc."},"6746":{"cik_str":882291,"ticker":"AEMD","title":"AETHLON MEDICAL INC"},"6747":{"cik_str":1394319,"ticker":"TCON","title":"Tracon Pharmaceuticals, Inc."},"6748":{"cik_str":1520118,"ticker":"INTV","title":"INTEGRATED VENTURES, INC."},"6749":{"cik_str":773717,"ticker":"ACRG","title":"American Clean Resources Group, Inc."},"6750":{"cik_str":1352952,"ticker":"CNFN","title":"CFN Enterprises Inc."},"6751":{"cik_str":1872964,"ticker":"MTEK","title":"Maris Tech Ltd."},"6752":{"cik_str":1563298,"ticker":"EAWD","title":"Energy & Water Development Corp"},"6753":{"cik_str":1083922,"ticker":"ARAO","title":"AuraSource, Inc."},"6754":{"cik_str":1634447,"ticker":"ISUN","title":"ISUN, INC."},"6755":{"cik_str":1735556,"ticker":"BTOG","title":"BIT ORIGIN Ltd"},"6756":{"cik_str":1021435,"ticker":"HPTO","title":"hopTo Inc."},"6757":{"cik_str":811522,"ticker":"QZMRF","title":"QUARTZ MOUNTAIN RESOURCES LTD"},"6758":{"cik_str":1608092,"ticker":"KITL","title":"Kisses From Italy Inc."},"6759":{"cik_str":1687542,"ticker":"FRGT","title":"Freight Technologies, Inc."},"6760":{"cik_str":1814067,"ticker":"LXEH","title":"Lixiang Education Holding Co. Ltd."},"6761":{"cik_str":1128189,"ticker":"PKTX","title":"ProtoKinetix, Inc."},"6762":{"cik_str":1448324,"ticker":"CFIC","title":"Cornerstone Financial Corp"},"6763":{"cik_str":1761696,"ticker":"CRKN","title":"Crown Electrokinetics Corp."},"6764":{"cik_str":1506928,"ticker":"AVGR","title":"Avinger Inc"},"6765":{"cik_str":1362516,"ticker":"CLRI","title":"Cleartronic, Inc."},"6766":{"cik_str":1810467,"ticker":"SNTG","title":"Sentage Holdings Inc."},"6767":{"cik_str":1618500,"ticker":"MDGS","title":"Medigus Ltd."},"6768":{"cik_str":1812727,"ticker":"RELI","title":"Reliance Global Group, Inc."},"6769":{"cik_str":1702318,"ticker":"AHG","title":"Akso Health Group"},"6770":{"cik_str":1603793,"ticker":"NRIS","title":"Norris Industries, Inc."},"6771":{"cik_str":1894693,"ticker":"SVRE","title":"SaverOne 2014 Ltd."},"6772":{"cik_str":1643301,"ticker":"CURR","title":"Avenir Wellness Solutions, Inc."},"6773":{"cik_str":1905511,"ticker":"JCSE","title":"JE Cleantech Holdings Ltd"},"6774":{"cik_str":1289850,"ticker":"NURO","title":"NeuroMetrix, Inc."},"6775":{"cik_str":1374567,"ticker":"LUVU","title":"Luvu Brands, Inc."},"6776":{"cik_str":1561880,"ticker":"LEAI","title":"Legacy Education Alliance, Inc."},"6777":{"cik_str":823546,"ticker":"RKFL","title":"ROCKETFUEL BLOCKCHAIN, INC."},"6778":{"cik_str":1415758,"ticker":"NVSGF","title":"Nevada Sunrise Gold Corp"},"6779":{"cik_str":914139,"ticker":"PRKR","title":"PARKERVISION INC"},"6780":{"cik_str":1492617,"ticker":"FWFW","title":"FLYWHEEL ADVANCED TECHNOLOGY, INC."},"6781":{"cik_str":1515139,"ticker":"MWRK","title":"METAWORKS PLATFORMS, INC."},"6782":{"cik_str":1529113,"ticker":"INPX","title":"INPIXON"},"6783":{"cik_str":1898643,"ticker":"MOB","title":"Mobilicom Ltd"},"6784":{"cik_str":1270436,"ticker":"COHN","title":"Cohen & Co Inc."},"6785":{"cik_str":1544400,"ticker":"NFTN","title":"NFiniTi inc."},"6786":{"cik_str":1514946,"ticker":"AXIM","title":"AXIM BIOTECHNOLOGIES, INC."},"6787":{"cik_str":1794905,"ticker":"CYXTQ","title":"Cyxtera Technologies, Inc."},"6788":{"cik_str":1510178,"ticker":"TGLTY","title":"GCDI S.A./ADR"},"6789":{"cik_str":1786182,"ticker":"FHSEY","title":"First High-School Education Group Co., Ltd."},"6790":{"cik_str":896493,"ticker":"AULT","title":"Ault Alliance, Inc."},"6791":{"cik_str":1885827,"ticker":"VRAX","title":"Virax Biolabs Group Ltd"},"6792":{"cik_str":1088638,"ticker":"MMND","title":"MASTERMIND, INC."},"6793":{"cik_str":1785494,"ticker":"WBQNL","title":"Woodbridge Liquidation Trust"},"6794":{"cik_str":1538210,"ticker":"NEXI","title":"NexImmune, Inc."},"6795":{"cik_str":1527613,"ticker":"NUZE","title":"NuZee, Inc."},"6796":{"cik_str":1130166,"ticker":"CYCC","title":"Cyclacel Pharmaceuticals, Inc."},"6797":{"cik_str":1668438,"ticker":"IMTE","title":"Integrated Media Technology Ltd"},"6798":{"cik_str":1403870,"ticker":"GARWF","title":"Golden Arrow Resources Corp"},"6799":{"cik_str":1875236,"ticker":"CDMNF","title":"Canadian Manganese Co Inc."},"6800":{"cik_str":1174891,"ticker":"BUUZ","title":"CalEthos, Inc."},"6801":{"cik_str":1372183,"ticker":"NXTP","title":"NextPlay Technologies Inc."},"6802":{"cik_str":1805594,"ticker":"JWEL","title":"Jowell Global Ltd."},"6803":{"cik_str":836564,"ticker":"CPMV","title":"Mosaic ImmunoEngineering Inc."},"6804":{"cik_str":1127475,"ticker":"DBMM","title":"Digital Brand Media & Marketing Group, Inc."},"6805":{"cik_str":1282847,"ticker":"TDCB","title":"THIRD CENTURY BANCORP"},"6806":{"cik_str":1536394,"ticker":"USLG","title":"U.S. Lighting Group, Inc."},"6807":{"cik_str":1506983,"ticker":"GCTK","title":"GlucoTrack, Inc."},"6808":{"cik_str":897078,"ticker":"KOAN","title":"Resonate Blends, Inc."},"6809":{"cik_str":1110607,"ticker":"KLNG","title":"Koil Energy Solutions, Inc."},"6810":{"cik_str":1140215,"ticker":"REED","title":"REED'S, INC."},"6811":{"cik_str":768710,"ticker":"RELV","title":"RELIV INTERNATIONAL INC"},"6812":{"cik_str":1061612,"ticker":"JPOTF","title":"Jackpot Digital Inc."},"6813":{"cik_str":1741489,"ticker":"ELVG","title":"Elvictor Group, Inc."},"6814":{"cik_str":1432290,"ticker":"AWON","title":"A1 Group, Inc."},"6815":{"cik_str":1600132,"ticker":"BLPH","title":"Bellerophon Therapeutics, Inc."},"6816":{"cik_str":1019034,"ticker":"BKYI","title":"BIO KEY INTERNATIONAL INC"},"6817":{"cik_str":1627041,"ticker":"VCOR","title":"VISIBER57 CORP."},"6818":{"cik_str":1831868,"ticker":"ICU","title":"SeaStar Medical Holding Corp"},"6819":{"cik_str":1498382,"ticker":"KTRA","title":"Kintara Therapeutics, Inc."},"6820":{"cik_str":1122993,"ticker":"BMXI","title":"BROOKMOUNT EXPLORATIONS INC"},"6821":{"cik_str":890821,"ticker":"ENVB","title":"Enveric Biosciences, Inc."},"6822":{"cik_str":1376793,"ticker":"CVAT","title":"Cavitation Technologies, Inc."},"6823":{"cik_str":1816172,"ticker":"JZXN","title":"Jiuzi Holdings, Inc."},"6824":{"cik_str":1504167,"ticker":"TMBR","title":"Timber Pharmaceuticals, Inc."},"6825":{"cik_str":1300734,"ticker":"SISI","title":"SHINECO, INC."},"6826":{"cik_str":1527352,"ticker":"NXL","title":"Nexalin Technology, Inc."},"6827":{"cik_str":1092570,"ticker":"FULO","title":"FULLNET COMMUNICATIONS INC"},"6828":{"cik_str":1682149,"ticker":"WISA","title":"WISA TECHNOLOGIES, INC."},"6829":{"cik_str":1768946,"ticker":"BPTS","title":"Biophytis SA"},"6830":{"cik_str":1572565,"ticker":"INQD","title":"Indoor Harvest Corp"},"6831":{"cik_str":1068897,"ticker":"FXBY","title":"FOXBY CORP."},"6832":{"cik_str":1205059,"ticker":"REPCF","title":"REPLICEL LIFE SCIENCES INC."},"6833":{"cik_str":1619096,"ticker":"SNTW","title":"Summit Networks Inc."},"6834":{"cik_str":1063537,"ticker":"RIBT","title":"RiceBran Technologies"},"6835":{"cik_str":1816815,"ticker":"BON","title":"Bon Natural Life Ltd"},"6836":{"cik_str":99106,"ticker":"TNLX","title":"TRANS LUX Corp"},"6837":{"cik_str":64463,"ticker":"SLNH","title":"Soluna Holdings, Inc"},"6838":{"cik_str":1471727,"ticker":"BTTR","title":"Better Choice Co Inc."},"6839":{"cik_str":1808898,"ticker":"BNTC","title":"Benitec Biopharma Inc."},"6840":{"cik_str":1875609,"ticker":"SWVL","title":"Swvl Holdings Corp"},"6841":{"cik_str":927761,"ticker":"MYMX","title":"MYMETICS CORP"},"6842":{"cik_str":1560293,"ticker":"TNON","title":"Tenon Medical, Inc."},"6843":{"cik_str":1174940,"ticker":"OGEN","title":"ORAGENICS INC"},"6844":{"cik_str":96793,"ticker":"SSY","title":"SUNLINK HEALTH SYSTEMS INC"},"6845":{"cik_str":96885,"ticker":"TIKK","title":"TEL INSTRUMENT ELECTRONICS CORP"},"6846":{"cik_str":1609988,"ticker":"FLCX","title":"flooidCX Corp."},"6847":{"cik_str":1071840,"ticker":"WNDW","title":"SolarWindow Technologies, Inc."},"6848":{"cik_str":1004724,"ticker":"RHE","title":"REGIONAL HEALTH PROPERTIES, INC"},"6849":{"cik_str":34956,"ticker":"TENX","title":"TENAX THERAPEUTICS, INC."},"6850":{"cik_str":1107280,"ticker":"OVTZ","title":"OCULUS VISIONTECH INC."},"6851":{"cik_str":1571934,"ticker":"SNPX","title":"Synaptogenix, Inc."},"6852":{"cik_str":1072772,"ticker":"TMBXF","title":"TOMBSTONE EXPLORATION CORP"},"6853":{"cik_str":1482753,"ticker":"PMCOF","title":"Prospector Metals Corp."},"6854":{"cik_str":1435064,"ticker":"CETX","title":"CEMTREX INC"},"6855":{"cik_str":1281845,"ticker":"UNQL","title":"Unique Logistics International, Inc."},"6856":{"cik_str":1668370,"ticker":"TBLT","title":"Toughbuilt Industries, Inc"},"6857":{"cik_str":1643721,"ticker":"LBUY","title":"LEAFBUYER TECHNOLOGIES, INC."},"6858":{"cik_str":1053691,"ticker":"DFFN","title":"Diffusion Pharmaceuticals Inc."},"6859":{"cik_str":1875547,"ticker":"LCFY","title":"Locafy Ltd"},"6860":{"cik_str":1339005,"ticker":"FEMY","title":"FEMASYS INC"},"6861":{"cik_str":946486,"ticker":"WINT","title":"WINDTREE THERAPEUTICS INC /DE/"},"6862":{"cik_str":811240,"ticker":"BIOL","title":"BIOLASE, INC"},"6863":{"cik_str":1604191,"ticker":"FWBI","title":"First Wave BioPharma, Inc."},"6864":{"cik_str":1745078,"ticker":"VYND","title":"Vynleads, Inc."},"6865":{"cik_str":1725430,"ticker":"INBS","title":"INTELLIGENT BIO SOLUTIONS INC."},"6866":{"cik_str":100716,"ticker":"UNAM","title":"UNICO AMERICAN CORP"},"6867":{"cik_str":1543268,"ticker":"BTB","title":"Bit Brother Ltd"},"6868":{"cik_str":1187953,"ticker":"CELZ","title":"CREATIVE MEDICAL TECHNOLOGY HOLDINGS, INC."},"6869":{"cik_str":1394108,"ticker":"SUIC","title":"SUIC Worldwide Holdings Ltd."},"6870":{"cik_str":1514183,"ticker":"SILO","title":"Silo Pharma, Inc."},"6871":{"cik_str":1687277,"ticker":"RETO","title":"ReTo Eco-Solutions, Inc."},"6872":{"cik_str":1419051,"ticker":"TSOI","title":"THERAPEUTIC SOLUTIONS INTERNATIONAL, INC."},"6873":{"cik_str":1067873,"ticker":"IPTK","title":"AS-IP TECH INC"},"6874":{"cik_str":1346346,"ticker":"CMOT","title":"Curtiss Motorcycle Company, Inc."},"6875":{"cik_str":105744,"ticker":"GBR","title":"New Concept Energy, Inc."},"6876":{"cik_str":1103090,"ticker":"RSCF","title":"REFLECT SCIENTIFIC, INC."},"6877":{"cik_str":1613979,"ticker":"ZPHYF","title":"ZEPHYR MINERALS LTD."},"6878":{"cik_str":887247,"ticker":"ADMP","title":"Adamis Pharmaceuticals Corp"},"6879":{"cik_str":1213660,"ticker":"BIMI","title":"BIMI International Medical Inc."},"6880":{"cik_str":1543652,"ticker":"FFLO","title":"Free Flow, Inc."},"6881":{"cik_str":922521,"ticker":"FALC","title":"FALCONSTOR SOFTWARE INC"},"6882":{"cik_str":1778651,"ticker":"JUVAF","title":"JUVA LIFE INC./Canada"},"6883":{"cik_str":1494558,"ticker":"AMBO","title":"Ambow Education Holding Ltd."},"6884":{"cik_str":1652452,"ticker":"CXXMF","title":"CMX GOLD & SILVER CORP."},"6885":{"cik_str":1861657,"ticker":"HILS","title":"Hillstream BioPharma Inc."},"6886":{"cik_str":1087329,"ticker":"RSHN","title":"RushNet, Inc."},"6887":{"cik_str":1029125,"ticker":"PBLA","title":"Panbela Therapeutics, Inc."},"6888":{"cik_str":1513525,"ticker":"ADIL","title":"ADIAL PHARMACEUTICALS, INC."},"6889":{"cik_str":1885408,"ticker":"JFBR","title":"Jeffs' Brands Ltd"},"6890":{"cik_str":1510964,"ticker":"CVSI","title":"CV Sciences, Inc."},"6891":{"cik_str":1734768,"ticker":"IMVIQ","title":"IMV Inc."},"6892":{"cik_str":1552189,"ticker":"PLSH","title":"PANACEA LIFE SCIENCES HOLDINGS, INC."},"6893":{"cik_str":1673475,"ticker":"GPOX","title":"GPO Plus, Inc."},"6894":{"cik_str":1622057,"ticker":"NBIO","title":"Nascent Biotech Inc."},"6895":{"cik_str":1348362,"ticker":"LEXX","title":"Lexaria Bioscience Corp."},"6896":{"cik_str":1759352,"ticker":"CGOLF","title":"Contact Gold Corp."},"6897":{"cik_str":1464165,"ticker":"BETRF","title":"BetterLife Pharma Inc."},"6898":{"cik_str":1482541,"ticker":"CEAD","title":"CEA Industries Inc."},"6899":{"cik_str":1825452,"ticker":"ONFO","title":"Onfolio Holdings, Inc"},"6900":{"cik_str":319458,"ticker":"ENSV","title":"Enservco Corp"},"6901":{"cik_str":1861522,"ticker":"PIK","title":"KIDPIK CORP."},"6902":{"cik_str":803578,"ticker":"WAVD","title":"WAVEDANCER, INC."},"6903":{"cik_str":1622244,"ticker":"OWPC","title":"One World Products, Inc."},"6904":{"cik_str":1324736,"ticker":"HENC","title":"Hero Technologies Inc."},"6905":{"cik_str":1043150,"ticker":"EEGI","title":"Eline Entertainment Group, Inc."},"6906":{"cik_str":1817760,"ticker":"SMTK","title":"SmartKem, Inc."},"6907":{"cik_str":1343465,"ticker":"SNPW","title":"Sun Pacific Holding Corp."},"6908":{"cik_str":1780097,"ticker":"BXRX","title":"Baudax Bio, Inc."},"6909":{"cik_str":1450894,"ticker":"IAALF","title":"IBC Advanced Alloys Corp."},"6910":{"cik_str":1633441,"ticker":"SECO","title":"Secoo Holding Ltd"},"6911":{"cik_str":1603519,"ticker":"ALXEF","title":"ALX RESOURCES CORP."},"6912":{"cik_str":20639,"ticker":"ABCP","title":"AmBase Corp"},"6913":{"cik_str":1690080,"ticker":"ATNF","title":"180 Life Sciences Corp."},"6914":{"cik_str":1141284,"ticker":"ASNS","title":"ACTELIS NETWORKS INC"},"6915":{"cik_str":1042418,"ticker":"INTI","title":"Inhibitor Therapeutics, Inc."},"6916":{"cik_str":1626644,"ticker":"ODYY","title":"Odyssey Health, Inc."},"6917":{"cik_str":1350102,"ticker":"ASTI","title":"Ascent Solar Technologies, Inc."},"6918":{"cik_str":1671502,"ticker":"QNRX","title":"Quoin Pharmaceuticals, Ltd."},"6919":{"cik_str":1589150,"ticker":"RGBP","title":"Regen BioPharma Inc"},"6920":{"cik_str":1191832,"ticker":"KNGRF","title":"Kingsmen Resources Ltd."},"6921":{"cik_str":1635077,"ticker":"ACON","title":"Aclarion, Inc."},"6922":{"cik_str":1164256,"ticker":"DBRM","title":"DAYBREAK OIL & GAS, INC."},"6923":{"cik_str":1527541,"ticker":"WHLR","title":"Wheeler Real Estate Investment Trust, Inc."},"6924":{"cik_str":812796,"ticker":"SNGX","title":"SOLIGENIX, INC."},"6925":{"cik_str":1342936,"ticker":"AVOI","title":"Advanced Voice Recognition Systems, Inc"},"6926":{"cik_str":1326706,"ticker":"NAOV","title":"NanoVibronix, Inc."},"6927":{"cik_str":1605888,"ticker":"SQL","title":"SeqLL, Inc."},"6928":{"cik_str":1810560,"ticker":"REVB","title":"REVELATION BIOSCIENCES, INC."},"6929":{"cik_str":1859007,"ticker":"ZVSA","title":"ZyVersa Therapeutics, Inc."},"6930":{"cik_str":1621221,"ticker":"ARTL","title":"ARTELO BIOSCIENCES, INC."},"6931":{"cik_str":758743,"ticker":"VIDE","title":"VIDEO DISPLAY CORP"},"6932":{"cik_str":1815566,"ticker":"IFBD","title":"Infobird Co., Ltd"},"6933":{"cik_str":102109,"ticker":"UUU","title":"UNIVERSAL SECURITY INSTRUMENTS INC"},"6934":{"cik_str":1362705,"ticker":"SNMP","title":"Evolve Transition Infrastructure LP"},"6935":{"cik_str":1806524,"ticker":"LGHL","title":"Lion Group Holding Ltd"},"6936":{"cik_str":1689084,"ticker":"QRON","title":"Qrons Inc."},"6937":{"cik_str":1639068,"ticker":"HBUV","title":"Hubilu Venture Corp"},"6938":{"cik_str":1883083,"ticker":"GPLL","title":"GPL Holdings, Inc."},"6939":{"cik_str":1014052,"ticker":"DTGI","title":"Digerati Technologies, Inc."},"6940":{"cik_str":1532619,"ticker":"PW","title":"Power REIT"},"6941":{"cik_str":1023549,"ticker":"XTLB","title":"XTL BIOPHARMACEUTICALS LTD"},"6942":{"cik_str":1705682,"ticker":"VNTRQ","title":"Venator Materials PLC"},"6943":{"cik_str":1387473,"ticker":"TIGCF","title":"Triumph Gold Corp."},"6944":{"cik_str":1638911,"ticker":"USDR","title":"UAS Drone Corp."},"6945":{"cik_str":1128252,"ticker":"SIPN","title":"SIPP International Industries, Inc."},"6946":{"cik_str":1432176,"ticker":"NSAV","title":"Net Savings Link, Inc./DE"},"6947":{"cik_str":1367083,"ticker":"SNOA","title":"Sonoma Pharmaceuticals, Inc."},"6948":{"cik_str":1493137,"ticker":"LCTC","title":"Lifeloc Technologies, Inc"},"6949":{"cik_str":1478069,"ticker":"CFRX","title":"CONTRAFECT Corp"},"6950":{"cik_str":1610853,"ticker":"HSDT","title":"HELIUS MEDICAL TECHNOLOGIES, INC."},"6951":{"cik_str":1424864,"ticker":"RYES","title":"Rise Gold Corp."},"6952":{"cik_str":1403708,"ticker":"EVOK","title":"Evoke Pharma Inc"},"6953":{"cik_str":768408,"ticker":"CYAN","title":"CYANOTECH CORP"},"6954":{"cik_str":1080448,"ticker":"PGOL","title":"PATRIOT GOLD CORP"},"6955":{"cik_str":1500123,"ticker":"INLB","title":"Item 9 Labs Corp."},"6956":{"cik_str":1838128,"ticker":"LOWLF","title":"Lowell Farms Inc."},"6957":{"cik_str":1533040,"ticker":"PHIO","title":"Phio Pharmaceuticals Corp."},"6958":{"cik_str":1095981,"ticker":"PSTV","title":"PLUS THERAPEUTICS, INC."},"6959":{"cik_str":700764,"ticker":"VYEY","title":"VICTORY OILFIELD TECH, INC."},"6960":{"cik_str":1551887,"ticker":"DUSYF","title":"DUESENBERG TECHNOLOGIES INC."},"6961":{"cik_str":1744345,"ticker":"RWBYF","title":"Red White & Bloom Brands Inc."},"6962":{"cik_str":1269026,"ticker":"SINT","title":"Sintx Technologies, Inc."},"6963":{"cik_str":1853816,"ticker":"DRMA","title":"Dermata Therapeutics, Inc."},"6964":{"cik_str":1580490,"ticker":"IONI","title":"I-ON Digital Corp."},"6965":{"cik_str":1281984,"ticker":"WDLF","title":"Decentral Life, Inc."},"6966":{"cik_str":1421636,"ticker":"CBNT","title":"C-Bond Systems, Inc"},"6967":{"cik_str":802257,"ticker":"MITI","title":"Mitesco, Inc."},"6968":{"cik_str":1595353,"ticker":"GLMD","title":"Galmed Pharmaceuticals Ltd."},"6969":{"cik_str":890725,"ticker":"GSAC","title":"GELSTAT CORP"},"6970":{"cik_str":1632121,"ticker":"BLNC","title":"Balance Labs, Inc."},"6971":{"cik_str":1563463,"ticker":"NLBS","title":"NUTRALIFE BIOSCIENCES, INC"},"6972":{"cik_str":1883898,"ticker":"STUPF","title":"Straightup Resources Inc"},"6973":{"cik_str":1293818,"ticker":"OPGN","title":"OPGEN INC"},"6974":{"cik_str":1609258,"ticker":"PTCO","title":"PetroGas Co"},"6975":{"cik_str":1108630,"ticker":"LIVC","title":"Live Current Media Inc."},"6976":{"cik_str":1411057,"ticker":"CHNC","title":"Cannabis Bioscience International Holdings, Inc."},"6977":{"cik_str":1282980,"ticker":"NVNT","title":"NOVINT TECHNOLOGIES INC"},"6978":{"cik_str":1528188,"ticker":"WNFT","title":"Worldwide NFT Inc."},"6979":{"cik_str":1103833,"ticker":"CRWE","title":"Crown Equity Holdings, Inc."},"6980":{"cik_str":1510832,"ticker":"BOPO","title":"Biopower Operations Corp"},"6981":{"cik_str":1674365,"ticker":"APTX","title":"Aptinyx Inc."},"6982":{"cik_str":1123596,"ticker":"BABB","title":"BAB, INC."},"6983":{"cik_str":1591913,"ticker":"IPSI","title":"Innovative Payment Solutions, Inc."},"6984":{"cik_str":923601,"ticker":"MICS","title":"SINGING MACHINE CO INC"},"6985":{"cik_str":1674227,"ticker":"WORX","title":"SCWorx Corp."},"6986":{"cik_str":1641640,"ticker":"NBRV","title":"Nabriva Therapeutics plc"},"6987":{"cik_str":1414953,"ticker":"MOJO","title":"EQUATOR Beverage Co"},"6988":{"cik_str":1454263,"ticker":"SZLSF","title":"STAGEZERO LIFE SCIENCES LTD."},"6989":{"cik_str":1549631,"ticker":"QURT","title":"Quarta-Rad, Inc."},"6990":{"cik_str":1100397,"ticker":"ADXS","title":"Ayala Pharmaceuticals, Inc."},"6991":{"cik_str":1704287,"ticker":"BJDX","title":"Bluejay Diagnostics, Inc."},"6992":{"cik_str":1508348,"ticker":"ACAN","title":"AmeriCann, Inc."},"6993":{"cik_str":924168,"ticker":"EFOI","title":"ENERGY FOCUS, INC/DE"},"6994":{"cik_str":1615219,"ticker":"SLRX","title":"Salarius Pharmaceuticals, Inc."},"6995":{"cik_str":1103310,"ticker":"ICCO","title":"INTERCARE DX INC"},"6996":{"cik_str":1798270,"ticker":"IONM","title":"Assure Holdings Corp."},"6997":{"cik_str":1559157,"ticker":"SSOK","title":"Sunstock, Inc."},"6998":{"cik_str":1593549,"ticker":"NUGN","title":"Livento Group, Inc."},"6999":{"cik_str":1581220,"ticker":"XALL","title":"Xalles Holdings Inc."},"7000":{"cik_str":1592058,"ticker":"PRTYQ","title":"Party City Holdco Inc."},"7001":{"cik_str":1460702,"ticker":"QLGN","title":"Qualigen Therapeutics, Inc."},"7002":{"cik_str":1802546,"ticker":"KGKG","title":"KONA GOLD BEVERAGE, INC."},"7003":{"cik_str":1611831,"ticker":"CDELF","title":"CANDELARIA MINING CORP."},"7004":{"cik_str":1716621,"ticker":"RMED","title":"Ra Medical Systems, Inc."},"7005":{"cik_str":1291855,"ticker":"SPCB","title":"SuperCom Ltd"},"7006":{"cik_str":1534525,"ticker":"XBIO","title":"Xenetic Biosciences, Inc."},"7007":{"cik_str":1054102,"ticker":"IDXG","title":"INTERPACE BIOSCIENCES, INC."},"7008":{"cik_str":1788841,"ticker":"MCOM","title":"micromobility.com Inc."},"7009":{"cik_str":1358190,"ticker":"ITP","title":"IT TECH PACKAGING, INC."},"7010":{"cik_str":1649009,"ticker":"SYTA","title":"Siyata Mobile Inc."},"7011":{"cik_str":1701963,"ticker":"VS","title":"Versus Systems Inc."},"7012":{"cik_str":1031093,"ticker":"SVBL","title":"SILVER BULL RESOURCES, INC."},"7013":{"cik_str":1534120,"ticker":"AVTX","title":"Avalo Therapeutics, Inc."},"7014":{"cik_str":1611747,"ticker":"BVXV","title":"BiondVax Pharmaceuticals Ltd."},"7015":{"cik_str":1468978,"ticker":"SITS","title":"SOUTHERN ITS INTERNATIONAL, INC."},"7016":{"cik_str":1729944,"ticker":"BACK","title":"IMAC Holdings, Inc."},"7017":{"cik_str":930775,"ticker":"ECIA","title":"ENCISION INC"},"7018":{"cik_str":1584693,"ticker":"HITC","title":"Healthcare Integrated Technologies Inc."},"7019":{"cik_str":1670869,"ticker":"RMHB","title":"Rocky Mountain High Brands, Inc."},"7020":{"cik_str":1480313,"ticker":"SMREF","title":"SUN SUMMIT MINERALS CORP."},"7021":{"cik_str":319016,"ticker":"FZMD","title":"Fuse Medical, Inc."},"7022":{"cik_str":1691077,"ticker":"YAYO","title":"EVmo, Inc."},"7023":{"cik_str":1419554,"ticker":"BBLG","title":"Bone Biologics Corp"},"7024":{"cik_str":707511,"ticker":"RGRX","title":"REGENERX BIOPHARMACEUTICALS INC"},"7025":{"cik_str":1178727,"ticker":"COMS","title":"COMSovereign Holding Corp."},"7026":{"cik_str":1782941,"ticker":"TIRX","title":"TIAN RUIXIANG HOLDINGS LTD"},"7027":{"cik_str":1434737,"ticker":"SRSG","title":"SPIRITS TIME INTERNATIONAL, INC."},"7028":{"cik_str":1427570,"ticker":"RSLS","title":"ReShape Lifesciences Inc."},"7029":{"cik_str":1976923,"ticker":"YQAI","title":"YOUNEEQAI TECHNICAL SERVICES, INC."},"7030":{"cik_str":1410187,"ticker":"CNNC","title":"Cannonau Corp."},"7031":{"cik_str":1559998,"ticker":"VINO","title":"Gaucho Group Holdings, Inc."},"7032":{"cik_str":1557798,"ticker":"CIIT","title":"Tianci International, Inc."},"7033":{"cik_str":1439264,"ticker":"BONZ","title":"Bonanza Goldfields Corp."},"7034":{"cik_str":1871181,"ticker":"QLIS","title":"Qualis Innovations, Inc."},"7035":{"cik_str":1711012,"ticker":"AIHS","title":"Senmiao Technology Ltd"},"7036":{"cik_str":1128281,"ticker":"SKAS","title":"Saker Aviation Services, Inc."},"7037":{"cik_str":1500620,"ticker":"BGAVF","title":"Bravada Gold Corp"},"7038":{"cik_str":1796514,"ticker":"METX","title":"Meten Holding Group Ltd."},"7039":{"cik_str":1815903,"ticker":"PTPI","title":"Petros Pharmaceuticals, Inc."},"7040":{"cik_str":1346022,"ticker":"ENRT","title":"Enertopia Corp."},"7041":{"cik_str":1292519,"ticker":"NVIV","title":"INVIVO THERAPEUTICS HOLDINGS CORP."},"7042":{"cik_str":1566610,"ticker":"VERB","title":"Verb Technology Company, Inc."},"7043":{"cik_str":819050,"ticker":"FRTX","title":"Fresh Tracks Therapeutics, Inc."},"7044":{"cik_str":1555972,"ticker":"STCC","title":"STERLING CONSOLIDATED Corp"},"7045":{"cik_str":1840102,"ticker":"SPTY","title":"SPECIFICITY, INC."},"7046":{"cik_str":1442492,"ticker":"LRDC","title":"Laredo Oil, Inc."},"7047":{"cik_str":1652350,"ticker":"RWRDP","title":"iConsumer Corp."},"7048":{"cik_str":1495584,"ticker":"SVVC","title":"Firsthand Technology Value Fund, Inc."},"7049":{"cik_str":709005,"ticker":"NROM","title":"NOBLE ROMANS INC"},"7050":{"cik_str":1829635,"ticker":"RNAZ","title":"Transcode Therapeutics, Inc."},"7051":{"cik_str":1375195,"ticker":"CRTG","title":"CORETEC GROUP INC."},"7052":{"cik_str":730349,"ticker":"TOFB","title":"TOFUTTI BRANDS INC"},"7053":{"cik_str":1755953,"ticker":"KERN","title":"Akerna Corp."},"7054":{"cik_str":1372514,"ticker":"KPRX","title":"KIORA PHARMACEUTICALS INC"},"7055":{"cik_str":28823,"ticker":"DBDQQ","title":"DIEBOLD NIXDORF, Inc"},"7056":{"cik_str":1555214,"ticker":"WLYW","title":"Wally World Media, Inc"},"7057":{"cik_str":1601936,"ticker":"CYTO","title":"Altamira Therapeutics Ltd."},"7058":{"cik_str":1357671,"ticker":"VOCL","title":"Creatd, Inc."},"7059":{"cik_str":1528172,"ticker":"ENDV","title":"ENDONOVO THERAPEUTICS, INC."},"7060":{"cik_str":1826397,"ticker":"AGRI","title":"AGRIFORCE GROWING SYSTEMS LTD."},"7061":{"cik_str":1536089,"ticker":"VRVR","title":"VIRTUAL INTERACTIVE TECHNOLOGIES CORP."},"7062":{"cik_str":1674440,"ticker":"CNXA","title":"Connexa Sports Technologies Inc."},"7063":{"cik_str":918573,"ticker":"GRVE","title":"GROOVE BOTANICALS INC."},"7064":{"cik_str":1100779,"ticker":"MLMN","title":"Millennium Prime, Inc."},"7065":{"cik_str":1644903,"ticker":"YCBD","title":"cbdMD, Inc."},"7066":{"cik_str":1462223,"ticker":"RCRT","title":"Recruiter.com Group, Inc."},"7067":{"cik_str":1451448,"ticker":"GMBL","title":"ESPORTS ENTERTAINMENT GROUP, INC."},"7068":{"cik_str":1611746,"ticker":"SPRC","title":"SciSparc Ltd."},"7069":{"cik_str":1892492,"ticker":"OCTO","title":"Eightco Holdings Inc."},"7070":{"cik_str":1389067,"ticker":"TTCM","title":"TAUTACHROME INC."},"7071":{"cik_str":1668010,"ticker":"DBGI","title":"Digital Brands Group, Inc."},"7072":{"cik_str":788611,"ticker":"SASI","title":"SIGMA ADDITIVE SOLUTIONS, INC."},"7073":{"cik_str":841533,"ticker":"SRMX","title":"SADDLE RANCH MEDIA, INC."},"7074":{"cik_str":1141964,"ticker":"PCMC","title":"PUBLIC CO MANAGEMENT CORP"},"7075":{"cik_str":1652958,"ticker":"EDGM","title":"Edgemode, Inc."},"7076":{"cik_str":1722731,"ticker":"FDCT","title":"FDCTECH, INC."},"7077":{"cik_str":1566826,"ticker":"LGMK","title":"LogicMark, Inc."},"7078":{"cik_str":1800637,"ticker":"AGFY","title":"Agrify Corp"},"7079":{"cik_str":1121795,"ticker":"GHST","title":"GHST World Inc."},"7080":{"cik_str":862861,"ticker":"JAN","title":"JanOne Inc."},"7081":{"cik_str":1537561,"ticker":"ARTH","title":"Arch Therapeutics, Inc."},"7082":{"cik_str":1716947,"ticker":"ENSC","title":"Ensysce Biosciences, Inc."},"7083":{"cik_str":1413909,"ticker":"DSGT","title":"DSG Global Inc."},"7084":{"cik_str":1156784,"ticker":"COWI","title":"CarbonMeta Technologies, Inc."},"7085":{"cik_str":1511820,"ticker":"STEK","title":"Stemtech Corp"},"7086":{"cik_str":1851682,"ticker":"MF","title":"Missfresh Ltd"},"7087":{"cik_str":1165320,"ticker":"GBLX","title":"GB SCIENCES INC"},"7088":{"cik_str":1284237,"ticker":"ABCFF","title":"ABACUS MINING & EXPLORATION CORP"},"7089":{"cik_str":1941771,"ticker":"FTHWF","title":"Field Trip Health & Wellness Ltd."},"7090":{"cik_str":1629205,"ticker":"IVBT","title":"Innovation1 Biotech Inc."},"7091":{"cik_str":1376804,"ticker":"VNUE","title":"VNUE, Inc."},"7092":{"cik_str":1867589,"ticker":"RWGI","title":"Rodedawg International Industries, Inc."},"7093":{"cik_str":1324759,"ticker":"HGYN","title":"HONG YUAN HOLDING GROUP"},"7094":{"cik_str":1342219,"ticker":"NLSC","title":"Namliong SkyCosmos, Inc."},"7095":{"cik_str":1725516,"ticker":"REII","title":"RENEWABLE INNOVATIONS, INC."},"7096":{"cik_str":1586554,"ticker":"CBDY","title":"Target Group Inc."},"7097":{"cik_str":1565146,"ticker":"GULTU","title":"Gulf Coast Ultra Deep Royalty Trust"},"7098":{"cik_str":1469443,"ticker":"RKDA","title":"Arcadia Biosciences, Inc."},"7099":{"cik_str":1417664,"ticker":"VEII","title":"Value Exchange International, Inc."},"7100":{"cik_str":1406588,"ticker":"MNGG","title":"Mining Global, Inc."},"7101":{"cik_str":1358403,"ticker":"BLCM","title":"BELLICUM PHARMACEUTICALS, INC"},"7102":{"cik_str":1686850,"ticker":"MOTS","title":"Motus GI Holdings, Inc."},"7103":{"cik_str":1790665,"ticker":"GNRS","title":"Greenrose Holding Co Inc."},"7104":{"cik_str":1766267,"ticker":"SHMY","title":"Synergy Empire Ltd"},"7105":{"cik_str":77281,"ticker":"PRET","title":"PENNSYLVANIA REAL ESTATE INVESTMENT TRUST"},"7106":{"cik_str":1106644,"ticker":"CPHI","title":"CHINA PHARMA HOLDINGS, INC."},"7107":{"cik_str":1951067,"ticker":"CISS","title":"C3is Inc."},"7108":{"cik_str":746210,"ticker":"OBLG","title":"Oblong, Inc."},"7109":{"cik_str":1411906,"ticker":"AMPE","title":"Ampio Pharmaceuticals, Inc."},"7110":{"cik_str":711034,"ticker":"THMG","title":"THUNDER MOUNTAIN GOLD INC"},"7111":{"cik_str":1725911,"ticker":"GDMK","title":"NetBrands Corp."},"7112":{"cik_str":1680378,"ticker":"SNES","title":"SenesTech, Inc."},"7113":{"cik_str":1507713,"ticker":"AUNFF","title":"Aurcana Silver Corp"},"7114":{"cik_str":1499494,"ticker":"DXF","title":"Dunxin Financial Holdings Ltd"},"7115":{"cik_str":1553846,"ticker":"RDHL","title":"RedHill Biopharma Ltd."},"7116":{"cik_str":1572386,"ticker":"GWTI","title":"GREENWAY TECHNOLOGIES INC"},"7117":{"cik_str":1946335,"ticker":"INUMF","title":"Infinitum Copper Corp."},"7118":{"cik_str":1940674,"ticker":"SMX","title":"SMX (Security Matters) Public Ltd Co"},"7119":{"cik_str":1593204,"ticker":"ADAD","title":"Huaizhong Health Group, Inc."},"7120":{"cik_str":1566469,"ticker":"NHIQ","title":"NantHealth, Inc."},"7121":{"cik_str":1001601,"ticker":"MGTI","title":"MGT CAPITAL INVESTMENTS, INC."},"7122":{"cik_str":318299,"ticker":"SRCO","title":"SPARTA COMMERCIAL SERVICES, INC."},"7123":{"cik_str":1290504,"ticker":"NUGS","title":"CANNABIS STRATEGIC VENTURES"},"7124":{"cik_str":1892500,"ticker":"CMND","title":"Clearmind Medicine Inc."},"7125":{"cik_str":1716324,"ticker":"RGMP","title":"Regnum Corp."},"7126":{"cik_str":1117228,"ticker":"MMMW","title":"MASS MEGAWATTS WIND POWER INC"},"7127":{"cik_str":810509,"ticker":"NAVB","title":"NAVIDEA BIOPHARMACEUTICALS, INC."},"7128":{"cik_str":1769768,"ticker":"EJH","title":"E-Home Household Service Holdings Ltd"},"7129":{"cik_str":1829966,"ticker":"EBET","title":"EBET, Inc."},"7130":{"cik_str":870826,"ticker":"IMBI","title":"iMedia Brands, Inc."},"7131":{"cik_str":1838937,"ticker":"ZMENY","title":"Zhangmen Education Inc."},"7132":{"cik_str":1261249,"ticker":"AGRX","title":"AGILE THERAPEUTICS INC"},"7133":{"cik_str":1697834,"ticker":"STMH","title":"Stem Holdings, Inc."},"7134":{"cik_str":1809750,"ticker":"EDBL","title":"Edible Garden AG Inc"},"7135":{"cik_str":1585380,"ticker":"INKW","title":"Greene Concepts, Inc"},"7136":{"cik_str":1680962,"ticker":"WGMCF","title":"WINSTON GOLD CORP."},"7137":{"cik_str":1045942,"ticker":"TRCK","title":"Track Group, Inc."},"7138":{"cik_str":1789192,"ticker":"SVFD","title":"Save Foods, Inc."},"7139":{"cik_str":1109504,"ticker":"EPWCF","title":"Empower Clinics Inc."},"7140":{"cik_str":1657214,"ticker":"ILAL","title":"International Land Alliance Inc."},"7141":{"cik_str":1081188,"ticker":"GEGP","title":"GOLD ENTERTAINMENT GROUP INC"},"7142":{"cik_str":1651166,"ticker":"GSBX","title":"Golden State Bancorp"},"7143":{"cik_str":1386049,"ticker":"BYOC","title":"Beyond Commerce, Inc."},"7144":{"cik_str":1304077,"ticker":"FMNJ","title":"Franklin Mining, Inc."},"7145":{"cik_str":1694617,"ticker":"ROYL","title":"Royale Energy, Inc."},"7146":{"cik_str":1506492,"ticker":"NUWE","title":"Nuwellis, Inc."},"7147":{"cik_str":1024095,"ticker":"CDJM","title":"Carnegie Development, Inc"},"7148":{"cik_str":88000,"ticker":"SLGD","title":"Scott's Liquid Gold - Inc."},"7149":{"cik_str":1375793,"ticker":"MLRT","title":"Metalert, Inc."},"7150":{"cik_str":1173281,"ticker":"NBSE","title":"NeuBase Therapeutics, Inc."},"7151":{"cik_str":1668082,"ticker":"BYLG","title":"BYLOG GROUP CORP."},"7152":{"cik_str":1792581,"ticker":"KRBP","title":"Kiromic Biopharma, Inc."},"7153":{"cik_str":1394638,"ticker":"DWAY","title":"Driveitaway Holdings, Inc."},"7154":{"cik_str":1829949,"ticker":"OGBLY","title":"Onion Global Ltd"},"7155":{"cik_str":1502152,"ticker":"GSTC","title":"GlobeStar Therapeutics Corp"},"7156":{"cik_str":884247,"ticker":"YVR","title":"Liquid Media Group Ltd."},"7157":{"cik_str":1084267,"ticker":"MOBQ","title":"Mobiquity Technologies, Inc."},"7158":{"cik_str":1285543,"ticker":"ZRFY","title":"Zerify, Inc."},"7159":{"cik_str":1383701,"ticker":"HSTO","title":"Histogen Inc."},"7160":{"cik_str":1424768,"ticker":"VYCO","title":"VYCOR MEDICAL INC"},"7161":{"cik_str":720875,"ticker":"DYNT","title":"DYNATRONICS CORP"},"7162":{"cik_str":1389545,"ticker":"NBY","title":"NovaBay Pharmaceuticals, Inc."},"7163":{"cik_str":1819074,"ticker":"PTNYF","title":"ParcelPal Logistics Inc."},"7164":{"cik_str":1728328,"ticker":"INM","title":"InMed Pharmaceuticals Inc."},"7165":{"cik_str":1099132,"ticker":"MHTX","title":"MANHATTAN SCIENTIFICS INC"},"7166":{"cik_str":1473334,"ticker":"NVFY","title":"Nova Lifestyle, Inc."},"7167":{"cik_str":1787740,"ticker":"TIVC","title":"Tivic Health Systems, Inc."},"7168":{"cik_str":1726711,"ticker":"ADTX","title":"Aditxt, Inc."},"7169":{"cik_str":1417926,"ticker":"INVO","title":"INVO Bioscience, Inc."},"7170":{"cik_str":1522222,"ticker":"CLSH","title":"CLS Holdings USA, Inc."},"7171":{"cik_str":1279715,"ticker":"IWSH","title":"Wright Investors Service Holdings, Inc."},"7172":{"cik_str":1593001,"ticker":"NGTF","title":"NightFood Holdings, Inc."},"7173":{"cik_str":1730773,"ticker":"BSFC","title":"Blue Star Foods Corp."},"7174":{"cik_str":1350073,"ticker":"ICNB","title":"Iconic Brands, Inc."},"7175":{"cik_str":1851860,"ticker":"SMFL","title":"SMART FOR LIFE, INC."},"7176":{"cik_str":1913210,"ticker":"BRSH","title":"Bruush Oral Care Inc."},"7177":{"cik_str":1763660,"ticker":"SEAV","title":"SEATech Ventures Corp."},"7178":{"cik_str":1639142,"ticker":"PLRTF","title":"Plymouth Rock Technologies Inc."},"7179":{"cik_str":932021,"ticker":"GTLL","title":"GLOBAL TECHNOLOGIES LTD"},"7180":{"cik_str":1706509,"ticker":"COSG","title":"Cosmos Group Holdings Inc."},"7181":{"cik_str":1499717,"ticker":"STAF","title":"Staffing 360 Solutions, Inc."},"7182":{"cik_str":1028357,"ticker":"BITTF","title":"BITTERROOT RESOURCES LTD"},"7183":{"cik_str":1821424,"ticker":"UK","title":"Ucommune International Ltd"},"7184":{"cik_str":1888014,"ticker":"AKAN","title":"AKANDA CORP."},"7185":{"cik_str":1760764,"ticker":"GMVD","title":"G Medical Innovations Holdings Ltd."},"7186":{"cik_str":1543623,"ticker":"UCLE","title":"US NUCLEAR CORP."},"7187":{"cik_str":1603345,"ticker":"AGTX","title":"Agentix Corp."},"7188":{"cik_str":1211805,"ticker":"MYSZ","title":"My Size, Inc."},"7189":{"cik_str":1525306,"ticker":"FSTJ","title":"First America Resources Corp"},"7190":{"cik_str":858043,"ticker":"CMDRF","title":"COMMANDER RESOURCES LTD"},"7191":{"cik_str":1443611,"ticker":"SING","title":"SinglePoint Inc."},"7192":{"cik_str":1497649,"ticker":"GSTX","title":"Graphene & Solar Technologies Ltd"},"7193":{"cik_str":1487091,"ticker":"QWTR","title":"Quest Water Global, Inc."},"7194":{"cik_str":1749849,"ticker":"OGAA","title":"Organic Agricultural Co Ltd"},"7195":{"cik_str":867028,"ticker":"FOMC","title":"FOMO WORLDWIDE, INC."},"7196":{"cik_str":1318268,"ticker":"MDEX","title":"Madison Technologies Inc."},"7197":{"cik_str":1805087,"ticker":"GLSH","title":"GELESIS HOLDINGS, INC."},"7198":{"cik_str":1755101,"ticker":"ALDS","title":"APPlife Digital Solutions Inc"},"7199":{"cik_str":720762,"ticker":"NIMU","title":"NON INVASIVE MONITORING SYSTEMS INC /FL/"},"7200":{"cik_str":1784567,"ticker":"HCDI","title":"Harbor Custom Development, Inc."},"7201":{"cik_str":1062506,"ticker":"ALDA","title":"ATLANTICA INC"},"7202":{"cik_str":1570937,"ticker":"ATAO","title":"ALTAIR INTERNATIONAL CORP."},"7203":{"cik_str":1534708,"ticker":"EAST","title":"Eastside Distilling, Inc."},"7204":{"cik_str":1662574,"ticker":"GROM","title":"Grom Social Enterprises, Inc."},"7205":{"cik_str":1088005,"ticker":"NXMR","title":"NextMart Inc."},"7206":{"cik_str":1803096,"ticker":"STRG","title":"STARGUIDE GROUP, INC."},"7207":{"cik_str":66600,"ticker":"MMMM","title":"Quad M Solutions, Inc."},"7208":{"cik_str":1452936,"ticker":"PCOK","title":"Pacific Oak Strategic Opportunity REIT, Inc."},"7209":{"cik_str":1300781,"ticker":"ENMI","title":"DH ENCHANTMENT, INC."},"7210":{"cik_str":1723980,"ticker":"SXTC","title":"China SXT Pharmaceuticals, Inc."},"7211":{"cik_str":1617669,"ticker":"UFAB","title":"Unique Fabricating, Inc."},"7212":{"cik_str":774415,"ticker":"KRFG","title":"KING RESOURCES, INC."},"7213":{"cik_str":1466301,"ticker":"CLVSQ","title":"Clovis Oncology, Inc."},"7214":{"cik_str":900475,"ticker":"SMKG","title":"SmartCard Marketing Systems Inc"},"7215":{"cik_str":873612,"ticker":"ABNAF","title":"Aben Resources Ltd."},"7216":{"cik_str":1401395,"ticker":"NEPT","title":"Neptune Wellness Solutions Inc."},"7217":{"cik_str":1319643,"ticker":"LSMG","title":"Lode-Star Mining Inc."},"7218":{"cik_str":811212,"ticker":"THMO","title":"ThermoGenesis Holdings, Inc."},"7219":{"cik_str":1547521,"ticker":"BBUZ","title":"Engage Mobility, Inc."},"7220":{"cik_str":1872812,"ticker":"TCBP","title":"TC BioPharm (Holdings) plc"},"7221":{"cik_str":1008653,"ticker":"EMAX","title":"Ecomax, Inc"},"7222":{"cik_str":1518336,"ticker":"DREM","title":"Dream Homes & Development Corp."},"7223":{"cik_str":1081938,"ticker":"CPMD","title":"CANNAPHARMARX, INC."},"7224":{"cik_str":1437750,"ticker":"TRXA","title":"T-REX Acquisition Corp."},"7225":{"cik_str":1599407,"ticker":"EFSH","title":"1847 Holdings LLC"},"7226":{"cik_str":1443089,"ticker":"WHSI","title":"WEARABLE HEALTH SOLUTIONS, INC."},"7227":{"cik_str":1597892,"ticker":"JRSS","title":"JRSIS HEALTH CARE Corp"},"7228":{"cik_str":1454742,"ticker":"GMER","title":"GOOD GAMING, INC."},"7229":{"cik_str":1039466,"ticker":"XSNX","title":"NovAccess Global Inc."},"7230":{"cik_str":1805024,"ticker":"AMHG","title":"Amergent Hospitality Group Inc."},"7231":{"cik_str":1530185,"ticker":"ATVK","title":"Ameritek Ventures, Inc."},"7232":{"cik_str":1625288,"ticker":"NXEN","title":"NEXIEN BIOPHARMA, INC."},"7233":{"cik_str":1911467,"ticker":"CRCE","title":"Circle Energy, Inc./NV"},"7234":{"cik_str":1064722,"ticker":"GMPW","title":"GIVEMEPOWER CORP"},"7235":{"cik_str":1473490,"ticker":"WSCO","title":"Wall Street Media Co, Inc."},"7236":{"cik_str":1642159,"ticker":"SIGY","title":"Sigyn Therapeutics, Inc."},"7237":{"cik_str":1424404,"ticker":"WOLV","title":"Wolverine Resources Corp."},"7238":{"cik_str":1133192,"ticker":"VPER","title":"VIPER NETWORKS INC"},"7239":{"cik_str":1661039,"ticker":"TPTW","title":"TPT GLOBAL TECH, INC."},"7240":{"cik_str":1384066,"ticker":"BMCS","title":"BMCS SUMCOIN INDEX FUND, INC."},"7241":{"cik_str":792935,"ticker":"GRST","title":"ETHEMA HEALTH Corp"},"7242":{"cik_str":1680132,"ticker":"CSUI","title":"CANNABIS SUISSE CORP."},"7243":{"cik_str":1445467,"ticker":"AVPMF","title":"AVRUPA MINERALS LTD."},"7244":{"cik_str":1341726,"ticker":"GSPE","title":"GULFSLOPE ENERGY, INC."},"7245":{"cik_str":925660,"ticker":"FLXT","title":"FLEXPOINT SENSOR SYSTEMS INC"},"7246":{"cik_str":1613685,"ticker":"TMIN","title":"TRENDMAKER INC. LTD."},"7247":{"cik_str":1402371,"ticker":"ELRA","title":"ELRAY RESOURCES, INC."},"7248":{"cik_str":1734262,"ticker":"CTKYY","title":"CooTek(Cayman)Inc."},"7249":{"cik_str":849997,"ticker":"FECOF","title":"FEC Resources Inc."},"7250":{"cik_str":1772028,"ticker":"SCPS","title":"Scopus BioPharma Inc."},"7251":{"cik_str":1481443,"ticker":"TCRI","title":"TechCom, Inc."},"7252":{"cik_str":1131903,"ticker":"FCCN","title":"SPECTRAL CAPITAL Corp"},"7253":{"cik_str":1437491,"ticker":"BNMV","title":"BitNile Metaverse, Inc."},"7254":{"cik_str":1199392,"ticker":"EGMCF","title":"EMERGENT METALS CORP."},"7255":{"cik_str":1084551,"ticker":"CRDV","title":"Community Redevelopment Inc."},"7256":{"cik_str":1869467,"ticker":"OP","title":"OceanPal Inc."},"7257":{"cik_str":1519177,"ticker":"SOLS","title":"SOLLENSYS CORP."},"7258":{"cik_str":1614556,"ticker":"STAL","title":"Star Alliance International Corp."},"7259":{"cik_str":1119897,"ticker":"PCTL","title":"PCT LTD"},"7260":{"cik_str":847942,"ticker":"STQN","title":"STRATEGIC ACQUISITIONS INC /NV/"},"7261":{"cik_str":1864448,"ticker":"WEJOQ","title":"Wejo Group Ltd"},"7262":{"cik_str":1644488,"ticker":"SHRG","title":"SHARING SERVICES GLOBAL Corp"},"7263":{"cik_str":1401835,"ticker":"SRGZ","title":"Star Gold Corp."},"7264":{"cik_str":1653247,"ticker":"ASAP","title":"Waitr Holdings Inc."},"7265":{"cik_str":812306,"ticker":"PCYN","title":"PROCYON CORP"},"7266":{"cik_str":1473579,"ticker":"NNMX","title":"NANOMIX Corp"},"7267":{"cik_str":1715433,"ticker":"LCHD","title":"Leader Capital Holdings Corp."},"7268":{"cik_str":824416,"ticker":"QPRC","title":"QUEST PATENT RESEARCH CORP"},"7269":{"cik_str":1737372,"ticker":"SYSX","title":"Sysorex, Inc."},"7270":{"cik_str":1416090,"ticker":"IMII","title":"INCEPTION MINING INC."},"7271":{"cik_str":1300050,"ticker":"GPLDF","title":"GREAT PANTHER MINING Ltd"},"7272":{"cik_str":1509786,"ticker":"GKIN","title":"Guskin Gold Corp."},"7273":{"cik_str":814926,"ticker":"CAPC","title":"CAPSTONE COMPANIES, INC."},"7274":{"cik_str":1830503,"ticker":"BZWR","title":"Business Warrior Corp"},"7275":{"cik_str":1138608,"ticker":"BLMS","title":"BLOOMIOS, INC."},"7276":{"cik_str":1389034,"ticker":"KALO","title":"Kallo Inc."},"7277":{"cik_str":1750777,"ticker":"HWKE","title":"Hawkeye Systems, Inc."},"7278":{"cik_str":1736865,"ticker":"DVLP","title":"Golden Developing Solutions, Inc."},"7279":{"cik_str":1568969,"ticker":"APYP","title":"APPYEA, INC"},"7280":{"cik_str":1856416,"ticker":"ECGI","title":"ECGI Holdings, Inc."},"7281":{"cik_str":1338929,"ticker":"AHRO","title":"Authentic Holdings, Inc."},"7282":{"cik_str":1318641,"ticker":"STAB","title":"Statera Biopharma, Inc."},"7283":{"cik_str":862668,"ticker":"ESMC","title":"ESCALON MEDICAL CORP"},"7284":{"cik_str":827099,"ticker":"CPWR","title":"Ocean Thermal Energy Corp"},"7285":{"cik_str":1618835,"ticker":"EVFM","title":"Evofem Biosciences, Inc."},"7286":{"cik_str":1044378,"ticker":"BIOC","title":"BIOCEPT INC"},"7287":{"cik_str":1666657,"ticker":"EBYH","title":"Strainsforpains, Inc."},"7288":{"cik_str":1492674,"ticker":"TTOO","title":"T2 Biosystems, Inc."},"7289":{"cik_str":730669,"ticker":"FCIC","title":"FCCC INC"},"7290":{"cik_str":1409036,"ticker":"TARSF","title":"Alianza Minerals Ltd."},"7291":{"cik_str":1409197,"ticker":"BSPK","title":"Bespoke Extracts, Inc."},"7292":{"cik_str":1389518,"ticker":"CMGR","title":"Clubhouse Media Group, Inc."},"7293":{"cik_str":1553264,"ticker":"APSI","title":"AQUA POWER SYSTEMS INC."},"7294":{"cik_str":1648087,"ticker":"AREB","title":"AMERICAN REBEL HOLDINGS INC"},"7295":{"cik_str":1501862,"ticker":"EPGG","title":"Empire Global Gaming, Inc."},"7296":{"cik_str":1358633,"ticker":"SNBH","title":"SENTIENT BRANDS HOLDINGS INC."},"7297":{"cik_str":1692068,"ticker":"TPPM","title":"Yotta Global, Inc."},"7298":{"cik_str":1935435,"ticker":"SSOF","title":"Sixty Six Oilfield Services, Inc."},"7299":{"cik_str":775057,"ticker":"ALTX","title":"ALTEX INDUSTRIES INC"},"7300":{"cik_str":1084475,"ticker":"NIHK","title":"Video River Networks, Inc."},"7301":{"cik_str":1743745,"ticker":"GNLN","title":"Greenlane Holdings, Inc."},"7302":{"cik_str":1635136,"ticker":"DUUO","title":"DUO WORLD INC"},"7303":{"cik_str":1099814,"ticker":"SVSN","title":"STEREO VISION ENTERTAINMENT INC"},"7304":{"cik_str":822411,"ticker":"IMUC","title":"EOM Pharmaceutical Holdings, Inc."},"7305":{"cik_str":1140586,"ticker":"NFEI","title":"NEW FRONTIER ENERGY INC"},"7306":{"cik_str":1524872,"ticker":"TNRG","title":"Thunder Energies Corp"},"7307":{"cik_str":1624985,"ticker":"XNDA","title":"Tribal Rides International Corp."},"7308":{"cik_str":1515251,"ticker":"SIXWF","title":"SIXTH WAVE INNOVATIONS INC."},"7309":{"cik_str":1096768,"ticker":"NUGL","title":"NUGL, INC."},"7310":{"cik_str":1658521,"ticker":"MTPP","title":"MOUNTAIN TOP PROPERTIES, INC."},"7311":{"cik_str":1543637,"ticker":"NUMD","title":"Nu-Med Plus, Inc."},"7312":{"cik_str":1693687,"ticker":"MSYN","title":"MS YOUNG ADVENTURE ENTERPRISE, INC."},"7313":{"cik_str":1811999,"ticker":"FMHS","title":"FARMHOUSE, INC. /NV"},"7314":{"cik_str":1434601,"ticker":"TMGI","title":"Marquie Group, Inc."},"7315":{"cik_str":1388319,"ticker":"USRM","title":"U.S. Stem Cell, Inc."},"7316":{"cik_str":1093636,"ticker":"ATYG","title":"Saxon Capital Group, Inc./DE"},"7317":{"cik_str":96699,"ticker":"TCCO","title":"TECHNICAL COMMUNICATIONS CORP"},"7318":{"cik_str":1301991,"ticker":"SMME","title":"SmartMetric, Inc."},"7319":{"cik_str":1346917,"ticker":"GNTOF","title":"GENTOR RESOURCES INC."},"7320":{"cik_str":1512886,"ticker":"NTRR","title":"NEUTRA CORP."},"7321":{"cik_str":1669400,"ticker":"VFRM","title":"Veritas Farms, Inc."},"7322":{"cik_str":1651992,"ticker":"ASFT","title":"Appsoft Technologies, Inc."},"7323":{"cik_str":945617,"ticker":"AMMJ","title":"American Cannabis Company, Inc."},"7324":{"cik_str":1474835,"ticker":"IPCIF","title":"Intellipharmaceutics International Inc."},"7325":{"cik_str":860543,"ticker":"JKSM","title":"Jacksam Corp"},"7326":{"cik_str":1474558,"ticker":"KATX","title":"KAT EXPLORATION, INC."},"7327":{"cik_str":1877461,"ticker":"CBDW","title":"1606 CORP."},"7328":{"cik_str":856984,"ticker":"USAQ","title":"QHSLab, Inc."},"7329":{"cik_str":1515317,"ticker":"MAGE","title":"MAGELLAN GOLD Corp"},"7330":{"cik_str":1661600,"ticker":"SATT","title":"SATIVUS TECH CORP."},"7331":{"cik_str":1530746,"ticker":"KAYS","title":"Kaya Holdings, Inc."},"7332":{"cik_str":1853718,"ticker":"TEVNF","title":"Tevano Systems Holdings Inc."},"7333":{"cik_str":1102942,"ticker":"INQR","title":"InnovaQor, Inc."},"7334":{"cik_str":1789330,"ticker":"MJHI","title":"MJ Harvest, Inc."},"7335":{"cik_str":1648903,"ticker":"FIFG","title":"First Foods Group, Inc."},"7336":{"cik_str":1884697,"ticker":"STRYQ","title":"Starry Group Holdings, Inc."},"7337":{"cik_str":1169138,"ticker":"GBUX","title":"GIVBUX, INC."},"7338":{"cik_str":1399352,"ticker":"WARM","title":"COOL TECHNOLOGIES, INC."},"7339":{"cik_str":1709542,"ticker":"XESP","title":"Electronic Servitor Publication Network Inc."},"7340":{"cik_str":1656501,"ticker":"BWMY","title":"BorrowMoney.com, Inc."},"7341":{"cik_str":1355677,"ticker":"MXSG","title":"Mexus Gold US"},"7342":{"cik_str":1456857,"ticker":"MJNE","title":"MJ Holdings, Inc."},"7343":{"cik_str":1619227,"ticker":"CLOW","title":"Cloudweb, Inc."},"7344":{"cik_str":1643918,"ticker":"BDRX","title":"Biodexa Pharmaceuticals Plc"},"7345":{"cik_str":1609436,"ticker":"MEXGF","title":"Mexican Gold Mining Corp."},"7346":{"cik_str":752294,"ticker":"ELST","title":"ELECTRONIC SYSTEMS TECHNOLOGY INC"},"7347":{"cik_str":1043894,"ticker":"WOEN","title":"WOLF ENERGY SERVICES INC."},"7348":{"cik_str":719274,"ticker":"GIGA","title":"GIGA TRONICS INC"},"7349":{"cik_str":1973160,"ticker":"GMZP","title":"GEMZ Corp"},"7350":{"cik_str":1300867,"ticker":"ONCI","title":"ON4 COMMUNICATIONS INC."},"7351":{"cik_str":799698,"ticker":"LADX","title":"LadRx Corp"},"7352":{"cik_str":1398713,"ticker":"NVDEF","title":"NEVADA EXPLORATION INC"},"7353":{"cik_str":1871928,"ticker":"LUXFF","title":"Luxxfolio Holdings Inc."},"7354":{"cik_str":1509957,"ticker":"CANB","title":"Can B Corp"},"7355":{"cik_str":930245,"ticker":"AASP","title":"GLOBAL ACQUISITIONS Corp"},"7356":{"cik_str":1413659,"ticker":"MSCH","title":"MainStreetChamber Holdings, Inc."},"7357":{"cik_str":1350156,"ticker":"PAXH","title":"PREAXIA HEALTH CARE PAYMENT SYSTEMS INC."},"7358":{"cik_str":1355790,"ticker":"ISCO","title":"International Stem Cell CORP"},"7359":{"cik_str":1703625,"ticker":"BLIS","title":"Treasure & Shipwreck Recovery, Inc."},"7360":{"cik_str":1407704,"ticker":"BOTY","title":"LINGERIE FIGHTING CHAMPIONSHIPS, INC."},"7361":{"cik_str":1290658,"ticker":"ITOX","title":"IIOT-OXYS, Inc."},"7362":{"cik_str":1393540,"ticker":"IGEN","title":"IGEN NETWORKS CORP"},"7363":{"cik_str":1302913,"ticker":"GEGI","title":"Genesis Electronics Group, Inc."},"7364":{"cik_str":1450307,"ticker":"COUV","title":"CORPORATE UNIVERSE INC"},"7365":{"cik_str":1671818,"ticker":"ONCR","title":"Oncorus, Inc."},"7366":{"cik_str":773318,"ticker":"VRTC","title":"VERITEC INC"},"7367":{"cik_str":1640251,"ticker":"WINSF","title":"Wins Finance Holdings Inc."},"7368":{"cik_str":1025771,"ticker":"WHLT","title":"CHASE PACKAGING CORP"},"7369":{"cik_str":825171,"ticker":"HHHEF","title":"37 CAPITAL INC"},"7370":{"cik_str":795212,"ticker":"KSPN","title":"Kaspien Holdings Inc."},"7371":{"cik_str":1308569,"ticker":"MLFB","title":"MAJOR LEAGUE FOOTBALL INC"},"7372":{"cik_str":1172178,"ticker":"LBSR","title":"LIBERTY STAR URANIUM & METALS CORP."},"7373":{"cik_str":1531477,"ticker":"HADV","title":"HEALTH ADVANCE INC."},"7374":{"cik_str":1485074,"ticker":"FRZT","title":"Freeze Tag, Inc."},"7375":{"cik_str":1161582,"ticker":"PHOT","title":"GROWLIFE, INC."},"7376":{"cik_str":1835567,"ticker":"PEARQ","title":"Pear Therapeutics, Inc."},"7377":{"cik_str":1426567,"ticker":"LBWR","title":"Labwire Inc"},"7378":{"cik_str":1125699,"ticker":"DLYT","title":"DAIS Corp"},"7379":{"cik_str":1300699,"ticker":"ATNXQ","title":"Athenex, Inc."},"7380":{"cik_str":1560905,"ticker":"TRTK","title":"TORtec Group Corp"},"7381":{"cik_str":1360442,"ticker":"CBDS","title":"Cannabis Sativa, Inc."},"7382":{"cik_str":1696411,"ticker":"CCCP","title":"Crona Corp."},"7383":{"cik_str":1335288,"ticker":"REOS","title":"ReoStar Energy CORP"},"7384":{"cik_str":1759424,"ticker":"EVVL","title":"Evil Empire Designs, Inc."},"7385":{"cik_str":727634,"ticker":"ISGN","title":"iSign Solutions Inc."},"7386":{"cik_str":1011901,"ticker":"WEBB","title":"Web Blockchain Media, Inc."},"7387":{"cik_str":90391,"ticker":"SILS","title":"SILVER SCOTT MINES INC"},"7388":{"cik_str":1471781,"ticker":"GTCH","title":"GBT Technologies Inc."},"7389":{"cik_str":1584480,"ticker":"LAAB","title":"Startech Labs, Inc."},"7390":{"cik_str":1594968,"ticker":"VEST","title":"Vestiage, Inc."},"7391":{"cik_str":822746,"ticker":"AMNI","title":"AMERICAN NOBLE GAS, INC."},"7392":{"cik_str":1652724,"ticker":"LHDXQ","title":"Lucira Health, Inc."},"7393":{"cik_str":1839412,"ticker":"QTEKQ","title":"QualTek Services Inc."},"7394":{"cik_str":1407878,"ticker":"DLOC","title":"Digital Locations, Inc."},"7395":{"cik_str":1388295,"ticker":"ONEI","title":"OneMeta Inc."},"7396":{"cik_str":1413754,"ticker":"MRZM","title":"MARIZYME, INC."},"7397":{"cik_str":1368275,"ticker":"WESC","title":"W&E Source Corp."},"7398":{"cik_str":894501,"ticker":"GRHI","title":"GOLD ROCK HOLDINGS, INC."},"7399":{"cik_str":1284454,"ticker":"YBCN","title":"Yong Bai Chao New Retail Corp"},"7400":{"cik_str":1645260,"ticker":"TOMDF","title":"TODOS MEDICAL LTD."},"7401":{"cik_str":1358654,"ticker":"RMESF","title":"RED METAL RESOURCES, LTD."},"7402":{"cik_str":1346655,"ticker":"CMGO","title":"CMG HOLDINGS GROUP, INC."},"7403":{"cik_str":1698370,"ticker":"TRLFF","title":"Maven Brands Inc."},"7404":{"cik_str":1083468,"ticker":"EARI","title":"ENTERTAINMENT ARTS RESEARCH, INC."},"7405":{"cik_str":1128353,"ticker":"ETCK","title":"ENERTECK CORP"},"7406":{"cik_str":1627480,"ticker":"PVNNF","title":"PV Nano Cell, Ltd."},"7407":{"cik_str":1607004,"ticker":"MRNJ","title":"METATRON APPS, INC."},"7408":{"cik_str":725929,"ticker":"BTDG","title":"B2Digital, Inc."},"7409":{"cik_str":1098462,"ticker":"MTLK","title":"METALINK LTD"},"7410":{"cik_str":1467505,"ticker":"PRRY","title":"Planet Resource Recovery, Inc."},"7411":{"cik_str":1089815,"ticker":"USNU","title":"U.S. NeuroSurgical Holdings, Inc."},"7412":{"cik_str":1433551,"ticker":"SGLA","title":"Sino Green Land Corp."},"7413":{"cik_str":1490054,"ticker":"VBHI","title":"VERDE BIO HOLDINGS, INC."},"7414":{"cik_str":797564,"ticker":"HSTC","title":"HST Global, Inc."},"7415":{"cik_str":1119643,"ticker":"NPHC","title":"NUTRA PHARMA CORP"},"7416":{"cik_str":1852707,"ticker":"BFYW","title":"Better For You Wellness, Inc."},"7417":{"cik_str":1653606,"ticker":"EHVVF","title":"Ehave, Inc."},"7418":{"cik_str":889608,"ticker":"HRBK","title":"HARBOR BANKSHARES CORP"},"7419":{"cik_str":1104280,"ticker":"SGBI","title":"SANGUI BIOTECH INTERNATIONAL INC"},"7420":{"cik_str":1076682,"ticker":"PTEIQ","title":"POLARITYTE, INC."},"7421":{"cik_str":1286768,"ticker":"UVSS","title":"UNIVERSAL SYSTEMS INC"},"7422":{"cik_str":1874138,"ticker":"SHGI","title":"Sparx Holdings Group, Inc."},"7423":{"cik_str":1616291,"ticker":"JPPYY","title":"Jupai Holdings Ltd"},"7424":{"cik_str":1559172,"ticker":"DPWW","title":"DIEGO PELLICER WORLDWIDE, INC"},"7425":{"cik_str":1704795,"ticker":"BANT","title":"Bantec, Inc."},"7426":{"cik_str":1435181,"ticker":"PHBI","title":"Pharmagreen Biotech Inc."},"7427":{"cik_str":1409253,"ticker":"NAFS","title":"North America Frac Sand, Inc."},"7428":{"cik_str":1162283,"ticker":"MGHL","title":"MORGAN GROUP HOLDING CO"},"7429":{"cik_str":820771,"ticker":"GFMH","title":"Goliath Film & Media Holdings"},"7430":{"cik_str":1169987,"ticker":"HTGMQ","title":"HTG MOLECULAR DIAGNOSTICS, INC"},"7431":{"cik_str":1587476,"ticker":"PSWW","title":"Principal Solar, Inc."},"7432":{"cik_str":1499275,"ticker":"SANP","title":"SANTO MINING CORP."},"7433":{"cik_str":1058330,"ticker":"ROAG","title":"Rogue One, Inc."},"7434":{"cik_str":1404804,"ticker":"OMTK","title":"Omnitek Engineering Corp"},"7435":{"cik_str":1261002,"ticker":"GNOLF","title":"GENOIL INC"},"7436":{"cik_str":1416697,"ticker":"BLPG","title":"Blue Line Protection Group, Inc."},"7437":{"cik_str":1975222,"ticker":"SSHT","title":"SSHT S&T Group Ltd."},"7438":{"cik_str":1591165,"ticker":"HCYT","title":"H-CYTE, INC."},"7439":{"cik_str":1438901,"ticker":"FLES","title":"Auto Parts 4Less Group, Inc."},"7440":{"cik_str":1132509,"ticker":"NNAX","title":"New Momentum Corp."},"7441":{"cik_str":1946585,"ticker":"BCNN","title":"Balincan USA, Inc"},"7442":{"cik_str":1612851,"ticker":"PLYN","title":"Palayan Resources, Inc."},"7443":{"cik_str":1556801,"ticker":"MYCB","title":"My City Builders, Inc."},"7444":{"cik_str":1599117,"ticker":"MNTR","title":"Mentor Capital, Inc."},"7445":{"cik_str":1382112,"ticker":"GMEV","title":"GME INNOTAINMENT, INC."},"7446":{"cik_str":1421204,"ticker":"RBSH","title":"Rebus Holdings, Inc."},"7447":{"cik_str":8328,"ticker":"AMNL","title":"Applied Minerals, Inc."},"7448":{"cik_str":1794621,"ticker":"QNGYQ","title":"Quanergy Systems, Inc."},"7449":{"cik_str":812152,"ticker":"RDGA","title":"RIDGEFIELD ACQUISITION CORP"},"7450":{"cik_str":1539778,"ticker":"RAYT","title":"RAYONT INC."},"7451":{"cik_str":1790515,"ticker":"EQOSQ","title":"EQONEX Ltd"},"7452":{"cik_str":1373853,"ticker":"NECA","title":"NEW AMERICA ENERGY CORP."},"7453":{"cik_str":352991,"ticker":"AOXY","title":"ADVANCED OXYGEN TECHNOLOGIES INC"},"7454":{"cik_str":1138586,"ticker":"XCRT","title":"Xcelerate, Inc."},"7455":{"cik_str":1444307,"ticker":"ONCSQ","title":"ONCOSEC MEDICAL Inc"},"7456":{"cik_str":811222,"ticker":"CDIX","title":"Cardiff Lexington Corp"},"7457":{"cik_str":1145898,"ticker":"CWNOF","title":"CHINESEWORLDNET COM INC"},"7458":{"cik_str":884650,"ticker":"IMCI","title":"INFINITE GROUP INC"},"7459":{"cik_str":1561686,"ticker":"SSET","title":"STARSTREAM ENTERTAINMENT, INC."},"7460":{"cik_str":1124197,"ticker":"AMLH","title":"AMERICAN LEISURE HOLDINGS, INC."},"7461":{"cik_str":1763657,"ticker":"OWVI","title":"One World Ventures Inc"},"7462":{"cik_str":1530425,"ticker":"ARRT","title":"Artisan Consumer Goods, Inc."},"7463":{"cik_str":1465311,"ticker":"ZICX","title":"Zicix Corp"},"7464":{"cik_str":1440280,"ticker":"XTRM","title":"EXTREME BIODIESEL, INC."},"7465":{"cik_str":882800,"ticker":"PACV","title":"Pacific Ventures Group, Inc."},"7466":{"cik_str":1517389,"ticker":"JFIL","title":"Jubilant Flame International, Ltd"},"7467":{"cik_str":1308027,"ticker":"VYST","title":"Vystar Corp"},"7468":{"cik_str":1575659,"ticker":"RTSL","title":"Rapid Therapeutic Science Laboratories, Inc."},"7469":{"cik_str":1158702,"ticker":"TAMG","title":"Transnational Group, Inc."},"7470":{"cik_str":915661,"ticker":"GRLF","title":"GREEN LEAF INNOVATIONS INC"},"7471":{"cik_str":1961,"ticker":"WDDD","title":"WORLDS INC"},"7472":{"cik_str":1763329,"ticker":"TPIA","title":"Mycotopia Therapies, Inc."},"7473":{"cik_str":1437476,"ticker":"GSFI","title":"Green Stream Holdings Inc."},"7474":{"cik_str":1502966,"ticker":"DIGP","title":"Digipath, Inc."},"7475":{"cik_str":1394220,"ticker":"RINO","title":"JOIN Entertainment Holdings, Inc."},"7476":{"cik_str":1321828,"ticker":"DCLT","title":"Data Call Technologies"},"7477":{"cik_str":1540615,"ticker":"PUGE","title":"PUGET TECHNOLOGIES, INC."},"7478":{"cik_str":844856,"ticker":"HCMC","title":"Healthier Choices Management Corp."},"7479":{"cik_str":896747,"ticker":"ABMC","title":"AMERICAN BIO MEDICA CORP"},"7480":{"cik_str":1517681,"ticker":"PPCB","title":"Propanc Biopharma, Inc."},"7481":{"cik_str":895287,"ticker":"VMHG","title":"VICTORY MARINE HOLDINGS CORP"},"7482":{"cik_str":1409446,"ticker":"NHMD","title":"NATE'S FOOD CO."},"7483":{"cik_str":1822529,"ticker":"TQLB","title":"Torque Lifestyle Brands, Inc."},"7484":{"cik_str":1575142,"ticker":"BDPT","title":"BIOADAPTIVES, INC."},"7485":{"cik_str":1590875,"ticker":"NSPDF","title":"NATURALLY SPLENDID ENTERPRISES LTD."},"7486":{"cik_str":1872292,"ticker":"RAKR","title":"Rainmaker Worldwide Inc."},"7487":{"cik_str":1695473,"ticker":"GCAN","title":"Greater Cannabis Company, Inc."},"7488":{"cik_str":1342792,"ticker":"WWSG","title":"WORLDWIDE STRATEGIES INC"},"7489":{"cik_str":1085277,"ticker":"SKVI","title":"SKINVISIBLE, INC."},"7490":{"cik_str":1440799,"ticker":"MMEX","title":"MMEX Resources Corp"},"7491":{"cik_str":731245,"ticker":"PDNLA","title":"PRESIDENTIAL REALTY CORP/DE/"},"7492":{"cik_str":1535079,"ticker":"MCCX","title":"MCX Technologies Corp"},"7493":{"cik_str":1596062,"ticker":"QBIO","title":"Q BioMed Inc."},"7494":{"cik_str":1539894,"ticker":"AFHIF","title":"Atlas Financial Holdings, Inc."},"7495":{"cik_str":1510247,"ticker":"LFAP","title":"LGBTQ Loyalty Holdings, Inc."},"7496":{"cik_str":1388994,"ticker":"CYDX","title":"CYduct Diagnostics, Inc."},"7497":{"cik_str":1574553,"ticker":"TRMNF","title":"NEW WAVE HOLDINGS CORP."},"7498":{"cik_str":819926,"ticker":"SEII","title":"SHARING ECONOMY INTERNATIONAL INC."},"7499":{"cik_str":1605331,"ticker":"ABQQ","title":"AB INTERNATIONAL GROUP CORP."},"7500":{"cik_str":1049011,"ticker":"KDCE","title":"KID CASTLE EDUCATIONAL CORP"},"7501":{"cik_str":1611852,"ticker":"RAHGF","title":"Roan Holdings Group Co., Ltd."},"7502":{"cik_str":1300524,"ticker":"AMIH","title":"AMERICAN INTERNATIONAL HOLDINGS CORP."},"7503":{"cik_str":1655075,"ticker":"AFIIQ","title":"Armstrong Flooring, Inc."},"7504":{"cik_str":1412659,"ticker":"RGIN","title":"Regenicin, Inc."},"7505":{"cik_str":1816906,"ticker":"GMGT","title":"Gaming Technologies, Inc."},"7506":{"cik_str":1171008,"ticker":"ADMG","title":"ADAMANT DRI PROCESSING & MINERALS GROUP"},"7507":{"cik_str":1331421,"ticker":"HLLK","title":"HALLMARK VENTURE GROUP, INC."},"7508":{"cik_str":1552979,"ticker":"CDXQ","title":"China De Xiao Quan Care Group Co., Ltd"},"7509":{"cik_str":1431880,"ticker":"PTAM","title":"POTASH AMERICA, INC."},"7510":{"cik_str":1506295,"ticker":"VIZC","title":"VIZCONNECT, INC."},"7511":{"cik_str":1602813,"ticker":"FRTG","title":"FRONTERA GROUP INC."},"7512":{"cik_str":1681941,"ticker":"IFMK","title":"iFresh Inc"},"7513":{"cik_str":1342916,"ticker":"HNOI","title":"HNO International, Inc."},"7514":{"cik_str":1528308,"ticker":"IINX","title":"IONIX TECHNOLOGY, INC."},"7515":{"cik_str":1017110,"ticker":"GTHR","title":"GENETHERA INC"},"7516":{"cik_str":1483646,"ticker":"BEGI","title":"BLACKSTAR ENTERPRISE GROUP, INC."},"7517":{"cik_str":1683131,"ticker":"FORZ","title":"Forza Innovations Inc"},"7518":{"cik_str":1138724,"ticker":"GAHC","title":"Global Arena Holding, Inc."},"7519":{"cik_str":1659352,"ticker":"CDAKQ","title":"Codiak BioSciences, Inc."},"7520":{"cik_str":1301838,"ticker":"PMPG","title":"Premier Product Group, Inc."},"7521":{"cik_str":1762546,"ticker":"GTRL","title":"Get Real USA, Inc."},"7522":{"cik_str":1715819,"ticker":"EMED","title":"Electromedical Technologies, Inc"},"7523":{"cik_str":1409624,"ticker":"HMLA","title":"HIMALAYA TECHNOLOGIES, INC"},"7524":{"cik_str":772263,"ticker":"NICH","title":"NITCHES INC"},"7525":{"cik_str":894560,"ticker":"BOTH","title":"BIOETHICS LTD"},"7526":{"cik_str":849636,"ticker":"RSPI","title":"RespireRx Pharmaceuticals Inc."},"7527":{"cik_str":1916879,"ticker":"VHLD","title":"VECTOR 21 HOLDINGS, INC."},"7528":{"cik_str":1522690,"ticker":"GWGHQ","title":"GWG Holdings, Inc."},"7529":{"cik_str":1410708,"ticker":"NUVI","title":"Emo Capital Corp."},"7530":{"cik_str":1708410,"ticker":"WINR","title":"SIMPLICITY ESPORTS & GAMING Co"},"7531":{"cik_str":1014111,"ticker":"GLAE","title":"GlassBridge Enterprises, Inc."},"7532":{"cik_str":1697935,"ticker":"MAPT","title":"MAPTELLIGENT, INC."},"7533":{"cik_str":1092802,"ticker":"AURI","title":"AURI INC"},"7534":{"cik_str":1713832,"ticker":"HYREQ","title":"HC LIQUIDATING, INC."},"7535":{"cik_str":1094032,"ticker":"QDMI","title":"QDM International Inc."},"7536":{"cik_str":1510524,"ticker":"GYST","title":"GRAYSTONE COMPANY, INC."},"7537":{"cik_str":790273,"ticker":"CONC","title":"CONECTISYS CORP"},"7538":{"cik_str":1706907,"ticker":"SNMN","title":"SNM Global Holdings"},"7539":{"cik_str":1486452,"ticker":"MASN","title":"Maison Luxe, Inc."},"7540":{"cik_str":1077319,"ticker":"SLCH","title":"Spotlight Capital Holdings, Inc"},"7541":{"cik_str":1528760,"ticker":"EXEO","title":"Exeo Entertainment, Inc."},"7542":{"cik_str":1178660,"ticker":"CSSI","title":"COSTAS INC"},"7543":{"cik_str":1813603,"ticker":"HSTA","title":"Hestia Insight Inc."},"7544":{"cik_str":1530163,"ticker":"SAML","title":"Samsara Luggage, Inc."},"7545":{"cik_str":737207,"ticker":"ATRX","title":"Adhera Therapeutics, Inc."},"7546":{"cik_str":1068689,"ticker":"ATDS","title":"Data443 Risk Mitigation, Inc."},"7547":{"cik_str":1681556,"ticker":"GXXM","title":"GEX MANAGEMENT, INC."},"7548":{"cik_str":1760026,"ticker":"MSSV","title":"MESO NUMISMATICS, INC."},"7549":{"cik_str":836937,"ticker":"UPDC","title":"UPD HOLDING CORP."},"7550":{"cik_str":1494413,"ticker":"TWOH","title":"Two Hands Corp"},"7551":{"cik_str":852447,"ticker":"TPII","title":"Triad Pro Innovators, Inc."},"7552":{"cik_str":1286648,"ticker":"GZIC","title":"GZ6G Technologies Corp."},"7553":{"cik_str":820608,"ticker":"QTXB","title":"QUANTRX BIOMEDICAL CORP"},"7554":{"cik_str":918545,"ticker":"BISA","title":"BALTIC INTERNATIONAL USA INC"},"7555":{"cik_str":1909152,"ticker":"KALRQ","title":"Kalera Public Ltd Co"},"7556":{"cik_str":813716,"ticker":"CIRX","title":"CIRTRAN CORP"},"7557":{"cik_str":1945619,"ticker":"ECGR","title":"Bellatora, Inc."},"7558":{"cik_str":1506814,"ticker":"JPEX","title":"JPX Global Inc."},"7559":{"cik_str":894552,"ticker":"EMDF","title":"E Med Future, Inc."},"7560":{"cik_str":1409999,"ticker":"BBBT","title":"Black Bird Biotech, Inc."},"7561":{"cik_str":1624517,"ticker":"YCRM","title":"Yuenglings Ice Cream Corp"},"7562":{"cik_str":1295961,"ticker":"QREE","title":"QUANTUM ENERGY INC."},"7563":{"cik_str":1082733,"ticker":"VISM","title":"VISIUM TECHNOLOGIES, INC."},"7564":{"cik_str":1230524,"ticker":"CYAP","title":"Cyber Apps World"},"7565":{"cik_str":1451433,"ticker":"NNRX","title":"NUTRANOMICS, INC."},"7566":{"cik_str":1592603,"ticker":"GWSN","title":"Gulf West Security Network, Inc."},"7567":{"cik_str":1463459,"ticker":"EMGE","title":"Emergent Health Corp."},"7568":{"cik_str":1883389,"ticker":"DZGH","title":"Da Zhong Trading Group Holding Co"},"7569":{"cik_str":1464865,"ticker":"ASII","title":"Accredited Solutions, Inc."},"7570":{"cik_str":747435,"ticker":"AHIX","title":"ALUF HOLDINGS, INC."},"7571":{"cik_str":1572384,"ticker":"PRCX","title":"Phoenix Rising Companies"},"7572":{"cik_str":1227282,"ticker":"SPOWF","title":"Strata Power Corp"},"7573":{"cik_str":1456802,"ticker":"CCOB","title":"Century Cobalt Corp."},"7574":{"cik_str":1688126,"ticker":"CRCW","title":"Crypto Co"},"7575":{"cik_str":1916741,"ticker":"LFEV","title":"Life Electric Vehicles Holdings, Inc."},"7576":{"cik_str":1715032,"ticker":"WTII","title":"Water Technologies International,Inc."},"7577":{"cik_str":1404935,"ticker":"THCT","title":"THC Therapeutics, Inc."},"7578":{"cik_str":1168663,"ticker":"ANSU","title":"AMANASU TECHNO HOLDINGS CORP"},"7579":{"cik_str":1504239,"ticker":"PCNT","title":"Point of Care Nano-Technology, Inc."},"7580":{"cik_str":1497230,"ticker":"SMCE","title":"SMC Entertainment, Inc."},"7581":{"cik_str":1438461,"ticker":"ELRE","title":"Yinfu Gold Corp."},"7582":{"cik_str":1492091,"ticker":"ASCK","title":"AUSCRETE Corp"},"7583":{"cik_str":831489,"ticker":"SCRH","title":"SCORES HOLDING CO INC"},"7584":{"cik_str":1497504,"ticker":"PLXPQ","title":"PLx Pharma Winddown Corp."},"7585":{"cik_str":806592,"ticker":"SUWN","title":"SUNWIN STEVIA INTERNATIONAL, INC."},"7586":{"cik_str":1277575,"ticker":"SCGX","title":"SAXON CAPITAL GROUP INC"},"7587":{"cik_str":1096759,"ticker":"SNWR","title":"Sanwire Corp"},"7588":{"cik_str":1641751,"ticker":"BBRW","title":"BrewBilt Manufacturing Inc."},"7589":{"cik_str":1614466,"ticker":"DLCR","title":"Kibush Capital Corp"},"7590":{"cik_str":1776073,"ticker":"CBDL","title":"CBD Life Sciences Inc."},"7591":{"cik_str":1877788,"ticker":"CSTF","title":"CuraScientific Corp."},"7592":{"cik_str":1373467,"ticker":"VTXB","title":"Vortex Brands Co."},"7593":{"cik_str":1295514,"ticker":"MDWK","title":"MDWerks, Inc."},"7594":{"cik_str":1431074,"ticker":"BRGO","title":"Bergio International, Inc."},"7595":{"cik_str":1517498,"ticker":"EWLU","title":"Merion, Inc."},"7596":{"cik_str":1384135,"ticker":"LBTI","title":"Lithium & Boron Technology, Inc."},"7597":{"cik_str":59860,"ticker":"GRMC","title":"GOLDRICH MINING CO"},"7598":{"cik_str":1489588,"ticker":"HBIS","title":"Home Bistro, Inc. /NV/"},"7599":{"cik_str":1569329,"ticker":"YGYI","title":"Youngevity International, Inc."},"7600":{"cik_str":1343009,"ticker":"CNBX","title":"CNBX Pharmaceuticals Inc."},"7601":{"cik_str":1468679,"ticker":"CRYO","title":"American CryoStem Corp"},"7602":{"cik_str":1127993,"ticker":"GAXY","title":"GALAXY NEXT GENERATION, INC."},"7603":{"cik_str":1331275,"ticker":"XCPL","title":"XCPCNL Business Services Corp"},"7604":{"cik_str":1418115,"ticker":"HSMD","title":"Healthcare Solutions Management Group, Inc."},"7605":{"cik_str":1384365,"ticker":"RDAR","title":"RAADR, INC."},"7606":{"cik_str":1456453,"ticker":"WTKN","title":"CLStv Corp."},"7607":{"cik_str":1558465,"ticker":"PGAS","title":"PETROGRESS, INC"},"7608":{"cik_str":1404356,"ticker":"CBIA","title":"Blue Heaven Coffee, Inc."},"7609":{"cik_str":1265521,"ticker":"TRFE","title":"Trustfeed Corp."},"7610":{"cik_str":931059,"ticker":"RNVA","title":"Rennova Health, Inc."},"7611":{"cik_str":1533357,"ticker":"DTII","title":"DEFENSE TECHNOLOGIES INTERNATIONAL CORP."},"7612":{"cik_str":1613895,"ticker":"BMXC","title":"Bemax, Inc."},"7613":{"cik_str":1569340,"ticker":"CLCS","title":"Cell Source, Inc."},"7614":{"cik_str":1419995,"ticker":"DNAX","title":"DNA BRANDS INC"},"7615":{"cik_str":1427644,"ticker":"SLDC","title":"Telco Cuba, Inc."},"7616":{"cik_str":869495,"ticker":"DWOG","title":"DEEP WELL OIL & GAS INC"},"7617":{"cik_str":1828672,"ticker":"BOXDQ","title":"Boxed, Inc."},"7618":{"cik_str":1579010,"ticker":"LFER","title":"Life On Earth, Inc."},"7619":{"cik_str":1863976,"ticker":"AQUI","title":"Aquarius I Acquisition Corp."},"7620":{"cik_str":786947,"ticker":"ACUR","title":"ACURA PHARMACEUTICALS, INC"},"7621":{"cik_str":1069996,"ticker":"FNHCQ","title":"FedNat Holding Co"},"7622":{"cik_str":1379043,"ticker":"XFCI","title":"XTREME FIGHTING CHAMPIONSHIPS, INC."},"7623":{"cik_str":1169245,"ticker":"PHASQ","title":"PhaseBio Pharmaceuticals Inc"},"7624":{"cik_str":1552358,"ticker":"CGSI","title":"CGS INTERNATIONAL, INC."},"7625":{"cik_str":1765826,"ticker":"BZRD","title":"Blubuzzard, Inc."},"7626":{"cik_str":1563227,"ticker":"SPUP","title":"Sipup Corp"},"7627":{"cik_str":1598308,"ticker":"GWHP","title":"Global Wholehealth Partners Corp"},"7628":{"cik_str":1284450,"ticker":"CHGI","title":"China Carbon Graphite Group, Inc."},"7629":{"cik_str":1624658,"ticker":"ALNAQ","title":"Allena Pharmaceuticals, Inc."},"7630":{"cik_str":1622231,"ticker":"BTIM","title":"BOATIM INC."},"7631":{"cik_str":1104265,"ticker":"GRSO","title":"GROW SOLUTIONS HOLDINGS, INC."},"7632":{"cik_str":1353499,"ticker":"MAXD","title":"Max Sound Corp"},"7633":{"cik_str":1484703,"ticker":"SFIO","title":"Starfleet Innotech, Inc."},"7634":{"cik_str":1399306,"ticker":"BRBL","title":"BrewBilt Brewing Co"},"7635":{"cik_str":1622408,"ticker":"FRFR","title":"Fritzy Tech Inc."},"7636":{"cik_str":1370816,"ticker":"FBCD","title":"FBC Holding, Inc."},"7637":{"cik_str":1108046,"ticker":"AFPW","title":"AlumiFuel Power Corp"},"7638":{"cik_str":1384939,"ticker":"BNCM","title":"BOUNCE MOBILE SYSTEMS, INC."},"7639":{"cik_str":1880419,"ticker":"FERN","title":"Fernhill Corp"},"7640":{"cik_str":892832,"ticker":"SDON","title":"SANDSTON CORP"},"7641":{"cik_str":1393548,"ticker":"CLIS","title":"ClickStream Corp"},"7642":{"cik_str":1734902,"ticker":"WEIDY","title":"Weidai Ltd."},"7643":{"cik_str":1782627,"ticker":"WBEVQ","title":"Winc, Inc."},"7644":{"cik_str":1580262,"ticker":"RTON","title":"Right On Brands, Inc."},"7645":{"cik_str":1499855,"ticker":"GTOR","title":"GGTOOR, INC."},"7646":{"cik_str":878726,"ticker":"TUEMQ","title":"TUESDAY MORNING CORP/DE"},"7647":{"cik_str":1833908,"ticker":"ALFIQ","title":"Alfi, Inc."},"7648":{"cik_str":1499684,"ticker":"QIAN","title":"Qiansui International Group Co. Ltd."},"7649":{"cik_str":1729089,"ticker":"PHCFF","title":"Puhui Wealth Investment Management Co., Ltd."},"7650":{"cik_str":884380,"ticker":"KEGS","title":"1812 Brewing Company, Inc."},"7651":{"cik_str":1758736,"ticker":"MKDTY","title":"Molecular Data Inc."},"7652":{"cik_str":81157,"ticker":"PGAI","title":"PGI INC"},"7653":{"cik_str":1280396,"ticker":"VTNA","title":"VetaNova Inc."},"7654":{"cik_str":723733,"ticker":"MUSS","title":"MULTI SOLUTIONS II, INC"},"7655":{"cik_str":1368620,"ticker":"CNNA","title":"Cann American Corp."},"7656":{"cik_str":1286459,"ticker":"AFOM","title":"ALL FOR ONE MEDIA CORP."},"7657":{"cik_str":1929281,"ticker":"BLFE","title":"BioLIfe Sciences Inc"},"7658":{"cik_str":1886958,"ticker":"YUKA","title":"Yuka Group Inc"},"7659":{"cik_str":1873504,"ticker":"NWAC","title":"Namaste World Acquisition Corp"},"7660":{"cik_str":1942808,"ticker":"EGOX","title":"Next.e.GO B.V."},"7661":{"cik_str":1934245,"ticker":"SPGC","title":"Sacks Parente Golf, Inc."},"7662":{"cik_str":1957132,"ticker":"SN","title":"SharkNinja, Inc."},"7663":{"cik_str":1868079,"ticker":"PMED","title":"Predictmedix Inc."},"7664":{"cik_str":1936224,"ticker":"SRFM","title":"SURF AIR MOBILITY INC."},"7665":{"cik_str":1951089,"ticker":"CRML","title":"Critical Metals Corp."},"7666":{"cik_str":1946025,"ticker":"SUNH","title":"Xuhang Holdings Ltd"},"7667":{"cik_str":1948099,"ticker":"MTEN","title":"Mingteng International Corp Inc."},"7668":{"cik_str":820318,"ticker":"COHR","title":"COHERENT CORP."},"7669":{"cik_str":1503658,"ticker":"LVDW","title":"LiquidValue Development Inc."},"7670":{"cik_str":1558924,"ticker":"TNT","title":"Tenet Fintech Group Inc."},"7671":{"cik_str":1405513,"ticker":"UNL","title":"United States 12 Month Natural Gas Fund, LP"},"7672":{"cik_str":1567264,"ticker":"INTS","title":"INTENSITY THERAPEUTICS, INC."},"7673":{"cik_str":1671132,"ticker":"SONG","title":"Music Licensing Inc."},"7674":{"cik_str":1869901,"ticker":"ZTOP","title":"Zi Toprun Acquisition Corp."},"7675":{"cik_str":1956055,"ticker":"BUJA","title":"Bukit Jalil Global Acquisition 1 Ltd."},"7676":{"cik_str":1907085,"ticker":"ODD","title":"Oddity Tech Ltd"},"7677":{"cik_str":1837824,"ticker":"SESG","title":"Sprott ESG Gold ETF"},"7678":{"cik_str":1340909,"ticker":"GJO","title":"STRATS SM TRUST FOR WAL-MART STORES, INC. SECURITIES, SERIES 2005-4"},"7679":{"cik_str":1690996,"ticker":"CNRLX","title":"City National Rochdale Select Strategies Fund"},"7680":{"cik_str":1450922,"ticker":"SIVR","title":"abrdn Silver ETF Trust"},"7681":{"cik_str":1267332,"ticker":"IPB","title":"MERRILL LYNCH DEPOSITOR INC INDEXPLUS TRUST SERIES 2003-1"},"7682":{"cik_str":1552164,"ticker":"NREG","title":"Springs Rejuvenation, Inc."},"7683":{"cik_str":1804189,"ticker":"THNK","title":"T1V, Inc."},"7684":{"cik_str":1826660,"ticker":"WETH","title":"Wetouch Technology Inc."},"7685":{"cik_str":1898722,"ticker":"MDRN","title":"Modern Mining Technology Corp."},"7686":{"cik_str":1663712,"ticker":"NRSAX","title":"NEXPOINT REAL ESTATE STRATEGIES FUND"},"7687":{"cik_str":1851651,"ticker":"BRNI","title":"Beroni Group Ltd"},"7688":{"cik_str":1888980,"ticker":"LRE","title":"LEAD REAL ESTATE CO., LTD"},"7689":{"cik_str":1887799,"ticker":"LIAI","title":"LEMENG HOLDINGS LTD"},"7690":{"cik_str":1759124,"ticker":"IAUM","title":"iShares Gold Trust Micro"},"7691":{"cik_str":1675426,"ticker":"SMGE","title":"Sigmata Electronics, Inc."},"7692":{"cik_str":1904958,"ticker":"TDAI","title":"Thornburg Durable Allocation & Income Trust"},"7693":{"cik_str":1843974,"ticker":"DXYZ","title":"Destiny Tech100 Inc."},"7694":{"cik_str":1884434,"ticker":"PGRM","title":"Panagram Capital, LLC"},"7695":{"cik_str":1929783,"ticker":"WOK","title":"WORK Medical Technology Group LTD"},"7696":{"cik_str":1943444,"ticker":"TRSG","title":"Tungray Technologies Inc"},"7697":{"cik_str":1838831,"ticker":"DISAW","title":"Disruptive Acquisition Corp I"},"7698":{"cik_str":1686515,"ticker":"EAWC","title":"Ecco Auto World Corp"},"7699":{"cik_str":1585389,"ticker":"SMST","title":"SmartStop Self Storage REIT, Inc."},"7700":{"cik_str":1545772,"ticker":"BPYPM","title":"Brookfield Property Partners L.P."},"7701":{"cik_str":1642563,"ticker":"XGEIX","title":"Guggenheim Energy & Income Fund"},"7702":{"cik_str":1787123,"ticker":"LQLY","title":"QLY Biotech Group Corp."},"7703":{"cik_str":1447100,"ticker":"YAMHY","title":"Yamaha Motor Co., Ltd."},"7704":{"cik_str":1631256,"ticker":"KBSG","title":"KBS Growth & Income REIT, Inc."},"7705":{"cik_str":1343491,"ticker":"GJP","title":"STRATS(SM) TRUST FOR DOMINION RESOURCES, INC. SECURITIES, SERIES 2005-6"},"7706":{"cik_str":1483386,"ticker":"GLTR","title":"abrdn Precious Metals Basket ETF Trust"},"7707":{"cik_str":1573901,"ticker":"OLSI","title":"Origin Life Sciences, Inc."},"7708":{"cik_str":1357660,"ticker":"GJT","title":"STRATS(SM) Trust for Allstate Corp Securities, Series 2006-3"},"7709":{"cik_str":1348952,"ticker":"ELC","title":"ENTERGY LOUISIANA, LLC"},"7710":{"cik_str":1011060,"ticker":"NORD","title":"Nordicus Partners Corp"},"7711":{"cik_str":926865,"ticker":"CECL","title":"CECIL BANCORP INC"},"7712":{"cik_str":1681717,"ticker":"VFLEX","title":"FIRST TRUST ALTERNATIVE OPPORTUNITIES FUND"},"7713":{"cik_str":1873723,"ticker":"HARD","title":"Harden Technologies Inc."},"7714":{"cik_str":1935092,"ticker":"CETI","title":"Cyber Enviro-Tech, Inc."},"7715":{"cik_str":1974952,"ticker":"AMGS","title":"Prospect Energy Holdings Corp."},"7716":{"cik_str":1892292,"ticker":"MSS","title":"Maison Solutions Inc."},"7717":{"cik_str":1959224,"ticker":"SWIN","title":"Solowin Holdings, Ltd."},"7718":{"cik_str":1911545,"ticker":"HRYU","title":"Hanryu Holdings, Inc."},"7719":{"cik_str":1867506,"ticker":"DDCIU","title":"Diffuse Digital 30, LP"},"7720":{"cik_str":1975218,"ticker":"NETD","title":"Nabors Energy Transition Corp. II"},"7721":{"cik_str":1973266,"ticker":"TKO","title":"New Whale Inc."},"7722":{"cik_str":1958489,"ticker":"CRGT","title":"Cortigent, Inc."},"7723":{"cik_str":1924482,"ticker":"IBG","title":"Innovation Beverage Group Ltd"},"7724":{"cik_str":1707079,"ticker":"EVTS","title":"EV TRANSPORTATION SERVICES, INC."},"7725":{"cik_str":1912884,"ticker":"RPGL","title":"Republic Power Group Ltd"},"7726":{"cik_str":1945240,"ticker":"ROMA","title":"Roma Green Finance Ltd"},"7727":{"cik_str":1753391,"ticker":"QSJC","title":"TANCHENG GROUP CO., LTD."},"7728":{"cik_str":1965044,"ticker":"FLXG","title":"Flexi Group Holdings Ltd"},"7729":{"cik_str":1807616,"ticker":"HIGR","title":"Hi-Great Group Holding Co"},"7730":{"cik_str":1805526,"ticker":"JNRV","title":"Janover Inc."},"7731":{"cik_str":1875016,"ticker":"YMAT","title":"J-Star Holding Co., Ltd."},"7732":{"cik_str":1944399,"ticker":"ELWS","title":"Earlyworks Co., Ltd."},"7733":{"cik_str":1953984,"ticker":"BCG","title":"Binah Capital Group, Inc."},"7734":{"cik_str":1973368,"ticker":"SVMH","title":"SRIVARU Holding Ltd"},"7735":{"cik_str":1668523,"ticker":"GSGG","title":"GSG GROUP INC."},"7736":{"cik_str":1930207,"ticker":"QTM","title":"CH AUTO Inc."},"7737":{"cik_str":1608016,"ticker":"PSOIX","title":"Palmer Square Opportunistic Income Fund"},"7738":{"cik_str":1848669,"ticker":"SWGC","title":"Stillwater Growth Corp. I"},"7739":{"cik_str":1883631,"ticker":"FOGO","title":"Fogo Hospitality, Inc."},"7740":{"cik_str":1968915,"ticker":"PHIN","title":"PHINIA INC."},"7741":{"cik_str":1806905,"ticker":"ALEH","title":"ALE Group Holding Ltd"},"7742":{"cik_str":1673362,"ticker":"CMCZ","title":"Curtis Mathes Corp"},"7743":{"cik_str":1951276,"ticker":"SRBK","title":"SR Bancorp, Inc."},"7744":{"cik_str":1891944,"ticker":"PMEC","title":"Primech Holdings Ltd"},"7745":{"cik_str":1933951,"ticker":"SAG","title":"SAG Holdings Ltd"},"7746":{"cik_str":1866390,"ticker":"BBLNF","title":"Babylon Holdings Ltd"},"7747":{"cik_str":1962845,"ticker":"PXDT","title":"PIXIE DUST TECHNOLOGIES, INC."},"7748":{"cik_str":1932777,"ticker":"PROQ","title":"PROTONIQ Acquisition Corp"},"7749":{"cik_str":1604477,"ticker":"SQZB","title":"SQZ Biotechnologies Co"},"7750":{"cik_str":1716885,"ticker":"ASCIX","title":"Angel Oak Strategic Credit Fund"},"7751":{"cik_str":1861785,"ticker":"SOSH","title":"SOS Hydration Inc."},"7752":{"cik_str":1916416,"ticker":"INTJ","title":"Intelligent Group Ltd"},"7753":{"cik_str":1762562,"ticker":"CPRDX","title":"Clarion Partners Real Estate Income Fund Inc."},"7754":{"cik_str":1857971,"ticker":"BTRY","title":"Clarios International Inc."},"7755":{"cik_str":1902930,"ticker":"PDPG","title":"Performance Drink Group, Inc."},"7756":{"cik_str":1944977,"ticker":"HLST","title":"Holisto Ltd."},"7757":{"cik_str":1887389,"ticker":"ALEF","title":"Aleph Group, Inc"},"7758":{"cik_str":1914513,"ticker":"RISE","title":"Rise Oil & Gas, Inc."},"7759":{"cik_str":1916902,"ticker":"BGAC","title":"Biotech Group Acquisition Corp"},"7760":{"cik_str":1964630,"ticker":"VSTE","title":"Vast Solar Pty Ltd"},"7761":{"cik_str":1938109,"ticker":"PAPL","title":"Pineapple Financial Inc."},"7762":{"cik_str":1897532,"ticker":"MJID","title":"Majestic Ideal Holdings Ltd"},"7763":{"cik_str":1771755,"ticker":"SKUR","title":"Sekur Private Data Ltd."},"7764":{"cik_str":1133519,"ticker":"MPH","title":"MEDICURE INC"},"7765":{"cik_str":1926451,"ticker":"TWEN","title":"T20 HOLDINGS LTD."},"7766":{"cik_str":1964657,"ticker":"LBIO","title":"Denali SPAC Holdco, Inc."},"7767":{"cik_str":1868395,"ticker":"YIBO","title":"Planet Image International Ltd"},"7768":{"cik_str":1835068,"ticker":"NPCT","title":"Nuveen Core Plus Impact Fund"},"7769":{"cik_str":1941158,"ticker":"WETO","title":"Webus International Ltd."},"7770":{"cik_str":1940322,"ticker":"MDLB","title":"Medlab Clinical Ltd."},"7771":{"cik_str":1940177,"ticker":"PODC","title":"Courtside Group, Inc."},"7772":{"cik_str":1904286,"ticker":"MIRA","title":"MIRA PHARMACEUTICALS, INC."},"7773":{"cik_str":1896425,"ticker":"LBGJ","title":"Li Bang International Corp Inc."},"7774":{"cik_str":1917890,"ticker":"PSIG","title":"PSI Group Holdings Ltd."},"7775":{"cik_str":1936574,"ticker":"IMSV","title":"IMMRSIV Inc."},"7776":{"cik_str":1888780,"ticker":"LEWY","title":"LEEWAY SERVICES, INC."},"7777":{"cik_str":1954694,"ticker":"LGCL","title":"Lucas GC Ltd"},"7778":{"cik_str":1941029,"ticker":"ADVB","title":"Advanced Biomed Inc."},"7779":{"cik_str":1673504,"ticker":"AGLY","title":"Atlantis Glory Inc."},"7780":{"cik_str":1927309,"ticker":"ALKD","title":"Alkaid Acquisition Corp."},"7781":{"cik_str":1975723,"ticker":"STA","title":"HWEL Holdings Corp."},"7782":{"cik_str":1960074,"ticker":"DVGR","title":"Digital Virgo Group S.A."},"7783":{"cik_str":1850767,"ticker":"ACMSY","title":"Accustem Sciences Inc."},"7784":{"cik_str":1803487,"ticker":"SCIT","title":"Sancai Holding Group Ltd."},"7785":{"cik_str":1670196,"ticker":"DSWR","title":"Deseo Swimwear Inc."},"7786":{"cik_str":1793895,"ticker":"CDTG","title":"CDT Environmental Technology Investment Holdings Ltd"},"7787":{"cik_str":1610718,"ticker":"ODRS","title":"Outdoor Specialty Products, Inc."},"7788":{"cik_str":1936031,"ticker":"LEC","title":"LAFAYETTE ENERGY CORP."},"7789":{"cik_str":1771951,"ticker":"WEIX","title":"Dynamic Shares Trust"},"7790":{"cik_str":1094831,"ticker":"BGC","title":"BGC Partners, Inc."},"7791":{"cik_str":1708646,"ticker":"AAAU","title":"Goldman Sachs Physical Gold ETF"},"7792":{"cik_str":1896677,"ticker":"GSOL","title":"Grayscale Solana Trust (SOL)"},"7793":{"cik_str":1947971,"ticker":"DRFY","title":"Droneify Holdings Ltd"},"7794":{"cik_str":1947654,"ticker":"NTV","title":"Neotv Group Ltd"},"7795":{"cik_str":1725210,"ticker":"ETHE","title":"Grayscale Ethereum Trust (ETH)"},"7796":{"cik_str":1645108,"ticker":"MDLS","title":"MDNA Life Sciences, Inc."},"7797":{"cik_str":1843165,"ticker":"LQR","title":"LQR House Inc."},"7798":{"cik_str":1396878,"ticker":"UGA","title":"United States Gasoline Fund, LP"},"7799":{"cik_str":1829311,"ticker":"BMNR","title":"BITMINE IMMERSION TECHNOLOGIES, INC."},"7800":{"cik_str":1956744,"ticker":"SRM","title":"SRM Entertainment, Inc."},"7801":{"cik_str":1403528,"ticker":"OAK-PA","title":"Oaktree Capital Group, LLC"},"7802":{"cik_str":1450923,"ticker":"SGOL","title":"abrdn Gold ETF Trust"},"7803":{"cik_str":1392449,"ticker":"GPLB","title":"Green Planet Bio Engineering Co. Ltd."},"7804":{"cik_str":1896511,"ticker":"ZKGCF","title":"ZKGC New Energy Ltd"},"7805":{"cik_str":1956410,"ticker":"JPO","title":"JP Outfitters, Inc."},"7806":{"cik_str":1767057,"ticker":"OBTC","title":"Osprey Bitcoin Trust"},"7807":{"cik_str":1920092,"ticker":"GELS","title":"Gelteq Ltd"},"7808":{"cik_str":1937692,"ticker":"GFCX","title":"GoodFaith Technology Inc."},"7809":{"cik_str":1790320,"ticker":"MSTH","title":"Mystic Holdings Inc./NV"},"7810":{"cik_str":1927596,"ticker":"WPA","title":"Whale Point Acquisition Corp."},"7811":{"cik_str":1947250,"ticker":"GDLT","title":"Greifenberg Digital Ltd"},"7812":{"cik_str":1841931,"ticker":"RPET","title":"New Ruipeng Pet Group Inc."},"7813":{"cik_str":1434389,"ticker":"UNSS","title":"UNIVERSAL SOLAR TECHNOLOGY, INC."},"7814":{"cik_str":1751156,"ticker":"EIOAX","title":"ELLINGTON INCOME OPPORTUNITIES FUND"},"7815":{"cik_str":1956552,"ticker":"NISM","title":"Newsight Imaging Ltd."},"7816":{"cik_str":1687898,"ticker":"VCRRX","title":"Versus Capital Real Assets Fund LLC"},"7817":{"cik_str":1983324,"ticker":"RMSG","title":"Real Messenger Corp"},"7818":{"cik_str":1948697,"ticker":"SPPL","title":"SIMPPLE LTD."},"7819":{"cik_str":1885840,"ticker":"XYGJ","title":"Fortune Joy International Acquisition Corp"},"7820":{"cik_str":1568905,"ticker":"PRDEX","title":"PREDEX"},"7821":{"cik_str":1481585,"ticker":"GXXY","title":"Galexxy Holdings, Inc."},"7822":{"cik_str":1963088,"ticker":"ATCH","title":"Calculator New Pubco, Inc."},"7823":{"cik_str":1932470,"ticker":"AGII","title":"AgiiPlus Inc."},"7824":{"cik_str":1959023,"ticker":"SGD","title":"Safe & Green Development Corp"},"7825":{"cik_str":1900564,"ticker":"WRNT","title":"WARRANTEE INC."},"7826":{"cik_str":1810200,"ticker":"ONS","title":"ONS Acquisition Corp."},"7827":{"cik_str":1922335,"ticker":"SYRA","title":"Syra Health Corp"},"7828":{"cik_str":1922639,"ticker":"OILS","title":"Permex Petroleum Corp"},"7829":{"cik_str":1753373,"ticker":"MTWO","title":"M2I Global, Inc."},"7830":{"cik_str":1927578,"ticker":"NTCL","title":"NetClass Technology Inc"},"7831":{"cik_str":1843573,"ticker":"UMAV","title":"UAV Corp"},"7832":{"cik_str":1946412,"ticker":"ECLP","title":"Eclipse Bancorp, Inc."},"7833":{"cik_str":1900720,"ticker":"EPWK","title":"EPWK Holdings Ltd."},"7834":{"cik_str":1901799,"ticker":"BTM","title":"Bitcoin Depot Inc."},"7835":{"cik_str":1353226,"ticker":"GJR","title":"STRATS(SM) Trust for Procter & Gamble Securities, Series 2006-1"},"7836":{"cik_str":1427437,"ticker":"ETI-P","title":"ENTERGY TEXAS, INC."},"7837":{"cik_str":1062128,"ticker":"MNKA","title":"Manuka, Inc."},"7838":{"cik_str":1124959,"ticker":"XSIAX","title":"VOYA CREDIT INCOME FUND"},"7839":{"cik_str":1888577,"ticker":"UERI","title":"UE Resorts International, Inc."},"7840":{"cik_str":1688554,"ticker":"PFFLX","title":"PIMCO Flexible Credit Income Fund"},"7841":{"cik_str":1765850,"ticker":"PWM","title":"Prestige Wealth Inc."},"7842":{"cik_str":1620704,"ticker":"MXRX","title":"MED-X, INC."},"7843":{"cik_str":1662991,"ticker":"SZL","title":"Sezzle Inc."},"7844":{"cik_str":1375348,"ticker":"GLNS","title":"Golden Star Resource Corp."},"7845":{"cik_str":1592651,"ticker":"LNZNY","title":"Lenzing AG/ADR"},"7846":{"cik_str":1472494,"ticker":"BNO","title":"United States Brent Oil Fund, LP"},"7847":{"cik_str":1254348,"ticker":"EMTX","title":"EMULATE THERAPEUTICS, INC."},"7848":{"cik_str":1283337,"ticker":"KTN","title":"STRUCTURED PRODUCTS CORP CRED ENHANCE CORTS TR FOR AON CAP A"},"7849":{"cik_str":846546,"ticker":"BNSOF","title":"BONSO ELECTRONICS INTERNATIONAL INC"},"7850":{"cik_str":1736510,"ticker":"NICHX","title":"Variant Alternative Income Fund"},"7851":{"cik_str":1873454,"ticker":"RYET","title":"Ruanyun Edai Technology Inc."},"7852":{"cik_str":1928581,"ticker":"FTEL","title":"Fitell Corp"},"7853":{"cik_str":1838406,"ticker":"BKV","title":"BKV Corp"},"7854":{"cik_str":1356284,"ticker":"GJS","title":"STRATS(SM) Trust for Goldman Sachs Group Securities, Series 2006-2"},"7855":{"cik_str":1353612,"ticker":"FXC","title":"Invesco CurrencyShares Canadian Dollar Trust"},"7856":{"cik_str":1052354,"ticker":"MADL","title":"MAN AHL DIVERSIFIED I LP"},"7857":{"cik_str":1624422,"ticker":"PHXM","title":"PHAXIAM Therapeutics S.A."},"7858":{"cik_str":1958713,"ticker":"MI","title":"NFT Ltd"},"7859":{"cik_str":1947756,"ticker":"MERQ","title":"Merqueo Holdings"},"7860":{"cik_str":1639953,"ticker":"SAGN","title":"Sagoon Inc."},"7861":{"cik_str":1383151,"ticker":"UUP","title":"Invesco DB US Dollar Index Bullish Fund"},"7862":{"cik_str":1496254,"ticker":"LTAFX","title":"Alternative Strategies Fund"},"7863":{"cik_str":1561032,"ticker":"HTIA","title":"Healthcare Trust, Inc."},"7864":{"cik_str":1674356,"ticker":"TPTA","title":"Terra Property Trust, Inc."},"7865":{"cik_str":1952202,"ticker":"JR","title":"JINRONG HOLDINGS LTD."},"7866":{"cik_str":1949478,"ticker":"DTCK","title":"DAVIS COMMODITIES Ltd"},"7867":{"cik_str":1717976,"ticker":"WEIB","title":"Weiss Strategic Interval Fund"},"7868":{"cik_str":1905951,"ticker":"BSME","title":"MED EIBY Holding Co., Ltd"},"7869":{"cik_str":1879754,"ticker":"EHGO","title":"EShallGo Inc."},"7870":{"cik_str":1937987,"ticker":"FBYD","title":"Falcon's Beyond Global, Inc."},"7871":{"cik_str":1873529,"ticker":"KLC","title":"KinderCare Learning Companies, Inc."},"7872":{"cik_str":1960081,"ticker":"XJET","title":"XJet Ltd."},"7873":{"cik_str":1893124,"ticker":"ZAAG","title":"ZA Group, Inc."},"7874":{"cik_str":1956955,"ticker":"UMAC","title":"Unusual Machines, Inc."},"7875":{"cik_str":1880431,"ticker":"VOCO","title":"Vocodia Holdings Corp"},"7876":{"cik_str":1900035,"ticker":"FP","title":"First Person Ltd."},"7877":{"cik_str":1764974,"ticker":"TSBX","title":"Turnstone Biologics Corp."},"7878":{"cik_str":1728162,"ticker":"IAMR","title":"Medical Industries of the Americas"},"7879":{"cik_str":1939365,"ticker":"IVP","title":"INSPIRE VETERINARY PARTNERS, INC."},"7880":{"cik_str":1918661,"ticker":"ESHA","title":"ESH Acquisition Corp."},"7881":{"cik_str":1827620,"ticker":"SLDX","title":"Stella Diagnostics, Inc."},"7882":{"cik_str":1282496,"ticker":"TDRK","title":"TIDEROCK COMPANIES, INC."},"7883":{"cik_str":1283464,"ticker":"KTH","title":"STRUCTURED PRODUCTS CORP CORTS TR FOR PECO ENERGY CAP TR III"},"7884":{"cik_str":1400118,"ticker":"SGMT","title":"Sagimet Biosciences Inc."},"7885":{"cik_str":1702015,"ticker":"AIDG","title":"AIS Holdings Group, Inc."},"7886":{"cik_str":1577134,"ticker":"TFSA","title":"Terra Income Fund 6, LLC"},"7887":{"cik_str":1688897,"ticker":"FCREX","title":"FS Credit Income Fund"},"7888":{"cik_str":1086313,"ticker":"SFLM","title":"SFLMaven Corp."},"7889":{"cik_str":1283789,"ticker":"RYNL","title":"Reynaldo's Mexican Food Company, Inc."},"7890":{"cik_str":1459862,"ticker":"PALL","title":"abrdn Palladium ETF Trust"},"7891":{"cik_str":1754927,"ticker":"AAIDX","title":"Axonic Alternative Income Fund"},"7892":{"cik_str":1880423,"ticker":"SEQP","title":"Santana Equestrian Private Financial, LLC"},"7893":{"cik_str":1919847,"ticker":"SFWJ","title":"Software Effective Solutions, Corp."},"7894":{"cik_str":1910053,"ticker":"MDRI","title":"Midori Group Inc."},"7895":{"cik_str":1974640,"ticker":"APGE","title":"Apogee Therapeutics, Inc."},"7896":{"cik_str":1570132,"ticker":"ANVI","title":"ANVI GLOBAL HOLDINGS, INC."},"7897":{"cik_str":1669368,"ticker":"ASAV","title":"ASI Aviation, Inc."},"7898":{"cik_str":1933567,"ticker":"NRXS","title":"Neuraxis, INC"},"7899":{"cik_str":1913715,"ticker":"PSE","title":"Prime Skyline Ltd"},"7900":{"cik_str":1720265,"ticker":"ZCSH","title":"Grayscale Zcash Trust (ZEC)"},"7901":{"cik_str":1734237,"ticker":"GRAZ","title":"Graze, Inc."},"7902":{"cik_str":1946703,"ticker":"WBUY","title":"WEBUY GLOBAL LTD"},"7903":{"cik_str":1951667,"ticker":"CTNT","title":"CHEETAH NET SUPPLY CHAIN SERVICE INC."},"7904":{"cik_str":1905448,"ticker":"WXT","title":"Wuxin Technology Holdings, Inc."},"7905":{"cik_str":1844450,"ticker":"ELVA","title":"Electrovaya Inc."},"7906":{"cik_str":1943661,"ticker":"CVTO","title":"Covalto Ltd."},"7907":{"cik_str":1792799,"ticker":"QHI","title":"QINHONG INTERNATIONAL GROUP"},"7908":{"cik_str":1966734,"ticker":"AITR","title":"AI Transportation Acquisition Corp"},"7909":{"cik_str":1748945,"ticker":"HZEN","title":"Grayscale Horizen Trust (ZEN)"},"7910":{"cik_str":1908233,"ticker":"MNX","title":"MN8 Energy, Inc."},"7911":{"cik_str":1831523,"ticker":"SFCO","title":"Southern Financial Corp"},"7912":{"cik_str":1891367,"ticker":"RVR","title":"REV Renewables, Inc."},"7913":{"cik_str":1946485,"ticker":"FPAQ","title":"FPA ENERGY ACQUISITION CORP."},"7914":{"cik_str":1725295,"ticker":"TSIFX","title":"Ecofin Tax-Exempt Private Credit Fund, Inc."},"7915":{"cik_str":1874907,"ticker":"DEFG","title":"Grayscale Decentralized Finance (DeFi) Fund LLC"},"7916":{"cik_str":1909142,"ticker":"BOUW","title":"Boustead Wavefront Inc."},"7917":{"cik_str":1879403,"ticker":"LRHC","title":"La Rosa Holdings Corp."},"7918":{"cik_str":1958217,"ticker":"LZM","title":"Lifezone Metals Ltd"},"7919":{"cik_str":1807689,"ticker":"FCCI","title":"FAST CASUAL CONCEPTS, INC."},"7920":{"cik_str":1791929,"ticker":"DPUI","title":"DISCOUNT PRINT USA, INC."},"7921":{"cik_str":1294808,"ticker":"PYT","title":"PPLUS Trust Series GSC-2"},"7922":{"cik_str":823277,"ticker":"CHSCP","title":"CHS INC"},"7923":{"cik_str":70502,"ticker":"NRUC","title":"NATIONAL RURAL UTILITIES COOPERATIVE FINANCE CORP /DC/"},"7924":{"cik_str":1714919,"ticker":"SAVU","title":"BioLife4D Corp"},"7925":{"cik_str":1821534,"ticker":"EXOD","title":"Exodus Movement, Inc."},"7926":{"cik_str":1381871,"ticker":"BOMO","title":"bowmo, Inc."},"7927":{"cik_str":1353611,"ticker":"FXB","title":"Invesco CurrencyShares British Pound Sterling Trust"},"7928":{"cik_str":1353614,"ticker":"FXA","title":"Invesco CurrencyShares Australian Dollar Trust"},"7929":{"cik_str":1284143,"ticker":"JBK","title":"LEHMAN ABS CORP GOLDMAN SACHS CAP 1 SEC BACKED SER 2004-6"},"7930":{"cik_str":1581005,"ticker":"SRRIX","title":"Stone Ridge Trust II"},"7931":{"cik_str":1535665,"ticker":"RBNK","title":"RiverBank Holding Co"},"7932":{"cik_str":1616037,"ticker":"XILSX","title":"Pioneer ILS Interval Fund"},"7933":{"cik_str":1690437,"ticker":"BAR","title":"GraniteShares Gold Trust"},"7934":{"cik_str":1814287,"ticker":"ABL","title":"East Resources Acquisition Co"},"7935":{"cik_str":1863974,"ticker":"AQUB","title":"Aquarius II Acquisition Corp."},"7936":{"cik_str":1954269,"ticker":"WIN","title":"Garden Stage Ltd"},"7937":{"cik_str":1858258,"ticker":"FGDL","title":"Franklin Templeton Holdings Trust"},"7938":{"cik_str":1831833,"ticker":"BDS","title":"Building DreamStar Technology Inc."},"7939":{"cik_str":1964246,"ticker":"ZBJ","title":"ZBJ FINANCE GROUP LTD"},"7940":{"cik_str":1732078,"ticker":"FROPX","title":"Flat Rock Opportunity Fund"},"7941":{"cik_str":1888525,"ticker":"UBXG","title":"U-BX Technology Ltd."},"7942":{"cik_str":1948370,"ticker":"PMC","title":"PIMCO Municipal Credit Income Fund"},"7943":{"cik_str":1964801,"ticker":"RUSA","title":"IWAC HOLDINGS INC."},"7944":{"cik_str":1861733,"ticker":"JJOC","title":"JJ Opportunity Corp."},"7945":{"cik_str":1577351,"ticker":"XYLB","title":"XY Labs, Inc."},"7946":{"cik_str":1696025,"ticker":"KCRD","title":"Kindcard, Inc."},"7947":{"cik_str":1723701,"ticker":"PMFAX","title":"PIMCO Flexible Municipal Income Fund"},"7948":{"cik_str":1902700,"ticker":"PGFF","title":"Pioneer Green Farms, Inc."},"7949":{"cik_str":1919246,"ticker":"CHRO","title":"Chromocell Therapeutics Corp"},"7950":{"cik_str":1946563,"ticker":"SXTP","title":"60 DEGREES PHARMACEUTICALS, INC."},"7951":{"cik_str":1967078,"ticker":"DAZS","title":"DA AI ZHI SHUI INTERNATIONAL HOLDING GROUP LTD"},"7952":{"cik_str":1969373,"ticker":"VTMX","title":"Vesta Real Estate Corporation, S.A.B. de C.V."},"7953":{"cik_str":1877332,"ticker":"NEAT","title":"Noble Education Acquisition Corp."},"7954":{"cik_str":1959304,"ticker":"BZFL","title":"BOZHI FANGLUE INTERNATIONAL INVESTMENT GROUP CO LTD"},"7955":{"cik_str":1872525,"ticker":"SWAG","title":"Stran & Company, Inc."},"7956":{"cik_str":1718271,"ticker":"PMTM","title":"Prometheum, Inc."},"7957":{"cik_str":1885493,"ticker":"ELGP","title":"Elate Group, Inc."},"7958":{"cik_str":1679379,"ticker":"NYX","title":"NYIAX, INC."},"7959":{"cik_str":1254699,"ticker":"QVCD","title":"QVC INC"},"7960":{"cik_str":1489300,"ticker":"ZLME","title":"Zhanling International Ltd"},"7961":{"cik_str":1467631,"ticker":"XCAPX","title":"ACAP Strategic Fund"},"7962":{"cik_str":1383149,"ticker":"UDN","title":"INVESCO DB US DOLLAR INDEX BEARISH FUND"},"7963":{"cik_str":1659207,"ticker":"FLLZ","title":"Fellazo Corp"},"7964":{"cik_str":1682662,"ticker":"XPASX","title":"Peachtree Alternative Strategies Fund"},"7965":{"cik_str":1691570,"ticker":"USQIX","title":"USQ Core Real Estate Fund"},"7966":{"cik_str":68622,"ticker":"CTBB","title":"QWEST CORP"},"7967":{"cik_str":1882781,"ticker":"CBLO","title":"C2 Blockchain,Inc."},"7968":{"cik_str":1949766,"ticker":"ODDB","title":"Odd Burger Corp"},"7969":{"cik_str":1519472,"ticker":"SMXT","title":"SolarMax Technology, Inc."},"7970":{"cik_str":1957252,"ticker":"AUMB","title":"AUM Biosciences Ltd"},"7971":{"cik_str":1958839,"ticker":"HBER","title":"HUBEIER GROUP LTD."},"7972":{"cik_str":1813744,"ticker":"WDSP","title":"World Scan Project, Inc."},"7973":{"cik_str":1805586,"ticker":"GC","title":"Gladstone Companies, Inc."},"7974":{"cik_str":1735964,"ticker":"CCLFX","title":"Cliffwater Corporate Lending Fund"},"7975":{"cik_str":1948294,"ticker":"NWGL","title":"Nature Wood Group Ltd"},"7976":{"cik_str":1934642,"ticker":"MDBH","title":"MDB Capital Holdings, LLC"},"7977":{"cik_str":1748680,"ticker":"OWSCX","title":"1WS Credit Income Fund"},"7978":{"cik_str":1756404,"ticker":"PDSKX","title":"Principal Diversified Select Real Asset Fund"},"7979":{"cik_str":1795091,"ticker":"OSTX","title":"OS Therapies Inc"},"7980":{"cik_str":1883934,"ticker":"ETZ","title":"Earntz Healthcare Products, Inc."},"7981":{"cik_str":1913749,"ticker":"GMM","title":"Global Mofy Metaverse Ltd"},"7982":{"cik_str":1979332,"ticker":"CPBI","title":"Central Plains Bancshares, Inc."},"7983":{"cik_str":1971975,"ticker":"ABXX","title":"Abaxx Technologies Inc."},"7984":{"cik_str":1826202,"ticker":"KPN","title":"Kepuni Holdings Inc."},"7985":{"cik_str":1935418,"ticker":"FMST","title":"Foremost Lithium Resources & Technology Ltd."},"7986":{"cik_str":1966522,"ticker":"SUBL","title":"BioLingus (Cayman) Ltd"},"7987":{"cik_str":1912287,"ticker":"FLCT","title":"Felicitex Therapeutics Inc."},"7988":{"cik_str":1895159,"ticker":"CJOY","title":"Cine Top Culture Holdings Ltd."},"7989":{"cik_str":1932629,"ticker":"BCGF","title":"BCGF Acquisition Corp."},"7990":{"cik_str":1979005,"ticker":"AFJK","title":"Aimei Health Technology Co., Ltd."},"7991":{"cik_str":1723788,"ticker":"BITW","title":"Bitwise 10 Crypto Index Fund"},"7992":{"cik_str":1852040,"ticker":"GBAT","title":"Grayscale Basic Attention Token Trust (BAT)"},"7993":{"cik_str":1690012,"ticker":"ICR-PA","title":"InPoint Commercial Real Estate Income, Inc."},"7994":{"cik_str":1586009,"ticker":"WEFCX","title":"Wildermuth Fund"},"7995":{"cik_str":1885514,"ticker":"YBZN","title":"Yi Po International Holdings Ltd"},"7996":{"cik_str":1729637,"ticker":"KARX","title":"Karbon-X Corp."},"7997":{"cik_str":1760173,"ticker":"SRGAQ","title":"SURGALIGN HOLDINGS, INC."},"7998":{"cik_str":1852024,"ticker":"GLIV","title":"Grayscale Livepeer Trust (LPT)"},"7999":{"cik_str":1104023,"ticker":"GEMZ","title":"Gemxx Corp."},"8000":{"cik_str":1515001,"ticker":"VCMIX","title":"Versus Capital Multi-Manager Real Estate Income Fund LLC"},"8001":{"cik_str":1407573,"ticker":"ACRU","title":"AmeriCrew Inc."},"8002":{"cik_str":1551047,"ticker":"TIPLX","title":"Bluerock Total Income (plus) Real Estate Fund"},"8003":{"cik_str":1722837,"ticker":"CEDAX","title":"BlueBay Destra International Event-Driven Credit Fund"},"8004":{"cik_str":1661166,"ticker":"LGCP","title":"Legion Capital Corp"},"8005":{"cik_str":1618181,"ticker":"GLDM","title":"World Gold Trust"},"8006":{"cik_str":1368757,"ticker":"GTTJ","title":"GTJ REIT, INC."},"8007":{"cik_str":1880249,"ticker":"PSGI","title":"Perfect Solutions Group, Inc."},"8008":{"cik_str":1916148,"ticker":"CEAC","title":"CE Energy Acquisition Corp."},"8009":{"cik_str":1765048,"ticker":"GCGJ","title":"Charmt, Inc."},"8010":{"cik_str":1754836,"ticker":"EIC","title":"Eagle Point Income Co Inc."},"8011":{"cik_str":1514587,"ticker":"TURO","title":"Turo Inc."},"8012":{"cik_str":1935398,"ticker":"BNTY","title":"Bounty Minerals, Inc."},"8013":{"cik_str":1945415,"ticker":"HUHU","title":"HUHUTECH International Group Inc."},"8014":{"cik_str":1853047,"ticker":"HUDA","title":"Hudson Acquisition I Corp."},"8015":{"cik_str":1923171,"ticker":"CKHL","title":"Chi Ko Holdings Ltd"},"8016":{"cik_str":1710523,"ticker":"BGFDX","title":"Blackstone Floating Rate Enhanced Income Fund"},"8017":{"cik_str":1874137,"ticker":"FDAN","title":"FD TECHNOLOGY INC."},"8018":{"cik_str":1677615,"ticker":"XPTFX","title":"Federated Hermes Project & Trade Finance Tender Fund"},"8019":{"cik_str":1865249,"ticker":"VACX","title":"Vistas Acquisition Co II Inc."},"8020":{"cik_str":1901336,"ticker":"ALCY","title":"Alchemy Investments Acquisition Corp 1"},"8021":{"cik_str":1908054,"ticker":"JETR","title":"Star Jets International Inc."},"8022":{"cik_str":1970653,"ticker":"USEE","title":"USEE ELECTRONIC COMMERCE LTD"},"8023":{"cik_str":1932072,"ticker":"LOBO","title":"LOBO EV TECHNOLOGIES LTD"},"8024":{"cik_str":1865833,"ticker":"ELEP","title":"Elephant Oil Corp."},"8025":{"cik_str":1958928,"ticker":"CTWO","title":"COTWO ADVISORS PHYSICAL EUROPEAN CARBON ALLOWANCE TRUST"},"8026":{"cik_str":1950429,"ticker":"NNAG","title":"99 Acquisition Group Inc."},"8027":{"cik_str":1965671,"ticker":"UNFL","title":"Unifoil Holdings, Inc."},"8028":{"cik_str":1964603,"ticker":"YZH","title":"YUEZHONGHUI INTERNATIONAL HOLDINGS GROUP LTD"},"8029":{"cik_str":1678124,"ticker":"CADCX","title":"CION Ares Diversified Credit Fund"},"8030":{"cik_str":1628040,"ticker":"RCIAX","title":"Alternative Credit Income Fund"},"8031":{"cik_str":1405528,"ticker":"USL","title":"United States 12 Month Oil Fund, LP"},"8032":{"cik_str":1858939,"ticker":"GOSC","title":"Giant Oak Acquisition Corp"},"8033":{"cik_str":1883037,"ticker":"SED","title":"Sedibelo Resources Ltd"},"8034":{"cik_str":1826474,"ticker":"WDHI","title":"Worldwide Diversified Holdings, Inc"},"8035":{"cik_str":1936817,"ticker":"HXHX","title":"Haoxin Holdings Ltd"},"8036":{"cik_str":1879293,"ticker":"MYSN","title":"Myson, Inc."},"8037":{"cik_str":1900830,"ticker":"TSIO","title":"Thornburg Strategic Income Opportunities Trust"},"8038":{"cik_str":1850079,"ticker":"TBIO","title":"Telesis Bio Inc."},"8039":{"cik_str":1353615,"ticker":"FXF","title":"Invesco CurrencyShares Swiss Franc Trust"},"8040":{"cik_str":1130889,"ticker":"CNCL","title":"CANCER CAPITAL CORP"},"8041":{"cik_str":1949257,"ticker":"RADX","title":"Radiopharm Theranostics Ltd"},"8042":{"cik_str":1875655,"ticker":"AEIB","title":"AEI CapForce II Investment Corp"},"8043":{"cik_str":1798458,"ticker":"CRGH","title":"Carriage House Event Center, Inc."},"8044":{"cik_str":1903464,"ticker":"AIMA","title":"Aimfinity Investment Corp. I"},"8045":{"cik_str":1852025,"ticker":"GLNK","title":"Grayscale Chainlink Trust (LINK)"},"8046":{"cik_str":1753945,"ticker":"OPHV","title":"Opti-Harvest, Inc."},"8047":{"cik_str":1794942,"ticker":"SKFG","title":"Stark Focus Group, Inc."},"8048":{"cik_str":1850059,"ticker":"FSHP","title":"Flag Ship Acquisition Corp"},"8049":{"cik_str":1912498,"ticker":"SCRP","title":"Scripps Safe, Inc."},"8050":{"cik_str":1938534,"ticker":"FBGL","title":"FBS Global Ltd"},"8051":{"cik_str":1957538,"ticker":"ESGL","title":"ESGL Holdings Ltd"},"8052":{"cik_str":1980034,"ticker":"BELR","title":"Bell Rose Capital, Inc."},"8053":{"cik_str":1729678,"ticker":"XALCX","title":"BNY Mellon Alcentra Global Multi-Strategy Credit Fund, Inc."},"8054":{"cik_str":1965143,"ticker":"NVNI","title":"Nvni Group Ltd"},"8055":{"cik_str":1916241,"ticker":"PRZO","title":"ParaZero Technologies Ltd."},"8056":{"cik_str":1886444,"ticker":"RVGO","title":"RVELOCITY, INC."},"8057":{"cik_str":1931717,"ticker":"CCTG","title":"CCSC Technology International Holdings Ltd"},"8058":{"cik_str":1965040,"ticker":"FTRE","title":"Fortrea Holdings Inc."},"8059":{"cik_str":1891856,"ticker":"GENK","title":"GEN Restaurant Group, Inc."},"8060":{"cik_str":1862044,"ticker":"ZKH","title":"ZKH Group Ltd"},"8061":{"cik_str":1882148,"ticker":"DESR","title":"DESRI Inc."},"8062":{"cik_str":1460235,"ticker":"PPLT","title":"abrdn Platinum ETF Trust"},"8063":{"cik_str":1309251,"ticker":"MALG","title":"MICROALLIANCE GROUP INC."},"8064":{"cik_str":1286405,"ticker":"GJH","title":"STRATS SM TRUST FOR U S CELL CORP SEC SERIES 2004 6"},"8065":{"cik_str":1546652,"ticker":"OUNZ","title":"VanEck Merk Gold Trust"},"8066":{"cik_str":1852023,"ticker":"MANA","title":"Grayscale Decentraland Trust (MANA)"},"8067":{"cik_str":1700844,"ticker":"EVOH","title":"EvoAir Holdings Inc."},"8068":{"cik_str":1912472,"ticker":"DCCA","title":"Decca Investment Ltd"},"8069":{"cik_str":1913510,"ticker":"VFS","title":"VinFast Auto Pte. Ltd."},"8070":{"cik_str":1923183,"ticker":"HGRN","title":"Healthy Green Group Holding Ltd"},"8071":{"cik_str":1859392,"ticker":"GLXY","title":"Galaxy Digital Inc."},"8072":{"cik_str":1761325,"ticker":"GXLM","title":"Grayscale Stellar Lumens Trust (XLM)"},"8073":{"cik_str":1680689,"ticker":"NRHI","title":"Natural Resource Holdings, Inc."},"8074":{"cik_str":1658645,"ticker":"LENDX","title":"Stone Ridge Trust V"},"8075":{"cik_str":1905920,"ticker":"GLXG","title":"Galaxy Payroll Group Ltd"},"8076":{"cik_str":1957146,"ticker":"EXTO","title":"Almacenes Exito S.A."},"8077":{"cik_str":1973056,"ticker":"BOWN","title":"Bowen Acquisition Corp"},"8078":{"cik_str":1979330,"ticker":"NBBK","title":"NB Bancorp, Inc."},"8079":{"cik_str":1690842,"ticker":"PLTM","title":"GraniteShares Platinum Trust"},"8080":{"cik_str":1970622,"ticker":"IPXX","title":"Inflection Point Acquisition Corp. II"},"8081":{"cik_str":1908705,"ticker":"GLE","title":"Global Engine Group Holding Ltd"},"8082":{"cik_str":1907702,"ticker":"HESP","title":"Hesperos, Inc"},"8083":{"cik_str":1843875,"ticker":"TRCC","title":"TRACCOM INC."},"8084":{"cik_str":1705181,"ticker":"ETCG","title":"Grayscale Ethereum Classic Trust (ETC)"},"8085":{"cik_str":1868110,"ticker":"HGIA","title":"HENGGUANG HOLDING CO, Ltd"},"8086":{"cik_str":1786888,"ticker":"JDRR","title":"F3 Platform Biologics INC"},"8087":{"cik_str":1737936,"ticker":"CNROX","title":"City National Rochdale Strategic Credit Fund"},"8088":{"cik_str":1603652,"ticker":"ULY","title":"Urgent.ly Inc."},"8089":{"cik_str":1726822,"ticker":"ADOB","title":"Soul Biotechnology Corp"},"8090":{"cik_str":1906195,"ticker":"ASPP","title":"Abri SPAC 2, Inc."},"8091":{"cik_str":1924064,"ticker":"RLND","title":"RoyaLand Co Ltd."},"8092":{"cik_str":1957489,"ticker":"ABLV","title":"Able View Global Inc."},"8093":{"cik_str":1835104,"ticker":"INTM","title":"INTERMEDIA CLOUD COMMUNICATIONS, INC."},"8094":{"cik_str":1753712,"ticker":"LARAX","title":"Lord Abbett Credit Opportunities Fund"},"8095":{"cik_str":1956401,"ticker":"MNGA","title":"MNG Havayollari ve Tasimacilik A.S."},"8096":{"cik_str":1652044,"ticker":"GOOG","title":"Alphabet Inc."},"8097":{"cik_str":36104,"ticker":"USB-PA","title":"US BANCORP \\DE\\"},"8098":{"cik_str":1067983,"ticker":"BRK-A","title":"BERKSHIRE HATHAWAY INC"},"8099":{"cik_str":824046,"ticker":"LVMHF","title":"LVMH MOET HENNESSY LOUIS VUITTON"},"8100":{"cik_str":353278,"ticker":"NONOF","title":"NOVO NORDISK A S"},"8101":{"cik_str":19617,"ticker":"JPM-PD","title":"JPMORGAN CHASE & CO"},"8102":{"cik_str":937966,"ticker":"ASMLF","title":"ASML HOLDING NV"},"8103":{"cik_str":70858,"ticker":"BML-PG","title":"BANK OF AMERICA CORP /DE/"},"8104":{"cik_str":70858,"ticker":"BML-PH","title":"BANK OF AMERICA CORP /DE/"},"8105":{"cik_str":70858,"ticker":"BAC-PE","title":"BANK OF AMERICA CORP /DE/"},"8106":{"cik_str":70858,"ticker":"BML-PJ","title":"BANK OF AMERICA CORP /DE/"},"8107":{"cik_str":1577552,"ticker":"BABAF","title":"Alibaba Group Holding Ltd"},"8108":{"cik_str":811809,"ticker":"BHPLF","title":"BHP Group Ltd"},"8109":{"cik_str":901832,"ticker":"AZNCF","title":"ASTRAZENECA PLC"},"8110":{"cik_str":1306965,"ticker":"RYDAF","title":"Shell plc"},"8111":{"cik_str":1114448,"ticker":"NVSEF","title":"NOVARTIS AG"},"8112":{"cik_str":72971,"ticker":"WFC-PQ","title":"WELLS FARGO & COMPANY/MN"},"8113":{"cik_str":1108329,"ticker":"PCCYF","title":"PETROCHINA CO LTD"},"8114":{"cik_str":72971,"ticker":"WFC-PY","title":"WELLS FARGO & COMPANY/MN"},"8115":{"cik_str":72971,"ticker":"WFC-PR","title":"WELLS FARGO & COMPANY/MN"},"8116":{"cik_str":1000184,"ticker":"SAPGF","title":"SAP SE"},"8117":{"cik_str":1089113,"ticker":"HBCYF","title":"HSBC HOLDINGS PLC"},"8118":{"cik_str":879764,"ticker":"TTFNF","title":"TotalEnergies SE"},"8119":{"cik_str":1121404,"ticker":"SNYNF","title":"Sanofi"},"8120":{"cik_str":72971,"ticker":"WFC-PC","title":"WELLS FARGO & COMPANY/MN"},"8121":{"cik_str":1268896,"ticker":"CILJF","title":"CHINA LIFE INSURANCE CO LTD"},"8122":{"cik_str":1378580,"ticker":"EADSF","title":"Airbus SE/ADR"},"8123":{"cik_str":313838,"ticker":"SNEJF","title":"Sony Group Corp"},"8124":{"cik_str":863064,"ticker":"RTPPF","title":"RIO TINTO PLC"},"8125":{"cik_str":835403,"ticker":"DGEAF","title":"DIAGEO PLC"},"8126":{"cik_str":1000275,"ticker":"RY-PT","title":"ROYAL BANK OF CANADA"},"8127":{"cik_str":1274152,"ticker":"CMXHF","title":"CSL LTD"},"8128":{"cik_str":1067318,"ticker":"MBGAF","title":"DAIMLER AG"},"8129":{"cik_str":886982,"ticker":"GS-PA","title":"GOLDMAN SACHS GROUP INC"},"8130":{"cik_str":1303523,"ticker":"BTAFF","title":"British American Tobacco p.l.c."},"8131":{"cik_str":895421,"ticker":"MS-PA","title":"MORGAN STANLEY"},"8132":{"cik_str":1131399,"ticker":"GLAXF","title":"GSK plc"},"8133":{"cik_str":1091587,"ticker":"ABLZF","title":"ABB LTD"},"8134":{"cik_str":886982,"ticker":"GS-PD","title":"GOLDMAN SACHS GROUP INC"},"8135":{"cik_str":886982,"ticker":"GS-PJ","title":"GOLDMAN SACHS GROUP INC"},"8136":{"cik_str":1443276,"ticker":"TOELF","title":"Tokyo Electron LTD"},"8137":{"cik_str":1096200,"ticker":"ESOCF","title":"ENEL SOCIETA PER AZIONI"},"8138":{"cik_str":895421,"ticker":"MS-PI","title":"MORGAN STANLEY"},"8139":{"cik_str":895421,"ticker":"MS-PF","title":"MORGAN STANLEY"},"8140":{"cik_str":748954,"ticker":"ATLCY","title":"ATLAS COPCO AB"},"8141":{"cik_str":886982,"ticker":"GS-PK","title":"GOLDMAN SACHS GROUP INC"},"8142":{"cik_str":1326160,"ticker":"DUK-PA","title":"Duke Energy CORP"},"8143":{"cik_str":891478,"ticker":"BCDRF","title":"Banco Santander, S.A."},"8144":{"cik_str":316709,"ticker":"SCHW-PD","title":"SCHWAB CHARLES CORP"},"8145":{"cik_str":5272,"ticker":"AIG-PA","title":"AMERICAN INTERNATIONAL GROUP, INC."},"8146":{"cik_str":1116578,"ticker":"PUKPF","title":"PRUDENTIAL PLC"},"8147":{"cik_str":1393311,"ticker":"PSA-PK","title":"Public Storage"},"8148":{"cik_str":895564,"ticker":"BAESF","title":"BAE SYSTEMS PLC /FI/"},"8149":{"cik_str":899051,"ticker":"ALL-PH","title":"ALLSTATE CORP"},"8150":{"cik_str":899051,"ticker":"ALL-PB","title":"ALLSTATE CORP"},"8151":{"cik_str":1038143,"ticker":"FNCTF","title":"ORANGE"},"8152":{"cik_str":839923,"ticker":"VODPF","title":"VODAFONE GROUP PUBLIC LTD CO"},"8153":{"cik_str":92122,"ticker":"SOJC","title":"SOUTHERN CO"},"8154":{"cik_str":924613,"ticker":"NOKBF","title":"NOKIA CORP"},"8155":{"cik_str":814052,"ticker":"TEFOF","title":"TELEFONICA S A"},"8156":{"cik_str":1297996,"ticker":"DLR-PJ","title":"DIGITAL REALTY TRUST, INC."},"8157":{"cik_str":1243429,"ticker":"AMSYF","title":"ArcelorMittal"},"8158":{"cik_str":46619,"ticker":"HEI-A","title":"HEICO CORP"},"8159":{"cik_str":1439124,"ticker":"EBR-B","title":"BRAZILIAN ELECTRIC POWER CO"},"8160":{"cik_str":1070304,"ticker":"ORXCF","title":"ORIX CORP"},"8161":{"cik_str":874766,"ticker":"HIG-PG","title":"HARTFORD FINANCIAL SERVICES GROUP, INC."},"8162":{"cik_str":798941,"ticker":"FCNCB","title":"FIRST CITIZENS BANCSHARES INC /DE/"},"8163":{"cik_str":1560385,"ticker":"FWONA","title":"Liberty Media Corp"},"8164":{"cik_str":1560385,"ticker":"FWONK","title":"Liberty Media Corp"},"8165":{"cik_str":35527,"ticker":"FITBI","title":"FIFTH THIRD BANCORP"},"8166":{"cik_str":1754301,"ticker":"FOX","title":"Fox Corp"},"8167":{"cik_str":929058,"ticker":"HUNGF","title":"HUANENG POWER INTERNATIONAL INC"},"8168":{"cik_str":1041668,"ticker":"CHKIF","title":"CHINA SOUTHERN AIRLINES CO LTD"},"8169":{"cik_str":759944,"ticker":"CFG-PD","title":"CITIZENS FINANCIAL GROUP INC/RI"},"8170":{"cik_str":947484,"ticker":"ACGLO","title":"ARCH CAPITAL GROUP LTD."},"8171":{"cik_str":1043219,"ticker":"NLY-PG","title":"ANNALY CAPITAL MANAGEMENT INC"},"8172":{"cik_str":1043219,"ticker":"NLY-PF","title":"ANNALY CAPITAL MANAGEMENT INC"},"8173":{"cik_str":932477,"ticker":"LNVGF","title":"LENOVO GROUP LTD"},"8174":{"cik_str":91576,"ticker":"KEY-PJ","title":"KEYCORP /NEW/"},"8175":{"cik_str":1617640,"ticker":"Z","title":"ZILLOW GROUP, INC."},"8176":{"cik_str":1161611,"ticker":"ACHHY","title":"ALUMINUM CORP OF CHINA LTD"},"8177":{"cik_str":1564708,"ticker":"NWS","title":"NEWS CORP"},"8178":{"cik_str":1026214,"ticker":"FMCCT","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8179":{"cik_str":1747748,"ticker":"XM","title":"Qualtrics International Inc."},"8180":{"cik_str":1560385,"ticker":"LSXMK","title":"Liberty Media Corp"},"8181":{"cik_str":1111711,"ticker":"NI-PB","title":"NISOURCE INC."},"8182":{"cik_str":1545772,"ticker":"BPYPP","title":"Brookfield Property Partners L.P."},"8183":{"cik_str":1076378,"ticker":"CXMSF","title":"CEMEX SAB DE CV"},"8184":{"cik_str":1127055,"ticker":"VIVEF","title":"VIVENDI"},"8185":{"cik_str":769218,"ticker":"AEGOF","title":"AEGON NV"},"8186":{"cik_str":898174,"ticker":"RZB","title":"REINSURANCE GROUP OF AMERICA INC"},"8187":{"cik_str":34903,"ticker":"FRT-PC","title":"FEDERAL REALTY INVESTMENT TRUST"},"8188":{"cik_str":1423689,"ticker":"AGNCN","title":"AGNC Investment Corp."},"8189":{"cik_str":1438569,"ticker":"GIKLY","title":"Grifols SA"},"8190":{"cik_str":1157557,"ticker":"CIG-C","title":"ENERGY CO OF MINAS GERAIS"},"8191":{"cik_str":886982,"ticker":"GS-PC","title":"GOLDMAN SACHS GROUP INC"},"8192":{"cik_str":1562401,"ticker":"AMH-PH","title":"American Homes 4 Rent"},"8193":{"cik_str":1562401,"ticker":"AMH-PG","title":"American Homes 4 Rent"},"8194":{"cik_str":879101,"ticker":"KIM-PM","title":"KIMCO REALTY CORP"},"8195":{"cik_str":1423689,"ticker":"AGNCM","title":"AGNC Investment Corp."},"8196":{"cik_str":1570585,"ticker":"LBTYK","title":"Liberty Global plc"},"8197":{"cik_str":879101,"ticker":"KIM-PL","title":"KIMCO REALTY CORP"},"8198":{"cik_str":913144,"ticker":"RNR-PF","title":"RENAISSANCERE HOLDINGS LTD"},"8199":{"cik_str":1535929,"ticker":"VOYA-PB","title":"Voya Financial, Inc."},"8200":{"cik_str":1527469,"ticker":"ATH-PA","title":"Athene Holding Ltd"},"8201":{"cik_str":1004980,"ticker":"PCG-PA","title":"PG&E Corp"},"8202":{"cik_str":1403528,"ticker":"OAK-PB","title":"Oaktree Capital Group, LLC"},"8203":{"cik_str":1567683,"ticker":"CWEN-A","title":"Clearway Energy, Inc."},"8204":{"cik_str":5513,"ticker":"UNMA","title":"Unum Group"},"8205":{"cik_str":1593538,"ticker":"JSM","title":"NAVIENT CORP"},"8206":{"cik_str":1040971,"ticker":"SLG-PI","title":"SL GREEN REALTY CORP"},"8207":{"cik_str":18349,"ticker":"SNV-PD","title":"SYNOVUS FINANCIAL CORP"},"8208":{"cik_str":1004980,"ticker":"PCG-PB","title":"PG&E Corp"},"8209":{"cik_str":794685,"ticker":"GAB-PG","title":"GABELLI EQUITY TRUST INC"},"8210":{"cik_str":1045450,"ticker":"EPR-PG","title":"EPR PROPERTIES"},"8211":{"cik_str":1172494,"ticker":"AUOTY","title":"AU OPTRONICS CORP"},"8212":{"cik_str":1004980,"ticker":"PCG-PE","title":"PG&E Corp"},"8213":{"cik_str":1126956,"ticker":"SR-PA","title":"SPIRE INC"},"8214":{"cik_str":1792045,"ticker":"THNPY","title":"Technip Energies N.V."},"8215":{"cik_str":1571283,"ticker":"REXR-PB","title":"Rexford Industrial Realty, Inc."},"8216":{"cik_str":803649,"ticker":"EQC-PD","title":"Equity Commonwealth"},"8217":{"cik_str":1685040,"ticker":"BHFAP","title":"Brighthouse Financial, Inc."},"8218":{"cik_str":720672,"ticker":"SF-PB","title":"STIFEL FINANCIAL CORP"},"8219":{"cik_str":1446702,"ticker":"UBSFF","title":"Ubisoft Entertainment SA"},"8220":{"cik_str":43920,"ticker":"GEF-B","title":"GREIF, INC"},"8221":{"cik_str":1308606,"ticker":"SRC-PA","title":"SPIRIT REALTY CAPITAL, INC."},"8222":{"cik_str":1363829,"ticker":"ESGRO","title":"Enstar Group LTD"},"8223":{"cik_str":1012019,"ticker":"RUSHB","title":"RUSH ENTERPRISES INC TX"},"8224":{"cik_str":1015328,"ticker":"WTFCM","title":"WINTRUST FINANCIAL CORP"},"8225":{"cik_str":67887,"ticker":"MOG-B","title":"MOOG INC."},"8226":{"cik_str":78814,"ticker":"PBI-PB","title":"PITNEY BOWES INC /DE/"},"8227":{"cik_str":1110805,"ticker":"NS-PB","title":"NuStar Energy L.P."},"8228":{"cik_str":1012139,"ticker":"GNGYF","title":"GUANGSHEN RAILWAY CO LTD"},"8229":{"cik_str":1110805,"ticker":"NS-PA","title":"NuStar Energy L.P."},"8230":{"cik_str":1336917,"ticker":"UA","title":"Under Armour, Inc."},"8231":{"cik_str":1465740,"ticker":"TWO-PC","title":"TWO HARBORS INVESTMENT CORP."},"8232":{"cik_str":1409493,"ticker":"CIM-PB","title":"CHIMERA INVESTMENT CORP"},"8233":{"cik_str":1409493,"ticker":"CIM-PD","title":"CHIMERA INVESTMENT CORP"},"8234":{"cik_str":310522,"ticker":"FNMAS","title":"FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE"},"8235":{"cik_str":99614,"ticker":"TY-P","title":"TRI-CONTINENTAL CORP"},"8236":{"cik_str":310522,"ticker":"FNMFN","title":"FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE"},"8237":{"cik_str":1465740,"ticker":"TWO-PA","title":"TWO HARBORS INVESTMENT CORP."},"8238":{"cik_str":1409493,"ticker":"CIM-PC","title":"CHIMERA INVESTMENT CORP"},"8239":{"cik_str":310522,"ticker":"FNMAH","title":"FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE"},"8240":{"cik_str":357294,"ticker":"HOVNP","title":"HOVNANIAN ENTERPRISES INC"},"8241":{"cik_str":1465740,"ticker":"TWO-PB","title":"TWO HARBORS INVESTMENT CORP."},"8242":{"cik_str":310522,"ticker":"FNMAJ","title":"FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE"},"8243":{"cik_str":1055160,"ticker":"MFA-PB","title":"MFA FINANCIAL, INC."},"8244":{"cik_str":310522,"ticker":"FNMAM","title":"FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE"},"8245":{"cik_str":925261,"ticker":"AKO-B","title":"ANDINA BOTTLING CO INC"},"8246":{"cik_str":1110805,"ticker":"NS-PC","title":"NuStar Energy L.P."},"8247":{"cik_str":714310,"ticker":"VLYPP","title":"VALLEY NATIONAL BANCORP"},"8248":{"cik_str":1660734,"ticker":"TRTN-PA","title":"Triton International Ltd"},"8249":{"cik_str":1628063,"ticker":"SRG-PA","title":"Seritage Growth Properties"},"8250":{"cik_str":7789,"ticker":"ASB-PE","title":"ASSOCIATED BANC-CORP"},"8251":{"cik_str":1560385,"ticker":"BATRK","title":"Liberty Media Corp"},"8252":{"cik_str":1829948,"ticker":"CIXXF","title":"CI Financial Corp."},"8253":{"cik_str":887733,"ticker":"CENTA","title":"CENTRAL GARDEN & PET CO"},"8254":{"cik_str":929351,"ticker":"LGF-B","title":"LIONS GATE ENTERTAINMENT CORP /CN/"},"8255":{"cik_str":1892316,"ticker":"LDDD","title":"Longduoduo Co Ltd"},"8256":{"cik_str":1259942,"ticker":"BLU","title":"BELLUS Health Inc."},"8257":{"cik_str":107140,"ticker":"WLYB","title":"JOHN WILEY & SONS, INC."},"8258":{"cik_str":1712184,"ticker":"LILAK","title":"Liberty Latin America Ltd."},"8259":{"cik_str":1504461,"ticker":"NGL-PB","title":"NGL Energy Partners LP"},"8260":{"cik_str":1094831,"ticker":"BGCP","title":"BGC Partners, Inc."},"8261":{"cik_str":1437071,"ticker":"IVR-PC","title":"Invesco Mortgage Capital Inc."},"8262":{"cik_str":1412100,"ticker":"MHLA","title":"Maiden Holdings, Ltd."},"8263":{"cik_str":845877,"ticker":"AGM-A","title":"FEDERAL AGRICULTURAL MORTGAGE CORP"},"8264":{"cik_str":1026214,"ticker":"FMCKL","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8265":{"cik_str":794685,"ticker":"GAB-PH","title":"GABELLI EQUITY TRUST INC"},"8266":{"cik_str":1026214,"ticker":"FMCCG","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8267":{"cik_str":1526113,"ticker":"GNL-PA","title":"Global Net Lease, Inc."},"8268":{"cik_str":1026214,"ticker":"FMCKI","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8269":{"cik_str":1534126,"ticker":"GLOG-PA","title":"GasLog Ltd."},"8270":{"cik_str":1026214,"ticker":"FMCCL","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8271":{"cik_str":1026214,"ticker":"FMCCO","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8272":{"cik_str":1280784,"ticker":"HCXY","title":"Hercules Capital, Inc."},"8273":{"cik_str":1026214,"ticker":"FMCCJ","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8274":{"cik_str":1026214,"ticker":"FMCCI","title":"FEDERAL HOME LOAN MORTGAGE CORP"},"8275":{"cik_str":1345126,"ticker":"CODI-PA","title":"Compass Diversified Holdings"},"8276":{"cik_str":1267395,"ticker":"AHL-PD","title":"ASPEN INSURANCE HOLDINGS LTD"},"8277":{"cik_str":1273685,"ticker":"NYMTN","title":"NEW YORK MORTGAGE TRUST INC"},"8278":{"cik_str":1569187,"ticker":"AHH-PA","title":"Armada Hoffler Properties, Inc."},"8279":{"cik_str":1598655,"ticker":"GLOP-PB","title":"GasLog Partners LP"},"8280":{"cik_str":1497645,"ticker":"INN-PE","title":"Summit Hotel Properties, Inc."},"8281":{"cik_str":1677576,"ticker":"IIPR-PA","title":"INNOVATIVE INDUSTRIAL PROPERTIES INC"},"8282":{"cik_str":1598655,"ticker":"GLOP-PC","title":"GasLog Partners LP"},"8283":{"cik_str":40417,"ticker":"GAM-PB","title":"GENERAL AMERICAN INVESTORS CO INC"},"8284":{"cik_str":1029800,"ticker":"UBA","title":"URSTADT BIDDLE PROPERTIES INC"},"8285":{"cik_str":1843862,"ticker":"EOCW","title":"Elliott Opportunity II Corp."},"8286":{"cik_str":729580,"ticker":"BELFB","title":"BEL FUSE INC /NJ"},"8287":{"cik_str":845877,"ticker":"AGM-PD","title":"FEDERAL AGRICULTURAL MORTGAGE CORP"},"8288":{"cik_str":845877,"ticker":"AGM-PC","title":"FEDERAL AGRICULTURAL MORTGAGE CORP"},"8289":{"cik_str":1029800,"ticker":"UBP-PH","title":"URSTADT BIDDLE PROPERTIES INC"},"8290":{"cik_str":1037390,"ticker":"INDT","title":"INDUS REALTY TRUST, INC."},"8291":{"cik_str":1709442,"ticker":"FSUN","title":"FIRSTSUN CAPITAL BANCORP"},"8292":{"cik_str":1209028,"ticker":"AIC","title":"Arlington Asset Investment Corp."},"8293":{"cik_str":1503584,"ticker":"CMRE-PB","title":"Costamare Inc."},"8294":{"cik_str":1726173,"ticker":"BH","title":"Biglari Holdings Inc."},"8295":{"cik_str":1503584,"ticker":"CMRE-PC","title":"Costamare Inc."},"8296":{"cik_str":1339422,"ticker":"KDSKF","title":"Royal DSM N.V."},"8297":{"cik_str":1503584,"ticker":"CMRE-PD","title":"Costamare Inc."},"8298":{"cik_str":1290877,"ticker":"CLVLF","title":"Clinuvel Pharmaceuticals Ltd"},"8299":{"cik_str":1503584,"ticker":"CMRE-PE","title":"Costamare Inc."},"8300":{"cik_str":921671,"ticker":"GGT-PE","title":"GABELLI MULTIMEDIA TRUST INC."},"8301":{"cik_str":1063344,"ticker":"HT-PD","title":"HERSHA HOSPITALITY TRUST"},"8302":{"cik_str":1313510,"ticker":"GGN-PB","title":"GAMCO Global Gold, Natural Resources & Income Trust"},"8303":{"cik_str":353184,"ticker":"AIRTP","title":"AIR T INC"},"8304":{"cik_str":216085,"ticker":"HVT-A","title":"HAVERTY FURNITURE COMPANIES INC"},"8305":{"cik_str":1060349,"ticker":"GAMI","title":"GAMCO INVESTORS, INC. ET AL"},"8306":{"cik_str":1896084,"ticker":"GSCCF","title":"ioneer Ltd"},"8307":{"cik_str":800457,"ticker":"DGICB","title":"DONEGAL GROUP INC"},"8308":{"cik_str":752642,"ticker":"UMH-PD","title":"UMH PROPERTIES, INC."},"8309":{"cik_str":1604174,"ticker":"ECCX","title":"Eagle Point Credit Co Inc."},"8310":{"cik_str":1887388,"ticker":"BRCHF","title":"Brainchip Holdings Limited/ADR"},"8311":{"cik_str":1214935,"ticker":"NCV-PA","title":"Virtus Convertible & Income Fund"},"8312":{"cik_str":1234006,"ticker":"GOODO","title":"GLADSTONE COMMERCIAL CORP"},"8313":{"cik_str":1355096,"ticker":"QRTEB","title":"Qurate Retail, Inc."},"8314":{"cik_str":1533615,"ticker":"GMRE-PA","title":"Global Medical REIT Inc."},"8315":{"cik_str":1629019,"ticker":"MBINP","title":"Merchants Bancorp"},"8316":{"cik_str":1495222,"ticker":"OXLCO","title":"Oxford Lane Capital Corp."},"8317":{"cik_str":1452857,"ticker":"SPLP-PA","title":"STEEL PARTNERS HOLDINGS L.P."},"8318":{"cik_str":1514281,"ticker":"MITT-PA","title":"AG Mortgage Investment Trust, Inc."},"8319":{"cik_str":1080720,"ticker":"GUT-PC","title":"GABELLI UTILITY TRUST"},"8320":{"cik_str":1944885,"ticker":"APLMW","title":"Apollomics Inc."},"8321":{"cik_str":1593222,"ticker":"CIO-PA","title":"City Office REIT, Inc."},"8322":{"cik_str":1574085,"ticker":"BHR-PD","title":"Braemar Hotels & Resorts Inc."},"8323":{"cik_str":1514281,"ticker":"MITT-PB","title":"AG Mortgage Investment Trust, Inc."},"8324":{"cik_str":923571,"ticker":"USX","title":"US XPRESS ENTERPRISES INC"},"8325":{"cik_str":1850271,"ticker":"DNAB","title":"Social Capital Suvretta Holdings Corp. II"},"8326":{"cik_str":1850272,"ticker":"DNAD","title":"Social Capital Suvretta Holdings Corp. IV"},"8327":{"cik_str":1166663,"ticker":"TNP-PD","title":"TSAKOS ENERGY NAVIGATION LTD"},"8328":{"cik_str":1859795,"ticker":"NVNXF","title":"NOVONIX Ltd"},"8329":{"cik_str":1166663,"ticker":"TNP-PE","title":"TSAKOS ENERGY NAVIGATION LTD"},"8330":{"cik_str":1041657,"ticker":"UONEK","title":"URBAN ONE, INC."},"8331":{"cik_str":1574085,"ticker":"BHR-PB","title":"Braemar Hotels & Resorts Inc."},"8332":{"cik_str":1232582,"ticker":"AHT-PG","title":"ASHFORD HOSPITALITY TRUST INC"},"8333":{"cik_str":1232582,"ticker":"AHT-PI","title":"ASHFORD HOSPITALITY TRUST INC"},"8334":{"cik_str":1232582,"ticker":"AHT-PH","title":"ASHFORD HOSPITALITY TRUST INC"},"8335":{"cik_str":1232582,"ticker":"AHT-PD","title":"ASHFORD HOSPITALITY TRUST INC"},"8336":{"cik_str":1571776,"ticker":"CHMI-PB","title":"Cherry Hill Mortgage Investment Corp"},"8337":{"cik_str":1232582,"ticker":"AHT-PF","title":"ASHFORD HOSPITALITY TRUST INC"},"8338":{"cik_str":922358,"ticker":"FGPRB","title":"FERRELLGAS PARTNERS L P"},"8339":{"cik_str":1434754,"ticker":"SB-PD","title":"SAFE BULKERS, INC."},"8340":{"cik_str":1564584,"ticker":"NIQ","title":"Nuveen Intermediate Duration Quality Municipal Term Fund"},"8341":{"cik_str":855874,"ticker":"TCFC","title":"COMMUNITY FINANCIAL CORP /MD/"},"8342":{"cik_str":1578453,"ticker":"DLNG-PA","title":"Dynagas LNG Partners LP"},"8343":{"cik_str":761648,"ticker":"CDR-PB","title":"CEDAR REALTY TRUST, INC."},"8344":{"cik_str":761648,"ticker":"CDR-PC","title":"CEDAR REALTY TRUST, INC."},"8345":{"cik_str":1901799,"ticker":"GSRM","title":"Bitcoin Depot Inc."},"8346":{"cik_str":1814287,"ticker":"ERES","title":"East Resources Acquisition Co"},"8347":{"cik_str":793040,"ticker":"ECF-PA","title":"ELLSWORTH GROWTH & INCOME FUND LTD"},"8348":{"cik_str":1301236,"ticker":"SOHON","title":"Sotherly Hotels Inc."},"8349":{"cik_str":1301236,"ticker":"SOHOB","title":"Sotherly Hotels Inc."},"8350":{"cik_str":9521,"ticker":"BCV-PA","title":"BANCROFT FUND LTD"},"8351":{"cik_str":1355848,"ticker":"GNUS","title":"Kartoon Studios, Inc."},"8352":{"cik_str":1429260,"ticker":"FBIOP","title":"Fortress Biotech, Inc."},"8353":{"cik_str":1838831,"ticker":"DISA","title":"Disruptive Acquisition Corp I"},"8354":{"cik_str":1841675,"ticker":"ARBKF","title":"Argo Blockchain Plc"},"8355":{"cik_str":1888274,"ticker":"DEFTF","title":"Defi Technologies, Inc."},"8356":{"cik_str":1787384,"ticker":"FFBW","title":"FFBW, Inc. /MD/"},"8357":{"cik_str":1823575,"ticker":"ZFOXW","title":"ZeroFox Holdings, Inc."},"8358":{"cik_str":1087456,"ticker":"UBOH","title":"UNITED BANCSHARES INC/OH"},"8359":{"cik_str":1737270,"ticker":"NEXCF","title":"NexTech AR Solutions Corp."},"8360":{"cik_str":1787005,"ticker":"CNNB","title":"Cincinnati Bancorp, Inc."},"8361":{"cik_str":838875,"ticker":"WVVIP","title":"WILLAMETTE VALLEY VINEYARDS INC"},"8362":{"cik_str":1463000,"ticker":"GRUSF","title":"Grown Rogue International Inc."},"8363":{"cik_str":942149,"ticker":"OCGSF","title":"Outcrop Silver & Gold Corp"},"8364":{"cik_str":1882839,"ticker":"WONDF","title":"WonderFi Technologies Inc."},"8365":{"cik_str":1636050,"ticker":"SIOX","title":"Sio Gene Therapies Inc."},"8366":{"cik_str":1444874,"ticker":"TETAA","title":"Teton Advisors, Inc."},"8367":{"cik_str":1624422,"ticker":"ERYP","title":"PHAXIAM Therapeutics S.A."},"8368":{"cik_str":943034,"ticker":"AATC","title":"AUTOSCOPE TECHNOLOGIES CORP"},"8369":{"cik_str":1130166,"ticker":"CYCCP","title":"Cyclacel Pharmaceuticals, Inc."},"8370":{"cik_str":1762359,"ticker":"ACRDF","title":"Acreage Holdings, Inc."},"8371":{"cik_str":1042046,"ticker":"AFGC","title":"AMERICAN FINANCIAL GROUP INC"},"8372":{"cik_str":1312109,"ticker":"SICP","title":"Silvergate Capital Corp"},"8373":{"cik_str":1002135,"ticker":"WSTL","title":"WESTELL TECHNOLOGIES INC"},"8374":{"cik_str":1336364,"ticker":"HNCKF","title":"Giga Metals Corp"},"8375":{"cik_str":1558924,"ticker":"PKKFF","title":"Tenet Fintech Group Inc."},"8376":{"cik_str":1645155,"ticker":"WBSR","title":"Webstar Technology Group Inc."},"8377":{"cik_str":1949766,"ticker":"ODDAF","title":"Odd Burger Corp"},"8378":{"cik_str":926423,"ticker":"MINDP","title":"MIND TECHNOLOGY, INC"},"8379":{"cik_str":1866390,"ticker":"BBLN","title":"Babylon Holdings Ltd"},"8380":{"cik_str":1819876,"ticker":"TRIRF","title":"Triterras, Inc."},"8381":{"cik_str":1756704,"ticker":"SIRC","title":"SOLAR INTEGRATED ROOFING CORP."},"8382":{"cik_str":1617765,"ticker":"LNDZF","title":"Salona Global Medical Device Corp"},"8383":{"cik_str":1604477,"ticker":"SQZ","title":"SQZ Biotechnologies Co"},"8384":{"cik_str":840551,"ticker":"TMDIF","title":"TITAN MEDICAL INC"},"8385":{"cik_str":1868079,"ticker":"PMEDF","title":"Predictmedix Inc."},"8386":{"cik_str":1376986,"ticker":"TVE","title":"Tennessee Valley Authority"},"8387":{"cik_str":1527541,"ticker":"WHLRD","title":"Wheeler Real Estate Investment Trust, Inc."},"8388":{"cik_str":1811942,"ticker":"FFLWF","title":"Fire & Flower Holdings Corp."},"8389":{"cik_str":1771755,"ticker":"SWISF","title":"Sekur Private Data Ltd."},"8390":{"cik_str":1804469,"ticker":"GRDAF","title":"Guardforce AI Co., Ltd."},"8391":{"cik_str":1318482,"ticker":"SGLDF","title":"KIDOZ INC."},"8392":{"cik_str":927720,"ticker":"SBSAA","title":"SPANISH BROADCASTING SYSTEM INC"},"8393":{"cik_str":1609139,"ticker":"INND","title":"INNERSCOPE HEARING TECHNOLOGIES, INC."},"8394":{"cik_str":1631463,"ticker":"BRLL","title":"Barrel Energy Inc."},"8395":{"cik_str":1263011,"ticker":"EXNRF","title":"EXCELLON RESOURCES INC"},"8396":{"cik_str":1508381,"ticker":"BNKL","title":"Bionik Laboratories Corp."},"8397":{"cik_str":1588014,"ticker":"ADMQ","title":"ADM ENDEAVORS, INC."},"8398":{"cik_str":1094084,"ticker":"TKOI","title":"TELKONET INC"},"8399":{"cik_str":1525852,"ticker":"BTZI","title":"BOTS, Inc./PR"},"8400":{"cik_str":846546,"ticker":"BNSO","title":"BONSO ELECTRONICS INTERNATIONAL INC"},"8401":{"cik_str":1435064,"ticker":"CETXP","title":"CEMTREX INC"},"8402":{"cik_str":1076262,"ticker":"GSPT","title":"Golden Star Enterprises Ltd."},"8403":{"cik_str":1922639,"ticker":"OILCF","title":"Permex Petroleum Corp"},"8404":{"cik_str":1593034,"ticker":"ENDPQ","title":"Endo International plc"},"8405":{"cik_str":1487198,"ticker":"ASPU","title":"ASPEN GROUP, INC."},"8406":{"cik_str":42050,"ticker":"WSRC","title":"WESTERN SIERRA RESOURCE Corp"},"8407":{"cik_str":1370496,"ticker":"MLLOF","title":"Medallion Resources Ltd"},"8408":{"cik_str":1000683,"ticker":"BDRL","title":"BLONDER TONGUE LABORATORIES INC"},"8409":{"cik_str":1000298,"ticker":"IMPM","title":"IMPAC MORTGAGE HOLDINGS INC"},"8410":{"cik_str":1760173,"ticker":"SRGA","title":"SURGALIGN HOLDINGS, INC."},"8411":{"cik_str":1444307,"ticker":"ONCS","title":"ONCOSEC MEDICAL Inc"},"8412":{"cik_str":1642178,"ticker":"AGNPF","title":"Algernon Pharmaceuticals Inc."},"8413":{"cik_str":1004724,"ticker":"RHE-PA","title":"REGIONAL HEALTH PROPERTIES, INC"},"8414":{"cik_str":1825270,"ticker":"MKGP","title":"Maverick Energy Group, Ltd."},"8415":{"cik_str":1794311,"ticker":"BLEG","title":"Branded Legacy, Inc."},"8416":{"cik_str":1349929,"ticker":"VYNT","title":"Vyant Bio, Inc."},"8417":{"cik_str":1664127,"ticker":"ALTD","title":"ALTITUDE INTERNATIONAL HOLDINGS, INC."},"8418":{"cik_str":1653821,"ticker":"CENBF","title":"CEN BIOTECH INC"},"8419":{"cik_str":1068618,"ticker":"GVSI","title":"Good Vibrations Shoes, Inc."},"8420":{"cik_str":842722,"ticker":"DROP","title":"Fuse Science, Inc."},"8421":{"cik_str":1313938,"ticker":"SNNC","title":"Sibannac, Inc."},"8422":{"cik_str":100716,"ticker":"UNAMQ","title":"UNICO AMERICAN CORP"},"8423":{"cik_str":1429859,"ticker":"CBTC","title":"XTRA Bitcoin Inc."},"8424":{"cik_str":1420108,"ticker":"GLUC","title":"Glucose Health, Inc."},"8425":{"cik_str":1839730,"ticker":"SLHGF","title":"Skylight Health Group Inc."},"8426":{"cik_str":1351573,"ticker":"PHCG","title":"Pure Harvest Corporate Group, Inc."},"8427":{"cik_str":1325950,"ticker":"MOCI","title":"Energy Holdings, Inc."},"8428":{"cik_str":58411,"ticker":"LPHM","title":"Lee Pharmaceuticals, Inc."},"8429":{"cik_str":1108248,"ticker":"KNOS","title":"KRONOS ADVANCED TECHNOLOGIES INC"},"8430":{"cik_str":1621221,"ticker":"ARTLW","title":"ARTELO BIOSCIENCES, INC."},"8431":{"cik_str":1084577,"ticker":"CYRNQ","title":"CYREN Ltd."},"8432":{"cik_str":1566610,"ticker":"VERBW","title":"Verb Technology Company, Inc."},"8433":{"cik_str":1414043,"ticker":"FDBL","title":"Friendable, Inc."},"8434":{"cik_str":842183,"ticker":"RPT-PD","title":"RPT Realty"},"8435":{"cik_str":70858,"ticker":"BAC-PL","title":"BANK OF AMERICA CORP /DE/"},"8436":{"cik_str":70858,"ticker":"BAC-PM","title":"BANK OF AMERICA CORP /DE/"},"8437":{"cik_str":70858,"ticker":"BAC-PN","title":"BANK OF AMERICA CORP /DE/"},"8438":{"cik_str":70858,"ticker":"BAC-PP","title":"BANK OF AMERICA CORP /DE/"},"8439":{"cik_str":70858,"ticker":"BAC-PO","title":"BANK OF AMERICA CORP /DE/"},"8440":{"cik_str":109380,"ticker":"ZIONO","title":"ZIONS BANCORPORATION, NATIONAL ASSOCIATION /UT/"},"8441":{"cik_str":109380,"ticker":"ZIONP","title":"ZIONS BANCORPORATION, NATIONAL ASSOCIATION /UT/"},"8442":{"cik_str":109380,"ticker":"ZIONL","title":"ZIONS BANCORPORATION, NATIONAL ASSOCIATION /UT/"},"8443":{"cik_str":813828,"ticker":"PARAP","title":"Paramount Global"},"8444":{"cik_str":846617,"ticker":"DCOMP","title":"Dime Community Bancshares, Inc. /NY/"},"8445":{"cik_str":821130,"ticker":"UZE","title":"UNITED STATES CELLULAR CORP"},"8446":{"cik_str":821130,"ticker":"UZF","title":"UNITED STATES CELLULAR CORP"},"8447":{"cik_str":1456772,"ticker":"OPINL","title":"OFFICE PROPERTIES INCOME TRUST"},"8448":{"cik_str":1004702,"ticker":"OCFCP","title":"OCEANFIRST FINANCIAL CORP"},"8449":{"cik_str":1014763,"ticker":"AIMDW","title":"Ainos, Inc."},"8450":{"cik_str":1582982,"ticker":"CCLDP","title":"CareCloud, Inc."},"8451":{"cik_str":1582982,"ticker":"CCLDO","title":"CareCloud, Inc."},"8452":{"cik_str":1853047,"ticker":"HUDAR","title":"Hudson Acquisition I Corp."},"8453":{"cik_str":1870129,"ticker":"ROSEW","title":"Rose Hill Acquisition Corp"},"8454":{"cik_str":1836176,"ticker":"FATH-WT","title":"Fathom Digital Manufacturing Corp"},"8455":{"cik_str":1834974,"ticker":"LEV-WT","title":"Lion Electric Co"},"8456":{"cik_str":1893325,"ticker":"SCRMW","title":"Screaming Eagle Acquisition Corp."},"8457":{"cik_str":1819989,"ticker":"CIFRW","title":"Cipher Mining Inc."},"8458":{"cik_str":1693577,"ticker":"MNSBP","title":"MainStreet Bancshares, Inc."},"8459":{"cik_str":1604174,"ticker":"ECCW","title":"Eagle Point Credit Co Inc."},"8460":{"cik_str":1604174,"ticker":"ECC-PD","title":"Eagle Point Credit Co Inc."},"8461":{"cik_str":1326160,"ticker":"DUKB","title":"Duke Energy CORP"},"8462":{"cik_str":7789,"ticker":"ASBA","title":"ASSOCIATED BANC-CORP"},"8463":{"cik_str":37996,"ticker":"F-PC","title":"FORD MOTOR CO"},"8464":{"cik_str":37996,"ticker":"F-PD","title":"FORD MOTOR CO"},"8465":{"cik_str":37996,"ticker":"F-PB","title":"FORD MOTOR CO"},"8466":{"cik_str":1867102,"ticker":"EVTL-WT","title":"Vertical Aerospace Ltd."},"8467":{"cik_str":1903870,"ticker":"AFRIW","title":"Forafric Global PLC"},"8468":{"cik_str":1892922,"ticker":"MARXU","title":"Mars Acquisition Corp."},"8469":{"cik_str":1862490,"ticker":"DCFCW","title":"Tritium DCFC Ltd"},"8470":{"cik_str":1929231,"ticker":"PLTNW","title":"Plutonian Acquisition Corp."},"8471":{"cik_str":1784851,"ticker":"SHPW-WT","title":"Shapeways Holdings, Inc."},"8472":{"cik_str":769218,"ticker":"AEFC","title":"AEGON NV"},"8473":{"cik_str":750577,"ticker":"HWCPZ","title":"HANCOCK WHITNEY CORP"},"8474":{"cik_str":1174169,"ticker":"AQNB","title":"ALGONQUIN POWER & UTILITIES CORP."},"8475":{"cik_str":1174169,"ticker":"AQNA","title":"ALGONQUIN POWER & UTILITIES CORP."},"8476":{"cik_str":1866782,"ticker":"CDROW","title":"Codere Online Luxembourg, S.A."},"8477":{"cik_str":1823878,"ticker":"MYPSW","title":"PLAYSTUDIOS, Inc."},"8478":{"cik_str":1830214,"ticker":"DNA-WT","title":"Ginkgo Bioworks Holdings, Inc."},"8479":{"cik_str":1844452,"ticker":"LUNRW","title":"Intuitive Machines, Inc."},"8480":{"cik_str":1801417,"ticker":"BYNOW","title":"byNordic Acquisition Corp"},"8481":{"cik_str":1835512,"ticker":"LLAP-WT","title":"Terran Orbital Corp"},"8482":{"cik_str":1821424,"ticker":"UKOMW","title":"Ucommune International Ltd"},"8483":{"cik_str":1857855,"ticker":"FNVTW","title":"Finnovate Acquisition Corp."},"8484":{"cik_str":1887673,"ticker":"WLDSW","title":"Wearable Devices Ltd."},"8485":{"cik_str":1847513,"ticker":"TRONU","title":"CORNER GROWTH ACQUISITION CORP. 2"},"8486":{"cik_str":1614806,"ticker":"AJXA","title":"Great Ajax Corp."},"8487":{"cik_str":1406234,"ticker":"BIP-PB","title":"Brookfield Infrastructure Partners L.P."},"8488":{"cik_str":1490349,"ticker":"PFXNZ","title":"PhenixFIN Corp"},"8489":{"cik_str":1744494,"ticker":"ADNWW","title":"ADVENT TECHNOLOGIES HOLDINGS, INC."},"8490":{"cik_str":1606268,"ticker":"VIASP","title":"Via Renewables, Inc."},"8491":{"cik_str":1635077,"ticker":"ACONW","title":"Aclarion, Inc."},"8492":{"cik_str":1553788,"ticker":"SBEV-WT","title":"SPLASH BEVERAGE GROUP, INC."},"8493":{"cik_str":1554818,"ticker":"AUUDW","title":"AUDDIA INC."},"8494":{"cik_str":1601712,"ticker":"SYF-PA","title":"Synchrony Financial"},"8495":{"cik_str":899689,"ticker":"VNO-PO","title":"VORNADO REALTY TRUST"},"8496":{"cik_str":1474098,"ticker":"PEB-PH","title":"Pebblebrook Hotel Trust"},"8497":{"cik_str":73124,"ticker":"NTRSO","title":"NORTHERN TRUST CORP"},"8498":{"cik_str":14930,"ticker":"BC-PC","title":"BRUNSWICK CORP"},"8499":{"cik_str":14930,"ticker":"BC-PB","title":"BRUNSWICK CORP"},"8500":{"cik_str":14930,"ticker":"BC-PA","title":"BRUNSWICK CORP"},"8501":{"cik_str":831001,"ticker":"C-PN","title":"CITIGROUP INC"},"8502":{"cik_str":832489,"ticker":"GOVXW","title":"GeoVax Labs, Inc."},"8503":{"cik_str":884614,"ticker":"UGIC","title":"UGI CORP /PA/"},"8504":{"cik_str":920112,"ticker":"HTLFP","title":"HEARTLAND FINANCIAL USA INC"},"8505":{"cik_str":1058307,"ticker":"NXPLW","title":"NextPlat Corp"},"8506":{"cik_str":1091748,"ticker":"ARGO-PA","title":"Argo Group International Holdings, Ltd."},"8507":{"cik_str":1859035,"ticker":"MCAGR","title":"Mountain Crest Acquisition Corp. V"},"8508":{"cik_str":720672,"ticker":"SF-PC","title":"STIFEL FINANCIAL CORP"},"8509":{"cik_str":720672,"ticker":"SF-PD","title":"STIFEL FINANCIAL CORP"},"8510":{"cik_str":720672,"ticker":"SFB","title":"STIFEL FINANCIAL CORP"},"8511":{"cik_str":712534,"ticker":"FRMEP","title":"FIRST MERCHANTS CORP"},"8512":{"cik_str":312070,"ticker":"VXX","title":"BARCLAYS BANK PLC"},"8513":{"cik_str":312070,"ticker":"VXZ","title":"BARCLAYS BANK PLC"},"8514":{"cik_str":312070,"ticker":"GRN","title":"BARCLAYS BANK PLC"},"8515":{"cik_str":312070,"ticker":"SGGFF","title":"BARCLAYS BANK PLC"},"8516":{"cik_str":312070,"ticker":"DJP","title":"BARCLAYS BANK PLC"},"8517":{"cik_str":61398,"ticker":"TELZ","title":"TELLURIAN INC. /DE/"},"8518":{"cik_str":874761,"ticker":"AESC","title":"AES CORP"},"8519":{"cik_str":1341317,"ticker":"BWBBP","title":"Bridgewater Bancshares Inc"},"8520":{"cik_str":1739174,"ticker":"PHGE-UN","title":"BiomX Inc."},"8521":{"cik_str":1624326,"ticker":"PAVMZ","title":"PAVmed Inc."},"8522":{"cik_str":1597033,"ticker":"SABRP","title":"Sabre Corp"},"8523":{"cik_str":1668010,"ticker":"DBGIW","title":"Digital Brands Group, Inc."},"8524":{"cik_str":1817218,"ticker":"SPKX","title":"ConvexityShares Trust"},"8525":{"cik_str":1817218,"ticker":"SPKY","title":"ConvexityShares Trust"},"8526":{"cik_str":1819794,"ticker":"HTOOW","title":"Fusion Fuel Green PLC"},"8527":{"cik_str":1828248,"ticker":"CVII-WT","title":"Churchill Capital Corp VII"},"8528":{"cik_str":1835654,"ticker":"INVZW","title":"Innoviz Technologies Ltd."},"8529":{"cik_str":1854587,"ticker":"CLBTW","title":"Cellebrite DI Ltd."},"8530":{"cik_str":1660734,"ticker":"TRTN-PE","title":"Triton International Ltd"},"8531":{"cik_str":1660734,"ticker":"TRTN-PB","title":"Triton International Ltd"},"8532":{"cik_str":1527469,"ticker":"ATH-PB","title":"Athene Holding Ltd"},"8533":{"cik_str":1527469,"ticker":"ATH-PE","title":"Athene Holding Ltd"},"8534":{"cik_str":1527469,"ticker":"ATH-PD","title":"Athene Holding Ltd"},"8535":{"cik_str":1527469,"ticker":"ATH-PC","title":"Athene Holding Ltd"},"8536":{"cik_str":1562463,"ticker":"INBKZ","title":"First Internet Bancorp"},"8537":{"cik_str":1748137,"ticker":"NEOVW","title":"NeoVolta Inc."},"8538":{"cik_str":1862068,"ticker":"RBT-WT","title":"Rubicon Technologies, Inc."},"8539":{"cik_str":1753706,"ticker":"FREEW","title":"Whole Earth Brands, Inc."},"8540":{"cik_str":1838987,"ticker":"FACT-WT","title":"Freedom Acquisition I Corp."},"8541":{"cik_str":1838672,"ticker":"ADTHW","title":"AdTheorent Holding Company, Inc."},"8542":{"cik_str":1412100,"ticker":"MHNC","title":"Maiden Holdings, Ltd."},"8543":{"cik_str":1051512,"ticker":"TDS-PV","title":"TELEPHONE & DATA SYSTEMS INC /DE/"},"8544":{"cik_str":1253986,"ticker":"ABR-PD","title":"ARBOR REALTY TRUST INC"},"8545":{"cik_str":1253986,"ticker":"ABR-PE","title":"ARBOR REALTY TRUST INC"},"8546":{"cik_str":1253986,"ticker":"ABR-PF","title":"ARBOR REALTY TRUST INC"},"8547":{"cik_str":1021162,"ticker":"TGIDW","title":"TRIUMPH GROUP INC"},"8548":{"cik_str":310522,"ticker":"FNMAP","title":"FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE"},"8549":{"cik_str":316709,"ticker":"SCHW-PJ","title":"SCHWAB CHARLES CORP"},"8550":{"cik_str":320335,"ticker":"GL-PD","title":"GLOBE LIFE INC."},"8551":{"cik_str":1946399,"ticker":"YSBPW","title":"YS Biopharma Co., Ltd."},"8552":{"cik_str":1844971,"ticker":"GREEL","title":"Greenidge Generation Holdings Inc."},"8553":{"cik_str":1825249,"ticker":"CFIVU","title":"CF ACQUISITION CORP. IV"},"8554":{"cik_str":4904,"ticker":"AEPPZ","title":"AMERICAN ELECTRIC POWER CO INC"},"8555":{"cik_str":1852407,"ticker":"FEXDR","title":"Fintech Ecosystem Development Corp."},"8556":{"cik_str":1495222,"ticker":"OXLCN","title":"Oxford Lane Capital Corp."},"8557":{"cik_str":1495222,"ticker":"OXLCP","title":"Oxford Lane Capital Corp."},"8558":{"cik_str":1495222,"ticker":"OXLCZ","title":"Oxford Lane Capital Corp."},"8559":{"cik_str":1287032,"ticker":"PSEC-PA","title":"PROSPECT CAPITAL CORP"},"8560":{"cik_str":1099219,"ticker":"MET-PF","title":"METLIFE INC"},"8561":{"cik_str":1408534,"ticker":"FGBIP","title":"First Guaranty Bancshares, Inc."},"8562":{"cik_str":1260729,"ticker":"GDV-PK","title":"GABELLI DIVIDEND & INCOME TRUST"},"8563":{"cik_str":1276187,"ticker":"ET-PE","title":"Energy Transfer LP"},"8564":{"cik_str":1276187,"ticker":"ET-PD","title":"Energy Transfer LP"},"8565":{"cik_str":1276187,"ticker":"ET-PC","title":"Energy Transfer LP"},"8566":{"cik_str":1297996,"ticker":"DLR-PL","title":"DIGITAL REALTY TRUST, INC."},"8567":{"cik_str":1212545,"ticker":"WAL-PA","title":"WESTERN ALLIANCE BANCORPORATION"},"8568":{"cik_str":1629019,"ticker":"MBINO","title":"Merchants Bancorp"},"8569":{"cik_str":1629019,"ticker":"MBINN","title":"Merchants Bancorp"},"8570":{"cik_str":1629019,"ticker":"MBINM","title":"Merchants Bancorp"},"8571":{"cik_str":1847986,"ticker":"DFLIW","title":"Dragonfly Energy Holdings Corp."},"8572":{"cik_str":1635088,"ticker":"ROIVW","title":"Roivant Sciences Ltd."},"8573":{"cik_str":1840199,"ticker":"WALDW","title":"Waldencast plc"},"8574":{"cik_str":1816233,"ticker":"SHCRW","title":"Sharecare, Inc."},"8575":{"cik_str":1898795,"ticker":"LVWR-WT","title":"LiveWire Group, Inc."},"8576":{"cik_str":1836875,"ticker":"NVVEW","title":"Nuvve Holding Corp."},"8577":{"cik_str":845877,"ticker":"AGM-PF","title":"FEDERAL AGRICULTURAL MORTGAGE CORP"},"8578":{"cik_str":845877,"ticker":"AGM-PE","title":"FEDERAL AGRICULTURAL MORTGAGE CORP"},"8579":{"cik_str":845877,"ticker":"AGM-PG","title":"FEDERAL AGRICULTURAL MORTGAGE CORP"},"8580":{"cik_str":927971,"ticker":"BULZ","title":"BANK OF MONTREAL /CAN/"},"8581":{"cik_str":927971,"ticker":"FLYD","title":"BANK OF MONTREAL /CAN/"},"8582":{"cik_str":927971,"ticker":"FNGS","title":"BANK OF MONTREAL /CAN/"},"8583":{"cik_str":927971,"ticker":"WTIU","title":"BANK OF MONTREAL /CAN/"},"8584":{"cik_str":927971,"ticker":"FLYU","title":"BANK OF MONTREAL /CAN/"},"8585":{"cik_str":927971,"ticker":"NRGD","title":"BANK OF MONTREAL /CAN/"},"8586":{"cik_str":927971,"ticker":"BNKU","title":"BANK OF MONTREAL /CAN/"},"8587":{"cik_str":927971,"ticker":"FNGO","title":"BANK OF MONTREAL /CAN/"},"8588":{"cik_str":927971,"ticker":"NRGU","title":"BANK OF MONTREAL /CAN/"},"8589":{"cik_str":927971,"ticker":"GDXD","title":"BANK OF MONTREAL /CAN/"},"8590":{"cik_str":927971,"ticker":"GDXU","title":"BANK OF MONTREAL /CAN/"},"8591":{"cik_str":927971,"ticker":"BNKD","title":"BANK OF MONTREAL /CAN/"},"8592":{"cik_str":927971,"ticker":"WTID","title":"BANK OF MONTREAL /CAN/"},"8593":{"cik_str":927971,"ticker":"OILD","title":"BANK OF MONTREAL /CAN/"},"8594":{"cik_str":927971,"ticker":"FNGU","title":"BANK OF MONTREAL /CAN/"},"8595":{"cik_str":927971,"ticker":"SHNY","title":"BANK OF MONTREAL /CAN/"},"8596":{"cik_str":1820144,"ticker":"GRND-WT","title":"Grindr Inc."},"8597":{"cik_str":1788841,"ticker":"MCOMW","title":"micromobility.com Inc."},"8598":{"cik_str":1837493,"ticker":"IINNW","title":"Inspira Technologies OXY B.H.N. Ltd"},"8599":{"cik_str":1840502,"ticker":"TBLAW","title":"Taboola.com Ltd."},"8600":{"cik_str":1831481,"ticker":"FLME-WT","title":"Flame Acquisition Corp."},"8601":{"cik_str":1822966,"ticker":"SMR-WT","title":"NUSCALE POWER Corp"},"8602":{"cik_str":1402328,"ticker":"SBFMW","title":"Sunshine Biopharma, Inc"},"8603":{"cik_str":1802749,"ticker":"ZEV-WT","title":"Lightning eMotors, Inc."},"8604":{"cik_str":1855756,"ticker":"LILMW","title":"Lilium N.V."},"8605":{"cik_str":1814140,"ticker":"APGNW","title":"Apexigen, Inc."},"8606":{"cik_str":1833835,"ticker":"PSFE-WT","title":"Paysafe Ltd"},"8607":{"cik_str":1814215,"ticker":"BURU-WT","title":"Nuburu, Inc."},"8608":{"cik_str":1826397,"ticker":"AGRIW","title":"AGRIFORCE GROWING SYSTEMS LTD."},"8609":{"cik_str":1468492,"ticker":"HSCSW","title":"Heart Test Laboratories, Inc."},"8610":{"cik_str":1254699,"ticker":"QVCC","title":"QVC INC"},"8611":{"cik_str":1224608,"ticker":"CNO-PA","title":"CNO Financial Group, Inc."},"8612":{"cik_str":1503290,"ticker":"ACP-PA","title":"abrdn Income Credit Strategies Fund"},"8613":{"cik_str":1730346,"ticker":"CHRB","title":"Charah Solutions, Inc."},"8614":{"cik_str":1512228,"ticker":"NIOBW","title":"NIOCORP DEVELOPMENTS LTD"},"8615":{"cik_str":1679063,"ticker":"CSSEN","title":"Chicken Soup for the Soul Entertainment, Inc."},"8616":{"cik_str":1556593,"ticker":"RITM-PA","title":"Rithm Capital Corp."},"8617":{"cik_str":1556593,"ticker":"RITM-PD","title":"Rithm Capital Corp."},"8618":{"cik_str":1556593,"ticker":"RITM-PB","title":"Rithm Capital Corp."},"8619":{"cik_str":1556593,"ticker":"RITM-PC","title":"Rithm Capital Corp."},"8620":{"cik_str":1475260,"ticker":"CVE-WT","title":"CENOVUS ENERGY INC."},"8621":{"cik_str":930236,"ticker":"RWT-PA","title":"REDWOOD TRUST INC"},"8622":{"cik_str":1043219,"ticker":"NLY-PI","title":"ANNALY CAPITAL MANAGEMENT INC"},"8623":{"cik_str":947484,"ticker":"ACGLN","title":"ARCH CAPITAL GROUP LTD."},"8624":{"cik_str":1070050,"ticker":"APCXW","title":"AppTech Payments Corp."},"8625":{"cik_str":1084267,"ticker":"MOBQW","title":"Mobiquity Technologies, Inc."},"8626":{"cik_str":1769624,"ticker":"AGBAW","title":"AGBA Group Holding Ltd."},"8627":{"cik_str":1905660,"ticker":"HUBCW","title":"Hub Cyber Security Ltd."},"8628":{"cik_str":1819810,"ticker":"RDW-WT","title":"Redwire Corp"},"8629":{"cik_str":1825024,"ticker":"OPAD-WT","title":"Offerpad Solutions Inc."},"8630":{"cik_str":1760764,"ticker":"GMVDW","title":"G Medical Innovations Holdings Ltd."},"8631":{"cik_str":1844981,"ticker":"APACW","title":"StoneBridge Acquisition Corp."},"8632":{"cik_str":1812173,"ticker":"RBOT-WT","title":"Vicarious Surgical Inc."},"8633":{"cik_str":1876581,"ticker":"IMPPP","title":"Imperial Petroleum Inc./Marshall Islands"},"8634":{"cik_str":1940674,"ticker":"SMXWW","title":"SMX (Security Matters) Public Ltd Co"},"8635":{"cik_str":1790665,"ticker":"GNRSW","title":"Greenrose Holding Co Inc."},"8636":{"cik_str":1822886,"ticker":"HHGCW","title":"HHG Capital Corp"},"8637":{"cik_str":1842356,"ticker":"PETWW","title":"Wag! Group Co."},"8638":{"cik_str":1877333,"ticker":"THCHW","title":"TH International Ltd"},"8639":{"cik_str":1843477,"ticker":"SVIIU","title":"Spring Valley Acquisition Corp. II"},"8640":{"cik_str":1855474,"ticker":"XPDBW","title":"Power & Digital Infrastructure Acquisition II Corp."},"8641":{"cik_str":1837607,"ticker":"PMGMW","title":"Priveterra Acquisition Corp."},"8642":{"cik_str":1042046,"ticker":"AFGE","title":"AMERICAN FINANCIAL GROUP INC"},"8643":{"cik_str":1042046,"ticker":"AFGD","title":"AMERICAN FINANCIAL GROUP INC"},"8644":{"cik_str":1042046,"ticker":"AFGB","title":"AMERICAN FINANCIAL GROUP INC"},"8645":{"cik_str":801337,"ticker":"WBS-PG","title":"WEBSTER FINANCIAL CORP"},"8646":{"cik_str":948320,"ticker":"LFMDP","title":"LifeMD, Inc."},"8647":{"cik_str":763901,"ticker":"BPOPM","title":"POPULAR, INC."},"8648":{"cik_str":870826,"ticker":"IMBIL","title":"iMedia Brands, Inc."},"8649":{"cik_str":913144,"ticker":"RNR-PG","title":"RENAISSANCERE HOLDINGS LTD"},"8650":{"cik_str":1411342,"ticker":"EFC-PA","title":"Ellington Financial Inc."},"8651":{"cik_str":1411342,"ticker":"EFC-PB","title":"Ellington Financial Inc."},"8652":{"cik_str":1411342,"ticker":"EFC-PC","title":"Ellington Financial Inc."},"8653":{"cik_str":1308106,"ticker":"SEAL-PA","title":"Seapeak LLC"},"8654":{"cik_str":1378701,"ticker":"GDL-PC","title":"GDL FUND"},"8655":{"cik_str":1850262,"ticker":"INTEW","title":"Integral Acquisition Corp 1"},"8656":{"cik_str":1620179,"ticker":"XELAP","title":"Exela Technologies, Inc."},"8657":{"cik_str":1814287,"ticker":"ERESW","title":"East Resources Acquisition Co"},"8658":{"cik_str":1899287,"ticker":"AMPX-WT","title":"Amprius Technologies, Inc."},"8659":{"cik_str":1836981,"ticker":"BBAI-WT","title":"BigBear.ai Holdings, Inc."},"8660":{"cik_str":1785424,"ticker":"KPLTW","title":"Katapult Holdings, Inc."},"8661":{"cik_str":1805077,"ticker":"EOSEW","title":"Eos Energy Enterprises, Inc."},"8662":{"cik_str":1640043,"ticker":"PXSAP","title":"Pyxis Tankers Inc."},"8663":{"cik_str":1640043,"ticker":"PXSAW","title":"Pyxis Tankers Inc."},"8664":{"cik_str":1780312,"ticker":"ASTSW","title":"AST SpaceMobile, Inc."},"8665":{"cik_str":1898416,"ticker":"ALVOW","title":"Alvotech"},"8666":{"cik_str":1856653,"ticker":"BIOSU","title":"BioPlus Acquisition Corp."},"8667":{"cik_str":1834026,"ticker":"GROY-WT","title":"Gold Royalty Corp."},"8668":{"cik_str":72971,"ticker":"WFC-PA","title":"WELLS FARGO & COMPANY/MN"},"8669":{"cik_str":72971,"ticker":"WFC-PD","title":"WELLS FARGO & COMPANY/MN"},"8670":{"cik_str":28917,"ticker":"DDT","title":"DILLARD'S, INC."},"8671":{"cik_str":1823896,"ticker":"GFX-WT","title":"Golden Falcon Acquisition Corp."},"8672":{"cik_str":1830081,"ticker":"RUMBW","title":"Rumble Inc."},"8673":{"cik_str":1800347,"ticker":"ETWO-WT","title":"E2open Parent Holdings, Inc."},"8674":{"cik_str":1879248,"ticker":"ADSEW","title":"Ads-Tec Energy Public Ltd Co"},"8675":{"cik_str":1849635,"ticker":"DWACW","title":"Digital World Acquisition Corp."},"8676":{"cik_str":1849635,"ticker":"DWACU","title":"Digital World Acquisition Corp."},"8677":{"cik_str":1822993,"ticker":"JXN-PA","title":"Jackson Financial Inc."},"8678":{"cik_str":1836833,"ticker":"PL-WT","title":"Planet Labs PBC"},"8679":{"cik_str":1842566,"ticker":"BYTSW","title":"BYTE Acquisition Corp."},"8680":{"cik_str":1321741,"ticker":"GAINN","title":"GLADSTONE INVESTMENT CORPORATION\\DE"},"8681":{"cik_str":1377936,"ticker":"SAJ","title":"SARATOGA INVESTMENT CORP."},"8682":{"cik_str":1377936,"ticker":"SAY","title":"SARATOGA INVESTMENT CORP."},"8683":{"cik_str":1377936,"ticker":"SAT","title":"SARATOGA INVESTMENT CORP."},"8684":{"cik_str":1820875,"ticker":"CXAIW","title":"CXApp Inc."},"8685":{"cik_str":1587987,"ticker":"NEWTZ","title":"NewtekOne, Inc."},"8686":{"cik_str":1587987,"ticker":"NEWTL","title":"NewtekOne, Inc."},"8687":{"cik_str":917251,"ticker":"ADC-PA","title":"AGREE REALTY CORP"},"8688":{"cik_str":1015328,"ticker":"WTFCP","title":"WINTRUST FINANCIAL CORP"},"8689":{"cik_str":314808,"ticker":"VAL-WT","title":"Valaris Ltd"},"8690":{"cik_str":895421,"ticker":"MS-PP","title":"MORGAN STANLEY"},"8691":{"cik_str":895421,"ticker":"MS-PL","title":"MORGAN STANLEY"},"8692":{"cik_str":823277,"ticker":"CHSCM","title":"CHS INC"},"8693":{"cik_str":823277,"ticker":"CHSCN","title":"CHS INC"},"8694":{"cik_str":823277,"ticker":"CHSCO","title":"CHS INC"},"8695":{"cik_str":895421,"ticker":"MS-PO","title":"MORGAN STANLEY"},"8696":{"cik_str":64463,"ticker":"SLNHP","title":"Soluna Holdings, Inc"},"8697":{"cik_str":59558,"ticker":"LNC-PD","title":"LINCOLN NATIONAL CORP"},"8698":{"cik_str":6176,"ticker":"AP-WT","title":"AMPCO PITTSBURGH CORP"},"8699":{"cik_str":49196,"ticker":"HBANM","title":"HUNTINGTON BANCSHARES INC /MD/"},"8700":{"cik_str":49196,"ticker":"HBANP","title":"HUNTINGTON BANCSHARES INC /MD/"},"8701":{"cik_str":1004980,"ticker":"PCGU","title":"PG&E Corp"},"8702":{"cik_str":1790625,"ticker":"AGILW","title":"AgileThought, Inc."},"8703":{"cik_str":46195,"ticker":"BOH-PA","title":"BANK OF HAWAII CORP"},"8704":{"cik_str":77281,"ticker":"PRETL","title":"PENNSYLVANIA REAL ESTATE INVESTMENT TRUST"},"8705":{"cik_str":77281,"ticker":"PRETM","title":"PENNSYLVANIA REAL ESTATE INVESTMENT TRUST"},"8706":{"cik_str":92103,"ticker":"SCE-PK","title":"SOUTHERN CALIFORNIA EDISON Co"},"8707":{"cik_str":92103,"ticker":"SCE-PL","title":"SOUTHERN CALIFORNIA EDISON Co"},"8708":{"cik_str":92230,"ticker":"TFC-PI","title":"TRUIST FINANCIAL CORP"},"8709":{"cik_str":92230,"ticker":"TFC-PO","title":"TRUIST FINANCIAL CORP"},"8710":{"cik_str":92230,"ticker":"TFC-PR","title":"TRUIST FINANCIAL CORP"},"8711":{"cik_str":759944,"ticker":"CFG-PE","title":"CITIZENS FINANCIAL GROUP INC/RI"},"8712":{"cik_str":1682220,"ticker":"SCCF","title":"Sachem Capital Corp."},"8713":{"cik_str":1682220,"ticker":"SCCB","title":"Sachem Capital Corp."},"8714":{"cik_str":1682220,"ticker":"SCCD","title":"Sachem Capital Corp."},"8715":{"cik_str":1682220,"ticker":"SCCG","title":"Sachem Capital Corp."},"8716":{"cik_str":1682220,"ticker":"SCCC","title":"Sachem Capital Corp."},"8717":{"cik_str":1855555,"ticker":"ENERR","title":"ACCRETION ACQUISITION CORP."},"8718":{"cik_str":1830210,"ticker":"BHIL-WT","title":"Benson Hill, Inc."},"8719":{"cik_str":1817640,"ticker":"BREZW","title":"Breeze Holdings Acquisition Corp."},"8720":{"cik_str":1853070,"ticker":"PACI-WT","title":"PROOF Acquisition Corp I"},"8721":{"cik_str":1845368,"ticker":"HCMAW","title":"HCM Acquisition Corp"},"8722":{"cik_str":1927719,"ticker":"CRGOW","title":"Freightos Ltd"},"8723":{"cik_str":1836274,"ticker":"ACAHW","title":"Atlantic Coastal Acquisition Corp."},"8724":{"cik_str":1856589,"ticker":"MBSC-UN","title":"M3-Brigade Acquisition III Corp."},"8725":{"cik_str":1843973,"ticker":"EGGF-WT","title":"EG Acquisition Corp."},"8726":{"cik_str":1533232,"ticker":"BEP-PA","title":"Brookfield Renewable Partners L.P."},"8727":{"cik_str":1415311,"ticker":"VIXY","title":"ProShares Trust II"},"8728":{"cik_str":1415311,"ticker":"ZSL","title":"ProShares Trust II"},"8729":{"cik_str":1415311,"ticker":"UVXY","title":"ProShares Trust II"},"8730":{"cik_str":1415311,"ticker":"GLL","title":"ProShares Trust II"},"8731":{"cik_str":1415311,"ticker":"SVXY","title":"ProShares Trust II"},"8732":{"cik_str":1415311,"ticker":"BOIL","title":"ProShares Trust II"},"8733":{"cik_str":1415311,"ticker":"KOLD","title":"ProShares Trust II"},"8734":{"cik_str":1415311,"ticker":"SCO","title":"ProShares Trust II"},"8735":{"cik_str":1415311,"ticker":"YCL","title":"ProShares Trust II"},"8736":{"cik_str":1415311,"ticker":"EUO","title":"ProShares Trust II"},"8737":{"cik_str":1415311,"ticker":"UCO","title":"ProShares Trust II"},"8738":{"cik_str":1415311,"ticker":"ULE","title":"ProShares Trust II"},"8739":{"cik_str":1415311,"ticker":"AGQ","title":"ProShares Trust II"},"8740":{"cik_str":1415311,"ticker":"VIXM","title":"ProShares Trust II"},"8741":{"cik_str":1335105,"ticker":"LIXTW","title":"LIXTE BIOTECHNOLOGY HOLDINGS, INC."},"8742":{"cik_str":1464343,"ticker":"ATLCP","title":"Atlanticus Holdings Corp"},"8743":{"cik_str":1464343,"ticker":"ATLCL","title":"Atlanticus Holdings Corp"},"8744":{"cik_str":707388,"ticker":"STRRP","title":"STAR EQUITY HOLDINGS, INC."},"8745":{"cik_str":19617,"ticker":"JPM-PK","title":"JPMORGAN CHASE & CO"},"8746":{"cik_str":1710680,"ticker":"HFRO-PA","title":"HIGHLAND INCOME FUND\\MA"},"8747":{"cik_str":1815086,"ticker":"BTWNW","title":"Bridgetown Holdings Ltd"},"8748":{"cik_str":19617,"ticker":"AMJ","title":"JPMORGAN CHASE & CO"},"8749":{"cik_str":19617,"ticker":"JPM-PL","title":"JPMORGAN CHASE & CO"},"8750":{"cik_str":1854458,"ticker":"NETC-WT","title":"Nabors Energy Transition Corp."},"8751":{"cik_str":1816581,"ticker":"OUST-WTA","title":"Ouster, Inc."},"8752":{"cik_str":1816581,"ticker":"OUST-WT","title":"Ouster, Inc."},"8753":{"cik_str":1799983,"ticker":"GB-WT","title":"Global Blue Group Holding AG"},"8754":{"cik_str":1844389,"ticker":"ACBAU","title":"Ace Global Business Acquisition Ltd"},"8755":{"cik_str":1870143,"ticker":"RCFA-WT","title":"RCF Acquisition Corp."},"8756":{"cik_str":1909417,"ticker":"SLNAW","title":"SELINA HOSPITALITY PLC"},"8757":{"cik_str":1861121,"ticker":"EVE-WT","title":"EVe Mobility Acquisition Corp"},"8758":{"cik_str":1807707,"ticker":"APPHW","title":"AppHarvest, Inc."},"8759":{"cik_str":1953530,"ticker":"OCSAW","title":"Oculis Holding AG"},"8760":{"cik_str":1819848,"ticker":"JOBY-WT","title":"Joby Aviation, Inc."},"8761":{"cik_str":1630805,"ticker":"BWSN","title":"Babcock & Wilcox Enterprises, Inc."},"8762":{"cik_str":1464790,"ticker":"RILYN","title":"B. Riley Financial, Inc."},"8763":{"cik_str":1464790,"ticker":"RILYM","title":"B. Riley Financial, Inc."},"8764":{"cik_str":1464790,"ticker":"RILYP","title":"B. Riley Financial, Inc."},"8765":{"cik_str":1509589,"ticker":"CIVII","title":"CIVITAS RESOURCES, INC."},"8766":{"cik_str":1509589,"ticker":"CIVIW","title":"CIVITAS RESOURCES, INC."},"8767":{"cik_str":1136174,"ticker":"OTRKP","title":"Ontrak, Inc."},"8768":{"cik_str":1411494,"ticker":"AAM-PA","title":"Apollo Asset Management, Inc."},"8769":{"cik_str":1411494,"ticker":"AAM-PB","title":"Apollo Asset Management, Inc."},"8770":{"cik_str":1413159,"ticker":"TGH-PB","title":"TEXTAINER GROUP HOLDINGS LTD"},"8771":{"cik_str":1464790,"ticker":"RILYT","title":"B. Riley Financial, Inc."},"8772":{"cik_str":1464790,"ticker":"RILYG","title":"B. Riley Financial, Inc."},"8773":{"cik_str":1464790,"ticker":"RILYZ","title":"B. Riley Financial, Inc."},"8774":{"cik_str":1464790,"ticker":"RILYL","title":"B. Riley Financial, Inc."},"8775":{"cik_str":907254,"ticker":"BFS-PE","title":"SAUL CENTERS, INC."},"8776":{"cik_str":1055160,"ticker":"MFA-PC","title":"MFA FINANCIAL, INC."},"8777":{"cik_str":1324948,"ticker":"RBCP","title":"RBC Bearings INC"},"8778":{"cik_str":1829432,"ticker":"AAC-UN","title":"Ares Acquisition Corp"},"8779":{"cik_str":1829432,"ticker":"AAC-WT","title":"Ares Acquisition Corp"},"8780":{"cik_str":1883814,"ticker":"SLND-WT","title":"Southland Holdings, Inc."},"8781":{"cik_str":1865975,"ticker":"AFTR-UN","title":"AfterNext HealthTech Acquisition Corp."},"8782":{"cik_str":1865975,"ticker":"AFTR-WT","title":"AfterNext HealthTech Acquisition Corp."},"8783":{"cik_str":1779128,"ticker":"BLDEW","title":"Blade Air Mobility, Inc."},"8784":{"cik_str":1809987,"ticker":"MIR-WT","title":"Mirion Technologies, Inc."},"8785":{"cik_str":36270,"ticker":"MTB-PH","title":"M&T BANK CORP"},"8786":{"cik_str":732717,"ticker":"TBC","title":"AT&T INC."},"8787":{"cik_str":732717,"ticker":"T-PC","title":"AT&T INC."},"8788":{"cik_str":230557,"ticker":"SIGIP","title":"SELECTIVE INSURANCE GROUP INC"},"8789":{"cik_str":732717,"ticker":"T-PA","title":"AT&T INC."},"8790":{"cik_str":1776661,"ticker":"ADVWW","title":"Advantage Solutions Inc."},"8791":{"cik_str":1789029,"ticker":"AEVA-WT","title":"Aeva Technologies, Inc."},"8792":{"cik_str":1818502,"ticker":"OPFI-WT","title":"OppFi Inc."},"8793":{"cik_str":1539638,"ticker":"TFINP","title":"Triumph Financial, Inc."},"8794":{"cik_str":1874315,"ticker":"SATLW","title":"Satellogic Inc."},"8795":{"cik_str":1273685,"ticker":"NYMTZ","title":"NEW YORK MORTGAGE TRUST INC"},"8796":{"cik_str":1273685,"ticker":"NYMTM","title":"NEW YORK MORTGAGE TRUST INC"},"8797":{"cik_str":1273685,"ticker":"NYMTL","title":"NEW YORK MORTGAGE TRUST INC"},"8798":{"cik_str":1464423,"ticker":"PMT-PC","title":"PennyMac Mortgage Investment Trust"},"8799":{"cik_str":1527590,"ticker":"RCB","title":"Ready Capital Corp"},"8800":{"cik_str":1393311,"ticker":"PSA-PR","title":"Public Storage"},"8801":{"cik_str":1393311,"ticker":"PSA-PO","title":"Public Storage"},"8802":{"cik_str":1393311,"ticker":"PSA-PG","title":"Public Storage"},"8803":{"cik_str":1393311,"ticker":"PSA-PQ","title":"Public Storage"},"8804":{"cik_str":1527590,"ticker":"RCC","title":"Ready Capital Corp"},"8805":{"cik_str":1393311,"ticker":"PSA-PN","title":"Public Storage"},"8806":{"cik_str":1393311,"ticker":"PSA-PM","title":"Public Storage"},"8807":{"cik_str":1393311,"ticker":"PSA-PP","title":"Public Storage"},"8808":{"cik_str":1393311,"ticker":"PSA-PL","title":"Public Storage"},"8809":{"cik_str":1393311,"ticker":"PSA-PF","title":"Public Storage"},"8810":{"cik_str":1393311,"ticker":"PSA-PJ","title":"Public Storage"},"8811":{"cik_str":1513525,"ticker":"ADILW","title":"ADIAL PHARMACEUTICALS, INC."},"8812":{"cik_str":1423689,"ticker":"AGNCP","title":"AGNC Investment Corp."},"8813":{"cik_str":1423689,"ticker":"AGNCO","title":"AGNC Investment Corp."},"8814":{"cik_str":1423689,"ticker":"AGNCL","title":"AGNC Investment Corp."},"8815":{"cik_str":1114446,"ticker":"AMUB","title":"UBS AG"},"8816":{"cik_str":1114446,"ticker":"BDCX","title":"UBS AG"},"8817":{"cik_str":1114446,"ticker":"BDCZ","title":"UBS AG"},"8818":{"cik_str":1114446,"ticker":"UCIB","title":"UBS AG"},"8819":{"cik_str":1114446,"ticker":"HDLB","title":"UBS AG"},"8820":{"cik_str":1114446,"ticker":"QULL","title":"UBS AG"},"8821":{"cik_str":1114446,"ticker":"SMHB","title":"UBS AG"},"8822":{"cik_str":1114446,"ticker":"MVRL","title":"UBS AG"},"8823":{"cik_str":1114446,"ticker":"SCDL","title":"UBS AG"},"8824":{"cik_str":1114446,"ticker":"MLPB","title":"UBS AG"},"8825":{"cik_str":1114446,"ticker":"MLPR","title":"UBS AG"},"8826":{"cik_str":1114446,"ticker":"CEFD","title":"UBS AG"},"8827":{"cik_str":1114446,"ticker":"WUCT","title":"UBS AG"},"8828":{"cik_str":1114446,"ticker":"IWML","title":"UBS AG"},"8829":{"cik_str":1114446,"ticker":"IFED","title":"UBS AG"},"8830":{"cik_str":1114446,"ticker":"FBGX","title":"UBS AG"},"8831":{"cik_str":1779372,"ticker":"BEATW","title":"HeartBeam, Inc."},"8832":{"cik_str":91576,"ticker":"KEY-PL","title":"KEYCORP /NEW/"},"8833":{"cik_str":1075415,"ticker":"DHCNL","title":"DIVERSIFIED HEALTHCARE TRUST"},"8834":{"cik_str":925741,"ticker":"BCDAW","title":"BioCardia, Inc."},"8835":{"cik_str":1582138,"ticker":"EQCDX","title":"Equalize Community Development Fund"},"8836":{"cik_str":1675033,"ticker":"GECCN","title":"Great Elm Capital Corp."},"8837":{"cik_str":1675033,"ticker":"GECCO","title":"Great Elm Capital Corp."},"8838":{"cik_str":1675033,"ticker":"GECCM","title":"Great Elm Capital Corp."},"8839":{"cik_str":1824734,"ticker":"BGRYW","title":"Berkshire Grey, Inc."},"8840":{"cik_str":1820190,"ticker":"SCLXW","title":"Scilex Holding Co"},"8841":{"cik_str":1545772,"ticker":"BPYPN","title":"Brookfield Property Partners L.P."},"8842":{"cik_str":1545772,"ticker":"BPYPO","title":"Brookfield Property Partners L.P."},"8843":{"cik_str":1722438,"ticker":"DOMA-WT","title":"Doma Holdings, Inc."},"8844":{"cik_str":1487428,"ticker":"HTFC","title":"Horizon Technology Finance Corp"},"8845":{"cik_str":1895262,"ticker":"NBLWF","title":"Noble Corp plc"},"8846":{"cik_str":1721386,"ticker":"LSEAW","title":"Landsea Homes Corp"},"8847":{"cik_str":1832765,"ticker":"EACPW","title":"Edify Acquisition Corp."},"8848":{"cik_str":1831270,"ticker":"BITE-WT","title":"Bite Acquisition Corp."},"8849":{"cik_str":1819438,"ticker":"GWH-WT","title":"ESS Tech, Inc."},"8850":{"cik_str":1834755,"ticker":"SLACU","title":"Social Leverage Acquisition Corp I"},"8851":{"cik_str":1834755,"ticker":"SLACW","title":"Social Leverage Acquisition Corp I"},"8852":{"cik_str":1703079,"ticker":"XFLT-PA","title":"XAI Octagon Floating Rate & Alternative Income Term Trust"},"8853":{"cik_str":1871321,"ticker":"DRTSW","title":"Alpha Tau Medical Ltd."},"8854":{"cik_str":1945711,"ticker":"LVROW","title":"Lavoro Ltd"},"8855":{"cik_str":4281,"ticker":"HWM-P","title":"Howmet Aerospace Inc."},"8856":{"cik_str":1874474,"ticker":"ALLG-WT","title":"Allego N.V."},"8857":{"cik_str":1823857,"ticker":"VHAQ-RT","title":"Viveon Health Acquisition Corp."},"8858":{"cik_str":1823857,"ticker":"VHAQ-WT","title":"Viveon Health Acquisition Corp."},"8859":{"cik_str":1137774,"ticker":"PRH","title":"PRUDENTIAL FINANCIAL INC"},"8860":{"cik_str":1137774,"ticker":"PRS","title":"PRUDENTIAL FINANCIAL INC"},"8861":{"cik_str":794685,"ticker":"GAB-PK","title":"GABELLI EQUITY TRUST INC"},"8862":{"cik_str":1471824,"ticker":"WEAT","title":"Teucrium Commodity Trust"},"8863":{"cik_str":1471824,"ticker":"SOYB","title":"Teucrium Commodity Trust"},"8864":{"cik_str":1471824,"ticker":"DEFI","title":"Teucrium Commodity Trust"},"8865":{"cik_str":1471824,"ticker":"CANE","title":"Teucrium Commodity Trust"},"8866":{"cik_str":1471824,"ticker":"TAGS","title":"Teucrium Commodity Trust"},"8867":{"cik_str":1471824,"ticker":"CORN","title":"Teucrium Commodity Trust"},"8868":{"cik_str":1502292,"ticker":"CNFRL","title":"Conifer Holdings, Inc."},"8869":{"cik_str":1851612,"ticker":"BHACW","title":"Crixus BH3 Acquisition Co"},"8870":{"cik_str":1819493,"ticker":"XOSWW","title":"Xos, Inc."},"8871":{"cik_str":1885408,"ticker":"JFBRW","title":"Jeffs' Brands Ltd"},"8872":{"cik_str":1863181,"ticker":"HPLTW","title":"Home Plate Acquisition Corp"},"8873":{"cik_str":1648960,"ticker":"DATSW","title":"DatChat, Inc."},"8874":{"cik_str":1723580,"ticker":"BFIIW","title":"BurgerFi International, Inc."},"8875":{"cik_str":1699880,"ticker":"AMRLF","title":"American Lithium Corp."},"8876":{"cik_str":1808377,"ticker":"LUCYW","title":"Innovative Eyewear Inc"},"8877":{"cik_str":1777946,"ticker":"IRNT-WT","title":"IronNet, Inc."},"8878":{"cik_str":1590364,"ticker":"FTAIN","title":"FTAI Aviation Ltd."},"8879":{"cik_str":1590364,"ticker":"FTAIP","title":"FTAI Aviation Ltd."},"8880":{"cik_str":1590364,"ticker":"FTAIM","title":"FTAI Aviation Ltd."},"8881":{"cik_str":1590364,"ticker":"FTAIO","title":"FTAI Aviation Ltd."},"8882":{"cik_str":1796514,"ticker":"METXW","title":"Meten Holding Group Ltd."},"8883":{"cik_str":1603016,"ticker":"HMLPF","title":"Hoegh LNG Partners LP"},"8884":{"cik_str":1865697,"ticker":"GGAAW","title":"Genesis Growth Tech Acquisition Corp."},"8885":{"cik_str":1783398,"ticker":"UWMC-WT","title":"UWM Holdings Corp"},"8886":{"cik_str":1818644,"ticker":"LIDRW","title":"AEye, Inc."},"8887":{"cik_str":1871983,"ticker":"ANGHW","title":"Anghami Inc"},"8888":{"cik_str":1610940,"ticker":"BDRY","title":"ETF Managers Group Commodity Trust I"},"8889":{"cik_str":1900402,"ticker":"EVGRW","title":"Evergreen Corp"},"8890":{"cik_str":897802,"ticker":"SPE-PC","title":"SPECIAL OPPORTUNITIES FUND, INC."},"8891":{"cik_str":911421,"ticker":"PRE-PJ","title":"PARTNERRE LTD"},"8892":{"cik_str":860748,"ticker":"KMPB","title":"KEMPER Corp"},"8893":{"cik_str":707179,"ticker":"ONBPP","title":"OLD NATIONAL BANCORP /IN/"},"8894":{"cik_str":1753539,"ticker":"BKSY-WT","title":"BlackSky Technology Inc."},"8895":{"cik_str":1633336,"ticker":"FCRX","title":"Crescent Capital BDC, Inc."},"8896":{"cik_str":1805833,"ticker":"SST-WT","title":"System1, Inc."},"8897":{"cik_str":1830033,"ticker":"PCTTW","title":"PureCycle Technologies, Inc."},"8898":{"cik_str":1847440,"ticker":"MITAU","title":"Coliseum Acquisition Corp."},"8899":{"cik_str":1868573,"ticker":"APXIW","title":"APx Acquisition Corp. I"},"8900":{"cik_str":1825473,"ticker":"PRSRU","title":"Prospector Capital Corp."},"8901":{"cik_str":1825473,"ticker":"PRSRW","title":"Prospector Capital Corp."},"8902":{"cik_str":1839132,"ticker":"MVLAW","title":"Movella Holdings Inc."},"8903":{"cik_str":1355096,"ticker":"QRTEP","title":"Qurate Retail, Inc."},"8904":{"cik_str":1209028,"ticker":"AAIC-PB","title":"Arlington Asset Investment Corp."},"8905":{"cik_str":1209028,"ticker":"AAIC-PC","title":"Arlington Asset Investment Corp."},"8906":{"cik_str":1209028,"ticker":"AAIN","title":"Arlington Asset Investment Corp."},"8907":{"cik_str":1267238,"ticker":"AIZN","title":"ASSURANT, INC."},"8908":{"cik_str":760498,"ticker":"BANFP","title":"BANCFIRST CORP /OK/"},"8909":{"cik_str":719739,"ticker":"SIVPQ","title":"SVB FINANCIAL GROUP"},"8910":{"cik_str":720005,"ticker":"RJF-PB","title":"RAYMOND JAMES FINANCIAL INC"},"8911":{"cik_str":811156,"ticker":"CMSD","title":"CMS ENERGY CORP"},"8912":{"cik_str":811156,"ticker":"CMS-PC","title":"CMS ENERGY CORP"},"8913":{"cik_str":1932737,"ticker":"NWTNW","title":"NWTN, Inc."},"8914":{"cik_str":35527,"ticker":"FITBP","title":"FIFTH THIRD BANCORP"},"8915":{"cik_str":35527,"ticker":"FITBO","title":"FIFTH THIRD BANCORP"},"8916":{"cik_str":1833214,"ticker":"SABSW","title":"SAB Biotherapeutics, Inc."},"8917":{"cik_str":1862327,"ticker":"NFNT-WT","title":"Infinite Acquisition Corp."},"8918":{"cik_str":1821850,"ticker":"SUNL-WT","title":"Sunlight Financial Holdings Inc."},"8919":{"cik_str":1806524,"ticker":"LGHLW","title":"Lion Group Holding Ltd"},"8920":{"cik_str":1840877,"ticker":"ANZUW","title":"Anzu Special Acquisition Corp I"},"8921":{"cik_str":1893219,"ticker":"ACABW","title":"Atlantic Coastal Acquisition Corp. II"},"8922":{"cik_str":1526113,"ticker":"GNL-PB","title":"Global Net Lease, Inc."},"8923":{"cik_str":1576018,"ticker":"SPNT-PB","title":"SiriusPoint Ltd"},"8924":{"cik_str":1618500,"ticker":"MDGSW","title":"Medigus Ltd."},"8925":{"cik_str":1822912,"ticker":"ADERW","title":"26 Capital Acquisition Corp."},"8926":{"cik_str":1828108,"ticker":"AUROW","title":"Aurora Innovation, Inc."},"8927":{"cik_str":1806347,"ticker":"WESTW","title":"Westrock Coffee Co"},"8928":{"cik_str":1514281,"ticker":"MITT-PC","title":"AG Mortgage Investment Trust, Inc."},"8929":{"cik_str":1835814,"ticker":"NSTD-WT","title":"Northern Star Investment Corp. IV"},"8930":{"cik_str":1872964,"ticker":"MTEKW","title":"Maris Tech Ltd."},"8931":{"cik_str":1810560,"ticker":"REVBW","title":"REVELATION BIOSCIENCES, INC."},"8932":{"cik_str":1837248,"ticker":"SDACW","title":"Sustainable Development Acquisition I Corp."},"8933":{"cik_str":1668370,"ticker":"TBLTW","title":"Toughbuilt Industries, Inc"},"8934":{"cik_str":1861449,"ticker":"BRDS-WT","title":"Bird Global, Inc."},"8935":{"cik_str":11544,"ticker":"WRB-PF","title":"BERKLEY W R CORP"},"8936":{"cik_str":11544,"ticker":"WRB-PH","title":"BERKLEY W R CORP"},"8937":{"cik_str":712771,"ticker":"CNOBP","title":"ConnectOne Bancorp, Inc."},"8938":{"cik_str":798941,"ticker":"FCNCP","title":"FIRST CITIZENS BANCSHARES INC /DE/"},"8939":{"cik_str":883948,"ticker":"AUB-PA","title":"Atlantic Union Bankshares Corp"},"8940":{"cik_str":866368,"ticker":"PSBZP","title":"PS BUSINESS PARKS, INC./MD"},"8941":{"cik_str":866368,"ticker":"PSBXP","title":"PS BUSINESS PARKS, INC./MD"},"8942":{"cik_str":866368,"ticker":"PSBYP","title":"PS BUSINESS PARKS, INC./MD"},"8943":{"cik_str":1828522,"ticker":"EFTRW","title":"eFFECTOR Therapeutics, Inc."},"8944":{"cik_str":1852940,"ticker":"GOGN-UN","title":"GoGreen Investments Corp"},"8945":{"cik_str":1738758,"ticker":"GSMGW","title":"GLORY STAR NEW MEDIA GROUP HOLDINGS Ltd"},"8946":{"cik_str":1849580,"ticker":"WAVC-WT","title":"Waverley Capital Acquisition Corp. 1"},"8947":{"cik_str":1831096,"ticker":"GEGGL","title":"Great Elm Group, Inc."},"8948":{"cik_str":1845618,"ticker":"TWCBW","title":"Bilander Acquisition Corp."},"8949":{"cik_str":1845618,"ticker":"TWCBU","title":"Bilander Acquisition Corp."},"8950":{"cik_str":1829247,"ticker":"BFRGW","title":"BullFrog AI Holdings, Inc."},"8951":{"cik_str":1852940,"ticker":"GOGN-WT","title":"GoGreen Investments Corp"},"8952":{"cik_str":1691421,"ticker":"LMND-WT","title":"Lemonade, Inc."},"8953":{"cik_str":1000209,"ticker":"MBNKP","title":"MEDALLION FINANCIAL CORP"},"8954":{"cik_str":899051,"ticker":"ALL-PI","title":"ALLSTATE CORP"},"8955":{"cik_str":1077428,"ticker":"TCBIO","title":"TEXAS CAPITAL BANCSHARES INC/TX"},"8956":{"cik_str":1029800,"ticker":"UBP-PK","title":"URSTADT BIDDLE PROPERTIES INC"},"8957":{"cik_str":1034957,"ticker":"CRESW","title":"CRESUD INC"},"8958":{"cik_str":1039828,"ticker":"AEL-PB","title":"AMERICAN EQUITY INVESTMENT LIFE HOLDING CO"},"8959":{"cik_str":1039828,"ticker":"AEL-PA","title":"AMERICAN EQUITY INVESTMENT LIFE HOLDING CO"},"8960":{"cik_str":1295810,"ticker":"SHO-PH","title":"Sunstone Hotel Investors, Inc."},"8961":{"cik_str":1466026,"ticker":"MSBIP","title":"Midland States Bancorp, Inc."},"8962":{"cik_str":1928446,"ticker":"GRNT-WT","title":"Granite Ridge Resources, Inc."},"8963":{"cik_str":1837412,"ticker":"SLGCW","title":"SomaLogic, Inc."},"8964":{"cik_str":1901799,"ticker":"GSRMW","title":"Bitcoin Depot Inc."},"8965":{"cik_str":1742927,"ticker":"RVPHW","title":"REVIVA PHARMACEUTICALS HOLDINGS, INC."},"8966":{"cik_str":1822145,"ticker":"PRSTW","title":"Presto Automation Inc."},"8967":{"cik_str":1853774,"ticker":"MCAFR","title":"Mountain Crest Acquisition Corp. IV"},"8968":{"cik_str":1936702,"ticker":"CETUW","title":"Cetus Capital Acquisition Corp."},"8969":{"cik_str":1794846,"ticker":"ATCO-PH","title":"Atlas Corp."},"8970":{"cik_str":1794846,"ticker":"ATCO-PI","title":"Atlas Corp."},"8971":{"cik_str":1794846,"ticker":"ATCOL","title":"Atlas Corp."},"8972":{"cik_str":1813914,"ticker":"CMAXW","title":"CareMax, Inc."},"8973":{"cik_str":1792849,"ticker":"HPKEW","title":"HighPeak Energy, Inc."},"8974":{"cik_str":1719406,"ticker":"NRXPW","title":"NRX Pharmaceuticals, Inc."},"8975":{"cik_str":1866501,"ticker":"WBX-WT","title":"Wallbox N.V."},"8976":{"cik_str":1255474,"ticker":"WLLAW","title":"Whiting Holdings LLC"},"8977":{"cik_str":927628,"ticker":"COF-PK","title":"CAPITAL ONE FINANCIAL CORP"},"8978":{"cik_str":927628,"ticker":"COF-PN","title":"CAPITAL ONE FINANCIAL CORP"},"8979":{"cik_str":927628,"ticker":"COF-PJ","title":"CAPITAL ONE FINANCIAL CORP"},"8980":{"cik_str":1673481,"ticker":"LTRYW","title":"Lottery.com Inc."},"8981":{"cik_str":1571283,"ticker":"REXR-PC","title":"Rexford Industrial Realty, Inc."},"8982":{"cik_str":1863006,"ticker":"VLN-WT","title":"Valens Semiconductor Ltd."},"8983":{"cik_str":1841425,"ticker":"VGASW","title":"Verde Clean Fuels, Inc."},"8984":{"cik_str":1538495,"ticker":"UNOV","title":"Earth Science Tech, Inc."},"8985":{"cik_str":910073,"ticker":"NYCB-PU","title":"NEW YORK COMMUNITY BANCORP INC"},"8986":{"cik_str":1045450,"ticker":"EPR-PC","title":"EPR PROPERTIES"},"8987":{"cik_str":1126956,"ticker":"SRCU","title":"SPIRE INC"},"8988":{"cik_str":1124198,"ticker":"FLRAP","title":"FLUOR CORP"},"8989":{"cik_str":1045450,"ticker":"EPR-PE","title":"EPR PROPERTIES"},"8990":{"cik_str":1561032,"ticker":"HTIBP","title":"Healthcare Trust, Inc."},"8991":{"cik_str":1653384,"ticker":"RWAYZ","title":"Runway Growth Finance Corp."},"8992":{"cik_str":1653384,"ticker":"RWAYL","title":"Runway Growth Finance Corp."},"8993":{"cik_str":1496254,"ticker":"LTCFX","title":"Alternative Strategies Fund"},"8994":{"cik_str":1479247,"ticker":"USCI","title":"United States Commodity Index Funds Trust"},"8995":{"cik_str":1479247,"ticker":"CPER","title":"United States Commodity Index Funds Trust"},"8996":{"cik_str":1360214,"ticker":"HROWM","title":"HARROW HEALTH, INC."},"8997":{"cik_str":1914023,"ticker":"ACACU","title":"Acri Capital Acquisition Corp"},"8998":{"cik_str":1914023,"ticker":"ACACW","title":"Acri Capital Acquisition Corp"},"8999":{"cik_str":1483934,"ticker":"SBBA","title":"Scorpio Tankers Inc."},"9000":{"cik_str":1824502,"ticker":"ACHR-WT","title":"Archer Aviation Inc."},"9001":{"cik_str":98222,"ticker":"TDW-WT","title":"TIDEWATER INC"},"9002":{"cik_str":98222,"ticker":"TDW-WTA","title":"TIDEWATER INC"},"9003":{"cik_str":1716951,"ticker":"OCCIN","title":"OFS Credit Company, Inc."},"9004":{"cik_str":1716951,"ticker":"OCCIO","title":"OFS Credit Company, Inc."},"9005":{"cik_str":1665300,"ticker":"PHUNW","title":"Phunware, Inc."},"9006":{"cik_str":1838615,"ticker":"ALTIW","title":"AlTi Global, Inc."},"9007":{"cik_str":1821812,"ticker":"SCAQW","title":"Stratim Cloud Acquisition Corp."},"9008":{"cik_str":1822791,"ticker":"CLNNW","title":"Clene Inc."},"9009":{"cik_str":1853816,"ticker":"DRMAW","title":"Dermata Therapeutics, Inc."},"9010":{"cik_str":1679688,"ticker":"DBRG-PI","title":"DigitalBridge Group, Inc."},"9011":{"cik_str":1679688,"ticker":"DBRG-PJ","title":"DigitalBridge Group, Inc."},"9012":{"cik_str":1784567,"ticker":"HCDIP","title":"Harbor Custom Development, Inc."},"9013":{"cik_str":1850767,"ticker":"ACUT","title":"Accustem Sciences Inc."},"9014":{"cik_str":1784567,"ticker":"HCDIW","title":"Harbor Custom Development, Inc."},"9015":{"cik_str":1844817,"ticker":"AACIW","title":"Armada Acquisition Corp. I"},"9016":{"cik_str":1843370,"ticker":"BLEUR","title":"bleuacacia ltd"},"9017":{"cik_str":1859807,"ticker":"NVACR","title":"NorthView Acquisition Corp"},"9018":{"cik_str":1759824,"ticker":"ALTG-PA","title":"ALTA EQUIPMENT GROUP INC."},"9019":{"cik_str":1855631,"ticker":"AWINW","title":"AERWINS Technologies Inc."},"9020":{"cik_str":1841925,"ticker":"INDIW","title":"indie Semiconductor, Inc."},"9021":{"cik_str":1812234,"ticker":"CCV-WT","title":"Churchill Capital Corp V"},"9022":{"cik_str":1882078,"ticker":"ARIZW","title":"Arisz Acquisition Corp."},"9023":{"cik_str":1811109,"ticker":"AUVIP","title":"Applied UV, Inc."},"9024":{"cik_str":1831979,"ticker":"GPACW","title":"Global Partner Acquisition Corp II"},"9025":{"cik_str":1844224,"ticker":"FREY-WT","title":"FREYR Battery"},"9026":{"cik_str":883569,"ticker":"FOSLL","title":"Fossil Group, Inc."},"9027":{"cik_str":1281761,"ticker":"RF-PE","title":"REGIONS FINANCIAL CORP"},"9028":{"cik_str":1847241,"ticker":"PEGRW","title":"Project Energy Reimagined Acquisition Corp."},"9029":{"cik_str":18349,"ticker":"SNV-PE","title":"SYNOVUS FINANCIAL CORP"},"9030":{"cik_str":36966,"ticker":"FHN-PE","title":"FIRST HORIZON CORP"},"9031":{"cik_str":36966,"ticker":"FHN-PB","title":"FIRST HORIZON CORP"},"9032":{"cik_str":36966,"ticker":"FHN-PF","title":"FIRST HORIZON CORP"},"9033":{"cik_str":1678130,"ticker":"OPP-PA","title":"RiverNorth/DoubleLine Strategic Opportunity Fund, Inc."},"9034":{"cik_str":1810140,"ticker":"POL-WT","title":"Polished.com Inc."},"9035":{"cik_str":1554625,"ticker":"PRIF-PJ","title":"Priority Income Fund, Inc."},"9036":{"cik_str":1554625,"ticker":"PRIF-PI","title":"Priority Income Fund, Inc."},"9037":{"cik_str":1554625,"ticker":"PRIF-PK","title":"Priority Income Fund, Inc."},"9038":{"cik_str":1865111,"ticker":"ALSAW","title":"Alpha Star Acquisition Corp"},"9039":{"cik_str":1857086,"ticker":"DMAQR","title":"Deep Medicine Acquisition Corp."},"9040":{"cik_str":1691936,"ticker":"SNAXW","title":"STRYVE FOODS, INC."},"9041":{"cik_str":1824884,"ticker":"ADOCR","title":"Edoc Acquisition Corp."},"9042":{"cik_str":1824884,"ticker":"ADOCW","title":"Edoc Acquisition Corp."},"9043":{"cik_str":1828608,"ticker":"CPUH-UN","title":"Compute Health Acquisition Corp."},"9044":{"cik_str":1830029,"ticker":"ADEX-WT","title":"Adit EdTech Acquisition Corp."},"9045":{"cik_str":1798562,"ticker":"TMCWW","title":"TMC the metals Co Inc."},"9046":{"cik_str":1854963,"ticker":"SHFSW","title":"SHF Holdings, Inc."},"9047":{"cik_str":1438893,"ticker":"GNT-PA","title":"GAMCO Natural Resources, Gold & Income Trust"},"9048":{"cik_str":1568162,"ticker":"RTLPO","title":"Necessity Retail REIT, Inc."},"9049":{"cik_str":1568162,"ticker":"RTLPP","title":"Necessity Retail REIT, Inc."},"9050":{"cik_str":1333986,"ticker":"EQH-PC","title":"Equitable Holdings, Inc."},"9051":{"cik_str":1159508,"ticker":"DGP","title":"DEUTSCHE BANK AKTIENGESELLSCHAFT"},"9052":{"cik_str":1332551,"ticker":"ACR-PC","title":"ACRES Commercial Realty Corp."},"9053":{"cik_str":1332551,"ticker":"ACR-PD","title":"ACRES Commercial Realty Corp."},"9054":{"cik_str":1398805,"ticker":"BEEMW","title":"Beam Global"},"9055":{"cik_str":1053092,"ticker":"UGLDF","title":"CREDIT SUISSE AG"},"9056":{"cik_str":1053092,"ticker":"USLVF","title":"CREDIT SUISSE AG"},"9057":{"cik_str":1053092,"ticker":"GLDI","title":"CREDIT SUISSE AG"},"9058":{"cik_str":1053092,"ticker":"USOI","title":"CREDIT SUISSE AG"},"9059":{"cik_str":1053092,"ticker":"TVIXF","title":"CREDIT SUISSE AG"},"9060":{"cik_str":1267395,"ticker":"AHL-PE","title":"ASPEN INSURANCE HOLDINGS LTD"},"9061":{"cik_str":791908,"ticker":"XOMAO","title":"XOMA Corp"},"9062":{"cik_str":791908,"ticker":"XOMAP","title":"XOMA Corp"},"9063":{"cik_str":895126,"ticker":"CHKEL","title":"CHESAPEAKE ENERGY CORP"},"9064":{"cik_str":1840776,"ticker":"HGTY-WT","title":"Hagerty, Inc."},"9065":{"cik_str":1823794,"ticker":"ARKOW","title":"ARKO Corp."},"9066":{"cik_str":1852039,"ticker":"FILG","title":"Grayscale Filecoin Trust (FIL)"},"9067":{"cik_str":1854863,"ticker":"BWCAW","title":"Blue Whale Acquisition Corp I"},"9068":{"cik_str":1793497,"ticker":"UVIX","title":"VS Trust"},"9069":{"cik_str":1793497,"ticker":"SVIX","title":"VS Trust"},"9070":{"cik_str":1411690,"ticker":"BNGOW","title":"Bionano Genomics, Inc."},"9071":{"cik_str":1631596,"ticker":"KREF-PA","title":"KKR Real Estate Finance Trust Inc."},"9072":{"cik_str":1723648,"ticker":"LVOXW","title":"LiveVox Holdings, Inc."},"9073":{"cik_str":1811856,"ticker":"VIEWW","title":"View, Inc."},"9074":{"cik_str":1547546,"ticker":"LFT-PA","title":"Lument Finance Trust, Inc."},"9075":{"cik_str":1855693,"ticker":"FICVU","title":"Frontier Investment Corp"},"9076":{"cik_str":1752828,"ticker":"CELUW","title":"Celularity Inc"},"9077":{"cik_str":1822366,"ticker":"ALTUW","title":"Altitude Acquisition Corp."},"9078":{"cik_str":1861974,"ticker":"ECXWW","title":"ECARX Holdings Inc."},"9079":{"cik_str":1861541,"ticker":"PGSS-WT","title":"Pegasus Digital Mobility Acquisition Corp."},"9080":{"cik_str":1860805,"ticker":"ASTLW","title":"Algoma Steel Group Inc."},"9081":{"cik_str":1886190,"ticker":"GGROW","title":"Gogoro Inc."},"9082":{"cik_str":1803901,"ticker":"TALKW","title":"Talkspace, Inc."},"9083":{"cik_str":1883085,"ticker":"PGYWW","title":"Pagaya Technologies Ltd."},"9084":{"cik_str":1852016,"ticker":"AEAEW","title":"AltEnergy Acquisition Corp"},"9085":{"cik_str":1838359,"ticker":"RGTIW","title":"Rigetti Computing, Inc."},"9086":{"cik_str":1845815,"ticker":"PAYOW","title":"Payoneer Global Inc."}}

### Moving to dataframe

In [172]:
# company_tickers

df_company_tickers = pd.DataFrame(company_tickers)
df_company_tickers

0               1              2               3  \
cik_str      320193          789019        1652044         1018724   
ticker         AAPL            MSFT          GOOGL            AMZN   
title    Apple Inc.  MICROSOFT CORP  Alphabet Inc.  AMAZON COM INC   

                   4            5                       6  \
cik_str      1045810      1318605                 1067983   
ticker          NVDA         TSLA                   BRK-B   
title    NVIDIA CORP  Tesla, Inc.  BERKSHIRE HATHAWAY INC   

                            7                                          8  \
cik_str               1326801                                    1046179   
ticker                   META                                        TSM   
title    Meta Platforms, Inc.  TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD   

                 9                                10                      11  \
cik_str    1403161                            824046                  731766   
ticker           V                             LVMUY                     UNH   
title    VISA INC.  LVMH MOET HENNESSY LOUIS VUITTON  UNITEDHEALTH GROUP INC   

                     12                13                   14            15  \
cik_str           59478             34088                19617        104169   
ticker              LLY               XOM                  JPM           WMT   
title    ELI LILLY & Co  EXXON MOBIL CORP  JPMORGAN CHASE & CO  Walmart Inc.   

                        16                      17              18  \
cik_str             200406                  884394         1141391   
ticker                 JNJ                     SPY              MA   
title    JOHNSON & JOHNSON  SPDR S&P 500 ETF TRUST  Mastercard Inc   

                    19                   20                21  \
cik_str        1730168                80424            353278   
ticker            AVGO                   PG               NVO   
title    Broadcom Inc.  PROCTER & GAMBLE Co  NOVO NORDISK A S   

                                22                           23           24  \
cik_str                    1047716                      1545460      1341439   
ticker                       LTMAY                        OLCLY         ORCL   
title    LATAM AIRLINES GROUP S.A.  Oriental Land Co., Ltd./ADR  ORACLE CORP   

                       25            26               27                 28  \
cik_str            354950         93410           937966             310158   
ticker                 HD           CVX             ASML                MRK   
title    HOME DEPOT, INC.  CHEVRON CORP  ASML HOLDING NV  Merck & Co., Inc.   

                   29           30                          31           32  \
cik_str         21344        77476                      909832      1551152   
ticker             KO          PEP                        COST         ABBV   
title    COCA COLA CO  PEPSICO INC  COSTCO WHOLESALE CORP /NEW  AbbVie Inc.   

                                33          34                  35  \
cik_str                      70858      796343             1094517   
ticker                         BAC        ADBE                  TM   
title    BANK OF AMERICA CORP /DE/  ADOBE INC.  TOYOTA MOTOR CORP/   

                    36                         37              38  \
cik_str         811809                    1577552           63908   
ticker             BHP                       BABA             MCD   
title    BHP Group Ltd  Alibaba Group Holding Ltd  MCDONALDS CORP   

                          39           40          41                42  \
cik_str               858877      1114448       78003           1108524   
ticker                  CSCO          NVS         PFE               CRM   
title    CISCO SYSTEMS, INC.  NOVARTIS AG  PFIZER INC  Salesforce, Inc.   

                43               44                                45  \
cik_str    1306965           901832                           1061736   
ticker        SHEL              AZN        

### Transpose data

In [173]:
df_company_tickers_transposed = df_company_tickers.transpose()


In [175]:
df_company_tickers_transposed.to_csv("df_company_tickers_transposed.csv")
df_company_tickers_transposed

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1652044,GOOGL,Alphabet Inc.
3,1018724,AMZN,AMAZON COM INC
4,1045810,NVDA,NVIDIA CORP
...,...,...,...
9082,1803901,TALKW,"Talkspace, Inc."
9083,1883085,PGYWW,Pagaya Technologies Ltd.
9084,1852016,AEAEW,AltEnergy Acquisition Corp
9085,1838359,RGTIW,"Rigetti Computing, Inc."


In [193]:
df_company_tickers_transposed.columns

Index(['cik_str', 'ticker', 'title'], dtype='object')

# Reading a table with largest public companies by market cap
* Source: https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/'
* As accessed on July 5th
* Using playwright because of blocker HTTPError: HTTP Error 403: Forbidden when using pd.read_html

In [178]:
url_large_companies_us=("https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/")

In [179]:
# Imports
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

await page.goto(url_large_companies_us)

<Response url='https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/' request=<Request url='https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/' method='GET'>>

In [180]:
large_companies_us_response= await page.content()

large_companies_us_soup_doc = BeautifulSoup(large_companies_us_response, 'html.parser')


In [185]:


large_companies_us_soup_doc_table = large_companies_us_soup_doc.find('table', class_='default-table table marketcap-table dataTable')
# large_companies_us_soup_doc_table

In [187]:

# Extract the data from the table
data = []
for row in large_companies_us_soup_doc_table.find_all('tr')[1:]:
    columns = row.find_all('td')
    name = columns[1].find('div', class_='company-name').text.strip()
    mcap = columns[2].text.strip()
    price = columns[3].text.strip()
    country = columns[6].text.strip()
    data.append([name, mcap, price, country])

# Create a DataFrame from the extracted data
df_large_companies_us_soup_doc_table = pd.DataFrame(data, columns=['Name', 'Market Cap', 'Price', 'Country'])

# Print the DataFrame
df_large_companies_us_soup_doc_table


,Name,Market Cap,Price,Country
0,Apple,$3.030 T,$191.33,🇺🇸 USA
1,Microsoft,$2.533 T,$338.15,🇺🇸 USA
2,Alphabet (Google),$1.553 T,$122.63,🇺🇸 USA
3,Amazon,$1.337 T,$130.38,🇺🇸 USA
4,NVIDIA,$1.045 T,$423.17,🇺🇸 USA
...,...,...,...,...
95,Regeneron Pharmaceuticals,$78.75 B,$717.83,🇺🇸 USA
96,Zoetis,$78.25 B,$169.35,🇺🇸 USA
97,Southern Company,$78.15 B,$71.68,🇺🇸 USA
98,Fiserv,$77.10 B,$124.90,🇺🇸 USA


In [188]:
df_large_companies_us_soup_doc_table.to_csv("df_large_companies_us_soup_doc_table.csv")

# Merging the two dataframes with slightly different names
* use fuzzy matching

In [191]:
# ! pip install fuzzywuzzy

In [205]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Convert 'title' column to string type
df_company_tickers_transposed['title'] = df_company_tickers_transposed['title'].astype(str)

# Define a function to find the best fuzzy match for each title
def find_best_match(title):
    choices = df_company_tickers_transposed['title'].tolist()  # Update column name to 'title'
    best_match, score = process.extractOne(str(title), choices)  # Convert title to string
    if score >= 80:  # Adjust the threshold as needed
        return best_match
    else:
        return None

# Apply the function to find the best matches for each title in df_large_companies_us_soup_doc_table
df_large_companies_us_soup_doc_table['best_match'] = df_large_companies_us_soup_doc_table['Name'].apply(find_best_match)

# Merge the dataframes based on the best matches
merged_df = pd.merge(df_large_companies_us_soup_doc_table, df_company_tickers_transposed, left_on='best_match', right_on='title', how='inner')  # Update column name to 'title'

# Remove the intermediate 'best_match' column
merged_df.drop('best_match', axis=1, inplace=True)

# Print the merged dataframe
print(merged_df)


                               Name Market Cap    Price Country  cik_str  \
0                             Apple   $3.030 T  $191.33  🇺🇸 USA   320193   
1                         Microsoft   $2.533 T  $338.15  🇺🇸 USA   789019   
2                            Amazon   $1.337 T  $130.38  🇺🇸 USA  1018724   
3    Booking Holdings (Booking.com)   $98.76 B   $2,674  🇺🇸 USA  1018724   
4                            NVIDIA   $1.045 T  $423.17  🇺🇸 USA  1045810   
..                              ...        ...      ...     ...      ...   
136              Palo Alto Networks   $78.87 B  $257.88  🇺🇸 USA  1327567   
137       Regeneron Pharmaceuticals   $78.75 B  $717.83  🇺🇸 USA   872589   
138                          Zoetis   $78.25 B  $169.35  🇺🇸 USA  1555280   
139                          Fiserv   $77.10 B  $124.90  🇺🇸 USA   798354   
140                     Progressive   $76.93 B  $131.43  🇺🇸 USA    80661   

    ticker                            title  
0     AAPL                       Apple In

In [206]:
merged_df.to_csv("merged_df.csv")

# Reading Data cleaned in Excel
* In Excel, I removed duplicates to make 100 rows
* I also manually put in the companies that did not automatically title themselves (just IBM and Google)

In [207]:
df_top_us_plus_sec_titles=pd.read_csv("df_top_us_plus_sec_titles.csv")
df_top_us_plus_sec_titles

,Unnamed: 0,name,market_cap,price,country,Unnamed: 5,Name_2,Market Cap_2,Price_2,Country_2,cik_str,ticker,title
0,0,Apple,$3.030 T,$191.33,🇺🇸 USA,95,Apple,$3.030 T,$191.33,🇺🇸 USA,320193,AAPL,Apple Inc.
1,1,Microsoft,$2.533 T,$338.15,🇺🇸 USA,111,Microsoft,$2.533 T,$338.15,🇺🇸 USA,789019,MSFT,MICROSOFT CORP
2,2,Alphabet (Google),$1.553 T,$122.63,🇺🇸 USA,135,NaN,NaN,NaN,NaN,1652044,GOOG,Alphabet Inc.
3,3,Amazon,$1.337 T,$130.38,🇺🇸 USA,49,Amazon,$1.337 T,$130.38,🇺🇸 USA,1018724,AMZN,AMAZON COM INC
4,4,NVIDIA,$1.045 T,$423.17,🇺🇸 USA,84,NVIDIA,$1.045 T,$423.17,🇺🇸 USA,1045810,NVDA,NVIDIA CORP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Regeneron Pharmaceuticals,$78.75 B,$717.83,🇺🇸 USA,85,Regeneron Pharmaceuticals,$78.75 B,$717.83,🇺🇸 USA,872589,REGN,"REGENERON PHARMACEUTICALS, INC."
96,96,Zoetis,$78.25 B,$169.35,🇺🇸 USA,45,Zoetis,$78.25 B,$169.35,🇺🇸 USA,1555280,ZTS,Zoetis Inc.
97,97,Southern Company,$78.15 B,$71.68,🇺🇸 USA,26,Southern Company,$78.15 B,$71.68,🇺🇸 USA,72971,WFC,WELLS FARGO & COMPANY/MN
98,98,Fiserv,$77.10 B,$124.90,🇺🇸 USA,25,Fiserv,$77.10 B,$124.90,🇺🇸 USA,798354,FI,FISERV INC
